In [ ]:
############################## This Version ########################################
# • Changed seq_pango check to seq_clade check for related groups of sequences (pairwise check for new groups only——existing-group check was already by clade)
#

In [26]:
@time begin
    using Pkg
    Pkg.add("JSON3")
    using JSON3
    Pkg.add("DataFrames")
    using DataFrames
    Pkg.add("JLD2")
    using JLD2
    Pkg.add("Dates")
    using Dates
    Pkg.add("CSV")
    using CSV
    Pkg.add("Tables")
    using Tables
    Pkg.add("PrettyTables")
    using PrettyTables
end
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
println(pwd())
cd("/Users/ryhisner")
println(pwd())

   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`


  8.879741 seconds (28.86 M allocations: 2.224 GiB, 10.18% gc time)
2025-05-26_735PM
/Users/ryhisner
/Users/ryhisner


In [8]:
################### Tons of Functions that have to be run first ##########################
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
AA_sort_key(m) = (split(m, ":")[1], parse(Int, split(m, ":")[2][2:end-1]))
AA_sort_key_pos_only(m) = (split(m, ":")[1], parse(Int, split(m, ":")[2]))
EPI_ISL(n) = split(n, "|")[2]
country(n) = split(n, "/")[2]
US_state(n) = split(split(n, "/")[3], "-")[1]
CAN_province(n) = split(split(n, "/")[3], "-")[1]
sequence_date(n) = split(n, "|")[3]
seq_lab(n) = split(n, "/")[3]
mut_pos(m) = parse(Int, m[2:end-1])
gene_name(m) = split(m, ":")[1]
AA_position(m) = split(m, ":")[2][2:end-1]
AA_mut_to_AA_pos(m) = split(m, ":")[1] * ":" * split(m, ":")[2][2:end-1]
gene_name_fx(m) = split(m, ":")[1]
species(m) = split(m, "/")[2]
#########################################################################################################################################
function seconds_to_hrs_min_sec(t)
    hours = 0; minutes = 0; seconds = 0; hours = t÷3600; minutes = (t%3600)÷60
    if t > 0.0001
        seconds = (t%3600)%60
    end
    hours_int = Int(hours); minutes_int = Int(minutes)
    minutes_str = split(string(minutes_int), ".")[1]
    hours_fin = split(string(hours_int), ".")[1]
    minutes_fin = lpad(minutes_str, 2, "0")
    seconds_1 = round(digits=2, seconds)
    seconds_fin = lpad(seconds_1, 5, "0")
    final_time = "$hours_fin:$minutes_fin:$seconds_fin"
    final_time2 = "$hours_fin hours, $minutes_fin minutes, $seconds_fin seconds"
    return final_time, final_time2
end
#########################################################################################################################################
function stringlist_to_strings(txt)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    arr_of_strings1 = Vector{String}()
    arr_of_strings2 = Vector{String}()
    no_newlines = replace(txt, 
        "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(arr_of_strings2, mseq)
            end
        else 
            push!(arr_of_strings2, seq)
        end
    end
    sort_arr_of_strings2 = sort(collect(arr_of_strings2), by = x -> epi_sortkey(x))
    return sort_arr_of_strings2
end
#########################################################################################################################################
function read_fasta(filepath::String)
    reader = FASTX.FASTA.Reader(open(filepath, "r"))
    fasta_in = [record for record in reader]
    close(reader)
    return[String(FASTX.FASTX.description(rec)) for rec in fasta_in],
    [uppercase(String(FASTX.FASTA.sequence(rec))) for rec in fasta_in]
end
#########################################################################################################################################
#########################################################################################################################################
#########################################################################################################################################
gene_AA_dict = Dict{String, String}()
gene_AA_dict["ORF1a"] = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV*"
gene_AA_dict["ORF1b"] = "RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
gene_AA_dict["S"] = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
gene_AA_dict["E"] = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
gene_AA_dict["M"] = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
gene_AA_dict["N"] = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
gene_AA_dict["ORF3a"] = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
gene_AA_dict["ORF6"] = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
gene_AA_dict["ORF7a"] = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
gene_AA_dict["ORF7b"] = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
gene_AA_dict["ORF8"] = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
gene_AA_dict["ORF9b"] = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
NSP_ranges1a = Dict{Int, BitSet}(1 => BitSet([1:180...]), 2 => BitSet([181:818...]), 3 => BitSet([819:2763...]), 4 => BitSet([2764:3263...]), 5 => BitSet([3264:3569...]), 6 => BitSet([3570:3859...]), 7 => BitSet([3860:3942...]), 8 => BitSet([3943:4140...]), 9 => BitSet([4141:4253...]), 10 => BitSet([4254:4392...]), 12 => BitSet([4393:4401...]))
NSP_ranges1b = Dict{Int, BitSet}(12 => BitSet([1:923...]), 13 => BitSet([924:1524...]), 14 => BitSet([1525:2051...]), 15 => BitSet([2052:2397...]), 16 => BitSet([2398:2696...]))
NSP1a_add = Dict{Int, Int}(1 => 0, 2 => 180, 3 => 818, 4 => 2763, 5 => 3263, 6 => 3569, 7 => 3859, 8 => 3942, 9 => 4140, 10 => 4253, 11 => 0, 12 => 4392)
NSP1b_add = Dict{Int, Int}(12 => -9, 13 => 923, 14 => 1524, 15 => 2051, 16 => 2397)
NSP_AA_dict = Dict{Int, String}()
for i in 1:10
    NSPkey = "NSP"*string(i)
    first = minimum(NSP_ranges1a[i])
    last = maximum(NSP_ranges1a[i])
    NSP_AA_str = gene_AA_dict["ORF1a"][first:last]
    NSP_AA_dict[i] = NSP_AA_str
end
NSP_AA_dict[11] = ""
NSP12_1a = gene_AA_dict["ORF1a"][4393:4401]
NSP12_1b = gene_AA_dict["ORF1b"][1:923]
NSP12_AA_str = NSP12_1a*NSP12_1b
NSP_AA_dict[12] = NSP12_AA_str
for i in 13:16
    NSPkey = "NSP"*string(i)
    first = minimum(NSP_ranges1b[i])
    last = maximum(NSP_ranges1b[i])
    NSP_AA_str = gene_AA_dict["ORF1b"][first:last]
    NSP_AA_dict[i] = NSP_AA_str
end
#########################################################################################################################################
#########################################################################################################################################
println(pwd())
cd("/Users/ryhisner")
println(pwd())    
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)

2025-05-14_836PM
/Users/ryhisner
/Users/ryhisner
2025-05-14_836PM


In [17]:
##################### load_all_seq_dicts, 2025_5_13 version #######################
date_now = Dates.format(now(), "yyyy-mm-dd__IMMp")
println(date_now)
function load_all_seq_dicts(date::String, fx_name::String, ndjson_name::String, max_AA_mut::Int, revs_thresh::Int, qc_max::Int)
#$    global date_nuc_mut_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_nuc_mut_ct.jld2", "date_nuc_mut_ct")
#$    global date_nuc_mut_ct_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels")
#$    global date_AA_mut_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct.jld2", "date_AA_mut_ct")
#$    global date_AA_mut_ct_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels")
#$    global date_AA_mut_ct_pos_only_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels")

#$    global seq_ct_by_year_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year.jld2", "seq_ct_by_year")
#$    global seq_ct_by_year_month_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year_month.jld2", "seq_ct_by_year_month")
#$    global seq_ct_by_year_month_day_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day")

#$    global seq_country_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_country.jld2", "seq_country")
#$    global seq_US_state_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_US_state.jld2", "seq_US_state")
#$    global seq_clade_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_clade.jld2", "seq_clade")
#$    global seq_pango_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_pango.jld2", "seq_pango")
#$    global seq_pango_unaliased_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_pango_unaliased.jld2", "seq_pango_unaliased")
#$    global seq_collection_date_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_collection_date.jld2", "seq_collection_date")
#$    global seq_date_index_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_date_index.jld2", "seq_date_index")
#$    global seq_date_tuple_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_date_tuple.jld2", "seq_date_tuple")
#$    global seq_lab_dict_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_lab_dict.jld2", "seq_lab_dict")
#$    global seq_nuc_del_ranges_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_nuc_del_ranges_ct.jld2", "seq_nuc_del_ranges_ct")
    # _all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_.jld2", "")
    # _all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_.jld2", "")
    
    global country_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_country_set.jld2", "country_set")
    global seq_lab_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_seq_lab_set.jld2", "seq_lab_set")
    global clade_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_clade_set.jld2", "clade_set")
    global pango_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_set.jld2", "pango_set")
    global pango_unaliased_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_unaliased_set.jld2", "pango_unaliased_set")
    global pango_to_pango_unaliased = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_to_pango_unaliased.jld2", "pango_to_pango_unaliased")
    global clade_pango_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_clade_pango_set.jld2", "clade_pango_set")
    global clade_pango_unaliased_set = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_clade_pango_unaliased_set.jld2", "clade_pango_unaliased_set")
#########################################################################################################################################    
    global clade_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_clade_date_index_ct.jld2", "clade_date_index_ct")
    global pango_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_date_index_ct.jld2", "pango_date_index_ct")
    global pango_unaliased_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_pango_unaliased_date_index_ct.jld2", "pango_unaliased_date_index_ct")

    global country_clade_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_country_clade_date_index_ct.jld2", "country_clade_date_index_ct")
    global country_pango_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_country_pango_date_index_ct.jld2", "country_pango_date_index_ct")
    global country_pango_unaliased_date_index_ct = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_country_pango_unaliased_date_index_ct.jld2", "country_pango_unaliased_date_index_ct")
#########################################################################################################################################    
    global ORF9b_CTD_muts_seq_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_ORF9b_CTD_muts_seq.jld2", "ORF9b_CTD_muts_seq")
    global multi_ORF9b_CTD_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD.jld2", "multi_ORF9b_CTD")
    global multi_ORF9b_CTD_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD_ct.jld2", "multi_ORF9b_CTD_ct")
    global qualifying_ORF9b_double_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_qualifying_ORF9b_double_ct.jld2", "qualifying_ORF9b_double_ct")

    global ORF9b_CTD_muts_seq_all_relaxed_qc = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_ORF9b_CTD_muts_seq_relaxed_qc.jld2", "ORF9b_CTD_muts_seq_relaxed_qc")
    global multi_ORF9b_CTD_all_relaxed_qc = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD_relaxed_qc.jld2", "multi_ORF9b_CTD_relaxed_qc")
    global multi_ORF9b_CTD_ct_all_relaxed_qc = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_multi_ORF9b_CTD_ct_relaxed_qc.jld2", "multi_ORF9b_CTD_ct_relaxed_qc")
    global qualifying_ORF9b_double_ct_all_relaxed_qc = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_qualifying_ORF9b_double_ct_relaxed_qc.jld2", "qualifying_ORF9b_double_ct_relaxed_qc")
#########################################################################################################################################
    
    global avg_private_AA_per_circ_seq = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_avg_private_AA_per_circ_seq.jld2", "avg_private_AA_per_circ_seq")
    
    global AA_muts_seq_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_seq.jld2", "AA_muts_seq")

    global all_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_all_seq_ct.jld2", "all_seq_ct")
    global qualifying_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_qualifying_seq_ct.jld2", "qualifying_seq_ct")

    global nuc_muts_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct.jld2", "nuc_muts_ct")
    global nuc_muts_ct_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels")
    global AA_muts_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct.jld2", "AA_muts_ct")
    global AA_muts_ct_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels")
    global AA_muts_ct_pos_only_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only")
    global AA_muts_ct_pos_only_no_dels_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels")
    global gene_mut_density_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density.jld2", "gene_mut_density")
    global domain_mut_density_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density.jld2", "domain_mut_density")

#$    global nuc_muts_ct_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_sort.jld2", "nuc_muts_ct_sort")
#$    global nuc_muts_ct_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_sort_by_seq_ct.jld2", "nuc_muts_ct_sort_by_seq_ct")
#$    global nuc_muts_ct_no_dels_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels_sort.jld2", "nuc_muts_ct_no_dels_sort")
#$    global nuc_muts_ct_no_dels_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_nuc_muts_ct_no_dels_sort_by_seq_ct.jld2", "nuc_muts_ct_no_dels_sort_by_seq_ct")

    global AA_muts_ct_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_sort.jld2", "AA_muts_ct_sort")
    global AA_muts_ct_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_sort_by_seq_ct.jld2", "AA_muts_ct_sort_by_seq_ct")
    global AA_muts_ct_pos_only_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_sort.jld2", "AA_muts_ct_pos_only_sort")
    global AA_muts_ct_pos_only_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_sort_by_seq_ct")
    global AA_muts_ct_no_dels_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels_sort.jld2", "AA_muts_ct_no_dels_sort")
    global AA_muts_ct_no_dels_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_sort_by_seq_ct")
    global AA_muts_ct_pos_only_no_dels_sort_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels_sort.jld2", "AA_muts_ct_pos_only_no_dels_sort")
    global AA_muts_ct_pos_only_no_dels_sort_by_seq_ct_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_seq_ct")

    global gene_mut_density_sort_by_gene_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density_sort_by_gene.jld2", "gene_mut_density_sort_by_gene")
    global gene_mut_density_sort_by_density_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_gene_mut_density_sort_by_density.jld2", "gene_mut_density_sort_by_density")
    global domain_mut_density_sort_by_gene_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density_sort_by_gene.jld2", "domain_mut_density_sort_by_gene")
    global domain_mut_density_sort_by_density_all = load("dictionaries/$(date)_$(fx_name)_$(ndjson_name)_maxAAmut$(max_AA_mut)_maxRevs$(revs_thresh)_qcMax$(qc_max)_domain_mut_density_sort_by_density.jld2", "domain_mut_density_sort_by_density")
    println("Dictionaries loaded!")
    println()
    finish = time() - start
    finish_rd = round(digits=1, finish)
    load_hms1, load_hms2 = seconds_to_hrs_min_sec(finish)
    println("Total Time to Load ALL Dictionaries = $(finish_rd)")
    println("Total Time to Load ALL Dictionaries = $(load_hms1)")
    println("Total Time to Load ALL Dictionaries = $(load_hms2)")
end

2025-05-15__903PM


load_all_seq_dicts (generic function with 1 method)

In [18]:
################ Execute Load All Dictionaries Function, EPI_ISL_400000_19860000 #########################
date_now = Dates.format(now(), "yyyy-mm-dd__IMMp")
println(date_now)
start = time()
load_all_seq_dicts("2025-05-13", "all_private_muts", "EPI_ISL_400000_19860000", 5, 1, 5)
finish = time() - start
finish_rd = round(digits=1, finish)
println("Total Time to Load ALL Dictionaries = $(finish_rd)")
println(date_now)
##load_all_seq_dicts(date::String, fx_name::String, ndjson_name::String, max_AA_mut::Int, revs_thresh::Int, qc_max::Int)
# 2025-02-17_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_avg_private_AA_per_circ_seq.jld2
for (dom, ct) in domain_mut_density_all
    println(dom, " = ", ct)
end
domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP12_NiRAN"=>20, "NSP12_intrfce"=>21, "NSP12_fingers"=>22, "NSP12_palm"=>23, "NSP12_palmLnk"=>24, "NSP12_thumb"=>25, "S_S1"=>26, "S_S2"=>27, "S_NTD"=>28, "S_RBD"=>29, "S_FCS_region"=>30, "N_N1"=>31, "N_N2"=>32, "N_N3"=>33, "N_N4"=>34, "N_N5"=>35, "N_SR"=>36, "N_L_helix"=>37, "N_CBP"=>38, "N_9b_overlap"=>39)
gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)
gene_mut_density_all_sort_by_gene = sort(collect(gene_mut_density_all), by = x -> gene_protein_order[x[1]])
gene_mut_density_all_sort_by_density = sort(collect(gene_mut_density_all), by = x -> x[2], rev=true)
domain_mut_density_all_sort_by_gene = sort(collect(domain_mut_density_all), by = x -> domain_order[x[1]])
domain_mut_density_all_sort_by_density = sort(collect(domain_mut_density_all), by = x -> x[2], rev=true)
println(date_now)

2025-05-15__904PM
Dictionaries loaded!

Total Time to Load ALL Dictionaries = 60.0
Total Time to Load ALL Dictionaries = 0:01:00.04
Total Time to Load ALL Dictionaries = 0 hours, 01 minutes, 00.04 seconds
Total Time to Load ALL Dictionaries = 60.1
2025-05-15__904PM
NSP3_Ubl2 = 1.6
NSP3_Y1 = 1.4
N_9b_overlap = 2.2
S_S1 = 2.1
NSP3_BSM = 3.2
NSP12_NiRAN = 1.4
N_L_helix = 2.1
NSP3_Mac2 = 1.7
NSP3_Ecto3 = 2.0
N_CBP = 1.9
S_RBD = 1.1
NSP12_palmLnk = 1.0
NSP4_TM2_TM6 = 1.1
N_SR = 4.0
NSP3_DPUP = 2.2
N_N1 = 3.3
NSP3_Ubl1 = 1.5
NSP3_Mac3 = 1.0
NSP3_CoVY = 1.2
NSP4_CTD = 0.6
NSP4_Ecto4 = 1.4
NSP12_thumb = 0.9
S_FCS_region = 4.3
NSP4_TM1 = 2.8
N_N5 = 4.0
N_N3 = 3.8
NSP3_Mac1 = 2.3
N_N2 = 2.1
NSP12_intrfce = 0.8
NSP3_TM234HLX = 1.1
NSP3_NAB = 1.6
S_S2 = 1.4
NSP3_TM1 = 1.6
NSP12_fingers = 0.6
NSP3_HVR = 3.3
NSP12_palm = 0.9
N_N4 = 1.9
NSP3_PLpro = 1.5
S_NTD = 3.2
2025-05-15__904PM


In [11]:
################ Execute Load All Dictionaries Function, EPI_ISL_400000_19720000 #########################
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
start = time()
load_all_seq_dicts("2025-03-17", "all_private_muts", "EPI_ISL_400000_19720000", 5, 1, 5)
finish = time() - start
finish_rd = round(digits=1, finish)
println("Total Time to Load ALL Dictionaries = $(finish_rd)")
println(date_right_now)
##load_all_seq_dicts(date::String, fx_name::String, ndjson_name::String, max_AA_mut::Int, revs_thresh::Int, qc_max::Int)
# 2025-02-17_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_avg_private_AA_per_circ_seq.jld2
# 

2025-05-04_1210PM
Dictionaries loaded!

Total Time to Load ALL Dictionaries = 147.0
Total Time to Load ALL Dictionaries = 0:02:26.97
Total Time to Load ALL Dictionaries = 0 hours, 02 minutes, 26.97 seconds
2025_05_04_1212PM
Total Time to Load ALL Dictionaries = 147.1
2025-05-04_1210PM


In [5]:
#date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
#println(date_right_now)
#start = time()

#nuc_muts_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct.jld2", "nuc_muts_ct")
#nuc_muts_ct_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels")
#AA_muts_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct.jld2", "AA_muts_ct")
#AA_muts_ct_pos_only_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only")
#AA_muts_ct_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels")
#AA_muts_ct_pos_only_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels")

#gene_mut_density_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_gene_mut_density.jld2", "gene_mut_density")
#domain_mut_density_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_domain_mut_density.jld2", "domain_mut_density")
#gene_mut_density_sort_by_gene_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_gene_mut_density_sort_by_gene.jld2", "gene_mut_density_sort_by_gene")
#gene_mut_density_sort_by_density_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_gene_mut_density_sort_by_density.jld2", "gene_mut_density_sort_by_density")
#domain_mut_density_sort_by_gene_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_domain_mut_density_sort_by_gene.jld2", "domain_mut_density_sort_by_gene")
#domain_mut_density_sort_by_density_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_domain_mut_density_sort_by_density.jld2", "domain_mut_density_sort_by_density")

#AA_muts_seq_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_seq.jld2", "AA_muts_seq")

#all_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_all_seq_ct.jld2", "all_seq_ct")
#qualifying_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_qualifying_seq_ct.jld2", "qualifying_seq_ct")

#nuc_muts_ct_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct_sort.jld2", "nuc_muts_ct_sort")
#nuc_muts_ct_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct_sort_by_seq_ct.jld2", "nuc_muts_ct_sort_by_seq_ct")
#nuc_muts_ct_no_dels_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct_no_dels_sort.jld2", "nuc_muts_ct_no_dels_sort")
#nuc_muts_ct_no_dels_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_nuc_muts_ct_no_dels_sort_by_seq_ct.jld2", "nuc_muts_ct_no_dels_sort_by_seq_ct")

#AA_muts_ct_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_sort.jld2", "AA_muts_ct_sort")
#AA_muts_ct_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_sort_by_seq_ct.jld2", "AA_muts_ct_sort_by_seq_ct")
#AA_muts_ct_no_dels_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_no_dels_sort.jld2", "AA_muts_ct_no_dels_sort")
#AA_muts_ct_no_dels_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_no_dels_sort_by_seq_ct")
#AA_muts_ct_pos_only_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only_sort.jld2", "AA_muts_ct_pos_only_sort")
#AA_muts_ct_pos_only_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_sort_by_seq_ct")
#AA_muts_ct_pos_only_no_dels_sort_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only_no_dels_sort.jld2", "AA_muts_ct_pos_only_no_dels_sort")
#AA_muts_ct_pos_only_no_dels_sort_by_seq_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", "AA_muts_ct_pos_only_no_dels_sort_by_seq_ct")

#seq_ct_by_year_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_ct_by_year.jld2", "seq_ct_by_year")
#seq_ct_by_year_month_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_ct_by_year_month.jld2", "seq_ct_by_year_month")
#seq_ct_by_year_month_day_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day")

#date_nuc_mut_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_date_nuc_mut_ct.jld2", "date_nuc_mut_ct")
#date_nuc_mut_ct_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels")
#date_AA_mut_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_date_AA_mut_ct.jld2", "date_AA_mut_ct")
#date_AA_mut_ct_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels")
#date_AA_mut_ct_pos_only_no_dels_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels")

#seq_country_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_country.jld2", "seq_country")
#seq_US_state_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_US_state.jld2", "seq_US_state")
#seq_clade_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_clade.jld2", "seq_clade")
#seq_pango_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_pango.jld2", "seq_pango")
#seq_pango_unaliased_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_pango_unaliased.jld2", "seq_pango_unaliased")
#seq_collection_date_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_collection_date.jld2", "seq_collection_date")
#seq_date_index_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_date_index.jld2", "seq_date_index")
#seq_date_tuple_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_date_tuple.jld2", "seq_date_tuple")
#seq_lab_dict_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_lab_dict.jld2", "seq_lab_dict")
#seq_nuc_del_ranges_ct_all = load("dictionaries/2025-01-27_all_private_muts_EPI_ISL_400000_19460000_v3_maxAAmut5_maxRevs1_qcMax5_seq_nuc_del_ranges_ct.jld2", "seq_nuc_del_ranges_ct")

#println("Dictionaries loaded!")
#println()
#finish = time() - start
#finish_rd = round(digits=1, finish)
#load_hms1, load_hms2 = seconds_to_hrs_min_sec(finish)
#println("Total Time to Load ALL Dictionaries = $(finish_rd)")
#println("Total Time to Load ALL Dictionaries = $(load_hms1)")
#println("Total Time to Load ALL Dictionaries = $(load_hms2)")
#println(date_right_now)

2025-04-11_259PM
Dictionaries loaded!

Total Time to Load ALL Dictionaries = 0.0
Total Time to Load ALL Dictionaries = 0:00:000.0
Total Time to Load ALL Dictionaries = 0 hours, 00 minutes, 000.0 seconds
2025-04-11_259PM


In [9]:
################################## common_frameshift_fixes function ################################

# Row 1 = N:30-33/ORF9b:27-29 deletion
# Row 2 = BA.2 S:24-26 deletion
# Row 3 = ∆69-70 deletion
# Row 4 = ∆N212, L212I
# Row 5 = JN.1's F157S,R158G
# Row 6 = NSP6 Deletion
# Row 7 = ∆143-145 deletion
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
frameshift_to_fix_dict = Dict{String, String}(
    "CCAGAATGGGTGGGGCG"=>"CCAGAATGGTGGGGCG", "CCAGAATGNGTGGGGCG"=>"CCAGAATGGTGGGGCG", 
    "ACTCAATTCATACACTA"=>"ACTCAATCATACACTA", "ACTCAATTACATACACTA"=>"ACTCAATCATACACTA", 
    "TANNNNNTCTCTGGGAC"=>"TATCTCTGGGAC", "TANNNNTCTCTGGGACC"=>"TATCTCTGGGACC", "TANNNTCTCTGGGACCA"=>"TATCTCTGGGACCA", "TANNTCTCTGGGACCAA"=>"TATCTCTGGGACCAA", "TANTCTCTGGGACCAAT"=>"TATCTCTGGGACCAAT", "TANTCTCTGGGACCAAT"=>"TATCTCTGGGACCAAT", "TATNNNTCTCTGGGACCAAT"=>"TATCTCTGGGACCAAT", 
    "CACGCCTATTATTAGGG"=>"CACGCCTATTATAGGG", "CACGCCTATTANTAGGG"=>"CACGCCTATTATAGGG",
    "GTGAGTCAGNGAGTTTA"=>"GTGAGTCAGGAGTTTA", "GTGAGTCNGAGTTTA"=>"GTGAGTCAGGAGTTTA", "GTGANNTCAGGAGTTTA"=>"GTGAGTCAGGAGTTTA",  "GTGAGTCNGAGTTTA"=>"GTGAGTCAGGAGTTTA",
    "GATACTANNNNGTTTT"=>"GATACTAGTTTT", "ACTAGTTTGGTTTGAAG"=>"ACTAGTTTGAAG", 
    "TTTTTGGGACCACAAAA"=>"TTTTTGGACCACAAAA")
    
# ""=>"",  ""=>"", ""=>"",  ""=>"", ""=>"", 
function common_frameshift_fixes(fasta)    #  GGGACCAAT
    for line in eachline(fasta)
        for baddy in keys(frameshift_to_fix_dict)
            for i in 5:length(line) - 15
                if line[i:i+16] == baddy
                    line[i:i+16] = frameshift_to_fix_dict[baddy]
                end
            end
        end
    end
end
function common_frameshift_fixes2(in_fasta, out_fasta)
    open(out_fasta, "w") do g
        for line in eachline(in_fasta)
            if line[1] == '>'
                println(g, line)
            else  
                frameshift_fixed_line = line
                for (baddy, fixed) in frameshift_to_fix_dict
                    frameshift_fixed_line = replace(frameshift_fixed_line, baddy => fixed)
                end
                println(g, frameshift_fixed_line)
            end
        end
    end
end 
#    , ""=>"", ""=>"", ""=>"", 
#    ""=>"", ""=>"", ""=>"", )

2025-05-14_838PM


common_frameshift_fixes2 (generic function with 1 method)

In [91]:
################################## Execute common_frameshift_fixes ################################
common_frameshift_fixes2("2025-04-06_EPI_ISL_16000001-17000000_minMut7_minChrMut4_Filtered.fasta", "2025-04-06_EPI_ISL_16000001-17000000_minMut7_minChrMut4_Filtered_fixed.fasta")

# common_frameshift_fixes2(in_fasta, out_fasta)

In [10]:
################### More Functions that have to be run first ##########################
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
index_to_date = Dict{Int, Tuple{Int, Int, Int}}()
date_to_index = Dict{Tuple{Int, Int, Int}, Int}()
index = 0
for year in 2020:2027
    for month in 1:12
        if year%4 == 0
            month_days = leap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_date[index] = (year, month, day)
            end
        else
            month_days = nonleap_month_day_dict[month]
            for day in 1:month_days
                index += 1
                index_to_date[index] = (year, month, day)
            end
        end
    end
end
for (index, date) in index_to_date
    date_to_index[date] = index
end
for y in 2019:2027
    date_to_index[(y, 0, 0)] = y*1000000
    index_to_date[y*1000000] = (y, 0, 0)
    for m in 1:12
        date_to_index[(y, m, 0)] = y*1000000 + m*1000
        index_to_date[y*1000000 + m*1000] = (y, m, 0)
    end
end
date_to_index[(0, 0, 0)] = 1000000000
index_to_date[1000000000] = (0, 0, 0)
##############################################################################################################
##############################################################################################################
function AA_mut_ct_by_date_range_all(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct_all[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function AA_mut_ct_no_dels_by_date_range_all(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_AA_mut_ct_no_dels_all[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function AA_mut_ct_pos_only_no_dels_by_date_range_all(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_AA_mut_ct_pos_only_no_dels_all[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function nuc_mut_ct_by_date_range_all(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct_all[i]
        for (nucmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, nucmut)
                date1_to_date2_ct[nucmut] = ct
            else
                date1_to_date2_ct[nucmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function nuc_mut_ct_no_dels_by_date_range_all(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct_no_dels_all[i]
        for (nucmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, nucmut)
                date1_to_date2_ct[nucmut] = ct
            else
                date1_to_date2_ct[nucmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
function AA_mut_ct_by_date_range(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function AA_mut_ct_no_dels_by_date_range(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_AA_mut_ct_no_dels[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function AA_mut_ct_pos_only_no_dels_by_date_range(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_AA_mut_ct_pos_only_no_dels[i]
        for (AAmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, AAmut)
                date1_to_date2_ct[AAmut] = ct
            else
                date1_to_date2_ct[AAmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function nuc_mut_ct_by_date_range(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct[i]
        for (nucmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, nucmut)
                date1_to_date2_ct[nucmut] = ct
            elseL
                date1_to_date2_ct[nucmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end
##############################################################################################################
function nuc_mut_ct_no_dels_by_date_range(date1::Int, date2::Int)
    date1_to_date2_ct = Dict{String, Int}()
    for i in date1:date2
        temp_dict = date_nuc_mut_ct_no_dels[i]
        for (nucmut, ct) in temp_dict
            if !haskey(date1_to_date2_ct, nucmut)
                date1_to_date2_ct[nucmut] = ct
            else
                date1_to_date2_ct[nucmut] += ct
            end
        end
    end
    return date1_to_date2_ct
end

2025-05-14_839PM


nuc_mut_ct_no_dels_by_date_range (generic function with 1 method)

In [10]:
excluded_seqs = stringlist_to_strings("EPI_ISL_12763802, EPI_ISL_15793981, EPI_ISL_16287253, EPI_ISL_18059726, EPI_ISL_15865257, EPI_ISL_13215742, EPI_ISL_14158264, EPI_ISL_12293549, EPI_ISL_15550525, EPI_ISL_15945504, EPI_ISL_19227219, EPI_ISL_14203206, EPI_ISL_19506572, EPI_ISL_13338249, EPI_ISL_13358962, EPI_ISL_13464004, EPI_ISL_13716400, EPI_ISL_13729045, EPI_ISL_14599772, EPI_ISL_16969756, EPI_ISL_16969757")
for s in excluded_seqs
    println(s)
end

EPI_ISL_12293549
EPI_ISL_12763802
EPI_ISL_13215742
EPI_ISL_13338249
EPI_ISL_13358962
EPI_ISL_13464004
EPI_ISL_13716400
EPI_ISL_13729045
EPI_ISL_14158264
EPI_ISL_14203206
EPI_ISL_14599772
EPI_ISL_15550525
EPI_ISL_15793981
EPI_ISL_15865257
EPI_ISL_15945504
EPI_ISL_16287253
EPI_ISL_16969756
EPI_ISL_16969757
EPI_ISL_18059726
EPI_ISL_19227219
EPI_ISL_19506572


In [10]:
println(all_seq_ct_all)
println(qualifying_seq_ct_all)

16728455
9903085


In [25]:
for line in eachline("chronics_2025_04_14__5383seq.ndjson")
    seq_country_temp = Dict{String, String}()
    seq_US_state_temp = Dict{String, String}()
    j = JSON3.read(line)
    name = EPI_ISL(j.seqName)
    seq_country_temp[name] = country(j.seqName)
    if seq_country_temp[name] == "USA"
        seq_US_state_temp[name] = US_state(j.seqName)
    end
    US_state_sort = sort(collect(seq_US_state_temp), by = x -> x[2])
#    for m in US_state_sort
#        println(m[1], " = ", m[2])
#    end
    if seq_country_temp[name] == "Netherlands"
        println(j.seqName)
    end
end

hCoV-19/Netherlands/FR-UMCG-MMB_6121/2022|EPI_ISL_14699555|2022-08-09
hCoV-19/Netherlands/FR-RIVM-109955/2022|EPI_ISL_14773839|2022-08-09
hCoV-19/Netherlands/FR-RIVM-111303/2022|EPI_ISL_14922929|2022-08-25
hCoV-19/Netherlands/ZH-EMC-5998/2022|EPI_ISL_12822483|2022-05-09
hCoV-19/Netherlands/ZH-EMC-5996/2022|EPI_ISL_12822481|2022-05-04
hCoV-19/Netherlands/ZE-MVD-CWGS2202303/2022|EPI_ISL_13028133|2022-05-17
hCoV-19/Netherlands/NH-AUMC-020860/2022|EPI_ISL_13132070|2022-05-27
hCoV-19/Netherlands/ZE-MVD-CWGS2202415/2022|EPI_ISL_13312837|2022-06-01
hCoV-19/Netherlands/LI-MUMC-6129/2022|EPI_ISL_13425805|2022-06-10
hCoV-19/Netherlands/ZH-EMC-6233/2022|EPI_ISL_13502894|2022-06-15
hCoV-19/Netherlands/ZH-EMC-6353/2022|EPI_ISL_13744799|2022-06-15
hCoV-19/Netherlands/NH-RIVM-103814/2022|EPI_ISL_13759811|2022-06-13
hCoV-19/Netherlands/ZH-EMC-6352/2022|EPI_ISL_13744798|2022-06-15
hCoV-19/Netherlands/GR-UMCG-MMB_5719/2022|EPI_ISL_13850726|2022-07-06
hCoV-19/Netherlands/OV-RIVM-105979/2022|EPI_ISL_13991

In [41]:
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
function chr_AA_nuc_del_adj_JSON3_2025_03_27_v2(ndjson::String, ndjson_name::String, minadj::Int, adj_score_ct_div::Int, rep_thresh::Int, revs_thresh::Int, print_ct_thresh::Int, mut_thresh::Int)
    excluded_seqs = stringlist_to_strings("EPI_ISL_1376522, EPI_ISL_12763802, EPI_ISL_15793981, EPI_ISL_16287253, EPI_ISL_18059726, EPI_ISL_15865257, EPI_ISL_13215742, EPI_ISL_14158264, EPI_ISL_12293549, EPI_ISL_15550525, EPI_ISL_15945504, EPI_ISL_19227219, EPI_ISL_14203206, EPI_ISL_19506572, EPI_ISL_13338249, EPI_ISL_13358962, EPI_ISL_13464004, EPI_ISL_13716400, EPI_ISL_13729045, EPI_ISL_14599772, EPI_ISL_16969756, EPI_ISL_16969757")
    legends = Set(["EPI_ISL_17344660", "EPI_ISL_17997249", "EPI_ISL_19081416", "EPI_ISL_19081419", "EPI_ISL_19081422", "EPI_ISL_19081425", "EPI_ISL_19108198", "EPI_ISL_19108199", "EPI_ISL_19108200", "EPI_ISL_19108208"])
    AA_position(m) = split(m, ":")[2][2:end-1]
    date = Dates.format(today(), "yyyy-mm-dd")
    ref_seq = "..."
    max_l = 12
    
    excluded_pos = BitSet([1:60..., 76:78..., 683:697..., 2453, 3241, 5515, 7926, 11283:11295..., 14960, 15521, 19209:19210..., 19212, 19214, 19217, 21595, 21766, 21772, 21987, 21995:21996..., 22882, 24130, 27291, 28360:28373..., 29632, 29732:29736..., 29757, 29759:29761..., 29830:29000...])
    excluded_AA = Set(["S:K113N", "S:Q115H", "S:N122K", "S:I794N", "ORF1b:N498I", "ORF1b:F685Y", "S:S27A", "S:D142G", "S:Y145D", "S:S408R", "S:N417K", "S:K440N"])
    reversion_exceptions = Set(["ORF1a:R135S", "ORF1a:I842T", "ORF1a:I3255T", "ORF1b:H264Y", "ORF1b:L314P", "ORF1b:V1566I", "S:N405D", "S:I19T", "S:F371S", "S:F375S", "S:A376T", "S:T444K", "S:S446G", "S:S455L", "S:K478T", "S:A484E", "S:V486F", "S:P486F", "S:S490F", "S:R493Q", "S:S496G", "S:H505Y", "S:K764N", "E:I9T", "ORF7a:Y47H", "ORF7a:A82V", "N:L13P", "ORF9b:S10P", "ORF9b:G16D"])
    
    counted_seq_total_private_AA_subs = Dict{String, Int}()
    total_private_AA_subs_all_counted_chronics = 0
    total_counted_sequences = 0
    
    nonleap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>28, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)
    leap_month_day_dict = Dict{Int, Int}(0=>0, 1=>31, 2=>29, 3=>31, 4=>30, 5=>31, 6=>30, 7=>31, 8=>31, 9=>30, 10=>31, 11=>30, 12=>31)

    date_nuc_mut_ct = Dict{Int, Dict{String, Int}}()
    date_nuc_mut_ct_no_dels = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct_no_dels = Dict{Int, Dict{String, Int}}()
    date_AA_mut_ct_pos_only_no_dels = Dict{Int, Dict{String, Int}}()

    seq_ct_by_year = Dict{Int, Int}(i=>0 for i in 2020:2027)
    seq_ct_by_year[0] = 0
    seq_ct_by_year_month = Dict{Tuple{Int, Int}, Int}()
    for i in 2020:2027
        for j in 0:12
            seq_ct_by_year_month[(i, j)] = 0
        end
    end
    seq_ct_by_year_month[(0, 0)] = 0
    seq_ct_by_year_month_day = Dict{Tuple{Int, Int, Int}, Int}()
    for i in 2020:2027
        if i%4 == 0
            for j in 0:12
                for k in 0:leap_month_day_dict[j]
                    seq_ct_by_year_month_day[(i, j, k)] = 0
                end
            end
        else
            for j in 0:12
                for k in 0:nonleap_month_day_dict[j]
                    seq_ct_by_year_month_day[(i, j, k)] = 0
                end
            end
        end
    end
    seq_ct_by_year_month_day[(0, 0, 0)] = 0
############################################################################################################    
    domain_mut_density = Dict{String, Float64}()
    gene_mut_density = Dict{String, Float64}()
############################################################################################################   
    nuc_muts_seq = Dict{String, Set{String}}()
    nuc_dels_seq = Dict{String, Set{String}}()
    nuc_muts_seq_no_dels = Dict{String, Set{String}}()
    
    seq_nuc_muts = Dict{String, Set{String}}()
    seq_nuc_dels = Dict{String, Set{String}}()
    seq_nuc_del_ranges_ct = Dict{String, Int}()
    seq_nuc_muts_no_dels = Dict{String, Set{String}}()
    
    seq_nuc_dropout = Dict{String, Set{Int}}()
    seq_mixed_nucs = Dict{String, Set{String}}()
############################################################################################################     
    AA_muts_seq = Dict{String, Set{String}}()
    AA_dels_seq = Dict{String, Set{String}}()
    AA_muts_seq_no_dels = Dict{String, Set{String}}()
    AA_muts_seq_pos_only = Dict{String, Set{String}}()
    AA_muts_seq_pos_only_no_dels = Dict{String, Set{String}}()
    
    seq_AA_muts = Dict{String, Set{String}}()
    seq_AA_dels = Dict{String, Set{String}}()
    seq_AA_muts_no_dels = Dict{String, Set{String}}()
    seq_AA_muts_pos_only = Dict{String, Set{String}}()
    seq_AA_muts_pos_only_no_dels = Dict{String, Set{String}}()
    
    seq_unknown_AA = Dict{String, Set{String}}()
    seq_mixed_AA_muts = Dict{String, Set{String}}()
    
## AA_muts_ct_pos_only: counts all muts at given AA position; e.g. "ORF1a:T1638" => 34
## seq_AA_muts_pos_only: keys=EPI_ISL, values=2-element array: 1=AA pos ("ORF1a:1638"), 2=AA res ("I")
## AA_muts_seq_pos_only: keys=AA pos ("ORF1a:1638"), values=EPI_ISLs
### WT means mutation relative to wild-type/ancestral
    seq_nuc_muts_WT = Dict{String, Set{String}}()
    seq_nuc_dels_WT = Dict{String, Set{String}}()
    seq_AA_muts_WT = Dict{String, Set{String}}()
    seq_AA_dels_WT = Dict{String, Set{String}}()
    
    nuc_muts_seq_WT = Dict{String, Set{String}}()
    nuc_dels_seq_WT = Dict{String, Set{String}}()
    
    AA_muts_seq_WT = Dict{String, Set{String}}()
    AA_dels_seq_WT = Dict{String, Set{String}}()
    
    seq_AA_muts_WT_pos_only = Dict{String, Set{String}}()
    AA_muts_seq_WT_pos_only = Dict{String, Set{String}}()
############################################################################################################    
############################################################################################################   
    nuc_muts_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_dels_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_muts_rep_seq_grps_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_muts = Dict{Int, Set{String}}()
    rep_seq_grps_dels = Dict{Int, Set{String}}()
    rep_seq_grps_muts_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_del_ranges_ct = Dict{String, Int}()
    
    rep_seq_grps_nuc_dropout = Dict{String, Set{Int}}()
    rep_seq_grps_mixed_nucs = Dict{String, Set{String}}()
############################################################################################################     
    AA_muts_rep_seq_grps = Dict{String, Set{Int}}()
    AA_dels_rep_seq_grps = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_no_dels = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_pos_only = Dict{String, Set{Int}}()
    
    rep_seq_grps_AA = Dict{Int, Set{String}}()
    rep_seq_grps_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_unknown_AA = Dict{Int, Set{String}}()
    rep_seq_grps_mixed_AA_muts = Dict{Int, Set{String}}() 
############################################################################################################     
    rep_seq_grps_nuc_muts_WT = Dict{String, Set{String}}()
    rep_seq_grps_nuc_dels_WT = Dict{String, Set{String}}()
    rep_seq_grps_AA_muts_WT = Dict{String, Set{String}}()
    rep_seq_grps_AA_dels_WT = Dict{String, Set{String}}()
    
    nuc_muts_rep_seq_grps_WT = Dict{String, Set{String}}()
    nuc_dels_rep_seq_grps_WT = Dict{String, Set{String}}()
    
    AA_muts_rep_seq_grps_WT = Dict{String, Set{String}}()
    AA_dels_rep_seq_grps_WT = Dict{String, Set{String}}()
    
    rep_seq_grps_AA_muts_WT_pos_only = Dict{String, Set{String}}()
    AA_muts_rep_seq_grps_WT_pos_only = Dict{String, Set{String}}()
############################################################################################################    
############################################################################################################    
    NSP_muts = Dict{String, Dict{String, Int}}
    NSP_muts_no_dels = Dict{String, Dict{String, Int}}
    NSP_ranges1a = Dict{Int, BitSet}(1 => BitSet([1:180...]), 2 => BitSet([181:818...]), 3 => BitSet([819:2763...]), 4 => BitSet([2764:3263...]), 5 => BitSet([3264:3569...]), 6 => BitSet([3570:3859...]), 7 => BitSet([3860:3942...]), 8 => BitSet([3943:4140...]), 9 => BitSet([4141:4253...]), 10 => BitSet([4254:4392...]), 12 => BitSet([4393:4401...]))
    NSP_ranges1b = Dict{Int, BitSet}(12 => BitSet([1:923...]), 13 => BitSet([924:1524...]), 14 => BitSet([1525:2051...]), 15 => BitSet([2052:2397...]), 16 => BitSet([2398:2696...]))
    NSP1a_add = Dict{Int, Int}(1 => 0, 2 => 180, 3 => 818, 4 => 2763, 5 => 3263, 6 => 3569, 7 => 3859, 8 => 3942, 9 => 4140, 10 => 4353, 11 => 0, 12 => 4392)
    NSP1b_add = Dict{Int, Int}(12 => -9, 13 => 923, 14 => 1524, 15 => 2051, 16 => 2397)
    NSP1ab_add = Dict{Int, Int}(1 => 0, 2 => 180, 3 => 818, 4 => 2763, 5 => 3263, 6 => 3569, 7 => 3859, 8 => 3942, 9 => 4140, 10 => 4353, 11 => 0, 12 => -9, 13 => 923, 14 => 1524, 15 => 2051, 16 => 2397)
    NSP_muts = Dict("NSP$(i)" => Dict{String, Int}() for i in 1:16 if i ≠ 11)
    NSP_muts_no_dels = Dict("NSP$(i)" => Dict{String, Int}() for i in 1:16 if i ≠ 11)
    domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM234HLX"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "S_S1"=>"S:1-669", "S_S2"=>"S:691-1273", "S_NTD"=>"S:2-306", "S_RBD"=>"S:331-527", "S_FCS_region"=>"S:670-690", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
    domain_residues_ORF = Dict{String, String}()
    NSP_muts_sortByCt_Arr = []
    NSP_muts_sortByPos_Arr = []
    NSP_muts_no_dels_sortByCt_Arr = []
    NSP_muts_no_dels_sortByPos_Arr = []
    
    function NSP_to_ORF(domain::String, NSP::Int)
        ORFadd = NSP1ab_add[NSP]
        range = split(domain, ":")[2]
        first1 = split(range, "-")[1]
        first = parse(Int, first1)
        last1 = split(range, "-")[2]
        last = parse(Int, last1)
        ORFfirst1 = first + ORFadd
        ORFfirst = string(ORFfirst1)
        ORFlast1 = last + ORFadd
        ORFlast = string(ORFlast1)
        ORF_range = ""
        if NSP < 12
            ORF_range = "ORF1a:$(ORFfirst)-$(ORFlast)"
        else
            ORF_range = "ORF1b:$(ORFfirst)-$(ORFlast)"
        end
        return ORF_range
    end

    domain_residues_ORF["NSP12_NiRAN"] = "1a:4393-1b:241"
    for (dom, range) in domain_residues_NSP
        if range[1:4] == "NSP3"
            ORF = NSP_to_ORF(range, 3)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:4] == "NSP4"
            ORF = NSP_to_ORF(range, 4)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:5] == "NSP12"
            ORF = NSP_to_ORF(range, 12)
            domain_residues_ORF[dom] = ORF
        end
        if range[1:3] ≠ "NSP"
            domain_residues_ORF[dom] = range
        end
    end
############################################################################################################    
############################################################################################################ 
    nuc_muts_ct = Dict{String, Int}()
    nuc_muts_ct_no_dels = Dict{String, Int}()
    nuc_muts_ct_pos_only_no_dels = Dict{Int, Int}()
    nuc_muts_ct_adj = Dict{String, Float64}()
    nuc_muts_ct_adj_score = Dict{String, Float64}()
    nuc_muts_ct_adj_no_dels = Dict{String, Float64}()
    nuc_muts_ct_adj_score_no_dels = Dict{String, Float64}()
    
    nuc_muts_ct_WT = Dict{String, Int}()
    nuc_dels_ct_WT = Dict{String, Int}()
    
    nuc_muts_ct_sort_by_site = Vector{Pair{String, Int64}}
    nuc_muts_ct_sort_by_seq_ct = Vector{Pair{String, Int64}}
    
    nuc_muts_ct_adj_sort_by_site = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_score_sort_by_site = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_score_sort_by_score = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    nuc_muts_ct_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}
############################################################################################################    
############################################################################################################ 
    nuc_dels_ct = Dict{String, Int}()
    mixed_nucs_ct = Dict{String, Int}()
    nuc_dropout_ct = Dict{Int, Int}()
    
    AA_dels_ct = Dict{String, Int}()
    mixed_AA_ct = Dict{String, Int}()
    unknown_AA_ct = Dict{String, Int}()
############################################################################################################    
############################################################################################################ 
    AA_muts_ct = Dict{String, Int}()
    AA_muts_ct_no_dels = Dict{String, Int}()
    AA_muts_ct_no_dels_no_revs = Dict{String, Int}()
    AA_muts_ct_pos_only = Dict{String, Int}()
    AA_muts_ct_pos_only_no_dels = Dict{String, Int}()
    AA_muts_ct_pos_only_no_dels_no_revs = Dict{String, Int}()
    AA_muts_ct_pos_only_reversion = Dict{String, Int}()

    AA_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{String, Int}()
    
    AA_muts_ct_WT = Dict{String, Int}()
    AA_dels_ct_WT = Dict{String, Int}()
    AA_muts_ct_WT_no_dels = Dict{String, Int}()
    AA_muts_ct_WT_pos_only = Dict{String, Int}()
    AA_muts_ct_WT_pos_only_no_dels = Dict{String, Int}()
    
    AA_muts_ct_adj = Dict{String, Float64}()
    AA_muts_ct_adj_no_dels = Dict{String, Float64}()
    AA_muts_ct_adj_score = Dict{String, Float64}()
    AA_muts_ct_adj_score_no_dels = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_no_dels = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_score = Dict{String, Float64}()
    AA_muts_ct_pos_only_adj_score_no_dels = Dict{String, Float64}()
    
    AA_muts_ct_sort_by_site = Vector{Pair{String, Int64}}
    AA_muts_ct_sort_by_seq_ct = Vector{Pair{String, Int64}}
    AA_muts_ct_no_dels_sort_by_site = Vector{Pair{String, Int64}}
    AA_muts_ct_no_dels_sort_by_seq_ct = Vector{Pair{String, Int64}}
    AA_muts_ct_no_dels_no_revs_sort_by_site = Vector{Pair{String, Int64}}
    AA_muts_ct_no_dels_no_revs_sort_by_seq_ct = Vector{Pair{String, Int64}}
    
    AA_muts_ct_adj_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}
    
    AA_muts_ct_adj_score_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_score_sort_by_score = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}
    
    AA_muts_ct_pos_only_sort_by_site = Vector{Pair{String, Int64}}
    AA_muts_ct_pos_only_sort_by_seq_ct = Vector{Pair{String, Int64}}
    AA_muts_ct_pos_only_no_dels_sort_by_site = Vector{Pair{String, Int64}}
    AA_muts_ct_pos_only_no_dels_sort_by_seq_ct = Vector{Pair{String, Int64}}

    AA_muts_ct_pos_only_adj_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_sort_by_seq_ct = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct = Vector{Pair{String, Float64}}
    
    AA_muts_ct_pos_only_adj_score_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_score_sort_by_score = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site = Vector{Pair{String, Float64}}
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score = Vector{Pair{String, Float64}}

    NSP1_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP2_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP3_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP4_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP5_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP6_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP7_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP8_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP9_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP10_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP11_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP12_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP13_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP14_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP15_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP16_muts_sortByCt = Vector{Pair{String, Int64}}()
    NSP1_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP2_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP3_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP4_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP5_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP6_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP7_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP8_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP9_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP10_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP11_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP12_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP13_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP14_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP15_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP16_muts_sortByPos = Vector{Pair{String, Int64}}()
    NSP1_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP2_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP3_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP4_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP5_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP6_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP7_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP8_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP9_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP10_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP11_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP12_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP13_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP14_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP15_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP16_muts_no_dels_sortByCt = Vector{Pair{String, Int64}}()
    NSP1_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP2_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP3_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP4_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP5_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP6_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP7_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP8_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP9_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP10_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP11_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP12_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP13_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP14_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP15_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    NSP16_muts_no_dels_sortByPos = Vector{Pair{String, Int64}}()
    
    ORF1a_mut_ct = 0
    ORF1b_mut_ct = 0
    NSP1_mut_ct = 0
    NSP2_mut_ct = 0
    NSP3_mut_ct = 0
    NSP4_mut_ct = 0
    NSP5_mut_ct = 0
    NSP6_mut_ct = 0
    NSP7_mut_ct = 0
    NSP8_mut_ct = 0
    NSP9_mut_ct = 0
    NSP10_mut_ct = 0
    NSP12_mut_ct = 0
    NSP13_mut_ct = 0
    NSP14_mut_ct = 0
    NSP15_mut_ct = 0
    NSP16_mut_ct = 0
    ORF3a_mut_ct = 0
    ORF6_mut_ct = 0
    ORF7a_mut_ct = 0
    ORF7b_mut_ct = 0
    ORF8_mut_ct = 0
    ORF9b_mut_ct = 0
    S_mut_ct = 0
    E_mut_ct = 0
    M_mut_ct = 0
    N_mut_ct = 0
    
    NSP3_Ubl1_ct = 0 
    NSP3_HVR_ct = 0
    NSP3_Mac1_ct = 0
    NSP3_Mac2_ct = 0
    NSP3_Mac3_ct = 0
    NSP3_DPUP_ct = 0
    NSP3_Ubl2_ct = 0
    NSP3_PLpro_ct = 0
    NSP3_NAB_ct = 0
    NSP3_BSM_ct = 0
    NSP3_TM1_ct = 0
    NSP3_Ecto3_ct = 0
    NSP3_TM234HLX_ct = 0
    NSP3_Y1_ct = 0
    NSP3_CoVY_ct = 0
    NSP4_TM1_ct = 0
    NSP4_Ecto4_ct = 0
    NSP4_TM2_TM6_ct = 0
    NSP4_CTD_ct = 0
    
    S_S1_ct = 0
    S_S2_ct = 0
    S_NTD_ct = 0
    S_RBD_ct = 0
    S_FCS_region_ct = 0
    
    NSP12_NiRAN_ct = 0
    NSP12_intrfce_ct = 0
    NSP12_fingers_ct =
    NSP12_palm_ct = 0
    NSP12_palmLnk_ct = 0
    NSP12_thumb_ct = 0
    
    N_N1_ct = 0
    N_N2_ct = 0
    N_N3_ct = 0
    N_N4_ct = 0
    N_N5_ct = 0
    N_SR_ct = 0
    N_L_helix_ct = 0
    N_CBP_ct = 0
    N_9b_overlap_ct = 0
    
    ORF1a_mut_ct_no_dels = 0
    ORF1b_mut_ct_no_dels = 0
    NSP1_mut_ct_no_dels = 0
    NSP2_mut_ct_no_dels = 0
    NSP3_mut_ct_no_dels = 0
    NSP4_mut_ct_no_dels = 0
    NSP5_mut_ct_no_dels = 0
    NSP6_mut_ct_no_dels = 0
    NSP7_mut_ct_no_dels = 0
    NSP8_mut_ct_no_dels = 0
    NSP9_mut_ct_no_dels = 0
    NSP10_mut_ct_no_dels = 0
    NSP12_mut_ct_no_dels = 0
    NSP13_mut_ct_no_dels = 0
    NSP14_mut_ct_no_dels = 0
    NSP15_mut_ct_no_dels = 0
    NSP16_mut_ct_no_dels = 0
    ORF3a_mut_ct_no_dels = 0
    ORF6_mut_ct_no_dels = 0
    ORF7a_mut_ct_no_dels = 0
    ORF7b_mut_ct_no_dels = 0
    ORF8_mut_ct_no_dels = 0
    ORF9b_mut_ct_no_dels = 0
    S_mut_ct_no_dels = 0
    E_mut_ct_no_dels = 0
    M_mut_ct_no_dels = 0
    N_mut_ct_no_dels = 0
    
    NSP3_Ubl1_ct_no_dels = 0 
    NSP3_HVR_ct_no_dels = 0
    NSP3_Mac1_ct_no_dels = 0
    NSP3_Mac2_ct_no_dels = 0
    NSP3_Mac3_ct_no_dels = 0
    NSP3_DPUP_ct_no_dels = 0
    NSP3_Ubl2_ct_no_dels = 0
    NSP3_PLpro_ct_no_dels = 0
    NSP3_NAB_ct_no_dels = 0
    NSP3_BSM_ct_no_dels = 0
    NSP3_TM1_ct_no_dels = 0
    NSP3_Ecto3_ct_no_dels = 0
    NSP3_TM234HLX_ct_no_dels = 0
    NSP3_Y1_ct_no_dels = 0
    NSP3_CoVY_ct_no_dels = 0
    NSP4_TM1_ct_no_dels = 0
    NSP4_Ecto4_ct_no_dels = 0
    NSP4_TM2_TM6_ct_no_dels = 0
    NSP4_CTD_ct_no_dels = 0
    
    S_S1_ct_no_dels = 0
    S_S2_ct_no_dels = 0
    S_NTD_ct_no_dels = 0
    S_RBD_ct_no_dels = 0
    S_FCS_region_ct_no_dels = 0
    
    NSP12_NiRAN_ct_no_dels = 0
    NSP12_intrfce_ct_no_dels = 0
    NSP12_fingers_ct_no_dels =
    NSP12_palm_ct_no_dels = 0
    NSP12_palmLnk_ct_no_dels = 0
    NSP12_thumb_ct_no_dels = 0
    
    N_N1_ct_no_dels = 0
    N_N2_ct_no_dels = 0
    N_N3_ct_no_dels = 0
    N_N4_ct_no_dels = 0
    N_N5_ct_no_dels = 0
    N_SR_ct_no_dels = 0
    N_L_helix_ct_no_dels = 0
    N_CBP_ct_no_dels = 0
    N_9b_overlap_ct_no_dels = 0
    
    gene_protein_length = Dict{String, Int}("NSP1"=>180, "NSP2"=>638, "NSP3"=>1945, "NSP4"=>500, "NSP5"=>306, "NSP6"=>290, "NSP7"=>83, "NSP8"=>198, "NSP9"=>113, "NSP10"=>139, "NSP12"=>932, "NSP13"=>601, "NSP14"=>527, "NSP15"=>346, "NSP16"=>298, "ORF3a"=>275, "ORF6"=>61, "ORF7a"=>121, "ORF7b"=>43, "ORF8"=>121, "ORF9b"=>97, "S"=>1273, "E"=>75, "M"=>222, "N"=>419)
    gene_protein_mut_ct = Dict{String, Int}("NSP1"=>NSP1_mut_ct, "NSP2"=>NSP2_mut_ct, "NSP3"=>NSP3_mut_ct, "NSP4"=>NSP4_mut_ct, "NSP5"=>NSP5_mut_ct, "NSP6"=>NSP6_mut_ct, "NSP7"=>NSP7_mut_ct, "NSP8"=>NSP8_mut_ct, "NSP9"=>NSP9_mut_ct, "NSP10"=>NSP10_mut_ct, "NSP12"=>NSP12_mut_ct, "NSP13"=>NSP13_mut_ct, "NSP14"=>NSP14_mut_ct, "NSP15"=>NSP15_mut_ct, "NSP16"=>NSP16_mut_ct, "ORF3a"=>ORF3a_mut_ct, "ORF6"=>ORF6_mut_ct, "ORF7a"=>ORF7a_mut_ct, "ORF7b"=>ORF7b_mut_ct, "ORF8"=>ORF8_mut_ct, "ORF9b"=>ORF9b_mut_ct, "S"=>S_mut_ct, "E"=>E_mut_ct, "M"=>M_mut_ct, "N"=>N_mut_ct)
    gene_protein_mut_ct_no_dels = Dict{String, Int}("NSP1"=>NSP1_mut_ct_no_dels, "NSP2"=>NSP2_mut_ct_no_dels, "NSP3"=>NSP3_mut_ct_no_dels, "NSP4"=>NSP4_mut_ct_no_dels, "NSP5"=>NSP5_mut_ct_no_dels, "NSP6"=>NSP6_mut_ct_no_dels, "NSP7"=>NSP7_mut_ct_no_dels, "NSP8"=>NSP8_mut_ct_no_dels, "NSP9"=>NSP9_mut_ct_no_dels, "NSP10"=>NSP10_mut_ct_no_dels, "NSP12"=>NSP12_mut_ct_no_dels, "NSP13"=>NSP13_mut_ct_no_dels, "NSP14"=>NSP14_mut_ct_no_dels, "NSP15"=>NSP15_mut_ct_no_dels, "NSP16"=>NSP16_mut_ct_no_dels, "ORF3a"=>ORF3a_mut_ct_no_dels, "ORF6"=>ORF6_mut_ct_no_dels, "ORF7a"=>ORF7a_mut_ct_no_dels, "ORF7b"=>ORF7b_mut_ct_no_dels, "ORF8"=>ORF8_mut_ct_no_dels, "ORF9b"=>ORF9b_mut_ct_no_dels, "S"=>S_mut_ct_no_dels, "E"=>E_mut_ct_no_dels, "M"=>M_mut_ct_no_dels, "N"=>N_mut_ct_no_dels)    
    gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)
    domain_length = Dict{String, Int}("NSP3_Ubl1"=>111, "NSP3_HVR"=>95, "NSP3_Mac1"=>168, "NSP3_Mac2"=>176, "NSP3_Mac3"=>123, "NSP3_DPUP"=>70, "NSP3_Ubl2"=>60, "NSP3_PLpro"=>259, "NSP3_NAB"=>151, "NSP3_BSM"=>131, "NSP3_TM1"=>106, "NSP3_Ecto3"=>59, "NSP3_TM234HLX"=>87, "NSP3_Y1"=>179, "NSP3_CoVY"=>181, "NSP4_TM1"=>31, "NSP4_Ecto4"=>240, "NSP4_TM2_TM6"=>139, "NSP4_CTD"=>90, "NSP12_NiRAN"=>250, "NSP12_intrfce"=>148, "NSP12_fingers"=>183, "NSP12_palm"=>171, "NSP12_palmLnk"=>60, "NSP12_thumb"=>120, "N_N1"=>43, "N_N2"=>131, "N_N3"=>71, "N_N4"=>118, "N_N5"=>55, "N_SR"=>31, "N_L_helix"=>21, "N_CBP"=>33, "N_9b_overlap"=>98)
    domain_mut_ct = Dict{String, Int}("NSP3_Ubl1"=>NSP3_Ubl1_ct, "NSP3_HVR"=>NSP3_HVR_ct, "NSP3_Mac1"=>NSP3_Mac1_ct, "NSP3_Mac2"=>NSP3_Mac2_ct, "NSP3_Mac3"=>NSP3_Mac3_ct, "NSP3_DPUP"=>NSP3_DPUP_ct, "NSP3_Ubl2"=>NSP3_Ubl2_ct, "NSP3_PLpro"=>NSP3_PLpro_ct, "NSP3_NAB"=>NSP3_NAB_ct, "NSP3_BSM"=>NSP3_BSM_ct, "NSP3_TM1"=>NSP3_TM1_ct, "NSP3_Ecto3"=>NSP3_Ecto3_ct, "NSP3_TM234HLX"=>NSP3_TM234HLX_ct, "NSP3_Y1"=>NSP3_Y1_ct, "NSP3_CoVY"=>NSP3_CoVY_ct, "NSP4_TM1"=>NSP4_TM1_ct, "NSP4_Ecto4"=>NSP4_Ecto4_ct, "NSP4_TM2_TM6"=>NSP4_TM2_TM6_ct, "NSP4_CTD"=>NSP4_CTD_ct, "NSP12_NiRAN"=>NSP12_NiRAN_ct, "NSP12_intrfce"=>NSP12_intrfce_ct, "NSP12_fingers"=>NSP12_fingers_ct, "NSP12_palm"=>NSP12_palm_ct, "NSP12_palmLnk"=>NSP12_palmLnk_ct, "NSP12_thumb"=>NSP12_thumb_ct, "S_S1"=>S_S1_ct, "S_S2"=>S_S2_ct, "S_NTD"=>S_NTD_ct, "S_RBD"=>S_RBD_ct, "S_FCS_region"=>S_FCS_region_ct, "N_N1"=>N_N1_ct, "N_N2"=>N_N2_ct, "N_N3"=>N_N3_ct, "N_N4"=>N_N4_ct, "N_N5"=>N_N5_ct, "N_SR"=>N_SR_ct, "N_L_helix"=>N_L_helix_ct, "N_CBP"=>N_CBP_ct, "N_9b_overlap"=>N_9b_overlap_ct)
    domain_mut_ct_no_dels = Dict{String, Int}("NSP3_Ubl1"=>NSP3_Ubl1_ct_no_dels, "NSP3_HVR"=>NSP3_HVR_ct_no_dels, "NSP3_Mac1"=>NSP3_Mac1_ct_no_dels, "NSP3_Mac2"=>NSP3_Mac2_ct_no_dels, "NSP3_Mac3"=>NSP3_Mac3_ct_no_dels, "NSP3_DPUP"=>NSP3_DPUP_ct_no_dels, "NSP3_Ubl2"=>NSP3_Ubl2_ct_no_dels, "NSP3_PLpro"=>NSP3_PLpro_ct_no_dels, "NSP3_NAB"=>NSP3_NAB_ct_no_dels, "NSP3_BSM"=>NSP3_BSM_ct_no_dels, "NSP3_TM1"=>NSP3_TM1_ct_no_dels, "NSP3_Ecto3"=>NSP3_Ecto3_ct_no_dels, "NSP3_TM234HLX"=>NSP3_TM234HLX_ct_no_dels, "NSP3_Y1"=>NSP3_Y1_ct_no_dels, "NSP3_CoVY"=>NSP3_CoVY_ct_no_dels, "NSP4_TM1"=>NSP4_TM1_ct_no_dels, "NSP4_Ecto4"=>NSP4_Ecto4_ct_no_dels, "NSP4_TM2_TM6"=>NSP4_TM2_TM6_ct_no_dels, "NSP4_CTD"=>NSP4_CTD_ct_no_dels, "NSP12_NiRAN"=>NSP12_NiRAN_ct_no_dels, "NSP12_intrfce"=>NSP12_intrfce_ct_no_dels, "NSP12_fingers"=>NSP12_fingers_ct_no_dels, "NSP12_palm"=>NSP12_palm_ct_no_dels, "NSP12_palmLnk"=>NSP12_palmLnk_ct_no_dels, "NSP12_thumb"=>NSP12_thumb_ct_no_dels, "S_S1"=>S_S1_ct_no_dels, "S_S2"=>S_S2_ct_no_dels, "S_NTD"=>S_NTD_ct_no_dels, "S_RBD"=>S_RBD_ct_no_dels, "S_FCS_region"=>S_FCS_region_ct_no_dels, "N_N1"=>N_N1_ct_no_dels, "N_N2"=>N_N2_ct_no_dels, "N_N3"=>N_N3_ct_no_dels, "N_N4"=>N_N4_ct_no_dels, "N_N5"=>N_N5_ct_no_dels, "N_SR"=>N_SR_ct_no_dels, "N_L_helix"=>N_L_helix_ct_no_dels, "N_CBP"=>N_CBP_ct_no_dels, "N_9b_overlap"=>N_9b_overlap_ct_no_dels)
    domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP12_NiRAN"=>20, "NSP12_intrfce"=>21, "NSP12_fingers"=>22, "NSP12_palm"=>23, "NSP12_palmLnk"=>24, "NSP12_thumb"=>25, "S_S1"=>26, "S_S2"=>27, "S_NTD"=>28, "S_RBD"=>29, "S_FCS_region"=>30, "N_N1"=>31, "N_N2"=>32, "N_N3"=>33, "N_N4"=>34, "N_N5"=>35, "N_SR"=>36, "N_L_helix"=>37, "N_CBP"=>38, "N_9b_overlap"=>39)
    domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM234HLX"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "S_S1"=>"S:1-669", "S_S2"=>"S:691-1273", "S_NTD"=>"S:2-306", "S_RBD"=>"S:331-527", "S_FCS_region"=>"S:670-690", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
    
    ORF1a_mut_ct_MINADJ = 0
    ORF1b_mut_ct_MINADJ = 0
    NSP1_mut_ct_MINADJ = 0
    NSP2_mut_ct_MINADJ = 0
    NSP3_mut_ct_MINADJ = 0
    NSP4_mut_ct_MINADJ = 0
    NSP5_mut_ct_MINADJ = 0
    NSP6_mut_ct_MINADJ = 0
    NSP7_mut_ct_MINADJ = 0
    NSP8_mut_ct_MINADJ = 0
    NSP9_mut_ct_MINADJ = 0
    NSP10_mut_ct_MINADJ = 0
    NSP12_mut_ct_MINADJ = 0
    NSP13_mut_ct_MINADJ = 0
    NSP14_mut_ct_MINADJ = 0
    NSP15_mut_ct_MINADJ = 0
    NSP16_mut_ct_MINADJ = 0
    ORF3a_mut_ct_MINADJ = 0
    ORF6_mut_ct_MINADJ = 0
    ORF7a_mut_ct_MINADJ = 0
    ORF7b_mut_ct_MINADJ = 0
    ORF8_mut_ct_MINADJ = 0
    ORF9b_mut_ct_MINADJ = 0
    S_mut_ct_MINADJ = 0
    E_mut_ct_MINADJ = 0
    M_mut_ct_MINADJ = 0
    N_mut_ct_MINADJ = 0
    
    NSP3_Ubl1_ct_MINADJ = 0 
    NSP3_HVR_ct_MINADJ = 0
    NSP3_Mac1_ct_MINADJ = 0
    NSP3_Mac2_ct_MINADJ = 0
    NSP3_Mac3_ct_MINADJ = 0
    NSP3_DPUP_ct_MINADJ = 0
    NSP3_Ubl2_ct_MINADJ = 0
    NSP3_PLpro_ct_MINADJ = 0
    NSP3_NAB_ct_MINADJ = 0
    NSP3_BSM_ct_MINADJ = 0
    NSP3_TM1_ct_MINADJ = 0
    NSP3_Ecto3_ct_MINADJ = 0
    NSP3_TM234HLX_ct_MINADJ = 0
    NSP3_Y1_ct_MINADJ = 0
    NSP3_CoVY_ct_MINADJ = 0
    NSP4_TM1_ct_MINADJ = 0
    NSP4_Ecto4_ct_MINADJ = 0
    NSP4_TM2_TM6_ct_MINADJ = 0
    NSP4_CTD_ct_MINADJ = 0
    
    S_S1_ct_MINADJ = 0
    S_S2_ct_MINADJ = 0
    S_NTD_ct_MINADJ = 0
    S_RBD_ct_MINADJ = 0
    S_FCS_region_ct_MINADJ = 0
    
    NSP12_NiRAN_ct_MINADJ = 0
    NSP12_intrfce_ct_MINADJ = 0
    NSP12_fingers_ct_MINADJ =
    NSP12_palm_ct_MINADJ = 0
    NSP12_palmLnk_ct_MINADJ = 0
    NSP12_thumb_ct_MINADJ = 0
    
    N_N1_ct_MINADJ = 0
    N_N2_ct_MINADJ = 0
    N_N3_ct_MINADJ = 0
    N_N4_ct_MINADJ = 0
    N_N5_ct_MINADJ = 0
    N_SR_ct_MINADJ = 0
    N_L_helix_ct_MINADJ = 0
    N_CBP_ct_MINADJ = 0
    N_9b_overlap_ct_MINADJ = 0
    
    seq_country = Dict{String, String}()
    seq_US_state = Dict{String, String}()
    seq_lab_dict = Dict{String, String}()
    seq_collection_date = Dict{String, String}()
    seq_date_index = Dict{String, Int}()
    seq_date_tuple = Dict{String, Tuple{Int, Int, Int}}()
    
    seq_clade = Dict{String, String}()
    seq_clade_display = Dict{String, String}()
    seq_pango = Dict{String, String}()
    seq_pango_unaliased = Dict{String, String}()

    rep_seq_grps_clade = Dict{Int, String}()
    rep_seq_grps_pango = Dict{Int, String}()
    rep_seq_grps_pango_unaliased = Dict{Int, String}()
    
    seq_clade_ct = Dict{String, Int}()
    seq_pango_ct = Dict{String, Int}()
    seq_pango_unaliased_ct = Dict{String, Int}()
    clade_set_complete = Set(["recombinant", "19A", "19B", "20A", "20B", "20C", "20D", "20E", "20F", "20G", "20H", "20I", "20J", "21A", "21B", "21C", "21D", "21E", "21F", "21G", "21H", "21I", "21J", "21K", "21L", "22A", "22B", "22C", "22D", "22E", "22F", "23A", "23B", "23C", "23D", "23E", "23F", "23G", "23H", "23I", "24A", "24B", "24C", "24D", "24E", "24F", "24G", "24H", "24I", "24J"])
    for clade in clade_set_complete
        seq_clade_ct[clade] = 0
    end

    seq_ct = 0
    total_subs = 0
    total_nuc_dels = 0

    all_seqs = Vector{String}()
    all_seqs_set = Set{String}()
    all_qualifying_seqs = Vector{String}()
    all_qualifying_seqs_set = Set{String}()
    all_nonqualifying_seqs = Vector{String}()
    all_nonqualifying_seqs_set = Set{String}()    
    
    non_rep_seqs = Set{String}()
    rep_seqs = Set{String}()
    rep_seq_grps_seqs = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_seqs = Dict{Int, String}()

    rep_seq_grps_maxmut_nuc = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_nuc_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_del_ranges_ct = Dict{Int, Int}()
    rep_seq_grps_maxmut_nuc_dropout = Dict{Int, Set{Int}}()
    rep_seq_grps_maxmut_mixed_nucs = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_AA = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_pos_only_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_unknown_AA = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_mixed_AA = Dict{Int, Set{String}}()
    
    rep_seq_grps_maxmut_nuc_muts_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_nuc_dels_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_muts_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_dels_WT = Dict{Int, Set{String}}()
    rep_seq_grps_maxmut_AA_muts_WT_pos_only = Dict{Int, Set{String}}()
############################################################################################################
############################################################################################################
    nuc_muts_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_dels_rep_seq_grps = Dict{Int, Set{String}}()
    nuc_muts_rep_seq_grps_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_muts = Dict{Int, Set{String}}()
    rep_seq_grps_dels = Dict{Int, Set{String}}()
    rep_seq_grps_muts_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_del_ranges_ct = Dict{String, Int}()
    
    rep_seq_grps_nuc_dropout = Dict{String, Set{Int}}()
    rep_seq_grps_mixed_nucs = Dict{String, Set{String}}()
############################################################################################################     
    AA_muts_rep_seq_grps = Dict{String, Set{Int}}()
    AA_dels_rep_seq_grps = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_no_dels = Dict{String, Set{Int}}()
    AA_muts_rep_seq_grps_pos_only = Dict{String, Set{Int}}()
    
    rep_seq_grps_AA = Dict{Int, Set{String}}()
    rep_seq_grps_AA_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_no_dels = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only = Dict{Int, Set{String}}()
    rep_seq_grps_AA_pos_only_no_dels = Dict{Int, Set{String}}()
    
    rep_seq_grps_unknown_AA = Dict{Int, Set{String}}()
    rep_seq_grps_mixed_AA_muts = Dict{Int, Set{String}}() 
############################################################################################################
############################################################################################################
    non_rep_seqs_AA = Dict{String, Set{String}}()
    non_rep_seqs_AA_pos_only = Dict{String, Set{String}}()
    non_rep_seqs_AA_pos_only_no_dels = Dict{String, Set{String}}()
    
    global chronic_search_muts = Set{String}()
    global chronic_search_muts_min2 = Set{String}()
    global chronic_search_muts_min2_no_dels = Set{String}()
    global chronic_search_muts_v2 = Set{String}()
    
    too_many_reversions = Vector{String}()
    
    subs_9 = Vector{String}()
    subs_8 = Vector{String}()
    subs_7 = Vector{String}()
    subs_6 = Vector{String}()
    subs_5 = Vector{String}()
    subs_4 = Vector{String}()
    subs_3 = Vector{String}()
    subs_9_ct = 0
    subs_8_ct = 0
    subs_7_ct = 0
    subs_6_ct = 0
    subs_5_ct = 0
    subs_4_ct = 0
    subs_3_ct = 0
    
    gene_AA_dict = Dict{String, String}()
    gene_AA_dict["ORF1a"] = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV*"
    gene_AA_dict["ORF1b"] = "RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
    gene_AA_dict["S"] = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
    gene_AA_dict["E"] = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
    gene_AA_dict["M"] = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
    gene_AA_dict["N"] = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
    gene_AA_dict["ORF3a"] = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
    gene_AA_dict["ORF6"] = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
    gene_AA_dict["ORF7a"] = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
    gene_AA_dict["ORF7b"] = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
    gene_AA_dict["ORF8"] = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
    gene_AA_dict["ORF9b"] = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
    
#    NSP_ranges1a_arr = Dict{Int, Vector{Int}}(1 => [1:180...], 2 => [181:818...], 3 => [819:2763...], 4 => [2764:3263...], 5 => [3264:3569...], 6 => [3570:3859...], 7 => [3860:3942...], 8 => [3943:4140...], 9 => [4141:4253...], 10 => [4254:4392...], 12 => [4393:4401...])
#    NSP_ranges1b_arr = Dict{Int, Vector{Int}}(12 => [1:923...], 13 => [924:1524...], 14 => [1525:2051...], 15 => [2052:2397...], 16 => [2398:2697...])
    NSP_AA_dict = Dict{Int, String}()
    for i in 1:10
        NSPkey = "NSP"*string(i)
        first = minimum(NSP_ranges1a[i])
        last = maximum(NSP_ranges1a[i])
        NSP_AA_str = gene_AA_dict["ORF1a"][first:last]
        NSP_AA_dict[i] = NSP_AA_str
    end
    NSP_AA_dict[11] = ""
    NSP12_1a = gene_AA_dict["ORF1a"][4393:4401]
    NSP12_1b = gene_AA_dict["ORF1b"][1:923]
    NSP12_AA_str = NSP12_1a*NSP12_1b
    NSP_AA_dict[12] = NSP12_AA_str
    for i in 13:16
        NSPkey = "NSP"*string(i)
        first = minimum(NSP_ranges1b[i])
        last = maximum(NSP_ranges1b[i])
        NSP_AA_str = gene_AA_dict["ORF1b"][first:last]
        NSP_AA_dict[i] = NSP_AA_str
    end
    
    line_ct = 0
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
    for line in eachline(ndjson)
        j = JSON3.read(line)
        name = EPI_ISL(j.seqName)
        push!(all_seqs_set, name)
        push!(all_seqs, name)
        seq_country[name] = country(j.seqName)
        if seq_country[name] == "USA" || seq_country[name] == "Canada"
            seq_US_state[name] = US_state(j.seqName)
        else
            seq_US_state[name] = ""
        end
        
        seq_clade[name] = j.clade
        seq_clade_display[name] = j.customNodeAttributes.clade_display
        seq_pango[name] = j.customNodeAttributes.Nextclade_pango
        seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
#        seq_date = sequence_date(j.seqName)
        lab = ""
        if !isempty(seq_lab(j.seqName))
            lab = seq_lab(j.seqName)
        end
        seq_lab_dict[name] = lab
        seq_date = ""
####################################################
        try
            seq_clade[name] = j.clade
        catch
            seq_clade[name] = "Unknown"
        end
####################################################
        try
            seq_pango[name] = j.customNodeAttributes.Nextclade_pango
        catch
            seq_pango[name] = "Unknown"
        end
####################################################
        try
            seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
        catch
            seq_pango_unaliased[name] = "Unknown"
        end
####################################################
        seq_date = ""
        try
            seq_date = sequence_date(j.seqName)
        catch
            seq_date = "0-0-0"
        end
####################################################
        lab = ""
        if !isempty(seq_lab(j.seqName))
            lab = seq_lab(j.seqName)
        end
        seq_lab_dict[name] = lab
        seq_year = 0        
        seq_month = 0
        seq_day = 0
        seq_yr = join(seq_date[1:4])
        if all(isdigit, seq_yr)
            seq_year = parse(Int, seq_yr)
        end
        if '-' in seq_date
            seq_mo = split(seq_date, "-")[2]
            if all(isdigit, seq_mo)
                seq_month = parse(Int, seq_mo)
            end
        end
####################################################
        dash_ct = 0
        for i in seq_date
            if i == '-'
                dash_ct += 1
            end
        end
####################################################
        if dash_ct == 2
            seq_da = split(seq_date, "-")[3]
            if all(isdigit, seq_da)
                seq_day = parse(Int, seq_da)
            end
        end
####################################################
        seq_collection_date[name] = seq_date
        if seq_month == 0 && seq_day == 0
            seq_collection_date[name] = string(seq_year)*"-0-0"
        end
        if seq_month ≠ 0 && seq_day == 0
            seq_collection_date[name] = string(seq_year)*"-"*string(seq_month)*"-0"
        end
        date_index = date_to_index[(seq_year, seq_month, seq_day)]
        seq_date_index[name] = date_index
        date_tuple = (seq_year, seq_month, seq_day)
        seq_date_tuple[name] = date_tuple
####################################################
#        if length(seq_date) ≥ 7
#            seq_mo = join(seq_date[6:7])
#            if all(isdigit, seq_mo)
#                seq_month = parse(Int, seq_mo)
#            end
#        end
#        if length(seq_date) ≥ 10
#            if all(isdigit, seq_da)
#            seq_da = join(seq_date[9:10])
#            seq_day = parse(Int, seq_da)
#        end
####################################################
#31 privateAaMutations
#          Type: Dict{String, Any}
#     j.privateAaMutations
#          j.privateAaMutations.ORF1a  ...
#          j.privateAaMutations.S
#               j.privateAaMutations.S.totalReversionSubstitutions
#               j.privateAaMutations.S.totalPrivateDeletions
#               j.privateAaMutations.S.totalPrivateSubstitutions
#               j.privateAaMutations.S.reversionSubstitutions
#                    j.privateAaMutations.S.reversionSubstitutions.1  ...
#                        j.privateAaMutations.S.reversionSubstitutions.1.refAa  ...
#               j.privateAaMutations.S.privateDeletionRanges
#                    j.privateAaMutations.S.privateDeletionRanges.1  ...
#                        j.privateAaMutations.S.privateDeletionRanges.1.range.begin
#                        j.privateAaMutations.S.privateDeletionRanges.1.range.end
        seq_ct += 1
        nuc_del_ranges = Set{String}()
        WT_nuc_del_ranges = Set{String}()
        genes = ["ORF1a", "ORF1b", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"]
        gene_AA_subs = Set([j.privateAaMutations.ORF1a.privateSubstitutions, j.privateAaMutations.ORF1b.privateSubstitutions, j.privateAaMutations.ORF3a.privateSubstitutions, j.privateAaMutations.ORF6.privateSubstitutions, j.privateAaMutations.ORF7a.privateSubstitutions, j.privateAaMutations.ORF7b.privateSubstitutions, j.privateAaMutations.ORF8.privateSubstitutions, j.privateAaMutations.ORF9b.privateSubstitutions, j.privateAaMutations.S.privateSubstitutions, j.privateAaMutations.E.privateSubstitutions, j.privateAaMutations.M.privateSubstitutions, j.privateAaMutations.N.privateSubstitutions])
        total_AA_subs = j.privateAaMutations.ORF1a.totalPrivateSubstitutions + j.privateAaMutations.ORF1b.totalPrivateSubstitutions + j.privateAaMutations.ORF3a.totalPrivateSubstitutions + j.privateAaMutations.ORF6.totalPrivateSubstitutions + j.privateAaMutations.ORF7a.totalPrivateSubstitutions + j.privateAaMutations.ORF7b.totalPrivateSubstitutions + j.privateAaMutations.ORF8.totalPrivateSubstitutions + j.privateAaMutations.ORF9b.totalPrivateSubstitutions + j.privateAaMutations.S.totalPrivateSubstitutions + j.privateAaMutations.E.totalPrivateSubstitutions + j.privateAaMutations.M.totalPrivateSubstitutions + j.privateAaMutations.N.totalPrivateSubstitutions
        total_del_ranges = length(j.privateAaMutations.ORF1a.privateDeletionRanges) + length(j.privateAaMutations.ORF1b.privateDeletionRanges) + length(j.privateAaMutations.ORF3a.privateDeletionRanges) + length(j.privateAaMutations.ORF6.privateDeletionRanges) + length(j.privateAaMutations.ORF7a.privateDeletionRanges) + length(j.privateAaMutations.ORF7b.privateDeletionRanges) + length(j.privateAaMutations.ORF8.privateDeletionRanges) + length(j.privateAaMutations.ORF9b.privateDeletionRanges) + length(j.privateAaMutations.S.privateDeletionRanges) + length(j.privateAaMutations.E.privateDeletionRanges) + length(j.privateAaMutations.M.privateDeletionRanges) + length(j.privateAaMutations.N.privateDeletionRanges)
        total_AA_reversions = j.privateAaMutations.ORF1a.totalReversionSubstitutions + j.privateAaMutations.ORF1b.totalReversionSubstitutions + j.privateAaMutations.ORF3a.totalReversionSubstitutions + j.privateAaMutations.ORF6.totalReversionSubstitutions + j.privateAaMutations.ORF7a.totalReversionSubstitutions + j.privateAaMutations.ORF7b.totalReversionSubstitutions + j.privateAaMutations.ORF8.totalReversionSubstitutions + j.privateAaMutations.ORF9b.totalReversionSubstitutions + j.privateAaMutations.S.totalReversionSubstitutions + j.privateAaMutations.E.totalReversionSubstitutions + j.privateAaMutations.M.totalReversionSubstitutions + j.privateAaMutations.N.totalReversionSubstitutions
        total_AA_subs_plus_del_ranges_minus_reversions = total_AA_subs + total_del_ranges - total_AA_reversions
        
        subs = Set(j.privateNucMutations.privateSubstitutions)
        dels = Set(j.privateNucMutations.privateDeletionRanges)
        nuc_dels = Set(j.privateNucMutations.privateDeletions)
        AA_deletions = Set([j.privateAaMutations.ORF1a.privateDeletionRanges, j.privateAaMutations.ORF1b.privateDeletionRanges, j.privateAaMutations.ORF3a.privateDeletionRanges, j.privateAaMutations.ORF6.privateDeletionRanges, j.privateAaMutations.ORF7a.privateDeletionRanges, j.privateAaMutations.ORF7b.privateDeletionRanges, j.privateAaMutations.ORF8.privateDeletionRanges, j.privateAaMutations.ORF9b.privateDeletionRanges, j.privateAaMutations.S.privateDeletionRanges, j.privateAaMutations.E.privateDeletionRanges, j.privateAaMutations.M.privateDeletionRanges, j.privateAaMutations.N.privateDeletionRanges])

        del_count = 0
        for range in dels
            del_count += 1
        end
        seq_nuc_del_ranges_ct[name] = del_count

        subs_WT = Set(j.substitutions)
        # WT_dels = Set(j.deletions)
        nuc_dels_WT = Set(j.deletions)
        gene_AA_del_WT = j.aaDeletions
        del_ct = 0
        function subs_n_ct(n, subs_n, subs_n_ct)
            if length(subs) + length(dels) == n
                subs_n_ct +=1
                push!(subs_n, name)
            end
        end
        subs_n_ct(9, subs_9, subs_9_ct)
        subs_n_ct(8, subs_8, subs_8_ct)
        subs_n_ct(7, subs_7, subs_7_ct)
        subs_n_ct(6, subs_6, subs_6_ct)
        subs_n_ct(5, subs_5, subs_5_ct)
        subs_n_ct(4, subs_4, subs_4_ct)
        subs_n_ct(3, subs_3, subs_3_ct)

        if j.privateNucMutations.totalReversionSubstitutions ≥ revs_thresh
            push!(too_many_reversions, name)
            push!(all_nonqualifying_seqs, name)
            push!(all_nonqualifying_seqs_set, name)
        end
        if name in excluded_seqs
            push!(all_nonqualifying_seqs, name)
            push!(all_nonqualifying_seqs_set, name)
        end
#        if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions < mut_thresh
         if total_AA_subs_plus_del_ranges_minus_reversions < mut_thresh
            push!(all_nonqualifying_seqs, name)
            push!(all_nonqualifying_seqs_set, name)
        end
######################################################################################################################################
        if j.privateNucMutations.totalReversionSubstitutions ≤ revs_thresh || name in legends
            if !(name in excluded_seqs)
# old                if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions ≥ mut_thresh
                if total_AA_subs_plus_del_ranges_minus_reversions ≥ mut_thresh
                    seq_ct_by_year[seq_year] += 1
                    seq_ct_by_year_month[(seq_year, seq_month)] += 1
                    seq_ct_by_year_month_day[(seq_year, seq_month, seq_day)] += 1
                    push!(all_qualifying_seqs, name)
                    push!(all_qualifying_seqs_set, name)
                    for k in subs
                        pos = k.pos + 1
                        if !(pos in excluded_pos) && !(k.refNuc == "-")
                            total_subs += 1
                            nuc_sub = k.refNuc * string(pos) * k.qryNuc    
                            if haskey(seq_nuc_muts, name)
                                push!(seq_nuc_muts[name], nuc_sub)
                            else 
                                seq_nuc_muts[name] = Set{String}()
                                push!(seq_nuc_muts[name], nuc_sub)
                            end
                            if !haskey(date_nuc_mut_ct, date_index)
                                date_nuc_mut_ct[date_index] = Dict{String, Int64}()
                            end
                            if !haskey(date_nuc_mut_ct[date_index], nuc_sub)
                                date_nuc_mut_ct[date_index][nuc_sub] = 1
                            else 
                                date_nuc_mut_ct[date_index][nuc_sub] += 1
                            end
    #                        if haskey(seq_nuc_muts_no_dels, name) && k.qryNuc ≠ "-"
    #                            push!(seq_nuc_muts_no_dels[name], nuc_sub)
    #                        else
    #                            seq_nuc_muts_no_dels[name] = Set{String}()
    #                            push!(seq_nuc_muts_no_dels[name], nuc_sub)
    #                        end
                            if haskey(nuc_muts_seq, nuc_sub)
                                push!(nuc_muts_seq[nuc_sub], name)
                            else nuc_muts_seq[nuc_sub] = Set{String}()
                                push!(nuc_muts_seq[nuc_sub], name)
                            end
    #*#                         if haskey(nuc_muts_seq_no_dels, nuc_sub) && k.qryNuc ≠ "-"
    #*#                             push!(nuc_muts_seq_no_dels[nuc_sub], name)
    #*#                         else
    #*#                             nuc_muts_seq_no_dels[nuc_sub] = Set{String}()
    #*#                             push!(nuc_muts_seq_no_dels[nuc_sub], name)
    #*#                         end
                        end
                    end
                    for k in subs_WT
                        pos = k.pos + 1
                        if !(pos in excluded_pos)
                            nuc_sub = k.refNuc * string(pos) * k.qryNuc  
                            if haskey(seq_nuc_muts_WT, name)
                                push!(seq_nuc_muts_WT[name], nuc_sub)
                            else 
                                seq_nuc_muts_WT[name] = Set{String}()
                                push!(seq_nuc_muts_WT[name], nuc_sub)
                            end
                            if haskey(nuc_muts_seq_WT, nuc_sub)
                                push!(nuc_muts_seq_WT[nuc_sub], name)
                            else nuc_muts_seq_WT[nuc_sub] = Set{String}()
                                push!(nuc_muts_seq_WT[nuc_sub], name)
                            end
                        end
                    end
                    for d in nuc_dels
                        pos = d.pos + 1
                        if !(pos in excluded_pos)
                            total_nuc_dels += 1
                            n_del = d.refNuc * string(pos) * "-"    
                            if haskey(seq_nuc_muts, name)
                                push!(seq_nuc_muts[name], n_del)
                            else 
                                seq_nuc_muts[name] = Set{String}()
                                push!(seq_nuc_muts[name], n_del)
                            end
                            if haskey(nuc_muts_seq, n_del)
                                push!(nuc_muts_seq[n_del], name)
                            else nuc_muts_seq[n_del] = Set{String}()
                                push!(nuc_muts_seq[n_del], name)
                            end
                            if !haskey(date_nuc_mut_ct, date_index)
                                date_nuc_mut_ct[date_index] = Dict{String, Int64}()
                            end    
                            if haskey(date_nuc_mut_ct[date_index], n_del)
                                date_nuc_mut_ct[date_index][n_del] += 1
                            else 
                                date_nuc_mut_ct[date_index][n_del] = 1
                            end
                        end
                    end
                    for d in dels
                        del_start = d.range[:begin] + 1
                        del_end = d.range[:end]
                        del_start_str = string(del_start)
                        del_end_str = string(del_end)
                        del_str = "_"*del_start_str*"-"*del_end_str
                        push!(nuc_del_ranges, del_str)
                        ## (might use later)  del_rg = [del_start_str, del_end_str]
                        ## (might use later)  nuc_del_ranges[del_ct+1] = [del_start, del_end]
                    end
                    for del in nuc_del_ranges
                        if haskey(seq_nuc_dels, name)
                            push!(seq_nuc_dels[name], del)
                        else 
                            seq_nuc_dels[name] = Set{String}()
                            push!(seq_nuc_dels[name], del)
                        end
                        if haskey(nuc_dels_seq, del)
                            push!(nuc_dels_seq[del], name)
                        else nuc_dels_seq[del] = Set{String}()
                            push!(nuc_dels_seq[del], name)
                        end
                    end
                    for d in nuc_dels_WT
                        del_start = d.range[:begin] + 1
                        del_end = d.range[:end]
                        del_start_str = string(del_start)
                        del_end_str = string(del_end)
                        del_str = "_"*del_start_str*"-"*del_end_str
                        push!(WT_nuc_del_ranges, del_str)
                    end
                    for del in WT_nuc_del_ranges
                        if haskey(seq_nuc_dels_WT, name)
                            push!(seq_nuc_dels_WT[name], del)
                        else 
                            seq_nuc_dels_WT[name] = Set{String}()
                            push!(seq_nuc_dels_WT[name], del)
                        end
                        if haskey(nuc_dels_seq_WT, del)
                            push!(nuc_dels_seq_WT[del], name)
                        else nuc_dels_seq_WT[del] = Set{String}()
                            push!(nuc_dels_seq_WT[del], name)
                        end
                    end

                    for i in gene_AA_subs
                        for k in i
                            pos = k.pos + 1
                            AA_sub = k.cdsName * ":" * k.refAa * string(pos) * k.qryAa
                            if !(k.refAa == "-") && !(AA_sub in excluded_AA)
                                if haskey(seq_AA_muts, name)
                                    push!(seq_AA_muts[name], AA_sub)
                                else 
                                    seq_AA_muts[name] = Set{String}()
                                    push!(seq_AA_muts[name], AA_sub)
                                end
    #                            if haskey(seq_AA_muts_no_dels, name) && k.qryAa ≠ "-"
    #                                push!(seq_AA_muts_no_dels[name], AA_sub)
    #                            else
    #                                seq_AA_muts_no_dels[name] = Set{String}()
    #                                push!(seq_AA_muts_no_dels[name], AA_sub)
    #                            end
    #                            if haskey(AA_muts_ct, AA_sub)
    #                                AA_muts_ct[AA_sub] += 1
    #                            else 
    #                                AA_muts_ct[AA_sub] = 1
    #                            end
    #######################################################################################################
                                if !haskey(date_AA_mut_ct, date_index)
                                    date_AA_mut_ct[date_index] = Dict{String, Int64}()
                                end
                                if !haskey(date_AA_mut_ct[date_index], AA_sub)
                                    date_AA_mut_ct[date_index][AA_sub] = 1
                                else 
                                    date_AA_mut_ct[date_index][AA_sub] += 1
                                end
    #######################################################################################################
                                if haskey(AA_muts_seq, AA_sub)
                                    push!(AA_muts_seq[AA_sub], name)
                                else AA_muts_seq[AA_sub] = Set{String}()
                                    push!(AA_muts_seq[AA_sub], name)
                                end
                            end
    #                        AA_sub_pos = k.cdsName * ":" * string(pos)
    #                        if !(k.refAa == "-") && !(AA_sub in excluded_AA)
    #                            if haskey(seq_AA_muts_pos_only, name)
    #                                push!(seq_AA_muts_pos_only[name], AA_sub_pos)
    #                            else 
    #                                seq_AA_muts_pos_only[name] = Set{String}()
    #                                push!(seq_AA_muts_pos_only[name], AA_sub_pos)
    #                            end
    #                            if haskey(AA_muts_seq_pos_only, AA_sub_pos)
    #                                push!(AA_muts_seq_pos_only[AA_sub_pos], name)
    #                            else AA_muts_seq_pos_only[AA_sub_pos] = Set{String}()
    #                                push!(AA_muts_seq_pos_only[AA_sub_pos], name)
    #                            end
    ##############################################################################################################                                
    #                            if haskey(seq_AA_muts_pos_only_no_dels, name) && k.qryAa ≠ "-"
    #                                push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
    #                            else 
    #                                seq_AA_muts_pos_only_no_dels[name] = Set{String}()
    #                                push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
    #                            end
    #                            if haskey(AA_muts_seq_pos_only_no_dels, AA_sub_pos) && k.qryAa ≠ "-"
    #                                push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
    #                            else AA_muts_seq_pos_only_no_dels[AA_sub_pos] = Set{String}()
    #                                push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
    #                            end
    #                        end
                        end 
                    end
    #27 aaSubstitutions
    # Type: Vector{Any}
    #   j["aaSubstitutions"]
    #   j["aaSubstitutions"][1]
    #   j["aaSubstitutions"][2]
    #   j["aaSubstitutions"][3]  (...etc, very long list)
    #   j["aaSubstitutions"][1]["refAa"]
    #   j["aaSubstitutions"][1]["cdsName"]
    #   j["aaSubstitutions"][1]["pos"]
    #   j["aaSubstitutions"][1]["qryAa"]
                    gene_AA_subs_WT = j.aaSubstitutions
                    for i in gene_AA_subs_WT
                        pos = i.pos + 1
                        AA_sub = i.cdsName * ":" * i.refAa * string(pos) * i.qryAa
                        if !(i.refAa == "-") && !(AA_sub in excluded_AA)
                            if haskey(seq_AA_muts_WT, name)
                                push!(seq_AA_muts_WT[name], AA_sub)
                            else 
                                seq_AA_muts_WT[name] = Set{String}()
                                push!(seq_AA_muts_WT[name], AA_sub)
                            end
                            if haskey(AA_muts_seq_WT, AA_sub)
                                push!(AA_muts_seq_WT[AA_sub], name)
                            else AA_muts_seq_WT[AA_sub] = Set{String}()
                                push!(AA_muts_seq_WT[AA_sub], name)
                            end
                        end
    #                    AA_sub_pos = i.cdsName * ":" * string(pos)
    #                    if !(i.refAa == "-") && !(AA_sub in excluded_AA)
    #                        if haskey(seq_AA_muts_WT_pos_only, name)
    #                            push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
    #                        else 
    #                            seq_AA_muts_WT_pos_only[name] = Set{String}()
    #                            push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
    #                        end
    #                        if haskey(AA_muts_seq_WT_pos_only, AA_sub_pos)
    #                            push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
    #                        else AA_muts_seq_WT_pos_only[AA_sub_pos] = Set{String}()
    #                            push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
    #                        end
    #                    end
                    end                   
    #46 aaDeletions
    #     Type: Vector{Any}
    #        j["aaDeletions"]
    #        j["aaDeletions"][1]
    #        j["aaDeletions"][2]
    #        j["aaDeletions"][3]  (...etc)
    #        j["aaDeletions"][1]["refAa"]
    #        j["aaDeletions"][1]["cdsName"]
    #        j["aaDeletions"][1]["pos"]
                    for i in gene_AA_del_WT
                        jeen = i.cdsName
                        refAa = i.refAa
                        pos = string(i.pos + 1)
                        AA_del_WT = jeen*":"*refAa*pos*"-"
                        if haskey(seq_AA_dels_WT, name)
                            push!(seq_AA_dels_WT[name], AA_del_WT)
                        else 
                            seq_AA_dels_WT[name] = Set{String}()
                            push!(seq_AA_dels_WT[name], AA_del_WT)
                        end
                        if haskey(AA_dels_seq_WT, AA_del_WT)
                            push!(AA_dels_seq_WT[AA_del_WT], name)
                        else AA_dels_seq_WT[AA_del_WT] = Set{String}()
                            push!(AA_dels_seq_WT[AA_del_WT], name)
                        end
                        if haskey(seq_AA_muts_WT, name)
                            push!(seq_AA_muts_WT[name], AA_del_WT)
                        else 
                            seq_AA_muts_WT[name] = Set{String}()
                            push!(seq_AA_muts_WT[name], AA_del_WT)
                        end
                        if haskey(AA_muts_seq_WT, AA_del_WT)
                            push!(AA_muts_seq_WT[AA_del_WT], name)
                        else AA_muts_seq_WT[AA_del_WT] = Set{String}()
                            push!(AA_muts_seq_WT[AA_del_WT], name)
                        end
                    end

                    for i in genes
                        gene_AA_del_ranges = j.privateAaMutations[i].privateDeletionRanges
                        for h in gene_AA_del_ranges
                            beg = h.range[:begin] + 1
                            fin = h.range[:end]
                            AA_ref_beg = ""
                            AA_ref_fin = ""
                            AA_del_res = j.privateAaMutations[i].privateDeletions
                            for n in AA_del_res
                                if n.pos == beg - 1
                                   AA_ref_beg = n.refAa
                                end
                                if n.pos == fin
                                    AA_ref_fin = n.refAa
                                end
                            end
                            beg_str = string(beg)
                            fin_str = string(fin)
                            AA_del_range = i * ":" * AA_ref_beg * beg_str * "-" * ":" * AA_ref_fin * fin_str * "-"
                            if haskey(seq_AA_dels, name)
                                push!(seq_AA_dels[name], AA_del_range)
                            else 
                                seq_AA_dels[name] = Set{String}()
                                push!(seq_AA_dels[name], AA_del_range)
                            end
                            if haskey(AA_dels_seq, AA_del_range)
                                push!(AA_dels_seq[AA_del_range], name)
                            else AA_dels_seq[AA_del_range] = Set{String}()
                                push!(AA_dels_seq[AA_del_range], name)
                            end

                        end
                        AA_dels = j.privateAaMutations[i].privateDeletions
                        for h in AA_dels
                            pos = h.pos + 1
                            del_AA = i * ":" * h.refAa * string(pos)*"-"
                            if haskey(seq_AA_muts, name)
                                push!(seq_AA_muts[name], del_AA)
                            else 
                                seq_AA_muts[name] = Set{String}()
                                push!(seq_AA_muts[name], del_AA)
                            end
                            if haskey(AA_muts_seq, del_AA)
                                push!(AA_muts_seq[del_AA], name)
                            else AA_muts_seq[del_AA] = Set{String}()
                                push!(AA_muts_seq[del_AA], name)
                            end
                        end 
                    end                   
    #4 unknownAaRanges
    #      Type: Vector{Any}                                       ### = Array of dictionaries
    #          j["unknownAaRanges"]
    #          j["unknownAaRanges"][1]                             ### = Dictionary, each # for 1 unknown AA range
    #          j["unknownAaRanges"][2]
    #          j["unknownAaRanges"][3] (...etc)
    #          j["unknownAaRanges"][1]["length"]
    #          j["unknownAaRanges"][1]["character"]
    #          j["unknownAaRanges"][1]["ranges"]                   ### = Array of Dictionaries
    #          j["unknownAaRanges"][1]["cdsName"]       
    #          j["unknownAaRanges"][1]["ranges"][1]                ### = Dictionary (2 keys)
    #          j["unknownAaRanges"][1]["ranges"][2]
    #          j["unknownAaRanges"][1]["ranges"][3]   (...etc)
    #          j["unknownAaRanges"][1]["ranges"][1]["character"]   
    #          j["unknownAaRanges"][1]["ranges"][1]["range"]       ### = Dictionary (2 keys)
    #          j["unknownAaRanges"][1]["ranges"][1]["range"]["begin"]
    #          j["unknownAaRanges"][1]["ranges"][1]["range"]["end"]

                    for i in j.unknownAaRanges
                        unkn_gene = i.cdsName
                        for p in i.ranges
                            bgin = p.range[:begin] + 1
                            nd = p.range[:end]
                            for res in bgin:nd
                                res_str = string(res)
                                if !haskey(seq_unknown_AA, name)
                                    seq_unknown_AA[name] = Set{String}()
                                end
                                push!(seq_unknown_AA[name], unkn_gene*":"*res_str)
                            end
                        end
                    end
    #10 missing
    #    j["missing"][1]["character"]
    #    j["missing"][1]["range"]
    #    j["missing"][1]["range"]["begin"]
    #    j["missing"][1]["range"]["end"]                    
                    for i in j.missing
                        bgin = i.range[:begin] + 1
                        nd = i.range[:end]
                        for nuc_site in bgin:nd
                            if !haskey(seq_nuc_dropout, name)
                                seq_nuc_dropout[name] = Set{Int}()
                            end
                            push!(seq_nuc_dropout[name], nuc_site)
                        end
                    end

                    ref_seq_arr = Vector{String}()
                    for i in 1:length(ref_seq)
                        q = string(ref_seq[i])
                        push!(ref_seq_arr, q)
                    end
                    for i in j.nonACGTNs
                        mixed_nuc = i.character
                        bgin = i.range[:begin] + 1
                        nd = i.range[:end]
                        for b in bgin:nd
                            if !haskey(seq_mixed_nucs, name)
                                seq_mixed_nucs[name] = Set{String}()
                            end
                            push!(seq_mixed_nucs[name], ref_seq_arr[b]*string(b)*mixed_nuc)
                        end
                    end      
                end
            end
        end
    end
    for (name, mutset) in seq_nuc_muts
        for nuc_sub in mutset
            if nuc_sub[end] ≠ '-'
                if haskey(seq_nuc_muts_no_dels, name)
                    push!(seq_nuc_muts_no_dels[name], nuc_sub)
                else
                    seq_nuc_muts_no_dels[name] = Set{String}()
                    push!(seq_nuc_muts_no_dels[name], nuc_sub)
                end
            end
        end
    end
    for (name, mutset) in seq_AA_muts
        for AA_sub in mutset
            AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
            if haskey(seq_AA_muts_pos_only, name)
                push!(seq_AA_muts_pos_only[name], AA_sub_pos)
            else 
                seq_AA_muts_pos_only[name] = Set{String}()
                push!(seq_AA_muts_pos_only[name], AA_sub_pos)
            end
            if AA_sub[end] ≠ '-'
                if haskey(seq_AA_muts_no_dels, name)
                    push!(seq_AA_muts_no_dels[name], AA_sub)
                else
                    seq_AA_muts_no_dels[name] = Set{String}()
                    push!(seq_AA_muts_no_dels[name], AA_sub)
                end
                if haskey(seq_AA_muts_pos_only_no_dels, name)
                    push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
                else 
                    seq_AA_muts_pos_only_no_dels[name] = Set{String}()
                    push!(seq_AA_muts_pos_only_no_dels[name], AA_sub_pos)
                end
            end
        end
    end
    for (AA_sub, name_set) in AA_muts_seq
        AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
        for name in name_set
            if haskey(AA_muts_seq_pos_only, AA_sub_pos)
                push!(AA_muts_seq_pos_only[AA_sub_pos], name)
            else AA_muts_seq_pos_only[AA_sub_pos] = Set{String}()
                push!(AA_muts_seq_pos_only[AA_sub_pos], name)
            end
            if AA_sub[end] ≠ '-'
                if haskey(AA_muts_seq_pos_only_no_dels, AA_sub_pos)
                    push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
                else AA_muts_seq_pos_only_no_dels[AA_sub_pos] = Set{String}()
                    push!(AA_muts_seq_pos_only_no_dels[AA_sub_pos], name)
                end
            end
        end
    end
    for (name, mutset) in seq_AA_muts_WT
        for AA_sub in mutset
            AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
            if haskey(seq_AA_muts_WT_pos_only, name)
                push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
            else 
                seq_AA_muts_WT_pos_only[name] = Set{String}()
                push!(seq_AA_muts_WT_pos_only[name], AA_sub_pos)
            end
        end
    end
    for (AA_sub, name_set) in AA_muts_seq_WT
        AA_sub_pos = AA_mut_to_AA_pos(AA_sub)
        for name in name_set
            if haskey(AA_muts_seq_WT_pos_only, AA_sub_pos)
                push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
            else AA_muts_seq_WT_pos_only[AA_sub_pos] = Set{String}()
                push!(AA_muts_seq_WT_pos_only[AA_sub_pos], name)
            end
        end
    end                                
#    print("\n"^2)
    for seq in all_qualifying_seqs_set
        if !haskey(seq_nuc_dropout, seq)
            seq_nuc_dropout[seq] = Set{Int}()
        end
        if !haskey(seq_mixed_nucs, seq)
            seq_mixed_nucs[seq] = Set{String}()
        end
        if !haskey(seq_unknown_AA, seq)
            seq_unknown_AA[seq] = Set{String}()
        end
        if !haskey(seq_nuc_dels, seq)
            seq_nuc_dels[seq] = Set{String}()
        end
        if !haskey(seq_AA_dels, seq)
            seq_AA_dels[seq] = Set{String}()
        end
        if !haskey(seq_nuc_dels_WT, seq)
            seq_nuc_dels_WT[seq] = Set{String}()
        end
        if !haskey(seq_AA_dels_WT, seq)
            seq_AA_dels_WT[seq] = Set{String}()
        end
    end
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
#    print("\n"^2)
    date = Dates.format(today(), "yyyy-mm-dd")
    subs_9_sort = sort(subs_9)
    subs_8_sort = sort(subs_8)
    subs_7_sort = sort(subs_7)
    subs_6_sort = sort(subs_6)
    subs_5_sort = sort(subs_5)
    subs_4_sort = sort(subs_4)
    subs_3_sort = sort(subs_3)
    too_many_reversions_sort = sort(too_many_reversions)
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh).txt", "w") do g
        println("######################### Sequences with more than $(revs_thresh) Reversions #########################")
        println(g, "######################### Sequences with more than $(revs_thresh) Reversions #########################")
        for s in too_many_reversions_sort
            print(s, ", ")
            print(g, s, ", ")
        end
        println()
        println(g)
        function seqs_with_n_subs(n::Int, subs_n_sort::Vector{String})
            println()
            println(g)
            println("EPI_ISLs for sequences with $(n) private subs + del ranges")
            println(g, "EPI_ISLs for sequences with $(n) private subs + del ranges")
            for seq in subs_n_sort
                print(seq, ", ")
                print(g, seq, ", ")
            end
            println()
            println(g)
        end
        seqs_with_n_subs(8, subs_8_sort)
        seqs_with_n_subs(7, subs_7_sort)
        seqs_with_n_subs(6, subs_6_sort)
        seqs_with_n_subs(5, subs_5_sort)
        seqs_with_n_subs(4, subs_4_sort)
        seqs_with_n_subs(3, subs_3_sort)
        println()
        print(g, "\n"^2)
    end
    for n in 1:length(all_qualifying_seqs) - 1
#        yep_already_in_repSeqGrpSeqs = 0
        NthSeq = all_qualifying_seqs[n]
        if !(NthSeq in rep_seqs)
            if haskey(rep_seq_grps_muts, 1) && haskey(rep_seq_grps_seqs, 1)
                if !(NthSeq in rep_seqs)
                    keez = keys(rep_seq_grps_muts)
                    for i in keys(rep_seq_grps_seqs)
                        if !(NthSeq in rep_seqs)
                            Grp_i_MutsSet = rep_seq_grps_muts_no_dels[i]
                            Grp_i_AAMutsSet = rep_seq_grps_AA_no_dels[i]
                            Grp_i_EpisSet = rep_seq_grps_seqs[i]
                            Grp_i_MutsSetNoDels = Set{String}()
                            Grp_i_AAMutsSetNoDels = Set{String}()
                            for grp_i_seq in Grp_i_EpisSet
                                for mut in seq_nuc_muts_no_dels[grp_i_seq]
                                    push!(Grp_i_MutsSetNoDels, mut)
                                end
                                for AA in seq_AA_muts_no_dels[grp_i_seq]
                                    push!(Grp_i_AAMutsSetNoDels, AA)
                                end
                            end
                            firstSeq = first(Grp_i_EpisSet)
                            NthSeqMuts = seq_nuc_muts_no_dels[NthSeq]
                            NthSeqAAMuts = seq_AA_muts_no_dels[NthSeq]
                            NthSeqMuts_Tot = length(NthSeqMuts)
                            NthSeqAAMuts_Tot = length(NthSeqAAMuts)
                            Grp_i_Muts_Tot = length(Grp_i_MutsSetNoDels)
                            Grp_i_AAMuts_Tot = length(Grp_i_AAMutsSetNoDels)
                            GrpSize = length(Grp_i_EpisSet)
                            common = intersect(seq_nuc_muts_no_dels[NthSeq], Grp_i_MutsSetNoDels)
                            commonAA = intersect(seq_AA_muts_no_dels[NthSeq], Grp_i_MutsSetNoDels)
                            sharedMuts_tot = length(common)
                            sharedMuts_NthSeq_prop = sharedMuts_tot/NthSeqMuts_Tot
                            sharedMuts_Grp_prop = sharedMuts_tot/Grp_i_Muts_Tot
                            if GrpSize < 3 && sharedMuts_NthSeq_prop ≤ 0.3 && sharedMuts_Grp_prop ≤ 0.3
                                continue
                            end
                            US_state_check = "no"
                            if seq_country[NthSeq] == seq_country[firstSeq] && (seq_country[NthSeq] == "USA" || seq_country[NthSeq] == "Canada") && seq_US_state[NthSeq] == seq_US_state[firstSeq]
                                US_state_check = "yes"
                            end 
                            if seq_country[NthSeq] == seq_country[firstSeq] && seq_country[NthSeq] ≠ "USA" && seq_country[NthSeq] ≠ "Canada"
                                US_state_check = "yes"
                            end  
                            if sharedMuts_tot ≥ rep_thresh && seq_country[NthSeq] == seq_country[firstSeq] && seq_clade[NthSeq] == seq_clade[firstSeq] && US_state_check == "yes"
        #                    if sharedMuts_tot ≥ rep_thresh && seq_country[NthSeq] == seq_country[firstSeq] && seq_pango[NthSeq] == seq_pango[firstSeq]        #                        if seq_country[NthSeq] == "USA" && seq_country[firstSeq] == "USA"
        #                           if seq_US_state[NthSeq] ≠ seq_US_state[firstSeq]
        #                           if haskey(seq_US_state, firstSeq) && haskey(seq_US_state, NthSeq) && seq_US_state[NthSeq] ≠ seq_US_state[firstSeq]
        #                               US_state_match_check = -1
        #                           end
        #                       end
        #                       if US_state_match_check ≥ 0
                                push!(rep_seqs, NthSeq)
                                push!(rep_seq_grps_seqs[i], NthSeq)
                                for mut in seq_nuc_muts[NthSeq]
                                    push!(rep_seq_grps_muts[i], mut)
                                    if mut[end] ≠ '-'
                                        push!(rep_seq_grps_muts_no_dels[i], mut)
                                    end
                                end
                                for mut in seq_AA_muts[NthSeq]
                                    push!(rep_seq_grps_AA[i], mut)
                                    if mut[end] ≠ '-'
                                        push!(rep_seq_grps_AA_no_dels[i], mut)
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
        for p in n+1:length(all_qualifying_seqs)
            if !(NthSeq in rep_seqs)
                PthSeq = all_qualifying_seqs[p]
                if !(PthSeq in rep_seqs)
                    common_muts = intersect(seq_AA_muts_no_dels[NthSeq], seq_AA_muts_no_dels[PthSeq])
                    US_state_check = "no"
                    if seq_country[NthSeq] == seq_country[PthSeq] && (seq_country[NthSeq] == "USA" || seq_country[NthSeq] == "Canada") && seq_US_state[NthSeq] == seq_US_state[PthSeq]
                        US_state_check = "yes"
                    end 
                    if seq_country[NthSeq] == seq_country[PthSeq] && seq_country[NthSeq] ≠ "USA" && seq_country[NthSeq] ≠ "Canada"
                        US_state_check = "yes"
                    end  
                    if length(common_muts) ≥ rep_thresh && seq_country[NthSeq] == seq_country[PthSeq] && seq_clade[NthSeq] == seq_clade[PthSeq] && US_state_check == "yes"
                        push!(rep_seqs, NthSeq)
                        push!(rep_seqs, PthSeq)
                        if !haskey(rep_seq_grps_seqs, 1)
                            rep_seq_grps_seqs[1] = Set{String}()
                            rep_seq_grps_muts[1] = Set{String}()
                            rep_seq_grps_muts_no_dels[1] = Set{String}()
                            rep_seq_grps_dels[1] = Set{String}()
                            rep_seq_grps_AA[1] = Set{String}()
                            rep_seq_grps_AA_no_dels[1] = Set{String}()
                            rep_seq_grps_AA_dels[1] = Set{String}()
                            push!(rep_seq_grps_seqs[1], NthSeq)
                            push!(rep_seq_grps_seqs[1], PthSeq)
                            rep_seq_grps_clade[1] = seq_clade[PthSeq]
                            rep_seq_grps_pango[1] = seq_pango[PthSeq]
                            rep_seq_grps_pango_unaliased[1] = seq_pango_unaliased[PthSeq]
                            for m in seq_nuc_muts[NthSeq]
                                push!(rep_seq_grps_muts[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[1], m)
                                end
                            end
                            for m in seq_nuc_muts[PthSeq]
                                push!(rep_seq_grps_muts[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[1], m)
                                end
                            end
                            for m in seq_AA_muts[NthSeq]
                                push!(rep_seq_grps_AA[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[1], m)
                                end
                            end
                            for m in seq_AA_muts[PthSeq]
                                push!(rep_seq_grps_AA[1], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[1], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[1], m)
                                end
                            end
                        else
                            Grp_Nums = keys(rep_seq_grps_seqs)
                            new_grp_num = length(Grp_Nums) + 1
                            rep_seq_grps_clade[new_grp_num] = seq_clade[PthSeq]
                            rep_seq_grps_pango[new_grp_num] = seq_pango[PthSeq]
                            rep_seq_grps_pango_unaliased[new_grp_num] = seq_pango_unaliased[PthSeq]
                            rep_seq_grps_seqs[new_grp_num] = Set{String}()
                            rep_seq_grps_muts[new_grp_num] = Set{String}()
                            rep_seq_grps_muts_no_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_AA[new_grp_num] = Set{String}()
                            rep_seq_grps_AA_no_dels[new_grp_num] = Set{String}()
                            rep_seq_grps_AA_dels[new_grp_num] = Set{String}()
                            push!(rep_seq_grps_seqs[new_grp_num], NthSeq)
                            push!(rep_seq_grps_seqs[new_grp_num], PthSeq)
                            for m in seq_nuc_muts[NthSeq]
                                push!(rep_seq_grps_muts[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_nuc_muts[PthSeq]
                                push!(rep_seq_grps_muts[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_muts_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_AA_muts[NthSeq]
                                push!(rep_seq_grps_AA[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[new_grp_num], m)
                                end
                            end
                            for m in seq_AA_muts[PthSeq]
                                push!(rep_seq_grps_AA[new_grp_num], m)
                                if m[end] ≠ '-'
                                    push!(rep_seq_grps_AA_no_dels[new_grp_num], m)
                                end
                                if m[end] == '-'
                                    push!(rep_seq_grps_AA_dels[new_grp_num], m)
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    
    
    for (grp_num, seq_set) in rep_seq_grps_seqs
        max_AA_muts = 0
        max_AA_mut_seq = ""
        for seq in seq_set
            seq_total_priv_AA_muts = length(seq_AA_muts[seq])
            if seq_total_priv_AA_muts > max_AA_muts
                max_AA_muts = seq_total_priv_AA_muts
                max_AA_mut_seq = seq
            end
        end
        rep_seq_grps_maxmut_seqs[grp_num] = max_AA_mut_seq
    end
    for (grp_num, seq) in rep_seq_grps_maxmut_seqs
        rep_seq_grps_maxmut_nuc[grp_num] = seq_nuc_muts[seq]
        rep_seq_grps_maxmut_nuc_no_dels[grp_num] = seq_nuc_muts_no_dels[seq]
        rep_seq_grps_maxmut_dels[grp_num] = seq_nuc_dels[seq]
        rep_seq_grps_maxmut_del_ranges_ct[grp_num] = seq_nuc_del_ranges_ct[seq]
        rep_seq_grps_maxmut_nuc_dropout[grp_num] = Set{Int}() 
        for num in seq_nuc_dropout[seq]
            push!(rep_seq_grps_maxmut_nuc_dropout[grp_num], num)
        end
        rep_seq_grps_maxmut_mixed_nucs[grp_num] = seq_mixed_nucs[seq]
        rep_seq_grps_maxmut_AA[grp_num] = seq_AA_muts[seq]
        rep_seq_grps_maxmut_AA_no_dels[grp_num] = seq_AA_muts_no_dels[seq]
        rep_seq_grps_maxmut_AA_dels[grp_num] = seq_AA_dels[seq]
        rep_seq_grps_maxmut_AA_pos_only[grp_num] = seq_AA_muts_pos_only[seq]
        rep_seq_grps_maxmut_AA_pos_only_no_dels[grp_num] = seq_AA_muts_pos_only_no_dels[seq]
        rep_seq_grps_maxmut_unknown_AA[grp_num] = Set{String}()
        for AAsite in seq_unknown_AA[seq]
            push!(rep_seq_grps_maxmut_unknown_AA[grp_num], AAsite)
        end
#        rep_seq_grps_maxmut_mixed_AA[grp_num] = seq_mixed_AA_muts[seq]
        rep_seq_grps_maxmut_nuc_muts_WT[grp_num] = seq_nuc_muts_WT[seq]
        rep_seq_grps_maxmut_nuc_dels_WT[grp_num] = seq_nuc_dels_WT[seq]
        rep_seq_grps_maxmut_AA_muts_WT[grp_num] = seq_AA_muts_WT[seq]
        rep_seq_grps_maxmut_AA_dels_WT[grp_num] = seq_AA_dels_WT[seq]
        rep_seq_grps_maxmut_AA_muts_WT_pos_only[grp_num] = seq_AA_muts_WT_pos_only[seq]
    end
    
    
    for i in 1:length(rep_seq_grps_seqs)
        for m in rep_seq_grps_maxmut_nuc[i]
            if haskey(nuc_muts_ct, m)
                nuc_muts_ct[m] += 1
            else 
                nuc_muts_ct[m] = 1
            end
        end
#*        for m in rep_seq_grps_maxmut_nuc[i]
#*            if m[end] ≠ '-'
#*                if haskey(nuc_muts_ct_no_dels, m)
#*                    nuc_muts_ct_no_dels[m] += 1
#*                else 
#*                    nuc_muts_ct_no_dels[m] = 1
#*                end
#*            end
#*        end
        for m in rep_seq_grps_maxmut_dels[i]
            if haskey(nuc_dels_ct, m)
                nuc_dels_ct[m] += 1
            else 
                nuc_dels_ct[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_mixed_nucs[i]
            if haskey(mixed_nucs_ct, m)
                mixed_nucs_ct[m] += 1
            else 
                mixed_nucs_ct[m] = 1
            end
        end
##############################################################################################################
##############################################################################################################        
        for m in rep_seq_grps_maxmut_AA[i]
            if haskey(AA_muts_ct, m)
                AA_muts_ct[m] += 1
            else 
                AA_muts_ct[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_AA_dels[i]
            if haskey(AA_dels_ct, m)
                AA_dels_ct[m] += 1
            else 
                AA_dels_ct[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_unknown_AA[i]
            if haskey(unknown_AA_ct, m)
                unknown_AA_ct[m] += 1
            else 
                unknown_AA_ct[m] = 1
            end
        end
##############################################################################################################
##############################################################################################################
        for m in rep_seq_grps_maxmut_nuc_muts_WT[i]
            if haskey(nuc_muts_ct_WT, m)
                nuc_muts_ct_WT[m] += 1
            else 
                nuc_muts_ct_WT[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_nuc_dels_WT[i]
            if haskey(nuc_dels_ct_WT, m)
                nuc_dels_ct_WT[m] += 1
            else 
                nuc_dels_ct_WT[m] = 1
            end
        end
##############################################################################################################
##############################################################################################################
        for m in rep_seq_grps_maxmut_AA_muts_WT[i]
            if haskey(AA_muts_ct_WT, m)
                AA_muts_ct_WT[m] += 1
            else 
                AA_muts_ct_WT[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_AA_dels_WT[i]
            if haskey(AA_dels_ct_WT, m)
                AA_dels_ct_WT[m] += 1
            else 
                AA_dels_ct_WT[m] = 1
            end
        end
        for m in rep_seq_grps_maxmut_AA[i]
            jeen = split(m, ":")[1]
            AA_pos = split(m, ":")[2][2:end-1]
            mpos = jeen*":"*AA_pos
            AA_mut_pos_int = parse(Int, AA_pos)
            qry_AA_str = string(m[end])
            ref_seq_AA_str = string(gene_AA_dict[jeen][AA_mut_pos_int])
            if m[end] == ref_seq_AA_str
                if haskey(mpos, AA_muts_ct_pos_only_reversion)
                    AA_muts_ct_pos_only_reversion[mpos] += 1
                else
                    AA_muts_ct_pos_only_reversion[mpos] = 1
                end
            end
        end
    end

##############################################################################################################
##############################################################################################################
#**#    rep_seq_grps_nuc_mut_keys = keys(rep_seq_grps_muts)
#**#    rep_seq_grps_nuc_mut_keys_ln = length(rep_seq_grps_nuc_mut_keys)
#**#    for i in 1:rep_seq_grps_nuc_mut_keys_ln
#**#        for m in rep_seq_grps_muts[i]
#**#            if m[end] ≠ '-'
#**#                if haskey(nuc_muts_ct_no_dels, m)
#**#                    nuc_muts_ct_no_dels[m] += 1
#**#                else 
#**#                    nuc_muts_ct_no_dels[m] = 1
#**#                end
#**#            end
#**#        end
#**#    end
#**#    print("\n"^2)
    println("Number of Sequences in rep_seqs list: ", length(rep_seqs))
    print("\n"^2)
    gene_name_fx(m) = split(m, ":")[1]
    for (grp_num, clade) in rep_seq_grps_clade
        if haskey(seq_clade_ct, clade)
            seq_clade_ct[clade] += 1
        else
            seq_clade_ct[clade] = 1
        end
    end
    for (grp_num, pango) in rep_seq_grps_pango
        if haskey(seq_pango_ct, pango)
            seq_pango_ct[pango] += 1
        else
            seq_pango_ct[pango] = 1
        end
    end
    for (grp_num, pango_unaliased) in rep_seq_grps_pango_unaliased
        if haskey(seq_pango_unaliased_ct, pango_unaliased)
            seq_pango_unaliased_ct[pango_unaliased] += 1
        else
            seq_pango_unaliased_ct[pango_unaliased] = 1
        end
    end
    for grp_num in keys(rep_seq_grps_AA)
        rep_seq_grps_AA_pos_only[grp_num] = Set{String}()
        rep_seq_grps_AA_pos_only_no_dels[grp_num] = Set{String}()
    end
    for (grp_num, mutset) in rep_seq_grps_AA
        for m in mutset
            AA_pos = AA_mut_to_AA_pos(m)
            push!(rep_seq_grps_AA_pos_only[grp_num], AA_pos)
        end
        for m in mutset
            if m[end] ≠ '-'
                AA_pos = AA_mut_to_AA_pos(m)
                push!(rep_seq_grps_AA_pos_only_no_dels[grp_num], AA_pos)
            end
        end
        for m in mutset
            if !haskey(AA_muts_rep_seq_grps, m)
                AA_muts_rep_seq_grps[m] = Set{Int}()
            end
            push!(AA_muts_rep_seq_grps[m], grp_num)
            if m[end] ≠ '-'
                if !haskey(AA_muts_rep_seq_grps_no_dels, m)
                    AA_muts_rep_seq_grps_no_dels[m] = Set{Int}()
                end
                push!(AA_muts_rep_seq_grps_no_dels[m], grp_num)
            end
            if m[end] == '-'
                if !haskey(AA_dels_rep_seq_grps, m)
                    AA_dels_rep_seq_grps[m] = Set{Int}()
                end
                push!(AA_dels_rep_seq_grps[m], grp_num)
            end
        end
    end
    for (grp_num, mutpos_set) in rep_seq_grps_AA_pos_only
        for m in mutpos_set
            if !haskey(AA_muts_rep_seq_grps_pos_only, m)
                AA_muts_rep_seq_grps_pos_only[m] = Set{Int}()
            end
            push!(AA_muts_rep_seq_grps_pos_only[m], grp_num)
        end      
    end
##############################################################################    
    for seq in all_qualifying_seqs_set
        if !(seq in rep_seqs)
            push!(non_rep_seqs, seq)
        end
    end    
    for seq in non_rep_seqs
        clade = seq_clade[seq]
        pango = seq_pango[seq]
        pango_unaliased = seq_pango_unaliased[seq]
        if haskey(seq_clade_ct, clade)
            seq_clade_ct[clade] += 1
        else
            seq_clade_ct[clade] = 1
        end
        if haskey(seq_pango_ct, pango)
            seq_pango_ct[pango] += 1
        else
            seq_pango_ct[pango] = 1
        end
        if haskey(seq_pango_unaliased_ct, pango_unaliased)
            seq_pango_unaliased_ct[pango_unaliased] += 1
        else
            seq_pango_unaliased_ct[pango_unaliased] = 1
        end
        for m in seq_nuc_muts[seq]
            if haskey(nuc_muts_ct, m)
                nuc_muts_ct[m] += 1
            else 
                nuc_muts_ct[m] = 1
            end
        end
        for m in seq_nuc_muts_WT[seq]
            if haskey(nuc_muts_ct_WT, m)
                nuc_muts_ct_WT[m] += 1
            else 
                nuc_muts_ct_WT[m] = 1
            end
        end
        for m in seq_nuc_dels_WT[seq]
            if haskey(nuc_dels_ct_WT, m)
                nuc_dels_ct_WT[m] += 1
            else 
                nuc_dels_ct_WT[m] = 1
            end
        end
        for m in seq_mixed_nucs[seq]
            if haskey(mixed_nucs_ct, m)
                mixed_nucs_ct[m] += 1
            else 
                mixed_nucs_ct[m] = 1
            end
        end
        for m in seq_AA_muts[seq]
            if haskey(AA_muts_ct, m)
                AA_muts_ct[m] += 1
            else 
                AA_muts_ct[m] = 1
            end
            if !haskey(non_rep_seqs_AA, seq)
                non_rep_seqs_AA[seq] = Set{String}()
                push!(non_rep_seqs_AA[seq], m)
            else
                push!(non_rep_seqs_AA[seq], m)
            end
        end
        for m in seq_AA_muts_WT[seq]
            if haskey(AA_muts_ct_WT, m)
                AA_muts_ct_WT[m] += 1
            else 
                AA_muts_ct_WT[m] = 1
            end
        end
        for m in seq_AA_dels_WT[seq]
            if haskey(AA_dels_ct_WT, m)
                AA_dels_ct_WT[m] += 1
            else 
                AA_dels_ct_WT[m] = 1
            end
        end
        for m in seq_unknown_AA[seq]
            if haskey(unknown_AA_ct, m)
                unknown_AA_ct[m] += 1
            else 
                unknown_AA_ct[m] = 1
            end
        end
        for m in seq_AA_muts[seq]
            jeen = split(m, ":")[1]
            AA_pos = split(m, ":")[2][2:end-1]
            mpos = jeen*":"*AA_pos
            AA_mut_pos_int = parse(Int, AA_pos)
            qry_AA_str = string(m[end])
            ref_seq_AA_str = string(gene_AA_dict[jeen][AA_mut_pos_int])
            if m[end] == ref_seq_AA_str
                if haskey(mpos, AA_muts_ct_pos_only_reversion)
                    AA_muts_ct_pos_only_reversion[mpos] += 1
                else
                    AA_muts_ct_pos_only_reversion[mpos] = 1
                end
            end
        end
        for (EPI, mutSet) in non_rep_seqs_AA
            for m in mutSet
                AA_pos = AA_mut_to_AA_pos(m)
                if !haskey(non_rep_seqs_AA_pos_only, EPI)
                    non_rep_seqs_AA_pos_only[EPI] = Set{String}()
                    push!(non_rep_seqs_AA_pos_only[EPI], AA_pos)
                else
                    push!(non_rep_seqs_AA_pos_only[EPI], AA_pos)
                end
            end
            for m in mutSet
                if m[end] ≠ '-'
                    AA_pos = AA_mut_to_AA_pos(m)
                    if !haskey(non_rep_seqs_AA_pos_only_no_dels, EPI)
                        non_rep_seqs_AA_pos_only_no_dels[EPI] = Set{String}()
                        push!(non_rep_seqs_AA_pos_only_no_dels[EPI], AA_pos)
                    else
                        push!(non_rep_seqs_AA_pos_only_no_dels[EPI], AA_pos)
                    end
                end
            end
        end
    end
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
    nuc_mut_pos(m) = parse(Int, m[2:end-1])
    for (mut, ct) in nuc_muts_ct
        nuc_pos = nuc_mut_pos(mut)
        if mut[end] ≠ '-'
            nuc_muts_ct_no_dels[mut] = ct
            if haskey(nuc_muts_ct_pos_only_no_dels, nuc_pos)
                nuc_muts_ct_pos_only_no_dels[nuc_pos] += ct
            else
                nuc_muts_ct_pos_only_no_dels[nuc_pos] = ct
            end
            if haskey(nuc_muts_ct_pos_only_no_dels, nuc_pos)
                nuc_muts_ct_pos_only_no_dels[nuc_pos] += ct
            else
                nuc_muts_ct_pos_only_no_dels[nuc_pos] = ct
            end
        end
    end
    for (mut, ct) in AA_muts_ct
        AA_pos = AA_mut_to_AA_pos(mut)
        AA_mut_pos_int = parse(Int, split(mut, ":")[2][2:end-1])
        gene = split(mut, ":")[1]
        gene = string(gene)
        refaa = gene_AA_dict[gene][AA_mut_pos_int]
        refAA = string(refaa)
        qryAA = string(mut[end])
        if haskey(AA_muts_ct_pos_only, AA_pos)
            AA_muts_ct_pos_only[AA_pos] += ct
        else
            AA_muts_ct_pos_only[AA_pos] = ct
        end
        if mut[end] ≠ '-'
            AA_muts_ct_no_dels[mut] = ct
            if haskey(AA_muts_ct_pos_only_no_dels, AA_pos)
                AA_muts_ct_pos_only_no_dels[AA_pos] += ct
            else
                AA_muts_ct_pos_only_no_dels[AA_pos] = ct
            end
            if !(qryAA == refAA) && !(mut in reversion_exceptions)
                AA_muts_ct_no_dels_no_revs[mut] = ct
                if haskey(AA_muts_ct_pos_only_no_dels_no_revs, AA_pos)
                    AA_muts_ct_pos_only_no_dels_no_revs[AA_pos] += ct
                else
                    AA_muts_ct_pos_only_no_dels_no_revs[AA_pos] = ct
                end
            end
        end
    end
    for (mut, ct) in AA_muts_ct_WT
        AA_pos = AA_mut_to_AA_pos(mut)
        if haskey(AA_muts_ct_WT_pos_only, AA_pos)
            AA_muts_ct_WT_pos_only[AA_pos] += ct
        else
            AA_muts_ct_WT_pos_only[AA_pos] = ct
        end
        if mut[end] ≠ '-'
            AA_muts_ct_WT_no_dels[mut] = ct
            if haskey(AA_muts_ct_WT_pos_only_no_dels, AA_pos)
                AA_muts_ct_WT_pos_only_no_dels[AA_pos] += ct
            else
                AA_muts_ct_WT_pos_only_no_dels[AA_pos] = ct
            end
        end
    end
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
    AA_muts_ct_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_chr_all_ratio = Dict{String, Float64}()
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio = Dict{String, Float64}()
    
    non_rep_seq_ct = length(non_rep_seqs)
    rep_seq_grps_maxmut_seqs_ct = length(keys(rep_seq_grps_maxmut_seqs))
    total_qualifying_chr_seq_ct = non_rep_seq_ct + rep_seq_grps_maxmut_seqs_ct
    all_ct = qualifying_seq_ct_all
    total_circ_AA_mut_ct = 0
    total_chr_AA_ct = 0
    total_chr_AA_no_revs_ct = 0
    for (mut, count) in AA_muts_ct_no_dels_all
        total_circ_AA_mut_ct += count
    end
    for (mut, count) in AA_muts_ct_no_dels
        total_chr_AA_ct += count
    end
    for (mut, count) in AA_muts_ct_no_dels_no_revs
        total_chr_AA_no_revs_ct += count
    end
    avg_AA_subs_per_chr_seq = total_chr_AA_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_per_chr_seq_rd = round(digits=2, avg_AA_subs_per_chr_seq)
    avg_AA_subs_per_circ_seq = total_circ_AA_mut_ct/all_ct
    avg_AA_subs_per_circ_seq_rd = round(digits=2, avg_AA_subs_per_circ_seq)
    avg_AA_subs_per_chr_seq_no_revs = total_chr_AA_no_revs_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_per_chr_seq_no_revs_rd = round(digits=2, avg_AA_subs_per_chr_seq_no_revs)
    println()
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic AA Substitution Ct = $(total_chr_AA_ct)  **********************")
    println("********************** Total Circulating AA Substitution Ct = $(total_circ_AA_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Substitution Ct = $(total_chr_AA_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating AA Substitution Ct = $(total_circ_AA_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
    chr_to_circ_mut_ratio_adj_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_per_circ_seq/avg_AA_subs_per_chr_seq)
    for (mut, chr_ct) in AA_muts_ct_no_dels
        if haskey(AA_muts_ct_no_dels_all, mut)
            circ_ct = AA_muts_ct_no_dels_all[mut]
            AA_muts_ct_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            AA_muts_ct_no_dels_all[mut] = 0
            AA_muts_ct_no_dels_chr_all_ratio[mut] = chr_ct*1000
        end
    end
    AA_muts_ct_no_dels_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_no_dels_chr_all_ratio), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_no_dels_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
    ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in AA_muts_ct_no_dels_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", rpad(ratio, 7))
        end
    end
    print("\n"^8)
#######################################################################################################################################
    println("********************** Total Counted Chronic Seq Ct = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Seq Ct = $(all_ct)  **********************")
    println()
    println("********************** Total Chronic AA Substitution Ct No Revs = $(total_chr_AA_no_revs_ct)  **********************")
    println("********************** Total Circulating AA Substitution Ct = $(total_circ_AA_mut_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Substitution Ct, No Revs = $(total_chr_AA_no_revs_ct/total_qualifying_chr_seq_ct)  **********************")
    println("********************** Avg Circulating AA Substitution Ct = $(total_circ_AA_mut_ct/qualifying_seq_ct_all)  **********************")
    println()
    chr_to_circ_mut_ratio_adj_factor_no_revs = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_per_circ_seq/avg_AA_subs_per_chr_seq_no_revs)
    for (mut, chr_ct_no_revs) in AA_muts_ct_no_dels_no_revs
        if haskey(AA_muts_ct_no_dels_all, mut)
            circ_ct = AA_muts_ct_no_dels_all[mut]
            AA_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            AA_muts_ct_no_dels_all[mut] = 0
            AA_muts_ct_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs*1000
        end
    end
    AA_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
    ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in AA_muts_ct_no_dels_no_revs_chr_all_ratio
        ratio_print_ct += 1
        if ratio_print_ct < 10
            println(rpad(mut, 12), " = ", rpad(ratio_no_revs, 7))
        end
    end
########################################################################################################################################
    total_circ_AA_pos_only_ct = 0
    total_chr_AA_pos_only_ct = 0
    total_chr_AA_pos_only_no_revs_ct = 0
    for (mut, count) in AA_muts_ct_pos_only_no_dels_all
        total_circ_AA_pos_only_ct += count
    end
    for (mut, count) in AA_muts_ct_pos_only_no_dels
        total_chr_AA_pos_only_ct += count
    end
    for (mut, count) in AA_muts_ct_pos_only_no_dels_no_revs
        total_chr_AA_pos_only_no_revs_ct += count
    end
    avg_AA_subs_pos_only_per_circ_seq = total_circ_AA_pos_only_ct/all_ct
    avg_AA_subs_pos_only_per_circ_seq_rd = round(digits=2, avg_AA_subs_pos_only_per_circ_seq)
    avg_AA_subs_pos_only_per_chr_seq = total_chr_AA_pos_only_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_pos_only_per_chr_seq_rd = round(digits=2, avg_AA_subs_pos_only_per_chr_seq)
    avg_AA_subs_pos_only_no_revs_per_chr_seq = total_chr_AA_pos_only_no_revs_ct/total_qualifying_chr_seq_ct
    avg_AA_subs_pos_only_no_revs_per_chr_seq_rd = round(digits=2, avg_AA_subs_pos_only_no_revs_per_chr_seq)
    println()
    println("********************** Total Counted Chronic Sequences = $(total_qualifying_chr_seq_ct)  **********************")
    println("********************** Total Counted Circulating Sequences = $(qualifying_seq_ct_all)  **********************")
    println()
    println("********************** Total Chronic AA Subs Pos Only Ct = $(total_chr_AA_pos_only_ct)  **********************")
    println("********************** Total Circulating AA Subs Pos Only Ct = $(total_circ_AA_pos_only_ct)  **********************")
    println()
    println("********************** Avg Chronic AA Subs Pos Only Ct = $(avg_AA_subs_pos_only_per_chr_seq_rd)  **********************")
    println("********************** Avg Circulating AA Subs Pos Only Ct = $(avg_AA_subs_pos_only_per_circ_seq_rd)  **********************")
    println()
#######################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_pos_only_per_circ_seq/avg_AA_subs_pos_only_per_chr_seq)
    for (mut, chr_ct) in AA_muts_ct_pos_only_no_dels
        if haskey(AA_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = AA_muts_ct_pos_only_no_dels_all[mut]
            AA_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor*chr_ct/circ_ct)
        else
            AA_muts_ct_pos_only_no_dels_chr_all_ratio[mut] = chr_ct*1000
            AA_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
    AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_pos_only_no_dels_chr_all_ratio), by = x -> x[2], rev=true)
    pos_only_ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_pos_only_no_dels_chr_all_ratio Keys and Values")
    for (mut, ratio) in AA_muts_ct_pos_only_no_dels_chr_all_ratio
        pos_only_ratio_print_ct += 1
        if pos_only_ratio_print_ct < 10
            println(rpad(mut, 11), " = ", rpad(ratio, 7))
        end
    end
    println()
    for mut in keys(AA_muts_ct_pos_only_no_dels_chr_all_ratio)
        if !haskey(AA_muts_ct_pos_only_no_dels, mut)
            AA_muts_ct_pos_only_no_dels[mut] = 0
        end
    end
#######################################################################################################################################
    chr_to_circ_mut_ratio_adj_pos_only_no_revs_factor = (all_ct/total_qualifying_chr_seq_ct)*(avg_AA_subs_pos_only_per_circ_seq/avg_AA_subs_pos_only_no_revs_per_chr_seq)
    for (mut, chr_ct_no_revs) in AA_muts_ct_pos_only_no_dels_no_revs
        if haskey(AA_muts_ct_pos_only_no_dels_all, mut)
            circ_ct = AA_muts_ct_pos_only_no_dels_all[mut]
            AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = round(digits=2, chr_to_circ_mut_ratio_adj_factor_no_revs*chr_ct_no_revs/circ_ct)
        else
            AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio[mut] = chr_ct_no_revs*1000
            AA_muts_ct_pos_only_no_dels_all[mut] = 0
        end
    end
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort = sort(collect(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort = sort(collect(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio), by = x -> x[2], rev=true)
    pos_only_no_revs_ratio_print_ct = 0
    println()
    println("10 random AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio Keys and Values")
    for (mut, ratio_no_revs) in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio
        pos_only_no_revs_ratio_print_ct += 1
        if pos_only_no_revs_ratio_print_ct < 10
            println(rpad(mut, 11), " = ", rpad(ratio_no_revs, 7))
        end
    end
    println()
    for mut in keys(AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
        if !haskey(AA_muts_ct_pos_only_no_dels_no_revs, mut)
            AA_muts_ct_pos_only_no_dels_no_revs[mut] = 0
        end
    end
#######################################################################################################################################
#######################################################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_Chr_Circ_Ratios.txt", "w") do g
        println(g, "******************************************************************************************************")
        println(g, "******************* AA Chronic/Circulating Ratios (no deletions), Sorted by Ratio ********************")
        println(g, "******************************************************************************************************")
        println(g)
        for w in AA_muts_ct_no_dels_chr_all_ratio_ct_sort
             if AA_muts_ct_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "******************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios (no deletions), Sorted by Genome Position ***************")
        println(g, "******************************************************************************************************")
        println(g)
        for w in AA_muts_ct_no_dels_chr_all_ratio_pos_sort
             if AA_muts_ct_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
    end
#######################################################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_Chr_Circ_Ratios_Pos_Only.txt", "w") do g
        println(g, "******************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Ratio ****************")
        println(g, "******************************************************************************************************")
        println(g)
        for w in AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort
            if !(haskey(AA_muts_ct_pos_only_no_dels, w[1]))
                AA_muts_ct_pos_only_no_dels[w[1]] = 0
            end
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
            end
        end
        print(g, "\n"^3)
        println(g, "******************************************************************************************************")
        println(g, "******** AA Chronic/Circulating Ratios Pos Only (no deletions), Sorted by Genome Position ************")
        println(g, "******************************************************************************************************")
        println(g)
        for w in AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
            end
        end
        print(g, "\n"^3)
    end
#######################################################################################################################################
#######################################################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_Chr_Circ_Ratios_No_Revs.txt", "w") do g
        println(g, "************************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Ratio ***************")
        println(g, "************************************************************************************************************************************")
        println(g)
        for w in AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort
             if AA_muts_ct_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels_no_revs[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels_no_revs[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "**********************************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios, No Deletions, No Reversions (with special exceptions), Sorted by Genome Position ***************")
        println(g, "**********************************************************************************************************************************************")
        println(g)
        for w in AA_muts_ct_no_dels_no_revs_chr_all_ratio_pos_sort
             if AA_muts_ct_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels_no_revs[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_no_dels_no_revs[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
    end
#######################################################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_Chr_Circ_Ratios_Pos_Only_No_Revs.txt", "w") do g
        println(g, "**********************************************************************************************************************************")
        println(g, "************** AA Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Ratio ****************")
        println(g, "**********************************************************************************************************************************")
        println(g)
        for w in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_ct_sort
            if !(haskey(AA_muts_ct_pos_only_no_dels_no_revs, w[1]))
                AA_muts_ct_pos_only_no_dels_no_revs[w[1]] = 0
            end
            if AA_muts_ct_pos_only_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels_no_revs[w[1]], ")")
            end
        end
        print(g, "\n"^3)
        println(g, "**********************************************************************************************************************************")
        println(g, "********* AA Chronic/Circulating Ratios Pos Only, No Dels, No Revs (with special exceptions) Sorted by Genome Position ***********")
        println(g, "**********************************************************************************************************************************")
        println(g)
        for w in AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio_pos_sort
            if AA_muts_ct_pos_only_no_dels_no_revs[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels_no_revs[w[1]], ")")
            end
        end
        print(g, "\n"^3)
    end
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
    for (nucmut, Ct) in nuc_muts_ct
        if haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = site1*site2*site3*site4*site5
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int]
                        v_all = nuc_muts_ct_all[nucmut]
                        nuc_muts_ct_adj[nucmut] = round(digits=1, 1000*Ct/v_all)
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = site1*site2*site3*site4
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = site1*site2*site3
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = site1*site2
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
        end
        if !haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = site1*site2*site3*site4*site5
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int]
                        nuc_muts_ct_adj[nucmut] = 1001*Ct
                    end
                    if nucmut[end] == ref_seq[nuc_site_int]
                        nuc_muts_ct_adj[nucmut] = 50*Ct
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = site1*site2*site3*site4
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = site1*site2*site3
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = site1*site2
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int]
                    nuc_muts_ct_adj[nucmut] = 50*Ct
                end
            end
        end
    end
    for (mut, adj) in nuc_muts_ct_adj
        adj_pt = sqrt(nuc_muts_ct_adj[mut])
        ct_pt = (nuc_muts_ct[mut])/adj_score_ct_div
        score = adj_pt + ct_pt
        score_rd = round(digits=2, score)
        nuc_muts_ct_adj_score[mut] = score_rd
    end
    for (nucmut, Ct) in nuc_muts_ct
        if haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = site1*site2*site3*site4*site5
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        v_all = nuc_muts_ct_all[nucmut]
                        nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, 1000*Ct/v_all)
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = site1*site2*site3*site4
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = site1*site2*site3
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = site1*site2
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    v_all = nuc_muts_ct_all[nucmut]
                    nuc_muts_ct_adj_no_dels[nucmut] = round(digits=1, 1000*Ct/v_all)
                end
            end
        end
        if !haskey(nuc_muts_ct_all, nucmut)
            if length(nucmut) == 7
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                site5 = string(nucmut[6])
                nuc_site_str = site1*site2*site3*site4*site5
                nuc_site_int = parse(Int, nuc_site_str)
                if nuc_site_int < 29870
                    if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                    end
                    if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                        nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                    end
                end
            end
            if length(nucmut) == 6
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                site4 = string(nucmut[5])
                nuc_site_str = site1*site2*site3*site4
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 5
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                site3 = string(nucmut[4])
                nuc_site_str = site1*site2*site3
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
            if length(nucmut) == 4
                site1 = string(nucmut[2])
                site2 = string(nucmut[3])
                nuc_site_str = site1*site2
                nuc_site_int = parse(Int, nuc_site_str)
                if nucmut[end] ≠ ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 1001*Ct
                end
                if nucmut[end] == ref_seq[nuc_site_int] && nucmut[end] ≠ '-'
                    nuc_muts_ct_adj_no_dels[nucmut] = 50*Ct
                end
            end
        end
    end
    for (mut, adj) in nuc_muts_ct_adj
        if mut[end] ≠ '-'
            adj_pt = sqrt(nuc_muts_ct_adj_no_dels[mut])
            ct_pt = (nuc_muts_ct[mut])/adj_score_ct_div
            #adj_pt_rd = round(digits=2, adj_pt)
            #ct_pt_rd = round(digits=2, ct_pt)
            #score = adj_pt_rd + ct_pt_rd
            score = adj_pt + ct_pt
            score_rd = round(digits=2, score)
            nuc_muts_ct_adj_score_no_dels[mut] = score_rd
        end
    end
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
    max_l = 12
    minimum_count_1 = 50
    minimum_score_1 = 10
    min_comb_score = 100
    for (AAmut, Ct) in AA_muts_ct
        if haskey(AA_muts_ct_all, AAmut)
            v_all = AA_muts_ct_all[AAmut]
            AA_muts_ct_adj[AAmut] = round(digits=1, 1000*Ct/v_all)
        end
        if !haskey(AA_muts_ct_all, AAmut)
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos = AA_mut[2:end-1]
            AA_mut_pos_int = parse(Int, AA_mut_pos)
            qry_AA = AA_mut[end]
            qry_AA_str = string(qry_AA)
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct[AAmut]
            if qry_AA_str == ref_seq_AA_str 
                AA_muts_ct_adj[AAmut] = round(digits=1, 50*Ct)
            else
                AA_muts_ct_adj[AAmut] = round(digits=1, 1001*Ct)
            end
        end
#######################################################################################################################################        
        if haskey(AA_muts_ct_all, AAmut) && AAmut[end] ≠ '-'
            v_all = AA_muts_ct_all[AAmut]
            AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 1000*Ct/v_all)
        end
        if !haskey(AA_muts_ct_all, AAmut) && AAmut[end] ≠ '-'
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos = AA_mut[2:end-1]
            AA_mut_pos_int = parse(Int, AA_mut_pos)
            qry_AA = AA_mut[end]
            qry_AA_str = string(qry_AA)
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct[AAmut]
            if qry_AA_str == ref_seq_AA_str 
                AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 50*Ct)
            else
                AA_muts_ct_adj_no_dels[AAmut] = round(digits=1, 1001*Ct)
            end
        end
#######################################################################################################################################
    end
#######################################################################################################################################
#######################################################################################################################################                
#######################################################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only
        rev_ct = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
        if haskey(AA_muts_ct_pos_only_all, AAmutpos)
            v_all = AA_muts_ct_pos_only_all[AAmutpos]
            non_rev_adj = 1000*non_rev_ct/v_all
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj[AAmutpos] = round(digits=1, total_adj)
        end
        if !haskey(AA_muts_ct_pos_only_all, AAmutpos)
            non_rev_adj = 1001*non_rev_ct
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj[AAmutpos] = round(digits=1, total_adj)
        end
    end
#############################################################################################################               
#############################################################################################################                
#############################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only_no_dels
        rev_ct = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
        if haskey(AA_muts_ct_pos_only_all, AAmutpos)
            v_all = AA_muts_ct_pos_only_all[AAmutpos]
            non_rev_adj = 1000*non_rev_ct/v_all
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj_no_dels[AAmutpos] = round(digits=1, total_adj)
        end
        if !haskey(AA_muts_ct_pos_only_all, AAmutpos)
            non_rev_adj = 1001*non_rev_ct
            rev_adj = 50*rev_ct
            total_adj = non_rev_adj + rev_adj
            AA_muts_ct_pos_only_adj_no_dels[AAmutpos] = round(digits=1, total_adj)
        end
    end
#############################################################################################################
    for (AAmutpos, Ct) in AA_muts_ct_pos_only_no_dels
        rev_ct = 0
        try
            rev_ct = AA_muts_ct_pos_only_reversion[AAmutpos]
        catch
            rev_ct = 0
        end
        non_rev_ct = Ct - rev_ct
    end
#############################################################################################################               
#############################################################################################################                
#############################################################################################################
#############################################################################################################               
#############################################################################################################                
    for (mut, adj) in AA_muts_ct_adj
        if mut[end] ≠ '*'
            adj_pt = sqrt(AA_muts_ct_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_adj_score[mut] = score_rd
        end
    end
#############################################################################################################               
#############################################################################################################                
#############################################################################################################
    for (mut, adj) in AA_muts_ct_adj
        if mut[end] ≠ '*' && mut[end] ≠ '-'
            adj_pt = sqrt(AA_muts_ct_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_adj_score_no_dels[mut] = score_rd
        end
    end
#############################################################################################################               
#############################################################################################################                
#############################################################################################################        
    for (mut, adj) in AA_muts_ct_pos_only_adj
        if mut[end] ≠ '*'
            adj_pt = sqrt(AA_muts_ct_pos_only_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct_pos_only_adj[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_pos_only_adj_score[mut] = score_rd                
        end
    end
#############################################################################################################               
#############################################################################################################                
#############################################################################################################
    for (mut, adj) in AA_muts_ct_pos_only_adj
        if mut[end] ≠ '*' && mut[end] ≠ '-'
            adj_pt = sqrt(AA_muts_ct_pos_only_adj[mut])
            adj_pt_rd = round(digits=2, adj_pt)
            ct_pt = (AA_muts_ct_pos_only_adj[mut])/adj_score_ct_div
            ct_pt_rd = round(digits=2, ct_pt)
            score = adj_pt_rd + ct_pt_rd
            score_rd = round(digits=2, score)
            AA_muts_ct_pos_only_adj_score_no_dels[mut] = score_rd                
        end
    end
    AA_pos_int(m) = parse(Int, AA_position(m))      
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "E" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            E_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "M" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            M_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF3a" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF3a_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF6" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF6_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF7a" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF7a_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF7b" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF7b_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF8" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF8_mut_ct_MINADJ += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF9b" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            ORF9b_mut_ct_MINADJ += AAmutct
        end
    end
#############################################################################################################
#############################################################################################################
#############################################################################################################
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF1a" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            if AA_pos_int(mut) ≤ 180
                NSP1_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [181:818...]        
                NSP2_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [819:2763...]  
                NSP3_mut_ct_MINADJ += AAmutct
                if AA_pos_int(mut) in [819:929...]    ### NSP3:1-111
                    NSP3_Ubl1_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [930:1024...]   ### NSP3:112-206
                    NSP3_HVR_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1025:1192...]  ### NSP3:207-374
                    NSP3_Mac1_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1193:1368...]  ### NSP3:375-550
                    NSP3_Mac2_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1369:1491...]  ### NSP3:551-673
                    NSP3_Mac3_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1492:1561...]  ### NSP3:674-743
                    NSP3_DPUP_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1562:1621...]  ### NSP3:744-803
                    NSP3_Ubl2_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1622:1870...]  ### NSP3:804-1052
                    NSP3_PLpro_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [1871:2021...]  ### NSP3:1053-1203
                    NSP3_NAB_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2022:2152...]  ### NSP3:1204-1334
                    NSP3_BSM_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2153:2258...]  ### NSP3:1335-1440
                    NSP3_TM1_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2259:2317...]  ### NSP3:1441-1499
                    NSP3_Ecto3_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2318:2404...]  ### NSP3:1500-1586
                    NSP3_TM234HLX_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2405:2582...]  ### NSP3:1587-1764
                    NSP3_Y1_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2583:2763...]  ### NSP3:1765-1945
                    NSP3_CoVY_ct_MINADJ += AAmutct
                end
            elseif AA_pos_int(mut) in [2764:3263...]  
                NSP4_mut_ct_MINADJ += AAmutct
                if AA_pos_int(mut) in [2764:2794...]  ### NSP4:1-31
                    NSP4_TM1_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [2795:3034...]  ### NSP4:32-271
                    NSP4_Ecto4_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [3035:3173...]  ### NSP4:272-410
                    NSP4_TM2_TM6_ct_MINADJ += AAmutct
                end
                if AA_pos_int(mut) in [3174:3263...]  ### NSP4:411-500
                    NSP4_CTD_ct_MINADJ += AAmutct
                end
            elseif AA_pos_int(mut) in [3264:3569...]  
                NSP5_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [3570:3859...]
                NSP6_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [3860:3942...]
                NSP7_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [3943:4140...]
                NSP8_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [4141:4253...]
                NSP9_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [4253:4392...]
                NSP10_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [4392:4400...]
                NSP12_mut_ct_MINADJ += AAmutct
                NSP12_NiRAN_ct_MINADJ += AAmutct   ### NSP12:1-250
            end
        end
        if gene_name_fx(mut) == "ORF1b" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            if AA_pos_int(mut) ≤ 923
                NSP12_mut_ct_MINADJ += AAmutct
                if AA_pos_int(mut) in [1:241...]
                    NSP12_NiRAN_ct_MINADJ += AAmutct        ### NSP12:1-250
                end
                if AA_pos_int(mut) in [242:389...]
                    NSP12_intrfce_ct_MINADJ += AAmutct    ### NSP12:251-398
                end
                if AA_pos_int(mut) in [390:572...]
                    NSP12_fingers_ct_MINADJ += AAmutct      ### NSP12:399-581
                end
                if AA_pos_int(mut) in [573:618...]
                    NSP12_palm_ct_MINADJ += AAmutct         ### NSP12:582-627
                end
                if AA_pos_int(mut) in [619:678...]
                    NSP12_palmLnk_ct_MINADJ += AAmutct  ### NSP12:628-687
                end
                if AA_pos_int(mut) in [679:803...]
                    NSP12_palm_ct_MINADJ += AAmutct         ### NSP12:688-812
                end
                if AA_pos_int(mut) in [804:923...]
                    NSP12_thumb_ct_MINADJ += AAmutct        ### NSP12:813-932
                end
            elseif AA_pos_int(mut) in [924:1524...]
                NSP13_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [1525:2051...]
                NSP14_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) in [2052:2397...]
                NSP15_mut_ct_MINADJ += AAmutct
            elseif AA_pos_int(mut) ≥ 2398
                NSP16_mut_ct_MINADJ += AAmutct
            end
        end
        if gene_name_fx(mut) == "S" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            S_mut_ct_MINADJ += AAmutct
            if AA_pos_int(mut) in [1:669...]
                S_S1_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [691:1273...] 
                S_S2_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [2:306...]
                S_NTD_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [331:527...]
                S_RBD_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [670:690...]
                S_FCS_region_ct_MINADJ += AAmutct
            end
        end
        if gene_name_fx(mut) == "N" && mut[end] ≠ '-' && AA_muts_ct_adj[mut] ≥ minadj
            N_mut_ct_MINADJ += AAmutct
            if AA_pos_int(mut) in [2:44...]
                N_N1_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [45:175...]
                N_N2_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [176:246...]
                N_N3_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [247:364...]
                N_N4_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [365:419...]
                N_N5_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [176:206...]
                N_SR_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [215:235...]
                N_L_helix_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [247:279...]
                N_CBP_ct_MINADJ += AAmutct
            end
            if AA_pos_int(mut) in [4:101...]
                N_9b_overlap_ct_MINADJ += AAmutct
            end
        end
    end
#############################################################################################################
#############################################################################################################
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "E" && mut[end] ≠ '-'
            E_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "M" && mut[end] ≠ '-'
            M_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF3a" && mut[end] ≠ '-'
            ORF3a_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF6" && mut[end] ≠ '-'
            ORF6_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF7a" && mut[end] ≠ '-'
            ORF7a_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF7b" && mut[end] ≠ '-'
            ORF7b_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF8" && mut[end] ≠ '-'
            ORF8_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF9b" && mut[end] ≠ '-'
            ORF9b_mut_ct += AAmutct
        end
    end
    for (mut, AAmutct) in AA_muts_ct
        if gene_name_fx(mut) == "ORF1a" && mut[end] ≠ '-'
            if AA_pos_int(mut) ≤ 180
                NSP1_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [181:818...]        
                NSP2_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [819:2763...]
                NSP3_mut_ct += AAmutct
                if AA_pos_int(mut) in [819:929...]
                    NSP3_Ubl1_ct += AAmutct
                end
                if AA_pos_int(mut) in [930:1024...]  
                    NSP3_HVR_ct += AAmutct
                end
                if AA_pos_int(mut) in [1025:1192...]
                    NSP3_Mac1_ct += AAmutct
                end
                if AA_pos_int(mut) in [1193:1368...]
                    NSP3_Mac2_ct += AAmutct
                end
                if AA_pos_int(mut) in [1369:1491...]   
                    NSP3_Mac3_ct += AAmutct
                end
                if AA_pos_int(mut) in [1492:1561...]    
                    NSP3_DPUP_ct += AAmutct
                end
                if AA_pos_int(mut) in [1562:1621...]
                    NSP3_Ubl2_ct += AAmutct
                end
                if AA_pos_int(mut) in [1622:1870...]
                    NSP3_PLpro_ct += AAmutct
                end
                if AA_pos_int(mut) in [1871:2021...]
                    NSP3_NAB_ct += AAmutct
                end
                if AA_pos_int(mut) in [2022:2152...]
                    NSP3_BSM_ct += AAmutct
                end
                if AA_pos_int(mut) in [2153:2258...]
                    NSP3_TM1_ct += AAmutct
                end
                if AA_pos_int(mut) in [2259:2317...]
                    NSP3_Ecto3_ct += AAmutct
                end
                if AA_pos_int(mut) in [2318:2404...]
                    NSP3_TM234HLX_ct += AAmutct
                end
                if AA_pos_int(mut) in [2405:2582...]
                    NSP3_Y1_ct += AAmutct
                end
                if AA_pos_int(mut) in [2583:2763...]
                    NSP3_CoVY_ct += AAmutct
                end
            elseif AA_pos_int(mut) in [2764:3263...]
                NSP4_mut_ct += AAmutct
                if AA_pos_int(mut) in [2764:2794...]  
                    NSP4_TM1_ct += AAmutct
                end
                if AA_pos_int(mut) in [2795:3034...]  
                    NSP4_Ecto4_ct += AAmutct
                end
                if AA_pos_int(mut) in [3035:3173...]
                    NSP4_TM2_TM6_ct += AAmutct
                end
                if AA_pos_int(mut) in [3174:3263...]
                    NSP4_CTD_ct += AAmutct
                end
            elseif AA_pos_int(mut) in [3264:3569...]
                NSP5_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [3570:3859...]
                NSP6_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [3860:3942...]
                NSP7_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [3943:4140...]
                NSP8_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [4141:4253...]
                NSP9_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [4253:4392...]
                NSP10_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [4392:4400...]
                NSP12_mut_ct += AAmutct
                NSP12_NiRAN_ct += AAmutct
            end
        end
        if gene_name_fx(mut) == "ORF1b" && mut[end] ≠ '-'
            if AA_pos_int(mut) ≤ 923
                NSP12_mut_ct += AAmutct
                if AA_pos_int(mut) in [1:241...]
                    NSP12_NiRAN_ct += AAmutct
                end
                if AA_pos_int(mut) in [242:389...]
                    NSP12_intrfce_ct += AAmutct
                end
                if AA_pos_int(mut) in [390:572...]
                    NSP12_fingers_ct += AAmutct
                end
                if AA_pos_int(mut) in [573:618...]
                    NSP12_palm_ct += AAmutct
                end
                if AA_pos_int(mut) in [619:678...]
                    NSP12_palmLnk_ct += AAmutct
                end
                if AA_pos_int(mut) in [679:803...]
                    NSP12_palm_ct += AAmutct
                end
                if AA_pos_int(mut) in [804:923...]
                    NSP12_thumb_ct += AAmutct
                end
            elseif AA_pos_int(mut) in [924:1524...]
                NSP13_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [1525:2051...]
                NSP14_mut_ct += AAmutct
            elseif AA_pos_int(mut) in [2052:2397...]
                NSP15_mut_ct += AAmutct
            elseif AA_pos_int(mut) ≥ 2398
                NSP16_mut_ct += AAmutct
            end
        end
        if gene_name_fx(mut) == "S" && mut[end] ≠ '-'
            S_mut_ct += AAmutct
            if AA_pos_int(mut) in [1:669...]
                S_S1_ct += AAmutct
            end
            if AA_pos_int(mut) in [691:1273...] 
                S_S2_ct += AAmutct
            end
            if AA_pos_int(mut) in [2:306...]
                S_NTD_ct += AAmutct
            end
            if AA_pos_int(mut) in [331:527...]
                S_RBD_ct += AAmutct
            end
            if AA_pos_int(mut) in [670:690...]
                S_FCS_region_ct += AAmutct
            end
        end
        if gene_name_fx(mut) == "N" && mut[end] ≠ '-'
            N_mut_ct += AAmutct
            if AA_pos_int(mut) in [2:44...]
                N_N1_ct += AAmutct
            end
            if AA_pos_int(mut) in [45:175...]
                N_N2_ct += AAmutct
            end
            if AA_pos_int(mut) in [176:246...]
                N_N3_ct += AAmutct
            end
            if AA_pos_int(mut) in [247:364...]
                N_N4_ct += AAmutct
            end
            if AA_pos_int(mut) in [365:419...]
                N_N5_ct += AAmutct
            end
            if AA_pos_int(mut) in [176:206...]
                N_SR_ct += AAmutct
            end
            if AA_pos_int(mut) in [215:235...]
                N_L_helix_ct += AAmutct
            end
            if AA_pos_int(mut) in [247:279...]
                N_CBP_ct += AAmutct
            end
            if AA_pos_int(mut) in [4:101...]
                N_9b_overlap_ct += AAmutct
            end
        end
    end
#############################################################################################################
    function AA_mutations_in_seq(g::IO, EPI_ISL::String)        
        println(g, "Mutations in sequence $(EPI_ISL)")
        collected_muts = collect(seq_AA_muts[EPI_ISL])
        mut_list = join(collected_muts, ", ")
        println(g, mut_list)
    end
    function print_seqs_with_AA_mut_plus_their_other_AA_muts(g::IO, mut::String)
        println(g, "$(mut) sequences")
        if haskey(AA_muts_seq, mut)
            seq_list = collect(AA_muts_seq[mut])
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************** Mutations in Sequences with $(mut) **********************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            for seq in seq_list
                AA_mutations_in_seq(g, seq)
                seq_mut_total = length(seq_AA_muts[seq])
                println(g, "Total Private Mutations in ", seq, " = ", seq_mut_total)
                println(g)
            end
        end
    end
    function nuc_muts_in_seq(g::IO, EPI_ISL)
        println(g, "Nuc Mutations in sequence ", EPI_ISL)
        seq_mut_list = join(collect(seq_AA_muts[EPI_ISL]), ", ")
        println(g, seq_mut_list)
    end
    function print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g::IO, nuc_mut::String)
        println("$(nuc_mut) sequences")
        println(g, "$(nuc_mut) sequences")
        if haskey(nuc_muts_seq, nuc_mut)
            seq_list = collect(nuc_muts_seq[nuc_mut])
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "******************** Mutations in Sequences with $(nuc_mut) ********************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            println(g, "********************************************************************************")
            for seq in seq_list
                nuc_muts_in_seq(g, seq)
                seq_mut_total = length(seq_nuc_muts[seq])
                println(g, "Total Private Mutations in ", seq, " = ", seq_mut_total)
                println(g)
            end
        end
    end
    
    function print_pos_only_adj(g::IO, gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
        print(g, "\n"^2)
        println(g, "***************** $(gene_name) AA Mutations *POS ONLY* Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) *****************")
        for w in AA_muts_ct_pos_only_adj_sort_by_seq_ct
            AAmut = w[1]
            AAct = w[2]
            max_l = 12
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct_pos_only[AAmut]
            len = length(gene_char_num)
            if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)    ## get(collection, key, default_value)
                if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                end
            end
        end
    end

    function print_pos_only_adj_score(g::IO, gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
        print(g, "\n"^2)
        println(g, "***************** $(gene_name) AA Mutations *POS ONLY* Sorted by Adjusted SCORE (Ratio of Chronic:Circulating Private Muts) *****************")
        for w in AA_muts_ct_pos_only_adj_score_sort_by_score
            AAmut = w[1]
            AAct = w[2]
            max_l = 12
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct_pos_only[AAmut]
            len = length(gene_char_num)
            if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)    ## get(collection, key, default_value)
                if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                end
            end
        end
    end

    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_rep_seq_grps.txt", "w") do g
        println(g)
        rep_seq_grps_seqs_grp_size_sort = sort(collect(rep_seq_grps_seqs), by = x -> length(x[2]), rev=true)
        println(g, "#############################################################################################")
        println(g, "############################### Groups with ≥5 Sequences ####################################")
        println(g, "#############################################################################################")
        for (grp_num, seq_EPIs_set) in rep_seq_grps_seqs_grp_size_sort
            seq_EPIs_arr = collect(seq_EPIs_set)
            seq_EPIs_sort = sort(seq_EPIs_arr)
            seq_EPIs_sort_str = join(seq_EPIs_sort, ", ")
            seq_EPI_set_len = length(seq_EPIs_set)
            if seq_EPI_set_len ≥ 5
                println(g)
                println(g, "#############################################################################################")
#                println("Group Number ", grp_num, "    Total Sequences in Group = ", seq_EPI_set_len)
                println(g, "Group Number ", grp_num, "    Total Sequences in Group = ", seq_EPI_set_len)
                println(g, "EPI_ISLs in Group Number ", grp_num, " = ", seq_EPIs_sort_str)
                println(g)
                rep_seq_grps_AA_sort = sort(collect(rep_seq_grps_AA[grp_num]))
                rep_seq_grps_muts_sort = sort(collect(rep_seq_grps_muts[grp_num]))
                println(g, "Collective Mutations in Group Number $(grp_num)")
                for mut in rep_seq_grps_muts_sort
                    print(g, mut, ", ")
                end
                println(g)
                println(g, "Collective AA Mutations in Group Number $(grp_num)")
                for AA in rep_seq_grps_AA_sort
                    print(g, AA, ", ")
                end
                println(g)
            end
            println(g, "#############################################################################################")
            println(g, "############################### Groups with <5 Sequences ####################################")
            println(g, "#############################################################################################")
            if seq_EPI_set_len < 5
                println(g)
                println(g, "#############################################################################################")
                println(g, "Group Number ", grp_num, "    Total Sequences in Group = ", seq_EPI_set_len)
                println(g, "EPI_ISLs in Group Number ", grp_num, " = ", seq_EPIs_sort_str)
                println(g)
                rep_seq_grps_AA_sort = sort(collect(rep_seq_grps_AA[grp_num]))
                rep_seq_grps_muts_sort = sort(collect(rep_seq_grps_muts[grp_num]))                        
                println(g, "Collective Mutations in Group Number $(grp_num)")
                for mut in rep_seq_grps_muts_sort
                    print(g, mut, ", ")
                end
                println(g)
                println(g, "Collective AA Mutations in Group Number $(grp_num)")
                for AA in rep_seq_grps_AA_sort
                    print(g, AA, ", ")
                end
                println(g)
            end
        end
    end
    nuc_num_from_key(k) = parse(Int, k[2:end-1])
    nuc_muts_ct_sort_by_site = sort(collect(nuc_muts_ct), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_sort_by_seq_ct = sort(collect(nuc_muts_ct), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_sort_by_site = sort(collect(nuc_muts_ct_adj), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_sort_by_seq_ct = sort(collect(nuc_muts_ct_adj), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_no_dels_sort_by_site = sort(collect(nuc_muts_ct_adj_no_dels), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_no_dels_sort_by_seq_ct = sort(collect(nuc_muts_ct_adj_no_dels), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_score_sort_by_site = sort(collect(nuc_muts_ct_adj_score), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_score_sort_by_score = sort(collect(nuc_muts_ct_adj_score), by = x -> x[2], rev = true)
    nuc_muts_ct_adj_score_no_dels_sort_by_site = sort(collect(nuc_muts_ct_adj_score_no_dels), by = x -> nuc_num_from_key(x[1]))
    nuc_muts_ct_adj_score_no_dels_sort_by_score = sort(collect(nuc_muts_ct_adj_score_no_dels), by = x -> x[2], rev = true)
#############################################################################################################    
    date = Dates.format(today(), "yyyy-mm-dd")
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_NUCS.txt", "a") do g       
        print(g, "\n"^2)
        println(g, "************** TOP NUC MUTATIONS Sorted by Count ***************")
        println(g)
        for w in nuc_muts_ct_adj_sort_by_seq_ct
            nucmut = w[1]
            nucct = w[2]
            if nuc_muts_ct[nucmut] ≥ print_ct_thresh
                nuc_mut_pos = nucmut[2:end-1]
                nuc_mut_pos_int = parse(Int, nuc_mut_pos)
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc = nucmut[end]
                    qry_nuc_str = string(qry_nuc)
                    ref_seq_nuc = ref_seq[nuc_mut_pos_int]
                    ref_seq_nuc_str = string(ref_seq_nuc)
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, nucmut, " = ", nucct, " (", nuc_muts_ct[nucmut], ")") 
                    else
                        println(g, nucmut, " = ", nucct, " (", nuc_muts_ct[nucmut], ", REVERSION)") 
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "************** TOP NUC MUTATIONS Sorted by Genome Position ***************")
        println(g)
        for w in nuc_muts_ct_adj_sort_by_site
            nucmut = w[1]
            nucct = w[2]
            if nuc_muts_ct[nucmut] ≥ print_ct_thresh
                nuc_mut_pos = nucmut[2:end-1]
                nuc_mut_pos_int = parse(Int, nuc_mut_pos)
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc = nucmut[end]
                    qry_nuc_str = string(qry_nuc)
                    ref_seq_nuc = ref_seq[nuc_mut_pos_int]
                    ref_seq_nuc_str = string(ref_seq_nuc)
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, nucmut, " = ", nucct, " (", nuc_muts_ct[nucmut], ")") 
                    else
                        println(g, nucmut, " = ", nucct, " (", nuc_muts_ct[nucmut], ", REVERSION)") 
                    end
                end
            end
        end
    end
#############################################################################################################    
    AA_muts_ct_sort_by_site = sort(collect(AA_muts_ct), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_sort_by_seq_ct = sort(collect(AA_muts_ct), by = x -> x[2], rev = true)

    AA_muts_ct_no_dels_sort_by_site = sort(collect(AA_muts_ct_no_dels), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_no_dels), by = x -> x[2], rev = true)

    AA_muts_ct_pos_only_sort_by_site = sort(collect(AA_muts_ct_pos_only), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only), by = x -> x[2], rev = true)
    AA_muts_ct_pos_only_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_no_dels), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_no_dels), by = x -> x[2], rev = true)
    
    AA_muts_ct_adj_sort_by_site = sort(collect(AA_muts_ct_adj), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_adj_sort_by_seq_ct = sort(collect(AA_muts_ct_adj), by = x -> x[2], rev = true)
    AA_muts_ct_adj_no_dels_sort_by_site = sort(collect(AA_muts_ct_adj_no_dels), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_adj_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_adj_no_dels), by = x -> x[2], rev = true)

    AA_muts_ct_adj_score_sort_by_site = sort(collect(AA_muts_ct_adj_score), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_adj_score_sort_by_score = sort(collect(AA_muts_ct_adj_score), by = x -> x[2], rev = true)
    AA_muts_ct_adj_score_no_dels_sort_by_site = sort(collect(AA_muts_ct_adj_score_no_dels), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_adj_score_no_dels_sort_by_score = sort(collect(AA_muts_ct_adj_score_no_dels), by = x -> x[2], rev = true)
    
    # println(typeof(AA_muts_ct_adj_sort_by_seq_ct))  ### Vector{Pair{String, Int64}}
    
#    gene_AA_dict = Dict{String, String}()
#    gene_AA_dict["ORF1a"] = "MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQGKPLEFGATSAALQPEEEQEEDWLDDDSQQTVGQQDGSEDNQTTTIQTIVEVQPQLEMELTPVVQTIEVNSFSGYLKLTDNVYIKNADIVEEAKKVKPTVVVNAANVYLKHGGGVAGALNKATNNAMQVESDDYIATNGPLKVGGSCVLSGHNLAKHCLHVVGPNVNKGEDIQLLKSAYENFNQHEVLLAPLLSAGIFGADPIHSLRVCVDTVRTNVYLAVFDKNLYDKLVSSFLEMKSEKQVEQKIAEIPKEEVKPFITESKPSVEQRKQDDKKIKACVEEVTTTLEETKFLTENLLLYIDINGNLHPDSATLVSDIDITFLKKDAPYIVGDVVQEGVLTAVVIPTKKAGGTTEMLAKALRKVPTDNYITTYPGQGLNGYTVEEAKTVLKKCKSAFYILPSIISNEKQEILGTVSWNLREMLAHAEETRKLMPVCVETKAIVSTIQRKYKGIKIQEGVVDYGARFYFYTSKTTVASLINTLNDLNETLVTMPLGYVTHGLNLEEAARYMRSLKVPATVSVSSPDAVTAYNGYLTSSSKTPEEHFIETISLAGSYKDWSYSGQSTQLGIEFLKRGDKSVYYTSNPTTFHLDGEVITFDNLKTLLSLREVRTIKVFTTVDNINLHTQVVDMSMTYGQQFGPTYLDGADVTKIKPHNSHEGKTFYVLPNDDTLRVEAFEYYHTTDPSFLGRYMSALNHTKKWKYPQVNGLTSIKWADNNCYLATALLTLQQIELKFNPPALQDAYYRARAGEAANFCALILAYCNKTVGELGDVRETMSYLFQHANLDSCKRVLNVVCKTCGQQQTTLKGVEAVMYMGTLSYEQFKKGVQIPCTCGKQATKYLVQQESPFVMMSAPPAQYELKHGTFTCASEYTGNYQCGHYKHITSKETLYCIDGALLTKSSEYKGPITDVFYKENSYTTTIKPVTYKLDGVVCTEIDPKLDNYYKKDNSYFTEQPIDLVPNQPYPNASFDNFKFVCDNIKFADDLNQLTGYKKPASRELKVTFFPDLNGDVVAIDYKHYTPSFKKGAKLLHKPIVWHVNNATNKATYKPNTWCIRCLWSTKPVETSNSFDVLKSEDAQGMDNLACEDLKPVSEEVVENPTIQKDVLECNVKTTEVVGDIILKPANNSLKITEEVGHTDLMAAYVDNSSLTIKKPNELSRVLGLKTLATHGLAAVNSVPWDTIANYAKPFLNKVVSTTTNIVTRCLNRVCTNYMPYFFTLLLQLCTFTRSTNSRIKASMPTTIAKNTVKSVGKFCLEASFNYLKSPNFSKLINIIIWFLLLSVCLGSLIYSTAALGVLMSNLGMPSYCTGYREGYLNSTNVTIATYCTGSIPCSVCLSGLDSLDTYPSLETIQITISSFKWDLTAFGLVAEWFLAYILFTRFFYVLGLAAIMQLFFSYFAVHFISNSWLMWLIINLVQMAPISAMVRMYIFFASFYYVWKSYVHVVDGCNSSTCMMCYKRNRATRVECTTIVNGVRRSFYVYANGGKGFCKLHNWNCVNCDTFCAGSTFISDEVARDLSLQFKRPINPTDQSSYIVDSVTVKNGSIHLYFDKAGQKTYERHSLSHFVNLDNLRANNTKGSLPINVIVFDGKSKCEESSAKSASVYYSQLMCQPILLLDQALVSDVGDSAEVAVKMFDAYVNTFSSTFNVPMEKLKTLVATAEAELAKNVSLDNVLSTFISAARQGFVDSDVETKDVVECLKLSHQSDIEVTGDSCNNYMLTYNKVENMTPRDLGACIDCSARHINAQVAKSHNIALIWNVKDFMSLSEQLRKQIRSAAKKNNLPFKLTCATTRQVVNVVTTKIALKGGKIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGYKAIDGGVTRDIASTDTCFANKHADFDTWFSQRGGSYTNDKACPLIAAVITREVGFVVPGLPGTILRTTNGDFLHFLPRVFSAVGNICYTPSKLIEYTDFATSACVLAAECTIFKDASGKPVPYCYDTNVLEGSVAYESLRPDTRYVLMDGSIIQFPNTYLEGSVRVVTTFDSEYCRHGTCERSEAGVCVSTSGRWVLNNDYYRSLPGVFCGVDAVNLLTNMFTPLIQPIGALDISASIVAGGIVAIVVTCLAYYFMRFRRAFGEYSHVVAFNTLLFLMSFTVLCLTPVYSFLPGVYSVIYLYLTFYLTNDVSFLAHIQWMVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSLSGFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRYFRLTLGVYDYLVSTQEFRYMNSQGLLPPKNSIDAFKLNIKLLGVGGKPCIKVATVQSKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTTEAFEKMVSLLSVLLSMQGAVDINKLCEEMLDNRATLQAIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAKSEFDRDAAMQRKLEKMADQAMTQMYKQARSEDKRAKVTSAMQTMLFTMLRKLDNDALNNIINNARDGCVPLNIIPLTTAAKLMVVIPDYNTYKNTCDGTTFTYASALWEIQQVVDADSKIVQLSEISMDNSPNLAWPLIVTALRANSAVKLQNNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLSDLQDLKWARFPKSDGTGTIYTELEPPCRFVTDTPKGPKVKYLYFIKGLNNLNRGMVLGSLAATVRLQAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLCTHTGTGQAITVTPEANMDQESFGGASCCLYCRCHIDHPNPKGFCDLKGKYVQIPTTCANDPVGFTLKNTVCTVCGMWKGYGCSCDQLREPMLQSADAQSFLNGFAV*"
#    gene_AA_dict["ORF1b"] = "RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKPPISFPLCANGQVFGLYKNTCVGSDNVTDFNAIATCDWTNAGDYILANTCTERLKLFAAETLKATEETFKLSYGIATVREVLSDRELHLSWEVGKPRPPLNRNYVFTGYRVTKNSKVQIGEYTFEKGDYGDAVVYRGTTTYKLNVGDYFVLTSHTVMPLSAPTLVPQEHYVRITGLYPTLNISDEFSSNVANYQKVGMQKYSTLQGPPGTGKSHFAIGLALYYPSARIVYTACSHAAVDALCEKALKYLPIDKCSRIIPARARVECFDKFKVNSTLEQYVFCTVNALPETTADIVVFDEISMATNYDLSVVNARLRAKHYVYIGDPAQLPAPRTLLTKGTLEPEYFNSVCRLMKTIGPDMFLGTCRRCPAEIVDTVSALVYDNKLKAHKDKSAQCFKMFYKGVITHDVSSAINRPQIGVVREFLTRNPAWRKAVFISPYNSQNAVASKILGLPTQTVDSSQGSEYDYVIFTQTTETAHSCNVNRFNVAITRAKVGILCIMSDRDLYDKLQFTSLEIPRRNVATLQAENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIPKDMTYRRLISMMGFKMNYQVNGYPNMFITREEAIRHVRAWIGFDVEGCHATREAVGTNLPLQLGFSTGVNLVAVPTGYVDTPNNTDFSRVSAKPPPGDQFKHLIPLMYKGLPWNVVRIKIVQMLSDTLKNLSDRVVFVLWAHGFELTSMKYFVKIGPERTCCLCDRRATCFSTASDTYACWHHSIGFDYVYNPFMIDVQQWGFTGNLQSNHDLYCQVHGNAHVASCDAIMTRCLAVHECFVKRVDWTIEYPIIGDELKINAACRKVQHMVVKAALLADKFPVLHDIGNPKAIKCVPQADVEWKFYDAQPCSDKAYKIEELFYSYATHSDKFTDGVCLFWNCNVDRYPANSIVCRFDTRVLSNLNLPGCDGGSLYVNKHAFHTPAFDKSAFVNLKQLPFFYYSDSPCESHGKQVVSDIDYVPLKSATCITRCNLGGAVCRHHANEYRLYLDAYNMMISAGFSLWVYKQFDTYNLWNTFTRLQSLENVAFNVVNKGHFDGQQGEVPVSIINNTVYTKVDGVDVELFENKTTLPVNVAFELWAKRNIKPVPEVKILNNLGVDIAANTVIWDYKRDAPAHISTIGVCSMTDIAKKPTETICAPLTVFFDGRVDGQVDLFRNARNGVLITEGSVKGLQPSVGPKQASLNGVTLIGEAVKTQFNYYKKVDGVVQQLPETYFTQSRNLQEFKPRSQMEIDFLELAMDEFIERYKLEGYAFEHIVYGDFSHSQLGGLHLLIGLAKRFKESPFELEDFIPMDSTVKNYFITDAQTGSSKCVCSVIDLLLDDFVEIIKSQDLSVVSKVVKVTIDYTEISFMLWCKDGHVETFYPKLQSSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAKYTQLCQYLNTLTLAVPYNMRVIHFGAGSDKGVAPGTAVLRQWLPTGTLLVDSDLNDFVSDADSTLIGDCATVHTANKWDLIISDMYDPKTKNVTKENDSKEGFFTYICGFIQQKLALGGSVAIKITEHSWNADLYKLMGHFAWWTAFVTNVNASSSEAFLIGCNYLGKPREQIDGYVMHANYIFWRNTNPIQLSSYSLFDMSKFPLKLRGTAVMSLKEGQINDMILSLLSKGRLIIRENNRVVISSDVLVNN*"
#    gene_AA_dict["S"] = "MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*"
#    gene_AA_dict["E"] = "MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYSRVKNLNSSRVPDLLV*"
#    gene_AA_dict["M"] = "MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFLWLLWPVTLACFVLAAVYRINWITGGIAIAMACLVGLMWLSYFIASFRLFARTRSMWSFNPETNILLNVPLHGTILTRPLLESELVIGAVILRGHLRIAGHHLGRCDIKDLPKEITVATSRTLSYYKLGASQRVAGDSGFAAYSRYRIGNYKLNTDHSSSSDNIALLVQ*"
#    gene_AA_dict["N"] = "MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFTALTQHGKEDLKFPRGQGVPINTNSSPDDQIGYYRRATRRIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSPARMAGNGGDAALALLLLDRLNQLESKMSGKGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSGTWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKTFPPTEPKKDKKKKADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA*"
#    gene_AA_dict["ORF3a"] = "MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLAVFQSASKIITLKKRWQLALSKGVHFVCNLLLLFVTVYSHLLLVAAGLEAPFLYLYALVYFLQSINFVRIIMRLWLCWKCRSKNPLLYDANYFLCWHTNCYDYCIPYNSVTSSIVITSGDGTTSPISEHDYQIGGYTEKWESGVKDCVVLHSYFTSDYYQLYSTQLSTDTGVEHVTFFIYNKIVDEPEEHVQIHTIDGSSGVVNPVMEPIYDEPTTTTSVPL*"
#    gene_AA_dict["ORF6"] = "MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDEEQPMEID*"
#    gene_AA_dict["ORF7a"] = "MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE*"
#    gene_AA_dict["ORF7b"] = "MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*"
#    gene_AA_dict["ORF8"] = "MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKSAPLIELCVDEAGSKSPIQYIDIGNYTVSCLPFTINCQEPKLGSLVVRCSFYEDFLEYHDVRVVLDFI*"
#    gene_AA_dict["ORF9b"] = "MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSLNMARKTLNSLEDKAFQLTPIAVQMTKLATTEELPDEFVVVTVK*"
    
#    NSP1 = 1:180
#    NSP2 = 181:818
#    NSP3 = 819:2763
#    NSP4 = 2764:3264
#    NSP5 = 3264:3569
#    NSP6 = 3570:3859
#    NSP7 = 3860:3942
#    NSP8 = 3943:4140
#    NSP9 = 4141:4253
#    NSP10 = 4254:4392
#    NSP12 = 4392:4401, 1:923
#    NSP13 = 924:1524
#    NSP14 = 1525:2051
#    NSP15 = 2052:2397
#    NSP16 = 2397:2696
    max_l = 12
#    NSP_ranges1a = Dict{Int, BitSet}(1 => BitSet([1:180...]), 2 => BitSet([181:818...]), 3 => BitSet([819:2763...]), 4 => BitSet([2764:3263...]), 5 => BitSet([3264:3569...]), 6 => BitSet([3570:3859...]), 7 => BitSet([3860:3942...]), 8 => BitSet([3943:4140...]), 9 => BitSet([4141:4253...]), 10 => BitSet([4254:4392...]), 12 => BitSet([4393:4401...]))
#    NSP_ranges1b = Dict{Int, BitSet}(12 => BitSet([1:923...]), 13 => BitSet([924:1524...]), 14 => BitSet([1525:2051...]), 15 => BitSet([2052:2397...]), 16 => BitSet([2398:2696...]))
#    NSP1a_add = Dict{Int, Int}(1 => 0, 2 => 180, 3 => 818, 4 => 2763, 5 => 3263, 6 => 3569, 7 => 3859, 8 => 3942, 9 => 4140, 10 => 4353, 11 => 0, 12 => 4392)
#    NSP1b_add = Dict{Int, Int}(12 => -9, 13 => 923, 14 => 1524, 15 => 2051, 16 => 2397)
#    NSP_muts = Dict("NSP$(i)" => Dict{String, Int}() for i in 1:16 if i ≠ 11)
    NSPmutPosKey(n) = parse(Int, split(n, ":")[2][2:end-1])
    NSPmutPosKey2(n) = parse(Int, split(n, ":")[2])
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_TOP_NSP_MUTS.txt", "w") do g 
        print(g, "\n"^2)
        println(g, "********************** TOP AA MUTS by NSP ************************")
        println(g)
        for w in AA_muts_ct
            AAmut = w[1]
            AAct = w[2]
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos = AA_mut[2:end-1]
            AA_mut_pos_int = parse(Int, AA_mut_pos)
            ref_AA = AA_mut[1]
            ref_AA_str = string(ref_AA)
            qry_AA = AA_mut[end]
            qry_AA_str = string(qry_AA)
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            for i in 1:12
                if i ≠ 11
                    if jeen == "ORF1a" && AA_mut_pos_int in NSP_ranges1a[i]
                        NSP_pos = AA_mut_pos_int - NSP1a_add[i]
                        if qry_AA_str == ref_seq_AA_str
                            NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            if qry_AA_str ≠ "-"
                                NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            end
                        else
                            NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            if qry_AA_str ≠ "-"
                                NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                            end
                        end
                    end
                end
            end
            for i in 12:16
                if jeen == "ORF1b" && AA_mut_pos_int in NSP_ranges1b[i]
                    NSP_pos = AA_mut_pos_int - NSP1b_add[i]
                    if qry_AA_str == ref_seq_AA_str
                        NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        if qry_AA_str ≠ "-"
                            NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        end
                    else
                        NSP_muts["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        if qry_AA_str ≠ "-"
                            NSP_muts_no_dels["NSP$(i)"]["NSP$(i):$(ref_AA)$(NSP_pos)$(qry_AA)"] = AAct
                        end
                    end
                end
            end
        end
        NSP1_muts_sortByCt = sort(collect(NSP_muts["NSP1"]), by = x -> x[2], rev=true)
        NSP1_muts_sortByPos = sort(collect(NSP_muts["NSP1"]), by = x -> NSPmutPosKey(x[1]))
        NSP2_muts_sortByCt = sort(collect(NSP_muts["NSP2"]), by = x -> x[2], rev=true)
        NSP2_muts_sortByPos = sort(collect(NSP_muts["NSP2"]), by = x -> NSPmutPosKey(x[1]))
        NSP3_muts_sortByCt = sort(collect(NSP_muts["NSP3"]), by = x -> x[2], rev=true)
        NSP3_muts_sortByPos = sort(collect(NSP_muts["NSP3"]), by = x -> NSPmutPosKey(x[1]))
        NSP4_muts_sortByCt = sort(collect(NSP_muts["NSP4"]), by = x -> x[2], rev=true)
        NSP4_muts_sortByPos = sort(collect(NSP_muts["NSP4"]), by = x -> NSPmutPosKey(x[1]))
        NSP5_muts_sortByCt = sort(collect(NSP_muts["NSP5"]), by = x -> x[2], rev=true)
        NSP5_muts_sortByPos = sort(collect(NSP_muts["NSP5"]), by = x -> NSPmutPosKey(x[1]))
        NSP6_muts_sortByCt = sort(collect(NSP_muts["NSP6"]), by = x -> x[2], rev=true)
        NSP6_muts_sortByPos = sort(collect(NSP_muts["NSP6"]), by = x -> NSPmutPosKey(x[1]))
        NSP7_muts_sortByCt = sort(collect(NSP_muts["NSP7"]), by = x -> x[2], rev=true)
        NSP7_muts_sortByPos = sort(collect(NSP_muts["NSP7"]), by = x -> NSPmutPosKey(x[1]))
        NSP8_muts_sortByCt = sort(collect(NSP_muts["NSP8"]), by = x -> x[2], rev=true)
        NSP8_muts_sortByPos = sort(collect(NSP_muts["NSP8"]), by = x -> NSPmutPosKey(x[1]))
        NSP9_muts_sortByCt = sort(collect(NSP_muts["NSP9"]), by = x -> x[2], rev=true)
        NSP9_muts_sortByPos = sort(collect(NSP_muts["NSP9"]), by = x -> NSPmutPosKey(x[1]))
        NSP10_muts_sortByCt = sort(collect(NSP_muts["NSP10"]), by = x -> x[2], rev=true)
        NSP10_muts_sortByPos = sort(collect(NSP_muts["NSP10"]), by = x -> NSPmutPosKey(x[1]))
        NSP12_muts_sortByCt = sort(collect(NSP_muts["NSP12"]), by = x -> x[2], rev=true)
        NSP12_muts_sortByPos = sort(collect(NSP_muts["NSP12"]), by = x -> NSPmutPosKey(x[1]))
        NSP13_muts_sortByCt = sort(collect(NSP_muts["NSP13"]), by = x -> x[2], rev=true)
        NSP13_muts_sortByPos = sort(collect(NSP_muts["NSP13"]), by = x -> NSPmutPosKey(x[1]))
        NSP14_muts_sortByCt = sort(collect(NSP_muts["NSP14"]), by = x -> x[2], rev=true)
        NSP14_muts_sortByPos = sort(collect(NSP_muts["NSP14"]), by = x -> NSPmutPosKey(x[1]))
        NSP15_muts_sortByCt = sort(collect(NSP_muts["NSP15"]), by = x -> x[2], rev=true)
        NSP15_muts_sortByPos = sort(collect(NSP_muts["NSP15"]), by = x -> NSPmutPosKey(x[1]))
        NSP16_muts_sortByCt = sort(collect(NSP_muts["NSP16"]), by = x -> x[2], rev=true)
        NSP16_muts_sortByPos = sort(collect(NSP_muts["NSP16"]), by = x -> NSPmutPosKey(x[1]))
######################################################################################################################################
        NSP11_placeholder = Vector{Pair{String, Int64}}
######################################################################################################################################
        NSP1_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP1"]), by = x -> x[2], rev=true)
        NSP1_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP1"]), by = x -> NSPmutPosKey(x[1]))
        NSP2_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP2"]), by = x -> x[2], rev=true)
        NSP2_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP2"]), by = x -> NSPmutPosKey(x[1]))
        NSP3_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP3"]), by = x -> x[2], rev=true)
        NSP3_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP3"]), by = x -> NSPmutPosKey(x[1]))
        NSP4_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP4"]), by = x -> x[2], rev=true)
        NSP4_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP4"]), by = x -> NSPmutPosKey(x[1]))
        NSP5_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP5"]), by = x -> x[2], rev=true)
        NSP5_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP5"]), by = x -> NSPmutPosKey(x[1]))
        NSP6_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP6"]), by = x -> x[2], rev=true)
        NSP6_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP6"]), by = x -> NSPmutPosKey(x[1]))
        NSP7_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP7"]), by = x -> x[2], rev=true)
        NSP7_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP7"]), by = x -> NSPmutPosKey(x[1]))
        NSP8_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP8"]), by = x -> x[2], rev=true)
        NSP8_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP8"]), by = x -> NSPmutPosKey(x[1]))
        NSP9_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP9"]), by = x -> x[2], rev=true)
        NSP9_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP9"]), by = x -> NSPmutPosKey(x[1]))
        NSP10_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP10"]), by = x -> x[2], rev=true)
        NSP10_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP10"]), by = x -> NSPmutPosKey(x[1]))
        NSP12_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP12"]), by = x -> x[2], rev=true)
        NSP12_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP12"]), by = x -> NSPmutPosKey(x[1]))
        NSP13_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP13"]), by = x -> x[2], rev=true)
        NSP13_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP13"]), by = x -> NSPmutPosKey(x[1]))
        NSP14_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP14"]), by = x -> x[2], rev=true)
        NSP14_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP14"]), by = x -> NSPmutPosKey(x[1]))
        NSP15_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP15"]), by = x -> x[2], rev=true)
        NSP15_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP15"]), by = x -> NSPmutPosKey(x[1]))
        NSP16_muts_no_dels_sortByCt = sort(collect(NSP_muts_no_dels["NSP16"]), by = x -> x[2], rev=true)
        NSP16_muts_no_dels_sortByPos = sort(collect(NSP_muts_no_dels["NSP16"]), by = x -> NSPmutPosKey(x[1]))
#        println("typeof NSP1_muts_sortByCt = ", typeof(NSP1_muts_sortByCt))
#        println("typeof NSP1_muts_no_dels_sortByCt", typeof(NSP1_muts_no_dels_sortByCt))
######################################################################################################################################
        NSP_muts_sortByCt_Arr = [NSP1_muts_sortByCt, NSP2_muts_sortByCt, NSP3_muts_sortByCt, NSP4_muts_sortByCt, NSP5_muts_sortByCt, NSP6_muts_sortByCt, NSP7_muts_sortByCt, NSP8_muts_sortByCt, NSP9_muts_sortByCt, NSP10_muts_sortByCt, NSP11_placeholder, NSP12_muts_sortByCt, NSP13_muts_sortByCt, NSP14_muts_sortByCt, NSP15_muts_sortByCt, NSP16_muts_sortByCt]
        NSP_muts_sortByPos_Arr = [NSP1_muts_sortByPos, NSP2_muts_sortByPos, NSP3_muts_sortByPos, NSP4_muts_sortByPos, NSP5_muts_sortByPos, NSP6_muts_sortByPos, NSP7_muts_sortByPos, NSP8_muts_sortByPos, NSP9_muts_sortByPos, NSP10_muts_sortByPos, NSP11_placeholder, NSP12_muts_sortByPos, NSP13_muts_sortByPos, NSP14_muts_sortByPos, NSP15_muts_sortByPos, NSP16_muts_sortByPos]
        NSP_muts_no_dels_sortByCt_Arr = [NSP1_muts_no_dels_sortByCt, NSP2_muts_no_dels_sortByCt, NSP3_muts_no_dels_sortByCt, NSP4_muts_no_dels_sortByCt, NSP5_muts_no_dels_sortByCt, NSP6_muts_no_dels_sortByCt, NSP7_muts_no_dels_sortByCt, NSP8_muts_no_dels_sortByCt, NSP9_muts_no_dels_sortByCt, NSP10_muts_no_dels_sortByCt, NSP11_placeholder, NSP12_muts_no_dels_sortByCt, NSP13_muts_no_dels_sortByCt, NSP14_muts_no_dels_sortByCt, NSP15_muts_no_dels_sortByCt, NSP16_muts_no_dels_sortByCt]
        NSP_muts_no_dels_sortByPos_Arr = [NSP1_muts_no_dels_sortByPos, NSP2_muts_no_dels_sortByPos, NSP3_muts_no_dels_sortByPos, NSP4_muts_no_dels_sortByPos, NSP5_muts_no_dels_sortByPos, NSP6_muts_no_dels_sortByPos, NSP7_muts_no_dels_sortByPos, NSP8_muts_no_dels_sortByPos, NSP9_muts_no_dels_sortByPos, NSP10_muts_no_dels_sortByPos, NSP11_placeholder, NSP12_muts_no_dels_sortByPos, NSP13_muts_no_dels_sortByPos, NSP14_muts_no_dels_sortByPos, NSP15_muts_no_dels_sortByPos, NSP16_muts_no_dels_sortByPos]
        for i in 1:16
            if i ≠ 11
                println(g)
                println(g, "************* NSP$(i) Muts Sorted by Ct *************")
                for w in NSP_muts_sortByCt_Arr[i]
                    println(g, w[1], " = ", w[2])
                end
            end
        end
        for i in 1:16
            if i ≠ 11
                println(g)
                println(g, "************* NSP$(i) Muts Sorted by Pos *************")
                for w in NSP_muts_sortByPos_Arr[i]
                    println(g, w[1], " = ", w[2])
                end
            end
        end      
    end
#############################################################################################################
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_NUC_COUNTS.txt", "w") do g   
        print(g, "\n"^3)
        println(g, "************** TOP NUC MUTATIONS Sorted by Count ***************")
        println(g)
        for w in nuc_muts_ct_sort_by_seq_ct
            if nuc_muts_ct[w[1]] ≥ print_ct_thresh
                nuc_mut_pos = w[1][2:end-1]
                nuc_mut_pos_int = parse(Int, nuc_mut_pos)
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc = w[1][end]
                    qry_nuc_str = string(qry_nuc)
                    ref_seq_nuc = ref_seq[nuc_mut_pos_int]
                    ref_seq_nuc_str = string(ref_seq_nuc)
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, w[1], " = ", w[2], " (", nuc_muts_ct[w[1]], ")") 
                    else
                        println(g, w[1], " = ", w[2], " (", nuc_muts_ct[w[1]], ", REVERSION)") 
                    end
                end
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP NUC MUTATIONS Sorted by Genome Position ***************")
        println(g)
        for w in nuc_muts_ct_sort_by_site
            if nuc_muts_ct[w[1]] ≥ print_ct_thresh
                nuc_mut_pos = w[1][2:end-1]
                nuc_mut_pos_int = parse(Int, nuc_mut_pos)
                if nuc_mut_pos_int ≤ 29850
                    qry_nuc = w[1][end]
                    qry_nuc_str = string(qry_nuc)
                    ref_seq_nuc = ref_seq[nuc_mut_pos_int]
                    ref_seq_nuc_str = string(ref_seq_nuc)
                    if qry_nuc_str ≠ ref_seq_nuc_str
                        println(g, w[1], " = ", w[2], " (", nuc_muts_ct[w[1]], ")") 
                    else
                        println(g, w[1], " = ", w[2], " (", nuc_muts_ct[w[1]], ", REVERSION)") 
                    end
                end
            end
        end
        print(g, "\n"^3)
    end
#############################################################################################################        
#############################################################################################################
    AA_muts_ct_sort_by_site = sort(collect(AA_muts_ct), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_sort_by_seq_ct = sort(collect(AA_muts_ct), by = x -> x[2], rev = true)
    AA_muts_ct_adj_sort = sort(collect(AA_muts_ct_adj), by = x -> AA_sort_key(x[1]))
    AA_muts_ct_adj_sort_by_seq_ct = sort(collect(AA_muts_ct_adj), by = x -> x[2], rev = true)
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_COUNTS.txt", "w") do g               
        print(g, "\n"^3)
        println(g, "************** TOP AA MUTATIONS Sorted by Count ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_sort_by_seq_ct
            if AA_muts_ct[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                     println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP AA MUTATIONS Sorted by Genome Position ***************")
        println(g)
        for w in AA_muts_ct_sort_by_site
             if AA_muts_ct[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut = split(w[1], ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                else
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                end
            end
        end
        print(g, "\n"^3)
        println(g, "**************** ORF1a AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][4] == '1' && w[1][5] == 'a'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF1b AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][4] == '1' && w[1][5] == 'b'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                         println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** Spike AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][1] == 'S'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** E AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][1] == 'E'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** M AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][1] == 'M'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                         println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** N AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][1] == 'N'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF3a AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][2] == 'R' && w[1][4] == '3'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                         println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF6 AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][2] == 'R' && w[1][4] == '6'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                         println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF7a AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][4] == '7' && w[1][5] == 'a'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF7b AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][4] == '7' && w[1][5] == 'b'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF8 AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][2] == 'R' && w[1][4] == '8'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF9b AA Mutations Sorted by Count *****************")
        for w in AA_muts_ct_sort_by_seq_ct
            if w[1][4] == '9' && w[1][5] == 'b'
                if AA_muts_ct[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut = split(w[1], ":")[2]
                    AA_mut_pos = AA_mut[2:end-1]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    qry_AA = AA_mut[end]
                    qry_AA_str = string(qry_AA)
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    ref_seq_AA_str = string(ref_seq_AA)
                    if qry_AA_str ≠ ref_seq_AA_str 
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ")")
                    else
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct[w[1]], ", REVERSION)")
                    end
                end
            end
        end
    end
#############################################################################################################    
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_COUNTS_POS_ONLY.txt", "w") do g               
        print(g, "\n"^3)
        println(g, "************** TOP AA Mutations Pos Only No Dels Sorted by Count ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
            end
        end
        print(g, "\n"^3)
        println(g, "************** TOP AA Mutations Pos Only No Dels Sorted by Genome Position ***************")
        println(g)
        for w in AA_muts_ct_pos_only_no_dels_sort_by_site
             if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                jeen = split(w[1], ":")[1]
                AA_mut_pos = split(w[1], ":")[2]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                left_pad = rpad(string(w[1]), max_l)
                println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
             end
        end
        print(g, "\n"^3)
        println(g, "**************** ORF1a AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 5
                if w[1][4] == '1' && w[1][5] == 'a'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF1b AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 5
                if w[1][4] == '1' && w[1][5] == 'b'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** Spike AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if w[1][1] == 'S'
                if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut_pos = split(w[1], ":")[2]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** E AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if w[1][1] == 'E'
                if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut_pos = split(w[1], ":")[2]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** M AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if w[1][1] == 'M'
                if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut_pos = split(w[1], ":")[2]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** N AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if w[1][1] == 'N'
                if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut_pos = split(w[1], ":")[2]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF3a AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if w[1][2] == 'R' && w[1][4] == '3'
                if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                    jeen = split(w[1], ":")[1]
                    AA_mut_pos = split(w[1], ":")[2]
                    AA_mut_pos_int = parse(Int, AA_mut_pos)
                    left_pad = rpad(string(w[1]), max_l)
                    println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF6 AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 4
                if w[1][2] == 'R' && w[1][4] == '6'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF7a AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 5
            if w[1][4] == '7' && w[1][5] == 'a'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF7b AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 5
                if w[1][4] == '7' && w[1][5] == 'b'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF8 AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 4
                if w[1][2] == 'R' && w[1][4] == '8'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
        print(g, "\n"^2)
        println(g, "***************** ORF9b AA Mutations Pos Only No Dels Sorted by Count *****************")
        for w in AA_muts_ct_pos_only_no_dels_sort_by_seq_ct
            if length(w[1]) ≥ 5
                if w[1][4] == '9' && w[1][5] == 'b'
                    if AA_muts_ct_pos_only_no_dels[w[1]] ≥ print_ct_thresh
                        jeen = split(w[1], ":")[1]
                        AA_mut_pos = split(w[1], ":")[2]
                        AA_mut_pos_int = parse(Int, AA_mut_pos)
                        left_pad = rpad(string(w[1]), max_l)
                        println(g, left_pad, " = ", w[2], " (", AA_muts_ct_pos_only_no_dels[w[1]], ")")
                    end
                end
            end
        end
    end
#######################################################################################################################################                
#######################################################################################################################################
    date = Dates.format(today(), "yyyy-mm-dd")   
    minimum_count_1 = 50
    minimum_score_1 = 10
    min_comb_score = 100
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_CHRONIC_SEARCH_MUTS.txt", "w") do g        
        print(g, "\n"^2)
        max_l = 12
        println(g, "********************** TOP AA MUTS for Chronic Search, CovSpectrum ************************")
        println(g)
        key_pre2(n) = (length(n), split(n, ":")[2][2:end-1])
        Cov_sortkey(n) = (split(n, ":")[1], key_pre2(n))
        CovSpec_muts = Set{String}()
        open("CovSpectrum_Search_Muts.txt", "w") do h
            for w in AA_muts_ct_adj_sort_by_seq_ct
                AAmut = w[1]
                AAct = w[2]
                jeen = split(AAmut, ":")[1]
                AA_mut = split(AAmut, ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                if AA_muts_ct[AAmut] > 5 && AAct > 20 || AA_muts_ct[AAmut] > 5 && AA_muts_ct[AAmut] + AAct > min_comb_score
                    if qry_AA_str ≠ ref_seq_AA_str && AA_mut[end] ≠ '-' && AA_mut[end] ≠ '*'
                        push!(CovSpec_muts, AAmut)
                    end
                end
            end
            CovSpec_arr = collect(CovSpec_muts)
            CovSpec_sort = sort(CovSpec_arr, by = x -> Cov_sortkey(x))
            for mut in CovSpec_sort
                print(h, mut, ", ")
            end
        end
        print(g, "\n"^2)
        println(g, "********************** TOP AA MUTS for Chronic Search ************************")
        println(g)
        for w in AA_muts_ct_adj_sort_by_seq_ct 
            AAmut = w[1]
            AAct = w[2]
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos = AA_mut[2:end-1]
            AA_mut_pos_int = parse(Int, AA_mut_pos)
            qry_AA = AA_mut[end]
            qry_AA_str = string(qry_AA)
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct[AAmut]
            if AA_muts_ct[AAmut] + AAct*10 ≥ min_comb_score || AAct > 20
                if qry_AA_str ≠ ref_seq_AA_str
                    push!(chronic_search_muts, AAmut)
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                else
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)], REVERSION")
                end
            end
        end
    end
#######################################################################################################################################
######################################### Chronic Search Muts Version 2, Improved #####################################################
#######################################################################################################################################
    minimum_count_1 = 2
#    minimum_score_1 = 10
    min_comb_score = 125
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_CHRONIC_SEARCH_MUTS_v2_improved.txt", "w") do g
        println(g, "********************** TOP AA MUTS for Chronic Search: New & Improved Version ************************")
        println(g)
        for AAmut___ratio in AA_muts_ct_no_dels_no_revs_chr_all_ratio_ct_sort 
            AAmut = AAmut___ratio[1]
            ratio = AAmut___ratio[2]
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos_int = parse(Int, AA_mut[2:end-1])
            qry_AA_str = string(AA_mut[end])
            ref_AA_str = string(AA_mut[1])
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct_no_dels_no_revs[AAmut]
            if AA_muts_ct_no_dels_no_revs[AAmut]*ratio ≥ min_comb_score || ratio > 10 && AAmutct ≥ minimum_count_1
                if qry_AA_str ≠ ref_seq_AA_str
                    push!(chronic_search_muts_v2, AAmut)
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(ratio), [$(AAmutct)]")
                else
                    push!(chronic_search_muts_v2, AAmut)
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(ratio), [$(AAmutct)], REVERSION")
                end
            end
        end
    end
#######################################################################################################################################
#######################################################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_ADJ_SORTED_BY_POS.txt", "w") do g        
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS, Adjusted, Sorted by Genome Position ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_adj_sort_by_site
            AAmut = w[1]
            AAct = w[2]
            if AA_muts_ct[AAmut] ≥ print_ct_thresh
                jeen = split(AAmut, ":")[1]
                AA_mut = split(AAmut, ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                AAmut_str = string(AAmut)
                AAmutct = AA_muts_ct[AAmut]
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                else
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)], REVERSION")
                end
            end
        end
        print(g, "\n"^2)
    end  
#############################################################################################################
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_prnt$(print_ct_thresh)_AA_ADJ_SCORE.txt", "w") do g        
        println(g, "************** TOP AA MUTATIONS Sorted by Adjusted SCORE = sqrt(chronic/circulating ratio) + (raw chronic mut ct)/$(adj_score_ct_div) ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_adj_score_sort_by_score
            AAmut = w[1]
            AAct = w[2]
            jeen = split(AAmut, ":")[1]
            AA_mut = split(AAmut, ":")[2]
            AA_mut_pos = AA_mut[2:end-1]
            AA_mut_pos_int = parse(Int, AA_mut_pos)
            qry_AA = AA_mut[end]
            qry_AA_str = string(qry_AA)
            ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
            ref_seq_AA_str = string(ref_seq_AA)
            adj_score = AAct
            raw_ct = AA_muts_ct[AAmut]
            adj_ct = AA_muts_ct_adj[AAmut]
            AAmut_str = string(AAmut)
            if qry_AA_str ≠ ref_seq_AA_str 
                left_pad = rpad(AAmut_str, max_l)
                println(g, "$(left_pad) = $(adj_score) ($(raw_ct), $(adj_ct))")
            else
                left_pad = rpad(AAmut_str, max_l)
                println(g, "$(left_pad) = $(adj_score) ($(raw_ct), $(adj_ct)), REVERSION")
            end
        end
        print(g, "\n"^2)   
        println(g, "************** TOP AA MUTATIONS Adjusted Sorted by Genome Position, w/Adjusted SCORE = sqrt(chronic/circulating ratio) + (raw chronic mut ct)/$(adj_score_ct_div) ***************")
        println(g)  
        for w in AA_muts_ct_adj_score_sort_by_site
            AAmut = w[1]
            AAct = w[2]
            if AA_muts_ct[AAmut] ≥ 2
                jeen = split(AAmut, ":")[1]
                AA_mut = split(AAmut, ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                mutkey = AAmut
                adj_score = AAct
                raw_ct = AA_muts_ct[mutkey]
                adj_ct = AA_muts_ct_adj[mutkey]
                AAmut_str = string(AAmut)
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(adj_score) ($(raw_ct), $(adj_ct))")
                else
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(adj_score) ($(raw_ct), $(adj_ct)), REVERSION")
                end
            end
        end
        print(g, "\n"^2)
        mutkey_1000 = AA_muts_ct_adj_score_sort_by_score[1000][1]
        mut_score_1000 = AA_muts_ct_adj_score_sort_by_score[1000][2]
    
        function nth_ranked_score(n)
            mutkey = AA_muts_ct_adj_score_sort_by_score[n][1]
            println(g)
            mut_score = AA_muts_ct_adj_score_sort_by_score[n][2]
            adj_ct = AA_muts_ct_adj[mutkey][1]
            raw_ct = AA_muts_ct[mutkey]
            println(g, "$(n)th-ranked AA Mut Score, (Adj_ct, Raw Ct); $(mutkey) = $(mut_score), ($(adj_ct), $(raw_ct))")
            println(g)
        end
        nth_ranked_score(2000)
        nth_ranked_score(1000)
        nth_ranked_score(750)
        nth_ranked_score(500)
        nth_ranked_score(400)
        nth_ranked_score(300)
        nth_ranked_score(200)
        nth_ranked_score(100)
        nth_ranked_score(50)
        nth_ranked_score(25)
        nth_ranked_score(10)
        nth_ranked_score(5)
        nth_ranked_score(1)
    end
#############################################################################################################
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_AA_ADJ_SORT_BY_COUNT.txt", "w") do g            
        println(g, "************** TOP AA MUTATIONS Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_adj_sort_by_seq_ct
            AAmut = w[1]
            AAct = w[2]
            if AA_muts_ct[AAmut] ≥ print_ct_thresh
                jeen = split(AAmut, ":")[1]
                AA_mut = split(AAmut, ":")[2]
                AA_mut_pos = AA_mut[2:end-1]
                AA_mut_pos_int = parse(Int, AA_mut_pos)
                qry_AA = AA_mut[end]
                qry_AA_str = string(qry_AA)
                ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                ref_seq_AA_str = string(ref_seq_AA)
                AAmut_str = string(AAmut)
                AAmutct = AA_muts_ct[AAmut]
                if qry_AA_str ≠ ref_seq_AA_str 
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                else
                    left_pad = rpad(AAmut_str, max_l)
                    println(g, "$(left_pad) = $(AAct), [$(AAmutct)], REVERSION")
                end
            end
        end
        print(g, "\n"^2)
        function AA_median_Q1_Q3_etc(key, key_name)
            println(g, "$(key_name) AA Adj Number = ", key)
            println(g, "$(key_name) AA Adj Mutation = ", AA_muts_ct_adj_sort_by_seq_ct[key][2])
            println(g, "$(key_name) AA adj Score = ", AA_muts_ct_adj_sort_by_seq_ct[key][1])
            println(g)
        end
#############################################################################################################
        kz = keys(AA_muts_ct_adj_sort_by_seq_ct)
        kzln = length(kz)
        half_k = round(digits=0, kzln÷2)
        half_ke = string(half_k)
        half_kee = split(half_ke, ".")[1]
        half_key = parse(Int, half_kee)
        d9k = round(digits=0, kzln÷(10)÷1)
        d9ke = string(d9k)
        d9kee = split(d9ke, ".")[1]
        d9key = parse(Int, d9kee)
        Q3mutk = round(digits=0, half_k÷2)
        Q3mutke = string(Q3mutk)
        Q3mutkee = split(Q3mutke, ".")[1]
        Q3mutkey = parse(Int, Q3mutkee)
        Q1mutk = round(digits=0, half_k + kzln÷2)
        Q1mutke = string(Q1mutk)
        Q1mutkee = split(Q1mutke, ".")[1]
        Q1mutkey = parse(Int, Q1mutkee)
        fac = 10/9
        d1k = round(digits=0, kzln÷fac)
        d1ke = string(d1k)
        d1kee = split(d1ke, ".")[1]
        d1key = parse(Int, d1kee)
#############################################################################################################
        AA_median_Q1_Q3_etc(d1key, "d9")
        AA_median_Q1_Q3_etc(Q1mutkey, "Q3")
        AA_median_Q1_Q3_etc(half_key, "Median")
        AA_median_Q1_Q3_etc(Q3mutkey, "Q1")
        AA_median_Q1_Q3_etc(d9key, "d1")
#############################################################################################################    
        function print_AA_adj_ct_sorted(gene_name::String, gene_char_num::Vector{Int}, gene_char::Vector{Char})
            max_l = 12
            println(g)
            println(g, "**************** $(gene_name) AA Mutations Sorted by Adjusted Ct (Ratio of Chronic:Circulating Private Muts) *****************")
            for w in AA_muts_ct_adj_sort_by_seq_ct
                AAmut = w[1]
                AAct = w[2]
                len = length(gene_char_num)
                if all(get(AAmut, gene_char_num[i], ' ') == gene_char[i] for i in 1:len)     
                    AAmutct = AA_muts_ct[AAmut]
                    jeen = split(AAmut, ":")[1]
                    AA_mut_pos_int = AA_pos_int(AAmut) 
                    qry_AA = AAmut[end]
                    ref_seq_AA = gene_AA_dict[jeen][AA_mut_pos_int]
                    AAmut_str = string(AAmut)
                    if AA_muts_ct[AAmut] ≥ print_ct_thresh
                        left_pad = rpad(AAmut_str, max_l)
                        if qry_AA ≠ ref_seq_AA
                            println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
                        else
                            println(g, "$(left_pad) = $(AAct), [$(AAmutct)], REVERSION")
                        end
                    end
                end
            end
        end
        print_AA_adj_ct_sorted("ORF1a", [4, 5], ['1', 'a'])
        print_AA_adj_ct_sorted("ORF1b", [4, 5], ['1', 'b'])
        print_AA_adj_ct_sorted("Spike", [1], ['S'])
        print_AA_adj_ct_sorted("E", [1], ['E'])
        print_AA_adj_ct_sorted("M", [1], ['M'])
        print_AA_adj_ct_sorted("N", [1], ['N'])
        print_AA_adj_ct_sorted("ORF3a", [2, 4], ['R', '3'])
        print_AA_adj_ct_sorted("ORF6", [2, 4], ['R', '6'])
        print_AA_adj_ct_sorted("ORF7a", [4, 5], ['7', 'a'])
        print_AA_adj_ct_sorted("ORF7b", [4, 5], ['7', 'b'])
        print_AA_adj_ct_sorted("ORF8", [2, 4], ['R', '8'])
        print_AA_adj_ct_sorted("ORF9b", [4, 5], ['9', 'b'])
    end
#############################################################################################################    
    gene_protein_length = Dict{String, Int}("NSP1"=>180, "NSP2"=>638, "NSP3"=>1945, "NSP4"=>500, "NSP5"=>306, "NSP6"=>290, "NSP7"=>83, "NSP8"=>198, "NSP9"=>113, "NSP10"=>139, "NSP12"=>932, "NSP13"=>601, "NSP14"=>527, "NSP15"=>346, "NSP16"=>298, "ORF3a"=>275, "ORF6"=>61, "ORF7a"=>121, "ORF7b"=>43, "ORF8"=>121, "ORF9b"=>97, "S"=>1273, "E"=>75, "M"=>222, "N"=>419)
    gene_protein_mut_ct = Dict{String, Int}("NSP1"=>NSP1_mut_ct, "NSP2"=>NSP2_mut_ct, "NSP3"=>NSP3_mut_ct, "NSP4"=>NSP4_mut_ct, "NSP5"=>NSP5_mut_ct, "NSP6"=>NSP6_mut_ct, "NSP7"=>NSP7_mut_ct, "NSP8"=>NSP8_mut_ct, "NSP9"=>NSP9_mut_ct, "NSP10"=>NSP10_mut_ct, "NSP12"=>NSP12_mut_ct, "NSP13"=>NSP13_mut_ct, "NSP14"=>NSP14_mut_ct, "NSP15"=>NSP15_mut_ct, "NSP16"=>NSP16_mut_ct, "ORF3a"=>ORF3a_mut_ct, "ORF6"=>ORF6_mut_ct, "ORF7a"=>ORF7a_mut_ct, "ORF7b"=>ORF7b_mut_ct, "ORF8"=>ORF8_mut_ct, "ORF9b"=>ORF9b_mut_ct, "S"=>S_mut_ct, "E"=>E_mut_ct, "M"=>M_mut_ct, "N"=>N_mut_ct)
    gene_protein_order = Dict{String, Int}("NSP1"=>1, "NSP2"=>2, "NSP3"=>3, "NSP4"=>4, "NSP5"=>5, "NSP6"=>6, "NSP7"=>7, "NSP8"=>8, "NSP9"=>9, "NSP10"=>10, "NSP12"=>12, "NSP13"=>13, "NSP14"=>14, "NSP15"=>15, "NSP16"=>16, "ORF3a"=>17, "ORF6"=>18, "ORF7a"=>19, "ORF7b"=>20, "ORF8"=>21, "ORF9b"=>22, "S"=>23, "E"=>24, "M"=>25, "N"=>26)
    gene_set = Set(["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"])

    domain_length = Dict{String, Int}("NSP3_Ubl1"=>111, "NSP3_HVR"=>95, "NSP3_Mac1"=>168, "NSP3_Mac2"=>176, "NSP3_Mac3"=>123, "NSP3_DPUP"=>70, "NSP3_Ubl2"=>60, "NSP3_PLpro"=>259, "NSP3_NAB"=>151, "NSP3_BSM"=>131, "NSP3_TM1"=>106, "NSP3_Ecto3"=>59, "NSP3_TM234HLX"=>87, "NSP3_Y1"=>179, "NSP3_CoVY"=>181, "NSP4_TM1"=>31, "NSP4_Ecto4"=>240, "NSP4_TM2_TM6"=>139, "NSP4_CTD"=>90, "NSP12_NiRAN"=>250, "NSP12_intrfce"=>148, "NSP12_fingers"=>183, "NSP12_palm"=>171, "NSP12_palmLnk"=>60, "NSP12_thumb"=>120, "S_S1"=>669, "S_S2"=>583, "S_NTD"=>305, "S_RBD"=>197, "S_FCS_region"=>21, "N_N1"=>43, "N_N2"=>131, "N_N3"=>71, "N_N4"=>118, "N_N5"=>55, "N_SR"=>31, "N_L_helix"=>21, "N_CBP"=>33, "N_9b_overlap"=>98)
    domain_mut_ct = Dict{String, Int}("NSP3_Ubl1"=>NSP3_Ubl1_ct, "NSP3_HVR"=>NSP3_HVR_ct, "NSP3_Mac1"=>NSP3_Mac1_ct, "NSP3_Mac2"=>NSP3_Mac2_ct, "NSP3_Mac3"=>NSP3_Mac3_ct, "NSP3_DPUP"=>NSP3_DPUP_ct, "NSP3_Ubl2"=>NSP3_Ubl2_ct, "NSP3_PLpro"=>NSP3_PLpro_ct, "NSP3_NAB"=>NSP3_NAB_ct, "NSP3_BSM"=>NSP3_BSM_ct, "NSP3_TM1"=>NSP3_TM1_ct, "NSP3_Ecto3"=>NSP3_Ecto3_ct, "NSP3_TM234HLX"=>NSP3_TM234HLX_ct, "NSP3_Y1"=>NSP3_Y1_ct, "NSP3_CoVY"=>NSP3_CoVY_ct, "NSP4_TM1"=>NSP4_TM1_ct, "NSP4_Ecto4"=>NSP4_Ecto4_ct, "NSP4_TM2_TM6"=>NSP4_TM2_TM6_ct, "NSP4_CTD"=>NSP4_CTD_ct, "NSP12_NiRAN"=>NSP12_NiRAN_ct, "NSP12_intrfce"=>NSP12_intrfce_ct, "NSP12_fingers"=>NSP12_fingers_ct, "NSP12_palm"=>NSP12_palm_ct, "NSP12_palmLnk"=>NSP12_palmLnk_ct, "NSP12_thumb"=>NSP12_thumb_ct, "S_S1"=>S_S1_ct, "S_S2"=>S_S2_ct, "S_NTD"=>S_NTD_ct, "S_RBD"=>S_RBD_ct, "S_FCS_region"=>S_FCS_region_ct, "N_N1"=>N_N1_ct, "N_N2"=>N_N2_ct, "N_N3"=>N_N3_ct, "N_N4"=>N_N4_ct, "N_N5"=>N_N5_ct, "N_SR"=>N_SR_ct, "N_L_helix"=>N_L_helix_ct, "N_CBP"=>N_CBP_ct, "N_9b_overlap"=>N_9b_overlap_ct)
    domain_order = Dict{String, Int}("NSP3_Ubl1"=>1, "NSP3_HVR"=>2, "NSP3_Mac1"=>3, "NSP3_Mac2"=>4, "NSP3_Mac3"=>5, "NSP3_DPUP"=>6, "NSP3_Ubl2"=>7, "NSP3_PLpro"=>8, "NSP3_NAB"=>9, "NSP3_BSM"=>10, "NSP3_TM1"=>11, "NSP3_Ecto3"=>12, "NSP3_TM234HLX"=>13, "NSP3_Y1"=>14, "NSP3_CoVY"=>15, "NSP4_TM1"=>16, "NSP4_Ecto4"=>17, "NSP4_TM2_TM6"=>18, "NSP4_CTD"=>19, "NSP12_NiRAN"=>20, "NSP12_intrfce"=>21, "NSP12_fingers"=>22, "NSP12_palm"=>23, "NSP12_palmLnk"=>24, "NSP12_thumb"=>25, "S_S1"=>26, "S_S2"=>27, "S_NTD"=>28, "S_RBD"=>29, "S_FCS_region"=>30, "N_N1"=>31, "N_N2"=>32, "N_N3"=>33, "N_N4"=>34, "N_N5"=>35, "N_SR"=>36, "N_L_helix"=>37, "N_CBP"=>38, "N_9b_overlap"=>39)
    domain_set = Set{String}(["NSP3_Ubl1", "NSP3_HVR", "NSP3_Mac1", "NSP3_Mac2", "NSP3_Mac3", "NSP3_DPUP", "NSP3_Ubl2", "NSP3_PLpro", "NSP3_NAB", "NSP3_BSM", "NSP3_TM1", "NSP3_Ecto3", "NSP3_TM234HLX", "NSP3_Y1", "NSP3_CoVY", "NSP4_TM1", "NSP4_Ecto4", "NSP4_TM2_TM6", "NSP4_CTD", "NSP12_NiRAN", "NSP12_intrfce", "NSP12_fingers", "NSP12_palm", "NSP12_palmLnk", "NSP12_thumb", "S_S1", "S_S2", "S_NTD", "S_RBD", "S_FCS_region", "N_N1", "N_N2", "N_N3", "N_N4", "N_N5", "N_SR", "N_L_helix", "N_CBP", "N_9b_overlap"])
    domain_residues_NSP = Dict{String, String}("NSP3_Ubl1"=>"NSP3:1-111", "NSP3_HVR"=>"NSP3:112-206", "NSP3_Mac1"=>"NSP3:207-374", "NSP3_Mac2"=>"NSP3:375-551", "NSP3_Mac3"=>"NSP3:552-673", "NSP3_DPUP"=>"NSP3:674-743", "NSP3_Ubl2"=>"NSP3:744-803", "NSP3_PLpro"=>"NSP3:804-1052", "NSP3_NAB"=>"NSP3:1053-1203", "NSP3_BSM"=>"NSP3:1204-1334", "NSP3_TM1"=>"NSP3:1335-1440", "NSP3_Ecto3"=>"NSP3:1441-1499", "NSP3_TM23_HLX_TM4"=>"NSP3:1500-1586", "NSP3_Y1"=>"NSP3:1587-1764", "NSP3_CoVY"=>"NSP3:1765-1945", "NSP4_TM1"=>"NSP4:1-31", "NSP4_Ecto4"=>"NSP4:32-271", "NSP4_TM2_TM6"=>"NSP4:272-410", "NSP4_CTD"=>"NSP4:411-500", "NSP12_NiRAN"=>"NSP12:1-250", "NSP12_intrfce"=>"NSP12:251-398", "NSP12_fingers"=>"NSP12:399-581", "NSP12_palm"=>"NSP12:582-627", "NSP12_palmLnk"=>"NSP12:628-687", "NSP12_palm2"=>"NSP12:688-812", "NSP12_thumb"=>"NSP12:813-932", "S_S1"=>"S:1-669", "S_S2"=>"S:691-1273", "S_NTD"=>"S:2-306", "S_RBD"=>"S:331-527", "S_FCS_region"=>"S:670-690", "N_N1"=>"N:2-44", "N_N2"=>"N:45-175", "N_N3"=>"N:176-246", "N_N4"=>"N:247-364", "N_N5"=>"N:365-419", "N_SR"=>"N:176-206", "N_L_helix"=>"N:215-235", "N_CBP"=>"N:247-279", "N_9b_overlap"=>"N:4-101")
    
#    domain_mut_density = Dict{String, Float64}()
#    gene_mut_density = Dict{String, Float64}()

    for geen in gene_set
        gene_mut_density[geen] = round(digits=2, gene_protein_mut_ct[geen]/gene_protein_length[geen])
    end
    for dom in domain_set
        domain_mut_density[dom] = round(digits=2, domain_mut_ct[dom]/domain_length[dom])
    end
    gene_sort(n) = gene_protein_order[n]
    gene_mut_density_sort_by_gene = sort(collect(gene_mut_density), by = x -> gene_sort(x[1]))
    gene_mut_density_sort_by_density = sort(collect(gene_mut_density), by = x -> x[2], rev=true)
    protein_dom_sort(n) = domain_order[n]
    domain_mut_density_sort_by_gene = sort(collect(domain_mut_density), by = x -> protein_dom_sort(x[1]))
    domain_mut_density_sort_by_density = sort(collect(domain_mut_density), by = x -> x[2], rev=true)

#############################################################################################################
    date = Dates.format(today(), "yyyy-mm-dd")
    function density_print(d)
        ds = string(d)
        ds_dec = split(ds, ".")[2]
        if length(ds_dec) < 2
            ds = ds*"0"
        end
        return ds
    end
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_GENE_MUT_DENSITY.txt", "w") do g
        print(g, "\n"^2)
        println(g, "*************** Protein Mut Density, Sorted by Gene ***************")
        for (prot, density) in gene_mut_density_sort_by_gene
            println(g, rpad("$(prot) = ", 8), lpad("$(density_print(density))", 5) )
        end
        println(g)
        println(g, "*************** Protein Mut Density, Sorted by Mut Density ***************")
        for (prot, density) in gene_mut_density_sort_by_density
            println(g, rpad("$(prot) = ", 8), lpad("$(density_print(density))", 5) )
        end
        print(g, "\n"^2)
        println(g, "*************** Protein Domain Mut Density, Sorted by Gene, Domain ***************")
        for (domain, density) in domain_mut_density_sort_by_gene
            println(g, rpad("$(domain) = ", 20), lpad("$(density_print(density))", 5) )
        end
        println(g)
        println(g, "*************** Protein Domain Mut Density, Sorted by Mut Density ***************")
        for (domain, density) in domain_mut_density_sort_by_density
            println(g, rpad("$(domain) = ", 20), lpad("$(density_print(density))", 5) )
        end
        print(g, "\n"^2)
    end
#############################################################################################################
    gene_protein_mut_ct_MINADJ = Dict{String, Int}("NSP1"=>NSP1_mut_ct_MINADJ, "NSP2"=>NSP2_mut_ct_MINADJ, "NSP3"=>NSP3_mut_ct_MINADJ, "NSP4"=>NSP4_mut_ct_MINADJ, "NSP5"=>NSP5_mut_ct_MINADJ, "NSP6"=>NSP6_mut_ct_MINADJ, "NSP7"=>NSP7_mut_ct_MINADJ, "NSP8"=>NSP8_mut_ct_MINADJ, "NSP9"=>NSP9_mut_ct_MINADJ, "NSP10"=>NSP10_mut_ct_MINADJ, "NSP12"=>NSP12_mut_ct_MINADJ, "NSP13"=>NSP13_mut_ct_MINADJ, "NSP14"=>NSP14_mut_ct_MINADJ, "NSP15"=>NSP15_mut_ct, "NSP16"=>NSP16_mut_ct_MINADJ, "ORF3a"=>ORF3a_mut_ct_MINADJ, "ORF6"=>ORF6_mut_ct_MINADJ, "ORF7a"=>ORF7a_mut_ct_MINADJ, "ORF7b"=>ORF7b_mut_ct_MINADJ, "ORF8"=>ORF8_mut_ct_MINADJ, "ORF9b"=>ORF9b_mut_ct_MINADJ, "S"=>S_mut_ct_MINADJ, "E"=>E_mut_ct_MINADJ, "M"=>M_mut_ct_MINADJ, "N"=>N_mut_ct_MINADJ)
    gene_mut_density_MINADJ = Dict{String, Float64}()
    domain_mut_ct_MINADJ = Dict{String, Int}("NSP3_Ubl1"=>NSP3_Ubl1_ct_MINADJ, "NSP3_HVR"=>NSP3_HVR_ct_MINADJ, "NSP3_Mac1"=>NSP3_Mac1_ct_MINADJ, "NSP3_Mac2"=>NSP3_Mac2_ct_MINADJ, "NSP3_Mac3"=>NSP3_Mac3_ct_MINADJ, "NSP3_DPUP"=>NSP3_DPUP_ct_MINADJ, "NSP3_Ubl2"=>NSP3_Ubl2_ct_MINADJ, "NSP3_PLpro"=>NSP3_PLpro_ct_MINADJ, "NSP3_NAB"=>NSP3_NAB_ct_MINADJ, "NSP3_BSM"=>NSP3_BSM_ct_MINADJ, "NSP3_TM1"=>NSP3_TM1_ct_MINADJ, "NSP3_Ecto3"=>NSP3_Ecto3_ct_MINADJ, "NSP3_TM234HLX"=>NSP3_TM234HLX_ct_MINADJ, "NSP3_Y1"=>NSP3_Y1_ct_MINADJ, "NSP3_CoVY"=>NSP3_CoVY_ct_MINADJ, "NSP4_TM1"=>NSP4_TM1_ct_MINADJ, "NSP4_Ecto4"=>NSP4_Ecto4_ct_MINADJ, "NSP4_TM2_TM6"=>NSP4_TM2_TM6_ct_MINADJ, "NSP4_CTD"=>NSP4_CTD_ct_MINADJ, "NSP12_NiRAN"=>NSP12_NiRAN_ct_MINADJ, "NSP12_intrfce"=>NSP12_intrfce_ct_MINADJ, "NSP12_fingers"=>NSP12_fingers_ct_MINADJ, "NSP12_palm"=>NSP12_palm_ct_MINADJ, "NSP12_palmLnk"=>NSP12_palmLnk_ct_MINADJ, "NSP12_thumb"=>NSP12_thumb_ct_MINADJ, "S_S1"=>S_S1_ct_MINADJ, "S_S2"=>S_S2_ct_MINADJ, "S_NTD"=>S_NTD_ct_MINADJ, "S_RBD"=>S_RBD_ct_MINADJ, "S_FCS_region"=>S_FCS_region_ct_MINADJ, "N_N1"=>N_N1_ct_MINADJ, "N_N2"=>N_N2_ct_MINADJ, "N_N3"=>N_N3_ct_MINADJ, "N_N4"=>N_N4_ct_MINADJ, "N_N5"=>N_N5_ct_MINADJ, "N_SR"=>N_SR_ct_MINADJ, "N_L_helix"=>N_L_helix_ct_MINADJ, "N_CBP"=>N_CBP_ct_MINADJ, "N_9b_overlap"=>N_9b_overlap_ct_MINADJ)
    domain_mut_density_MINADJ = Dict{String, Float64}()
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_GENE_MUT_DENSITY_MINADJ.txt", "w") do g   
        print(g, "\n"^2)
        for geen in gene_set
            gene_mut_density_MINADJ[geen] = round(digits=2, gene_protein_mut_ct_MINADJ[geen]/gene_protein_length[geen])
        end
        gene_sort(n) = gene_protein_order[n]
        gene_mut_density_sort_by_gene_MINADJ = sort(collect(gene_mut_density_MINADJ), by = x -> gene_sort(x[1]))
        gene_mut_density_sort_by_density_MINADJ = sort(collect(gene_mut_density_MINADJ), by = x -> x[2], rev=true)
        for dom in domain_set
            domain_mut_density_MINADJ[dom] = round(digits=2, domain_mut_ct_MINADJ[dom]/domain_length[dom])
        end    
        protein_dom_sort(n) = domain_order[n]
        domain_mut_density_sort_by_gene_MINADJ = sort(collect(domain_mut_density_MINADJ), by = x -> protein_dom_sort(x[1]))
        domain_mut_density_sort_by_density_MINADJ = sort(collect(domain_mut_density_MINADJ), by = x -> x[2], rev=true)
        println(g, "*************** Protein Mut Density MINADJ, Sorted by Gene ***************")
        for (prot, density) in gene_mut_density_sort_by_gene_MINADJ
            println(g, "$(prot) = $(density)")
        end
        println(g)
        println(g, "*************** Protein Mut Density MINADJ, Sorted by Mut Density ***************")
        for (prot, density) in gene_mut_density_sort_by_density_MINADJ
            println(g, "$(prot) = $(density_print(density))")
        end
        print(g, "\n"^2)
        println(g, "*************** Protein Mut Density MINADJ, Sorted by Gene ***************")
        for (prot, density) in gene_mut_density_sort_by_gene_MINADJ
            println(g, "$(prot) = $(density_print(density))")
        end
        println(g)
        println(g, "*************** Protein Mut Density MINADJ, Sorted by Mut Density ***************")
        for (prot, density) in gene_mut_density_sort_by_density_MINADJ
            println(g, "$(prot) = $(density_print(density))")
        end
        print(g, "\n"^2)
        println(g, "*************** Protein Domain Mut Density MINADJ, Sorted by Gene, Domain ***************")
        for (prot_domain, density) in domain_mut_density_sort_by_gene_MINADJ
            println(g, "$(prot_domain) = $(density_print(density))")
        end
        println(g)
        println(g, "*************** Protein Domain Mut Density MINADJ, Sorted by Mut Density ***************")
        for (prot_domain, density) in domain_mut_density_sort_by_density_MINADJ
            println(g, "$(prot_domain) = $(density_print(density))")
        end
        print(g, "\n"^2)
    end
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_MISCELLANEOUS.txt", "w") do g        
        println(g, "Number of Sequences in rep_seqs list: ", length(rep_seqs))
        println(g, "Number of keys in rep_seq_grps_seqs: ", length(keys(rep_seq_grps_seqs)))
        println(g)
        related_grps = []
        for (k, v) in rep_seq_grps_seqs
            z = []
            p = sort(collect(v))
            q = join(p, ", ")
            push!(z, k)
            push!(z, q)
            push!(related_grps, z)
        end
        println(g)
        print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g, "T241C")
        print_seqs_with_nuc_mut_plus_their_other_nuc_muts(g, "A28877T")
        print_seqs_with_AA_mut_plus_their_other_AA_muts(g, "ORF7a:R39I")
        print_seqs_with_AA_mut_plus_their_other_AA_muts(g, "S:A372T")
        AA_mutations_in_seq(g, "EPI_ISL_18337738")
        nuc_muts_in_seq(g, "EPI_ISL_18337738")
    end
#############################################################################################################    
    AA_muts_ct_pos_only_adj_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_adj_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_adj), by = x -> x[2], rev = true)

    AA_muts_ct_pos_only_adj_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_no_dels), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct = sort(collect(AA_muts_ct_pos_only_adj_no_dels), by = x -> x[2], rev = true)
    
    AA_muts_ct_pos_only_adj_score_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_score), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_adj_score_sort_by_score = sort(collect(AA_muts_ct_pos_only_adj_score), by = x -> x[2], rev = true)
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site = sort(collect(AA_muts_ct_pos_only_adj_score_no_dels), by = x -> AA_sort_key_pos_only(x[1]))
    AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score = sort(collect(AA_muts_ct_pos_only_adj_score_no_dels), by = x -> x[2], rev = true)
#############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_POS_ONLY_ADJ_COUNT.txt", "w") do g       
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS *POS ONLY* Sorted by Count ***************")
        println(g)
        max_l = 12
        for w in AA_muts_ct_pos_only_adj_sort_by_seq_ct
            AAmut = w[1]
            AAct = w[2]
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                left_pad = rpad(string(AAmut), max_l)
                println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
            end
        end
        print(g, "\n"^2)
        println(g, "************** TOP AA MUTATIONS ADJ *POS ONLY* Sorted by Genome Position ***************")
        println(g)
        for w in AA_muts_ct_pos_only_adj_sort_by_site
            AAmut = w[1]
            AAct = w[2]
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                left_pad = rpad(AAmut_str, max_l)
                println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
            end
        end
        print_pos_only_adj(g, "ORF1a", [4, 5], ['1', 'a'])
        print_pos_only_adj(g, "ORF1b", [4, 5], ['1', 'b'])
        print_pos_only_adj(g, "Spike", [1], ['S'])
        print_pos_only_adj(g, "E", [1], ['E'])
        print_pos_only_adj(g, "M", [1], ['M'])
        print_pos_only_adj(g, "N", [1], ['N'])
        print_pos_only_adj(g, "ORF3a", [2, 4], ['R', '3'])
        print_pos_only_adj(g, "ORF6", [2, 4], ['R', '6'])
        print_pos_only_adj(g, "ORF7a", [4, 5], ['7', 'a'])
        print_pos_only_adj(g, "ORF7b", [4, 5], ['7', 'b'])
        print_pos_only_adj(g, "ORF8", [2, 4], ['R', '8'])
        print_pos_only_adj(g, "ORF9b", [4, 5], ['9', 'b'])
    end
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
    for (date, nuc_mut_ct_dict) in date_nuc_mut_ct
        for (mut, ct) in nuc_mut_ct_dict
            if !haskey(date_nuc_mut_ct_no_dels, date)
                date_nuc_mut_ct_no_dels[date] = Dict{String, Int}()
            end
            if mut[end] ≠ '-'
                date_nuc_mut_ct_no_dels[date][mut] = ct
            end
        end
    end
    for (date, AA_mut_ct_dict) in date_AA_mut_ct
        for (mut, ct) in AA_mut_ct_dict
            AA_pos = AA_mut_to_AA_pos(mut)
            if mut[end] ≠ '-'
                if !haskey(date_AA_mut_ct_no_dels, date)
                    date_AA_mut_ct_no_dels[date] = Dict{String, Int}()
                end
                date_AA_mut_ct_no_dels[date][mut] = ct
                if !haskey(date_AA_mut_ct_pos_only_no_dels, date)
                    date_AA_mut_ct_pos_only_no_dels[date] = Dict{String, Int}()
                end
####################################################
                if haskey(date_AA_mut_ct_pos_only_no_dels[date], AA_pos)
                    date_AA_mut_ct_pos_only_no_dels[date][AA_pos] += ct
                else
                    date_AA_mut_ct_pos_only_no_dels[date][AA_pos] = ct
                end
####################################################
            end
        end
    end
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
    open("$(date)_$(ndjson_name)_rep$(rep_thresh)_rev$(revs_thresh)_minmut$(mut_thresh)_adjDiv$(adj_score_ct_div)_prnt$(print_ct_thresh)_POS_ONLY_ADJ_RATIO.txt", "w") do g       
        print(g, "\n"^2)
        println(g, "******************************************************************************************************************")  
        println(g, "* AA Mutations Ratio (Proportion of Chronic/Proportion of Circulating Muts) *POS ONLY* Sorted by Genome Position *")
        println(g, "******************************************************************************************************************")
        println(g)
        for w in AA_muts_ct_pos_only_adj_score_sort_by_site
            AAmut = w[1]
            AAct = w[2]
            AAmut_str = string(AAmut)
            AAmutct = AA_muts_ct_pos_only[AAmut]
            if AA_muts_ct_pos_only[AAmut] ≥ print_ct_thresh
                left_pad = rpad(AAmut_str, max_l)
                println(g, "$(left_pad) = $(AAct), [$(AAmutct)]")
            end
        end
        print_pos_only_adj_score(g, "ORF1a", [4, 5], ['1', 'a'])
        print_pos_only_adj_score(g, "ORF1b", [4, 5], ['1', 'b'])
        print_pos_only_adj_score(g, "Spike", [1], ['S'])
        print_pos_only_adj_score(g, "E", [1], ['E'])
        print_pos_only_adj_score(g, "M", [1], ['M'])
        print_pos_only_adj_score(g, "N", [1], ['N'])
        print_pos_only_adj_score(g, "ORF3a", [2, 4], ['R', '3'])
        print_pos_only_adj_score(g, "ORF6", [2, 4], ['R', '6'])
        print_pos_only_adj_score(g, "ORF7a", [4, 5], ['7', 'a'])
        print_pos_only_adj_score(g, "ORF7b", [4, 5], ['7', 'b'])
        print_pos_only_adj_score(g, "ORF8", [2, 4], ['R', '8'])
        print_pos_only_adj_score(g, "ORF9b", [4, 5], ['9', 'b'])
    end
########################################################################################################################################
########################################################################################################################################   
########################################################################################################################################
########################################################################################################################################    
    save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs.jld2", "AA_muts_ct_no_dels_no_revs", AA_muts_ct_no_dels_no_revs)
    save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_no_dels_no_revs_chr_all_ratio", AA_muts_ct_no_dels_no_revs_chr_all_ratio)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio", AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)
########################################################################################################################################
    save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_chr_all_ratio.jld2", "AA_muts_ct_no_dels_chr_all_ratio", AA_muts_ct_no_dels_chr_all_ratio)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_chr_all_ratio", AA_muts_ct_pos_only_no_dels_chr_all_ratio)
########################################################################################################################################
    save("$(date)_$(ndjson_name)_gene_mut_density.jld2", "gene_mut_density", gene_mut_density)
    save("$(date)_$(ndjson_name)_domain_mut_density.jld2", "domain_mut_density", domain_mut_density)
########################################################################################################################################    
    save("$(date)_$(ndjson_name)_date_nuc_mut_ct.jld2", "date_nuc_mut_ct", date_nuc_mut_ct)
    save("$(date)_$(ndjson_name)_date_nuc_mut_ct_no_dels.jld2", "date_nuc_mut_ct_no_dels", date_nuc_mut_ct_no_dels)
    save("$(date)_$(ndjson_name)_date_AA_mut_ct.jld2", "date_AA_mut_ct", date_AA_mut_ct)
    save("$(date)_$(ndjson_name)_date_AA_mut_ct_no_dels.jld2", "date_AA_mut_ct_no_dels", date_AA_mut_ct_no_dels)
    save("$(date)_$(ndjson_name)_date_AA_mut_ct_pos_only_no_dels.jld2", "date_AA_mut_ct_pos_only_no_dels", date_AA_mut_ct_pos_only_no_dels)

    save("$(date)_$(ndjson_name)_seq_ct_by_year_month_day.jld2", "seq_ct_by_year_month_day", seq_ct_by_year_month_day)
    save("$(date)_$(ndjson_name)_seq_ct_by_year_month.jld2", "seq_ct_by_year_month", seq_ct_by_year_month)
    save("$(date)_$(ndjson_name)_seq_ct_by_year.jld2", "seq_ct_by_year", seq_ct_by_year)
########################################################################################################################################    
    save("$(date)_$(ndjson_name)_nuc_muts_seq.jld2", "nuc_muts_seq", nuc_muts_seq)
    save("$(date)_$(ndjson_name)_nuc_dels_seq.jld2", "nuc_dels_seq", nuc_dels_seq)
    
    save("$(date)_$(ndjson_name)_AA_muts_seq.jld2", "AA_muts_seq", AA_muts_seq)
    save("$(date)_$(ndjson_name)_AA_muts_seq_pos_only.jld2", "AA_muts_seq_pos_only", AA_muts_seq_pos_only)
    save("$(date)_$(ndjson_name)_AA_dels_seq.jld2", "AA_dels_seq", AA_dels_seq)
    
    save("$(date)_$(ndjson_name)_seq_nuc_muts.jld2", "seq_nuc_muts", seq_nuc_muts)
    save("$(date)_$(ndjson_name)_seq_nuc_dels.jld2", "seq_nuc_dels", seq_nuc_dels)
    
    save("$(date)_$(ndjson_name)_seq_AA_muts.jld2", "seq_AA_muts", seq_AA_muts)
    save("$(date)_$(ndjson_name)_seq_AA_muts_pos_only.jld2", "seq_AA_muts_pos_only", seq_AA_muts_pos_only)
    save("$(date)_$(ndjson_name)_seq_AA_dels.jld2", "seq_AA_dels", seq_AA_dels)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_seq_nuc_muts_WT.jld2", "seq_nuc_muts_WT", seq_nuc_muts_WT)
    save("$(date)_$(ndjson_name)_seq_nuc_dels_WT.jld2", "seq_nuc_dels_WT", seq_nuc_dels_WT)
    
    save("$(date)_$(ndjson_name)_seq_AA_muts_WT.jld2", "seq_AA_muts_WT", seq_AA_muts_WT)
    save("$(date)_$(ndjson_name)_seq_AA_muts_WT_pos_only.jld2", "seq_AA_muts_WT_pos_only", seq_AA_muts_WT_pos_only)
    save("$(date)_$(ndjson_name)_seq_AA_dels_WT.jld2", "seq_AA_dels_WT", seq_AA_dels_WT)
    
    save("$(date)_$(ndjson_name)_nuc_muts_seq_WT.jld2", "nuc_muts_seq_WT", nuc_muts_seq_WT)
    save("$(date)_$(ndjson_name)_nuc_dels_seq_WT.jld2", "nuc_dels_seq_WT", nuc_dels_seq_WT)
    
    save("$(date)_$(ndjson_name)_AA_muts_seq_WT.jld2", "AA_muts_seq_WT", AA_muts_seq_WT)
    save("$(date)_$(ndjson_name)_AA_muts_seq_WT_pos_only.jld2", "AA_muts_seq_WT_pos_only", AA_muts_seq_WT_pos_only)
    save("$(date)_$(ndjson_name)_AA_dels_seq_WT.jld2", "AA_dels_seq_WT", AA_dels_seq_WT)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_seq_country.jld2", "seq_country", seq_country)
    save("$(date)_$(ndjson_name)_seq_US_state.jld2", "seq_US_state", seq_US_state)
    save("$(date)_$(ndjson_name)_seq_collection_date.jld2", "seq_collection_date", seq_collection_date)
    save("$(date)_$(ndjson_name)_seq_date_index.jld2", "seq_date_index", seq_date_index)
    save("$(date)_$(ndjson_name)_seq_date_tuple.jld2", "seq_date_tuple", seq_date_tuple)
    save("$(date)_$(ndjson_name)_seq_lab_dict.jld2", "seq_lab_dict", seq_lab_dict)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_seq_unknown_AA.jld2", "seq_unknown_AA", seq_unknown_AA)
    save("$(date)_$(ndjson_name)_seq_nuc_dropout.jld2", "seq_nuc_dropout", seq_nuc_dropout)
    save("$(date)_$(ndjson_name)_seq_mixed_nucs.jld2", "seq_mixed_nucs", seq_mixed_nucs)
########################################################################################################################################
######################################################################################################################################## 
    save("$(date)_$(ndjson_name)_NSP_muts.jld2", "NSP_muts", NSP_muts)
    save("$(date)_$(ndjson_name)_NSP_muts_no_dels.jld2", "NSP_muts_no_dels", NSP_muts_no_dels)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_seq_clade.jld2", "seq_clade", seq_clade)
    save("$(date)_$(ndjson_name)_seq_clade_display.jld2", "seq_clade_display", seq_clade_display)
    save("$(date)_$(ndjson_name)_seq_pango.jld2", "seq_pango", seq_pango)
    save("$(date)_$(ndjson_name)_seq_pango_unaliased.jld2", "seq_pango_unaliased", seq_pango_unaliased)

    save("$(date)_$(ndjson_name)_rep_seq_grps_clade.jld2", "rep_seq_grps_clade", rep_seq_grps_clade)

    save("$(date)_$(ndjson_name)_seq_clade_ct.jld2", "seq_clade_ct", seq_clade_ct)
    save("$(date)_$(ndjson_name)_seq_pango_ct.jld2", "seq_pango_ct", seq_pango_ct)
    save("$(date)_$(ndjson_name)_seq_pango_unaliased_ct.jld2", "seq_pango_unaliased_ct", seq_pango_unaliased_ct)  
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_rep_seq_grps_seqs.jld2", "rep_seq_grps_seqs", rep_seq_grps_seqs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_muts.jld2", "rep_seq_grps_muts", rep_seq_grps_muts)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA.jld2", "rep_seq_grps_AA", rep_seq_grps_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_pos_only.jld2", "rep_seq_grps_AA_pos_only", rep_seq_grps_AA_pos_only)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_pos_only_no_dels.jld2", "rep_seq_grps_AA_pos_only_no_dels", rep_seq_grps_AA_pos_only_no_dels)
    save("$(date)_$(ndjson_name)_non_rep_seqs_AA.jld2", "non_rep_seqs_AA", non_rep_seqs_AA)
    save("$(date)_$(ndjson_name)_non_rep_seqs_AA_pos_only.jld2", "non_rep_seqs_AA_pos_only", non_rep_seqs_AA_pos_only)
    save("$(date)_$(ndjson_name)_non_rep_seqs_AA_pos_only_no_dels.jld2", "non_rep_seqs_AA_pos_only_no_dels", non_rep_seqs_AA_pos_only_no_dels)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_nuc_dels_ct.jld2", "nuc_dels_ct", nuc_dels_ct)
    
    save("$(date)_$(ndjson_name)_nuc_muts_ct.jld2", "nuc_muts_ct", nuc_muts_ct)
    save("$(date)_$(ndjson_name)_nuc_muts_ct_no_dels.jld2", "nuc_muts_ct_no_dels", nuc_muts_ct_no_dels)
    
    save("$(date)_$(ndjson_name)_nuc_muts_ct_adj.jld2", "nuc_muts_ct_adj", nuc_muts_ct_adj)
    save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_no_dels.jld2", "nuc_muts_ct_adj_no_dels", nuc_muts_ct_adj_no_dels)
    save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score.jld2", "nuc_muts_ct_adj_score", nuc_muts_ct_adj_score)
    save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score_no_dels.jld2", "nuc_muts_ct_adj_score_no_dels", nuc_muts_ct_adj_score_no_dels)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_AA_dels_ct.jld2", "AA_dels_ct", AA_dels_ct)
    
    save("$(date)_$(ndjson_name)_AA_muts_ct.jld2", "AA_muts_ct", AA_muts_ct)
    save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels.jld2", "AA_muts_ct_no_dels", AA_muts_ct_no_dels)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only.jld2", "AA_muts_ct_pos_only", AA_muts_ct_pos_only)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels.jld2", "AA_muts_ct_pos_only_no_dels", AA_muts_ct_pos_only_no_dels)
    
    save("$(date)_$(ndjson_name)_AA_muts_ct_adj.jld2", "AA_muts_ct_adj", AA_muts_ct_adj)
    save("$(date)_$(ndjson_name)_AA_muts_ct_adj_no_dels.jld2", "AA_muts_ct_adj_no_dels", AA_muts_ct_adj_no_dels)    
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj.jld2", "AA_muts_ct_pos_only_adj", AA_muts_ct_pos_only_adj)    
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_no_dels.jld2", "AA_muts_ct_pos_only_adj_no_dels", AA_muts_ct_pos_only_adj_no_dels)
########################################################################################################################################
########################################################################################################################################    
    save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score.jld2", "AA_muts_ct_adj_score", AA_muts_ct_adj_score)
    save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score_no_dels.jld2", "AA_muts_ct_adj_score_no_dels", AA_muts_ct_adj_score_no_dels)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score.jld2", "AA_muts_ct_pos_only_adj_score", AA_muts_ct_pos_only_adj_score)
    save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score_no_dels.jld2", "AA_muts_ct_pos_only_adj_score_no_dels", AA_muts_ct_pos_only_adj_score_no_dels)
########################################################################################################################################
########################################################################################################################################
    save("$(date)_$(ndjson_name)_AA_muts_seq_pos_only_no_dels.jld2", "AA_muts_seq_pos_only_no_dels", AA_muts_seq_pos_only_no_dels)
    save("$(date)_$(ndjson_name)_seq_AA_muts_pos_only_no_dels.jld2", "seq_AA_muts_pos_only_no_dels", seq_AA_muts_pos_only_no_dels)
    save("$(date)_$(ndjson_name)_nuc_muts_rep_seq_grps.jld2", "nuc_muts_rep_seq_grps", nuc_muts_rep_seq_grps)
    save("$(date)_$(ndjson_name)_nuc_dels_rep_seq_grps.jld2", "nuc_dels_rep_seq_grps", nuc_dels_rep_seq_grps)
    save("$(date)_$(ndjson_name)_nuc_muts_rep_seq_grps_no_dels.jld2", "nuc_muts_rep_seq_grps_no_dels", nuc_muts_rep_seq_grps_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_dels.jld2", "rep_seq_grps_dels", rep_seq_grps_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_muts_no_dels.jld2", "rep_seq_grps_muts_no_dels", rep_seq_grps_muts_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_del_ranges_ct.jld2", "rep_seq_grps_del_ranges_ct", rep_seq_grps_del_ranges_ct)
    save("$(date)_$(ndjson_name)_rep_seq_grps_nuc_dropout.jld2", "rep_seq_grps_nuc_dropout", rep_seq_grps_nuc_dropout)
    save("$(date)_$(ndjson_name)_rep_seq_grps_mixed_nucs.jld2", "rep_seq_grps_mixed_nucs", rep_seq_grps_mixed_nucs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_dels.jld2", "rep_seq_grps_AA_dels", rep_seq_grps_AA_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_no_dels.jld2", "rep_seq_grps_AA_no_dels", rep_seq_grps_AA_no_dels)
    save("$(date)_$(ndjson_name)_AA_muts_rep_seq_grps.jld2", "AA_muts_rep_seq_grps", AA_muts_rep_seq_grps)
    save("$(date)_$(ndjson_name)_AA_dels_rep_seq_grps.jld2", "AA_dels_rep_seq_grps", AA_dels_rep_seq_grps)
    save("$(date)_$(ndjson_name)_AA_muts_rep_seq_grps_no_dels.jld2", "AA_muts_rep_seq_grps_no_dels", AA_muts_rep_seq_grps_no_dels)
    save("$(date)_$(ndjson_name)_AA_muts_rep_seq_grps_pos_only.jld2", "AA_muts_rep_seq_grps_pos_only", AA_muts_rep_seq_grps_pos_only)
    save("$(date)_$(ndjson_name)_rep_seq_grps_unknown_AA.jld2", "rep_seq_grps_unknown_AA", rep_seq_grps_unknown_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_mixed_AA_muts.jld2", "rep_seq_grps_mixed_AA_muts", rep_seq_grps_mixed_AA_muts)
    save("$(date)_$(ndjson_name)_rep_seq_grps_nuc_muts_WT.jld2", "rep_seq_grps_nuc_muts_WT", rep_seq_grps_nuc_muts_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_nuc_dels_WT.jld2", "rep_seq_grps_nuc_dels_WT", rep_seq_grps_nuc_dels_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_muts_WT.jld2", "rep_seq_grps_AA_muts_WT", rep_seq_grps_AA_muts_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_dels_WT.jld2", "rep_seq_grps_AA_dels_WT", rep_seq_grps_AA_dels_WT)
    save("$(date)_$(ndjson_name)_nuc_muts_rep_seq_grps_WT.jld2", "nuc_muts_rep_seq_grps_WT", nuc_muts_rep_seq_grps_WT)
    save("$(date)_$(ndjson_name)_nuc_dels_rep_seq_grps_WT.jld2", "nuc_dels_rep_seq_grps_WT", nuc_dels_rep_seq_grps_WT)
    save("$(date)_$(ndjson_name)_AA_muts_rep_seq_grps_WT.jld2", "AA_muts_rep_seq_grps_WT", AA_muts_rep_seq_grps_WT)
    save("$(date)_$(ndjson_name)_AA_dels_rep_seq_grps_WT.jld2", "AA_dels_rep_seq_grps_WT", AA_dels_rep_seq_grps_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_AA_muts_WT_pos_only.jld2", "rep_seq_grps_AA_muts_WT_pos_only", rep_seq_grps_AA_muts_WT_pos_only)
    save("$(date)_$(ndjson_name)_AA_muts_rep_seq_grps_WT_pos_only.jld2", "AA_muts_rep_seq_grps_WT_pos_only", AA_muts_rep_seq_grps_WT_pos_only)
    save("$(date)_$(ndjson_name)_rep_seq_grps_pango.jld2", "rep_seq_grps_pango", rep_seq_grps_pango)
    save("$(date)_$(ndjson_name)_rep_seq_grps_pango_unaliased.jld2", "rep_seq_grps_pango_unaliased", rep_seq_grps_pango_unaliased)

    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_seqs.jld2", "rep_seq_grps_maxmut_seqs", rep_seq_grps_maxmut_seqs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc.jld2", "rep_seq_grps_maxmut_nuc", rep_seq_grps_maxmut_nuc)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_no_dels.jld2", "rep_seq_grps_maxmut_nuc_no_dels", rep_seq_grps_maxmut_nuc_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_dels.jld2", "rep_seq_grps_maxmut_dels", rep_seq_grps_maxmut_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_dropout.jld2", "rep_seq_grps_maxmut_nuc_dropout", rep_seq_grps_maxmut_nuc_dropout)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_mixed_nucs.jld2", "rep_seq_grps_maxmut_mixed_nucs", rep_seq_grps_maxmut_mixed_nucs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA.jld2", "rep_seq_grps_maxmut_AA", rep_seq_grps_maxmut_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_no_dels.jld2", "rep_seq_grps_maxmut_AA_no_dels", rep_seq_grps_maxmut_AA_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_dels.jld2", "rep_seq_grps_maxmut_AA_dels", rep_seq_grps_maxmut_AA_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_pos_only.jld2", "rep_seq_grps_maxmut_AA_pos_only", rep_seq_grps_maxmut_AA_pos_only)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_pos_only_no_dels.jld2", "rep_seq_grps_maxmut_AA_pos_only_no_dels", rep_seq_grps_maxmut_AA_pos_only_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_unknown_AA.jld2", "rep_seq_grps_maxmut_unknown_AA", rep_seq_grps_maxmut_unknown_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_mixed_AA.jld2", "rep_seq_grps_maxmut_mixed_AA", rep_seq_grps_maxmut_mixed_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_seqs.jld2", "rep_seq_grps_maxmut_seqs", rep_seq_grps_maxmut_seqs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc.jld2", "rep_seq_grps_maxmut_nuc", rep_seq_grps_maxmut_nuc)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_no_dels.jld2", "rep_seq_grps_maxmut_nuc_no_dels", rep_seq_grps_maxmut_nuc_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_dels.jld2", "rep_seq_grps_maxmut_dels", rep_seq_grps_maxmut_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_del_ranges_ct.jld2", "rep_seq_grps_maxmut_del_ranges_ct", rep_seq_grps_maxmut_del_ranges_ct)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_dropout.jld2", "rep_seq_grps_maxmut_nuc_dropout", rep_seq_grps_maxmut_nuc_dropout)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_mixed_nucs.jld2", "rep_seq_grps_maxmut_mixed_nucs", rep_seq_grps_maxmut_mixed_nucs)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA.jld2", "rep_seq_grps_maxmut_AA", rep_seq_grps_maxmut_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_no_dels.jld2", "rep_seq_grps_maxmut_AA_no_dels", rep_seq_grps_maxmut_AA_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_dels.jld2", "rep_seq_grps_maxmut_AA_dels", rep_seq_grps_maxmut_AA_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_pos_only.jld2", "rep_seq_grps_maxmut_AA_pos_only", rep_seq_grps_maxmut_AA_pos_only)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_pos_only_no_dels.jld2", "rep_seq_grps_maxmut_AA_pos_only_no_dels", rep_seq_grps_maxmut_AA_pos_only_no_dels)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_unknown_AA.jld2", "rep_seq_grps_maxmut_unknown_AA", rep_seq_grps_maxmut_unknown_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_mixed_AA.jld2", "rep_seq_grps_maxmut_mixed_AA", rep_seq_grps_maxmut_mixed_AA)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_muts_WT.jld2", "rep_seq_grps_maxmut_nuc_muts_WT", rep_seq_grps_maxmut_nuc_muts_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_dels_WT.jld2", "rep_seq_grps_maxmut_nuc_dels_WT", rep_seq_grps_maxmut_nuc_dels_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_muts_WT.jld2", "rep_seq_grps_maxmut_AA_muts_WT", rep_seq_grps_maxmut_AA_muts_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_dels_WT.jld2", "rep_seq_grps_maxmut_AA_dels_WT", rep_seq_grps_maxmut_AA_dels_WT)
    save("$(date)_$(ndjson_name)_rep_seq_grps_maxmut_AA_muts_WT_pos_only.jld2", "rep_seq_grps_maxmut_AA_muts_WT_pos_only", rep_seq_grps_maxmut_AA_muts_WT_pos_only)
#######################################################################################################################################
#######################################################################################################################################
    @save("$(date)_$(ndjson_name)_all_seqs.jld2", all_seqs)
    @save("$(date)_$(ndjson_name)_all_seqs_set.jld2", all_seqs_set)
    @save("$(date)_$(ndjson_name)_all_qualifying_seqs.jld2", all_qualifying_seqs)
    @save("$(date)_$(ndjson_name)_all_qualifying_seqs_set.jld2", all_qualifying_seqs_set)
    @save("$(date)_$(ndjson_name)_all_nonqualifying_seqs.jld2", all_nonqualifying_seqs)
    @save("$(date)_$(ndjson_name)_all_nonqualifying_seqs_set.jld2", all_nonqualifying_seqs_set)
    @save("$(date)_$(ndjson_name)_rep_seqs.jld2", rep_seqs)
    @save("$(date)_$(ndjson_name)_non_rep_seqs.jld2", non_rep_seqs)
    @save("$(date)_$(ndjson_name)_too_many_reversions.jld2", too_many_reversions)


    @save("$(date)_$(ndjson_name)_NSP_muts_sortByCt_Arr.jld2", NSP_muts_sortByCt_Arr)
    @save("$(date)_$(ndjson_name)_NSP_muts_sortByPos_Arr.jld2", NSP_muts_sortByPos_Arr)
    @save("$(date)_$(ndjson_name)_NSP_muts_no_dels_sortByCt_Arr.jld2", NSP_muts_no_dels_sortByCt_Arr)
    @save("$(date)_$(ndjson_name)_NSP_muts_no_dels_sortByPos_Arr.jld2", NSP_muts_no_dels_sortByPos_Arr)

    @save("$(date)_$(ndjson_name)_NSP1_muts_sortByCt.jld2", NSP1_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP2_muts_sortByCt.jld2", NSP2_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP3_muts_sortByCt.jld2", NSP3_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP4_muts_sortByCt.jld2", NSP4_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP5_muts_sortByCt.jld2", NSP5_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP6_muts_sortByCt.jld2", NSP6_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP7_muts_sortByCt.jld2", NSP7_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP8_muts_sortByCt.jld2", NSP8_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP9_muts_sortByCt.jld2", NSP9_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP10_muts_sortByCt.jld2", NSP10_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP11_muts_sortByCt.jld2", NSP11_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP12_muts_sortByCt.jld2", NSP12_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP13_muts_sortByCt.jld2", NSP13_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP14_muts_sortByCt.jld2", NSP14_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP15_muts_sortByCt.jld2", NSP15_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP16_muts_sortByCt.jld2", NSP16_muts_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP1_muts_sortByPos.jld2", NSP1_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP2_muts_sortByPos.jld2", NSP2_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP3_muts_sortByPos.jld2", NSP3_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP4_muts_sortByPos.jld2", NSP4_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP5_muts_sortByPos.jld2", NSP5_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP6_muts_sortByPos.jld2", NSP6_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP7_muts_sortByPos.jld2", NSP7_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP8_muts_sortByPos.jld2", NSP8_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP9_muts_sortByPos.jld2", NSP9_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP10_muts_sortByPos.jld2", NSP10_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP11_muts_sortByPos.jld2", NSP11_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP12_muts_sortByPos.jld2", NSP12_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP13_muts_sortByPos.jld2", NSP13_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP14_muts_sortByPos.jld2", NSP14_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP15_muts_sortByPos.jld2", NSP15_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP16_muts_sortByPos.jld2", NSP16_muts_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP1_muts_no_dels_sortByCt.jld2", NSP1_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP2_muts_no_dels_sortByCt.jld2", NSP2_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP3_muts_no_dels_sortByCt.jld2", NSP3_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP4_muts_no_dels_sortByCt.jld2", NSP4_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP5_muts_no_dels_sortByCt.jld2", NSP5_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP6_muts_no_dels_sortByCt.jld2", NSP6_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP7_muts_no_dels_sortByCt.jld2", NSP7_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP8_muts_no_dels_sortByCt.jld2", NSP8_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP9_muts_no_dels_sortByCt.jld2", NSP9_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP10_muts_no_dels_sortByCt.jld2", NSP10_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP11_muts_no_dels_sortByCt.jld2", NSP11_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP12_muts_no_dels_sortByCt.jld2", NSP12_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP13_muts_no_dels_sortByCt.jld2", NSP13_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP14_muts_no_dels_sortByCt.jld2", NSP14_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP15_muts_no_dels_sortByCt.jld2", NSP15_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP16_muts_no_dels_sortByCt.jld2", NSP16_muts_no_dels_sortByCt)
    @save("$(date)_$(ndjson_name)_NSP1_muts_no_dels_sortByPos.jld2", NSP1_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP2_muts_no_dels_sortByPos.jld2", NSP2_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP3_muts_no_dels_sortByPos.jld2", NSP3_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP4_muts_no_dels_sortByPos.jld2", NSP4_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP5_muts_no_dels_sortByPos.jld2", NSP5_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP6_muts_no_dels_sortByPos.jld2", NSP6_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP7_muts_no_dels_sortByPos.jld2", NSP7_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP8_muts_no_dels_sortByPos.jld2", NSP8_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP9_muts_no_dels_sortByPos.jld2", NSP9_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP10_muts_no_dels_sortByPos.jld2", NSP10_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP11_muts_no_dels_sortByPos.jld2", NSP11_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP12_muts_no_dels_sortByPos.jld2", NSP12_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP13_muts_no_dels_sortByPos.jld2", NSP13_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP14_muts_no_dels_sortByPos.jld2", NSP14_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP15_muts_no_dels_sortByPos.jld2", NSP15_muts_no_dels_sortByPos)
    @save("$(date)_$(ndjson_name)_NSP16_muts_no_dels_sortByPos.jld2", NSP16_muts_no_dels_sortByPos)
#######################################################################################################################################
#######################################################################################################################################
                                                                                  
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_chr_all_ratio_ct_sort.jld2", AA_muts_ct_no_dels_chr_all_ratio_ct_sort)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_chr_all_ratio_pos_sort.jld2", AA_muts_ct_no_dels_chr_all_ratio_pos_sort)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort.jld2", AA_muts_ct_pos_only_no_dels_chr_all_ratio_pos_sort)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort.jld2", AA_muts_ct_pos_only_no_dels_chr_all_ratio_ct_sort)
    @save("$(date)_$(ndjson_name)_avg_AA_subs_per_chr_seq.jld2", avg_AA_subs_per_chr_seq)
    @save("$(date)_$(ndjson_name)_avg_AA_subs_per_circ_seq.jld2", avg_AA_subs_per_circ_seq)

    
    @save("$(date)_$(ndjson_name)_gene_mut_density_sort_by_gene.jld2", gene_mut_density_sort_by_gene)
    @save("$(date)_$(ndjson_name)_gene_mut_density_sort_by_density.jld2", gene_mut_density_sort_by_density)
    @save("$(date)_$(ndjson_name)_domain_mut_density_sort_by_gene.jld2", domain_mut_density_sort_by_gene)
    @save("$(date)_$(ndjson_name)_domain_mut_density_sort_by_density.jld2", domain_mut_density_sort_by_density)
########################################################################################################################################
########################################################################################################################################
    @save("$(date)_$(ndjson_name)_excluded_pos.jld2", excluded_pos)
    @save("$(date)_$(ndjson_name)_excluded_AA.jld2", excluded_AA)
    
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_sort_by_site.jld2", nuc_muts_ct_sort_by_site)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_sort_by_seq_ct.jld2", nuc_muts_ct_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_sort_by_site.jld2", nuc_muts_ct_adj_sort_by_site)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_sort_by_seq_ct.jld2", nuc_muts_ct_adj_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_no_dels_sort_by_site.jld2", nuc_muts_ct_adj_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", nuc_muts_ct_adj_no_dels_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score_sort_by_site.jld2", nuc_muts_ct_adj_score_sort_by_site)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score_sort_by_score.jld2", nuc_muts_ct_adj_score_sort_by_score)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score_no_dels_sort_by_site.jld2", nuc_muts_ct_adj_score_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_nuc_muts_ct_adj_score_no_dels_sort_by_score.jld2", nuc_muts_ct_adj_score_no_dels_sort_by_score)
########################################################################################################################################
########################################################################################################################################    
    @save("$(date)_$(ndjson_name)_AA_muts_ct_sort_by_site.jld2", AA_muts_ct_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_sort_by_seq_ct.jld2", AA_muts_ct_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_sort_by_site.jld2", AA_muts_ct_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_sort_by_seq_ct.jld2", AA_muts_ct_no_dels_sort_by_seq_ct)
    
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_sort_by_site.jld2", AA_muts_ct_adj_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_sort_by_seq_ct.jld2", AA_muts_ct_adj_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_no_dels_sort_by_site.jld2", AA_muts_ct_adj_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_no_dels_sort_by_seq_ct.jld2", AA_muts_ct_adj_no_dels_sort_by_seq_ct)
        
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score_sort_by_score.jld2", AA_muts_ct_adj_score_sort_by_score) 
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score_sort_by_site.jld2", AA_muts_ct_adj_score_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score_no_dels_sort_by_score.jld2", AA_muts_ct_adj_score_no_dels_sort_by_score) 
    @save("$(date)_$(ndjson_name)_AA_muts_ct_adj_score_no_dels_sort_by_site.jld2", AA_muts_ct_adj_score_no_dels_sort_by_site)
    
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_sort_by_site.jld2", AA_muts_ct_pos_only_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_sort_by_seq_ct.jld2", AA_muts_ct_pos_only_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_sort_by_site.jld2", AA_muts_ct_pos_only_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_sort_by_seq_ct.jld2", AA_muts_ct_pos_only_no_dels_sort_by_seq_ct)
    
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_sort_by_site.jld2", AA_muts_ct_pos_only_adj_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_sort_by_seq_ct.jld2", AA_muts_ct_pos_only_adj_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_no_dels_sort_by_site.jld2", AA_muts_ct_pos_only_adj_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct.jld2", AA_muts_ct_pos_only_adj_no_dels_sort_by_seq_ct)
    
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score_sort_by_site.jld2", AA_muts_ct_pos_only_adj_score_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score_sort_by_score.jld2", AA_muts_ct_pos_only_adj_score_sort_by_score)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site.jld2", AA_muts_ct_pos_only_adj_score_no_dels_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score.jld2", AA_muts_ct_pos_only_adj_score_no_dels_sort_by_score)
########################################################################################################################################
########################################################################################################################################  
    @save("$(date)_$(ndjson_name)_chronic_search_muts.jld2", chronic_search_muts)
    @save("$(date)_$(ndjson_name)_chronic_search_muts_min2.jld2", chronic_search_muts_min2)
    @save("$(date)_$(ndjson_name)_chronic_search_muts_min2_no_dels.jld2", chronic_search_muts_min2_no_dels)
########################################################################################################################################
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_sort_by_site.jld2", AA_muts_ct_no_dels_no_revs_sort_by_site)
    @save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_sort_by_seq_ct.jld2", AA_muts_ct_no_dels_no_revs_sort_by_seq_ct)
    @save("$(date)_$(ndjson_name)_avg_AA_subs_per_chr_seq_no_revs.jld2", avg_AA_subs_per_chr_seq_no_revs)
    @save("$(date)_$(ndjson_name)_chronic_search_muts_v2.jld2", chronic_search_muts_v2)
end

2025-05-26_750PM


chr_AA_nuc_del_adj_JSON3_2025_03_27_v2 (generic function with 1 method)

In [43]:
gene(n) = split(n, ":")[1]
poz(n) = parse(Int, string(split(n, ":")[2][2:end-1]))
chronic_search_muts_v2_arr = Vector{String}()
for mut in chronic_search_muts_v2
    push!(chronic_search_muts_v2_arr, mut)
end
chr_srch_mut_sort = sort(chronic_search_muts_v2_arr, by = x -> (gene(x), poz(x)))
chr_srch_mut_sort_len = length(chr_srch_mut_sort)
println("Number of Mutations = $(chr_srch_mut_sort_len)")
#######################################
df_chr_search_muts_v2 = DataFrame("Chronic Search Mutations" => chr_srch_mut_sort)
#display(df_chr_search_muts_v2)
CSV.write("__CSV_Files/df_chr_search_muts_v2.csv", df_chr_search_muts_v2)
########################################
chronic_search_muts_table = pretty_table(df_chr_search_muts_v2)
########################################

Number of Mutations = 1735
┌──────────────────────────┐
│ Chronic Search Mutations │
│                   String │
├──────────────────────────┤
│                    E:V5D │
│                    E:V5A │
│                    E:V5F │
│                    E:V5L │
│                    E:S6L │
│                    E:E8A │
│                   E:G10S │
│                   E:G10C │
│                   E:G10A │
│                   E:G10V │
│                   E:T11A │
│                   E:V14A │
│                   E:N15I │
│                   E:N15K │
│                   E:N15H │
│                   E:N15Y │
│                   E:N15T │
│                   E:N15S │
│                   E:S16I │
│                   E:S16R │
│                   E:S16N │
│                   E:L18R │
│                   E:L19P │
│                   E:L19V │
│                   E:L19F │
│                   E:L19S │
│                   E:F20S │
│                   E:A22D │
│                   E:A22T │
│               

In [42]:
################ Execute Load Chronic Dictionaries Function, chronics_2025_05_07__5566seq #########################
dddate = Dates.format(now(), "yyyy_mm_dd_IMMp")
println(dddate)
@time begin
    chr_AA_nuc_del_adj_JSON3_2025_03_27_v2("chronics_2025_05_07__5566seq.ndjson", "chronics_2025_05_07__5566seq", 1, 8, 5, 6, 1, 5)
end
# chr_AA_nuc_del_adj_JSON3_2025_1_13_v2  (ndjson,                          ndjson_name,        minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh,    fx_name)
# (ndjson, ndjson_name, minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)
println("\n"^3)

2025_05_26_750PM
######################### Sequences with more than 6 Reversions #########################
EPI_ISL_10816733, EPI_ISL_10816736, EPI_ISL_10816738, EPI_ISL_10816741, EPI_ISL_12805921, EPI_ISL_13050078, EPI_ISL_13386427, EPI_ISL_13417595, EPI_ISL_13454711, EPI_ISL_13464004, EPI_ISL_13637734, EPI_ISL_13653660, EPI_ISL_13858664, EPI_ISL_13889482, EPI_ISL_13957684, EPI_ISL_14676287, EPI_ISL_14960911, EPI_ISL_15014035, EPI_ISL_15170512, EPI_ISL_15296403, EPI_ISL_15380518, EPI_ISL_15387248, EPI_ISL_15537619, EPI_ISL_15550525, EPI_ISL_15581446, EPI_ISL_15614383, EPI_ISL_15635022, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15685810, EPI_ISL_15688500, EPI_ISL_15793981, EPI_ISL_15895625, EPI_ISL_15920754, EPI_ISL_15920755, EPI_ISL_15945504, EPI_ISL_15962045, EPI_ISL_16178634, EPI_ISL_16201173, EPI_ISL_16287253, EPI_ISL_16847425, EPI_ISL_16942000, EPI_ISL_16997638, EPI_ISL_17077446, EPI_ISL_17189360, EPI_ISL_17344660, EPI_ISL_17431238, EPI_ISL_17503268, EPI_ISL_17595116, EPI_ISL_175

In [33]:
################ Execute Load Chronic Dictionaries Function, chronics_2025_04_14__5383seq #########################
dddate = Dates.format(now(), "yyyy_mm_dd_IMMp")
println(dddate)
@time begin
    chr_AA_nuc_del_adj_JSON3_2025_03_27_v2("chronics_2025_04_14__5383seq.ndjson", "chronics_2025_04_14__5383seq", 1, 8, 6, 6, 1, 5)
end
# chr_AA_nuc_del_adj_JSON3_2025_1_13_v2  (ndjson,                          ndjson_name,        minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh,    fx_name)
# (ndjson, ndjson_name, minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)
println("\n"^3)

2025_05_04_1255PM
######################### Sequences with more than 6 Reversions #########################
EPI_ISL_10816733, EPI_ISL_10816736, EPI_ISL_10816738, EPI_ISL_10816741, EPI_ISL_12805921, EPI_ISL_13050078, EPI_ISL_13386427, EPI_ISL_13417595, EPI_ISL_13454711, EPI_ISL_13464004, EPI_ISL_13637734, EPI_ISL_13653660, EPI_ISL_13858664, EPI_ISL_13889482, EPI_ISL_13957684, EPI_ISL_14676287, EPI_ISL_14960911, EPI_ISL_15014035, EPI_ISL_15170512, EPI_ISL_15296403, EPI_ISL_15380518, EPI_ISL_15387248, EPI_ISL_15537619, EPI_ISL_15550525, EPI_ISL_15581446, EPI_ISL_15614383, EPI_ISL_15635022, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15685810, EPI_ISL_15688500, EPI_ISL_15793981, EPI_ISL_15895625, EPI_ISL_15920754, EPI_ISL_15920755, EPI_ISL_15945504, EPI_ISL_15962045, EPI_ISL_16178634, EPI_ISL_16201173, EPI_ISL_16287253, EPI_ISL_16847425, EPI_ISL_16942000, EPI_ISL_16997638, EPI_ISL_17077446, EPI_ISL_17189360, EPI_ISL_17344660, EPI_ISL_17431238, EPI_ISL_17503268, EPI_ISL_17595116, EPI_ISL_17

In [133]:
@time begin
    chr_AA_nuc_del_adj_JSON3_2025_03_27_v2("test1.ndjson", "test1", 1, 8, 6, 6, 1, 7)
end
# chr_AA_nuc_del_adj_JSON3_2025_1_13_v2  (ndjson,                          ndjson_name,        minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)
# (ndjson, ndjson_name, minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)

######################### Sequences with more than 6 Reversions #########################


EPI_ISLs for sequences with 8 private subs + del ranges


EPI_ISLs for sequences with 7 private subs + del ranges


EPI_ISLs for sequences with 6 private subs + del ranges
EPI_ISL_19258185, 

EPI_ISLs for sequences with 5 private subs + del ranges


EPI_ISLs for sequences with 4 private subs + del ranges


EPI_ISLs for sequences with 3 private subs + del ranges


Number of Sequences in rep_seqs list: 0



********************** Total Counted Chronic Seq Ct = 0  **********************
********************** Total Counted Circulating Seq Ct = 9976242  **********************

********************** Total Chronic AA Substitution Ct = 0  **********************
********************** Total Circulating AA Substitution Ct = 17857986  **********************

********************** Avg Chronic AA Substitution Ct = NaN  **********************
********************** Avg Circulating AA Substitution Ct = 1.79

LoadError: BoundsError: attempt to access 0-element Vector{Pair{String, Float64}} at index [1000]

In [109]:
# save("$(date)_$(ndjson_name)_rep_seq_grps_unknown_AA.jld2", "rep_seq_grps_unknown_AA", rep_seq_grps_unknown_AA)
# "save(\"\$(date)_\$(ndjson_name)_$(name).jld2\", \"$(name)\", $(name))"
# @save("$(date)_$(ndjson_name)_excluded_AA.jld2", excluded_AA)
# "\@save(\"\$(date)_\$(ndjson_name)_$(name).jld2\", $(name))"
function print_save_dict(dicts::Vector{String})
    for name in dicts
        println("save(\"\$(date)_\$(ndjson_name)_$(name).jld2\", \"$(name)\", $(name))")
    end
end
function print_vect_set_ct(vecs::Vector{String})
    for name in vecs
        println("@save(\"\$(date)_\$(ndjson_name)_$(name).jld2\", $(name))")
    end
end

print_vect_set_ct (generic function with 1 method)

In [112]:
dict_vec = ["AA_muts_ct_no_dels_no_revs", "AA_muts_ct_no_dels_no_revs_chr_all_ratio", "AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio"]
print_save_dict(dict_vec)

# print_save_dict(dicts::Vector{String})

save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs.jld2", "AA_muts_ct_no_dels_no_revs", AA_muts_ct_no_dels_no_revs)
save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_chr_ratio.jld2", "AA_muts_ct_no_dels_no_revs_chr_ratio", AA_muts_ct_no_dels_no_revs_chr_ratio)
save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_no_dels_no_revs_chr_all_ratio", AA_muts_ct_no_dels_no_revs_chr_all_ratio)
save("$(date)_$(ndjson_name)_AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio.jld2", "AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio", AA_muts_ct_pos_only_no_dels_no_revs_chr_all_ratio)


In [113]:
vec_vec = ["AA_muts_ct_no_dels_no_revs_sort_by_site", "AA_muts_ct_no_dels_no_revs_sort_by_seq_ct", "avg_AA_subs_per_chr_seq_no_revs"]
print_vect_set_ct(vec_vec)

# print_vect_set_ct(vecs::Vector{String})

@save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_sort_by_site.jld2", AA_muts_ct_no_dels_no_revs_sort_by_site)
@save("$(date)_$(ndjson_name)_AA_muts_ct_no_dels_no_revs_sort_by_seq_ct.jld2", AA_muts_ct_no_dels_no_revs_sort_by_seq_ct)
@save("$(date)_$(ndjson_name)_avg_AA_subs_per_chr_seq_no_revs.jld2", avg_AA_subs_per_chr_seq_no_revs)


In [3]:
vec_name = "NSP_muts_sortByCt_Arr"
println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
vec_name = "NSP_muts_sortByPos_Arr"
println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
    

@save("$(date)_$(fx_name)_$(ndjson_name)_NSP_muts_sortByCt_Arr.jld2", NSP_muts_sortByCt_Arr)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP_muts_sortByCt_Arr.jld2", NSP_muts_sortByCt_Arr)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP_muts_sortByPos_Arr.jld2", NSP_muts_sortByPos_Arr)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP_muts_sortByPos_Arr.jld2", NSP_muts_sortByPos_Arr)


In [21]:
NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
for i in 1:length(NSP_array)
    println("$(NSP_array[i])_muts_sortByCt = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> x[2], rev=true)")
    println("$(NSP_array[i])_muts_sortByPos = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> NSPmutPosKey(x[1]))")
end
for i in 1:length(NSP_array)
    println("$(NSP_array[i])_muts_no_dels_sortByCt = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> x[2], rev=true)")
    println("$(NSP_array[i])_muts_no_dels_sortByPos = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> NSPmutPosKey(x[1]))")
end

NSP1_muts_sortByCt = sort(collect(NSP_muts["NSP1"]), by = x -> x[2], rev=true)
NSP1_muts_sortByCt = sort(collect(NSP_muts["NSP1"]), by = x -> NSPmutPosKey(x[1]))
NSP2_muts_sortByCt = sort(collect(NSP_muts["NSP2"]), by = x -> x[2], rev=true)
NSP2_muts_sortByCt = sort(collect(NSP_muts["NSP2"]), by = x -> NSPmutPosKey(x[1]))
NSP3_muts_sortByCt = sort(collect(NSP_muts["NSP3"]), by = x -> x[2], rev=true)
NSP3_muts_sortByCt = sort(collect(NSP_muts["NSP3"]), by = x -> NSPmutPosKey(x[1]))
NSP4_muts_sortByCt = sort(collect(NSP_muts["NSP4"]), by = x -> x[2], rev=true)
NSP4_muts_sortByCt = sort(collect(NSP_muts["NSP4"]), by = x -> NSPmutPosKey(x[1]))
NSP5_muts_sortByCt = sort(collect(NSP_muts["NSP5"]), by = x -> x[2], rev=true)
NSP5_muts_sortByCt = sort(collect(NSP_muts["NSP5"]), by = x -> NSPmutPosKey(x[1]))
NSP6_muts_sortByCt = sort(collect(NSP_muts["NSP6"]), by = x -> x[2], rev=true)
NSP6_muts_sortByCt = sort(collect(NSP_muts["NSP6"]), by = x -> NSPmutPosKey(x[1]))
NSP7_muts_sortByCt = sort(co

In [ ]:
NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
for i in 1:length(NSP_array)
    println("$(NSP_array[i])_muts_sortByCt = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> x[2], rev=true)")
    println("$(NSP_array[i])_muts_sortByPos = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> NSPmutPosKey(x[1]))")
end
for i in 1:length(NSP_array)
    println("$(NSP_array[i])_muts_no_dels_sortByCt = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> x[2], rev=true)")
    println("$(NSP_array[i])_muts_no_dels_sortByPos = sort(collect(NSP_muts[\"$(NSP_array[i])\"]), by = x -> NSPmutPosKey(x[1]))")
end

In [9]:
dict_name = "rep_seq_grps_maxmut_nuc_muts_WT"
println("save(\"\$(date)_\\$(ndjson_name)_$(dict_name).jld2\", \"$(dict_name)\", $(dict_name))")
dict_name = "rep_seq_grps_maxmut_nuc_dels_WT"
println("save(\"\$(date)_\\$(ndjson_name)_$(dict_name).jld2\", \"$(dict_name)\", $(dict_name))")
dict_name = "rep_seq_grps_maxmut_AA_muts_WT"
println("save(\"\$(date)_\\$(ndjson_name)_$(dict_name).jld2\", \"$(dict_name)\", $(dict_name))")
dict_name = "rep_seq_grps_maxmut_AA_dels_WT"
println("save(\"\$(date)_\\$(ndjson_name)_$(dict_name).jld2\", \"$(dict_name)\", $(dict_name))")
dict_name = "rep_seq_grps_maxmut_AA_muts_WT_pos_only"
println("save(\"\$(date)_\\$(ndjson_name)_$(dict_name).jld2\", \"$(dict_name)\", $(dict_name))")

save("$(date)_$(fx_name)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_muts_WT.jld2", "rep_seq_grps_maxmut_nuc_muts_WT", rep_seq_grps_maxmut_nuc_muts_WT)
save("$(date)_$(fx_name)_$(ndjson_name)_rep_seq_grps_maxmut_nuc_dels_WT.jld2", "rep_seq_grps_maxmut_nuc_dels_WT", rep_seq_grps_maxmut_nuc_dels_WT)
save("$(date)_$(fx_name)_$(ndjson_name)_rep_seq_grps_maxmut_AA_muts_WT.jld2", "rep_seq_grps_maxmut_AA_muts_WT", rep_seq_grps_maxmut_AA_muts_WT)
save("$(date)_$(fx_name)_$(ndjson_name)_rep_seq_grps_maxmut_AA_dels_WT.jld2", "rep_seq_grps_maxmut_AA_dels_WT", rep_seq_grps_maxmut_AA_dels_WT)
save("$(date)_$(fx_name)_$(ndjson_name)_rep_seq_grps_maxmut_AA_muts_WT_pos_only.jld2", "rep_seq_grps_maxmut_AA_muts_WT_pos_only", rep_seq_grps_maxmut_AA_muts_WT_pos_only)


In [26]:
NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_sortByCt"
    println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_sortByPos"
    println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_no_dels_sortByCt"
    println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_no_dels_sortByPos"
    println("@save(\"\$(date)_\\$(ndjson_name)_$(vec_name).jld2\", $(vec_name))")
end
{Pair{String, Int64}}

@save("$(date)_$(fx_name)_$(ndjson_name)_NSP1_muts_sortByCt.jld2", NSP1_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP2_muts_sortByCt.jld2", NSP2_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP3_muts_sortByCt.jld2", NSP3_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP4_muts_sortByCt.jld2", NSP4_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP5_muts_sortByCt.jld2", NSP5_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP6_muts_sortByCt.jld2", NSP6_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP7_muts_sortByCt.jld2", NSP7_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP8_muts_sortByCt.jld2", NSP8_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP9_muts_sortByCt.jld2", NSP9_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP10_muts_sortByCt.jld2", NSP10_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name)_NSP11_muts_sortByCt.jld2", NSP11_muts_sortByCt)
@save("$(date)_$(fx_name)_$(ndjson_name

In [39]:
NSP_array = ["NSP1", "NSP2", "NSP3", "NSP4", "NSP5", "NSP6", "NSP7", "NSP8", "NSP9", "NSP10", "NSP11", "NSP12", "NSP13", "NSP14", "NSP15", "NSP16"]
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_sortByCt"
    println("$(vec_name) = {Pair{String, Int64}}()")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_sortByPos"
    println("$(vec_name) = {Pair{String, Int64}}()")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_no_dels_sortByCt"
    println("$(vec_name) = {Pair{String, Int64}}()")
end
for i in 1:length(NSP_array)
    vec_name = "$(NSP_array[i])_muts_no_dels_sortByPos"
    println("$(vec_name) = {Pair{String, Int64}}()")
end

NSP1_muts_sortByCt = {Pair{String, Int64}}()
NSP2_muts_sortByCt = {Pair{String, Int64}}()
NSP3_muts_sortByCt = {Pair{String, Int64}}()
NSP4_muts_sortByCt = {Pair{String, Int64}}()
NSP5_muts_sortByCt = {Pair{String, Int64}}()
NSP6_muts_sortByCt = {Pair{String, Int64}}()
NSP7_muts_sortByCt = {Pair{String, Int64}}()
NSP8_muts_sortByCt = {Pair{String, Int64}}()
NSP9_muts_sortByCt = {Pair{String, Int64}}()
NSP10_muts_sortByCt = {Pair{String, Int64}}()
NSP11_muts_sortByCt = {Pair{String, Int64}}()
NSP12_muts_sortByCt = {Pair{String, Int64}}()
NSP13_muts_sortByCt = {Pair{String, Int64}}()
NSP14_muts_sortByCt = {Pair{String, Int64}}()
NSP15_muts_sortByCt = {Pair{String, Int64}}()
NSP16_muts_sortByCt = {Pair{String, Int64}}()
NSP1_muts_sortByPos = {Pair{String, Int64}}()
NSP2_muts_sortByPos = {Pair{String, Int64}}()
NSP3_muts_sortByPos = {Pair{String, Int64}}()
NSP4_muts_sortByPos = {Pair{String, Int64}}()
NSP5_muts_sortByPos = {Pair{String, Int64}}()
NSP6_muts_sortByPos = {Pair{String, Int64}}

In [11]:
@time begin
    chr_AA_nuc_del_adj_JSON3_2025_1_13_v2("chronics_2024_12_31__4491_seq_v2_frameshifts_fixed.ndjson", "chronics_2024_12_31__4491_seq_v2_frameshifts_fixed", 1, 8, 6, 10, 1, 7, "chr_v2")
end
# chr_AA_nuc_del_adj_JSON3_2025_1_13_v2  (ndjson,                            ndjson_name,    minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)
# (ndjson, ndjson_name, minadj, adj_score_ct_div, rep_thresh, revs_thresh, print_ct_thresh, mut_thresh)

######################### Sequences with more than 10 Reversions #########################
EPI_ISL_12805921, EPI_ISL_13417595, EPI_ISL_13454711, EPI_ISL_13858664, EPI_ISL_13889482, EPI_ISL_14676287, EPI_ISL_14960911, EPI_ISL_15014035, EPI_ISL_15170512, EPI_ISL_15296403, EPI_ISL_15380518, EPI_ISL_15537619, EPI_ISL_15635022, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15685810, EPI_ISL_15920754, EPI_ISL_15962045, EPI_ISL_16178634, EPI_ISL_16201173, EPI_ISL_16942000, EPI_ISL_17737562, EPI_ISL_18391451, EPI_ISL_18416870, EPI_ISL_18668995, EPI_ISL_18953132, EPI_ISL_18971336, EPI_ISL_19506387, 

EPI_ISLs for sequences with 8 private subs + del ranges
EPI_ISL_10548919, EPI_ISL_12063598, EPI_ISL_12063602, EPI_ISL_12095320, EPI_ISL_12564261, EPI_ISL_12652423, EPI_ISL_12685124, EPI_ISL_12685126, EPI_ISL_12744193, EPI_ISL_13203062, EPI_ISL_13332921, EPI_ISL_13357233, EPI_ISL_13358809, EPI_ISL_13483653, EPI_ISL_13723111, EPI_ISL_13726415, EPI_ISL_13780813, EPI_ISL_13989484, EPI_ISL_14208835, EPI_I

T241C sequences
A28877T sequences
103.104583 seconds (1.04 G allocations: 73.728 GiB, 7.74% gc time, 4.45% compilation time)


In [26]:
for line in eachline("chronics_2024_12_31__4491_seq_v2_frameshifts_fixed.ndjson")
    j = JSON3.read(line)
    name = EPI_ISL(j.seqName)
    if j.privateNucMutations.totalReversionSubstitutions == 7
        print(name, ", ")
    end
end

EPI_ISL_17997249, EPI_ISL_13464004, EPI_ISL_15793981, EPI_ISL_15945504, EPI_ISL_15550525, EPI_ISL_13957684, EPI_ISL_16287253, EPI_ISL_17595980, EPI_ISL_17954106, EPI_ISL_17503268, EPI_ISL_18359229, EPI_ISL_18110496, EPI_ISL_19203296, EPI_ISL_19506410, EPI_ISL_19506412, EPI_ISL_19606872, EPI_ISL_19606873, EPI_ISL_19606893, 

In [14]:
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
date = Dates.format(today(),"yyyy-mm-dd")
println(date_right_now)
function chronic_epis(chronic_ndjson)
    chr_epi_set = Set{String}()
    for line in eachline(chronic_ndjson)
        j = JSON3.read(line)
        name = EPI_ISL(j.seqName)
        push!(chr_epi_set, name)
        # print(name, ", ")
    end
    return chr_epi_set
end
function multiepi_to_epis(multi)  
    epi_num_only_pre(n) = split(n, "_")[3]
    epi_num_only_first(n) = parse(Int, split(epi_num_only_pre(n), "-")[1])
    epi_num_only_last(n) = parse(Int, split(epi_num_only_pre(n), "-")[2])
    first = epi_num_only_first(multi)
    last = epi_num_only_last(multi)
    epi_arr = Vector{String}()
    for i in first:last
        i_str = string(i)
        epi = "EPI_ISL_"*i_str
        push!(epi_arr, epi)
    end
    return epi_arr
end
function stringlist_to_strings(txt)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    arr_of_strings1 = Vector{String}()
    arr_of_strings2 = Vector{String}()
    no_newlines = replace(txt, 
        "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(arr_of_strings2, mseq)
            end
        else 
            push!(arr_of_strings2, seq)
        end
    end
    sort_arr_of_strings2 = sort(collect(arr_of_strings2), by = x -> epi_sortkey(x))
    return sort_arr_of_strings2
end
function stringlist_to_strings_set(txt::String)
    epi_num_only_pre(n) = split(n, "_")[3]
    function epi_sortkey(epi)
        epinum = epi_num_only_pre(epi)
        epi_key = (length(epinum), epinum)
        return epi_key
    end
    set_of_strings = Set{String}()
    no_newlines = replace(txt, "\n" =>" ")
    for seq in split(no_newlines, ", ")
        if '-' in seq
            multis = multiepi_to_epis(seq)
            for mseq in multis
                push!(set_of_strings, mseq)
            end
        else 
            push!(set_of_strings, seq)
        end
    end
    return set_of_strings
end 

    chr_epi_2025_04_07 = stringlist_to_strings_set("EPI_ISL_474824, EPI_ISL_508674, EPI_ISL_510148, EPI_ISL_516999, EPI_ISL_517000, EPI_ISL_528438, EPI_ISL_534720, EPI_ISL_539541, EPI_ISL_539542, EPI_ISL_540582, EPI_ISL_593478, EPI_ISL_593479, EPI_ISL_593480, EPI_ISL_593553, EPI_ISL_593554, EPI_ISL_593555, EPI_ISL_593556, EPI_ISL_593557, EPI_ISL_593558, EPI_ISL_596228, EPI_ISL_602912, EPI_ISL_654166, EPI_ISL_654170, EPI_ISL_654172, EPI_ISL_654173, EPI_ISL_654182, EPI_ISL_654186, EPI_ISL_654191, EPI_ISL_654194, EPI_ISL_678289, EPI_ISL_686537, EPI_ISL_732971, EPI_ISL_753676, EPI_ISL_776770, EPI_ISL_801876, EPI_ISL_812862, EPI_ISL_831496, EPI_ISL_856750, EPI_ISL_872778, EPI_ISL_941340, EPI_ISL_949208, EPI_ISL_959309, EPI_ISL_979349, EPI_ISL_1014810, EPI_ISL_1039159, EPI_ISL_1039839, EPI_ISL_1059094, EPI_ISL_1081672, EPI_ISL_1081673, EPI_ISL_1090851, EPI_ISL_1104882, EPI_ISL_1105146, EPI_ISL_1105179, EPI_ISL_1105235, EPI_ISL_1158169, EPI_ISL_1200867, EPI_ISL_1200912, EPI_ISL_1209365, EPI_ISL_1225620, EPI_ISL_1241756, EPI_ISL_1248458, EPI_ISL_1248485, EPI_ISL_1248497, EPI_ISL_1257978, EPI_ISL_1261009, EPI_ISL_1295569, EPI_ISL_1295575, EPI_ISL_1309105, EPI_ISL_1347940, EPI_ISL_1347941, EPI_ISL_1347942, EPI_ISL_1366562, EPI_ISL_1366563, EPI_ISL_1366564, EPI_ISL_1366565, EPI_ISL_1366566, EPI_ISL_1366567, EPI_ISL_1366568, EPI_ISL_1366569, EPI_ISL_1366570, EPI_ISL_1366571, EPI_ISL_1366572, EPI_ISL_1366573, EPI_ISL_1366792, EPI_ISL_1372287, EPI_ISL_1372288, EPI_ISL_1376522, EPI_ISL_1376523, EPI_ISL_1378739, EPI_ISL_1403404, EPI_ISL_1469973, EPI_ISL_1470396, EPI_ISL_1473700, EPI_ISL_1477334, EPI_ISL_1483302, EPI_ISL_1490655, EPI_ISL_1490669, EPI_ISL_1495749, EPI_ISL_1517099, EPI_ISL_1522107, EPI_ISL_1534324, EPI_ISL_1547461, EPI_ISL_1575358, EPI_ISL_1578495, EPI_ISL_1626185, EPI_ISL_1637040, EPI_ISL_1668821, EPI_ISL_1668822, EPI_ISL_1668823, EPI_ISL_1668824, EPI_ISL_1668825, EPI_ISL_1670378, EPI_ISL_1671330, EPI_ISL_1675190, EPI_ISL_1675203, EPI_ISL_1678377, EPI_ISL_1738308, EPI_ISL_1743263, EPI_ISL_1744401, EPI_ISL_1756179, EPI_ISL_1756180, EPI_ISL_1792929, EPI_ISL_1829054, EPI_ISL_1829108, EPI_ISL_1840893, EPI_ISL_1855854, EPI_ISL_1904901, EPI_ISL_1904903, EPI_ISL_1908476, EPI_ISL_1935116, EPI_ISL_1935282, EPI_ISL_1941336, EPI_ISL_1941816, EPI_ISL_1965009, EPI_ISL_1965714, EPI_ISL_1965722, EPI_ISL_2001260, EPI_ISL_2001292, EPI_ISL_2030332, EPI_ISL_2035047, EPI_ISL_2036230, EPI_ISL_2080876, EPI_ISL_2096935, EPI_ISL_2098974, EPI_ISL_2140680, EPI_ISL_2142447, EPI_ISL_2159603, EPI_ISL_2170618, EPI_ISL_2179080, EPI_ISL_2179597, EPI_ISL_2179598, EPI_ISL_2179600, EPI_ISL_2179601, EPI_ISL_2179635, EPI_ISL_2193387, EPI_ISL_2193781, EPI_ISL_2193790, EPI_ISL_2229473, EPI_ISL_2232987, EPI_ISL_2232988, EPI_ISL_2245655, EPI_ISL_2246946, EPI_ISL_2272316, EPI_ISL_2281463, EPI_ISL_2285732, EPI_ISL_2289324, EPI_ISL_2331631, EPI_ISL_2335139, EPI_ISL_2355027, EPI_ISL_2373667, EPI_ISL_2373676, EPI_ISL_2373689, EPI_ISL_2373915, EPI_ISL_2373976, EPI_ISL_2376734, EPI_ISL_2385134, EPI_ISL_2397307, EPI_ISL_2397308, EPI_ISL_2397309, EPI_ISL_2397310, EPI_ISL_2397311, EPI_ISL_2397312, EPI_ISL_2397313, EPI_ISL_2403056, EPI_ISL_2408213, EPI_ISL_2443102, EPI_ISL_2443306, EPI_ISL_2451852, EPI_ISL_2453771, EPI_ISL_2456706, EPI_ISL_2466638, EPI_ISL_2482552, EPI_ISL_2482891, EPI_ISL_2484152, EPI_ISL_2492266, EPI_ISL_2501697, EPI_ISL_2504017, EPI_ISL_2510252, EPI_ISL_2526835, EPI_ISL_2537393, EPI_ISL_2545260, EPI_ISL_2567482, EPI_ISL_2567516, EPI_ISL_2589968, EPI_ISL_2598472, EPI_ISL_2621566, EPI_ISL_2622092, EPI_ISL_2626505, EPI_ISL_2629070, EPI_ISL_2629071, EPI_ISL_2629072, EPI_ISL_2629073, EPI_ISL_2629074, EPI_ISL_2629075, EPI_ISL_2646107, EPI_ISL_2652487, EPI_ISL_2658958, EPI_ISL_2658962, EPI_ISL_2658963, EPI_ISL_2658970, EPI_ISL_2658971, EPI_ISL_2658972, EPI_ISL_2681259, EPI_ISL_2686814, EPI_ISL_2686837, EPI_ISL_2713004, EPI_ISL_2716246, EPI_ISL_2746031, EPI_ISL_2758178, EPI_ISL_2758179, EPI_ISL_2776212, EPI_ISL_2790083, EPI_ISL_2791260, EPI_ISL_2810326, EPI_ISL_2811857, EPI_ISL_2817504, EPI_ISL_2820526, EPI_ISL_2828407, EPI_ISL_2833904, EPI_ISL_2858161, EPI_ISL_2858877, EPI_ISL_2860316, EPI_ISL_2868572, EPI_ISL_2868616, EPI_ISL_2876377, EPI_ISL_2894033, EPI_ISL_2903438, EPI_ISL_2931876, EPI_ISL_2931884, EPI_ISL_2931896, EPI_ISL_2931903, EPI_ISL_2955288, EPI_ISL_2955320, EPI_ISL_2966985, EPI_ISL_2978243, EPI_ISL_2978352, EPI_ISL_2984725, EPI_ISL_2990101, EPI_ISL_2993722, EPI_ISL_3010321, EPI_ISL_3029841, EPI_ISL_3030114, EPI_ISL_3030118, EPI_ISL_3030145, EPI_ISL_3030738, EPI_ISL_3032627, EPI_ISL_3033635, EPI_ISL_3039352, EPI_ISL_3045789, EPI_ISL_3053903, EPI_ISL_3061061, EPI_ISL_3064314, EPI_ISL_3129808, EPI_ISL_3130077, EPI_ISL_3130081, EPI_ISL_3130177, EPI_ISL_3130302, EPI_ISL_3132631, EPI_ISL_3133023, EPI_ISL_3152200, EPI_ISL_3153240, EPI_ISL_3164424, EPI_ISL_3185346, EPI_ISL_3209041, EPI_ISL_3212959, EPI_ISL_3246237, EPI_ISL_3251444, EPI_ISL_3251605, EPI_ISL_3259560, EPI_ISL_3275376, EPI_ISL_3339536, EPI_ISL_3356734, EPI_ISL_3358574, EPI_ISL_3370176, EPI_ISL_3394321, EPI_ISL_3396491, EPI_ISL_3414767, EPI_ISL_3414889, EPI_ISL_3415104, EPI_ISL_3415226, EPI_ISL_3426474, EPI_ISL_3446827, EPI_ISL_3447712, EPI_ISL_3453279, EPI_ISL_3457423, EPI_ISL_3459118, EPI_ISL_3471360, EPI_ISL_3475993, EPI_ISL_3497667, EPI_ISL_3503811, EPI_ISL_3504036, EPI_ISL_3509013, EPI_ISL_3556945, EPI_ISL_3634003, EPI_ISL_3634004, EPI_ISL_3635506, EPI_ISL_3657112, EPI_ISL_3666069, EPI_ISL_3712919, EPI_ISL_3771882, EPI_ISL_3779849, EPI_ISL_3813731, EPI_ISL_3838306, EPI_ISL_3891136, EPI_ISL_3933252, EPI_ISL_3937027, EPI_ISL_3957778, EPI_ISL_3958461, EPI_ISL_3958994, EPI_ISL_3982251, EPI_ISL_4029567, EPI_ISL_4052911, EPI_ISL_4072038, EPI_ISL_4096626, EPI_ISL_4096639, EPI_ISL_4114033, EPI_ISL_4124532, EPI_ISL_4178790, EPI_ISL_4193135, EPI_ISL_4198270, EPI_ISL_4203869, EPI_ISL_4251611, EPI_ISL_4261403, EPI_ISL_4261411, EPI_ISL_4281194, EPI_ISL_4284228, EPI_ISL_4295678, EPI_ISL_4298277, EPI_ISL_4298278, EPI_ISL_4298279, EPI_ISL_4309817, EPI_ISL_4415808, EPI_ISL_4440075, EPI_ISL_4515444, EPI_ISL_4525691, EPI_ISL_4525698, EPI_ISL_4525700, EPI_ISL_4531313, EPI_ISL_4536418, EPI_ISL_4576991, EPI_ISL_4577474, EPI_ISL_4625101, EPI_ISL_4652284, EPI_ISL_4769386, EPI_ISL_4775547, EPI_ISL_4875939, EPI_ISL_4930863, EPI_ISL_4935777, EPI_ISL_4935949, EPI_ISL_4936095, EPI_ISL_4936533, EPI_ISL_4949584, EPI_ISL_5018695, EPI_ISL_5033183, EPI_ISL_5056434, EPI_ISL_5059980, EPI_ISL_5099310, EPI_ISL_5132437, EPI_ISL_5132595, EPI_ISL_5145656, EPI_ISL_5196003, EPI_ISL_5265214, EPI_ISL_5280146, EPI_ISL_5307398, EPI_ISL_5323016, EPI_ISL_5332877, EPI_ISL_5332878, EPI_ISL_5395558, EPI_ISL_5446154, EPI_ISL_5463914, EPI_ISL_5532714, EPI_ISL_5592605, EPI_ISL_5620309, EPI_ISL_5621224, EPI_ISL_5627313, EPI_ISL_5628248, EPI_ISL_5639913, EPI_ISL_5640459, EPI_ISL_5649323, EPI_ISL_5650474, EPI_ISL_5655562, EPI_ISL_5680241, EPI_ISL_5692774, EPI_ISL_5749185, EPI_ISL_5780324, EPI_ISL_5814411, EPI_ISL_5865553, EPI_ISL_5892132, EPI_ISL_5922347, EPI_ISL_5935407, EPI_ISL_5944665, EPI_ISL_5944669, EPI_ISL_5944842, EPI_ISL_5944948, EPI_ISL_5946914, EPI_ISL_6017746, EPI_ISL_6017747, EPI_ISL_6076460, EPI_ISL_6208674, EPI_ISL_6208675, EPI_ISL_6208676, EPI_ISL_6227177, EPI_ISL_6227208, EPI_ISL_6262165, EPI_ISL_6281381, EPI_ISL_6324366, EPI_ISL_6327943, EPI_ISL_6381841, EPI_ISL_6384755, EPI_ISL_6574278, EPI_ISL_6584511, EPI_ISL_6604686, EPI_ISL_6605003, EPI_ISL_6605659, EPI_ISL_6628662, EPI_ISL_6642599, EPI_ISL_6666037, EPI_ISL_6710721, EPI_ISL_6735468, EPI_ISL_6737833, EPI_ISL_6757093, EPI_ISL_6783610, EPI_ISL_6810267, EPI_ISL_6814822, EPI_ISL_6826536, EPI_ISL_6842893, EPI_ISL_6863316, EPI_ISL_6863457, EPI_ISL_6865741, EPI_ISL_6930836, EPI_ISL_6938691, EPI_ISL_6976497, EPI_ISL_6977941, EPI_ISL_7000663, EPI_ISL_7015624, EPI_ISL_7015625, EPI_ISL_7135374, EPI_ISL_7159687, EPI_ISL_7175748, EPI_ISL_7204318, EPI_ISL_7334013, EPI_ISL_7361483, EPI_ISL_7361527, EPI_ISL_7452581, EPI_ISL_7452603, EPI_ISL_7456462, EPI_ISL_7458003, EPI_ISL_7502990, EPI_ISL_7503221, EPI_ISL_7507393, EPI_ISL_7592661, EPI_ISL_7592687, EPI_ISL_7652115, EPI_ISL_7660179, EPI_ISL_7707631, EPI_ISL_7711813, EPI_ISL_7729239, EPI_ISL_7806535, EPI_ISL_7806536, EPI_ISL_7806549, EPI_ISL_7813896, EPI_ISL_7861580, EPI_ISL_7908114, EPI_ISL_7961502, EPI_ISL_7976931, EPI_ISL_7980711, EPI_ISL_8001538, EPI_ISL_8035582, EPI_ISL_8131224, EPI_ISL_8132253, EPI_ISL_8151798, EPI_ISL_8153087, EPI_ISL_8166542, EPI_ISL_8189765, EPI_ISL_8189775, EPI_ISL_8204828, EPI_ISL_8205040, EPI_ISL_8207600, EPI_ISL_8215783, EPI_ISL_8215787, EPI_ISL_8251200, EPI_ISL_8263463, EPI_ISL_8376888, EPI_ISL_8479639, EPI_ISL_8479640, EPI_ISL_8563217, EPI_ISL_8563218, EPI_ISL_8563219, EPI_ISL_8615077, EPI_ISL_8627379, EPI_ISL_8669281, EPI_ISL_8712661, EPI_ISL_8725398, EPI_ISL_8725399, EPI_ISL_8725400, EPI_ISL_8725401, EPI_ISL_8725402, EPI_ISL_8725403, EPI_ISL_8725404, EPI_ISL_8725405, EPI_ISL_8725406, EPI_ISL_8725407, EPI_ISL_8725408, EPI_ISL_8725409, EPI_ISL_8732699, EPI_ISL_8732807, EPI_ISL_8732841, EPI_ISL_8750545, EPI_ISL_8754305, EPI_ISL_8766992, EPI_ISL_8800409, EPI_ISL_8806077, EPI_ISL_8806082, EPI_ISL_8806084, EPI_ISL_8819629, EPI_ISL_8825833, EPI_ISL_8828662, EPI_ISL_8887845, EPI_ISL_8887874, EPI_ISL_8973278, EPI_ISL_9021214, EPI_ISL_9141923, EPI_ISL_9155607, EPI_ISL_9201951, EPI_ISL_9242265, EPI_ISL_9242269, EPI_ISL_9316982, EPI_ISL_9319180, EPI_ISL_9570633, EPI_ISL_9630717, EPI_ISL_9637481, EPI_ISL_9637840, EPI_ISL_9702285, EPI_ISL_9735644, EPI_ISL_9844246, EPI_ISL_9869512, EPI_ISL_9873278, EPI_ISL_9907655, EPI_ISL_9949797, EPI_ISL_9949799, EPI_ISL_10124646, EPI_ISL_10127751, EPI_ISL_10128185, EPI_ISL_10132276, EPI_ISL_10185453, EPI_ISL_10195257, EPI_ISL_10195262, EPI_ISL_10195263, EPI_ISL_10195264, EPI_ISL_10195305, EPI_ISL_10230612, EPI_ISL_10239201, EPI_ISL_10251304, EPI_ISL_10329391, EPI_ISL_10329558, EPI_ISL_10352747, EPI_ISL_10397517, EPI_ISL_10451205, EPI_ISL_10451252, EPI_ISL_10548912, EPI_ISL_10548913, EPI_ISL_10548915, EPI_ISL_10548916, EPI_ISL_10548917, EPI_ISL_10548918, EPI_ISL_10548919, EPI_ISL_10548920, EPI_ISL_10548921, EPI_ISL_10548922, EPI_ISL_10549162, EPI_ISL_10549163, EPI_ISL_10549164, EPI_ISL_10549165, EPI_ISL_10549166, EPI_ISL_10590270, EPI_ISL_10590760, EPI_ISL_10591808, EPI_ISL_10657890, EPI_ISL_10681118, EPI_ISL_10706292, EPI_ISL_10712909, EPI_ISL_10717525, EPI_ISL_10815044, EPI_ISL_10816730, EPI_ISL_10816731, EPI_ISL_10816732, EPI_ISL_10816733, EPI_ISL_10816734, EPI_ISL_10816735, EPI_ISL_10816736, EPI_ISL_10816737, EPI_ISL_10816738, EPI_ISL_10816739, EPI_ISL_10816741, EPI_ISL_10816742, EPI_ISL_10816743, EPI_ISL_10816744, EPI_ISL_10824028, EPI_ISL_10876034, EPI_ISL_10876749, EPI_ISL_10899907, EPI_ISL_10942195, EPI_ISL_10981395, EPI_ISL_10995323, EPI_ISL_10998528, EPI_ISL_11025821, EPI_ISL_11025897, EPI_ISL_11030507, EPI_ISL_11036385, EPI_ISL_11036386, EPI_ISL_11036389, EPI_ISL_11036399, EPI_ISL_11036451, EPI_ISL_11036666, EPI_ISL_11036688, EPI_ISL_11036712, EPI_ISL_11036915, EPI_ISL_11036917, EPI_ISL_11050902, EPI_ISL_11055609, EPI_ISL_11106543, EPI_ISL_11110730, EPI_ISL_11167907, EPI_ISL_11173072, EPI_ISL_11219235, EPI_ISL_11219236, EPI_ISL_11221773, EPI_ISL_11221782, EPI_ISL_11222620, EPI_ISL_11229672, EPI_ISL_11239958, EPI_ISL_11242266, EPI_ISL_11248919, EPI_ISL_11256669, EPI_ISL_11290054, EPI_ISL_11295642, EPI_ISL_11296415, EPI_ISL_11349763, EPI_ISL_11356268, EPI_ISL_11356269, EPI_ISL_11403393, EPI_ISL_11424263, EPI_ISL_11424578, EPI_ISL_11437359, EPI_ISL_11482304, EPI_ISL_11503909, EPI_ISL_11504189, EPI_ISL_11517385, EPI_ISL_11565840, EPI_ISL_11576757, EPI_ISL_11600821, EPI_ISL_11600822, EPI_ISL_11621597, EPI_ISL_11657715, EPI_ISL_11661806, EPI_ISL_11695384, EPI_ISL_11742572, EPI_ISL_11742812, EPI_ISL_11747289, EPI_ISL_11778548, EPI_ISL_11787443, EPI_ISL_11798407, EPI_ISL_11798458, EPI_ISL_11801453, EPI_ISL_11816904, EPI_ISL_11825798, EPI_ISL_11826326, EPI_ISL_11826898, EPI_ISL_11871462, EPI_ISL_11886470, EPI_ISL_11886479, EPI_ISL_11886624, EPI_ISL_11897546, EPI_ISL_11919916, EPI_ISL_11961223, EPI_ISL_11968830, EPI_ISL_11968876, EPI_ISL_11970393, EPI_ISL_11976211, EPI_ISL_11992954, EPI_ISL_11994571, EPI_ISL_11995938, EPI_ISL_12001179, EPI_ISL_12001180, EPI_ISL_12021469, EPI_ISL_12039060, EPI_ISL_12060087, EPI_ISL_12063598, EPI_ISL_12063599, EPI_ISL_12063600, EPI_ISL_12063601, EPI_ISL_12063602, EPI_ISL_12079999, EPI_ISL_12080356, EPI_ISL_12083619, EPI_ISL_12089943, EPI_ISL_12097355, EPI_ISL_12108965, EPI_ISL_12109074, EPI_ISL_12127282, EPI_ISL_12137683, EPI_ISL_12139045, EPI_ISL_12139066, EPI_ISL_12145506, EPI_ISL_12146579, EPI_ISL_12146859, EPI_ISL_12146865, EPI_ISL_12146872, EPI_ISL_12148419, EPI_ISL_12150077, EPI_ISL_12150259, EPI_ISL_12150361, EPI_ISL_12150484, EPI_ISL_12155759, EPI_ISL_12155809, EPI_ISL_12157165, EPI_ISL_12157166, EPI_ISL_12157187, EPI_ISL_12158960, EPI_ISL_12162547, EPI_ISL_12168418, EPI_ISL_12171333, EPI_ISL_12171674, EPI_ISL_12172132, EPI_ISL_12172842, EPI_ISL_12173486, EPI_ISL_12173730, EPI_ISL_12173879, EPI_ISL_12174612, EPI_ISL_12174734, EPI_ISL_12174735, EPI_ISL_12174736, EPI_ISL_12174739, EPI_ISL_12174942, EPI_ISL_12175020, EPI_ISL_12175024, EPI_ISL_12175203, EPI_ISL_12176184, EPI_ISL_12207682, EPI_ISL_12220762, EPI_ISL_12240087, EPI_ISL_12240983, EPI_ISL_12251561, EPI_ISL_12259859, EPI_ISL_12261629, EPI_ISL_12268492, EPI_ISL_12278477, EPI_ISL_12278678, EPI_ISL_12278997, EPI_ISL_12284821, EPI_ISL_12293549, EPI_ISL_12296891, EPI_ISL_12310532, EPI_ISL_12323992, EPI_ISL_12325408, EPI_ISL_12350967, EPI_ISL_12351281, EPI_ISL_12355622, EPI_ISL_12371147, EPI_ISL_12401928, EPI_ISL_12422504, EPI_ISL_12425033, EPI_ISL_12430022, EPI_ISL_12444737, EPI_ISL_12447557, EPI_ISL_12454820, EPI_ISL_12454840, EPI_ISL_12464077, EPI_ISL_12467081, EPI_ISL_12467157, EPI_ISL_12473693, EPI_ISL_12475004, EPI_ISL_12486436, EPI_ISL_12486761, EPI_ISL_12488441, EPI_ISL_12495863, EPI_ISL_12501519, EPI_ISL_12511246, EPI_ISL_12530780, EPI_ISL_12531462, EPI_ISL_12535815, EPI_ISL_12539663, EPI_ISL_12564261, EPI_ISL_12568162, EPI_ISL_12568208, EPI_ISL_12575298, EPI_ISL_12579825, EPI_ISL_12581914, EPI_ISL_12590958, EPI_ISL_12602903, EPI_ISL_12611697, EPI_ISL_12611721, EPI_ISL_12616586, EPI_ISL_12622901, EPI_ISL_12622902, EPI_ISL_12623284, EPI_ISL_12628520, EPI_ISL_12639714, EPI_ISL_12639788, EPI_ISL_12639917, EPI_ISL_12645823, EPI_ISL_12646116, EPI_ISL_12646785, EPI_ISL_12647336, EPI_ISL_12652423, EPI_ISL_12654179, EPI_ISL_12661097, EPI_ISL_12663222, EPI_ISL_12664764, EPI_ISL_12680798, EPI_ISL_12685124, EPI_ISL_12685126, EPI_ISL_12691923, EPI_ISL_12698937, EPI_ISL_12701772, EPI_ISL_12701820, EPI_ISL_12701858, EPI_ISL_12701867, EPI_ISL_12701871, EPI_ISL_12701895, EPI_ISL_12703517, EPI_ISL_12707647, EPI_ISL_12710394, EPI_ISL_12717870, EPI_ISL_12735772, EPI_ISL_12739317, EPI_ISL_12739425, EPI_ISL_12741485, EPI_ISL_12742126, EPI_ISL_12742222, EPI_ISL_12744193, EPI_ISL_12753427, EPI_ISL_12754976, EPI_ISL_12763802, EPI_ISL_12765459, EPI_ISL_12771614, EPI_ISL_12784028, EPI_ISL_12789812, EPI_ISL_12789846, EPI_ISL_12799990, EPI_ISL_12805921, EPI_ISL_12808264, EPI_ISL_12809016, EPI_ISL_12811012, EPI_ISL_12822481, EPI_ISL_12822483, EPI_ISL_12829036, EPI_ISL_12830215, EPI_ISL_12830827, EPI_ISL_12843060, EPI_ISL_12844170, EPI_ISL_12851188, EPI_ISL_12851233, EPI_ISL_12851285, EPI_ISL_12862705, EPI_ISL_12862970, EPI_ISL_12871249, EPI_ISL_12879252, EPI_ISL_12892238, EPI_ISL_12892312, EPI_ISL_12892482, EPI_ISL_12896994, EPI_ISL_12903760, EPI_ISL_12906172, EPI_ISL_12911895, EPI_ISL_12911898, EPI_ISL_12914019, EPI_ISL_12926555, EPI_ISL_12926955, EPI_ISL_12932770, EPI_ISL_12953136, EPI_ISL_12953419, EPI_ISL_12955795, EPI_ISL_12958668, EPI_ISL_12961699, EPI_ISL_12980420, EPI_ISL_12993020, EPI_ISL_12995035, EPI_ISL_12995123, EPI_ISL_12995230, EPI_ISL_12995281, EPI_ISL_12995422, EPI_ISL_12995533, EPI_ISL_13000815, EPI_ISL_13001148, EPI_ISL_13011225, EPI_ISL_13011226, EPI_ISL_13019919, EPI_ISL_13026163, EPI_ISL_13028133, EPI_ISL_13040401, EPI_ISL_13044133, EPI_ISL_13047387, EPI_ISL_13050078, EPI_ISL_13050270, EPI_ISL_13050793, EPI_ISL_13051431, EPI_ISL_13051740, EPI_ISL_13051790, EPI_ISL_13051970, EPI_ISL_13052096, EPI_ISL_13052204, EPI_ISL_13055324, EPI_ISL_13055618, EPI_ISL_13056142, EPI_ISL_13056190, EPI_ISL_13065554, EPI_ISL_13066396, EPI_ISL_13066472, EPI_ISL_13066603, EPI_ISL_13073691, EPI_ISL_13085784, EPI_ISL_13085991, EPI_ISL_13086417, EPI_ISL_13086737, EPI_ISL_13086826, EPI_ISL_13086831, EPI_ISL_13088769, EPI_ISL_13088942, EPI_ISL_13089020, EPI_ISL_13089384, EPI_ISL_13091908, EPI_ISL_13091912, EPI_ISL_13091925, EPI_ISL_13091929, EPI_ISL_13092725, EPI_ISL_13093369, EPI_ISL_13093922, EPI_ISL_13094368, EPI_ISL_13108591, EPI_ISL_13110029, EPI_ISL_13110031, EPI_ISL_13123133, EPI_ISL_13129353, EPI_ISL_13129387, EPI_ISL_13131091, EPI_ISL_13131118, EPI_ISL_13131514, EPI_ISL_13132070, EPI_ISL_13133128, EPI_ISL_13133359, EPI_ISL_13152570, EPI_ISL_13155404, EPI_ISL_13157537, EPI_ISL_13157638, EPI_ISL_13158312, EPI_ISL_13158314, EPI_ISL_13160040, EPI_ISL_13166402, EPI_ISL_13166803, EPI_ISL_13169047, EPI_ISL_13169050, EPI_ISL_13172328, EPI_ISL_13172329, EPI_ISL_13176279, EPI_ISL_13176281, EPI_ISL_13178754, EPI_ISL_13183984, EPI_ISL_13187136, EPI_ISL_13192066, EPI_ISL_13192072, EPI_ISL_13192202, EPI_ISL_13199746, EPI_ISL_13199759, EPI_ISL_13202578, EPI_ISL_13210987, EPI_ISL_13211404, EPI_ISL_13214299, EPI_ISL_13215742, EPI_ISL_13230467, EPI_ISL_13231553, EPI_ISL_13242111, EPI_ISL_13242155, EPI_ISL_13244164, EPI_ISL_13244243, EPI_ISL_13244707, EPI_ISL_13251406, EPI_ISL_13251514, EPI_ISL_13251524, EPI_ISL_13253132, EPI_ISL_13253164, EPI_ISL_13253244, EPI_ISL_13253296, EPI_ISL_13253344, EPI_ISL_13253353, EPI_ISL_13253416, EPI_ISL_13253427, EPI_ISL_13253489, EPI_ISL_13253493, EPI_ISL_13253550, EPI_ISL_13253551, EPI_ISL_13253570, EPI_ISL_13264387, EPI_ISL_13271922, EPI_ISL_13272001, EPI_ISL_13272223, EPI_ISL_13273550, EPI_ISL_13284168, EPI_ISL_13288721, EPI_ISL_13289213, EPI_ISL_13289619, EPI_ISL_13289774, EPI_ISL_13294595, EPI_ISL_13299119, EPI_ISL_13304429, EPI_ISL_13304451, EPI_ISL_13304482, EPI_ISL_13306995, EPI_ISL_13307081, EPI_ISL_13312837, EPI_ISL_13314960, EPI_ISL_13317150, EPI_ISL_13317160, EPI_ISL_13318832, EPI_ISL_13319769, EPI_ISL_13320180, EPI_ISL_13322028, EPI_ISL_13322954, EPI_ISL_13322975, EPI_ISL_13328732, EPI_ISL_13329792, EPI_ISL_13332433, EPI_ISL_13332459, EPI_ISL_13332921, EPI_ISL_13333047, EPI_ISL_13338249, EPI_ISL_13339980, EPI_ISL_13345908, EPI_ISL_13349432, EPI_ISL_13350581, EPI_ISL_13354243, EPI_ISL_13354366, EPI_ISL_13356160, EPI_ISL_13356194, EPI_ISL_13357233, EPI_ISL_13358809, EPI_ISL_13358893, EPI_ISL_13358894, EPI_ISL_13358962, EPI_ISL_13361313, EPI_ISL_13361419, EPI_ISL_13362130, EPI_ISL_13368501, EPI_ISL_13368552, EPI_ISL_13369326, EPI_ISL_13371824, EPI_ISL_13376289, EPI_ISL_13385215, EPI_ISL_13386427, EPI_ISL_13388894, EPI_ISL_13389863, EPI_ISL_13389864, EPI_ISL_13394010, EPI_ISL_13398372, EPI_ISL_13398391, EPI_ISL_13400530, EPI_ISL_13405240, EPI_ISL_13406133, EPI_ISL_13407391, EPI_ISL_13407748, EPI_ISL_13407802, EPI_ISL_13408380, EPI_ISL_13410054, EPI_ISL_13410128, EPI_ISL_13412509, EPI_ISL_13417422, EPI_ISL_13417595, EPI_ISL_13417637, EPI_ISL_13422063, EPI_ISL_13425805, EPI_ISL_13426860, EPI_ISL_13426861, EPI_ISL_13426862, EPI_ISL_13440246, EPI_ISL_13440269, EPI_ISL_13440488, EPI_ISL_13443257, EPI_ISL_13454711, EPI_ISL_13455972, EPI_ISL_13463270, EPI_ISL_13464004, EPI_ISL_13465109, EPI_ISL_13466588, EPI_ISL_13466629, EPI_ISL_13466644, EPI_ISL_13467321, EPI_ISL_13467676, EPI_ISL_13470130, EPI_ISL_13470158, EPI_ISL_13476507, EPI_ISL_13477158, EPI_ISL_13478276, EPI_ISL_13478413, EPI_ISL_13478421, EPI_ISL_13478425, EPI_ISL_13478448, EPI_ISL_13478487, EPI_ISL_13480851, EPI_ISL_13481704, EPI_ISL_13482848, EPI_ISL_13483067, EPI_ISL_13483538, EPI_ISL_13483605, EPI_ISL_13483653, EPI_ISL_13489191, EPI_ISL_13501768, EPI_ISL_13502856, EPI_ISL_13502894, EPI_ISL_13503354, EPI_ISL_13503413, EPI_ISL_13504084, EPI_ISL_13504103, EPI_ISL_13504568, EPI_ISL_13518485, EPI_ISL_13519541, EPI_ISL_13522554, EPI_ISL_13522707, EPI_ISL_13529086, EPI_ISL_13534848, EPI_ISL_13535844, EPI_ISL_13536846, EPI_ISL_13536992, EPI_ISL_13538620, EPI_ISL_13538621, EPI_ISL_13539746, EPI_ISL_13552269, EPI_ISL_13560139, EPI_ISL_13563849, EPI_ISL_13563900, EPI_ISL_13564453, EPI_ISL_13564901, EPI_ISL_13566717, EPI_ISL_13571629, EPI_ISL_13572579, EPI_ISL_13572829, EPI_ISL_13573543, EPI_ISL_13573707, EPI_ISL_13578717, EPI_ISL_13585459, EPI_ISL_13585850, EPI_ISL_13592668, EPI_ISL_13605240, EPI_ISL_13605930, EPI_ISL_13608346, EPI_ISL_13611346, EPI_ISL_13611373, EPI_ISL_13612632, EPI_ISL_13614328, EPI_ISL_13615804, EPI_ISL_13615826, EPI_ISL_13616458, EPI_ISL_13617390, EPI_ISL_13617475, EPI_ISL_13617493, EPI_ISL_13619646, EPI_ISL_13622657, EPI_ISL_13622831, EPI_ISL_13622874, EPI_ISL_13624440, EPI_ISL_13624441, EPI_ISL_13626226, EPI_ISL_13633558, EPI_ISL_13633729, EPI_ISL_13636934, EPI_ISL_13637141, EPI_ISL_13637734, EPI_ISL_13638494, EPI_ISL_13642059, EPI_ISL_13653660, EPI_ISL_13665096, EPI_ISL_13665110, EPI_ISL_13666764, EPI_ISL_13677872, EPI_ISL_13688333, EPI_ISL_13691966, EPI_ISL_13692397, EPI_ISL_13694663, EPI_ISL_13696042, EPI_ISL_13698648, EPI_ISL_13700128, EPI_ISL_13700243, EPI_ISL_13700756, EPI_ISL_13701029, EPI_ISL_13701810, EPI_ISL_13710357, EPI_ISL_13715129, EPI_ISL_13715746, EPI_ISL_13716400, EPI_ISL_13716943, EPI_ISL_13723111, EPI_ISL_13723805, EPI_ISL_13726415, EPI_ISL_13726482, EPI_ISL_13729045, EPI_ISL_13734474, EPI_ISL_13734683, EPI_ISL_13738059, EPI_ISL_13740111, EPI_ISL_13740674, EPI_ISL_13741330, EPI_ISL_13744798, EPI_ISL_13744799, EPI_ISL_13745638, EPI_ISL_13745641, EPI_ISL_13748166, EPI_ISL_13750726, EPI_ISL_13750730, EPI_ISL_13750760, EPI_ISL_13750771, EPI_ISL_13750936, EPI_ISL_13750937, EPI_ISL_13751254, EPI_ISL_13755996, EPI_ISL_13757795, EPI_ISL_13757902, EPI_ISL_13757914, EPI_ISL_13759674, EPI_ISL_13759811, EPI_ISL_13760929, EPI_ISL_13762803, EPI_ISL_13764852, EPI_ISL_13765234, EPI_ISL_13769441, EPI_ISL_13774980, EPI_ISL_13776103, EPI_ISL_13776118, EPI_ISL_13778691, EPI_ISL_13780813, EPI_ISL_13788917, EPI_ISL_13794761, EPI_ISL_13795106, EPI_ISL_13795308, EPI_ISL_13802466, EPI_ISL_13805350, EPI_ISL_13806026, EPI_ISL_13806197, EPI_ISL_13810913, EPI_ISL_13812067, EPI_ISL_13824458, EPI_ISL_13826362, EPI_ISL_13830194, EPI_ISL_13830195, EPI_ISL_13830196, EPI_ISL_13830197, EPI_ISL_13830445, EPI_ISL_13830454, EPI_ISL_13839105, EPI_ISL_13839285, EPI_ISL_13842068, EPI_ISL_13844161, EPI_ISL_13850726, EPI_ISL_13855446, EPI_ISL_13856866, EPI_ISL_13858143, EPI_ISL_13858664, EPI_ISL_13860426, EPI_ISL_13860879, EPI_ISL_13866687, EPI_ISL_13866688, EPI_ISL_13866691, EPI_ISL_13867130, EPI_ISL_13867135, EPI_ISL_13871326, EPI_ISL_13873100, EPI_ISL_13875348, EPI_ISL_13875677, EPI_ISL_13876290, EPI_ISL_13876318, EPI_ISL_13876612, EPI_ISL_13876760, EPI_ISL_13881123, EPI_ISL_13884353, EPI_ISL_13884360, EPI_ISL_13884439, EPI_ISL_13889482, EPI_ISL_13891697, EPI_ISL_13896135, EPI_ISL_13896156, EPI_ISL_13896578, EPI_ISL_13900930, EPI_ISL_13907795, EPI_ISL_13907925, EPI_ISL_13908668, EPI_ISL_13913047, EPI_ISL_13915527, EPI_ISL_13915530, EPI_ISL_13931117, EPI_ISL_13931975, EPI_ISL_13931997, EPI_ISL_13932039, EPI_ISL_13932080, EPI_ISL_13937613, EPI_ISL_13939223, EPI_ISL_13947566, EPI_ISL_13948007, EPI_ISL_13951654, EPI_ISL_13957684, EPI_ISL_13958481, EPI_ISL_13958566, EPI_ISL_13963277, EPI_ISL_13963776, EPI_ISL_13963832, EPI_ISL_13967204, EPI_ISL_13968198, EPI_ISL_13970237, EPI_ISL_13970242, EPI_ISL_13970249, EPI_ISL_13970257, EPI_ISL_13970276, EPI_ISL_13975822, EPI_ISL_13981101, EPI_ISL_13984460, EPI_ISL_13986492, EPI_ISL_13986494, EPI_ISL_13986497, EPI_ISL_13986498, EPI_ISL_13986501, EPI_ISL_13989484, EPI_ISL_13991375, EPI_ISL_13993708, EPI_ISL_13994781, EPI_ISL_14000155, EPI_ISL_14004519, EPI_ISL_14005794, EPI_ISL_14010460, EPI_ISL_14010462, EPI_ISL_14011475, EPI_ISL_14015047, EPI_ISL_14019093, EPI_ISL_14019109, EPI_ISL_14019330, EPI_ISL_14020697, EPI_ISL_14022780, EPI_ISL_14022892, EPI_ISL_14023662, EPI_ISL_14027304, EPI_ISL_14027788, EPI_ISL_14028215, EPI_ISL_14029314, EPI_ISL_14029803, EPI_ISL_14030175, EPI_ISL_14032717, EPI_ISL_14035970, EPI_ISL_14036069, EPI_ISL_14041069, EPI_ISL_14044698, EPI_ISL_14044704, EPI_ISL_14046291, EPI_ISL_14047361, EPI_ISL_14050544, EPI_ISL_14051041, EPI_ISL_14051049, EPI_ISL_14051052, EPI_ISL_14057503, EPI_ISL_14062117, EPI_ISL_14064598, EPI_ISL_14064601, EPI_ISL_14066591, EPI_ISL_14066852, EPI_ISL_14071587, EPI_ISL_14071795, EPI_ISL_14097542, EPI_ISL_14097629, EPI_ISL_14124074, EPI_ISL_14127563, EPI_ISL_14128514, EPI_ISL_14134678, EPI_ISL_14135854, EPI_ISL_14147202, EPI_ISL_14155218, EPI_ISL_14158264, EPI_ISL_14161024, EPI_ISL_14170603, EPI_ISL_14171301, EPI_ISL_14172905, EPI_ISL_14173767, EPI_ISL_14175092, EPI_ISL_14175097, EPI_ISL_14175103, EPI_ISL_14175182, EPI_ISL_14181740, EPI_ISL_14193000, EPI_ISL_14193833, EPI_ISL_14196068, EPI_ISL_14196701, EPI_ISL_14197724, EPI_ISL_14198080, EPI_ISL_14200342, EPI_ISL_14200801, EPI_ISL_14203206, EPI_ISL_14203613, EPI_ISL_14208740, EPI_ISL_14208835, EPI_ISL_14209372, EPI_ISL_14209934, EPI_ISL_14210619, EPI_ISL_14211976, EPI_ISL_14215014, EPI_ISL_14215818, EPI_ISL_14216595, EPI_ISL_14217745, EPI_ISL_14222817, EPI_ISL_14223595, EPI_ISL_14223736, EPI_ISL_14223750, EPI_ISL_14224871, EPI_ISL_14225747, EPI_ISL_14226883, EPI_ISL_14228030, EPI_ISL_14229584, EPI_ISL_14230429, EPI_ISL_14230544, EPI_ISL_14231739, EPI_ISL_14231749, EPI_ISL_14231751, EPI_ISL_14232221, EPI_ISL_14236377, EPI_ISL_14241722, EPI_ISL_14243471, EPI_ISL_14243503, EPI_ISL_14243509, EPI_ISL_14249979, EPI_ISL_14257394, EPI_ISL_14257905, EPI_ISL_14259114, EPI_ISL_14259141, EPI_ISL_14259905, EPI_ISL_14260215, EPI_ISL_14261704, EPI_ISL_14262778, EPI_ISL_14263077, EPI_ISL_14277057, EPI_ISL_14285562, EPI_ISL_14287370, EPI_ISL_14288695, EPI_ISL_14289901, EPI_ISL_14289906, EPI_ISL_14292615, EPI_ISL_14292645, EPI_ISL_14292727, EPI_ISL_14292796, EPI_ISL_14296586, EPI_ISL_14298637, EPI_ISL_14299991, EPI_ISL_14301376, EPI_ISL_14305792, EPI_ISL_14311909, EPI_ISL_14311965, EPI_ISL_14312743, EPI_ISL_14321789, EPI_ISL_14321842, EPI_ISL_14329477, EPI_ISL_14340832, EPI_ISL_14353536, EPI_ISL_14356738, EPI_ISL_14359010, EPI_ISL_14377405, EPI_ISL_14377421, EPI_ISL_14382623, EPI_ISL_14386850, EPI_ISL_14387989, EPI_ISL_14389796, EPI_ISL_14391372, EPI_ISL_14393120, EPI_ISL_14393933, EPI_ISL_14409468, EPI_ISL_14416474, EPI_ISL_14417374, EPI_ISL_14425116, EPI_ISL_14425894, EPI_ISL_14426235, EPI_ISL_14426336, EPI_ISL_14430592, EPI_ISL_14433737, EPI_ISL_14434989, EPI_ISL_14436225, EPI_ISL_14437098, EPI_ISL_14439513, EPI_ISL_14439514, EPI_ISL_14439530, EPI_ISL_14439649, EPI_ISL_14439686, EPI_ISL_14440238, EPI_ISL_14448667, EPI_ISL_14455168, EPI_ISL_14459779, EPI_ISL_14462783, EPI_ISL_14464386, EPI_ISL_14467169, EPI_ISL_14469810, EPI_ISL_14470997, EPI_ISL_14471721, EPI_ISL_14478172, EPI_ISL_14478208, EPI_ISL_14479146, EPI_ISL_14479735, EPI_ISL_14483275, EPI_ISL_14485183, EPI_ISL_14485890, EPI_ISL_14487304, EPI_ISL_14487315, EPI_ISL_14490179, EPI_ISL_14493139, EPI_ISL_14493608, EPI_ISL_14493822, EPI_ISL_14493989, EPI_ISL_14494322, EPI_ISL_14496407, EPI_ISL_14496491, EPI_ISL_14496822, EPI_ISL_14497316, EPI_ISL_14498244, EPI_ISL_14502420, EPI_ISL_14503169, EPI_ISL_14503437, EPI_ISL_14504973, EPI_ISL_14505974, EPI_ISL_14507199, EPI_ISL_14507200, EPI_ISL_14508231, EPI_ISL_14509715, EPI_ISL_14513137, EPI_ISL_14518038, EPI_ISL_14518039, EPI_ISL_14518040, EPI_ISL_14518101, EPI_ISL_14518137, EPI_ISL_14523885, EPI_ISL_14527351, EPI_ISL_14527924, EPI_ISL_14535112, EPI_ISL_14540192, EPI_ISL_14544667, EPI_ISL_14545270, EPI_ISL_14551066, EPI_ISL_14552116, EPI_ISL_14553998, EPI_ISL_14554016, EPI_ISL_14554020, EPI_ISL_14554042, EPI_ISL_14554047, EPI_ISL_14556650, EPI_ISL_14560721, EPI_ISL_14560826, EPI_ISL_14560873, EPI_ISL_14561036, EPI_ISL_14561487, EPI_ISL_14562820, EPI_ISL_14571645, EPI_ISL_14572777, EPI_ISL_14573093, EPI_ISL_14574572, EPI_ISL_14577541, EPI_ISL_14577981, EPI_ISL_14578599, EPI_ISL_14583728, EPI_ISL_14596883, EPI_ISL_14599772, EPI_ISL_14599778, EPI_ISL_14599779, EPI_ISL_14599780, EPI_ISL_14602583, EPI_ISL_14602992, EPI_ISL_14606016, EPI_ISL_14608770, EPI_ISL_14610722, EPI_ISL_14610723, EPI_ISL_14610724, EPI_ISL_14610725, EPI_ISL_14610726, EPI_ISL_14610727, EPI_ISL_14610728, EPI_ISL_14610729, EPI_ISL_14610730, EPI_ISL_14610731, EPI_ISL_14610732, EPI_ISL_14610733, EPI_ISL_14610734, EPI_ISL_14613632, EPI_ISL_14613671, EPI_ISL_14616144, EPI_ISL_14616443, EPI_ISL_14616543, EPI_ISL_14616681, EPI_ISL_14618772, EPI_ISL_14619952, EPI_ISL_14623599, EPI_ISL_14624407, EPI_ISL_14625263, EPI_ISL_14630170, EPI_ISL_14647032, EPI_ISL_14650043, EPI_ISL_14652006, EPI_ISL_14665394, EPI_ISL_14666760, EPI_ISL_14666762, EPI_ISL_14666764, EPI_ISL_14666765, EPI_ISL_14666766, EPI_ISL_14667656, EPI_ISL_14667834, EPI_ISL_14669160, EPI_ISL_14676287, EPI_ISL_14681429, EPI_ISL_14683500, EPI_ISL_14687471, EPI_ISL_14691921, EPI_ISL_14694460, EPI_ISL_14695466, EPI_ISL_14699501, EPI_ISL_14699555, EPI_ISL_14700183, EPI_ISL_14700285, EPI_ISL_14701161, EPI_ISL_14701325, EPI_ISL_14702629, EPI_ISL_14706169, EPI_ISL_14707196, EPI_ISL_14707197, EPI_ISL_14707948, EPI_ISL_14710821, EPI_ISL_14710834, EPI_ISL_14711613, EPI_ISL_14711614, EPI_ISL_14715522, EPI_ISL_14721837, EPI_ISL_14721894, EPI_ISL_14722630, EPI_ISL_14725600, EPI_ISL_14727457, EPI_ISL_14728608, EPI_ISL_14728814, EPI_ISL_14732990, EPI_ISL_14744620, EPI_ISL_14744804, EPI_ISL_14744809, EPI_ISL_14745146, EPI_ISL_14746124, EPI_ISL_14746196, EPI_ISL_14746271, EPI_ISL_14746905, EPI_ISL_14747246, EPI_ISL_14747247, EPI_ISL_14747621, EPI_ISL_14752384, EPI_ISL_14754570, EPI_ISL_14755727, EPI_ISL_14755766, EPI_ISL_14755933, EPI_ISL_14762572, EPI_ISL_14763447, EPI_ISL_14763711, EPI_ISL_14763898, EPI_ISL_14765653, EPI_ISL_14766331, EPI_ISL_14766361, EPI_ISL_14766363, EPI_ISL_14766433, EPI_ISL_14770484, EPI_ISL_14771903, EPI_ISL_14772260, EPI_ISL_14773203, EPI_ISL_14773569, EPI_ISL_14773570, EPI_ISL_14773839, EPI_ISL_14778614, EPI_ISL_14780448, EPI_ISL_14785887, EPI_ISL_14786677, EPI_ISL_14788048, EPI_ISL_14788099, EPI_ISL_14788860, EPI_ISL_14789391, EPI_ISL_14789392, EPI_ISL_14789508, EPI_ISL_14791420, EPI_ISL_14793146, EPI_ISL_14793618, EPI_ISL_14796633, EPI_ISL_14804570, EPI_ISL_14806018, EPI_ISL_14806413, EPI_ISL_14809350, EPI_ISL_14811078, EPI_ISL_14812412, EPI_ISL_14813068, EPI_ISL_14813161, EPI_ISL_14813215, EPI_ISL_14813300, EPI_ISL_14813995, EPI_ISL_14816346, EPI_ISL_14817985, EPI_ISL_14832276, EPI_ISL_14832977, EPI_ISL_14836181, EPI_ISL_14837867, EPI_ISL_14838049, EPI_ISL_14841625, EPI_ISL_14842196, EPI_ISL_14845057, EPI_ISL_14847727, EPI_ISL_14856139, EPI_ISL_14859457, EPI_ISL_14859713, EPI_ISL_14859716, EPI_ISL_14859829, EPI_ISL_14862263, EPI_ISL_14886333, EPI_ISL_14888736, EPI_ISL_14890020, EPI_ISL_14891391, EPI_ISL_14891763, EPI_ISL_14891765, EPI_ISL_14892114, EPI_ISL_14892395, EPI_ISL_14892970, EPI_ISL_14896074, EPI_ISL_14901195, EPI_ISL_14901198, EPI_ISL_14901422, EPI_ISL_14901439, EPI_ISL_14901444, EPI_ISL_14901672, EPI_ISL_14901693, EPI_ISL_14903212, EPI_ISL_14904331, EPI_ISL_14907633, EPI_ISL_14912863, EPI_ISL_14913043, EPI_ISL_14916417, EPI_ISL_14917292, EPI_ISL_14919989, EPI_ISL_14920419, EPI_ISL_14921805, EPI_ISL_14922117, EPI_ISL_14922327, EPI_ISL_14922929, EPI_ISL_14922954, EPI_ISL_14924448, EPI_ISL_14925471, EPI_ISL_14925480, EPI_ISL_14925483, EPI_ISL_14925487, EPI_ISL_14926371, EPI_ISL_14929774, EPI_ISL_14931103, EPI_ISL_14934229, EPI_ISL_14934234, EPI_ISL_14935361, EPI_ISL_14935895, EPI_ISL_14935908, EPI_ISL_14935930, EPI_ISL_14935931, EPI_ISL_14937654, EPI_ISL_14937852, EPI_ISL_14937864, EPI_ISL_14942094, EPI_ISL_14942184, EPI_ISL_14942530, EPI_ISL_14943290, EPI_ISL_14945818, EPI_ISL_14946958, EPI_ISL_14949065, EPI_ISL_14950282, EPI_ISL_14950545, EPI_ISL_14951595, EPI_ISL_14951609, EPI_ISL_14951892, EPI_ISL_14952059, EPI_ISL_14952205, EPI_ISL_14952220, EPI_ISL_14953487, EPI_ISL_14955378, EPI_ISL_14959923, EPI_ISL_14960752, EPI_ISL_14960911, EPI_ISL_14961972, EPI_ISL_14962212, EPI_ISL_14962429, EPI_ISL_14962617, EPI_ISL_14971640, EPI_ISL_14975894, EPI_ISL_14977865, EPI_ISL_14980656, EPI_ISL_14992324, EPI_ISL_14993023, EPI_ISL_14994380, EPI_ISL_14995958, EPI_ISL_14997830, EPI_ISL_15003867, EPI_ISL_15005362, EPI_ISL_15005560, EPI_ISL_15010697, EPI_ISL_15012784, EPI_ISL_15013151, EPI_ISL_15013344, EPI_ISL_15014035, EPI_ISL_15014516, EPI_ISL_15015307, EPI_ISL_15017244, EPI_ISL_15018502, EPI_ISL_15020427, EPI_ISL_15022783, EPI_ISL_15022788, EPI_ISL_15024848, EPI_ISL_15026124, EPI_ISL_15030291, EPI_ISL_15030370, EPI_ISL_15032101, EPI_ISL_15036387, EPI_ISL_15038137, EPI_ISL_15040463, EPI_ISL_15040845, EPI_ISL_15040855, EPI_ISL_15040867, EPI_ISL_15044102, EPI_ISL_15047139, EPI_ISL_15048524, EPI_ISL_15050379, EPI_ISL_15051633, EPI_ISL_15058728, EPI_ISL_15058903, EPI_ISL_15071928, EPI_ISL_15072261, EPI_ISL_15072543, EPI_ISL_15072550, EPI_ISL_15072553, EPI_ISL_15072554, EPI_ISL_15072999, EPI_ISL_15075043, EPI_ISL_15075836, EPI_ISL_15076071, EPI_ISL_15077422, EPI_ISL_15078481, EPI_ISL_15080832, EPI_ISL_15083622, EPI_ISL_15084091, EPI_ISL_15085357, EPI_ISL_15085883, EPI_ISL_15085910, EPI_ISL_15086100, EPI_ISL_15086132, EPI_ISL_15086246, EPI_ISL_15088435, EPI_ISL_15088854, EPI_ISL_15090098, EPI_ISL_15093244, EPI_ISL_15093817, EPI_ISL_15093818, EPI_ISL_15094085, EPI_ISL_15096597, EPI_ISL_15096672, EPI_ISL_15098367, EPI_ISL_15101602, EPI_ISL_15107059, EPI_ISL_15107248, EPI_ISL_15107529, EPI_ISL_15108940, EPI_ISL_15108982, EPI_ISL_15109913, EPI_ISL_15111016, EPI_ISL_15111369, EPI_ISL_15114528, EPI_ISL_15114696, EPI_ISL_15116712, EPI_ISL_15118484, EPI_ISL_15119416, EPI_ISL_15120575, EPI_ISL_15125352, EPI_ISL_15126616, EPI_ISL_15129252, EPI_ISL_15137908, EPI_ISL_15137948, EPI_ISL_15140027, EPI_ISL_15140068, EPI_ISL_15145892, EPI_ISL_15145981, EPI_ISL_15156399, EPI_ISL_15157666, EPI_ISL_15159584, EPI_ISL_15160596, EPI_ISL_15161674, EPI_ISL_15169761, EPI_ISL_15169791, EPI_ISL_15170512, EPI_ISL_15171102, EPI_ISL_15172949, EPI_ISL_15173621, EPI_ISL_15175083, EPI_ISL_15175896, EPI_ISL_15177304, EPI_ISL_15177330, EPI_ISL_15177334, EPI_ISL_15178067, EPI_ISL_15178215, EPI_ISL_15184076, EPI_ISL_15184330, EPI_ISL_15191490, EPI_ISL_15191491, EPI_ISL_15191642, EPI_ISL_15191714, EPI_ISL_15191804, EPI_ISL_15193406, EPI_ISL_15195342, EPI_ISL_15195410, EPI_ISL_15195634, EPI_ISL_15195645, EPI_ISL_15195724, EPI_ISL_15198987, EPI_ISL_15211295, EPI_ISL_15211305, EPI_ISL_15213088, EPI_ISL_15215446, EPI_ISL_15216639, EPI_ISL_15218165, EPI_ISL_15222709, EPI_ISL_15229199, EPI_ISL_15231108, EPI_ISL_15236061, EPI_ISL_15236355, EPI_ISL_15241555, EPI_ISL_15247582, EPI_ISL_15248681, EPI_ISL_15250551, EPI_ISL_15251240, EPI_ISL_15251241, EPI_ISL_15251242, EPI_ISL_15251243, EPI_ISL_15257404, EPI_ISL_15264055, EPI_ISL_15268715, EPI_ISL_15268834, EPI_ISL_15271184, EPI_ISL_15273578, EPI_ISL_15275240, EPI_ISL_15278730, EPI_ISL_15278787, EPI_ISL_15279743, EPI_ISL_15284364, EPI_ISL_15284373, EPI_ISL_15284374, EPI_ISL_15284586, EPI_ISL_15286527, EPI_ISL_15287393, EPI_ISL_15288333, EPI_ISL_15292331, EPI_ISL_15294656, EPI_ISL_15296403, EPI_ISL_15305368, EPI_ISL_15305862, EPI_ISL_15306067, EPI_ISL_15307010, EPI_ISL_15307651, EPI_ISL_15310561, EPI_ISL_15312119, EPI_ISL_15314949, EPI_ISL_15316398, EPI_ISL_15317884, EPI_ISL_15322793, EPI_ISL_15325687, EPI_ISL_15325946, EPI_ISL_15328668, EPI_ISL_15330077, EPI_ISL_15330418, EPI_ISL_15331994, EPI_ISL_15332094, EPI_ISL_15332912, EPI_ISL_15333310, EPI_ISL_15338015, EPI_ISL_15338081, EPI_ISL_15340355, EPI_ISL_15341321, EPI_ISL_15347054, EPI_ISL_15348674, EPI_ISL_15348926, EPI_ISL_15348927, EPI_ISL_15348928, EPI_ISL_15348929, EPI_ISL_15352149, EPI_ISL_15354679, EPI_ISL_15354775, EPI_ISL_15357057, EPI_ISL_15357237, EPI_ISL_15357960, EPI_ISL_15357988, EPI_ISL_15362650, EPI_ISL_15363544, EPI_ISL_15368893, EPI_ISL_15370137, EPI_ISL_15370885, EPI_ISL_15370889, EPI_ISL_15376124, EPI_ISL_15376348, EPI_ISL_15376376, EPI_ISL_15379719, EPI_ISL_15380518, EPI_ISL_15384507, EPI_ISL_15384545, EPI_ISL_15385232, EPI_ISL_15385971, EPI_ISL_15387248, EPI_ISL_15387296, EPI_ISL_15387509, EPI_ISL_15387686, EPI_ISL_15388463, EPI_ISL_15389278, EPI_ISL_15392529, EPI_ISL_15393294, EPI_ISL_15393302, EPI_ISL_15396381, EPI_ISL_15397576, EPI_ISL_15403852, EPI_ISL_15403865, EPI_ISL_15403898, EPI_ISL_15408226, EPI_ISL_15408697, EPI_ISL_15409673, EPI_ISL_15415649, EPI_ISL_15416349, EPI_ISL_15420040, EPI_ISL_15420212, EPI_ISL_15420431, EPI_ISL_15420631, EPI_ISL_15423234, EPI_ISL_15424211, EPI_ISL_15424884, EPI_ISL_15427653, EPI_ISL_15434634, EPI_ISL_15434919, EPI_ISL_15435185, EPI_ISL_15436140, EPI_ISL_15436494, EPI_ISL_15436498, EPI_ISL_15436499, EPI_ISL_15442625, EPI_ISL_15442735, EPI_ISL_15446553, EPI_ISL_15456143, EPI_ISL_15462878, EPI_ISL_15471419, EPI_ISL_15471420, EPI_ISL_15472394, EPI_ISL_15472759, EPI_ISL_15473981, EPI_ISL_15473994, EPI_ISL_15476105, EPI_ISL_15476158, EPI_ISL_15476180, EPI_ISL_15476724, EPI_ISL_15479591, EPI_ISL_15481002, EPI_ISL_15486348, EPI_ISL_15490572, EPI_ISL_15492743, EPI_ISL_15492887, EPI_ISL_15494260, EPI_ISL_15494897, EPI_ISL_15495028, EPI_ISL_15495125, EPI_ISL_15496641, EPI_ISL_15501222, EPI_ISL_15502864, EPI_ISL_15505215, EPI_ISL_15505985, EPI_ISL_15506333, EPI_ISL_15507204, EPI_ISL_15507296, EPI_ISL_15507616, EPI_ISL_15508143, EPI_ISL_15509746, EPI_ISL_15509755, EPI_ISL_15509992, EPI_ISL_15511119, EPI_ISL_15511841, EPI_ISL_15511842, EPI_ISL_15511843, EPI_ISL_15513583, EPI_ISL_15513588, EPI_ISL_15513663, EPI_ISL_15514216, EPI_ISL_15514265, EPI_ISL_15514302, EPI_ISL_15514923, EPI_ISL_15523458, EPI_ISL_15523591, EPI_ISL_15528152, EPI_ISL_15528174, EPI_ISL_15528328, EPI_ISL_15528329, EPI_ISL_15528330, EPI_ISL_15528331, EPI_ISL_15528332, EPI_ISL_15528333, EPI_ISL_15528334, EPI_ISL_15535800, EPI_ISL_15537619, EPI_ISL_15538513, EPI_ISL_15538645, EPI_ISL_15542503, EPI_ISL_15546790, EPI_ISL_15549778, EPI_ISL_15549981, EPI_ISL_15550525, EPI_ISL_15557544, EPI_ISL_15568780, EPI_ISL_15579728, EPI_ISL_15579786, EPI_ISL_15580359, EPI_ISL_15580699, EPI_ISL_15581446, EPI_ISL_15581681, EPI_ISL_15581931, EPI_ISL_15581932, EPI_ISL_15581939, EPI_ISL_15582076, EPI_ISL_15582517, EPI_ISL_15583376, EPI_ISL_15585338, EPI_ISL_15587950, EPI_ISL_15588132, EPI_ISL_15594682, EPI_ISL_15595518, EPI_ISL_15598104, EPI_ISL_15598966, EPI_ISL_15602198, EPI_ISL_15604595, EPI_ISL_15606951, EPI_ISL_15607872, EPI_ISL_15608835, EPI_ISL_15609106, EPI_ISL_15609107, EPI_ISL_15610726, EPI_ISL_15610881, EPI_ISL_15612047, EPI_ISL_15612048, EPI_ISL_15614101, EPI_ISL_15614116, EPI_ISL_15614150, EPI_ISL_15614383, EPI_ISL_15614456, EPI_ISL_15614490, EPI_ISL_15616889, EPI_ISL_15617621, EPI_ISL_15617635, EPI_ISL_15619675, EPI_ISL_15626705, EPI_ISL_15626859, EPI_ISL_15626955, EPI_ISL_15628252, EPI_ISL_15630041, EPI_ISL_15631537, EPI_ISL_15635022, EPI_ISL_15636641, EPI_ISL_15637121, EPI_ISL_15637182, EPI_ISL_15638597, EPI_ISL_15639067, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15649157, EPI_ISL_15650076, EPI_ISL_15650225, EPI_ISL_15653695, EPI_ISL_15654640, EPI_ISL_15656922, EPI_ISL_15659847, EPI_ISL_15661609, EPI_ISL_15666595, EPI_ISL_15667047, EPI_ISL_15669004, EPI_ISL_15671244, EPI_ISL_15671388, EPI_ISL_15671577, EPI_ISL_15671878, EPI_ISL_15671888, EPI_ISL_15673934, EPI_ISL_15675248, EPI_ISL_15678339, EPI_ISL_15681826, EPI_ISL_15685722, EPI_ISL_15685724, EPI_ISL_15685810, EPI_ISL_15687681, EPI_ISL_15687965, EPI_ISL_15688500, EPI_ISL_15692625, EPI_ISL_15693169, EPI_ISL_15693174, EPI_ISL_15693676, EPI_ISL_15700160, EPI_ISL_15703834, EPI_ISL_15705061, EPI_ISL_15712450, EPI_ISL_15712747, EPI_ISL_15715244, EPI_ISL_15719141, EPI_ISL_15719142, EPI_ISL_15719143, EPI_ISL_15720605, EPI_ISL_15721137, EPI_ISL_15721185, EPI_ISL_15721190, EPI_ISL_15723589, EPI_ISL_15725799, EPI_ISL_15728467, EPI_ISL_15728613, EPI_ISL_15728673, EPI_ISL_15729287, EPI_ISL_15729288, EPI_ISL_15729308, EPI_ISL_15729309, EPI_ISL_15729310, EPI_ISL_15729311, EPI_ISL_15729315, EPI_ISL_15729341, EPI_ISL_15729358, EPI_ISL_15731233, EPI_ISL_15731409, EPI_ISL_15732413, EPI_ISL_15736424, EPI_ISL_15739498, EPI_ISL_15739617, EPI_ISL_15741410, EPI_ISL_15742450, EPI_ISL_15743318, EPI_ISL_15743816, EPI_ISL_15749060, EPI_ISL_15754145, EPI_ISL_15754794, EPI_ISL_15758565, EPI_ISL_15758745, EPI_ISL_15760224, EPI_ISL_15760382, EPI_ISL_15760554, EPI_ISL_15760812, EPI_ISL_15761520, EPI_ISL_15761543, EPI_ISL_15761663, EPI_ISL_15763216, EPI_ISL_15765022, EPI_ISL_15768827, EPI_ISL_15773132, EPI_ISL_15773881, EPI_ISL_15773907, EPI_ISL_15776989, EPI_ISL_15778527, EPI_ISL_15779724, EPI_ISL_15780387, EPI_ISL_15781197, EPI_ISL_15781220, EPI_ISL_15781937, EPI_ISL_15785782, EPI_ISL_15786114, EPI_ISL_15786255, EPI_ISL_15789537, EPI_ISL_15790179, EPI_ISL_15790657, EPI_ISL_15791223, EPI_ISL_15791252, EPI_ISL_15792351, EPI_ISL_15793981, EPI_ISL_15797751, EPI_ISL_15798331, EPI_ISL_15801425, EPI_ISL_15801499, EPI_ISL_15801515, EPI_ISL_15803807, EPI_ISL_15815337, EPI_ISL_15815525, EPI_ISL_15815889, EPI_ISL_15818486, EPI_ISL_15820055, EPI_ISL_15822919, EPI_ISL_15824080, EPI_ISL_15824099, EPI_ISL_15824207, EPI_ISL_15825627, EPI_ISL_15826800, EPI_ISL_15826867, EPI_ISL_15829108, EPI_ISL_15837751, EPI_ISL_15837827, EPI_ISL_15838124, EPI_ISL_15839941, EPI_ISL_15843473, EPI_ISL_15843671, EPI_ISL_15844032, EPI_ISL_15844165, EPI_ISL_15845753, EPI_ISL_15845778, EPI_ISL_15845946, EPI_ISL_15846023, EPI_ISL_15846264, EPI_ISL_15849690, EPI_ISL_15850759, EPI_ISL_15850865, EPI_ISL_15850872, EPI_ISL_15853809, EPI_ISL_15853943, EPI_ISL_15856103, EPI_ISL_15856463, EPI_ISL_15856822, EPI_ISL_15857383, EPI_ISL_15857468, EPI_ISL_15860163, EPI_ISL_15864217, EPI_ISL_15864218, EPI_ISL_15865257, EPI_ISL_15865301, EPI_ISL_15865421, EPI_ISL_15865482, EPI_ISL_15866887, EPI_ISL_15873398, EPI_ISL_15874567, EPI_ISL_15878818, EPI_ISL_15882637, EPI_ISL_15883009, EPI_ISL_15883480, EPI_ISL_15887656, EPI_ISL_15894376, EPI_ISL_15895625, EPI_ISL_15896804, EPI_ISL_15896845, EPI_ISL_15896923, EPI_ISL_15897067, EPI_ISL_15897092, EPI_ISL_15898992, EPI_ISL_15900796, EPI_ISL_15905452, EPI_ISL_15905511, EPI_ISL_15911160, EPI_ISL_15912221, EPI_ISL_15912222, EPI_ISL_15912223, EPI_ISL_15912224, EPI_ISL_15914119, EPI_ISL_15917272, EPI_ISL_15920181, EPI_ISL_15920505, EPI_ISL_15920753, EPI_ISL_15920754, EPI_ISL_15920755, EPI_ISL_15926083, EPI_ISL_15926723, EPI_ISL_15928156, EPI_ISL_15928909, EPI_ISL_15929002, EPI_ISL_15929151, EPI_ISL_15932554, EPI_ISL_15934274, EPI_ISL_15937718, EPI_ISL_15938074, EPI_ISL_15941879, EPI_ISL_15941880, EPI_ISL_15945504, EPI_ISL_15949533, EPI_ISL_15953870, EPI_ISL_15955458, EPI_ISL_15958934, EPI_ISL_15961456, EPI_ISL_15962045, EPI_ISL_15965716, EPI_ISL_15966131, EPI_ISL_15966527, EPI_ISL_15969420, EPI_ISL_15969421, EPI_ISL_15969437, EPI_ISL_15969438, EPI_ISL_15970088, EPI_ISL_15970187, EPI_ISL_15976036, EPI_ISL_15982641, EPI_ISL_15984958, EPI_ISL_15992803, EPI_ISL_15998627, EPI_ISL_16001974, EPI_ISL_16001995, EPI_ISL_16005457, EPI_ISL_16006665, EPI_ISL_16007931, EPI_ISL_16008877, EPI_ISL_16012424, EPI_ISL_16012845, EPI_ISL_16013074, EPI_ISL_16013086, EPI_ISL_16015099, EPI_ISL_16017107, EPI_ISL_16018930, EPI_ISL_16019056, EPI_ISL_16024407, EPI_ISL_16024682, EPI_ISL_16027431, EPI_ISL_16027937, EPI_ISL_16029135, EPI_ISL_16029382, EPI_ISL_16029654, EPI_ISL_16030181, EPI_ISL_16033087, EPI_ISL_16033186, EPI_ISL_16039444, EPI_ISL_16043974, EPI_ISL_16044428, EPI_ISL_16045410, EPI_ISL_16046711, EPI_ISL_16050095, EPI_ISL_16050127, EPI_ISL_16054133, EPI_ISL_16054451, EPI_ISL_16054953, EPI_ISL_16054963, EPI_ISL_16055434, EPI_ISL_16055461, EPI_ISL_16055527, EPI_ISL_16055697, EPI_ISL_16055721, EPI_ISL_16056123, EPI_ISL_16057031, EPI_ISL_16060790, EPI_ISL_16062229, EPI_ISL_16066333, EPI_ISL_16068281, EPI_ISL_16068914, EPI_ISL_16073080, EPI_ISL_16073469, EPI_ISL_16073474, EPI_ISL_16074871, EPI_ISL_16075086, EPI_ISL_16075127, EPI_ISL_16079016, EPI_ISL_16080170, EPI_ISL_16080871, EPI_ISL_16091870, EPI_ISL_16102480, EPI_ISL_16111875, EPI_ISL_16113331, EPI_ISL_16113603, EPI_ISL_16113806, EPI_ISL_16114631, EPI_ISL_16115703, EPI_ISL_16116190, EPI_ISL_16116234, EPI_ISL_16116659, EPI_ISL_16116707, EPI_ISL_16119498, EPI_ISL_16119508, EPI_ISL_16119512, EPI_ISL_16119517, EPI_ISL_16119519, EPI_ISL_16119805, EPI_ISL_16131965, EPI_ISL_16131986, EPI_ISL_16131997, EPI_ISL_16136901, EPI_ISL_16137616, EPI_ISL_16151030, EPI_ISL_16151463, EPI_ISL_16151651, EPI_ISL_16153650, EPI_ISL_16153658, EPI_ISL_16153800, EPI_ISL_16154520, EPI_ISL_16157031, EPI_ISL_16157875, EPI_ISL_16158109, EPI_ISL_16158197, EPI_ISL_16158326, EPI_ISL_16158363, EPI_ISL_16160252, EPI_ISL_16160296, EPI_ISL_16160313, EPI_ISL_16165250, EPI_ISL_16167761, EPI_ISL_16178283, EPI_ISL_16178634, EPI_ISL_16179355, EPI_ISL_16180564, EPI_ISL_16180574, EPI_ISL_16181797, EPI_ISL_16181828, EPI_ISL_16181950, EPI_ISL_16183022, EPI_ISL_16190977, EPI_ISL_16191476, EPI_ISL_16196167, EPI_ISL_16197958, EPI_ISL_16201173, EPI_ISL_16215808, EPI_ISL_16218191, EPI_ISL_16219709, EPI_ISL_16219753, EPI_ISL_16221691, EPI_ISL_16230801, EPI_ISL_16233000, EPI_ISL_16233650, EPI_ISL_16233865, EPI_ISL_16234790, EPI_ISL_16235313, EPI_ISL_16235462, EPI_ISL_16235523, EPI_ISL_16235930, EPI_ISL_16244367, EPI_ISL_16244373, EPI_ISL_16244408, EPI_ISL_16244923, EPI_ISL_16245232, EPI_ISL_16245289, EPI_ISL_16245433, EPI_ISL_16245601, EPI_ISL_16245627, EPI_ISL_16247208, EPI_ISL_16247263, EPI_ISL_16247490, EPI_ISL_16247545, EPI_ISL_16247708, EPI_ISL_16257294, EPI_ISL_16264400, EPI_ISL_16265325, EPI_ISL_16268074, EPI_ISL_16270258, EPI_ISL_16271444, EPI_ISL_16271604, EPI_ISL_16273936, EPI_ISL_16284103, EPI_ISL_16284311, EPI_ISL_16287253, EPI_ISL_16287690, EPI_ISL_16290877, EPI_ISL_16293662, EPI_ISL_16312661, EPI_ISL_16327295, EPI_ISL_16334679, EPI_ISL_16338847, EPI_ISL_16338862, EPI_ISL_16343221, EPI_ISL_16344593, EPI_ISL_16348840, EPI_ISL_16348868, EPI_ISL_16351967, EPI_ISL_16354229, EPI_ISL_16355537, EPI_ISL_16356453, EPI_ISL_16356910, EPI_ISL_16358915, EPI_ISL_16360495, EPI_ISL_16365715, EPI_ISL_16368903, EPI_ISL_16369869, EPI_ISL_16370037, EPI_ISL_16378181, EPI_ISL_16379359, EPI_ISL_16380313, EPI_ISL_16381332, EPI_ISL_16381679, EPI_ISL_16384522, EPI_ISL_16385377, EPI_ISL_16385455, EPI_ISL_16385456, EPI_ISL_16391752, EPI_ISL_16394844, EPI_ISL_16394922, EPI_ISL_16395667, EPI_ISL_16398472, EPI_ISL_16399824, EPI_ISL_16400033, EPI_ISL_16414127, EPI_ISL_16422834, EPI_ISL_16424130, EPI_ISL_16425691, EPI_ISL_16428100, EPI_ISL_16428101, EPI_ISL_16428102, EPI_ISL_16429066, EPI_ISL_16434308, EPI_ISL_16435903, EPI_ISL_16436533, EPI_ISL_16439413, EPI_ISL_16443129, EPI_ISL_16443688, EPI_ISL_16443874, EPI_ISL_16444178, EPI_ISL_16444600, EPI_ISL_16452054, EPI_ISL_16454044, EPI_ISL_16460278, EPI_ISL_16460823, EPI_ISL_16461302, EPI_ISL_16464657, EPI_ISL_16467436, EPI_ISL_16470710, EPI_ISL_16471153, EPI_ISL_16473435, EPI_ISL_16473762, EPI_ISL_16474400, EPI_ISL_16479826, EPI_ISL_16482060, EPI_ISL_16489594, EPI_ISL_16491494, EPI_ISL_16492150, EPI_ISL_16492585, EPI_ISL_16492756, EPI_ISL_16493785, EPI_ISL_16497702, EPI_ISL_16498515, EPI_ISL_16503949, EPI_ISL_16507701, EPI_ISL_16507927, EPI_ISL_16520597, EPI_ISL_16520598, EPI_ISL_16520637, EPI_ISL_16520640, EPI_ISL_16520641, EPI_ISL_16520711, EPI_ISL_16520763, EPI_ISL_16520788, EPI_ISL_16521101, EPI_ISL_16524906, EPI_ISL_16528641, EPI_ISL_16528645, EPI_ISL_16528903, EPI_ISL_16534552, EPI_ISL_16535376, EPI_ISL_16536212, EPI_ISL_16539692, EPI_ISL_16541774, EPI_ISL_16542553, EPI_ISL_16544506, EPI_ISL_16567779, EPI_ISL_16574574, EPI_ISL_16581578, EPI_ISL_16584104, EPI_ISL_16585286, EPI_ISL_16586683, EPI_ISL_16586702, EPI_ISL_16587574, EPI_ISL_16597363, EPI_ISL_16598790, EPI_ISL_16599846, EPI_ISL_16607452, EPI_ISL_16607500, EPI_ISL_16607535, EPI_ISL_16611498, EPI_ISL_16611571, EPI_ISL_16613287, EPI_ISL_16613482, EPI_ISL_16615597, EPI_ISL_16615617, EPI_ISL_16615668, EPI_ISL_16616642, EPI_ISL_16625690, EPI_ISL_16626611, EPI_ISL_16626666, EPI_ISL_16627067, EPI_ISL_16628854, EPI_ISL_16630260, EPI_ISL_16630261, EPI_ISL_16637089, EPI_ISL_16637607, EPI_ISL_16637631, EPI_ISL_16638190, EPI_ISL_16638453, EPI_ISL_16643406, EPI_ISL_16647535, EPI_ISL_16647537, EPI_ISL_16647544, EPI_ISL_16649988, EPI_ISL_16650006, EPI_ISL_16650011, EPI_ISL_16653618, EPI_ISL_16669313, EPI_ISL_16669829, EPI_ISL_16672301, EPI_ISL_16672327, EPI_ISL_16672352, EPI_ISL_16676267, EPI_ISL_16677015, EPI_ISL_16678917, EPI_ISL_16678946, EPI_ISL_16679654, EPI_ISL_16681917, EPI_ISL_16682342, EPI_ISL_16688219, EPI_ISL_16688525, EPI_ISL_16688688, EPI_ISL_16688713, EPI_ISL_16691397, EPI_ISL_16691487, EPI_ISL_16694176, EPI_ISL_16695435, EPI_ISL_16697861, EPI_ISL_16700160, EPI_ISL_16702838, EPI_ISL_16705882, EPI_ISL_16706498, EPI_ISL_16708798, EPI_ISL_16711038, EPI_ISL_16711095, EPI_ISL_16711417, EPI_ISL_16711531, EPI_ISL_16716967, EPI_ISL_16721930, EPI_ISL_16722183, EPI_ISL_16722215, EPI_ISL_16722270, EPI_ISL_16722970, EPI_ISL_16723215, EPI_ISL_16725887, EPI_ISL_16727241, EPI_ISL_16728257, EPI_ISL_16728383, EPI_ISL_16728411, EPI_ISL_16729998, EPI_ISL_16731753, EPI_ISL_16736343, EPI_ISL_16736346, EPI_ISL_16736400, EPI_ISL_16739452, EPI_ISL_16740104, EPI_ISL_16740406, EPI_ISL_16741567, EPI_ISL_16741573, EPI_ISL_16749999, EPI_ISL_16750878, EPI_ISL_16751721, EPI_ISL_16751722, EPI_ISL_16751789, EPI_ISL_16751791, EPI_ISL_16751977, EPI_ISL_16752073, EPI_ISL_16752138, EPI_ISL_16757168, EPI_ISL_16757210, EPI_ISL_16758981, EPI_ISL_16764861, EPI_ISL_16765888, EPI_ISL_16766196, EPI_ISL_16807740, EPI_ISL_16807787, EPI_ISL_16807844, EPI_ISL_16811091, EPI_ISL_16812565, EPI_ISL_16815494, EPI_ISL_16816293, EPI_ISL_16818458, EPI_ISL_16818471, EPI_ISL_16825222, EPI_ISL_16828876, EPI_ISL_16828896, EPI_ISL_16829188, EPI_ISL_16831507, EPI_ISL_16833893, EPI_ISL_16834974, EPI_ISL_16835399, EPI_ISL_16842787, EPI_ISL_16842790, EPI_ISL_16847425, EPI_ISL_16847642, EPI_ISL_16847674, EPI_ISL_16847675, EPI_ISL_16847676, EPI_ISL_16847677, EPI_ISL_16853227, EPI_ISL_16853229, EPI_ISL_16853597, EPI_ISL_16856355, EPI_ISL_16856565, EPI_ISL_16856637, EPI_ISL_16856833, EPI_ISL_16857514, EPI_ISL_16857776, EPI_ISL_16858310, EPI_ISL_16858617, EPI_ISL_16858667, EPI_ISL_16861084, EPI_ISL_16862130, EPI_ISL_16863260, EPI_ISL_16866580, EPI_ISL_16867553, EPI_ISL_16868647, EPI_ISL_16868655, EPI_ISL_16868993, EPI_ISL_16869007, EPI_ISL_16875752, EPI_ISL_16876039, EPI_ISL_16876784, EPI_ISL_16877428, EPI_ISL_16878720, EPI_ISL_16883240, EPI_ISL_16883873, EPI_ISL_16884622, EPI_ISL_16894717, EPI_ISL_16895138, EPI_ISL_16895290, EPI_ISL_16895522, EPI_ISL_16903492, EPI_ISL_16903494, EPI_ISL_16904444, EPI_ISL_16904536, EPI_ISL_16908472, EPI_ISL_16910025, EPI_ISL_16910165, EPI_ISL_16910272, EPI_ISL_16911226, EPI_ISL_16921530, EPI_ISL_16921542, EPI_ISL_16925257, EPI_ISL_16927736, EPI_ISL_16931901, EPI_ISL_16941750, EPI_ISL_16942000, EPI_ISL_16945429, EPI_ISL_16946783, EPI_ISL_16947592, EPI_ISL_16947625, EPI_ISL_16951592, EPI_ISL_16953741, EPI_ISL_16954486, EPI_ISL_16955471, EPI_ISL_16957015, EPI_ISL_16966997, EPI_ISL_16967082, EPI_ISL_16967083, EPI_ISL_16967084, EPI_ISL_16967085, EPI_ISL_16967086, EPI_ISL_16969756, EPI_ISL_16969757, EPI_ISL_16970279, EPI_ISL_16973343, EPI_ISL_16977317, EPI_ISL_16977653, EPI_ISL_16977749, EPI_ISL_16979482, EPI_ISL_16980683, EPI_ISL_16981030, EPI_ISL_16981047, EPI_ISL_16981048, EPI_ISL_16981102, EPI_ISL_16987088, EPI_ISL_16987375, EPI_ISL_16987376, EPI_ISL_16995491, EPI_ISL_16995525, EPI_ISL_16997638, EPI_ISL_17001974, EPI_ISL_17001987, EPI_ISL_17002442, EPI_ISL_17006258, EPI_ISL_17008393, EPI_ISL_17008502, EPI_ISL_17016219, EPI_ISL_17016344, EPI_ISL_17018731, EPI_ISL_17020636, EPI_ISL_17021187, EPI_ISL_17022063, EPI_ISL_17022081, EPI_ISL_17024099, EPI_ISL_17025560, EPI_ISL_17025998, EPI_ISL_17026052, EPI_ISL_17026537, EPI_ISL_17027430, EPI_ISL_17032070, EPI_ISL_17032664, EPI_ISL_17035345, EPI_ISL_17036551, EPI_ISL_17037388, EPI_ISL_17040117, EPI_ISL_17040120, EPI_ISL_17040127, EPI_ISL_17040133, EPI_ISL_17041105, EPI_ISL_17041117, EPI_ISL_17041143, EPI_ISL_17044002, EPI_ISL_17046406, EPI_ISL_17047667, EPI_ISL_17050958, EPI_ISL_17051743, EPI_ISL_17056159, EPI_ISL_17057279, EPI_ISL_17061716, EPI_ISL_17062081, EPI_ISL_17065016, EPI_ISL_17067007, EPI_ISL_17068616, EPI_ISL_17068621, EPI_ISL_17073286, EPI_ISL_17074720, EPI_ISL_17076011, EPI_ISL_17076926, EPI_ISL_17077233, EPI_ISL_17077446, EPI_ISL_17079150, EPI_ISL_17079151, EPI_ISL_17079427, EPI_ISL_17080036, EPI_ISL_17080146, EPI_ISL_17080283, EPI_ISL_17080510, EPI_ISL_17081567, EPI_ISL_17084330, EPI_ISL_17086936, EPI_ISL_17086958, EPI_ISL_17090268, EPI_ISL_17090730, EPI_ISL_17092242, EPI_ISL_17092260, EPI_ISL_17099321, EPI_ISL_17099446, EPI_ISL_17101049, EPI_ISL_17101231, EPI_ISL_17104807, EPI_ISL_17105674, EPI_ISL_17105677, EPI_ISL_17105777, EPI_ISL_17105786, EPI_ISL_17105804, EPI_ISL_17106895, EPI_ISL_17109738, EPI_ISL_17109787, EPI_ISL_17112198, EPI_ISL_17113114, EPI_ISL_17118740, EPI_ISL_17126699, EPI_ISL_17126727, EPI_ISL_17127510, EPI_ISL_17129671, EPI_ISL_17139969, EPI_ISL_17149697, EPI_ISL_17150312, EPI_ISL_17150571, EPI_ISL_17152522, EPI_ISL_17152602, EPI_ISL_17152816, EPI_ISL_17154843, EPI_ISL_17154893, EPI_ISL_17158601, EPI_ISL_17158659, EPI_ISL_17158660, EPI_ISL_17158661, EPI_ISL_17158662, EPI_ISL_17158663, EPI_ISL_17158664, EPI_ISL_17158665, EPI_ISL_17164529, EPI_ISL_17165387, EPI_ISL_17165528, EPI_ISL_17170921, EPI_ISL_17173754, EPI_ISL_17174278, EPI_ISL_17174323, EPI_ISL_17175107, EPI_ISL_17180776, EPI_ISL_17182281, EPI_ISL_17188691, EPI_ISL_17188772, EPI_ISL_17188836, EPI_ISL_17189286, EPI_ISL_17189360, EPI_ISL_17189372, EPI_ISL_17190813, EPI_ISL_17191784, EPI_ISL_17193988, EPI_ISL_17194121, EPI_ISL_17194564, EPI_ISL_17195807, EPI_ISL_17198415, EPI_ISL_17199165, EPI_ISL_17199381, EPI_ISL_17199743, EPI_ISL_17200348, EPI_ISL_17200520, EPI_ISL_17201694, EPI_ISL_17202051, EPI_ISL_17205892, EPI_ISL_17206016, EPI_ISL_17206140, EPI_ISL_17207424, EPI_ISL_17210230, EPI_ISL_17210689, EPI_ISL_17214413, EPI_ISL_17214693, EPI_ISL_17214774, EPI_ISL_17214805, EPI_ISL_17214933, EPI_ISL_17215427, EPI_ISL_17215676, EPI_ISL_17215686, EPI_ISL_17215790, EPI_ISL_17216822, EPI_ISL_17216978, EPI_ISL_17221710, EPI_ISL_17222365, EPI_ISL_17223438, EPI_ISL_17226531, EPI_ISL_17226637, EPI_ISL_17232350, EPI_ISL_17232448, EPI_ISL_17237921, EPI_ISL_17239049, EPI_ISL_17239405, EPI_ISL_17239499, EPI_ISL_17241376, EPI_ISL_17244668, EPI_ISL_17245140, EPI_ISL_17245198, EPI_ISL_17245226, EPI_ISL_17245255, EPI_ISL_17246876, EPI_ISL_17246931, EPI_ISL_17247186, EPI_ISL_17247325, EPI_ISL_17247333, EPI_ISL_17251028, EPI_ISL_17252934, EPI_ISL_17253364, EPI_ISL_17253589, EPI_ISL_17257608, EPI_ISL_17262137, EPI_ISL_17265160, EPI_ISL_17270165, EPI_ISL_17270215, EPI_ISL_17270950, EPI_ISL_17270964, EPI_ISL_17270974, EPI_ISL_17271226, EPI_ISL_17271272, EPI_ISL_17272946, EPI_ISL_17273453, EPI_ISL_17273454, EPI_ISL_17273455, EPI_ISL_17273456, EPI_ISL_17275616, EPI_ISL_17275984, EPI_ISL_17276025, EPI_ISL_17276962, EPI_ISL_17284010, EPI_ISL_17284045, EPI_ISL_17284573, EPI_ISL_17284791, EPI_ISL_17284792, EPI_ISL_17284793, EPI_ISL_17284794, EPI_ISL_17284795, EPI_ISL_17285690, EPI_ISL_17288589, EPI_ISL_17290740, EPI_ISL_17292666, EPI_ISL_17292834, EPI_ISL_17297993, EPI_ISL_17298321, EPI_ISL_17298323, EPI_ISL_17299688, EPI_ISL_17300150, EPI_ISL_17304801, EPI_ISL_17304899, EPI_ISL_17305358, EPI_ISL_17319411, EPI_ISL_17319528, EPI_ISL_17319601, EPI_ISL_17321362, EPI_ISL_17322993, EPI_ISL_17334027, EPI_ISL_17342544, EPI_ISL_17344004, EPI_ISL_17344178, EPI_ISL_17344660, EPI_ISL_17345445, EPI_ISL_17347577, EPI_ISL_17348219, EPI_ISL_17349770, EPI_ISL_17349983, EPI_ISL_17350301, EPI_ISL_17352192, EPI_ISL_17358767, EPI_ISL_17359772, EPI_ISL_17370155, EPI_ISL_17371048, EPI_ISL_17374605, EPI_ISL_17374609, EPI_ISL_17374807, EPI_ISL_17376230, EPI_ISL_17381216, EPI_ISL_17387122, EPI_ISL_17389140, EPI_ISL_17389210, EPI_ISL_17389223, EPI_ISL_17389779, EPI_ISL_17390660, EPI_ISL_17390743, EPI_ISL_17390873, EPI_ISL_17391460, EPI_ISL_17394837, EPI_ISL_17397497, EPI_ISL_17408352, EPI_ISL_17409157, EPI_ISL_17411543, EPI_ISL_17414235, EPI_ISL_17414543, EPI_ISL_17418615, EPI_ISL_17421962, EPI_ISL_17423074, EPI_ISL_17424014, EPI_ISL_17429770, EPI_ISL_17430458, EPI_ISL_17430487, EPI_ISL_17431238, EPI_ISL_17434223, EPI_ISL_17434227, EPI_ISL_17437940, EPI_ISL_17440507, EPI_ISL_17441169, EPI_ISL_17441208, EPI_ISL_17441815, EPI_ISL_17445401, EPI_ISL_17446132, EPI_ISL_17464711, EPI_ISL_17466081, EPI_ISL_17470229, EPI_ISL_17470269, EPI_ISL_17471181, EPI_ISL_17471185, EPI_ISL_17471619, EPI_ISL_17471674, EPI_ISL_17472531, EPI_ISL_17475799, EPI_ISL_17476568, EPI_ISL_17476871, EPI_ISL_17477106, EPI_ISL_17480516, EPI_ISL_17481180, EPI_ISL_17481517, EPI_ISL_17481597, EPI_ISL_17482811, EPI_ISL_17482813, EPI_ISL_17482815, EPI_ISL_17482819, EPI_ISL_17485780, EPI_ISL_17487963, EPI_ISL_17494731, EPI_ISL_17497461, EPI_ISL_17497688, EPI_ISL_17497854, EPI_ISL_17497868, EPI_ISL_17501535, EPI_ISL_17501536, EPI_ISL_17501576, EPI_ISL_17501763, EPI_ISL_17502219, EPI_ISL_17502972, EPI_ISL_17503268, EPI_ISL_17503711, EPI_ISL_17504816, EPI_ISL_17504835, EPI_ISL_17505072, EPI_ISL_17508749, EPI_ISL_17509597, EPI_ISL_17510495, EPI_ISL_17510856, EPI_ISL_17511096, EPI_ISL_17511836, EPI_ISL_17512412, EPI_ISL_17512876, EPI_ISL_17512968, EPI_ISL_17513218, EPI_ISL_17513312, EPI_ISL_17514540, EPI_ISL_17515086, EPI_ISL_17515177, EPI_ISL_17516651, EPI_ISL_17516658, EPI_ISL_17516659, EPI_ISL_17517664, EPI_ISL_17517834, EPI_ISL_17517844, EPI_ISL_17521302, EPI_ISL_17521772, EPI_ISL_17521778, EPI_ISL_17522610, EPI_ISL_17522687, EPI_ISL_17522934, EPI_ISL_17523535, EPI_ISL_17523620, EPI_ISL_17523782, EPI_ISL_17523873, EPI_ISL_17524106, EPI_ISL_17524432, EPI_ISL_17524502, EPI_ISL_17524503, EPI_ISL_17535664, EPI_ISL_17535979, EPI_ISL_17541088, EPI_ISL_17541797, EPI_ISL_17543006, EPI_ISL_17544283, EPI_ISL_17545970, EPI_ISL_17546923, EPI_ISL_17547529, EPI_ISL_17547545, EPI_ISL_17548526, EPI_ISL_17549129, EPI_ISL_17550129, EPI_ISL_17550538, EPI_ISL_17553063, EPI_ISL_17553974, EPI_ISL_17556705, EPI_ISL_17559150, EPI_ISL_17559165, EPI_ISL_17559166, EPI_ISL_17559167, EPI_ISL_17559168, EPI_ISL_17563568, EPI_ISL_17565071, EPI_ISL_17565211, EPI_ISL_17565212, EPI_ISL_17566854, EPI_ISL_17579120, EPI_ISL_17580420, EPI_ISL_17583157, EPI_ISL_17584277, EPI_ISL_17585020, EPI_ISL_17585021, EPI_ISL_17585022, EPI_ISL_17585023, EPI_ISL_17585036, EPI_ISL_17585039, EPI_ISL_17586115, EPI_ISL_17587423, EPI_ISL_17587656, EPI_ISL_17587671, EPI_ISL_17587859, EPI_ISL_17588127, EPI_ISL_17588216, EPI_ISL_17588460, EPI_ISL_17589845, EPI_ISL_17590449, EPI_ISL_17590486, EPI_ISL_17591005, EPI_ISL_17591014, EPI_ISL_17591015, EPI_ISL_17591028, EPI_ISL_17592236, EPI_ISL_17592618, EPI_ISL_17593692, EPI_ISL_17593865, EPI_ISL_17595116, EPI_ISL_17595117, EPI_ISL_17595967, EPI_ISL_17595980, EPI_ISL_17597954, EPI_ISL_17598384, EPI_ISL_17599326, EPI_ISL_17599427, EPI_ISL_17600948, EPI_ISL_17600958, EPI_ISL_17600978, EPI_ISL_17600988, EPI_ISL_17601066, EPI_ISL_17601144, EPI_ISL_17601196, EPI_ISL_17601219, EPI_ISL_17601261, EPI_ISL_17601276, EPI_ISL_17601933, EPI_ISL_17602469, EPI_ISL_17602756, EPI_ISL_17605514, EPI_ISL_17612035, EPI_ISL_17612050, EPI_ISL_17612051, EPI_ISL_17612052, EPI_ISL_17615127, EPI_ISL_17615387, EPI_ISL_17617168, EPI_ISL_17617538, EPI_ISL_17618306, EPI_ISL_17620360, EPI_ISL_17620361, EPI_ISL_17621930, EPI_ISL_17623470, EPI_ISL_17623785, EPI_ISL_17623810, EPI_ISL_17626289, EPI_ISL_17628383, EPI_ISL_17628855, EPI_ISL_17630096, EPI_ISL_17632950, EPI_ISL_17633442, EPI_ISL_17634290, EPI_ISL_17634585, EPI_ISL_17634799, EPI_ISL_17637409, EPI_ISL_17637499, EPI_ISL_17637946, EPI_ISL_17640029, EPI_ISL_17640079, EPI_ISL_17642112, EPI_ISL_17642765, EPI_ISL_17643093, EPI_ISL_17644186, EPI_ISL_17645081, EPI_ISL_17645416, EPI_ISL_17646422, EPI_ISL_17648924, EPI_ISL_17651803, EPI_ISL_17652508, EPI_ISL_17652513, EPI_ISL_17654325, EPI_ISL_17654831, EPI_ISL_17655018, EPI_ISL_17656002, EPI_ISL_17657287, EPI_ISL_17658392, EPI_ISL_17659247, EPI_ISL_17659794, EPI_ISL_17661435, EPI_ISL_17661709, EPI_ISL_17661736, EPI_ISL_17661772, EPI_ISL_17662111, EPI_ISL_17664370, EPI_ISL_17666708, EPI_ISL_17667360, EPI_ISL_17667562, EPI_ISL_17669441, EPI_ISL_17669457, EPI_ISL_17671157, EPI_ISL_17671162, EPI_ISL_17671689, EPI_ISL_17675889, EPI_ISL_17677128, EPI_ISL_17677325, EPI_ISL_17678395, EPI_ISL_17679253, EPI_ISL_17679612, EPI_ISL_17680172, EPI_ISL_17683135, EPI_ISL_17683747, EPI_ISL_17683879, EPI_ISL_17683882, EPI_ISL_17683902, EPI_ISL_17683926, EPI_ISL_17684194, EPI_ISL_17685960, EPI_ISL_17685982, EPI_ISL_17686409, EPI_ISL_17686485, EPI_ISL_17686694, EPI_ISL_17686736, EPI_ISL_17688072, EPI_ISL_17689247, EPI_ISL_17695348, EPI_ISL_17696086, EPI_ISL_17696551, EPI_ISL_17696575, EPI_ISL_17697616, EPI_ISL_17699149, EPI_ISL_17699879, EPI_ISL_17699884, EPI_ISL_17700051, EPI_ISL_17700270, EPI_ISL_17701278, EPI_ISL_17701782, EPI_ISL_17703815, EPI_ISL_17704713, EPI_ISL_17705564, EPI_ISL_17706013, EPI_ISL_17706030, EPI_ISL_17708288, EPI_ISL_17709926, EPI_ISL_17710268, EPI_ISL_17710278, EPI_ISL_17710307, EPI_ISL_17710673, EPI_ISL_17710974, EPI_ISL_17711012, EPI_ISL_17711646, EPI_ISL_17712964, EPI_ISL_17713423, EPI_ISL_17713709, EPI_ISL_17714880, EPI_ISL_17714882, EPI_ISL_17714902, EPI_ISL_17714948, EPI_ISL_17715122, EPI_ISL_17715974, EPI_ISL_17716296, EPI_ISL_17718358, EPI_ISL_17718497, EPI_ISL_17719162, EPI_ISL_17721620, EPI_ISL_17721941, EPI_ISL_17722142, EPI_ISL_17722884, EPI_ISL_17726746, EPI_ISL_17727194, EPI_ISL_17728144, EPI_ISL_17728250, EPI_ISL_17731387, EPI_ISL_17731388, EPI_ISL_17732098, EPI_ISL_17733269, EPI_ISL_17734236, EPI_ISL_17735972, EPI_ISL_17736284, EPI_ISL_17736388, EPI_ISL_17737562, EPI_ISL_17739108, EPI_ISL_17739543, EPI_ISL_17741957, EPI_ISL_17743681, EPI_ISL_17744022, EPI_ISL_17747309, EPI_ISL_17759354, EPI_ISL_17759925, EPI_ISL_17760156, EPI_ISL_17760279, EPI_ISL_17762387, EPI_ISL_17762760, EPI_ISL_17763721, EPI_ISL_17764011, EPI_ISL_17764066, EPI_ISL_17764072, EPI_ISL_17764496, EPI_ISL_17766060, EPI_ISL_17766100, EPI_ISL_17766112, EPI_ISL_17767434, EPI_ISL_17767435, EPI_ISL_17767436, EPI_ISL_17767437, EPI_ISL_17769030, EPI_ISL_17769081, EPI_ISL_17769169, EPI_ISL_17769170, EPI_ISL_17769216, EPI_ISL_17769229, EPI_ISL_17769310, EPI_ISL_17769888, EPI_ISL_17770729, EPI_ISL_17770732, EPI_ISL_17770736, EPI_ISL_17770779, EPI_ISL_17771047, EPI_ISL_17771051, EPI_ISL_17772012, EPI_ISL_17775344, EPI_ISL_17776736, EPI_ISL_17777061, EPI_ISL_17777067, EPI_ISL_17777729, EPI_ISL_17778593, EPI_ISL_17778602, EPI_ISL_17780724, EPI_ISL_17780726, EPI_ISL_17780860, EPI_ISL_17780886, EPI_ISL_17781122, EPI_ISL_17781585, EPI_ISL_17781712, EPI_ISL_17782148, EPI_ISL_17782366, EPI_ISL_17782502, EPI_ISL_17783358, EPI_ISL_17784545, EPI_ISL_17784546, EPI_ISL_17784547, EPI_ISL_17784552, EPI_ISL_17784558, EPI_ISL_17784569, EPI_ISL_17784585, EPI_ISL_17784593, EPI_ISL_17784775, EPI_ISL_17784803, EPI_ISL_17784804, EPI_ISL_17786165, EPI_ISL_17786546, EPI_ISL_17786769, EPI_ISL_17786827, EPI_ISL_17787009, EPI_ISL_17787597, EPI_ISL_17787864, EPI_ISL_17788384, EPI_ISL_17789385, EPI_ISL_17789475, EPI_ISL_17789808, EPI_ISL_17790033, EPI_ISL_17790116, EPI_ISL_17791306, EPI_ISL_17791796, EPI_ISL_17792172, EPI_ISL_17792191, EPI_ISL_17794816, EPI_ISL_17796500, EPI_ISL_17796537, EPI_ISL_17796598, EPI_ISL_17796704, EPI_ISL_17797704, EPI_ISL_17798165, EPI_ISL_17799068, EPI_ISL_17799108, EPI_ISL_17802597, EPI_ISL_17803325, EPI_ISL_17803653, EPI_ISL_17806504, EPI_ISL_17806524, EPI_ISL_17809334, EPI_ISL_17809574, EPI_ISL_17810512, EPI_ISL_17812915, EPI_ISL_17813049, EPI_ISL_17813537, EPI_ISL_17813637, EPI_ISL_17813862, EPI_ISL_17815222, EPI_ISL_17816174, EPI_ISL_17817657, EPI_ISL_17817985, EPI_ISL_17818039, EPI_ISL_17819921, EPI_ISL_17820257, EPI_ISL_17820258, EPI_ISL_17820602, EPI_ISL_17821850, EPI_ISL_17823538, EPI_ISL_17824292, EPI_ISL_17824608, EPI_ISL_17824670, EPI_ISL_17826285, EPI_ISL_17830573, EPI_ISL_17830591, EPI_ISL_17830762, EPI_ISL_17831005, EPI_ISL_17831639, EPI_ISL_17831941, EPI_ISL_17833161, EPI_ISL_17833549, EPI_ISL_17837092, EPI_ISL_17837097, EPI_ISL_17837134, EPI_ISL_17837135, EPI_ISL_17837188, EPI_ISL_17837432, EPI_ISL_17837459, EPI_ISL_17837460, EPI_ISL_17837914, EPI_ISL_17837915, EPI_ISL_17838109, EPI_ISL_17838506, EPI_ISL_17850070, EPI_ISL_17850078, EPI_ISL_17851276, EPI_ISL_17853355, EPI_ISL_17853579, EPI_ISL_17855226, EPI_ISL_17856975, EPI_ISL_17857949, EPI_ISL_17857950, EPI_ISL_17859477, EPI_ISL_17860390, EPI_ISL_17860984, EPI_ISL_17862677, EPI_ISL_17871595, EPI_ISL_17879222, EPI_ISL_17884376, EPI_ISL_17884518, EPI_ISL_17885064, EPI_ISL_17885128, EPI_ISL_17885331, EPI_ISL_17885459, EPI_ISL_17891004, EPI_ISL_17899627, EPI_ISL_17949029, EPI_ISL_17949339, EPI_ISL_17949978, EPI_ISL_17950840, EPI_ISL_17952015, EPI_ISL_17952019, EPI_ISL_17953343, EPI_ISL_17953610, EPI_ISL_17954106, EPI_ISL_17954662, EPI_ISL_17954669, EPI_ISL_17954940, EPI_ISL_17956164, EPI_ISL_17958015, EPI_ISL_17959424, EPI_ISL_17960600, EPI_ISL_17960747, EPI_ISL_17964403, EPI_ISL_17964415, EPI_ISL_17964828, EPI_ISL_17965635, EPI_ISL_17965636, EPI_ISL_17966200, EPI_ISL_17966205, EPI_ISL_17968777, EPI_ISL_17968962, EPI_ISL_17969108, EPI_ISL_17971223, EPI_ISL_17971936, EPI_ISL_17972242, EPI_ISL_17972372, EPI_ISL_17973367, EPI_ISL_17974574, EPI_ISL_17974688, EPI_ISL_17974927, EPI_ISL_17974952, EPI_ISL_17975003, EPI_ISL_17975174, EPI_ISL_17976113, EPI_ISL_17976116, EPI_ISL_17976531, EPI_ISL_17977985, EPI_ISL_17978344, EPI_ISL_17978693, EPI_ISL_17978839, EPI_ISL_17979017, EPI_ISL_17979965, EPI_ISL_17979979, EPI_ISL_17979981, EPI_ISL_17982411, EPI_ISL_17982453, EPI_ISL_17982543, EPI_ISL_17985757, EPI_ISL_17988396, EPI_ISL_17989190, EPI_ISL_17989433, EPI_ISL_17989516, EPI_ISL_17989740, EPI_ISL_17989749, EPI_ISL_17989792, EPI_ISL_17989829, EPI_ISL_17989860, EPI_ISL_17990304, EPI_ISL_17993966, EPI_ISL_17994784, EPI_ISL_17994786, EPI_ISL_17995488, EPI_ISL_17995513, EPI_ISL_17995955, EPI_ISL_17996897, EPI_ISL_17997249, EPI_ISL_17997251, EPI_ISL_17997917, EPI_ISL_17997982, EPI_ISL_17998406, EPI_ISL_18000155, EPI_ISL_18000245, EPI_ISL_18000414, EPI_ISL_18000654, EPI_ISL_18000825, EPI_ISL_18001789, EPI_ISL_18001862, EPI_ISL_18008246, EPI_ISL_18008262, EPI_ISL_18008673, EPI_ISL_18009591, EPI_ISL_18009602, EPI_ISL_18010720, EPI_ISL_18011449, EPI_ISL_18011518, EPI_ISL_18012526, EPI_ISL_18012547, EPI_ISL_18012806, EPI_ISL_18014700, EPI_ISL_18016999, EPI_ISL_18019246, EPI_ISL_18028785, EPI_ISL_18029979, EPI_ISL_18030390, EPI_ISL_18030391, EPI_ISL_18030395, EPI_ISL_18031842, EPI_ISL_18032297, EPI_ISL_18032322, EPI_ISL_18032338, EPI_ISL_18033013, EPI_ISL_18033516, EPI_ISL_18033631, EPI_ISL_18034109, EPI_ISL_18037119, EPI_ISL_18037474, EPI_ISL_18037476, EPI_ISL_18037744, EPI_ISL_18038269, EPI_ISL_18039728, EPI_ISL_18040070, EPI_ISL_18041130, EPI_ISL_18041968, EPI_ISL_18042110, EPI_ISL_18044024, EPI_ISL_18044164, EPI_ISL_18044400, EPI_ISL_18044754, EPI_ISL_18044755, EPI_ISL_18044759, EPI_ISL_18045809, EPI_ISL_18048708, EPI_ISL_18048972, EPI_ISL_18048978, EPI_ISL_18049009, EPI_ISL_18049160, EPI_ISL_18049161, EPI_ISL_18049174, EPI_ISL_18049809, EPI_ISL_18049917, EPI_ISL_18050065, EPI_ISL_18050520, EPI_ISL_18050523, EPI_ISL_18051914, EPI_ISL_18051918, EPI_ISL_18052440, EPI_ISL_18052776, EPI_ISL_18052929, EPI_ISL_18053022, EPI_ISL_18053315, EPI_ISL_18054466, EPI_ISL_18054899, EPI_ISL_18056643, EPI_ISL_18056644, EPI_ISL_18056759, EPI_ISL_18056769, EPI_ISL_18058525, EPI_ISL_18058567, EPI_ISL_18058881, EPI_ISL_18059074, EPI_ISL_18059075, EPI_ISL_18059076, EPI_ISL_18059726, EPI_ISL_18060973, EPI_ISL_18062475, EPI_ISL_18064366, EPI_ISL_18064383, EPI_ISL_18064405, EPI_ISL_18064413, EPI_ISL_18064431, EPI_ISL_18064456, EPI_ISL_18070310, EPI_ISL_18071883, EPI_ISL_18071901, EPI_ISL_18072343, EPI_ISL_18073924, EPI_ISL_18074072, EPI_ISL_18075985, EPI_ISL_18076065, EPI_ISL_18076069, EPI_ISL_18076165, EPI_ISL_18076251, EPI_ISL_18076473, EPI_ISL_18077275, EPI_ISL_18078878, EPI_ISL_18079417, EPI_ISL_18080566, EPI_ISL_18083488, EPI_ISL_18091808, EPI_ISL_18093840, EPI_ISL_18094397, EPI_ISL_18094429, EPI_ISL_18094476, EPI_ISL_18094560, EPI_ISL_18094842, EPI_ISL_18095961, EPI_ISL_18097327, EPI_ISL_18097349, EPI_ISL_18097786, EPI_ISL_18098270, EPI_ISL_18098276, EPI_ISL_18098299, EPI_ISL_18098479, EPI_ISL_18098976, EPI_ISL_18099952, EPI_ISL_18100455, EPI_ISL_18100457, EPI_ISL_18100607, EPI_ISL_18104072, EPI_ISL_18104501, EPI_ISL_18106416, EPI_ISL_18106460, EPI_ISL_18106464, EPI_ISL_18106662, EPI_ISL_18106788, EPI_ISL_18106910, EPI_ISL_18106912, EPI_ISL_18106920, EPI_ISL_18106929, EPI_ISL_18106930, EPI_ISL_18106931, EPI_ISL_18106933, EPI_ISL_18106934, EPI_ISL_18106950, EPI_ISL_18106951, EPI_ISL_18107900, EPI_ISL_18109285, EPI_ISL_18110014, EPI_ISL_18110496, EPI_ISL_18110776, EPI_ISL_18111020, EPI_ISL_18111021, EPI_ISL_18111040, EPI_ISL_18111041, EPI_ISL_18111086, EPI_ISL_18112015, EPI_ISL_18115442, EPI_ISL_18115451, EPI_ISL_18115956, EPI_ISL_18116015, EPI_ISL_18116176, EPI_ISL_18118289, EPI_ISL_18118388, EPI_ISL_18118556, EPI_ISL_18118855, EPI_ISL_18119265, EPI_ISL_18120201, EPI_ISL_18123396, EPI_ISL_18124840, EPI_ISL_18125049, EPI_ISL_18125050, EPI_ISL_18126834, EPI_ISL_18127203, EPI_ISL_18127526, EPI_ISL_18127527, EPI_ISL_18127685, EPI_ISL_18127834, EPI_ISL_18129019, EPI_ISL_18129038, EPI_ISL_18129213, EPI_ISL_18129656, EPI_ISL_18129944, EPI_ISL_18131053, EPI_ISL_18131109, EPI_ISL_18134315, EPI_ISL_18134392, EPI_ISL_18134395, EPI_ISL_18134442, EPI_ISL_18134610, EPI_ISL_18134691, EPI_ISL_18134700, EPI_ISL_18134706, EPI_ISL_18134984, EPI_ISL_18135040, EPI_ISL_18136392, EPI_ISL_18136968, EPI_ISL_18139400, EPI_ISL_18139409, EPI_ISL_18141686, EPI_ISL_18141739, EPI_ISL_18141844, EPI_ISL_18142202, EPI_ISL_18142317, EPI_ISL_18142357, EPI_ISL_18142978, EPI_ISL_18142994, EPI_ISL_18146885, EPI_ISL_18147456, EPI_ISL_18147966, EPI_ISL_18151975, EPI_ISL_18151976, EPI_ISL_18151977, EPI_ISL_18153000, EPI_ISL_18159587, EPI_ISL_18160510, EPI_ISL_18160518, EPI_ISL_18160530, EPI_ISL_18160538, EPI_ISL_18162564, EPI_ISL_18162567, EPI_ISL_18163680, EPI_ISL_18164441, EPI_ISL_18166642, EPI_ISL_18166643, EPI_ISL_18168780, EPI_ISL_18169117, EPI_ISL_18205057, EPI_ISL_18207613, EPI_ISL_18210510, EPI_ISL_18212559, EPI_ISL_18213104, EPI_ISL_18215123, EPI_ISL_18215226, EPI_ISL_18215482, EPI_ISL_18215552, EPI_ISL_18217564, EPI_ISL_18217995, EPI_ISL_18218776, EPI_ISL_18219916, EPI_ISL_18219931, EPI_ISL_18219970, EPI_ISL_18220073, EPI_ISL_18220494, EPI_ISL_18220495, EPI_ISL_18220498, EPI_ISL_18221521, EPI_ISL_18221524, EPI_ISL_18221527, EPI_ISL_18221982, EPI_ISL_18221985, EPI_ISL_18222367, EPI_ISL_18224410, EPI_ISL_18224514, EPI_ISL_18225473, EPI_ISL_18227366, EPI_ISL_18227596, EPI_ISL_18227611, EPI_ISL_18227624, EPI_ISL_18227629, EPI_ISL_18228307, EPI_ISL_18232124, EPI_ISL_18233906, EPI_ISL_18234431, EPI_ISL_18236180, EPI_ISL_18236291, EPI_ISL_18237063, EPI_ISL_18238117, EPI_ISL_18241087, EPI_ISL_18241705, EPI_ISL_18241707, EPI_ISL_18241719, EPI_ISL_18245571, EPI_ISL_18247259, EPI_ISL_18248695, EPI_ISL_18249682, EPI_ISL_18253248, EPI_ISL_18253249, EPI_ISL_18255994, EPI_ISL_18256173, EPI_ISL_18256714, EPI_ISL_18256980, EPI_ISL_18258766, EPI_ISL_18259784, EPI_ISL_18260202, EPI_ISL_18263919, EPI_ISL_18263945, EPI_ISL_18263981, EPI_ISL_18269234, EPI_ISL_18271265, EPI_ISL_18273982, EPI_ISL_18274346, EPI_ISL_18276415, EPI_ISL_18277439, EPI_ISL_18277736, EPI_ISL_18278627, EPI_ISL_18278909, EPI_ISL_18279614, EPI_ISL_18281186, EPI_ISL_18281259, EPI_ISL_18281287, EPI_ISL_18281288, EPI_ISL_18281494, EPI_ISL_18281574, EPI_ISL_18282077, EPI_ISL_18282082, EPI_ISL_18286773, EPI_ISL_18287351, EPI_ISL_18290890, EPI_ISL_18290989, EPI_ISL_18291808, EPI_ISL_18292038, EPI_ISL_18292398, EPI_ISL_18294574, EPI_ISL_18295441, EPI_ISL_18298019, EPI_ISL_18299948, EPI_ISL_18301587, EPI_ISL_18302636, EPI_ISL_18303012, EPI_ISL_18303592, EPI_ISL_18303595, EPI_ISL_18303758, EPI_ISL_18306254, EPI_ISL_18306922, EPI_ISL_18308642, EPI_ISL_18311951, EPI_ISL_18313683, EPI_ISL_18315747, EPI_ISL_18315789, EPI_ISL_18319306, EPI_ISL_18319903, EPI_ISL_18319904, EPI_ISL_18319906, EPI_ISL_18319907, EPI_ISL_18320079, EPI_ISL_18321271, EPI_ISL_18322273, EPI_ISL_18322420, EPI_ISL_18322438, EPI_ISL_18323536, EPI_ISL_18324107, EPI_ISL_18324168, EPI_ISL_18324491, EPI_ISL_18324976, EPI_ISL_18325145, EPI_ISL_18325563, EPI_ISL_18326430, EPI_ISL_18326597, EPI_ISL_18326806, EPI_ISL_18326807, EPI_ISL_18330957, EPI_ISL_18330966, EPI_ISL_18331347, EPI_ISL_18334903, EPI_ISL_18334945, EPI_ISL_18334986, EPI_ISL_18335084, EPI_ISL_18336165, EPI_ISL_18336602, EPI_ISL_18336862, EPI_ISL_18337738, EPI_ISL_18338137, EPI_ISL_18338143, EPI_ISL_18338144, EPI_ISL_18338502, EPI_ISL_18338504, EPI_ISL_18338709, EPI_ISL_18342412, EPI_ISL_18343598, EPI_ISL_18345777, EPI_ISL_18345926, EPI_ISL_18346109, EPI_ISL_18351588, EPI_ISL_18352473, EPI_ISL_18352485, EPI_ISL_18352489, EPI_ISL_18359229, EPI_ISL_18359328, EPI_ISL_18359679, EPI_ISL_18360507, EPI_ISL_18360944, EPI_ISL_18361202, EPI_ISL_18362265, EPI_ISL_18362515, EPI_ISL_18363170, EPI_ISL_18363300, EPI_ISL_18365170, EPI_ISL_18365256, EPI_ISL_18367086, EPI_ISL_18367563, EPI_ISL_18367586, EPI_ISL_18367599, EPI_ISL_18367615, EPI_ISL_18367908, EPI_ISL_18367992, EPI_ISL_18370898, EPI_ISL_18370960, EPI_ISL_18370967, EPI_ISL_18371749, EPI_ISL_18373201, EPI_ISL_18377021, EPI_ISL_18377214, EPI_ISL_18377245, EPI_ISL_18377248, EPI_ISL_18378384, EPI_ISL_18380731, EPI_ISL_18381066, EPI_ISL_18383121, EPI_ISL_18383423, EPI_ISL_18384846, EPI_ISL_18384936, EPI_ISL_18385358, EPI_ISL_18385924, EPI_ISL_18386091, EPI_ISL_18386114, EPI_ISL_18387037, EPI_ISL_18388509, EPI_ISL_18388585, EPI_ISL_18389783, EPI_ISL_18391451, EPI_ISL_18391597, EPI_ISL_18392259, EPI_ISL_18392502, EPI_ISL_18392841, EPI_ISL_18393366, EPI_ISL_18395551, EPI_ISL_18398210, EPI_ISL_18398259, EPI_ISL_18400843, EPI_ISL_18400856, EPI_ISL_18400946, EPI_ISL_18400987, EPI_ISL_18401313, EPI_ISL_18403047, EPI_ISL_18403051, EPI_ISL_18403054, EPI_ISL_18403509, EPI_ISL_18403523, EPI_ISL_18404585, EPI_ISL_18405535, EPI_ISL_18405621, EPI_ISL_18406078, EPI_ISL_18406394, EPI_ISL_18408561, EPI_ISL_18410987, EPI_ISL_18414567, EPI_ISL_18414568, EPI_ISL_18414808, EPI_ISL_18415823, EPI_ISL_18415832, EPI_ISL_18415834, EPI_ISL_18415840, EPI_ISL_18415854, EPI_ISL_18416870, EPI_ISL_18417129, EPI_ISL_18417211, EPI_ISL_18419485, EPI_ISL_18419748, EPI_ISL_18421674, EPI_ISL_18422693, EPI_ISL_18422715, EPI_ISL_18422771, EPI_ISL_18423785, EPI_ISL_18423814, EPI_ISL_18423907, EPI_ISL_18424281, EPI_ISL_18424468, EPI_ISL_18426836, EPI_ISL_18428844, EPI_ISL_18429684, EPI_ISL_18429702, EPI_ISL_18429725, EPI_ISL_18429773, EPI_ISL_18429797, EPI_ISL_18432077, EPI_ISL_18433350, EPI_ISL_18434194, EPI_ISL_18435557, EPI_ISL_18435892, EPI_ISL_18435949, EPI_ISL_18436693, EPI_ISL_18437342, EPI_ISL_18438723, EPI_ISL_18439733, EPI_ISL_18440037, EPI_ISL_18440370, EPI_ISL_18440660, EPI_ISL_18440866, EPI_ISL_18441868, EPI_ISL_18443784, EPI_ISL_18443944, EPI_ISL_18448894, EPI_ISL_18449647, EPI_ISL_18449794, EPI_ISL_18449820, EPI_ISL_18449892, EPI_ISL_18450249, EPI_ISL_18450812, EPI_ISL_18451678, EPI_ISL_18453400, EPI_ISL_18455292, EPI_ISL_18455564, EPI_ISL_18455706, EPI_ISL_18455950, EPI_ISL_18457808, EPI_ISL_18459512, EPI_ISL_18461774, EPI_ISL_18462852, EPI_ISL_18463490, EPI_ISL_18463766, EPI_ISL_18466251, EPI_ISL_18468149, EPI_ISL_18470400, EPI_ISL_18472311, EPI_ISL_18472312, EPI_ISL_18473559, EPI_ISL_18474126, EPI_ISL_18474555, EPI_ISL_18474665, EPI_ISL_18475534, EPI_ISL_18475535, EPI_ISL_18480741, EPI_ISL_18486919, EPI_ISL_18487225, EPI_ISL_18489646, EPI_ISL_18489793, EPI_ISL_18489829, EPI_ISL_18491841, EPI_ISL_18491844, EPI_ISL_18492277, EPI_ISL_18492305, EPI_ISL_18492307, EPI_ISL_18492412, EPI_ISL_18492455, EPI_ISL_18493129, EPI_ISL_18495416, EPI_ISL_18496252, EPI_ISL_18496585, EPI_ISL_18498001, EPI_ISL_18498420, EPI_ISL_18498499, EPI_ISL_18500316, EPI_ISL_18500771, EPI_ISL_18501087, EPI_ISL_18503287, EPI_ISL_18509817, EPI_ISL_18512421, EPI_ISL_18512438, EPI_ISL_18513936, EPI_ISL_18514552, EPI_ISL_18515280, EPI_ISL_18515328, EPI_ISL_18515343, EPI_ISL_18515511, EPI_ISL_18515749, EPI_ISL_18516916, EPI_ISL_18518769, EPI_ISL_18518932, EPI_ISL_18519113, EPI_ISL_18520677, EPI_ISL_18520678, EPI_ISL_18521575, EPI_ISL_18521765, EPI_ISL_18522184, EPI_ISL_18522580, EPI_ISL_18523129, EPI_ISL_18524926, EPI_ISL_18525067, EPI_ISL_18525840, EPI_ISL_18526641, EPI_ISL_18528453, EPI_ISL_18529555, EPI_ISL_18530445, EPI_ISL_18530449, EPI_ISL_18536100, EPI_ISL_18536853, EPI_ISL_18537013, EPI_ISL_18537032, EPI_ISL_18537373, EPI_ISL_18537428, EPI_ISL_18537814, EPI_ISL_18538015, EPI_ISL_18543268, EPI_ISL_18543705, EPI_ISL_18546112, EPI_ISL_18546287, EPI_ISL_18546551, EPI_ISL_18546715, EPI_ISL_18551440, EPI_ISL_18552697, EPI_ISL_18553587, EPI_ISL_18554053, EPI_ISL_18556084, EPI_ISL_18556539, EPI_ISL_18557145, EPI_ISL_18558360, EPI_ISL_18558385, EPI_ISL_18558412, EPI_ISL_18558468, EPI_ISL_18558477, EPI_ISL_18559317, EPI_ISL_18560556, EPI_ISL_18560725, EPI_ISL_18560872, EPI_ISL_18561098, EPI_ISL_18563181, EPI_ISL_18563821, EPI_ISL_18564403, EPI_ISL_18564755, EPI_ISL_18566696, EPI_ISL_18567985, EPI_ISL_18568124, EPI_ISL_18576266, EPI_ISL_18576754, EPI_ISL_18577842, EPI_ISL_18577862, EPI_ISL_18577966, EPI_ISL_18578195, EPI_ISL_18579981, EPI_ISL_18580011, EPI_ISL_18580750, EPI_ISL_18580874, EPI_ISL_18581347, EPI_ISL_18584141, EPI_ISL_18588773, EPI_ISL_18589012, EPI_ISL_18589243, EPI_ISL_18589475, EPI_ISL_18589669, EPI_ISL_18590820, EPI_ISL_18591717, EPI_ISL_18593579, EPI_ISL_18594183, EPI_ISL_18594233, EPI_ISL_18594266, EPI_ISL_18595212, EPI_ISL_18598503, EPI_ISL_18598525, EPI_ISL_18603922, EPI_ISL_18604501, EPI_ISL_18604502, EPI_ISL_18604503, EPI_ISL_18604504, EPI_ISL_18604505, EPI_ISL_18605012, EPI_ISL_18605132, EPI_ISL_18606460, EPI_ISL_18607149, EPI_ISL_18607150, EPI_ISL_18608694, EPI_ISL_18609973, EPI_ISL_18612246, EPI_ISL_18615968, EPI_ISL_18622139, EPI_ISL_18624843, EPI_ISL_18625316, EPI_ISL_18626713, EPI_ISL_18626714, EPI_ISL_18626750, EPI_ISL_18630930, EPI_ISL_18633829, EPI_ISL_18634703, EPI_ISL_18635526, EPI_ISL_18635546, EPI_ISL_18635599, EPI_ISL_18635961, EPI_ISL_18639151, EPI_ISL_18640058, EPI_ISL_18641470, EPI_ISL_18641499, EPI_ISL_18642608, EPI_ISL_18646912, EPI_ISL_18646945, EPI_ISL_18648209, EPI_ISL_18652556, EPI_ISL_18653898, EPI_ISL_18654501, EPI_ISL_18668995, EPI_ISL_18672102, EPI_ISL_18674382, EPI_ISL_18677704, EPI_ISL_18681665, EPI_ISL_18681942, EPI_ISL_18681975, EPI_ISL_18683430, EPI_ISL_18687895, EPI_ISL_18689964, EPI_ISL_18693860, EPI_ISL_18694187, EPI_ISL_18702737, EPI_ISL_18712665, EPI_ISL_18712806, EPI_ISL_18712994, EPI_ISL_18713198, EPI_ISL_18714360, EPI_ISL_18714857, EPI_ISL_18715065, EPI_ISL_18715624, EPI_ISL_18717261, EPI_ISL_18727247, EPI_ISL_18727294, EPI_ISL_18729637, EPI_ISL_18730563, EPI_ISL_18740049, EPI_ISL_18742836, EPI_ISL_18743266, EPI_ISL_18743442, EPI_ISL_18743504, EPI_ISL_18743721, EPI_ISL_18759772, EPI_ISL_18760241, EPI_ISL_18763780, EPI_ISL_18766752, EPI_ISL_18772294, EPI_ISL_18777179, EPI_ISL_18778198, EPI_ISL_18778941, EPI_ISL_18782000, EPI_ISL_18782699, EPI_ISL_18784365, EPI_ISL_18784385, EPI_ISL_18784404, EPI_ISL_18784438, EPI_ISL_18787342, EPI_ISL_18787343, EPI_ISL_18792770, EPI_ISL_18796930, EPI_ISL_18797833, EPI_ISL_18798193, EPI_ISL_18798198, EPI_ISL_18798202, EPI_ISL_18798204, EPI_ISL_18798234, EPI_ISL_18799019, EPI_ISL_18801405, EPI_ISL_18803784, EPI_ISL_18805014, EPI_ISL_18809509, EPI_ISL_18809652, EPI_ISL_18811047, EPI_ISL_18811325, EPI_ISL_18811375, EPI_ISL_18811389, EPI_ISL_18814271, EPI_ISL_18814272, EPI_ISL_18814273, EPI_ISL_18815556, EPI_ISL_18816278, EPI_ISL_18816628, EPI_ISL_18817415, EPI_ISL_18818949, EPI_ISL_18820996, EPI_ISL_18824608, EPI_ISL_18825438, EPI_ISL_18828912, EPI_ISL_18829708, EPI_ISL_18832540, EPI_ISL_18832845, EPI_ISL_18839074, EPI_ISL_18839275, EPI_ISL_18839824, EPI_ISL_18839850, EPI_ISL_18839932, EPI_ISL_18841696, EPI_ISL_18842709, EPI_ISL_18846230, EPI_ISL_18851473, EPI_ISL_18853389, EPI_ISL_18853925, EPI_ISL_18854197, EPI_ISL_18854790, EPI_ISL_18854974, EPI_ISL_18856190, EPI_ISL_18856328, EPI_ISL_18859868, EPI_ISL_18860821, EPI_ISL_18863734, EPI_ISL_18864357, EPI_ISL_18864823, EPI_ISL_18865364, EPI_ISL_18865427, EPI_ISL_18868940, EPI_ISL_18869806, EPI_ISL_18870297, EPI_ISL_18873437, EPI_ISL_18873899, EPI_ISL_18874096, EPI_ISL_18874749, EPI_ISL_18875008, EPI_ISL_18876158, EPI_ISL_18876159, EPI_ISL_18876160, EPI_ISL_18876161, EPI_ISL_18876162, EPI_ISL_18876164, EPI_ISL_18876166, EPI_ISL_18876167, EPI_ISL_18876171, EPI_ISL_18876174, EPI_ISL_18876176, EPI_ISL_18876178, EPI_ISL_18876356, EPI_ISL_18877288, EPI_ISL_18877358, EPI_ISL_18877379, EPI_ISL_18877954, EPI_ISL_18878665, EPI_ISL_18880075, EPI_ISL_18882888, EPI_ISL_18884114, EPI_ISL_18885028, EPI_ISL_18885545, EPI_ISL_18886798, EPI_ISL_18888145, EPI_ISL_18888340, EPI_ISL_18889879, EPI_ISL_18892766, EPI_ISL_18896020, EPI_ISL_18897180, EPI_ISL_18897181, EPI_ISL_18899656, EPI_ISL_18899840, EPI_ISL_18900159, EPI_ISL_18901224, EPI_ISL_18901411, EPI_ISL_18901513, EPI_ISL_18901762, EPI_ISL_18901997, EPI_ISL_18904234, EPI_ISL_18906411, EPI_ISL_18906577, EPI_ISL_18908924, EPI_ISL_18914858, EPI_ISL_18915560, EPI_ISL_18916107, EPI_ISL_18916798, EPI_ISL_18916914, EPI_ISL_18917364, EPI_ISL_18917482, EPI_ISL_18917496, EPI_ISL_18917541, EPI_ISL_18918931, EPI_ISL_18918932, EPI_ISL_18918944, EPI_ISL_18919477, EPI_ISL_18919544, EPI_ISL_18924428, EPI_ISL_18927022, EPI_ISL_18927251, EPI_ISL_18927258, EPI_ISL_18927292, EPI_ISL_18927500, EPI_ISL_18927514, EPI_ISL_18927587, EPI_ISL_18930149, EPI_ISL_18930177, EPI_ISL_18930182, EPI_ISL_18930190, EPI_ISL_18930681, EPI_ISL_18931503, EPI_ISL_18931504, EPI_ISL_18931535, EPI_ISL_18931538, EPI_ISL_18931542, EPI_ISL_18932507, EPI_ISL_18933211, EPI_ISL_18933315, EPI_ISL_18933410, EPI_ISL_18933606, EPI_ISL_18933647, EPI_ISL_18935631, EPI_ISL_18936396, EPI_ISL_18939566, EPI_ISL_18939949, EPI_ISL_18945392, EPI_ISL_18945748, EPI_ISL_18947086, EPI_ISL_18947189, EPI_ISL_18947944, EPI_ISL_18948077, EPI_ISL_18948078, EPI_ISL_18948422, EPI_ISL_18948613, EPI_ISL_18948756, EPI_ISL_18948757, EPI_ISL_18949161, EPI_ISL_18949911, EPI_ISL_18952873, EPI_ISL_18953132, EPI_ISL_18953134, EPI_ISL_18953234, EPI_ISL_18953461, EPI_ISL_18953493, EPI_ISL_18953503, EPI_ISL_18953522, EPI_ISL_18955186, EPI_ISL_18956070, EPI_ISL_18956087, EPI_ISL_18956161, EPI_ISL_18958247, EPI_ISL_18958295, EPI_ISL_18958383, EPI_ISL_18960149, EPI_ISL_18960183, EPI_ISL_18960224, EPI_ISL_18962896, EPI_ISL_18962928, EPI_ISL_18963652, EPI_ISL_18965082, EPI_ISL_18965451, EPI_ISL_18966035, EPI_ISL_18966186, EPI_ISL_18968116, EPI_ISL_18968120, EPI_ISL_18969119, EPI_ISL_18969697, EPI_ISL_18969735, EPI_ISL_18970692, EPI_ISL_18971336, EPI_ISL_18972187, EPI_ISL_18972699, EPI_ISL_18972708, EPI_ISL_18972719, EPI_ISL_18972734, EPI_ISL_18972995, EPI_ISL_18975064, EPI_ISL_18975148, EPI_ISL_18975189, EPI_ISL_18975333, EPI_ISL_18977928, EPI_ISL_18977929, EPI_ISL_18979948, EPI_ISL_18980164, EPI_ISL_18981298, EPI_ISL_18982763, EPI_ISL_18986641, EPI_ISL_18986642, EPI_ISL_18986747, EPI_ISL_18987173, EPI_ISL_18987258, EPI_ISL_18987259, EPI_ISL_18987545, EPI_ISL_18990023, EPI_ISL_18992540, EPI_ISL_18999516, EPI_ISL_19000455, EPI_ISL_19002243, EPI_ISL_19002832, EPI_ISL_19003837, EPI_ISL_19004571, EPI_ISL_19005468, EPI_ISL_19005470, EPI_ISL_19005472, EPI_ISL_19005481, EPI_ISL_19006725, EPI_ISL_19008875, EPI_ISL_19009123, EPI_ISL_19009131, EPI_ISL_19010135, EPI_ISL_19012038, EPI_ISL_19012114, EPI_ISL_19012460, EPI_ISL_19012472, EPI_ISL_19012554, EPI_ISL_19012663, EPI_ISL_19013418, EPI_ISL_19014517, EPI_ISL_19014601, EPI_ISL_19015115, EPI_ISL_19015882, EPI_ISL_19016044, EPI_ISL_19016053, EPI_ISL_19016370, EPI_ISL_19017499, EPI_ISL_19019122, EPI_ISL_19019127, EPI_ISL_19019128, EPI_ISL_19021045, EPI_ISL_19021046, EPI_ISL_19022071, EPI_ISL_19022479, EPI_ISL_19024260, EPI_ISL_19026222, EPI_ISL_19026577, EPI_ISL_19026932, EPI_ISL_19029499, EPI_ISL_19030027, EPI_ISL_19030112, EPI_ISL_19030117, EPI_ISL_19030129, EPI_ISL_19030183, EPI_ISL_19031583, EPI_ISL_19032054, EPI_ISL_19032056, EPI_ISL_19032179, EPI_ISL_19032598, EPI_ISL_19033249, EPI_ISL_19035801, EPI_ISL_19036115, EPI_ISL_19036116, EPI_ISL_19036117, EPI_ISL_19036644, EPI_ISL_19036878, EPI_ISL_19036921, EPI_ISL_19041379, EPI_ISL_19041967, EPI_ISL_19042675, EPI_ISL_19043835, EPI_ISL_19043836, EPI_ISL_19044085, EPI_ISL_19044163, EPI_ISL_19044218, EPI_ISL_19044280, EPI_ISL_19044474, EPI_ISL_19044644, EPI_ISL_19044737, EPI_ISL_19046891, EPI_ISL_19046892, EPI_ISL_19047408, EPI_ISL_19049407, EPI_ISL_19049586, EPI_ISL_19050509, EPI_ISL_19051602, EPI_ISL_19051976, EPI_ISL_19052024, EPI_ISL_19052034, EPI_ISL_19052090, EPI_ISL_19052102, EPI_ISL_19052117, EPI_ISL_19053178, EPI_ISL_19053803, EPI_ISL_19053806, EPI_ISL_19053808, EPI_ISL_19054784, EPI_ISL_19055364, EPI_ISL_19055368, EPI_ISL_19055382, EPI_ISL_19055909, EPI_ISL_19058120, EPI_ISL_19060002, EPI_ISL_19060327, EPI_ISL_19060922, EPI_ISL_19061139, EPI_ISL_19061646, EPI_ISL_19062162, EPI_ISL_19062523, EPI_ISL_19062524, EPI_ISL_19064175, EPI_ISL_19065060, EPI_ISL_19065882, EPI_ISL_19066171, EPI_ISL_19066817, EPI_ISL_19067782, EPI_ISL_19067788, EPI_ISL_19070471, EPI_ISL_19071618, EPI_ISL_19073169, EPI_ISL_19073731, EPI_ISL_19073732, EPI_ISL_19073800, EPI_ISL_19073807, EPI_ISL_19074673, EPI_ISL_19075261, EPI_ISL_19075267, EPI_ISL_19075350, EPI_ISL_19076088, EPI_ISL_19081415, EPI_ISL_19081416, EPI_ISL_19081419, EPI_ISL_19081422, EPI_ISL_19081425, EPI_ISL_19082080, EPI_ISL_19082092, EPI_ISL_19082200, EPI_ISL_19082275, EPI_ISL_19082487, EPI_ISL_19085329, EPI_ISL_19085381, EPI_ISL_19085539, EPI_ISL_19085540, EPI_ISL_19085542, EPI_ISL_19086378, EPI_ISL_19086393, EPI_ISL_19086523, EPI_ISL_19091019, EPI_ISL_19091144, EPI_ISL_19094153, EPI_ISL_19094369, EPI_ISL_19095154, EPI_ISL_19095156, EPI_ISL_19095501, EPI_ISL_19095606, EPI_ISL_19095734, EPI_ISL_19095768, EPI_ISL_19100255, EPI_ISL_19100981, EPI_ISL_19105055, EPI_ISL_19106368, EPI_ISL_19106842, EPI_ISL_19106844, EPI_ISL_19106999, EPI_ISL_19108198, EPI_ISL_19108199, EPI_ISL_19108200, EPI_ISL_19108208, EPI_ISL_19108340, EPI_ISL_19108650, EPI_ISL_19108704, EPI_ISL_19108706, EPI_ISL_19108725, EPI_ISL_19131415, EPI_ISL_19131416, EPI_ISL_19131417, EPI_ISL_19131418, EPI_ISL_19131419, EPI_ISL_19132785, EPI_ISL_19132846, EPI_ISL_19133800, EPI_ISL_19135448, EPI_ISL_19135481, EPI_ISL_19135505, EPI_ISL_19135510, EPI_ISL_19135511, EPI_ISL_19137782, EPI_ISL_19137844, EPI_ISL_19140762, EPI_ISL_19141912, EPI_ISL_19142916, EPI_ISL_19143060, EPI_ISL_19143411, EPI_ISL_19143412, EPI_ISL_19143551, EPI_ISL_19143854, EPI_ISL_19143866, EPI_ISL_19143953, EPI_ISL_19143954, EPI_ISL_19145335, EPI_ISL_19146181, EPI_ISL_19146282, EPI_ISL_19146283, EPI_ISL_19146308, EPI_ISL_19146747, EPI_ISL_19147578, EPI_ISL_19148775, EPI_ISL_19151438, EPI_ISL_19153859, EPI_ISL_19158778, EPI_ISL_19158782, EPI_ISL_19159275, EPI_ISL_19159918, EPI_ISL_19159922, EPI_ISL_19159923, EPI_ISL_19161805, EPI_ISL_19164072, EPI_ISL_19164956, EPI_ISL_19165347, EPI_ISL_19165721, EPI_ISL_19167420, EPI_ISL_19167714, EPI_ISL_19169236, EPI_ISL_19169239, EPI_ISL_19169240, EPI_ISL_19169597, EPI_ISL_19169598, EPI_ISL_19173640, EPI_ISL_19173786, EPI_ISL_19175160, EPI_ISL_19175165, EPI_ISL_19175555, EPI_ISL_19175939, EPI_ISL_19176642, EPI_ISL_19176853, EPI_ISL_19176930, EPI_ISL_19177366, EPI_ISL_19177549, EPI_ISL_19177604, EPI_ISL_19177633, EPI_ISL_19178043, EPI_ISL_19178311, EPI_ISL_19179820, EPI_ISL_19182830, EPI_ISL_19182893, EPI_ISL_19183095, EPI_ISL_19183664, EPI_ISL_19183910, EPI_ISL_19183917, EPI_ISL_19183924, EPI_ISL_19184000, EPI_ISL_19184031, EPI_ISL_19184858, EPI_ISL_19185391, EPI_ISL_19186372, EPI_ISL_19187674, EPI_ISL_19187980, EPI_ISL_19188471, EPI_ISL_19189250, EPI_ISL_19190171, EPI_ISL_19190915, EPI_ISL_19191046, EPI_ISL_19192623, EPI_ISL_19192806, EPI_ISL_19192819, EPI_ISL_19193007, EPI_ISL_19193543, EPI_ISL_19193606, EPI_ISL_19193607, EPI_ISL_19193617, EPI_ISL_19195977, EPI_ISL_19195978, EPI_ISL_19195979, EPI_ISL_19195980, EPI_ISL_19195981, EPI_ISL_19195982, EPI_ISL_19196018, EPI_ISL_19196019, EPI_ISL_19196020, EPI_ISL_19196021, EPI_ISL_19197906, EPI_ISL_19198127, EPI_ISL_19198259, EPI_ISL_19199717, EPI_ISL_19199719, EPI_ISL_19200980, EPI_ISL_19202065, EPI_ISL_19202066, EPI_ISL_19203296, EPI_ISL_19205927, EPI_ISL_19209656, EPI_ISL_19210836, EPI_ISL_19211459, EPI_ISL_19213222, EPI_ISL_19213351, EPI_ISL_19214251, EPI_ISL_19214614, EPI_ISL_19216679, EPI_ISL_19216828, EPI_ISL_19217899, EPI_ISL_19219811, EPI_ISL_19221626, EPI_ISL_19223353, EPI_ISL_19225911, EPI_ISL_19226495, EPI_ISL_19227219, EPI_ISL_19228129, EPI_ISL_19229097, EPI_ISL_19229565, EPI_ISL_19230859, EPI_ISL_19230867, EPI_ISL_19230971, EPI_ISL_19232972, EPI_ISL_19234843, EPI_ISL_19237942, EPI_ISL_19239019, EPI_ISL_19239628, EPI_ISL_19239713, EPI_ISL_19243158, EPI_ISL_19243166, EPI_ISL_19243171, EPI_ISL_19243231, EPI_ISL_19243516, EPI_ISL_19243810, EPI_ISL_19244002, EPI_ISL_19245397, EPI_ISL_19245398, EPI_ISL_19245399, EPI_ISL_19251154, EPI_ISL_19254798, EPI_ISL_19256132, EPI_ISL_19256137, EPI_ISL_19256138, EPI_ISL_19256151, EPI_ISL_19256152, EPI_ISL_19257013, EPI_ISL_19257014, EPI_ISL_19257017, EPI_ISL_19257108, EPI_ISL_19257977, EPI_ISL_19259365, EPI_ISL_19259369, EPI_ISL_19259383, EPI_ISL_19260849, EPI_ISL_19260850, EPI_ISL_19261083, EPI_ISL_19261711, EPI_ISL_19264977, EPI_ISL_19265076, EPI_ISL_19267631, EPI_ISL_19268349, EPI_ISL_19269083, EPI_ISL_19269483, EPI_ISL_19271166, EPI_ISL_19271182, EPI_ISL_19271183, EPI_ISL_19271567, EPI_ISL_19271746, EPI_ISL_19273074, EPI_ISL_19277027, EPI_ISL_19277033, EPI_ISL_19280260, EPI_ISL_19281017, EPI_ISL_19282183, EPI_ISL_19282184, EPI_ISL_19283996, EPI_ISL_19286133, EPI_ISL_19286135, EPI_ISL_19286138, EPI_ISL_19286139, EPI_ISL_19286260, EPI_ISL_19287266, EPI_ISL_19287684, EPI_ISL_19288763, EPI_ISL_19290906, EPI_ISL_19292343, EPI_ISL_19292838, EPI_ISL_19292841, EPI_ISL_19297116, EPI_ISL_19297576, EPI_ISL_19298817, EPI_ISL_19299051, EPI_ISL_19300378, EPI_ISL_19300385, EPI_ISL_19300400, EPI_ISL_19301040, EPI_ISL_19302363, EPI_ISL_19302407, EPI_ISL_19304883, EPI_ISL_19308329, EPI_ISL_19308670, EPI_ISL_19308767, EPI_ISL_19308875, EPI_ISL_19308876, EPI_ISL_19308877, EPI_ISL_19308906, EPI_ISL_19309906, EPI_ISL_19310234, EPI_ISL_19311057, EPI_ISL_19311769, EPI_ISL_19312520, EPI_ISL_19318476, EPI_ISL_19319006, EPI_ISL_19320511, EPI_ISL_19322462, EPI_ISL_19324934, EPI_ISL_19326359, EPI_ISL_19331433, EPI_ISL_19332294, EPI_ISL_19333087, EPI_ISL_19335441, EPI_ISL_19340140, EPI_ISL_19341128, EPI_ISL_19341144, EPI_ISL_19344211, EPI_ISL_19345465, EPI_ISL_19345845, EPI_ISL_19346102, EPI_ISL_19346538, EPI_ISL_19348673, EPI_ISL_19351027, EPI_ISL_19351032, EPI_ISL_19351033, EPI_ISL_19351648, EPI_ISL_19351927, EPI_ISL_19359999, EPI_ISL_19360941, EPI_ISL_19362874, EPI_ISL_19362955, EPI_ISL_19363093, EPI_ISL_19364322, EPI_ISL_19364675, EPI_ISL_19365917, EPI_ISL_19369553, EPI_ISL_19369713, EPI_ISL_19374392, EPI_ISL_19374843, EPI_ISL_19374844, EPI_ISL_19374845, EPI_ISL_19380467, EPI_ISL_19381264, EPI_ISL_19381428, EPI_ISL_19381638, EPI_ISL_19381992, EPI_ISL_19381994, EPI_ISL_19382602, EPI_ISL_19383694, EPI_ISL_19384121, EPI_ISL_19385914, EPI_ISL_19385980, EPI_ISL_19387703, EPI_ISL_19388165, EPI_ISL_19388758, EPI_ISL_19391206, EPI_ISL_19391216, EPI_ISL_19393434, EPI_ISL_19393708, EPI_ISL_19398369, EPI_ISL_19405918, EPI_ISL_19408692, EPI_ISL_19408693, EPI_ISL_19410044, EPI_ISL_19410056, EPI_ISL_19410058, EPI_ISL_19411869, EPI_ISL_19412054, EPI_ISL_19414452, EPI_ISL_19414669, EPI_ISL_19414842, EPI_ISL_19415183, EPI_ISL_19415272, EPI_ISL_19415273, EPI_ISL_19415333, EPI_ISL_19417987, EPI_ISL_19418385, EPI_ISL_19418789, EPI_ISL_19419991, EPI_ISL_19425659, EPI_ISL_19427049, EPI_ISL_19427050, EPI_ISL_19427051, EPI_ISL_19428450, EPI_ISL_19428673, EPI_ISL_19431719, EPI_ISL_19433335, EPI_ISL_19434973, EPI_ISL_19438222, EPI_ISL_19441794, EPI_ISL_19446721, EPI_ISL_19446726, EPI_ISL_19447859, EPI_ISL_19450094, EPI_ISL_19452022, EPI_ISL_19456759, EPI_ISL_19457982, EPI_ISL_19458104, EPI_ISL_19458866, EPI_ISL_19459469, EPI_ISL_19463531, EPI_ISL_19463787, EPI_ISL_19463811, EPI_ISL_19464534, EPI_ISL_19465468, EPI_ISL_19467713, EPI_ISL_19467725, EPI_ISL_19468710, EPI_ISL_19473728, EPI_ISL_19474606, EPI_ISL_19474613, EPI_ISL_19477107, EPI_ISL_19478383, EPI_ISL_19478598, EPI_ISL_19479514, EPI_ISL_19480237, EPI_ISL_19482235, EPI_ISL_19483178, EPI_ISL_19483184, EPI_ISL_19483313, EPI_ISL_19486141, EPI_ISL_19488917, EPI_ISL_19491347, EPI_ISL_19495855, EPI_ISL_19498393, EPI_ISL_19499089, EPI_ISL_19499133, EPI_ISL_19499199, EPI_ISL_19499228, EPI_ISL_19499640, EPI_ISL_19499789, EPI_ISL_19500407, EPI_ISL_19502648, EPI_ISL_19503869, EPI_ISL_19506293, EPI_ISL_19506337, EPI_ISL_19506387, EPI_ISL_19506391, EPI_ISL_19506410, EPI_ISL_19506412, EPI_ISL_19506489, EPI_ISL_19506517, EPI_ISL_19506572, EPI_ISL_19508878, EPI_ISL_19510518, EPI_ISL_19511239, EPI_ISL_19511242, EPI_ISL_19511245, EPI_ISL_19511259, EPI_ISL_19511353, EPI_ISL_19512930, EPI_ISL_19513189, EPI_ISL_19513373, EPI_ISL_19513376, EPI_ISL_19517945, EPI_ISL_19519860, EPI_ISL_19521406, EPI_ISL_19522101, EPI_ISL_19526975, EPI_ISL_19527216, EPI_ISL_19527499, EPI_ISL_19529679, EPI_ISL_19529699, EPI_ISL_19530201, EPI_ISL_19530213, EPI_ISL_19530220, EPI_ISL_19532518, EPI_ISL_19535971, EPI_ISL_19536358, EPI_ISL_19537504, EPI_ISL_19537517, EPI_ISL_19537806, EPI_ISL_19539586, EPI_ISL_19539602, EPI_ISL_19541318, EPI_ISL_19543648, EPI_ISL_19543801, EPI_ISL_19546384, EPI_ISL_19550039, EPI_ISL_19550107, EPI_ISL_19551038, EPI_ISL_19551134, EPI_ISL_19555102, EPI_ISL_19555114, EPI_ISL_19555115, EPI_ISL_19557993, EPI_ISL_19558034, EPI_ISL_19560436, EPI_ISL_19560828, EPI_ISL_19560838, EPI_ISL_19561072, EPI_ISL_19561079, EPI_ISL_19562666, EPI_ISL_19575892, EPI_ISL_19578009, EPI_ISL_19578384, EPI_ISL_19585411, EPI_ISL_19585472, EPI_ISL_19588050, EPI_ISL_19588617, EPI_ISL_19588630, EPI_ISL_19589209, EPI_ISL_19589226, EPI_ISL_19589644, EPI_ISL_19591990, EPI_ISL_19600727, EPI_ISL_19601813, EPI_ISL_19603076, EPI_ISL_19603396, EPI_ISL_19603766, EPI_ISL_19606872, EPI_ISL_19606873, EPI_ISL_19606893, EPI_ISL_19613477, EPI_ISL_19613481, EPI_ISL_19615536, EPI_ISL_19615541, EPI_ISL_19615759, EPI_ISL_19616027, EPI_ISL_19616479, EPI_ISL_19618593, EPI_ISL_19619689, EPI_ISL_19619734, EPI_ISL_19619757, EPI_ISL_19620010, EPI_ISL_19621954, EPI_ISL_19622266, EPI_ISL_19623358, EPI_ISL_19623503, EPI_ISL_19627737, EPI_ISL_19627776, EPI_ISL_19627778, EPI_ISL_19630975, EPI_ISL_19631111, EPI_ISL_19632962, EPI_ISL_19634022, EPI_ISL_19636173, EPI_ISL_19638051, EPI_ISL_19639319, EPI_ISL_19640624, EPI_ISL_19641389, EPI_ISL_19641840, EPI_ISL_19642771, EPI_ISL_19643417, EPI_ISL_19643444, EPI_ISL_19643542, EPI_ISL_19643616, EPI_ISL_19643666, EPI_ISL_19645977, EPI_ISL_19648297, EPI_ISL_19655163, EPI_ISL_19656451, EPI_ISL_19657328, EPI_ISL_19657519, EPI_ISL_19657528, EPI_ISL_19660180, EPI_ISL_19661881, EPI_ISL_19661997, EPI_ISL_19665170, EPI_ISL_19666574, EPI_ISL_19671026, EPI_ISL_19671598, EPI_ISL_19671605, EPI_ISL_19671606, EPI_ISL_19671607, EPI_ISL_19671887, EPI_ISL_19673194, EPI_ISL_19673201, EPI_ISL_19676136, EPI_ISL_19680534, EPI_ISL_19683226, EPI_ISL_19683235, EPI_ISL_19683886, EPI_ISL_19685790, EPI_ISL_19685811, EPI_ISL_19685815, EPI_ISL_19685993, EPI_ISL_19688052, EPI_ISL_19689294, EPI_ISL_19690695, EPI_ISL_19693383, EPI_ISL_19693416, EPI_ISL_19693419, EPI_ISL_19696986, EPI_ISL_19703902, EPI_ISL_19704129, EPI_ISL_19704643, EPI_ISL_19704646, EPI_ISL_19706712, EPI_ISL_19706713, EPI_ISL_19710331, EPI_ISL_19710359, EPI_ISL_19711118, EPI_ISL_19711122, EPI_ISL_19713810, EPI_ISL_19713821, EPI_ISL_19713832, EPI_ISL_19714066, EPI_ISL_19715063, EPI_ISL_19717002, EPI_ISL_19719176, EPI_ISL_19719216, EPI_ISL_19719218, EPI_ISL_19720736, EPI_ISL_19720749, EPI_ISL_19720751, EPI_ISL_19720757, EPI_ISL_19720765, EPI_ISL_19720774, EPI_ISL_19720781, EPI_ISL_19721494, EPI_ISL_19721498, EPI_ISL_19729765, EPI_ISL_19729811, EPI_ISL_19729840, EPI_ISL_19730527, EPI_ISL_19735517, EPI_ISL_19736097, EPI_ISL_19736266, EPI_ISL_19736299, EPI_ISL_19736349, EPI_ISL_19736527, EPI_ISL_19739887, EPI_ISL_19742521, EPI_ISL_19743439, EPI_ISL_19743558, EPI_ISL_19749286, EPI_ISL_19749348, EPI_ISL_19750081, EPI_ISL_19750087, EPI_ISL_19750479, EPI_ISL_19751001, EPI_ISL_19752288, EPI_ISL_19755631, EPI_ISL_19760571, EPI_ISL_19761402, EPI_ISL_19765520, EPI_ISL_19765527, EPI_ISL_19766581, EPI_ISL_19766606, EPI_ISL_19770265, EPI_ISL_19771106, EPI_ISL_19773769, EPI_ISL_19773770, EPI_ISL_19775232, EPI_ISL_19776542, EPI_ISL_19782005, EPI_ISL_19783524, EPI_ISL_19788439, EPI_ISL_19791384, EPI_ISL_19794365, EPI_ISL_19795466, EPI_ISL_19799368, EPI_ISL_19799392, EPI_ISL_19799712, EPI_ISL_19799810, EPI_ISL_19800009, EPI_ISL_19800089, EPI_ISL_19800094, EPI_ISL_19800098, EPI_ISL_19800151, EPI_ISL_19801979, EPI_ISL_19803172, EPI_ISL_19803215, EPI_ISL_19803225, EPI_ISL_19803382, EPI_ISL_19804293, EPI_ISL_19804491, EPI_ISL_19805234, EPI_ISL_19806441, EPI_ISL_19806502, EPI_ISL_19806836, EPI_ISL_19806981, EPI_ISL_19807128, EPI_ISL_19807229, EPI_ISL_19808706, EPI_ISL_19810078, EPI_ISL_19810444, EPI_ISL_19810684, EPI_ISL_19812168, EPI_ISL_19814897, EPI_ISL_19815994, EPI_ISL_19816020, EPI_ISL_19816100, EPI_ISL_19816143, EPI_ISL_19816182, EPI_ISL_19816207, EPI_ISL_19816266, EPI_ISL_19816276, EPI_ISL_19816297, EPI_ISL_19816344, EPI_ISL_19816356, EPI_ISL_19816434, EPI_ISL_19819753")
    MOV = stringlist_to_strings_set("EPI_ISL_15609973, EPI_ISL_15907744, EPI_ISL_7390853, EPI_ISL_7714464, EPI_ISL_7833680, EPI_ISL_7974433, EPI_ISL_7974434, EPI_ISL_7974435, EPI_ISL_9246286, EPI_ISL_9961511, EPI_ISL_10512732, EPI_ISL_10513884, EPI_ISL_11022200, EPI_ISL_11141125, EPI_ISL_11174220, EPI_ISL_11224116, EPI_ISL_11353416, EPI_ISL_11664910, EPI_ISL_11782428, EPI_ISL_12014556, EPI_ISL_12325408, EPI_ISL_12559054, EPI_ISL_12666444, EPI_ISL_12830302, EPI_ISL_13276436, EPI_ISL_13294593, EPI_ISL_13358749, EPI_ISL_13358804, EPI_ISL_13376289, EPI_ISL_13723805, EPI_ISL_14138850, EPI_ISL_14346370, EPI_ISL_14429354, EPI_ISL_14466939, EPI_ISL_14467031, EPI_ISL_14487304, EPI_ISL_14526287, EPI_ISL_14573187, EPI_ISL_14580651, EPI_ISL_14601448, EPI_ISL_14616144, EPI_ISL_14667656, EPI_ISL_14788088, EPI_ISL_14807137, EPI_ISL_14809350, EPI_ISL_14818681, EPI_ISL_14830201, EPI_ISL_14916287, EPI_ISL_14916417, EPI_ISL_14916471, EPI_ISL_14929774, EPI_ISL_14945818, EPI_ISL_14946908, EPI_ISL_15081472, EPI_ISL_15106590, EPI_ISL_15106739, EPI_ISL_15111552, EPI_ISL_15113210, EPI_ISL_15118530, EPI_ISL_15118652, EPI_ISL_15119550, EPI_ISL_15119586, EPI_ISL_15119587, EPI_ISL_15119899, EPI_ISL_15125352, EPI_ISL_15156296, EPI_ISL_15157642, EPI_ISL_15160596, EPI_ISL_15176036, EPI_ISL_15192964, EPI_ISL_15212355, EPI_ISL_15212402, EPI_ISL_15258694, EPI_ISL_15258697, EPI_ISL_15258702, EPI_ISL_15258714, EPI_ISL_15258754, EPI_ISL_15260772, EPI_ISL_15276586, EPI_ISL_15301009, EPI_ISL_15301025, EPI_ISL_15341577, EPI_ISL_15343310, EPI_ISL_15352141, EPI_ISL_15354679, EPI_ISL_15356322, EPI_ISL_15357173, EPI_ISL_15357186, EPI_ISL_15357564, EPI_ISL_15385709, EPI_ISL_15385769, EPI_ISL_15395718, EPI_ISL_15410430, EPI_ISL_15428954, EPI_ISL_15471118, EPI_ISL_15471127, EPI_ISL_15476782, EPI_ISL_15476782-15476784, EPI_ISL_15476783, EPI_ISL_15476784, EPI_ISL_15476838, EPI_ISL_15476890, EPI_ISL_15477002, EPI_ISL_15477006, EPI_ISL_15477014, EPI_ISL_15477080, EPI_ISL_15477087, EPI_ISL_15477089, EPI_ISL_15477122, EPI_ISL_15477123, EPI_ISL_15477133, EPI_ISL_15477167, EPI_ISL_15523458, EPI_ISL_15532754, EPI_ISL_15532755, EPI_ISL_15532776, EPI_ISL_15532778, EPI_ISL_15533551, EPI_ISL_15546710, EPI_ISL_15578381, EPI_ISL_15581681, EPI_ISL_15595984, EPI_ISL_15629811, EPI_ISL_15641087, EPI_ISL_15652740, EPI_ISL_15666161, EPI_ISL_15670984, EPI_ISL_15673084, EPI_ISL_15673196, EPI_ISL_15673587, EPI_ISL_15693720, EPI_ISL_15697288, EPI_ISL_15708555, EPI_ISL_15729147, EPI_ISL_15731617, EPI_ISL_15731631, EPI_ISL_15732415, EPI_ISL_15741410, EPI_ISL_15755946, EPI_ISL_15755997, EPI_ISL_15756004, EPI_ISL_15756044, EPI_ISL_15765732, EPI_ISL_15797427, EPI_ISL_15797464, EPI_ISL_15797751, EPI_ISL_15797823, EPI_ISL_15797872, EPI_ISL_15805129, EPI_ISL_15808072, EPI_ISL_15820336, EPI_ISL_15820414, EPI_ISL_15825627, EPI_ISL_15829108, EPI_ISL_15830052, EPI_ISL_15838528, EPI_ISL_15856822, EPI_ISL_15857468, EPI_ISL_15870952, EPI_ISL_15871047, EPI_ISL_15894376, EPI_ISL_15895664, EPI_ISL_15895805, EPI_ISL_15906500, EPI_ISL_15910110, EPI_ISL_15910797, EPI_ISL_15926723, EPI_ISL_15938365, EPI_ISL_15938543, EPI_ISL_15941958, EPI_ISL_15955458, EPI_ISL_15955461, EPI_ISL_15956078, EPI_ISL_15957375, EPI_ISL_15959194, EPI_ISL_15979054, EPI_ISL_15979220, EPI_ISL_15980454, EPI_ISL_15980534, EPI_ISL_15991477, EPI_ISL_16004112, EPI_ISL_16004139, EPI_ISL_16004173, EPI_ISL_16018930, EPI_ISL_16036592, EPI_ISL_16046816, EPI_ISL_16048672, EPI_ISL_16050568, EPI_ISL_16058939, EPI_ISL_16074147, EPI_ISL_16077721, EPI_ISL_16077819, EPI_ISL_16077827, EPI_ISL_16078136, EPI_ISL_16081297, EPI_ISL_16089903, EPI_ISL_16093705, EPI_ISL_16101357, EPI_ISL_16119461, EPI_ISL_16119652, EPI_ISL_16119687, EPI_ISL_16119697, EPI_ISL_16130620, EPI_ISL_16131605, EPI_ISL_16133537, EPI_ISL_16133850, EPI_ISL_16134022, EPI_ISL_16137724, EPI_ISL_16152967, EPI_ISL_16172206, EPI_ISL_16172628, EPI_ISL_16173251, EPI_ISL_16175780, EPI_ISL_16191277, EPI_ISL_16193751, EPI_ISL_16194226, EPI_ISL_16196595, EPI_ISL_16205170, EPI_ISL_16210278, EPI_ISL_16210314, EPI_ISL_16210314-16210315, EPI_ISL_16210315, EPI_ISL_16210321, EPI_ISL_16210344, EPI_ISL_16210352, EPI_ISL_16210356, EPI_ISL_16257294, EPI_ISL_16272927, EPI_ISL_16273053, EPI_ISL_16273094, EPI_ISL_16273106, EPI_ISL_16273691, EPI_ISL_16273904, EPI_ISL_16279440, EPI_ISL_16279922, EPI_ISL_16298573, EPI_ISL_16298574, EPI_ISL_16315643, EPI_ISL_16315645, EPI_ISL_16315710, EPI_ISL_16342216, EPI_ISL_16356910, EPI_ISL_16359576, EPI_ISL_16359648, EPI_ISL_16359676, EPI_ISL_16359695, EPI_ISL_16359707, EPI_ISL_16359708, EPI_ISL_16359709, EPI_ISL_16359721, EPI_ISL_16359724, EPI_ISL_16359730, EPI_ISL_16359737, EPI_ISL_16359739, EPI_ISL_16360560, EPI_ISL_16360578, EPI_ISL_16360615, EPI_ISL_16360631, EPI_ISL_16364267, EPI_ISL_16366337, EPI_ISL_16374061, EPI_ISL_16374062, EPI_ISL_16374063, EPI_ISL_16374064, EPI_ISL_16374077, EPI_ISL_16374103, EPI_ISL_16378479, EPI_ISL_16380729, EPI_ISL_16386695, EPI_ISL_16388543, EPI_ISL_16391752, EPI_ISL_16394844, EPI_ISL_16394849, EPI_ISL_16433986, EPI_ISL_16434219, EPI_ISL_16434682, EPI_ISL_16435411, EPI_ISL_16435758, EPI_ISL_16435776, EPI_ISL_16435834, EPI_ISL_16435848, EPI_ISL_16436045, EPI_ISL_16455847, EPI_ISL_16466074, EPI_ISL_16474400, EPI_ISL_16474501, EPI_ISL_16479627, EPI_ISL_16480279, EPI_ISL_16482060, EPI_ISL_16491755, EPI_ISL_16491962, EPI_ISL_16493846, EPI_ISL_16493881, EPI_ISL_16494105, EPI_ISL_16495643, EPI_ISL_16507760, EPI_ISL_16514725, EPI_ISL_16514726, EPI_ISL_16514727, EPI_ISL_16514728, EPI_ISL_16514745, EPI_ISL_16521266, EPI_ISL_16521681, EPI_ISL_16522889, EPI_ISL_16523533, EPI_ISL_16523565, EPI_ISL_16523595, EPI_ISL_16527181, EPI_ISL_16527529, EPI_ISL_16535383, EPI_ISL_16545014, EPI_ISL_16546118, EPI_ISL_16548794, EPI_ISL_16554670, EPI_ISL_16559981, EPI_ISL_16568182, EPI_ISL_16568197, EPI_ISL_16570236, EPI_ISL_16570568, EPI_ISL_16570602, EPI_ISL_16574300, EPI_ISL_16581363, EPI_ISL_16581388, EPI_ISL_16581578, EPI_ISL_16585008, EPI_ISL_16585433, EPI_ISL_16600965, EPI_ISL_16611373, EPI_ISL_16611412, EPI_ISL_16611422, EPI_ISL_16611506, EPI_ISL_16611514, EPI_ISL_16611550, EPI_ISL_16611563, EPI_ISL_16611624, EPI_ISL_16611667, EPI_ISL_16611673, EPI_ISL_16611682, EPI_ISL_16611698, EPI_ISL_16611778, EPI_ISL_16611838, EPI_ISL_16611848, EPI_ISL_16611855, EPI_ISL_16612037, EPI_ISL_16616330, EPI_ISL_16617719, EPI_ISL_16617775, EPI_ISL_16633089, EPI_ISL_16633129, EPI_ISL_16639468, EPI_ISL_16639472, EPI_ISL_16641545, EPI_ISL_16642506, EPI_ISL_16645057, EPI_ISL_16645120, EPI_ISL_16648131, EPI_ISL_16648231, EPI_ISL_16662824, EPI_ISL_16664013, EPI_ISL_16670808, EPI_ISL_16675393, EPI_ISL_16678683, EPI_ISL_16691130, EPI_ISL_16691138, EPI_ISL_16691144, EPI_ISL_16691244, EPI_ISL_16702617, EPI_ISL_16702731, EPI_ISL_16707041, EPI_ISL_16709805, EPI_ISL_16711169, EPI_ISL_16716050, EPI_ISL_16716112, EPI_ISL_16716666, EPI_ISL_16716667, EPI_ISL_16717190, EPI_ISL_16723860, EPI_ISL_16729430, EPI_ISL_16742981, EPI_ISL_16746019, EPI_ISL_16783884, EPI_ISL_16806139, EPI_ISL_16824117, EPI_ISL_16825886, EPI_ISL_16825887, EPI_ISL_16825902, EPI_ISL_16825911, EPI_ISL_16825973, EPI_ISL_16825975, EPI_ISL_16826011, EPI_ISL_16826162, EPI_ISL_16827461, EPI_ISL_16829544, EPI_ISL_16830824, EPI_ISL_16831676, EPI_ISL_16832813, EPI_ISL_16836551, EPI_ISL_16840631, EPI_ISL_16846006, EPI_ISL_16850006, EPI_ISL_16850282, EPI_ISL_16856565, EPI_ISL_16886081, EPI_ISL_16887730, EPI_ISL_16887870, EPI_ISL_16888824, EPI_ISL_16889192, EPI_ISL_16900219, EPI_ISL_16901108, EPI_ISL_16902309, EPI_ISL_16902361, EPI_ISL_16902363, EPI_ISL_16902379, EPI_ISL_16904536, EPI_ISL_16909958, EPI_ISL_16910033, EPI_ISL_16910057, EPI_ISL_16910058, EPI_ISL_16910139, EPI_ISL_16910202, EPI_ISL_16910382, EPI_ISL_16910409, EPI_ISL_16912192, EPI_ISL_16912224, EPI_ISL_16912699, EPI_ISL_16925018, EPI_ISL_16931898, EPI_ISL_16939136, EPI_ISL_16939502, EPI_ISL_16939509, EPI_ISL_16939520, EPI_ISL_16947370, EPI_ISL_16949310, EPI_ISL_16949324, EPI_ISL_16949325, EPI_ISL_16949387, EPI_ISL_16949791, EPI_ISL_16950012, EPI_ISL_16950629, EPI_ISL_16953777, EPI_ISL_16953781, EPI_ISL_16953798, EPI_ISL_16953799, EPI_ISL_16953818, EPI_ISL_16953822, EPI_ISL_16953828, EPI_ISL_16953830, EPI_ISL_16953838, EPI_ISL_16953839, EPI_ISL_16953840, EPI_ISL_16953847, EPI_ISL_16953922, EPI_ISL_16954432, EPI_ISL_16957344, EPI_ISL_16957451, EPI_ISL_16964941, EPI_ISL_16969564, EPI_ISL_16970275, EPI_ISL_16971354, EPI_ISL_16973054, EPI_ISL_16973491, EPI_ISL_16973514, EPI_ISL_16974198, EPI_ISL_16974315, EPI_ISL_16974382, EPI_ISL_16974837, EPI_ISL_16976164, EPI_ISL_16977847, EPI_ISL_16983895, EPI_ISL_16993160, EPI_ISL_16994359, EPI_ISL_16994360, EPI_ISL_16994528, EPI_ISL_16994534, EPI_ISL_16994641, EPI_ISL_16994642, EPI_ISL_16994643, EPI_ISL_16994655, EPI_ISL_16997635, EPI_ISL_17000779, EPI_ISL_17001818, EPI_ISL_17013912, EPI_ISL_17013920, EPI_ISL_17014551, EPI_ISL_17014690, EPI_ISL_17015386, EPI_ISL_17015403, EPI_ISL_17015508, EPI_ISL_17024862, EPI_ISL_17027606, EPI_ISL_17027639, EPI_ISL_17027948, EPI_ISL_17027949, EPI_ISL_17028105, EPI_ISL_17028190, EPI_ISL_17029568, EPI_ISL_17029620, EPI_ISL_17040153, EPI_ISL_17040170, EPI_ISL_17040185, EPI_ISL_17042013, EPI_ISL_17046545, EPI_ISL_17046657, EPI_ISL_17046658, EPI_ISL_17046849, EPI_ISL_17047075, EPI_ISL_17047300, EPI_ISL_17053152, EPI_ISL_17053374, EPI_ISL_17058072, EPI_ISL_17058073, EPI_ISL_17062470, EPI_ISL_17062513, EPI_ISL_17068606, EPI_ISL_17076266, EPI_ISL_17079912, EPI_ISL_17089619, EPI_ISL_17090505, EPI_ISL_17090748, EPI_ISL_17091144, EPI_ISL_17092036, EPI_ISL_17092037, EPI_ISL_17092038, EPI_ISL_17120889, EPI_ISL_17124976, EPI_ISL_17131649, EPI_ISL_17144768, EPI_ISL_17146391, EPI_ISL_17150326, EPI_ISL_17150374, EPI_ISL_17150380, EPI_ISL_17154893, EPI_ISL_17156691, EPI_ISL_17162825, EPI_ISL_17162929, EPI_ISL_17163260, EPI_ISL_17164036, EPI_ISL_17164425, EPI_ISL_17168296, EPI_ISL_17171052, EPI_ISL_17171054, EPI_ISL_17171074, EPI_ISL_17173504, EPI_ISL_17180537, EPI_ISL_17185359, EPI_ISL_17185391, EPI_ISL_17185393, EPI_ISL_17185521, EPI_ISL_17187061, EPI_ISL_17187082, EPI_ISL_17187115, EPI_ISL_17187405, EPI_ISL_17190415, EPI_ISL_17193320, EPI_ISL_17193409, EPI_ISL_17197288, EPI_ISL_17198749, EPI_ISL_17198796, EPI_ISL_17198959, EPI_ISL_17198971, EPI_ISL_17205507, EPI_ISL_17212042, EPI_ISL_17212254, EPI_ISL_17212450, EPI_ISL_17213648, EPI_ISL_17213677, EPI_ISL_17233016, EPI_ISL_17237758, EPI_ISL_17239970, EPI_ISL_17241376, EPI_ISL_17243067, EPI_ISL_17244428, EPI_ISL_17244430, EPI_ISL_17245777, EPI_ISL_17249116, EPI_ISL_17259971, EPI_ISL_17273231, EPI_ISL_17273546, EPI_ISL_17279072, EPI_ISL_17283738, EPI_ISL_17288373, EPI_ISL_17292738, EPI_ISL_17292778, EPI_ISL_17292812, EPI_ISL_17295274, EPI_ISL_17299688, EPI_ISL_17305358, EPI_ISL_17322295, EPI_ISL_17326431, EPI_ISL_17329306, EPI_ISL_17342779, EPI_ISL_17350941, EPI_ISL_17372713, EPI_ISL_17373645, EPI_ISL_17373894, EPI_ISL_17374733, EPI_ISL_17374799, EPI_ISL_17374806, EPI_ISL_17374807, EPI_ISL_17375007, EPI_ISL_17382768, EPI_ISL_17387866, EPI_ISL_17388895, EPI_ISL_17388906, EPI_ISL_17393458, EPI_ISL_17404084, EPI_ISL_17409157, EPI_ISL_17413069, EPI_ISL_17413882, EPI_ISL_17419568, EPI_ISL_17422869, EPI_ISL_17430511, EPI_ISL_17433569, EPI_ISL_17463210, EPI_ISL_17464836, EPI_ISL_17465459, EPI_ISL_17468440, EPI_ISL_17477106, EPI_ISL_17485616, EPI_ISL_17485727, EPI_ISL_17485728, EPI_ISL_17493590, EPI_ISL_17501569, EPI_ISL_17503268, EPI_ISL_17504112, EPI_ISL_17511739, EPI_ISL_17516925, EPI_ISL_17517664, EPI_ISL_17521432, EPI_ISL_17527442, EPI_ISL_17527471, EPI_ISL_17527478, EPI_ISL_17527567, EPI_ISL_17539348, EPI_ISL_17544585, EPI_ISL_17544916, EPI_ISL_17547750, EPI_ISL_17551316, EPI_ISL_17556166, EPI_ISL_17558242, EPI_ISL_17558487, EPI_ISL_17558870, EPI_ISL_17559934, EPI_ISL_17592407, EPI_ISL_17604727, EPI_ISL_17604844, EPI_ISL_17605096, EPI_ISL_17621484, EPI_ISL_17622137, EPI_ISL_17628855, EPI_ISL_17642930, EPI_ISL_17647324, EPI_ISL_17649109, EPI_ISL_17656014, EPI_ISL_17656052, EPI_ISL_17657007, EPI_ISL_17657210, EPI_ISL_17660818, EPI_ISL_17661045, EPI_ISL_17661237, EPI_ISL_17661261, EPI_ISL_17661270, EPI_ISL_17662899, EPI_ISL_17666331, EPI_ISL_17682891, EPI_ISL_17682942, EPI_ISL_17683117, EPI_ISL_17683214, EPI_ISL_17683243, EPI_ISL_17685027, EPI_ISL_17692258, EPI_ISL_17695126, EPI_ISL_17695180, EPI_ISL_17696632, EPI_ISL_17696673, EPI_ISL_17696719, EPI_ISL_17698434, EPI_ISL_17703866, EPI_ISL_17714932, EPI_ISL_17716296, EPI_ISL_17718464, EPI_ISL_17719091, EPI_ISL_17720695, EPI_ISL_17722014, EPI_ISL_17724271, EPI_ISL_17724304, EPI_ISL_17737041, EPI_ISL_17740508, EPI_ISL_17742125, EPI_ISL_17762760, EPI_ISL_17762820, EPI_ISL_17764084, EPI_ISL_17765632, EPI_ISL_17765706, EPI_ISL_17765717, EPI_ISL_17775797, EPI_ISL_17775798, EPI_ISL_17776606, EPI_ISL_17777067, EPI_ISL_17784540, EPI_ISL_17784548, EPI_ISL_17784550, EPI_ISL_17784552, EPI_ISL_17784569, EPI_ISL_17784593, EPI_ISL_17790241, EPI_ISL_17791006, EPI_ISL_17793597, EPI_ISL_17793709, EPI_ISL_17796704, EPI_ISL_17802162, EPI_ISL_17802448, EPI_ISL_17802471, EPI_ISL_17802863, EPI_ISL_17803152, EPI_ISL_17806624, EPI_ISL_17806780, EPI_ISL_17806785, EPI_ISL_17807899, EPI_ISL_17821941, EPI_ISL_17822275, EPI_ISL_17822422, EPI_ISL_17822859, EPI_ISL_17822940, EPI_ISL_17823538, EPI_ISL_17824267, EPI_ISL_17824292, EPI_ISL_17824608, EPI_ISL_17824670, EPI_ISL_17826031, EPI_ISL_17826225, EPI_ISL_17826355, EPI_ISL_17826392, EPI_ISL_17827176, EPI_ISL_17827221, EPI_ISL_17827261, EPI_ISL_17829319, EPI_ISL_17829398, EPI_ISL_17829993, EPI_ISL_17832368, EPI_ISL_17849659, EPI_ISL_17851798, EPI_ISL_17857957, EPI_ISL_17859873, EPI_ISL_17859887, EPI_ISL_17879222, EPI_ISL_17882742, EPI_ISL_17960272, EPI_ISL_17975460, EPI_ISL_17977889, EPI_ISL_17977890, EPI_ISL_17977939, EPI_ISL_17980375, EPI_ISL_17980376, EPI_ISL_17980377, EPI_ISL_17985070, EPI_ISL_17987531, EPI_ISL_17994215, EPI_ISL_17997273, EPI_ISL_17999525, EPI_ISL_17999535, EPI_ISL_17999631, EPI_ISL_17999795, EPI_ISL_17999836, EPI_ISL_18000297, EPI_ISL_18001169, EPI_ISL_18010323, EPI_ISL_18014681, EPI_ISL_18031645, EPI_ISL_18032375, EPI_ISL_18039667, EPI_ISL_18040500, EPI_ISL_18041690, EPI_ISL_18042063, EPI_ISL_18043809, EPI_ISL_18051511, EPI_ISL_18052776, EPI_ISL_18057695, EPI_ISL_18060832, EPI_ISL_18063266, EPI_ISL_18063275, EPI_ISL_18063343, EPI_ISL_18063346, EPI_ISL_18063361, EPI_ISL_18063384, EPI_ISL_18063710, EPI_ISL_18063741, EPI_ISL_18063779, EPI_ISL_18063898, EPI_ISL_18064050, EPI_ISL_18064055, EPI_ISL_18064103, EPI_ISL_18064142, EPI_ISL_18064155, EPI_ISL_18064235, EPI_ISL_18064253, EPI_ISL_18064299, EPI_ISL_18064307, EPI_ISL_18064328, EPI_ISL_18064337, EPI_ISL_18064383, EPI_ISL_18065323, EPI_ISL_18066648, EPI_ISL_18066669, EPI_ISL_18066701, EPI_ISL_18076272, EPI_ISL_18077730, EPI_ISL_18082776, EPI_ISL_18093842, EPI_ISL_18100370, EPI_ISL_18100403, EPI_ISL_18100404, EPI_ISL_18100416, EPI_ISL_18100417, EPI_ISL_18100418, EPI_ISL_18106912, EPI_ISL_18108242, EPI_ISL_18112575, EPI_ISL_18112668, EPI_ISL_18115399, EPI_ISL_18117671, EPI_ISL_18127619, EPI_ISL_18127909, EPI_ISL_18127961, EPI_ISL_18134274, EPI_ISL_18134756, EPI_ISL_18138526, EPI_ISL_18139339, EPI_ISL_18139659, EPI_ISL_18166414, EPI_ISL_18208967, EPI_ISL_18213578, EPI_ISL_18215762, EPI_ISL_18215772, EPI_ISL_18216065, EPI_ISL_18218555, EPI_ISL_18218775, EPI_ISL_18218935, EPI_ISL_18218936, EPI_ISL_18225292, EPI_ISL_18226914, EPI_ISL_18237064, EPI_ISL_18247242, EPI_ISL_18263981, EPI_ISL_18265252, EPI_ISL_18272809, EPI_ISL_18280058, EPI_ISL_18285922, EPI_ISL_18285953, EPI_ISL_18287351, EPI_ISL_18300222, EPI_ISL_18300587, EPI_ISL_18306489, EPI_ISL_18316786, EPI_ISL_18320556, EPI_ISL_18343894, EPI_ISL_18347703, EPI_ISL_18350437, EPI_ISL_18352397, EPI_ISL_18354145, EPI_ISL_18355510, EPI_ISL_18365833, EPI_ISL_18367086, EPI_ISL_18375185, EPI_ISL_18375538, EPI_ISL_18376234, EPI_ISL_18382930, EPI_ISL_18382993, EPI_ISL_18383001, EPI_ISL_18390814, EPI_ISL_18423785, EPI_ISL_18432517, EPI_ISL_18432566, EPI_ISL_18434659, EPI_ISL_18435860, EPI_ISL_18440660, EPI_ISL_18449837, EPI_ISL_18461692, EPI_ISL_18466251, EPI_ISL_18471046, EPI_ISL_18475421, EPI_ISL_18475423, EPI_ISL_18475424, EPI_ISL_18475534, EPI_ISL_18498513, EPI_ISL_18502004, EPI_ISL_18505899, EPI_ISL_18509397, EPI_ISL_18509398, EPI_ISL_18509399, EPI_ISL_18512167, EPI_ISL_18512168, EPI_ISL_18512169, EPI_ISL_18520761, EPI_ISL_18520956, EPI_ISL_18551236, EPI_ISL_18551245, EPI_ISL_18566482, EPI_ISL_18566696, EPI_ISL_18575585, EPI_ISL_18623910, EPI_ISL_18624229, EPI_ISL_18624532, EPI_ISL_18633916, EPI_ISL_18639764, EPI_ISL_18650558, EPI_ISL_18650648, EPI_ISL_18651037, EPI_ISL_18662705, EPI_ISL_18667571, EPI_ISL_18667607, EPI_ISL_18667615, EPI_ISL_18680062, EPI_ISL_18687076, EPI_ISL_18687807, EPI_ISL_18687813, EPI_ISL_18687815, EPI_ISL_18690025, EPI_ISL_18693344, EPI_ISL_18697820, EPI_ISL_18701826, EPI_ISL_18705856, EPI_ISL_18706440, EPI_ISL_18706492, EPI_ISL_18706515, EPI_ISL_18706569, EPI_ISL_18706570, EPI_ISL_18706571, EPI_ISL_18706903, EPI_ISL_18706995, EPI_ISL_18707254, EPI_ISL_18708577, EPI_ISL_18709081, EPI_ISL_18709099, EPI_ISL_18709184, EPI_ISL_18709429, EPI_ISL_18709961, EPI_ISL_18710009, EPI_ISL_18710348, EPI_ISL_18710474, EPI_ISL_18711358, EPI_ISL_18711689, EPI_ISL_18712220, EPI_ISL_18712343, EPI_ISL_18712994, EPI_ISL_18713143, EPI_ISL_18713223, EPI_ISL_18713224, EPI_ISL_18713225, EPI_ISL_18713497, EPI_ISL_18713949, EPI_ISL_18714155, EPI_ISL_18714183, EPI_ISL_18714275, EPI_ISL_18714276, EPI_ISL_18714439, EPI_ISL_18714442, EPI_ISL_18714536, EPI_ISL_18714543, EPI_ISL_18714907, EPI_ISL_18715065, EPI_ISL_18715624, EPI_ISL_18715664, EPI_ISL_18715665, EPI_ISL_18715677, EPI_ISL_18715701, EPI_ISL_18715897, EPI_ISL_18716498, EPI_ISL_18716943, EPI_ISL_18717071, EPI_ISL_18717321, EPI_ISL_18717565, EPI_ISL_18717861, EPI_ISL_18718348, EPI_ISL_18718713, EPI_ISL_18719340, EPI_ISL_18719410, EPI_ISL_18719472, EPI_ISL_18719480, EPI_ISL_18719851, EPI_ISL_18720533, EPI_ISL_18720935, EPI_ISL_18721314, EPI_ISL_18721321, EPI_ISL_18722226, EPI_ISL_18722597, EPI_ISL_18723310, EPI_ISL_18723528, EPI_ISL_18723713, EPI_ISL_18723731, EPI_ISL_18723732, EPI_ISL_18724340, EPI_ISL_18724573, EPI_ISL_18724658, EPI_ISL_18724796, EPI_ISL_18724900, EPI_ISL_18724922, EPI_ISL_18725354, EPI_ISL_18725460, EPI_ISL_18725508, EPI_ISL_18725514, EPI_ISL_18725521, EPI_ISL_18725530, EPI_ISL_18725796, EPI_ISL_18725907, EPI_ISL_18726354, EPI_ISL_18726942, EPI_ISL_18727294, EPI_ISL_18728195, EPI_ISL_18728633, EPI_ISL_18728656, EPI_ISL_18728730, EPI_ISL_18728782, EPI_ISL_18728864, EPI_ISL_18728920, EPI_ISL_18729096, EPI_ISL_18729113, EPI_ISL_18729988, EPI_ISL_18730108, EPI_ISL_18730464, EPI_ISL_18730496, EPI_ISL_18730535, EPI_ISL_18730710, EPI_ISL_18731017, EPI_ISL_18731338, EPI_ISL_18731357, EPI_ISL_18731441, EPI_ISL_18733077, EPI_ISL_18733249, EPI_ISL_18733263, EPI_ISL_18733280, EPI_ISL_18734070, EPI_ISL_18734413, EPI_ISL_18734528, EPI_ISL_18734867, EPI_ISL_18735083, EPI_ISL_18735276, EPI_ISL_18736105, EPI_ISL_18736157, EPI_ISL_18736221, EPI_ISL_18736669, EPI_ISL_18736840, EPI_ISL_18736871, EPI_ISL_18736888, EPI_ISL_18736949, EPI_ISL_18736972, EPI_ISL_18736973, EPI_ISL_18736984, EPI_ISL_18737037, EPI_ISL_18737405, EPI_ISL_18738178, EPI_ISL_18738316, EPI_ISL_18738757, EPI_ISL_18738982, EPI_ISL_18739424, EPI_ISL_18739567, EPI_ISL_18740428, EPI_ISL_18740649, EPI_ISL_18740758, EPI_ISL_18740840, EPI_ISL_18740939, EPI_ISL_18741548, EPI_ISL_18741657, EPI_ISL_18741663, EPI_ISL_18741696, EPI_ISL_18742182, EPI_ISL_18742836, EPI_ISL_18743070, EPI_ISL_18743109, EPI_ISL_18744493, EPI_ISL_18744991, EPI_ISL_18745138, EPI_ISL_18745328, EPI_ISL_18745405, EPI_ISL_18745708, EPI_ISL_18746655, EPI_ISL_18747194, EPI_ISL_18747288, EPI_ISL_18747569, EPI_ISL_18747605, EPI_ISL_18747620, EPI_ISL_18747634, EPI_ISL_18747739, EPI_ISL_18748060, EPI_ISL_18748066, EPI_ISL_18748200, EPI_ISL_18748214, EPI_ISL_18749061, EPI_ISL_18749160, EPI_ISL_18749619, EPI_ISL_18749629, EPI_ISL_18749846, EPI_ISL_18749878, EPI_ISL_18749911, EPI_ISL_18750236, EPI_ISL_18750265, EPI_ISL_18750464, EPI_ISL_18750796, EPI_ISL_18751169, EPI_ISL_18751192, EPI_ISL_18751363, EPI_ISL_18751654, EPI_ISL_18751663, EPI_ISL_18751719, EPI_ISL_18751845, EPI_ISL_18751862, EPI_ISL_18752479, EPI_ISL_18753192, EPI_ISL_18753336, EPI_ISL_18754529, EPI_ISL_18754585, EPI_ISL_18754587, EPI_ISL_18755048, EPI_ISL_18755102, EPI_ISL_18755433, EPI_ISL_18756002, EPI_ISL_18756291, EPI_ISL_18756311, EPI_ISL_18756366, EPI_ISL_18756413, EPI_ISL_18756587, EPI_ISL_18756772, EPI_ISL_18756860, EPI_ISL_18757884, EPI_ISL_18757906, EPI_ISL_18757921, EPI_ISL_18758080, EPI_ISL_18758111, EPI_ISL_18758235, EPI_ISL_18758518, EPI_ISL_18758538, EPI_ISL_18760281, EPI_ISL_18760288, EPI_ISL_18760322, EPI_ISL_18792280, EPI_ISL_18792992, EPI_ISL_18793204, EPI_ISL_18798820, EPI_ISL_18799141, EPI_ISL_18843955, EPI_ISL_18857091, EPI_ISL_18877237, EPI_ISL_18878774, EPI_ISL_18886497, EPI_ISL_18895660, EPI_ISL_18904633, EPI_ISL_18904635, EPI_ISL_18922726, EPI_ISL_18924174, EPI_ISL_18931648, EPI_ISL_18931705, EPI_ISL_18931859, EPI_ISL_18932507, EPI_ISL_18932551, EPI_ISL_18960210, EPI_ISL_18967490, EPI_ISL_18992540, EPI_ISL_18994815, EPI_ISL_19005505, EPI_ISL_19009620, EPI_ISL_19009621, EPI_ISL_19009623, EPI_ISL_19009624, EPI_ISL_19015609, EPI_ISL_19021722, EPI_ISL_19024035, EPI_ISL_19024044, EPI_ISL_19024068, EPI_ISL_19024081, EPI_ISL_19037171, EPI_ISL_19050132, EPI_ISL_19054784, EPI_ISL_19057886, EPI_ISL_19059047, EPI_ISL_19071524, EPI_ISL_19080112, EPI_ISL_19082638, EPI_ISL_19084871, EPI_ISL_19084883, EPI_ISL_19085033, EPI_ISL_19085070, EPI_ISL_19085078, EPI_ISL_19085135, EPI_ISL_19096287, EPI_ISL_19108340, EPI_ISL_19122484, EPI_ISL_19135284, EPI_ISL_19140391, EPI_ISL_19150867, EPI_ISL_19150870, EPI_ISL_19168138, EPI_ISL_19169477, EPI_ISL_19169870, EPI_ISL_19169976, EPI_ISL_19175684, EPI_ISL_19180359, EPI_ISL_19180367, EPI_ISL_19197718, EPI_ISL_19226538, EPI_ISL_19239735, EPI_ISL_19269562, EPI_ISL_19269713, EPI_ISL_19281165, EPI_ISL_19289747, EPI_ISL_19292254, EPI_ISL_19298398, EPI_ISL_19300597, EPI_ISL_19305340, EPI_ISL_19308874, EPI_ISL_19308875, EPI_ISL_19308876, EPI_ISL_19308877, EPI_ISL_19323028, EPI_ISL_19328684, EPI_ISL_19333890, EPI_ISL_19337826, EPI_ISL_19340412, EPI_ISL_19340513, EPI_ISL_19340884, EPI_ISL_19363557, EPI_ISL_19369244, EPI_ISL_19374121, EPI_ISL_19374151, EPI_ISL_19374152, EPI_ISL_19374153, EPI_ISL_19374154, EPI_ISL_19377637, EPI_ISL_19377761, EPI_ISL_19377778, EPI_ISL_19383580, EPI_ISL_19383680, EPI_ISL_19392805, EPI_ISL_19405488, EPI_ISL_19427278, EPI_ISL_19434372, EPI_ISL_19444130, EPI_ISL_19472214, EPI_ISL_19473889, EPI_ISL_19506224, EPI_ISL_19515856, EPI_ISL_19517266, EPI_ISL_19522346, EPI_ISL_19526058, EPI_ISL_19526073, EPI_ISL_19526074, EPI_ISL_19527099, EPI_ISL_19531769, EPI_ISL_19531772, EPI_ISL_19531776, EPI_ISL_19534803, EPI_ISL_19555103, EPI_ISL_19572558, EPI_ISL_19585472, EPI_ISL_19588464, EPI_ISL_19591990, EPI_ISL_19620039, EPI_ISL_19750637, EPI_ISL_19778907, EPI_ISL_19788441")
    AT1 = stringlist_to_strings_set("EPI_ISL_2385089, EPI_ISL_18014433, EPI_ISL_18014355, EPI_ISL_3454837, EPI_ISL_2245684, EPI_ISL_2385137, EPI_ISL_18016392, EPI_ISL_18027763, EPI_ISL_1652622, EPI_ISL_18027770, EPI_ISL_2245728, EPI_ISL_2258756, EPI_ISL_2450459, EPI_ISL_2245675, EPI_ISL_2698607, EPI_ISL_3026520, EPI_ISL_2385127, EPI_ISL_11246080, EPI_ISL_1652580, EPI_ISL_2245775, EPI_ISL_2385430, EPI_ISL_2385240, EPI_ISL_18014354, EPI_ISL_2258757, EPI_ISL_2385264, EPI_ISL_2245737, EPI_ISL_2385132, EPI_ISL_2038887, EPI_ISL_2245644, EPI_ISL_11267839, EPI_ISL_1491610, EPI_ISL_1919575, EPI_ISL_1652607, EPI_ISL_3101247, EPI_ISL_2816166, EPI_ISL_3101347, EPI_ISL_3101296, EPI_ISL_2385268, EPI_ISL_1491611, EPI_ISL_2245680, EPI_ISL_1652608, EPI_ISL_18014306, EPI_ISL_3076755, EPI_ISL_2694579, EPI_ISL_3040116, EPI_ISL_2385306, EPI_ISL_2385222, EPI_ISL_3021542, EPI_ISL_1652639, EPI_ISL_2385317, EPI_ISL_2609255, EPI_ISL_2609191, EPI_ISL_2609371, EPI_ISL_18018216, EPI_ISL_2258962, EPI_ISL_2385285, EPI_ISL_18015138, EPI_ISL_2385294, EPI_ISL_11267844, EPI_ISL_2523392, EPI_ISL_2385305, EPI_ISL_12389148, EPI_ISL_11267840, EPI_ISL_2385426, EPI_ISL_2385141, EPI_ISL_1652545, EPI_ISL_3101302, EPI_ISL_2363834, EPI_ISL_2616208, EPI_ISL_2385265, EPI_ISL_2642249, EPI_ISL_18014342, EPI_ISL_2363825, EPI_ISL_2523425, EPI_ISL_18018316, EPI_ISL_3040109, EPI_ISL_2385427, EPI_ISL_2245760, EPI_ISL_1548041, EPI_ISL_4093675, EPI_ISL_2609203, EPI_ISL_2385115, EPI_ISL_2385223, EPI_ISL_2385086, EPI_ISL_2967111, EPI_ISL_18014405, EPI_ISL_2757968, EPI_ISL_18015053, EPI_ISL_11267843, EPI_ISL_18017208, EPI_ISL_11225838, EPI_ISL_2657493, EPI_ISL_2385327, EPI_ISL_1652585, EPI_ISL_2032642, EPI_ISL_2523721, EPI_ISL_2245670, EPI_ISL_18029419, EPI_ISL_1823179, EPI_ISL_7850437, EPI_ISL_2385286, EPI_ISL_3076760, EPI_ISL_2485133, EPI_ISL_18018233, EPI_ISL_18027824, EPI_ISL_2385376, EPI_ISL_2038912, EPI_ISL_2385269, EPI_ISL_3040113, EPI_ISL_2385325, EPI_ISL_2887541, EPI_ISL_3454841, EPI_ISL_2245716, EPI_ISL_2523667, EPI_ISL_2385234, EPI_ISL_2689418, EPI_ISL_2663108, EPI_ISL_2609134, EPI_ISL_2967104, EPI_ISL_5049878, EPI_ISL_2245717, EPI_ISL_2644858, EPI_ISL_2523696, EPI_ISL_2385238, EPI_ISL_18014312, EPI_ISL_2887421, EPI_ISL_2245679, EPI_ISL_2385171, EPI_ISL_13287205, EPI_ISL_3798354, EPI_ISL_3101286, EPI_ISL_2523759, EPI_ISL_3101315, EPI_ISL_1919459, EPI_ISL_2385201, EPI_ISL_8112527, EPI_ISL_1259282, EPI_ISL_2245761, EPI_ISL_2385198, EPI_ISL_2385307, EPI_ISL_1263541, EPI_ISL_3135785, EPI_ISL_5415169, EPI_ISL_18014259, EPI_ISL_2038886, EPI_ISL_2263172, EPI_ISL_2385216, EPI_ISL_2245665, EPI_ISL_2245649, EPI_ISL_3040110, EPI_ISL_2245663, EPI_ISL_7850438, EPI_ISL_11246081, EPI_ISL_1259283, EPI_ISL_11267841, EPI_ISL_2608656, EPI_ISL_2694559, EPI_ISL_2385421, EPI_ISL_18014137, EPI_ISL_2450562, EPI_ISL_2038871, EPI_ISL_2609088, EPI_ISL_2385082, EPI_ISL_11267842, EPI_ISL_3076749, EPI_ISL_6686962, EPI_ISL_2608657, EPI_ISL_1652635, EPI_ISL_12389150, EPI_ISL_2385109, EPI_ISL_3101381, EPI_ISL_2385229, EPI_ISL_1664699, EPI_ISL_2245763, EPI_ISL_2385110, EPI_ISL_11267838, EPI_ISL_18015144, EPI_ISL_2450534, EPI_ISL_1664729, EPI_ISL_2450556, EPI_ISL_2523448, EPI_ISL_2626347, EPI_ISL_2245677, EPI_ISL_1548040, EPI_ISL_2363838, EPI_ISL_2523400, EPI_ISL_2698617, EPI_ISL_3101291, EPI_ISL_2609233, EPI_ISL_1823178, EPI_ISL_2887542, EPI_ISL_18014235, EPI_ISL_18014407, EPI_ISL_2457502, EPI_ISL_2816398, EPI_ISL_2385263, EPI_ISL_2608651, EPI_ISL_2385431, EPI_ISL_2523611, EPI_ISL_2450517, EPI_ISL_1652557")
    AV1 = stringlist_to_strings_set("EPI_ISL_4566929, EPI_ISL_2125482, EPI_ISL_2239779, EPI_ISL_2004704, EPI_ISL_2240090, EPI_ISL_2240151, EPI_ISL_2122546, EPI_ISL_2198382, EPI_ISL_1987067, EPI_ISL_2021628, EPI_ISL_2624764, EPI_ISL_2434445, EPI_ISL_2127047, EPI_ISL_2236823, EPI_ISL_2520581, EPI_ISL_2236967, EPI_ISL_2705045, EPI_ISL_2120895, EPI_ISL_1857951, EPI_ISL_2237157, EPI_ISL_2239580, EPI_ISL_2171756, EPI_ISL_2434675, EPI_ISL_2237958, EPI_ISL_2239457, EPI_ISL_2518284, EPI_ISL_2240292, EPI_ISL_2570109, EPI_ISL_2457326, EPI_ISL_2706055, EPI_ISL_2625296, EPI_ISL_2518253, EPI_ISL_2518281, EPI_ISL_2517875, EPI_ISL_2021658, EPI_ISL_2625337, EPI_ISL_2625442, EPI_ISL_2318105, EPI_ISL_2518496, EPI_ISL_2721970, EPI_ISL_2236548, EPI_ISL_1857666, EPI_ISL_2253970, EPI_ISL_2092569, EPI_ISL_2437210, EPI_ISL_2514457, EPI_ISL_2517895, EPI_ISL_1857609, EPI_ISL_2434746, EPI_ISL_2355937, EPI_ISL_2127052, EPI_ISL_2240008, EPI_ISL_2434695, EPI_ISL_2240274, EPI_ISL_2240118, EPI_ISL_2518887, EPI_ISL_2004581, EPI_ISL_2239978, EPI_ISL_2518712, EPI_ISL_2717065, EPI_ISL_2138347, EPI_ISL_2126474, EPI_ISL_2517941, EPI_ISL_2347631, EPI_ISL_2356078, EPI_ISL_2004900, EPI_ISL_2356178, EPI_ISL_2171691, EPI_ISL_2239677, EPI_ISL_2707108, EPI_ISL_2356235, EPI_ISL_2240054, EPI_ISL_2240678, EPI_ISL_2517872, EPI_ISL_2518852, EPI_ISL_2625597, EPI_ISL_2005439, EPI_ISL_3291235, EPI_ISL_2434355, EPI_ISL_2238485, EPI_ISL_2239850, EPI_ISL_1912100, EPI_ISL_2517910, EPI_ISL_2355982, EPI_ISL_2092318, EPI_ISL_2664976, EPI_ISL_2518249, EPI_ISL_2122397, EPI_ISL_2394252, EPI_ISL_2434509, EPI_ISL_2742613, EPI_ISL_2240131, EPI_ISL_2355981, EPI_ISL_2434451, EPI_ISL_2625398, EPI_ISL_2394056, EPI_ISL_2355966, EPI_ISL_2120972, EPI_ISL_2353408, EPI_ISL_2199581, EPI_ISL_2240019, EPI_ISL_2743887, EPI_ISL_2623351, EPI_ISL_2487128, EPI_ISL_2120960, EPI_ISL_2138441, EPI_ISL_2240680, EPI_ISL_1980365, EPI_ISL_2128461, EPI_ISL_2240079, EPI_ISL_2119669, EPI_ISL_2518053, EPI_ISL_1698537, EPI_ISL_2434465, EPI_ISL_2236937, EPI_ISL_2239489, EPI_ISL_2348421, EPI_ISL_2518704, EPI_ISL_2355974, EPI_ISL_2236504, EPI_ISL_2356883, EPI_ISL_2434473, EPI_ISL_2518367, EPI_ISL_2004376, EPI_ISL_2518126, EPI_ISL_2518270, EPI_ISL_1857753, EPI_ISL_2199394, EPI_ISL_2239848, EPI_ISL_2240116, EPI_ISL_2366158, EPI_ISL_2022294, EPI_ISL_2120273, EPI_ISL_2723162, EPI_ISL_2138170, EPI_ISL_2356106, EPI_ISL_2239443, EPI_ISL_1980414, EPI_ISL_2240058, EPI_ISL_2240677, EPI_ISL_2121628, EPI_ISL_2356045, EPI_ISL_1980353, EPI_ISL_2392572, EPI_ISL_2706009, EPI_ISL_2437243, EPI_ISL_2022182, EPI_ISL_2138472, EPI_ISL_2199542, EPI_ISL_1921237, EPI_ISL_2239816")
    B1_243_2 = stringlist_to_strings_set("EPI_ISL_5655176, EPI_ISL_5217681, EPI_ISL_5925120, EPI_ISL_6327614, EPI_ISL_4299659, EPI_ISL_4370973, EPI_ISL_4959931, EPI_ISL_4706700, EPI_ISL_4706704, EPI_ISL_4571920, EPI_ISL_4571923, EPI_ISL_4508892, EPI_ISL_4508893, EPI_ISL_4508918, EPI_ISL_4359868, EPI_ISL_2245655, EPI_ISL_2385087, EPI_ISL_2385112, EPI_ISL_2385117, EPI_ISL_2385134, EPI_ISL_2450564, EPI_ISL_3015043, EPI_ISL_17105781, EPI_ISL_17105784, EPI_ISL_17105799, EPI_ISL_17105800, EPI_ISL_17105801")
    B1_616 = stringlist_to_strings_set("EPI_ISL_1526787, EPI_ISL_1526780, EPI_ISL_2178439, EPI_ISL_2178448, EPI_ISL_2178432, EPI_ISL_1655920, EPI_ISL_1739315, EPI_ISL_1655921, EPI_ISL_1118893, EPI_ISL_2178447, EPI_ISL_1443880, EPI_ISL_1526779, EPI_ISL_1111064, EPI_ISL_1409708, EPI_ISL_1118892, EPI_ISL_1381829, EPI_ISL_1259307, EPI_ISL_1489729, EPI_ISL_1526783, EPI_ISL_1739314, EPI_ISL_1739317, EPI_ISL_1526778, EPI_ISL_1318289, EPI_ISL_1526786, EPI_ISL_1655918, EPI_ISL_1540445, EPI_ISL_2281746, EPI_ISL_1526793, EPI_ISL_1526794, EPI_ISL_2178418, EPI_ISL_1381830, EPI_ISL_1739313, EPI_ISL_2178417, EPI_ISL_1110211, EPI_ISL_1655919, EPI_ISL_1239370, EPI_ISL_1525263")
    B1_632 = stringlist_to_strings_set("EPI_ISL_2343008, EPI_ISL_2402068, EPI_ISL_2479894, EPI_ISL_2479898, EPI_ISL_2479906, EPI_ISL_2490409, EPI_ISL_2490411, EPI_ISL_2616958, EPI_ISL_2658951, EPI_ISL_2658952, EPI_ISL_2658953, EPI_ISL_2671630, EPI_ISL_2681236, EPI_ISL_2681271, EPI_ISL_2681324, EPI_ISL_2778011, EPI_ISL_2801662, EPI_ISL_2801665, EPI_ISL_2801886, EPI_ISL_2835074, EPI_ISL_2835075, EPI_ISL_2835076, EPI_ISL_2835085, EPI_ISL_2835086, EPI_ISL_2835087, EPI_ISL_2835088, EPI_ISL_2920739, EPI_ISL_2926853, EPI_ISL_2928207, EPI_ISL_2942474, EPI_ISL_2942481, EPI_ISL_2942483, EPI_ISL_2942635, EPI_ISL_2942683, EPI_ISL_2942695, EPI_ISL_2970046, EPI_ISL_2978610, EPI_ISL_2987720, EPI_ISL_3048080, EPI_ISL_3067876, EPI_ISL_3067941, EPI_ISL_3084636, EPI_ISL_3084750, EPI_ISL_3085041, EPI_ISL_3085047, EPI_ISL_3110921, EPI_ISL_3155372, EPI_ISL_3155430, EPI_ISL_3155433, EPI_ISL_3160927, EPI_ISL_3219930, EPI_ISL_3239305, EPI_ISL_3239820, EPI_ISL_3265665, EPI_ISL_3265672, EPI_ISL_3265675, EPI_ISL_3277764, EPI_ISL_3277772, EPI_ISL_3327183, EPI_ISL_3328423, EPI_ISL_3330563, EPI_ISL_3330564, EPI_ISL_3347599, EPI_ISL_3347600, EPI_ISL_3347601, EPI_ISL_3347602, EPI_ISL_3347603, EPI_ISL_3347604, EPI_ISL_3347611, EPI_ISL_3385228, EPI_ISL_3388731, EPI_ISL_3393484, EPI_ISL_3408494, EPI_ISL_3408975, EPI_ISL_3428767, EPI_ISL_3431392, EPI_ISL_3491927, EPI_ISL_3492216, EPI_ISL_3494886, EPI_ISL_3495427, EPI_ISL_3496863, EPI_ISL_3501228, EPI_ISL_3503159, EPI_ISL_3503427, EPI_ISL_3503636, EPI_ISL_3520843, EPI_ISL_3556922, EPI_ISL_3556959, EPI_ISL_3556963, EPI_ISL_3610010, EPI_ISL_3610012, EPI_ISL_3713850, EPI_ISL_3713864, EPI_ISL_3715035, EPI_ISL_3721729, EPI_ISL_3721730, EPI_ISL_3721785, EPI_ISL_3721792, EPI_ISL_3721953, EPI_ISL_3757608, EPI_ISL_3769078, EPI_ISL_3805701, EPI_ISL_3805702, EPI_ISL_3814483, EPI_ISL_3817319, EPI_ISL_3817320, EPI_ISL_3848578, EPI_ISL_3848863, EPI_ISL_3851025, EPI_ISL_3855894, EPI_ISL_3905876, EPI_ISL_3912953, EPI_ISL_3913297, EPI_ISL_3945907, EPI_ISL_4006304, EPI_ISL_4006313, EPI_ISL_4035603, EPI_ISL_4071438, EPI_ISL_4158860, EPI_ISL_4158861, EPI_ISL_4232402, EPI_ISL_4232459, EPI_ISL_4276964, EPI_ISL_4298883, EPI_ISL_4298887, EPI_ISL_4412175, EPI_ISL_4455120, EPI_ISL_4466556, EPI_ISL_4507562, EPI_ISL_4550213, EPI_ISL_4743240, EPI_ISL_4743997, EPI_ISL_4917854, EPI_ISL_4917855, EPI_ISL_5014464, EPI_ISL_5193918, EPI_ISL_5337621, EPI_ISL_5338716, EPI_ISL_5340253, EPI_ISL_5340620, EPI_ISL_5342459, EPI_ISL_5344003, EPI_ISL_5344344, EPI_ISL_5344451, EPI_ISL_5517977, EPI_ISL_5781530, EPI_ISL_6475949, EPI_ISL_6483957, EPI_ISL_6486767, EPI_ISL_7812871, EPI_ISL_7813075, EPI_ISL_7813116, EPI_ISL_7813229, EPI_ISL_7813230, EPI_ISL_7813236, EPI_ISL_9540705, EPI_ISL_9545466, EPI_ISL_9546569, EPI_ISL_9548140, EPI_ISL_11090742, EPI_ISL_19570451, EPI_ISL_19587372")
    B1_633 = stringlist_to_strings_set("EPI_ISL_4132923, EPI_ISL_3006978, EPI_ISL_5978028, EPI_ISL_2556441, EPI_ISL_2253528, EPI_ISL_3006966, EPI_ISL_2880232, EPI_ISL_3277096, EPI_ISL_2461370, EPI_ISL_2556087, EPI_ISL_6024311, EPI_ISL_2881152, EPI_ISL_3267406, EPI_ISL_3267216, EPI_ISL_2461371, EPI_ISL_2880231, EPI_ISL_3277103")
    B1_638 = stringlist_to_strings_set("EPI_ISL_3451089, EPI_ISL_2693841, EPI_ISL_3718070, EPI_ISL_3451090, EPI_ISL_2955458, EPI_ISL_3717989, EPI_ISL_3717991, EPI_ISL_3451094, EPI_ISL_3451091, EPI_ISL_3451093, EPI_ISL_3451092")
    C12 = stringlist_to_strings_set("EPI_ISL_18141392, EPI_ISL_5510293, EPI_ISL_6966915, EPI_ISL_13830454, EPI_ISL_4877022, EPI_ISL_4572846, EPI_ISL_3451295, EPI_ISL_4505691, EPI_ISL_4029943, EPI_ISL_6201919, EPI_ISL_17118749, EPI_ISL_5416314, EPI_ISL_4301822, EPI_ISL_6422305, EPI_ISL_4029946, EPI_ISL_3411457, EPI_ISL_4649835, EPI_ISL_6966431, EPI_ISL_3237233, EPI_ISL_6010139, EPI_ISL_3451195, EPI_ISL_3132623, EPI_ISL_3746871, EPI_ISL_4301836, EPI_ISL_3149300, EPI_ISL_2770450, EPI_ISL_9439179, EPI_ISL_5250003, EPI_ISL_4030023, EPI_ISL_5918093, EPI_ISL_3451362, EPI_ISL_3482536, EPI_ISL_3730315, EPI_ISL_3717993, EPI_ISL_7310266, EPI_ISL_3261918, EPI_ISL_3236953, EPI_ISL_6327852, EPI_ISL_3643862, EPI_ISL_5196557, EPI_ISL_6202863, EPI_ISL_2988404, EPI_ISL_17118744, EPI_ISL_4498625, EPI_ISL_4575101, EPI_ISL_3643966, EPI_ISL_6422315, EPI_ISL_6966423, EPI_ISL_11435082, EPI_ISL_6704861, EPI_ISL_5416342, EPI_ISL_5196625, EPI_ISL_4301774, EPI_ISL_2988405, EPI_ISL_5510296, EPI_ISL_3451144, EPI_ISL_3799102, EPI_ISL_3729072, EPI_ISL_4651575, EPI_ISL_4121731, EPI_ISL_4816930, EPI_ISL_5501810, EPI_ISL_3838520, EPI_ISL_7015167, EPI_ISL_3838614, EPI_ISL_4030024, EPI_ISL_5196561, EPI_ISL_3411459, EPI_ISL_2803815, EPI_ISL_3451214, EPI_ISL_3746811, EPI_ISL_3342732, EPI_ISL_10716937, EPI_ISL_5249928, EPI_ISL_4029912, EPI_ISL_4301764, EPI_ISL_4651914, EPI_ISL_9201638, EPI_ISL_3281602, EPI_ISL_5416309, EPI_ISL_7971616, EPI_ISL_5918098, EPI_ISL_8801146, EPI_ISL_6327942, EPI_ISL_5510294, EPI_ISL_6308315, EPI_ISL_16078910, EPI_ISL_6900134, EPI_ISL_6585209, EPI_ISL_5603133, EPI_ISL_6966611, EPI_ISL_16073449, EPI_ISL_5510477, EPI_ISL_5416370, EPI_ISL_7971718, EPI_ISL_5918082, EPI_ISL_4572320, EPI_ISL_7751200, EPI_ISL_3859880, EPI_ISL_6693554, EPI_ISL_10878448, EPI_ISL_2989113, EPI_ISL_4029941, EPI_ISL_3746752, EPI_ISL_17118742, EPI_ISL_11376225, EPI_ISL_4029927, EPI_ISL_3236186, EPI_ISL_6203560, EPI_ISL_4572840, EPI_ISL_7456442, EPI_ISL_5917927, EPI_ISL_5918091, EPI_ISL_3859890, EPI_ISL_3838321, EPI_ISL_12954333, EPI_ISL_3827640, EPI_ISL_3643965, EPI_ISL_3451544, EPI_ISL_4891676, EPI_ISL_7337403, EPI_ISL_4891529, EPI_ISL_4253663, EPI_ISL_5250000, EPI_ISL_8182904, EPI_ISL_3722231, EPI_ISL_5416341, EPI_ISL_8801148, EPI_ISL_4473177, EPI_ISL_7605669, EPI_ISL_3447713, EPI_ISL_3411467, EPI_ISL_5429037, EPI_ISL_12954319, EPI_ISL_4572340, EPI_ISL_8182902, EPI_ISL_6202177, EPI_ISL_8801145, EPI_ISL_7852873, EPI_ISL_3411463, EPI_ISL_16073445, EPI_ISL_3342730, EPI_ISL_3236951, EPI_ISL_6585226, EPI_ISL_6202861, EPI_ISL_4301828, EPI_ISL_3451391, EPI_ISL_6327902, EPI_ISL_11673422, EPI_ISL_6966874, EPI_ISL_4030022, EPI_ISL_4473159, EPI_ISL_6327957, EPI_ISL_4572321, EPI_ISL_4572845, EPI_ISL_5099204, EPI_ISL_11978044, EPI_ISL_7456446, EPI_ISL_3237063, EPI_ISL_3447717, EPI_ISL_5099171, EPI_ISL_4884442, EPI_ISL_3411681, EPI_ISL_3149306, EPI_ISL_9439165, EPI_ISL_7337381, EPI_ISL_4572347, EPI_ISL_3237084, EPI_ISL_4121681, EPI_ISL_3799162, EPI_ISL_5510297, EPI_ISL_5250001, EPI_ISL_7740878, EPI_ISL_7337385, EPI_ISL_7605582, EPI_ISL_4891546, EPI_ISL_4650213, EPI_ISL_2942287, EPI_ISL_3838556, EPI_ISL_4968419, EPI_ISL_5416369, EPI_ISL_5098944, EPI_ISL_3033753, EPI_ISL_4891545, EPI_ISL_4253610, EPI_ISL_3261970, EPI_ISL_6010138, EPI_ISL_5416297, EPI_ISL_3447773, EPI_ISL_5098812, EPI_ISL_3447714, EPI_ISL_3451378, EPI_ISL_17758039, EPI_ISL_3149313, EPI_ISL_5098807, EPI_ISL_17758080, EPI_ISL_6203099, EPI_ISL_4968413, EPI_ISL_6308256, EPI_ISL_2841668, EPI_ISL_4968415, EPI_ISL_8801151, EPI_ISL_3447758, EPI_ISL_4193905, EPI_ISL_3722270, EPI_ISL_5249995, EPI_ISL_3722264, EPI_ISL_5510295, EPI_ISL_5416523, EPI_ISL_2931281, EPI_ISL_7456406, EPI_ISL_7337388, EPI_ISL_7337401, EPI_ISL_6327890, EPI_ISL_4575118, EPI_ISL_6327851, EPI_ISL_13037920, EPI_ISL_3838375, EPI_ISL_5196617, EPI_ISL_5540191, EPI_ISL_3281600, EPI_ISL_3149299, EPI_ISL_5501902, EPI_ISL_14594777, EPI_ISL_3149307, EPI_ISL_3838569, EPI_ISL_5196616, EPI_ISL_5249952, EPI_ISL_12954336, EPI_ISL_8957611, EPI_ISL_3451358, EPI_ISL_7337390, EPI_ISL_5018646, EPI_ISL_3287712, EPI_ISL_4253656, EPI_ISL_4029935, EPI_ISL_3746874, EPI_ISL_3643842, EPI_ISL_6327948, EPI_ISL_6585204, EPI_ISL_12954342, EPI_ISL_4968411, EPI_ISL_6202316, EPI_ISL_4336900, EPI_ISL_2984801, EPI_ISL_9439008, EPI_ISL_5603137, EPI_ISL_4253637, EPI_ISL_5196531, EPI_ISL_3717995, EPI_ISL_4030021, EPI_ISL_3860015, EPI_ISL_6436394, EPI_ISL_5429026, EPI_ISL_3237098, EPI_ISL_3697115, EPI_ISL_6308257, EPI_ISL_3132566, EPI_ISL_3799095, EPI_ISL_3342734, EPI_ISL_9230060, EPI_ISL_3342733, EPI_ISL_3451569, EPI_ISL_3237092, EPI_ISL_2718062, EPI_ISL_6327850, EPI_ISL_4876926, EPI_ISL_3717994, EPI_ISL_4029948, EPI_ISL_17758081, EPI_ISL_5918092, EPI_ISL_3149312, EPI_ISL_5917886, EPI_ISL_11435083, EPI_ISL_12954141, EPI_ISL_6201944, EPI_ISL_3746788, EPI_ISL_7881950, EPI_ISL_8801147, EPI_ISL_3219805, EPI_ISL_4650090, EPI_ISL_4029923, EPI_ISL_7310320, EPI_ISL_3663539, EPI_ISL_4891705, EPI_ISL_3074033, EPI_ISL_3799035, EPI_ISL_3838541, EPI_ISL_17758085, EPI_ISL_6585202, EPI_ISL_6782048, EPI_ISL_18215881, EPI_ISL_3482519, EPI_ISL_3451222, EPI_ISL_13963277, EPI_ISL_8616711, EPI_ISL_4891659, EPI_ISL_5249934, EPI_ISL_4572381, EPI_ISL_3411458, EPI_ISL_4301791, EPI_ISL_3451369, EPI_ISL_3453877, EPI_ISL_9439176, EPI_ISL_6436401, EPI_ISL_3799163, EPI_ISL_3643860, EPI_ISL_3506424, EPI_ISL_3128775, EPI_ISL_5416371, EPI_ISL_3746772, EPI_ISL_2828749, EPI_ISL_6203297, EPI_ISL_5477673, EPI_ISL_4003563, EPI_ISL_2827937, EPI_ISL_3237237, EPI_ISL_3746842, EPI_ISL_3838512, EPI_ISL_3132608, EPI_ISL_3838515, EPI_ISL_6203350, EPI_ISL_2726854, EPI_ISL_6327853, EPI_ISL_4474416, EPI_ISL_3860060, EPI_ISL_3859884, EPI_ISL_6966660, EPI_ISL_4003556, EPI_ISL_6693553, EPI_ISL_5098934, EPI_ISL_4651145, EPI_ISL_7971590, EPI_ISL_4108306, EPI_ISL_3643903, EPI_ISL_4968412, EPI_ISL_4651357, EPI_ISL_3451301, EPI_ISL_6966959, EPI_ISL_4301837, EPI_ISL_11516060, EPI_ISL_3219868, EPI_ISL_12954357, EPI_ISL_4968418, EPI_ISL_3149301, EPI_ISL_4029964, EPI_ISL_3411589, EPI_ISL_11435084, EPI_ISL_5540181, EPI_ISL_5917846, EPI_ISL_7456429, EPI_ISL_5250002, EPI_ISL_2988409, EPI_ISL_4650166, EPI_ISL_3838489, EPI_ISL_3722284, EPI_ISL_4473174, EPI_ISL_6900144, EPI_ISL_3838621, EPI_ISL_5099194, EPI_ISL_17118750, EPI_ISL_12954142, EPI_ISL_2695610, EPI_ISL_9439051, EPI_ISL_6906407, EPI_ISL_6201931, EPI_ISL_11435085, EPI_ISL_3717932, EPI_ISL_6203343, EPI_ISL_3101505, EPI_ISL_6201939, EPI_ISL_6327854, EPI_ISL_6913983, EPI_ISL_3281601, EPI_ISL_6966691, EPI_ISL_4575181, EPI_ISL_4030025, EPI_ISL_4474432, EPI_ISL_4336918, EPI_ISL_2726855, EPI_ISL_3799137")
    manual = stringlist_to_strings_set("EPI_ISL_16120995, EPI_ISL_16553517, EPI_ISL_16814940, EPI_ISL_16814949, EPI_ISL_16815034, EPI_ISL_16599636, EPI_ISL_16271048, EPI_ISL_17275297, EPI_ISL_16699252, EPI_ISL_15473936, EPI_ISL_9629866, EPI_ISL_1109976, EPI_ISL_482775, EPI_ISL_12511485, EPI_ISL_7945646, EPI_ISL_7944977, EPI_ISL_7792628, EPI_ISL_5470943, EPI_ISL_467299, EPI_ISL_1517192, EPI_ISL_13384362, EPI_ISL_1034151, EPI_ISL_7947025, EPI_ISL_2754678, EPI_ISL_1625031, EPI_ISL_9629922, EPI_ISL_9077212, EPI_ISL_17597388, EPI_ISL_13544323, EPI_ISL_955140, EPI_ISL_12736844, EPI_ISL_2887404, EPI_ISL_5934793, EPI_ISL_17034838, EPI_ISL_12955760, EPI_ISL_7591215, EPI_ISL_7591217, EPI_ISL_7591219, EPI_ISL_7591222, EPI_ISL_1696687, EPI_ISL_1696692, EPI_ISL_1696714, EPI_ISL_1696715, EPI_ISL_1133157, EPI_ISL_1704072, EPI_ISL_17238817, EPI_ISL_3299585, EPI_ISL_2095083, EPI_ISL_2095085, EPI_ISL_2095099, EPI_ISL_2095111, EPI_ISL_2095114, EPI_ISL_2095125, EPI_ISL_19013684, EPI_ISL_18110550, EPI_ISL_17785884, EPI_ISL_17821633, EPI_ISL_17821640, EPI_ISL_17821665, EPI_ISL_1960965, EPI_ISL_1960981, EPI_ISL_1960987, EPI_ISL_1961000, EPI_ISL_8422490, EPI_ISL_12582703, EPI_ISL_18572143, EPI_ISL_2931767, EPI_ISL_12146277, EPI_ISL_12320529, EPI_ISL_12320662, EPI_ISL_10697706, EPI_ISL_10697708, EPI_ISL_10697709, EPI_ISL_10697711, EPI_ISL_10697714, EPI_ISL_10697716, EPI_ISL_10697718, EPI_ISL_10697720, EPI_ISL_10697721, EPI_ISL_10697722, EPI_ISL_10697723, EPI_ISL_10697724, EPI_ISL_18352009, EPI_ISL_18139468, EPI_ISL_15316453, EPI_ISL_10432651, EPI_ISL_3941723, EPI_ISL_1133149, EPI_ISL_12152767, EPI_ISL_12321743, EPI_ISL_12906443, EPI_ISL_17372560, EPI_ISL_19014042, EPI_ISL_17097728, EPI_ISL_14552727, EPI_ISL_14584747, EPI_ISL_14584752, EPI_ISL_14584768, EPI_ISL_14584783, EPI_ISL_961464, EPI_ISL_1704083, EPI_ISL_2106988, EPI_ISL_2547087, EPI_ISL_3631655, EPI_ISL_7873234, EPI_ISL_8445235, EPI_ISL_8445256, EPI_ISL_8975271, EPI_ISL_9093175, EPI_ISL_9112420, EPI_ISL_9305884, EPI_ISL_9348945, EPI_ISL_10121550, EPI_ISL_10716792, EPI_ISL_10716808, EPI_ISL_10716815, EPI_ISL_11336335, EPI_ISL_11799946, EPI_ISL_12906191, EPI_ISL_12906192, EPI_ISL_12906193, EPI_ISL_12906194, EPI_ISL_12906195, EPI_ISL_12906196, EPI_ISL_12906197, EPI_ISL_12906198, EPI_ISL_12906199, EPI_ISL_12906200, EPI_ISL_12906201, EPI_ISL_12906202, EPI_ISL_12906203, EPI_ISL_12906204, EPI_ISL_12906205, EPI_ISL_12906206, EPI_ISL_12906207, EPI_ISL_12906208, EPI_ISL_12906209, EPI_ISL_12906213, EPI_ISL_12906215, EPI_ISL_12906216, EPI_ISL_12906217, EPI_ISL_12906221, EPI_ISL_12906223, EPI_ISL_12906224, EPI_ISL_12906225, EPI_ISL_12906226, EPI_ISL_12906227, EPI_ISL_12906228, EPI_ISL_12906229, EPI_ISL_12906230, EPI_ISL_12906231, EPI_ISL_12906232, EPI_ISL_12906233, EPI_ISL_12906234, EPI_ISL_12906236, EPI_ISL_12906239, EPI_ISL_12906241, EPI_ISL_12906245, EPI_ISL_12906246, EPI_ISL_12906252, EPI_ISL_12906256, EPI_ISL_12906263, EPI_ISL_12906264, EPI_ISL_12906268, EPI_ISL_12906270, EPI_ISL_12906271, EPI_ISL_12906272, EPI_ISL_12906273, EPI_ISL_12906276, EPI_ISL_12906281, EPI_ISL_12906282, EPI_ISL_12906283, EPI_ISL_12906284, EPI_ISL_12906285, EPI_ISL_12906287, EPI_ISL_12906288, EPI_ISL_12906289, EPI_ISL_12906292, EPI_ISL_12906294, EPI_ISL_12906300, EPI_ISL_12906301, EPI_ISL_12906302, EPI_ISL_12906305, EPI_ISL_12906306, EPI_ISL_12906307, EPI_ISL_12906310, EPI_ISL_12906311, EPI_ISL_12906312, EPI_ISL_12906315, EPI_ISL_12906316, EPI_ISL_12906317, EPI_ISL_12906318, EPI_ISL_12906319, EPI_ISL_12906320, EPI_ISL_12906321, EPI_ISL_12906322, EPI_ISL_12906323, EPI_ISL_12906325, EPI_ISL_12906339, EPI_ISL_12906442, EPI_ISL_12906445, EPI_ISL_12906446, EPI_ISL_12906447, EPI_ISL_12906448, EPI_ISL_12906449, EPI_ISL_12906450, EPI_ISL_12906589, EPI_ISL_12906591, EPI_ISL_12906593, EPI_ISL_12906596, EPI_ISL_12906598, EPI_ISL_12906610, EPI_ISL_12906621, EPI_ISL_12906626, EPI_ISL_12906635, EPI_ISL_12906639, EPI_ISL_12934416, EPI_ISL_12955769, EPI_ISL_12999272, EPI_ISL_13304547, EPI_ISL_15038277, EPI_ISL_15251283, EPI_ISL_15348547, EPI_ISL_16407943, EPI_ISL_17275302, EPI_ISL_17301938, EPI_ISL_17301940, EPI_ISL_17301944, EPI_ISL_17302062, EPI_ISL_17302064, EPI_ISL_17302068, EPI_ISL_17302070, EPI_ISL_17302071, EPI_ISL_17466156, EPI_ISL_17466157, EPI_ISL_17466158, EPI_ISL_17466159, EPI_ISL_17466160, EPI_ISL_17466161, EPI_ISL_17466162, EPI_ISL_17466163, EPI_ISL_17466164, EPI_ISL_17466165, EPI_ISL_17466166, EPI_ISL_17466167, EPI_ISL_17466168, EPI_ISL_17466169, EPI_ISL_17466170, EPI_ISL_17466171, EPI_ISL_17466172, EPI_ISL_17466173, EPI_ISL_17466174, EPI_ISL_17466175, EPI_ISL_17466176, EPI_ISL_17466177, EPI_ISL_17466178, EPI_ISL_17466179, EPI_ISL_12583843, EPI_ISL_12146044, EPI_ISL_12146227, EPI_ISL_1957278, EPI_ISL_16061073, EPI_ISL_1173555, EPI_ISL_17682075, EPI_ISL_17989401, EPI_ISL_14561987, EPI_ISL_12194616, EPI_ISL_449800, EPI_ISL_8688196, EPI_ISL_2389452, EPI_ISL_2375846, EPI_ISL_1825065, EPI_ISL_2375511, EPI_ISL_11468163, EPI_ISL_10551554, EPI_ISL_10551562, EPI_ISL_13478523, EPI_ISL_10314946, EPI_ISL_5484223, EPI_ISL_17445536, EPI_ISL_18385487, EPI_ISL_1703777, EPI_ISL_1703826, EPI_ISL_16354122, EPI_ISL_15348741, EPI_ISL_15415391, EPI_ISL_9861349, EPI_ISL_1133148, EPI_ISL_1166096, EPI_ISL_1166102, EPI_ISL_1166104, EPI_ISL_1469239, EPI_ISL_10523160, EPI_ISL_10851659, EPI_ISL_10851661, EPI_ISL_10858436, EPI_ISL_10858527, EPI_ISL_10858541, EPI_ISL_11333353, EPI_ISL_11507483, EPI_ISL_11507489, EPI_ISL_11507524, EPI_ISL_11793451, EPI_ISL_12146193, EPI_ISL_12291347, EPI_ISL_12308587, EPI_ISL_12321607, EPI_ISL_12583926, EPI_ISL_12694427, EPI_ISL_12955756, EPI_ISL_12955775, EPI_ISL_12955828, EPI_ISL_13176027, EPI_ISL_13307898, EPI_ISL_13847367, EPI_ISL_14999066, EPI_ISL_15038548, EPI_ISL_1185776, EPI_ISL_4551799, EPI_ISL_1184558, EPI_ISL_1185776, EPI_ISL_18098351, EPI_ISL_8827492, EPI_ISL_9196492, EPI_ISL_8193909, EPI_ISL_8251487, EPI_ISL_5686636, EPI_ISL_3743165, EPI_ISL_1915533, EPI_ISL_11887232, EPI_ISL_11336333, EPI_ISL_12753427, EPI_ISL_12753104, EPI_ISL_3775926, EPI_ISL_6599771, EPI_ISL_12955519, EPI_ISL_14847621, EPI_ISL_9658674, EPI_ISL_9202505, EPI_ISL_9202539, EPI_ISL_9030329, EPI_ISL_9746968, EPI_ISL_9518504, EPI_ISL_9133317, EPI_ISL_9300549, EPI_ISL_9437783, EPI_ISL_9669302, EPI_ISL_9669737, EPI_ISL_9519903, EPI_ISL_9437783, EPI_ISL_9133317, EPI_ISL_9518504, EPI_ISL_9746968, EPI_ISL_9030329, EPI_ISL_9202505, EPI_ISL_9202539, EPI_ISL_9437783, EPI_ISL_9093609, EPI_ISL_9093690, EPI_ISL_7566355, EPI_ISL_6949207, EPI_ISL_6117425, EPI_ISL_6898986, EPI_ISL_6208342, EPI_ISL_6208343, EPI_ISL_6208344, EPI_ISL_6208345, EPI_ISL_6208346, EPI_ISL_6368268, EPI_ISL_6582943, EPI_ISL_5736276, EPI_ISL_5736289, EPI_ISL_5736297, EPI_ISL_5736302, EPI_ISL_5736307, EPI_ISL_5736314, EPI_ISL_5736811, EPI_ISL_5738176, EPI_ISL_5367954, EPI_ISL_5123044, EPI_ISL_5384223, EPI_ISL_5344440, EPI_ISL_5854760, EPI_ISL_5536462, EPI_ISL_5536463, EPI_ISL_5599480, EPI_ISL_5599487, EPI_ISL_5599496, EPI_ISL_5599500, EPI_ISL_5867029, EPI_ISL_5324096, EPI_ISL_5070742, EPI_ISL_5073213, EPI_ISL_5390697, EPI_ISL_2428882, EPI_ISL_2428956, EPI_ISL_7370200, EPI_ISL_7380510, EPI_ISL_7456430, EPI_ISL_7456435, EPI_ISL_7456462, EPI_ISL_7547036, EPI_ISL_7566326, EPI_ISL_7566334, EPI_ISL_7566340, EPI_ISL_7566342, EPI_ISL_7566350, EPI_ISL_7566352, EPI_ISL_7566354, EPI_ISL_7566356, EPI_ISL_7566358, EPI_ISL_7566359, EPI_ISL_7566361, EPI_ISL_7566362, EPI_ISL_7566367, EPI_ISL_7566369, EPI_ISL_7566372, EPI_ISL_7566374, EPI_ISL_7566375, EPI_ISL_7566381, EPI_ISL_7566383, EPI_ISL_7566386, EPI_ISL_7568581, EPI_ISL_7568583, EPI_ISL_7568585, EPI_ISL_7568586, EPI_ISL_7568588, EPI_ISL_7568590, EPI_ISL_7568595, EPI_ISL_7568596, EPI_ISL_7568598, EPI_ISL_7575090, EPI_ISL_7594185, EPI_ISL_7605583, EPI_ISL_7605585, EPI_ISL_7615994, EPI_ISL_7634556, EPI_ISL_7660560, EPI_ISL_7698118, EPI_ISL_7716422, EPI_ISL_7716481, EPI_ISL_7719334, EPI_ISL_7729239, EPI_ISL_7729406, EPI_ISL_7734039, EPI_ISL_7734070, EPI_ISL_7734081, EPI_ISL_7738202, EPI_ISL_7743919, EPI_ISL_7747278, EPI_ISL_7762218, EPI_ISL_7763437, EPI_ISL_7795567, EPI_ISL_7807933, EPI_ISL_7808511, EPI_ISL_7818414, EPI_ISL_7842429, EPI_ISL_7848702, EPI_ISL_7850777, EPI_ISL_7850785, EPI_ISL_7860076, EPI_ISL_7862534, EPI_ISL_7862924, EPI_ISL_7863030, EPI_ISL_7863256, EPI_ISL_7868872, EPI_ISL_7869228, EPI_ISL_7882980, EPI_ISL_7895735, EPI_ISL_7948596, EPI_ISL_7952860, EPI_ISL_7957890, EPI_ISL_7961502, EPI_ISL_7961514, EPI_ISL_7961912, EPI_ISL_7963486, EPI_ISL_7969832, EPI_ISL_7971033, EPI_ISL_7976101, EPI_ISL_7977800, EPI_ISL_7977825, EPI_ISL_7977909, EPI_ISL_7978204, EPI_ISL_7978212, EPI_ISL_7978362, EPI_ISL_7978364, EPI_ISL_7978604, EPI_ISL_7979198, EPI_ISL_7979224, EPI_ISL_7980072, EPI_ISL_7992055, EPI_ISL_7994652, EPI_ISL_7994984, EPI_ISL_7605585, EPI_ISL_7337601, EPI_ISL_7850777, EPI_ISL_7850785, EPI_ISL_7698118, EPI_ISL_7734039, EPI_ISL_7734070, EPI_ISL_7734081, EPI_ISL_7977800, EPI_ISL_7977825, EPI_ISL_7977909, EPI_ISL_7978204, EPI_ISL_7978212, EPI_ISL_7978362, EPI_ISL_7978364, EPI_ISL_7978604, EPI_ISL_7979198, EPI_ISL_7979224, EPI_ISL_7980072, EPI_ISL_7738202, EPI_ISL_7848702, EPI_ISL_7842429, EPI_ISL_7969832, EPI_ISL_7868872, EPI_ISL_7807933, EPI_ISL_7808511, EPI_ISL_7615994, EPI_ISL_7594185, EPI_ISL_7265940, EPI_ISL_7952860, EPI_ISL_7869228, EPI_ISL_7994984, EPI_ISL_7994652, EPI_ISL_7992055, EPI_ISL_7795567, EPI_ISL_7895735, EPI_ISL_7862534, EPI_ISL_7862924, EPI_ISL_7863030, EPI_ISL_7863256, EPI_ISL_7575090, EPI_ISL_7963486, EPI_ISL_7762218, EPI_ISL_7763437, EPI_ISL_7818414, EPI_ISL_7634556, EPI_ISL_7719334, EPI_ISL_7860076, EPI_ISL_7605583, EPI_ISL_7337522, EPI_ISL_7337591, EPI_ISL_7167723, EPI_ISL_7747278, EPI_ISL_7716422, EPI_ISL_7716481, EPI_ISL_7976101, EPI_ISL_7566355, EPI_ISL_14210694, EPI_ISL_15846254, EPI_ISL_15945485, EPI_ISL_9456848, EPI_ISL_9456983, EPI_ISL_9156804, EPI_ISL_15709506, EPI_ISL_15025376, EPI_ISL_15940837, EPI_ISL_15975710, EPI_ISL_15025373, EPI_ISL_15270610, EPI_ISL_15670087, EPI_ISL_15250585, EPI_ISL_15709506, EPI_ISL_15880471, EPI_ISL_15879843, EPI_ISL_15897084, EPI_ISL_15897053, EPI_ISL_15886020, EPI_ISL_15866417, EPI_ISL_15874062, EPI_ISL_15179344, EPI_ISL_15237955, EPI_ISL_15399812, EPI_ISL_15494652, EPI_ISL_15482029, EPI_ISL_15947183, EPI_ISL_15982525, EPI_ISL_15846260, EPI_ISL_15831957")
    India = stringlist_to_strings_set("EPI_ISL_479776, EPI_ISL_2547229, EPI_ISL_1703710, EPI_ISL_541731, EPI_ISL_577654, EPI_ISL_2966131, EPI_ISL_11106543, EPI_ISL_11887241, EPI_ISL_11887313, EPI_ISL_11887728, EPI_ISL_11887754, EPI_ISL_11887820, EPI_ISL_12291249, EPI_ISL_12291339, EPI_ISL_9782271, EPI_ISL_8049924, EPI_ISL_8049928, EPI_ISL_17372400, EPI_ISL_17372447, EPI_ISL_17372505, EPI_ISL_17372556, EPI_ISL_17372557, EPI_ISL_17372567, EPI_ISL_15294031, EPI_ISL_15363265, EPI_ISL_14910936, EPI_ISL_14495310, EPI_ISL_14495319, EPI_ISL_14495329, EPI_ISL_14495334, EPI_ISL_14148984, EPI_ISL_13158620, EPI_ISL_13158678, EPI_ISL_13158687, EPI_ISL_13158700, EPI_ISL_12494547, EPI_ISL_12494558, EPI_ISL_12494582, EPI_ISL_12494585, EPI_ISL_12494587, EPI_ISL_12494596, EPI_ISL_12494638, EPI_ISL_12494874, EPI_ISL_10149349, EPI_ISL_14975997, EPI_ISL_14975999, EPI_ISL_14976001, EPI_ISL_14976002, EPI_ISL_14976004, EPI_ISL_14976005, EPI_ISL_14976006, EPI_ISL_14976008, EPI_ISL_14976014, EPI_ISL_14976017, EPI_ISL_14976018, EPI_ISL_14976094, EPI_ISL_14976096, EPI_ISL_14976099, EPI_ISL_14976106, EPI_ISL_14976107, EPI_ISL_14976109, EPI_ISL_14976111, EPI_ISL_14976112, EPI_ISL_14976113, EPI_ISL_14976123, EPI_ISL_14976129, EPI_ISL_14976139, EPI_ISL_14976142, EPI_ISL_14976146, EPI_ISL_14976147, EPI_ISL_14976148, EPI_ISL_14976150, EPI_ISL_14976153, EPI_ISL_14976154, EPI_ISL_14986408, EPI_ISL_14986472, EPI_ISL_14986473, EPI_ISL_14986476, EPI_ISL_14986478, EPI_ISL_14986480, EPI_ISL_14986481, EPI_ISL_14986482, EPI_ISL_14986484, EPI_ISL_14986486, EPI_ISL_14986487, EPI_ISL_14986488, EPI_ISL_14986489, EPI_ISL_14986490, EPI_ISL_14986493, EPI_ISL_14986505, EPI_ISL_14986525, EPI_ISL_14986529, EPI_ISL_14986530, EPI_ISL_14986532, EPI_ISL_14986533, EPI_ISL_14986539, EPI_ISL_14986542, EPI_ISL_14986543, EPI_ISL_14986544, EPI_ISL_14986545, EPI_ISL_14986547, EPI_ISL_14986550, EPI_ISL_14986551, EPI_ISL_14986554, EPI_ISL_14986555, EPI_ISL_14986556, EPI_ISL_14986560, EPI_ISL_14986563, EPI_ISL_14986567, EPI_ISL_14986568, EPI_ISL_14986570, EPI_ISL_14986574, EPI_ISL_14986580, EPI_ISL_14986583, EPI_ISL_14986591, EPI_ISL_14986594, EPI_ISL_14986600, EPI_ISL_14986601, EPI_ISL_14986606, EPI_ISL_14986608, EPI_ISL_14986612, EPI_ISL_14986613, EPI_ISL_14986619, EPI_ISL_14986623, EPI_ISL_14986624, EPI_ISL_14986625, EPI_ISL_14986630, EPI_ISL_14986631, EPI_ISL_14986633, EPI_ISL_14992926, EPI_ISL_14992971, EPI_ISL_14993025, EPI_ISL_14993053, EPI_ISL_14993093, EPI_ISL_14993097, EPI_ISL_14993098, EPI_ISL_14993100, EPI_ISL_14993111, EPI_ISL_14993113, EPI_ISL_14993114, EPI_ISL_14993139, EPI_ISL_14993140, EPI_ISL_14993141, EPI_ISL_14993142, EPI_ISL_14993146, EPI_ISL_14993173, EPI_ISL_14993174, EPI_ISL_15172564, EPI_ISL_15172596, EPI_ISL_15172598, EPI_ISL_15172634, EPI_ISL_15172657, EPI_ISL_15172658, EPI_ISL_15172662, EPI_ISL_15172663, EPI_ISL_15172678, EPI_ISL_15172680, EPI_ISL_15172702, EPI_ISL_15172709, EPI_ISL_15172719, EPI_ISL_15172720, EPI_ISL_15172726, EPI_ISL_15172727, EPI_ISL_15172729, EPI_ISL_15172730, EPI_ISL_15172733, EPI_ISL_15172741, EPI_ISL_15172743, EPI_ISL_15172745, EPI_ISL_15190291, EPI_ISL_15262286, EPI_ISL_15293825, EPI_ISL_15293919, EPI_ISL_15293920, EPI_ISL_15293921, EPI_ISL_15293922, EPI_ISL_15293923, EPI_ISL_15293924, EPI_ISL_15293951, EPI_ISL_15293996, EPI_ISL_15294000, EPI_ISL_15294013, EPI_ISL_15294014, EPI_ISL_15294017, EPI_ISL_15294021, EPI_ISL_15294023, EPI_ISL_15294024, EPI_ISL_15294025, EPI_ISL_15294027, EPI_ISL_15294028, EPI_ISL_11336342, EPI_ISL_11336416, EPI_ISL_11359809, EPI_ISL_11359818, EPI_ISL_11373282, EPI_ISL_11373303, EPI_ISL_11373308, EPI_ISL_11373310, EPI_ISL_11373330, EPI_ISL_11373343, EPI_ISL_11373365, EPI_ISL_11373384, EPI_ISL_11373430, EPI_ISL_11507507, EPI_ISL_11736106, EPI_ISL_11736364, EPI_ISL_11736368, EPI_ISL_11736370, EPI_ISL_11736383, EPI_ISL_11736392, EPI_ISL_11736399, EPI_ISL_11736401, EPI_ISL_11737568, EPI_ISL_11737648, EPI_ISL_11737649, EPI_ISL_11737670, EPI_ISL_11737703, EPI_ISL_11737704, EPI_ISL_11737705, EPI_ISL_11737757, EPI_ISL_11738154, EPI_ISL_11743906, EPI_ISL_11743950, EPI_ISL_11743951, EPI_ISL_11743952, EPI_ISL_11743956, EPI_ISL_11743957, EPI_ISL_11743964, EPI_ISL_11743967, EPI_ISL_11743971, EPI_ISL_11743972, EPI_ISL_11743980, EPI_ISL_11743981, EPI_ISL_11743982, EPI_ISL_11743984, EPI_ISL_11743985, EPI_ISL_11743987, EPI_ISL_11743992, EPI_ISL_11744005, EPI_ISL_11744006, EPI_ISL_11744012, EPI_ISL_11744047, EPI_ISL_11744059, EPI_ISL_11746406, EPI_ISL_11746517, EPI_ISL_11746523, EPI_ISL_11746556, EPI_ISL_11746561, EPI_ISL_11746579, EPI_ISL_11746642, EPI_ISL_11746682, EPI_ISL_11746691, EPI_ISL_11767469, EPI_ISL_11767883, EPI_ISL_11777028, EPI_ISL_11777627, EPI_ISL_11777640, EPI_ISL_11777683, EPI_ISL_11777799, EPI_ISL_11777817, EPI_ISL_11789967, EPI_ISL_11790070, EPI_ISL_11790073, EPI_ISL_11790078, EPI_ISL_11790082, EPI_ISL_11790083, EPI_ISL_11790088, EPI_ISL_11790094, EPI_ISL_11790096, EPI_ISL_11790105, EPI_ISL_11790135, EPI_ISL_11790203, EPI_ISL_11790222, EPI_ISL_11790247, EPI_ISL_11790250, EPI_ISL_11790253, EPI_ISL_11790259, EPI_ISL_11790274, EPI_ISL_11790278, EPI_ISL_11790290, EPI_ISL_11790294, EPI_ISL_11790305, EPI_ISL_11790310, EPI_ISL_11802438, EPI_ISL_11802439, EPI_ISL_11802546, EPI_ISL_11802599, EPI_ISL_11802600, EPI_ISL_11802602, EPI_ISL_11802603, EPI_ISL_11802608, EPI_ISL_11802616, EPI_ISL_11802639, EPI_ISL_11803132, EPI_ISL_11803143, EPI_ISL_11803157, EPI_ISL_11803274, EPI_ISL_11803277, EPI_ISL_11803351, EPI_ISL_11803374, EPI_ISL_11803386, EPI_ISL_12906222, EPI_ISL_12906238, EPI_ISL_12906247, EPI_ISL_12906251, EPI_ISL_12906255, EPI_ISL_12906258, EPI_ISL_12906259, EPI_ISL_12906261, EPI_ISL_12906265, EPI_ISL_12906267, EPI_ISL_12906274, EPI_ISL_12906280, EPI_ISL_12906286, EPI_ISL_12906290, EPI_ISL_12906293, EPI_ISL_12906296, EPI_ISL_12906299, EPI_ISL_12906303, EPI_ISL_12906304, EPI_ISL_12906309, EPI_ISL_12906313, EPI_ISL_12906314, EPI_ISL_12906369, EPI_ISL_12906374, EPI_ISL_12906383, EPI_ISL_12906455, EPI_ISL_12906464, EPI_ISL_12906470, EPI_ISL_12906479, EPI_ISL_12906498, EPI_ISL_12906503, EPI_ISL_12906506, EPI_ISL_12906509, EPI_ISL_12906511, EPI_ISL_12906518, EPI_ISL_12906542, EPI_ISL_12906559, EPI_ISL_12906562, EPI_ISL_12906590, EPI_ISL_12906592, EPI_ISL_12906597, EPI_ISL_12906599, EPI_ISL_12906600, EPI_ISL_12906601, EPI_ISL_12906602, EPI_ISL_12906604, EPI_ISL_12906609, EPI_ISL_12906618, EPI_ISL_12906619, EPI_ISL_12906622, EPI_ISL_12906627, EPI_ISL_12906628, EPI_ISL_12906632, EPI_ISL_12906633, EPI_ISL_12906636, EPI_ISL_12906637, EPI_ISL_12906638, EPI_ISL_12906640, EPI_ISL_12906210, EPI_ISL_12906211, EPI_ISL_12906212, EPI_ISL_12906214, EPI_ISL_12906218, EPI_ISL_12906219, EPI_ISL_12906220, EPI_ISL_12906235, EPI_ISL_12906237, EPI_ISL_12906240, EPI_ISL_12906242, EPI_ISL_12906243, EPI_ISL_12906244, EPI_ISL_12906248, EPI_ISL_12906249, EPI_ISL_12906250, EPI_ISL_12906253, EPI_ISL_12906254, EPI_ISL_12906257, EPI_ISL_12906260, EPI_ISL_12906262, EPI_ISL_12906266, EPI_ISL_12906269, EPI_ISL_12906275, EPI_ISL_12906277, EPI_ISL_12906278, EPI_ISL_12906279, EPI_ISL_12906291, EPI_ISL_12906295, EPI_ISL_12906297, EPI_ISL_12906298, EPI_ISL_12906308, EPI_ISL_12906324, EPI_ISL_12906326, EPI_ISL_12906327, EPI_ISL_12906328, EPI_ISL_12906329, EPI_ISL_12906330, EPI_ISL_12906331, EPI_ISL_12906332, EPI_ISL_12906333, EPI_ISL_12906334, EPI_ISL_12906335, EPI_ISL_12906336, EPI_ISL_12906337, EPI_ISL_12906338, EPI_ISL_12906340, EPI_ISL_12906341, EPI_ISL_12906342, EPI_ISL_12906343, EPI_ISL_12906344, EPI_ISL_12906345, EPI_ISL_12906346, EPI_ISL_12906347, EPI_ISL_12906348, EPI_ISL_12906349, EPI_ISL_12906350, EPI_ISL_12906351, EPI_ISL_12906352, EPI_ISL_12906353, EPI_ISL_12906354, EPI_ISL_12906355, EPI_ISL_12906356, EPI_ISL_12906357, EPI_ISL_12906358, EPI_ISL_12906359, EPI_ISL_12906360, EPI_ISL_12906361, EPI_ISL_12906362, EPI_ISL_12906363, EPI_ISL_12906364, EPI_ISL_12906365, EPI_ISL_12906366, EPI_ISL_12906367, EPI_ISL_12906368, EPI_ISL_12906370, EPI_ISL_12906371, EPI_ISL_12906372, EPI_ISL_12906373, EPI_ISL_12906375, EPI_ISL_12906376, EPI_ISL_12906377, EPI_ISL_12906378, EPI_ISL_12906379, EPI_ISL_12906380, EPI_ISL_12906381, EPI_ISL_12906382, EPI_ISL_12906384, EPI_ISL_12906385, EPI_ISL_12906386, EPI_ISL_12906387, EPI_ISL_12906388, EPI_ISL_12906389, EPI_ISL_12906390, EPI_ISL_12906391, EPI_ISL_12906392, EPI_ISL_12906393, EPI_ISL_12906394, EPI_ISL_12906395, EPI_ISL_12906396, EPI_ISL_12906397, EPI_ISL_12906398, EPI_ISL_12906399, EPI_ISL_12906400, EPI_ISL_12906401, EPI_ISL_12906402, EPI_ISL_12906403, EPI_ISL_12906404, EPI_ISL_12906405, EPI_ISL_12906406, EPI_ISL_12906407, EPI_ISL_12906408, EPI_ISL_12906409, EPI_ISL_12906410, EPI_ISL_12906411, EPI_ISL_12906412, EPI_ISL_12906413, EPI_ISL_12906414, EPI_ISL_12906415, EPI_ISL_12906416, EPI_ISL_12906417, EPI_ISL_12906418, EPI_ISL_12906419, EPI_ISL_12906420, EPI_ISL_12906421, EPI_ISL_12906422, EPI_ISL_12906423, EPI_ISL_12906424, EPI_ISL_12906425, EPI_ISL_12906426, EPI_ISL_12906427, EPI_ISL_12906428, EPI_ISL_12906429, EPI_ISL_12906430, EPI_ISL_12906431, EPI_ISL_12906432, EPI_ISL_12906433, EPI_ISL_12906434, EPI_ISL_12906435, EPI_ISL_12906436, EPI_ISL_12906437, EPI_ISL_12906438, EPI_ISL_12906439, EPI_ISL_12906440, EPI_ISL_12906441, EPI_ISL_12906444, EPI_ISL_12906451, EPI_ISL_12906452, EPI_ISL_12906453, EPI_ISL_12906454, EPI_ISL_12906456, EPI_ISL_12906457, EPI_ISL_12906458, EPI_ISL_12906459, EPI_ISL_12906460, EPI_ISL_12906461, EPI_ISL_12906462, EPI_ISL_12906463, EPI_ISL_12906465, EPI_ISL_12906466, EPI_ISL_12906467, EPI_ISL_12906468, EPI_ISL_12906469, EPI_ISL_12906471, EPI_ISL_12906472, EPI_ISL_12906473, EPI_ISL_12906474, EPI_ISL_12906475, EPI_ISL_12906476, EPI_ISL_12906477, EPI_ISL_12906478, EPI_ISL_12906480, EPI_ISL_12906481, EPI_ISL_12906482, EPI_ISL_12906483, EPI_ISL_12906484, EPI_ISL_12906485, EPI_ISL_12906486, EPI_ISL_12906487, EPI_ISL_12906488, EPI_ISL_12906489, EPI_ISL_12906490, EPI_ISL_12906491, EPI_ISL_12906492, EPI_ISL_12906493, EPI_ISL_12906494, EPI_ISL_12906495, EPI_ISL_12906496, EPI_ISL_12906497, EPI_ISL_12906499, EPI_ISL_12906500, EPI_ISL_12906501, EPI_ISL_12906502, EPI_ISL_12906504, EPI_ISL_12906505, EPI_ISL_12906507, EPI_ISL_12906508, EPI_ISL_12906510, EPI_ISL_12906512, EPI_ISL_12906513, EPI_ISL_12906514, EPI_ISL_12906515, EPI_ISL_12906516, EPI_ISL_12906517, EPI_ISL_12906519, EPI_ISL_12906520, EPI_ISL_12906521, EPI_ISL_12906522, EPI_ISL_12906523, EPI_ISL_12906524, EPI_ISL_12906525, EPI_ISL_12906526, EPI_ISL_12906527, EPI_ISL_12906528, EPI_ISL_12906529, EPI_ISL_12906530, EPI_ISL_12906531, EPI_ISL_12906532, EPI_ISL_12906533, EPI_ISL_12906534, EPI_ISL_12906535, EPI_ISL_12906536, EPI_ISL_12906537, EPI_ISL_12906538, EPI_ISL_12906539, EPI_ISL_12906540, EPI_ISL_12906541, EPI_ISL_12906543, EPI_ISL_12906544, EPI_ISL_12906545, EPI_ISL_12906546, EPI_ISL_12906547, EPI_ISL_12906548, EPI_ISL_12906549, EPI_ISL_12906550, EPI_ISL_12906551, EPI_ISL_12906552, EPI_ISL_12906553, EPI_ISL_12906554, EPI_ISL_12906555, EPI_ISL_12906556, EPI_ISL_12906557, EPI_ISL_12906558, EPI_ISL_12906560, EPI_ISL_12906561, EPI_ISL_12906563, EPI_ISL_12906564, EPI_ISL_12906565, EPI_ISL_12906566, EPI_ISL_12906567, EPI_ISL_12906568, EPI_ISL_12906569, EPI_ISL_12906570, EPI_ISL_12906571, EPI_ISL_12906572, EPI_ISL_12906573, EPI_ISL_12906574, EPI_ISL_12906575, EPI_ISL_12906576, EPI_ISL_12906577, EPI_ISL_12906578, EPI_ISL_12906579, EPI_ISL_12906580, EPI_ISL_12906581, EPI_ISL_12906582, EPI_ISL_12906583, EPI_ISL_12906584, EPI_ISL_12906585, EPI_ISL_12906586, EPI_ISL_12906587, EPI_ISL_12906588, EPI_ISL_12906594, EPI_ISL_12906595, EPI_ISL_12906603, EPI_ISL_12906605, EPI_ISL_12906606, EPI_ISL_12906607, EPI_ISL_12906608, EPI_ISL_12906611, EPI_ISL_12906612, EPI_ISL_12906613, EPI_ISL_12906614, EPI_ISL_12906615, EPI_ISL_12906616, EPI_ISL_12906617, EPI_ISL_12906620, EPI_ISL_12906623, EPI_ISL_12906624, EPI_ISL_12906625, EPI_ISL_12906629, EPI_ISL_12906630, EPI_ISL_12906631, EPI_ISL_12906634")
    Mexico = stringlist_to_strings_set("EPI_ISL_13398383, EPI_ISL_7716412, EPI_ISL_7716415, EPI_ISL_7716416, EPI_ISL_7716417, EPI_ISL_7716418, EPI_ISL_7716419, EPI_ISL_7716421, EPI_ISL_7716425, EPI_ISL_7716427, EPI_ISL_7716429, EPI_ISL_7716431, EPI_ISL_7716432, EPI_ISL_7716433, EPI_ISL_7716435, EPI_ISL_7716439, EPI_ISL_7716441, EPI_ISL_7716442, EPI_ISL_7716450, EPI_ISL_7716454, EPI_ISL_7716455, EPI_ISL_7716456, EPI_ISL_7716459, EPI_ISL_7716460, EPI_ISL_7716482, EPI_ISL_7716483, EPI_ISL_7716485, EPI_ISL_7716486, EPI_ISL_7716487, EPI_ISL_7716488, EPI_ISL_7716489, EPI_ISL_7730969")
    Indonesia = stringlist_to_strings_set("EPI_ISL_3019945, EPI_ISL_2854687, EPI_ISL_2854775, EPI_ISL_2868873, EPI_ISL_2868876, EPI_ISL_1824617, EPI_ISL_8889354, EPI_ISL_14583732, EPI_ISL_14584687, EPI_ISL_14584697")
    Thailand = stringlist_to_strings_set("EPI_ISL_17785884, EPI_ISL_6136058, EPI_ISL_6599859, EPI_ISL_5916960, EPI_ISL_5916998, EPI_ISL_3636272, EPI_ISL_4348475")
    Turkey = stringlist_to_strings_set("EPI_ISL_15927979, EPI_ISL_2102044, EPI_ISL_15681715, EPI_ISL_15295325")
    Texas = stringlist_to_strings_set("EPI_ISL_16762249, EPI_ISL_16762269, EPI_ISL_16542938, EPI_ISL_16398020, EPI_ISL_16012710, EPI_ISL_16397783, EPI_ISL_16762458, EPI_ISL_16762297, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599980, EPI_ISL_16599842, EPI_ISL_16599826, EPI_ISL_16599959, EPI_ISL_16599794, EPI_ISL_16599636, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599492, EPI_ISL_16599498, EPI_ISL_16599495, EPI_ISL_16599649, EPI_ISL_16599554, EPI_ISL_16599460, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599299, EPI_ISL_16599467, EPI_ISL_16542453, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16398022, EPI_ISL_16397883, EPI_ISL_16398016, EPI_ISL_16397988, EPI_ISL_16397860, EPI_ISL_16397721, EPI_ISL_16397554, EPI_ISL_16397720, EPI_ISL_16397699, EPI_ISL_16271386, EPI_ISL_16271102, EPI_ISL_16271048, EPI_ISL_16271255, EPI_ISL_16114505, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16113336, EPI_ISL_16113255, EPI_ISL_16113399, EPI_ISL_16013449, EPI_ISL_16013309, EPI_ISL_16013227, EPI_ISL_16012987, EPI_ISL_16012954, EPI_ISL_16012943, EPI_ISL_16013209, EPI_ISL_16013267, EPI_ISL_16599525, EPI_ISL_16762084, EPI_ISL_16599288, EPI_ISL_16599747, EPI_ISL_16270893, EPI_ISL_16762700, EPI_ISL_16762753, EPI_ISL_16762699, EPI_ISL_16762631, EPI_ISL_16762654, EPI_ISL_16762714, EPI_ISL_16762735, EPI_ISL_16762653, EPI_ISL_16762559, EPI_ISL_16762726, EPI_ISL_16762652, EPI_ISL_16762600, EPI_ISL_16762614, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762327, EPI_ISL_16762455, EPI_ISL_16762310, EPI_ISL_16762611, EPI_ISL_16762475, EPI_ISL_16762245, EPI_ISL_16762132, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762273, EPI_ISL_16762070, EPI_ISL_16762022, EPI_ISL_16762148, EPI_ISL_16761962, EPI_ISL_16762255, EPI_ISL_16762066, EPI_ISL_16761978, EPI_ISL_16761930, EPI_ISL_16761912, EPI_ISL_16542189, EPI_ISL_16542369, EPI_ISL_16542267, EPI_ISL_16762176, EPI_ISL_16762028, EPI_ISL_16762462, EPI_ISL_16599325, EPI_ISL_16542552, EPI_ISL_16271333, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16919306, EPI_ISL_16919337, EPI_ISL_16919315, EPI_ISL_16919296, EPI_ISL_16919348, EPI_ISL_16919365, EPI_ISL_16919303, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919362, EPI_ISL_16919367, EPI_ISL_16919388, EPI_ISL_16919412, EPI_ISL_16919396, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919430, EPI_ISL_16919399, EPI_ISL_16919496, EPI_ISL_16919468, EPI_ISL_16919526, EPI_ISL_16919523, EPI_ISL_16919467, EPI_ISL_16919558, EPI_ISL_16919504, EPI_ISL_16919456, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919586, EPI_ISL_16919660, EPI_ISL_16919546, EPI_ISL_16919662, EPI_ISL_16919695, EPI_ISL_16919680, EPI_ISL_16919670, EPI_ISL_16919640, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919794, EPI_ISL_16919804, EPI_ISL_16919796, EPI_ISL_16919806, EPI_ISL_16919772, EPI_ISL_16919841, EPI_ISL_16919706, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919810, EPI_ISL_16919894, EPI_ISL_16919852, EPI_ISL_16919902, EPI_ISL_16919932, EPI_ISL_16919919, EPI_ISL_16919944, EPI_ISL_16919906, EPI_ISL_16919897, EPI_ISL_16919870, EPI_ISL_16919856, EPI_ISL_16919962")
    Good_But_Not_Chronic = stringlist_to_strings_set("EPI_ISL_5336026, EPI_ISL_1419150, EPI_ISL_894228, EPI_ISL_894229, EPI_ISL_1165744, EPI_ISL_1232443, EPI_ISL_1370821")
    EPI_8mil_9mil = stringlist_to_strings_set("EPI_ISL_8001937, EPI_ISL_8014799, EPI_ISL_8017918, EPI_ISL_8029713, EPI_ISL_8030450, EPI_ISL_8030625, EPI_ISL_8038409, EPI_ISL_8044393, EPI_ISL_8046586, EPI_ISL_8049906, EPI_ISL_8049915, EPI_ISL_8049917, EPI_ISL_8049918, EPI_ISL_8049919, EPI_ISL_8060345, EPI_ISL_8060406, EPI_ISL_8060833, EPI_ISL_8060894, EPI_ISL_8060974, EPI_ISL_8062145, EPI_ISL_8063083, EPI_ISL_8068107, EPI_ISL_8078285, EPI_ISL_8079424, EPI_ISL_8079488, EPI_ISL_8079555, EPI_ISL_8080096, EPI_ISL_8080419, EPI_ISL_8082607, EPI_ISL_8082623, EPI_ISL_8082825, EPI_ISL_8087834, EPI_ISL_8091177, EPI_ISL_8092036, EPI_ISL_8097113, EPI_ISL_8097119, EPI_ISL_8106398, EPI_ISL_8109581, EPI_ISL_8111073, EPI_ISL_8112683, EPI_ISL_8115748, EPI_ISL_8116089, EPI_ISL_8116090, EPI_ISL_8122388, EPI_ISL_8126914, EPI_ISL_8132420, EPI_ISL_8133923, EPI_ISL_8149958, EPI_ISL_8157487, EPI_ISL_8159346, EPI_ISL_8163293, EPI_ISL_8163307, EPI_ISL_8163414, EPI_ISL_8166517, EPI_ISL_8170854, EPI_ISL_8180529, EPI_ISL_8182884, EPI_ISL_8183017, EPI_ISL_8183131, EPI_ISL_8183263, EPI_ISL_8185416, EPI_ISL_8186670, EPI_ISL_8188582, EPI_ISL_8188583, EPI_ISL_8188584, EPI_ISL_8188585, EPI_ISL_8188586, EPI_ISL_8188587, EPI_ISL_8188588, EPI_ISL_8188589, EPI_ISL_8189141, EPI_ISL_8193597, EPI_ISL_8200642, EPI_ISL_8206829, EPI_ISL_8206840, EPI_ISL_8206864, EPI_ISL_8207740, EPI_ISL_8229935, EPI_ISL_8229990, EPI_ISL_8233602, EPI_ISL_8236893, EPI_ISL_8238189, EPI_ISL_8238279, EPI_ISL_8241972, EPI_ISL_8242673, EPI_ISL_8243729, EPI_ISL_8248465, EPI_ISL_8248962, EPI_ISL_8255974, EPI_ISL_8257133, EPI_ISL_8257994, EPI_ISL_8258023, EPI_ISL_8259443, EPI_ISL_8260129, EPI_ISL_8260467, EPI_ISL_8260727, EPI_ISL_8260969, EPI_ISL_8261006, EPI_ISL_8261145, EPI_ISL_8261170, EPI_ISL_8261175, EPI_ISL_8261182, EPI_ISL_8261191, EPI_ISL_8261196, EPI_ISL_8261207, EPI_ISL_8261636, EPI_ISL_8262817, EPI_ISL_8263131, EPI_ISL_8265180, EPI_ISL_8267137, EPI_ISL_8276676, EPI_ISL_8280181, EPI_ISL_8281741, EPI_ISL_8282119, EPI_ISL_8282662, EPI_ISL_8287151, EPI_ISL_8287534, EPI_ISL_8289103, EPI_ISL_8289305, EPI_ISL_8289538, EPI_ISL_8289922, EPI_ISL_8290718, EPI_ISL_8290776, EPI_ISL_8293444, EPI_ISL_8294105, EPI_ISL_8295523, EPI_ISL_8303487, EPI_ISL_8304935, EPI_ISL_8305718, EPI_ISL_8306738, EPI_ISL_8307368, EPI_ISL_8307404, EPI_ISL_8307812, EPI_ISL_8308045, EPI_ISL_8308655, EPI_ISL_8308964, EPI_ISL_8309887, EPI_ISL_8309894, EPI_ISL_8310980, EPI_ISL_8311137, EPI_ISL_8311149, EPI_ISL_8311897, EPI_ISL_8314392, EPI_ISL_8315214, EPI_ISL_8317836, EPI_ISL_8321859, EPI_ISL_8328897, EPI_ISL_8328910, EPI_ISL_8329948, EPI_ISL_8336265, EPI_ISL_8336433, EPI_ISL_8336449, EPI_ISL_8336450, EPI_ISL_8337681, EPI_ISL_8337736, EPI_ISL_8338155, EPI_ISL_8351717, EPI_ISL_8352253, EPI_ISL_8353002, EPI_ISL_8353108, EPI_ISL_8354726, EPI_ISL_8355349, EPI_ISL_8355392, EPI_ISL_8356546, EPI_ISL_8356698, EPI_ISL_8357683, EPI_ISL_8358788, EPI_ISL_8364636, EPI_ISL_8365056, EPI_ISL_8365900, EPI_ISL_8373815, EPI_ISL_8373830, EPI_ISL_8373837, EPI_ISL_8374787, EPI_ISL_8374830, EPI_ISL_8374840, EPI_ISL_8374865, EPI_ISL_8374872, EPI_ISL_8374901, EPI_ISL_8377503, EPI_ISL_8381143, EPI_ISL_8381594, EPI_ISL_8381916, EPI_ISL_8381956, EPI_ISL_8384493, EPI_ISL_8392598, EPI_ISL_8393372, EPI_ISL_8393961, EPI_ISL_8394061, EPI_ISL_8396883, EPI_ISL_8402134, EPI_ISL_8402159, EPI_ISL_8404859, EPI_ISL_8404938, EPI_ISL_8404940, EPI_ISL_8404941, EPI_ISL_8405256, EPI_ISL_8405273, EPI_ISL_8405275, EPI_ISL_8405278, EPI_ISL_8405315, EPI_ISL_8405318, EPI_ISL_8405319, EPI_ISL_8405320, EPI_ISL_8405322, EPI_ISL_8405323, EPI_ISL_8405380, EPI_ISL_8405381, EPI_ISL_8405386, EPI_ISL_8410467, EPI_ISL_8410970, EPI_ISL_8411771, EPI_ISL_8416220, EPI_ISL_8416492, EPI_ISL_8418256, EPI_ISL_8420864, EPI_ISL_8424966, EPI_ISL_8425444, EPI_ISL_8425523, EPI_ISL_8425592, EPI_ISL_8425672, EPI_ISL_8425704, EPI_ISL_8425738, EPI_ISL_8426120, EPI_ISL_8426360, EPI_ISL_8427960, EPI_ISL_8428965, EPI_ISL_8433278, EPI_ISL_8433519, EPI_ISL_8434288, EPI_ISL_8434905, EPI_ISL_8436270, EPI_ISL_8449160, EPI_ISL_8456090, EPI_ISL_8457545, EPI_ISL_8458445, EPI_ISL_8462049, EPI_ISL_8465951, EPI_ISL_8467603, EPI_ISL_8468086, EPI_ISL_8468812, EPI_ISL_8470809, EPI_ISL_8471061, EPI_ISL_8471158, EPI_ISL_8471718, EPI_ISL_8471777, EPI_ISL_8472812, EPI_ISL_8472952, EPI_ISL_8475117, EPI_ISL_8480319, EPI_ISL_8482448, EPI_ISL_8484894, EPI_ISL_8494317, EPI_ISL_8494323, EPI_ISL_8494345, EPI_ISL_8495309, EPI_ISL_8508586, EPI_ISL_8512920, EPI_ISL_8513858, EPI_ISL_8513971, EPI_ISL_8514031, EPI_ISL_8517222, EPI_ISL_8517765, EPI_ISL_8517847, EPI_ISL_8521955, EPI_ISL_8524441, EPI_ISL_8524457, EPI_ISL_8525646, EPI_ISL_8525691, EPI_ISL_8527625, EPI_ISL_8527626, EPI_ISL_8529690, EPI_ISL_8530292, EPI_ISL_8530350, EPI_ISL_8530555, EPI_ISL_8530614, EPI_ISL_8538969, EPI_ISL_8541464, EPI_ISL_8541468, EPI_ISL_8541479, EPI_ISL_8546707, EPI_ISL_8546869, EPI_ISL_8546880, EPI_ISL_8546881, EPI_ISL_8546882, EPI_ISL_8546887, EPI_ISL_8546891, EPI_ISL_8546904, EPI_ISL_8546914, EPI_ISL_8546921, EPI_ISL_8546924, EPI_ISL_8546931, EPI_ISL_8546941, EPI_ISL_8546942, EPI_ISL_8546943, EPI_ISL_8546950, EPI_ISL_8546961, EPI_ISL_8546971, EPI_ISL_8546974, EPI_ISL_8546975, EPI_ISL_8546976, EPI_ISL_8546980, EPI_ISL_8546991, EPI_ISL_8546994, EPI_ISL_8547002, EPI_ISL_8547004, EPI_ISL_8551701, EPI_ISL_8551761, EPI_ISL_8552453, EPI_ISL_8552532, EPI_ISL_8552581, EPI_ISL_8552772, EPI_ISL_8554484, EPI_ISL_8554672, EPI_ISL_8554757, EPI_ISL_8555445, EPI_ISL_8555469, EPI_ISL_8556379, EPI_ISL_8557231, EPI_ISL_8573331, EPI_ISL_8578788, EPI_ISL_8581082, EPI_ISL_8581168, EPI_ISL_8581612, EPI_ISL_8581634, EPI_ISL_8584284, EPI_ISL_8584288, EPI_ISL_8584342, EPI_ISL_8584781, EPI_ISL_8584953, EPI_ISL_8585021, EPI_ISL_8588323, EPI_ISL_8588361, EPI_ISL_8588364, EPI_ISL_8588380, EPI_ISL_8588388, EPI_ISL_8588391, EPI_ISL_8588427, EPI_ISL_8588430, EPI_ISL_8588432, EPI_ISL_8588435, EPI_ISL_8591364, EPI_ISL_8592531, EPI_ISL_8593220, EPI_ISL_8603155, EPI_ISL_8605978, EPI_ISL_8606026, EPI_ISL_8611160, EPI_ISL_8623916, EPI_ISL_8624527, EPI_ISL_8625737, EPI_ISL_8632712, EPI_ISL_8637660, EPI_ISL_8637762, EPI_ISL_8647028, EPI_ISL_8647980, EPI_ISL_8648923, EPI_ISL_8650390, EPI_ISL_8652362, EPI_ISL_8652364, EPI_ISL_8652383, EPI_ISL_8652384, EPI_ISL_8654313, EPI_ISL_8658036, EPI_ISL_8662478, EPI_ISL_8665279, EPI_ISL_8665287, EPI_ISL_8668093, EPI_ISL_8674922, EPI_ISL_8674923, EPI_ISL_8674925, EPI_ISL_8674927, EPI_ISL_8674928, EPI_ISL_8674929, EPI_ISL_8674930, EPI_ISL_8674933, EPI_ISL_8674934, EPI_ISL_8674936, EPI_ISL_8674938, EPI_ISL_8675368, EPI_ISL_8676067, EPI_ISL_8676068, EPI_ISL_8676069, EPI_ISL_8676070, EPI_ISL_8676071, EPI_ISL_8676072, EPI_ISL_8676073, EPI_ISL_8676074, EPI_ISL_8676075, EPI_ISL_8676076, EPI_ISL_8676078, EPI_ISL_8676079, EPI_ISL_8676080, EPI_ISL_8676081, EPI_ISL_8676082, EPI_ISL_8676083, EPI_ISL_8676084, EPI_ISL_8676085, EPI_ISL_8676086, EPI_ISL_8676087, EPI_ISL_8676088, EPI_ISL_8676089, EPI_ISL_8676091, EPI_ISL_8676092, EPI_ISL_8676093, EPI_ISL_8676094, EPI_ISL_8676095, EPI_ISL_8676096, EPI_ISL_8678185, EPI_ISL_8678971, EPI_ISL_8679259, EPI_ISL_8679268, EPI_ISL_8679715, EPI_ISL_8679849, EPI_ISL_8681601, EPI_ISL_8683706, EPI_ISL_8683806, EPI_ISL_8684846, EPI_ISL_8685784, EPI_ISL_8686045, EPI_ISL_8686128, EPI_ISL_8686207, EPI_ISL_8686269, EPI_ISL_8686300, EPI_ISL_8686328, EPI_ISL_8686493, EPI_ISL_8686597, EPI_ISL_8686682, EPI_ISL_8686690, EPI_ISL_8686754, EPI_ISL_8686761, EPI_ISL_8686789, EPI_ISL_8686819, EPI_ISL_8686839, EPI_ISL_8686847, EPI_ISL_8687000, EPI_ISL_8687449, EPI_ISL_8687931, EPI_ISL_8687964, EPI_ISL_8688006, EPI_ISL_8688043, EPI_ISL_8688224, EPI_ISL_8688225, EPI_ISL_8688245, EPI_ISL_8700224, EPI_ISL_8703745, EPI_ISL_8703751, EPI_ISL_8703820, EPI_ISL_8704528, EPI_ISL_8705856, EPI_ISL_8707751, EPI_ISL_8707755, EPI_ISL_8707757, EPI_ISL_8707765, EPI_ISL_8713502, EPI_ISL_8717411, EPI_ISL_8717997, EPI_ISL_8718031, EPI_ISL_8718035, EPI_ISL_8718060, EPI_ISL_8718446, EPI_ISL_8719196, EPI_ISL_8719218, EPI_ISL_8719280, EPI_ISL_8719337, EPI_ISL_8719483, EPI_ISL_8719489, EPI_ISL_8719543, EPI_ISL_8719614, EPI_ISL_8720079, EPI_ISL_8720127, EPI_ISL_8733411, EPI_ISL_8733650, EPI_ISL_8739680, EPI_ISL_8739918, EPI_ISL_8742288, EPI_ISL_8742542, EPI_ISL_8744107, EPI_ISL_8746666, EPI_ISL_8746676, EPI_ISL_8746677, EPI_ISL_8746824, EPI_ISL_8748244, EPI_ISL_8754017, EPI_ISL_8766584, EPI_ISL_8766632, EPI_ISL_8766766, EPI_ISL_8767028, EPI_ISL_8767346, EPI_ISL_8768280, EPI_ISL_8769246, EPI_ISL_8769320, EPI_ISL_8769322, EPI_ISL_8769379, EPI_ISL_8769462, EPI_ISL_8769569, EPI_ISL_8769627, EPI_ISL_8769628, EPI_ISL_8769648, EPI_ISL_8769680, EPI_ISL_8769788, EPI_ISL_8769790, EPI_ISL_8769824, EPI_ISL_8769847, EPI_ISL_8769857, EPI_ISL_8769918, EPI_ISL_8770192, EPI_ISL_8781930, EPI_ISL_8781931, EPI_ISL_8784470, EPI_ISL_8784572, EPI_ISL_8790835, EPI_ISL_8790931, EPI_ISL_8792919, EPI_ISL_8799640, EPI_ISL_8800910, EPI_ISL_8800997, EPI_ISL_8801344, EPI_ISL_8801345, EPI_ISL_8802474, EPI_ISL_8802508, EPI_ISL_8804682, EPI_ISL_8809124, EPI_ISL_8809149, EPI_ISL_8809538, EPI_ISL_8809542, EPI_ISL_8809554, EPI_ISL_8809566, EPI_ISL_8809593, EPI_ISL_8809600, EPI_ISL_8809623, EPI_ISL_8809629, EPI_ISL_8809822, EPI_ISL_8809828, EPI_ISL_8809841, EPI_ISL_8809843, EPI_ISL_8809845, EPI_ISL_8809848, EPI_ISL_8809852, EPI_ISL_8809853, EPI_ISL_8809855, EPI_ISL_8809857, EPI_ISL_8809858, EPI_ISL_8809859, EPI_ISL_8809861, EPI_ISL_8809862, EPI_ISL_8809864, EPI_ISL_8809897, EPI_ISL_8809906, EPI_ISL_8809914, EPI_ISL_8809918, EPI_ISL_8809920, EPI_ISL_8809921, EPI_ISL_8809922, EPI_ISL_8809925, EPI_ISL_8809929, EPI_ISL_8809933, EPI_ISL_8809939, EPI_ISL_8809941, EPI_ISL_8809973, EPI_ISL_8809974, EPI_ISL_8809978, EPI_ISL_8809989, EPI_ISL_8809995, EPI_ISL_8814315, EPI_ISL_8814384, EPI_ISL_8814429, EPI_ISL_8814445, EPI_ISL_8814732, EPI_ISL_8815416, EPI_ISL_8815595, EPI_ISL_8815638, EPI_ISL_8815659, EPI_ISL_8818869, EPI_ISL_8819629, EPI_ISL_8825863, EPI_ISL_8826031, EPI_ISL_8826840, EPI_ISL_8827045, EPI_ISL_8827050, EPI_ISL_8827103, EPI_ISL_8827128, EPI_ISL_8827454, EPI_ISL_8827488, EPI_ISL_8827489, EPI_ISL_8827490, EPI_ISL_8827491, EPI_ISL_8827728, EPI_ISL_8827959, EPI_ISL_8827960, EPI_ISL_8828192, EPI_ISL_8828260, EPI_ISL_8829541, EPI_ISL_8829676, EPI_ISL_8829684, EPI_ISL_8830413, EPI_ISL_8830421, EPI_ISL_8830422, EPI_ISL_8830425, EPI_ISL_8830429, EPI_ISL_8830757, EPI_ISL_8830989, EPI_ISL_8831792, EPI_ISL_8831848, EPI_ISL_8831882, EPI_ISL_8834694, EPI_ISL_8835906, EPI_ISL_8835936, EPI_ISL_8837359, EPI_ISL_8837952, EPI_ISL_8841708, EPI_ISL_8843285, EPI_ISL_8851373, EPI_ISL_8867091, EPI_ISL_8867332, EPI_ISL_8869019, EPI_ISL_8869364, EPI_ISL_8870629, EPI_ISL_8870728, EPI_ISL_8871465, EPI_ISL_8871471, EPI_ISL_8872057, EPI_ISL_8872528, EPI_ISL_8876645, EPI_ISL_8876658, EPI_ISL_8877091, EPI_ISL_8878410, EPI_ISL_8881555, EPI_ISL_8881557, EPI_ISL_8881656, EPI_ISL_8881657, EPI_ISL_8881665, EPI_ISL_8881732, EPI_ISL_8881761, EPI_ISL_8881764, EPI_ISL_8881774, EPI_ISL_8881783, EPI_ISL_8881789, EPI_ISL_8881790, EPI_ISL_8881792, EPI_ISL_8881795, EPI_ISL_8881796, EPI_ISL_8881802, EPI_ISL_8881803, EPI_ISL_8881825, EPI_ISL_8881826, EPI_ISL_8881828, EPI_ISL_8881832, EPI_ISL_8881834, EPI_ISL_8881836, EPI_ISL_8881845, EPI_ISL_8881846, EPI_ISL_8881850, EPI_ISL_8881852, EPI_ISL_8881854, EPI_ISL_8885399, EPI_ISL_8887525, EPI_ISL_8887824, EPI_ISL_8892298, EPI_ISL_8897108, EPI_ISL_8898058, EPI_ISL_8901948, EPI_ISL_8901957, EPI_ISL_8911076, EPI_ISL_8917553, EPI_ISL_8917567, EPI_ISL_8917574, EPI_ISL_8917631, EPI_ISL_8917683, EPI_ISL_8917706, EPI_ISL_8917711, EPI_ISL_8917729, EPI_ISL_8917754, EPI_ISL_8917822, EPI_ISL_8917831, EPI_ISL_8921555, EPI_ISL_8921859, EPI_ISL_8921872, EPI_ISL_8921892, EPI_ISL_8921935, EPI_ISL_8922141, EPI_ISL_8934076, EPI_ISL_8940932, EPI_ISL_8941919, EPI_ISL_8942426, EPI_ISL_8942952, EPI_ISL_8943696, EPI_ISL_8944656, EPI_ISL_8945151, EPI_ISL_8945557, EPI_ISL_8946014, EPI_ISL_8947563, EPI_ISL_8949486, EPI_ISL_8950554, EPI_ISL_8950655, EPI_ISL_8950704, EPI_ISL_8950735, EPI_ISL_8951104, EPI_ISL_8951359, EPI_ISL_8957606, EPI_ISL_8960442, EPI_ISL_8962449, EPI_ISL_8974204, EPI_ISL_8975553, EPI_ISL_8976190, EPI_ISL_8978128, EPI_ISL_8978129, EPI_ISL_8984121, EPI_ISL_8990258, EPI_ISL_8991619")
    EPI_9mil_10mil = stringlist_to_strings_set("EPI_ISL_9005352, EPI_ISL_9005356, EPI_ISL_9009178, EPI_ISL_9010456, EPI_ISL_9011268, EPI_ISL_9011275, EPI_ISL_9011283, EPI_ISL_9011289, EPI_ISL_9011290, EPI_ISL_9011298, EPI_ISL_9011310, EPI_ISL_9012199, EPI_ISL_9012200, EPI_ISL_9012288, EPI_ISL_9012305, EPI_ISL_9013737, EPI_ISL_9025020, EPI_ISL_9028109, EPI_ISL_9029649, EPI_ISL_9029749, EPI_ISL_9030071, EPI_ISL_9031908, EPI_ISL_9034666, EPI_ISL_9034671, EPI_ISL_9034712, EPI_ISL_9034715, EPI_ISL_9035097, EPI_ISL_9036190, EPI_ISL_9036685, EPI_ISL_9036935, EPI_ISL_9037616, EPI_ISL_9037649, EPI_ISL_9038072, EPI_ISL_9040293, EPI_ISL_9041163, EPI_ISL_9044369, EPI_ISL_9044458, EPI_ISL_9044535, EPI_ISL_9044608, EPI_ISL_9045074, EPI_ISL_9045075, EPI_ISL_9045087, EPI_ISL_9045225, EPI_ISL_9046021, EPI_ISL_9046155, EPI_ISL_9046511, EPI_ISL_9046768, EPI_ISL_9047380, EPI_ISL_9047385, EPI_ISL_9047396, EPI_ISL_9047413, EPI_ISL_9047424, EPI_ISL_9047438, EPI_ISL_9047479, EPI_ISL_9047560, EPI_ISL_9047929, EPI_ISL_9048577, EPI_ISL_9048645, EPI_ISL_9048813, EPI_ISL_9048832, EPI_ISL_9049061, EPI_ISL_9049124, EPI_ISL_9051207, EPI_ISL_9053050, EPI_ISL_9053093, EPI_ISL_9058521, EPI_ISL_9061430, EPI_ISL_9063947, EPI_ISL_9064017, EPI_ISL_9064261, EPI_ISL_9065596, EPI_ISL_9066315, EPI_ISL_9068610, EPI_ISL_9074323, EPI_ISL_9074392, EPI_ISL_9074669, EPI_ISL_9075663, EPI_ISL_9076234, EPI_ISL_9078947, EPI_ISL_9080382, EPI_ISL_9086997, EPI_ISL_9087004, EPI_ISL_9087015, EPI_ISL_9087019, EPI_ISL_9090751, EPI_ISL_9090837, EPI_ISL_9090929, EPI_ISL_9090961, EPI_ISL_9090962, EPI_ISL_9090967, EPI_ISL_9090980, EPI_ISL_9090984, EPI_ISL_9090991, EPI_ISL_9090992, EPI_ISL_9090993, EPI_ISL_9090995, EPI_ISL_9091000, EPI_ISL_9091036, EPI_ISL_9091066, EPI_ISL_9091067, EPI_ISL_9091077, EPI_ISL_9091088, EPI_ISL_9091091, EPI_ISL_9091097, EPI_ISL_9091117, EPI_ISL_9091195, EPI_ISL_9091242, EPI_ISL_9092486, EPI_ISL_9092543, EPI_ISL_9093024, EPI_ISL_9093324, EPI_ISL_9100643, EPI_ISL_9100660, EPI_ISL_9100671, EPI_ISL_9100690, EPI_ISL_9103139, EPI_ISL_9104320, EPI_ISL_9108186, EPI_ISL_9109762, EPI_ISL_9109898, EPI_ISL_9110303, EPI_ISL_9120288, EPI_ISL_9121666, EPI_ISL_9121730, EPI_ISL_9126398, EPI_ISL_9129541, EPI_ISL_9131755, EPI_ISL_9133265, EPI_ISL_9133362, EPI_ISL_9133533, EPI_ISL_9133534, EPI_ISL_9133560, EPI_ISL_9133651, EPI_ISL_9133676, EPI_ISL_9133770, EPI_ISL_9133811, EPI_ISL_9133912, EPI_ISL_9134047, EPI_ISL_9134101, EPI_ISL_9134165, EPI_ISL_9134203, EPI_ISL_9134204, EPI_ISL_9134714, EPI_ISL_9135550, EPI_ISL_9135558, EPI_ISL_9135571, EPI_ISL_9139305, EPI_ISL_9139589, EPI_ISL_9139682, EPI_ISL_9139789, EPI_ISL_9139793, EPI_ISL_9139806, EPI_ISL_9139841, EPI_ISL_9139906, EPI_ISL_9139953, EPI_ISL_9146287, EPI_ISL_9147675, EPI_ISL_9151196, EPI_ISL_9151405, EPI_ISL_9151417, EPI_ISL_9151754, EPI_ISL_9152643, EPI_ISL_9156804, EPI_ISL_9157476, EPI_ISL_9163218, EPI_ISL_9166254, EPI_ISL_9168068, EPI_ISL_9169546, EPI_ISL_9169574, EPI_ISL_9169859, EPI_ISL_9170032, EPI_ISL_9172208, EPI_ISL_9176038, EPI_ISL_9180127, EPI_ISL_9182632, EPI_ISL_9182645, EPI_ISL_9182662, EPI_ISL_9187834, EPI_ISL_9187843, EPI_ISL_9187872, EPI_ISL_9187891, EPI_ISL_9191078, EPI_ISL_9191147, EPI_ISL_9191309, EPI_ISL_9196222, EPI_ISL_9196330, EPI_ISL_9196339, EPI_ISL_9196351, EPI_ISL_9196358, EPI_ISL_9196364, EPI_ISL_9196485, EPI_ISL_9196486, EPI_ISL_9196487, EPI_ISL_9196489, EPI_ISL_9196490, EPI_ISL_9196491, EPI_ISL_9196493, EPI_ISL_9196494, EPI_ISL_9196495, EPI_ISL_9196496, EPI_ISL_9196497, EPI_ISL_9196498, EPI_ISL_9196499, EPI_ISL_9196500, EPI_ISL_9196502, EPI_ISL_9196508, EPI_ISL_9197135, EPI_ISL_9197952, EPI_ISL_9198020, EPI_ISL_9198230, EPI_ISL_9198767, EPI_ISL_9199183, EPI_ISL_9200048, EPI_ISL_9200866, EPI_ISL_9202367, EPI_ISL_9202404, EPI_ISL_9202411, EPI_ISL_9202498, EPI_ISL_9202934, EPI_ISL_9202999, EPI_ISL_9204848, EPI_ISL_9205605, EPI_ISL_9206643, EPI_ISL_9210610, EPI_ISL_9210834, EPI_ISL_9210948, EPI_ISL_9211469, EPI_ISL_9214019, EPI_ISL_9214153, EPI_ISL_9217742, EPI_ISL_9218276, EPI_ISL_9227643, EPI_ISL_9227757, EPI_ISL_9229700, EPI_ISL_9229876, EPI_ISL_9230962, EPI_ISL_9232065, EPI_ISL_9232175, EPI_ISL_9232315, EPI_ISL_9232396, EPI_ISL_9232404, EPI_ISL_9232441, EPI_ISL_9232442, EPI_ISL_9233990, EPI_ISL_9237445, EPI_ISL_9244079, EPI_ISL_9244229, EPI_ISL_9244241, EPI_ISL_9244346, EPI_ISL_9244347, EPI_ISL_9244349, EPI_ISL_9244352, EPI_ISL_9244354, EPI_ISL_9244356, EPI_ISL_9244361, EPI_ISL_9244362, EPI_ISL_9244364, EPI_ISL_9244365, EPI_ISL_9244366, EPI_ISL_9244367, EPI_ISL_9244368, EPI_ISL_9244371, EPI_ISL_9244393, EPI_ISL_9244407, EPI_ISL_9244414, EPI_ISL_9244430, EPI_ISL_9244436, EPI_ISL_9244438, EPI_ISL_9244443, EPI_ISL_9257372, EPI_ISL_9263448, EPI_ISL_9268007, EPI_ISL_9272940, EPI_ISL_9273139, EPI_ISL_9275801, EPI_ISL_9287159, EPI_ISL_9301892, EPI_ISL_9303894, EPI_ISL_9320000, EPI_ISL_9320048, EPI_ISL_9320701, EPI_ISL_9322799, EPI_ISL_9323194, EPI_ISL_9324184, EPI_ISL_9324261, EPI_ISL_9324445, EPI_ISL_9325956, EPI_ISL_9329574, EPI_ISL_9329597, EPI_ISL_9329613, EPI_ISL_9329709, EPI_ISL_9329949, EPI_ISL_9332338, EPI_ISL_9340194, EPI_ISL_9346264, EPI_ISL_9357293, EPI_ISL_9361245, EPI_ISL_9366656, EPI_ISL_9366674, EPI_ISL_9366695, EPI_ISL_9366701, EPI_ISL_9366748, EPI_ISL_9369408, EPI_ISL_9369483, EPI_ISL_9369654, EPI_ISL_9369696, EPI_ISL_9369748, EPI_ISL_9369840, EPI_ISL_9374568, EPI_ISL_9375639, EPI_ISL_9375644, EPI_ISL_9393238, EPI_ISL_9393371, EPI_ISL_9393552, EPI_ISL_9393561, EPI_ISL_9393590, EPI_ISL_9393601, EPI_ISL_9393617, EPI_ISL_9393658, EPI_ISL_9397486, EPI_ISL_9397545, EPI_ISL_9397580, EPI_ISL_9397591, EPI_ISL_9402130, EPI_ISL_9402568, EPI_ISL_9402729, EPI_ISL_9402868, EPI_ISL_9405606, EPI_ISL_9406530, EPI_ISL_9406661, EPI_ISL_9406745, EPI_ISL_9407327, EPI_ISL_9408663, EPI_ISL_9408966, EPI_ISL_9408994, EPI_ISL_9409022, EPI_ISL_9409061, EPI_ISL_9409074, EPI_ISL_9409098, EPI_ISL_9409875, EPI_ISL_9409936, EPI_ISL_9410965, EPI_ISL_9412364, EPI_ISL_9416349, EPI_ISL_9416383, EPI_ISL_9416496, EPI_ISL_9416811, EPI_ISL_9417829, EPI_ISL_9417856, EPI_ISL_9418351, EPI_ISL_9429907, EPI_ISL_9429910, EPI_ISL_9429914, EPI_ISL_9429916, EPI_ISL_9429917, EPI_ISL_9429928, EPI_ISL_9429953, EPI_ISL_9430010, EPI_ISL_9430074, EPI_ISL_9430075, EPI_ISL_9430083, EPI_ISL_9430088, EPI_ISL_9430093, EPI_ISL_9430095, EPI_ISL_9430097, EPI_ISL_9430100, EPI_ISL_9430104, EPI_ISL_9430105, EPI_ISL_9430142, EPI_ISL_9430149, EPI_ISL_9430156, EPI_ISL_9430807, EPI_ISL_9433224, EPI_ISL_9436219, EPI_ISL_9436655, EPI_ISL_9436667, EPI_ISL_9436800, EPI_ISL_9436819, EPI_ISL_9439014, EPI_ISL_9443344, EPI_ISL_9445585, EPI_ISL_9445587, EPI_ISL_9448294, EPI_ISL_9448303, EPI_ISL_9450747, EPI_ISL_9451725, EPI_ISL_9454967, EPI_ISL_9455817, EPI_ISL_9455959, EPI_ISL_9456039, EPI_ISL_9456611, EPI_ISL_9457664, EPI_ISL_9457860, EPI_ISL_9458247, EPI_ISL_9459219, EPI_ISL_9460511, EPI_ISL_9465685, EPI_ISL_9466125, EPI_ISL_9466330, EPI_ISL_9466376, EPI_ISL_9466488, EPI_ISL_9466563, EPI_ISL_9466712, EPI_ISL_9466730, EPI_ISL_9466742, EPI_ISL_9466762, EPI_ISL_9466822, EPI_ISL_9466824, EPI_ISL_9466827, EPI_ISL_9467584, EPI_ISL_9468165, EPI_ISL_9471815, EPI_ISL_9471816, EPI_ISL_9471828, EPI_ISL_9471831, EPI_ISL_9471852, EPI_ISL_9471897, EPI_ISL_9471996, EPI_ISL_9471999, EPI_ISL_9472306, EPI_ISL_9472308, EPI_ISL_9472322, EPI_ISL_9472338, EPI_ISL_9472385, EPI_ISL_9472416, EPI_ISL_9472426, EPI_ISL_9472506, EPI_ISL_9472507, EPI_ISL_9472531, EPI_ISL_9472535, EPI_ISL_9475129, EPI_ISL_9487490, EPI_ISL_9487619, EPI_ISL_9487621, EPI_ISL_9487630, EPI_ISL_9487635, EPI_ISL_9487640, EPI_ISL_9487643, EPI_ISL_9487651, EPI_ISL_9487652, EPI_ISL_9487655, EPI_ISL_9487657, EPI_ISL_9487660, EPI_ISL_9487662, EPI_ISL_9487676, EPI_ISL_9487692, EPI_ISL_9487693, EPI_ISL_9487694, EPI_ISL_9487703, EPI_ISL_9487706, EPI_ISL_9487716, EPI_ISL_9487722, EPI_ISL_9487739, EPI_ISL_9487741, EPI_ISL_9487742, EPI_ISL_9487743, EPI_ISL_9487750, EPI_ISL_9487753, EPI_ISL_9487768, EPI_ISL_9487787, EPI_ISL_9487793, EPI_ISL_9487803, EPI_ISL_9487831, EPI_ISL_9487857, EPI_ISL_9487898, EPI_ISL_9487928, EPI_ISL_9487970, EPI_ISL_9487982, EPI_ISL_9487983, EPI_ISL_9488006, EPI_ISL_9488029, EPI_ISL_9488474, EPI_ISL_9488477, EPI_ISL_9488501, EPI_ISL_9488515, EPI_ISL_9488519, EPI_ISL_9488524, EPI_ISL_9488545, EPI_ISL_9488570, EPI_ISL_9488575, EPI_ISL_9488581, EPI_ISL_9488619, EPI_ISL_9488636, EPI_ISL_9488661, EPI_ISL_9488677, EPI_ISL_9488757, EPI_ISL_9488782, EPI_ISL_9488784, EPI_ISL_9488805, EPI_ISL_9488811, EPI_ISL_9488829, EPI_ISL_9488834, EPI_ISL_9489247, EPI_ISL_9489252, EPI_ISL_9497113, EPI_ISL_9504005, EPI_ISL_9504259, EPI_ISL_9510032, EPI_ISL_9510046, EPI_ISL_9511693, EPI_ISL_9511841, EPI_ISL_9511897, EPI_ISL_9511947, EPI_ISL_9511949, EPI_ISL_9511951, EPI_ISL_9512035, EPI_ISL_9512183, EPI_ISL_9512188, EPI_ISL_9512193, EPI_ISL_9512195, EPI_ISL_9512200, EPI_ISL_9512213, EPI_ISL_9512376, EPI_ISL_9519799, EPI_ISL_9523248, EPI_ISL_9532527, EPI_ISL_9533075, EPI_ISL_9533563, EPI_ISL_9533609, EPI_ISL_9549192, EPI_ISL_9549456, EPI_ISL_9561760, EPI_ISL_9569515, EPI_ISL_9569535, EPI_ISL_9572644, EPI_ISL_9573034, EPI_ISL_9573172, EPI_ISL_9587585, EPI_ISL_9587875, EPI_ISL_9589057, EPI_ISL_9594728, EPI_ISL_9594734, EPI_ISL_9594736, EPI_ISL_9594746, EPI_ISL_9594748, EPI_ISL_9594790, EPI_ISL_9594793, EPI_ISL_9594813, EPI_ISL_9594816, EPI_ISL_9594822, EPI_ISL_9594823, EPI_ISL_9594825, EPI_ISL_9596778, EPI_ISL_9596984, EPI_ISL_9601628, EPI_ISL_9601700, EPI_ISL_9601744, EPI_ISL_9604897, EPI_ISL_9609009, EPI_ISL_9609069, EPI_ISL_9610381, EPI_ISL_9610390, EPI_ISL_9610393, EPI_ISL_9610402, EPI_ISL_9613674, EPI_ISL_9614658, EPI_ISL_9617758, EPI_ISL_9620216, EPI_ISL_9623090, EPI_ISL_9625693, EPI_ISL_9625727, EPI_ISL_9625751, EPI_ISL_9625759, EPI_ISL_9625770, EPI_ISL_9625891, EPI_ISL_9636300, EPI_ISL_9637481, EPI_ISL_9640020, EPI_ISL_9640740, EPI_ISL_9642609, EPI_ISL_9647293, EPI_ISL_9650799, EPI_ISL_9651096, EPI_ISL_9651353, EPI_ISL_9653335, EPI_ISL_9656370, EPI_ISL_9666874, EPI_ISL_9668614, EPI_ISL_9673566, EPI_ISL_9674935, EPI_ISL_9678134, EPI_ISL_9678135, EPI_ISL_9679129, EPI_ISL_9679144, EPI_ISL_9679158, EPI_ISL_9679269, EPI_ISL_9679403, EPI_ISL_9683434, EPI_ISL_9683447, EPI_ISL_9683466, EPI_ISL_9683467, EPI_ISL_9683478, EPI_ISL_9683483, EPI_ISL_9683486, EPI_ISL_9683487, EPI_ISL_9689797, EPI_ISL_9691942, EPI_ISL_9695242, EPI_ISL_9695386, EPI_ISL_9696667, EPI_ISL_9696682, EPI_ISL_9696732, EPI_ISL_9696744, EPI_ISL_9696756, EPI_ISL_9696794, EPI_ISL_9696809, EPI_ISL_9697283, EPI_ISL_9697342, EPI_ISL_9697401, EPI_ISL_9697409, EPI_ISL_9697412, EPI_ISL_9697419, EPI_ISL_9697420, EPI_ISL_9697622, EPI_ISL_9697669, EPI_ISL_9697674, EPI_ISL_9697678, EPI_ISL_9697695, EPI_ISL_9697740, EPI_ISL_9697742, EPI_ISL_9697750, EPI_ISL_9697760, EPI_ISL_9697779, EPI_ISL_9697780, EPI_ISL_9697786, EPI_ISL_9697797, EPI_ISL_9697801, EPI_ISL_9697802, EPI_ISL_9697803, EPI_ISL_9697810, EPI_ISL_9697813, EPI_ISL_9697815, EPI_ISL_9697816, EPI_ISL_9702292, EPI_ISL_9705038, EPI_ISL_9705152, EPI_ISL_9707770, EPI_ISL_9709081, EPI_ISL_9709115, EPI_ISL_9709126, EPI_ISL_9709128, EPI_ISL_9709139, EPI_ISL_9713712, EPI_ISL_9714143, EPI_ISL_9714174, EPI_ISL_9715503, EPI_ISL_9715636, EPI_ISL_9715645, EPI_ISL_9721311, EPI_ISL_9721351, EPI_ISL_9721359, EPI_ISL_9721365, EPI_ISL_9721370, EPI_ISL_9721377, EPI_ISL_9721381, EPI_ISL_9721390, EPI_ISL_9727117, EPI_ISL_9727142, EPI_ISL_9727150, EPI_ISL_9727169, EPI_ISL_9727186, EPI_ISL_9727194, EPI_ISL_9727223, EPI_ISL_9727268, EPI_ISL_9728278, EPI_ISL_9729033, EPI_ISL_9729063, EPI_ISL_9729127, EPI_ISL_9729128, EPI_ISL_9736232, EPI_ISL_9743775, EPI_ISL_9746931, EPI_ISL_9754259, EPI_ISL_9754610, EPI_ISL_9770596, EPI_ISL_9770856, EPI_ISL_9770870, EPI_ISL_9771473, EPI_ISL_9773064, EPI_ISL_9775562, EPI_ISL_9781644, EPI_ISL_9781649, EPI_ISL_9781675, EPI_ISL_9781690, EPI_ISL_9783712, EPI_ISL_9788022, EPI_ISL_9791141, EPI_ISL_9792573, EPI_ISL_9806761, EPI_ISL_9806777, EPI_ISL_9806780, EPI_ISL_9806795, EPI_ISL_9806825, EPI_ISL_9806831, EPI_ISL_9806849, EPI_ISL_9806890, EPI_ISL_9806897, EPI_ISL_9806909, EPI_ISL_9806941, EPI_ISL_9806943, EPI_ISL_9806949, EPI_ISL_9806960, EPI_ISL_9806977, EPI_ISL_9806985, EPI_ISL_9806992, EPI_ISL_9806995, EPI_ISL_9808141, EPI_ISL_9808146, EPI_ISL_9808159, EPI_ISL_9808311, EPI_ISL_9808318, EPI_ISL_9813896, EPI_ISL_9814061, EPI_ISL_9817504, EPI_ISL_9818106, EPI_ISL_9818145, EPI_ISL_9831433, EPI_ISL_9831863, EPI_ISL_9832011, EPI_ISL_9832049, EPI_ISL_9832223, EPI_ISL_9832265, EPI_ISL_9832266, EPI_ISL_9833929, EPI_ISL_9835767, EPI_ISL_9835963, EPI_ISL_9837460, EPI_ISL_9837718, EPI_ISL_9837719, EPI_ISL_9843631, EPI_ISL_9844747, EPI_ISL_9844816, EPI_ISL_9844820, EPI_ISL_9849476, EPI_ISL_9849487, EPI_ISL_9849501, EPI_ISL_9854654, EPI_ISL_9854923, EPI_ISL_9863925, EPI_ISL_9873050, EPI_ISL_9873051, EPI_ISL_9873829, EPI_ISL_9873872, EPI_ISL_9873933, EPI_ISL_9873964, EPI_ISL_9877792, EPI_ISL_9880735, EPI_ISL_9882111, EPI_ISL_9884555, EPI_ISL_9886029, EPI_ISL_9886100, EPI_ISL_9911424, EPI_ISL_9911578, EPI_ISL_9919496, EPI_ISL_9925948, EPI_ISL_9925959, EPI_ISL_9925965, EPI_ISL_9925989, EPI_ISL_9925993, EPI_ISL_9925995, EPI_ISL_9925997, EPI_ISL_9925999, EPI_ISL_9926001, EPI_ISL_9926003, EPI_ISL_9926004, EPI_ISL_9926007, EPI_ISL_9926010, EPI_ISL_9926017, EPI_ISL_9926019, EPI_ISL_9926022, EPI_ISL_9926024, EPI_ISL_9926030, EPI_ISL_9926058, EPI_ISL_9926078, EPI_ISL_9926099, EPI_ISL_9926112, EPI_ISL_9926118, EPI_ISL_9926127, EPI_ISL_9926154, EPI_ISL_9926162, EPI_ISL_9926327, EPI_ISL_9926334, EPI_ISL_9940202, EPI_ISL_9961706, EPI_ISL_9961927, EPI_ISL_9963072, EPI_ISL_9966882, EPI_ISL_9970992, EPI_ISL_9979109, EPI_ISL_9983756, EPI_ISL_9983883, EPI_ISL_9987524, EPI_ISL_9994180")
    EPI_10mil_11mil = stringlist_to_strings_set("EPI_ISL_10005959, EPI_ISL_10006826, EPI_ISL_10008204, EPI_ISL_10008459, EPI_ISL_10008460, EPI_ISL_10008465, EPI_ISL_10009329, EPI_ISL_10011197, EPI_ISL_10013260, EPI_ISL_10015927, EPI_ISL_10021110, EPI_ISL_10022743, EPI_ISL_10023162, EPI_ISL_10023287, EPI_ISL_10023339, EPI_ISL_10025493, EPI_ISL_10026752, EPI_ISL_10026977, EPI_ISL_10027022, EPI_ISL_10027035, EPI_ISL_10027100, EPI_ISL_10027276, EPI_ISL_10028336, EPI_ISL_10029029, EPI_ISL_10034133, EPI_ISL_10034481, EPI_ISL_10034593, EPI_ISL_10034637, EPI_ISL_10045301, EPI_ISL_10058780, EPI_ISL_10058783, EPI_ISL_10063882, EPI_ISL_10065608, EPI_ISL_10065768, EPI_ISL_10065974, EPI_ISL_10065986, EPI_ISL_10065987, EPI_ISL_10065994, EPI_ISL_10066000, EPI_ISL_10066001, EPI_ISL_10066002, EPI_ISL_10066010, EPI_ISL_10066015, EPI_ISL_10066022, EPI_ISL_10066023, EPI_ISL_10066029, EPI_ISL_10066034, EPI_ISL_10066036, EPI_ISL_10066048, EPI_ISL_10066050, EPI_ISL_10066056, EPI_ISL_10066057, EPI_ISL_10066058, EPI_ISL_10066059, EPI_ISL_10066061, EPI_ISL_10066062, EPI_ISL_10066068, EPI_ISL_10066070, EPI_ISL_10066073, EPI_ISL_10066084, EPI_ISL_10066090, EPI_ISL_10066091, EPI_ISL_10066093, EPI_ISL_10066102, EPI_ISL_10066130, EPI_ISL_10066134, EPI_ISL_10066137, EPI_ISL_10066150, EPI_ISL_10066153, EPI_ISL_10066164, EPI_ISL_10066168, EPI_ISL_10066186, EPI_ISL_10066213, EPI_ISL_10066243, EPI_ISL_10066246, EPI_ISL_10066254, EPI_ISL_10066267, EPI_ISL_10072285, EPI_ISL_10072674, EPI_ISL_10072698, EPI_ISL_10073148, EPI_ISL_10074087, EPI_ISL_10074702, EPI_ISL_10074707, EPI_ISL_10077456, EPI_ISL_10077457, EPI_ISL_10077740, EPI_ISL_10077789, EPI_ISL_10077805, EPI_ISL_10077939, EPI_ISL_10077958, EPI_ISL_10077960, EPI_ISL_10077963, EPI_ISL_10077965, EPI_ISL_10077966, EPI_ISL_10077980, EPI_ISL_10077987, EPI_ISL_10078149, EPI_ISL_10078155, EPI_ISL_10078205, EPI_ISL_10078216, EPI_ISL_10078218, EPI_ISL_10078365, EPI_ISL_10078399, EPI_ISL_10078400, EPI_ISL_10078405, EPI_ISL_10078406, EPI_ISL_10078749, EPI_ISL_10078881, EPI_ISL_10078882, EPI_ISL_10078977, EPI_ISL_10079008, EPI_ISL_10079124, EPI_ISL_10079482, EPI_ISL_10079485, EPI_ISL_10079730, EPI_ISL_10080088, EPI_ISL_10080659, EPI_ISL_10081351, EPI_ISL_10081394, EPI_ISL_10081402, EPI_ISL_10081662, EPI_ISL_10081729, EPI_ISL_10081881, EPI_ISL_10081886, EPI_ISL_10082052, EPI_ISL_10082630, EPI_ISL_10082989, EPI_ISL_10083756, EPI_ISL_10083777, EPI_ISL_10083787, EPI_ISL_10083812, EPI_ISL_10083866, EPI_ISL_10083944, EPI_ISL_10083960, EPI_ISL_10083972, EPI_ISL_10083979, EPI_ISL_10083981, EPI_ISL_10083985, EPI_ISL_10083988, EPI_ISL_10084004, EPI_ISL_10084007, EPI_ISL_10084008, EPI_ISL_10084093, EPI_ISL_10085377, EPI_ISL_10087734, EPI_ISL_10087752, EPI_ISL_10092537, EPI_ISL_10093299, EPI_ISL_10093694, EPI_ISL_10101118, EPI_ISL_10101120, EPI_ISL_10101124, EPI_ISL_10121115, EPI_ISL_10121312, EPI_ISL_10121522, EPI_ISL_10123102, EPI_ISL_10123125, EPI_ISL_10123126, EPI_ISL_10123127, EPI_ISL_10123136, EPI_ISL_10123138, EPI_ISL_10123144, EPI_ISL_10123149, EPI_ISL_10123151, EPI_ISL_10123184, EPI_ISL_10123237, EPI_ISL_10123242, EPI_ISL_10123246, EPI_ISL_10123256, EPI_ISL_10123278, EPI_ISL_10129960, EPI_ISL_10138346, EPI_ISL_10138347, EPI_ISL_10138351, EPI_ISL_10138509, EPI_ISL_10149308, EPI_ISL_10149344, EPI_ISL_10149570, EPI_ISL_10173852, EPI_ISL_10174207, EPI_ISL_10174960, EPI_ISL_10175390, EPI_ISL_10182087, EPI_ISL_10182644, EPI_ISL_10182902, EPI_ISL_10184975, EPI_ISL_10185121, EPI_ISL_10185137, EPI_ISL_10185347, EPI_ISL_10188771, EPI_ISL_10188772, EPI_ISL_10188801, EPI_ISL_10188808, EPI_ISL_10188819, EPI_ISL_10188829, EPI_ISL_10192722, EPI_ISL_10192735, EPI_ISL_10198777, EPI_ISL_10204977, EPI_ISL_10204983, EPI_ISL_10204986, EPI_ISL_10204992, EPI_ISL_10205004, EPI_ISL_10205039, EPI_ISL_10205073, EPI_ISL_10205094, EPI_ISL_10205098, EPI_ISL_10205106, EPI_ISL_10205142, EPI_ISL_10205178, EPI_ISL_10205180, EPI_ISL_10205182, EPI_ISL_10205253, EPI_ISL_10205289, EPI_ISL_10205338, EPI_ISL_10205375, EPI_ISL_10207392, EPI_ISL_10207536, EPI_ISL_10207539, EPI_ISL_10207777, EPI_ISL_10210308, EPI_ISL_10210437, EPI_ISL_10210486, EPI_ISL_10212487, EPI_ISL_10212488, EPI_ISL_10213774, EPI_ISL_10213775, EPI_ISL_10214094, EPI_ISL_10214108, EPI_ISL_10214427, EPI_ISL_10214452, EPI_ISL_10214463, EPI_ISL_10220260, EPI_ISL_10224838, EPI_ISL_10246896, EPI_ISL_10246913, EPI_ISL_10246929, EPI_ISL_10247077, EPI_ISL_10247741, EPI_ISL_10248161, EPI_ISL_10251088, EPI_ISL_10251304, EPI_ISL_10255340, EPI_ISL_10255376, EPI_ISL_10283352, EPI_ISL_10283711, EPI_ISL_10283724, EPI_ISL_10290459, EPI_ISL_10293291, EPI_ISL_10293513, EPI_ISL_10295769, EPI_ISL_10296275, EPI_ISL_10298437, EPI_ISL_10298438, EPI_ISL_10298464, EPI_ISL_10298491, EPI_ISL_10298494, EPI_ISL_10300795, EPI_ISL_10306024, EPI_ISL_10306120, EPI_ISL_10306134, EPI_ISL_10306140, EPI_ISL_10306148, EPI_ISL_10306161, EPI_ISL_10306164, EPI_ISL_10306559, EPI_ISL_10306869, EPI_ISL_10309965, EPI_ISL_10311702, EPI_ISL_10311711, EPI_ISL_10311753, EPI_ISL_10312030, EPI_ISL_10312031, EPI_ISL_10312032, EPI_ISL_10312033, EPI_ISL_10312042, EPI_ISL_10312044, EPI_ISL_10312052, EPI_ISL_10312056, EPI_ISL_10312057, EPI_ISL_10312059, EPI_ISL_10312060, EPI_ISL_10312064, EPI_ISL_10312065, EPI_ISL_10312066, EPI_ISL_10312081, EPI_ISL_10312085, EPI_ISL_10312086, EPI_ISL_10312087, EPI_ISL_10312094, EPI_ISL_10312096, EPI_ISL_10312370, EPI_ISL_10312525, EPI_ISL_10312548, EPI_ISL_10312553, EPI_ISL_10312557, EPI_ISL_10312559, EPI_ISL_10312565, EPI_ISL_10312568, EPI_ISL_10312573, EPI_ISL_10312582, EPI_ISL_10312584, EPI_ISL_10312585, EPI_ISL_10312592, EPI_ISL_10312596, EPI_ISL_10312597, EPI_ISL_10312598, EPI_ISL_10312615, EPI_ISL_10312617, EPI_ISL_10312618, EPI_ISL_10312623, EPI_ISL_10312624, EPI_ISL_10312626, EPI_ISL_10312627, EPI_ISL_10312635, EPI_ISL_10312638, EPI_ISL_10312754, EPI_ISL_10313149, EPI_ISL_10313150, EPI_ISL_10313183, EPI_ISL_10313845, EPI_ISL_10314883, EPI_ISL_10314886, EPI_ISL_10314887, EPI_ISL_10314899, EPI_ISL_10314904, EPI_ISL_10314914, EPI_ISL_10314915, EPI_ISL_10314916, EPI_ISL_10314924, EPI_ISL_10314930, EPI_ISL_10314931, EPI_ISL_10314940, EPI_ISL_10314941, EPI_ISL_10314942, EPI_ISL_10314943, EPI_ISL_10314944, EPI_ISL_10316983, EPI_ISL_10317379, EPI_ISL_10327362, EPI_ISL_10327716, EPI_ISL_10328262, EPI_ISL_10328772, EPI_ISL_10369251, EPI_ISL_10379783, EPI_ISL_10382204, EPI_ISL_10386226, EPI_ISL_10391525, EPI_ISL_10392765, EPI_ISL_10398645, EPI_ISL_10401810, EPI_ISL_10401837, EPI_ISL_10401856, EPI_ISL_10401865, EPI_ISL_10401868, EPI_ISL_10401984, EPI_ISL_10404067, EPI_ISL_10404103, EPI_ISL_10404131, EPI_ISL_10405495, EPI_ISL_10431884, EPI_ISL_10431934, EPI_ISL_10431989, EPI_ISL_10433657, EPI_ISL_10435634, EPI_ISL_10437641, EPI_ISL_10441356, EPI_ISL_10443386, EPI_ISL_10443404, EPI_ISL_10443409, EPI_ISL_10443411, EPI_ISL_10443425, EPI_ISL_10443430, EPI_ISL_10443433, EPI_ISL_10443434, EPI_ISL_10443436, EPI_ISL_10443439, EPI_ISL_10443440, EPI_ISL_10443441, EPI_ISL_10443444, EPI_ISL_10443446, EPI_ISL_10443447, EPI_ISL_10443448, EPI_ISL_10443450, EPI_ISL_10443452, EPI_ISL_10443454, EPI_ISL_10443456, EPI_ISL_10443458, EPI_ISL_10443460, EPI_ISL_10443461, EPI_ISL_10443466, EPI_ISL_10443469, EPI_ISL_10443471, EPI_ISL_10443490, EPI_ISL_10443494, EPI_ISL_10443495, EPI_ISL_10443503, EPI_ISL_10443505, EPI_ISL_10443506, EPI_ISL_10443507, EPI_ISL_10443510, EPI_ISL_10443511, EPI_ISL_10443512, EPI_ISL_10443516, EPI_ISL_10443517, EPI_ISL_10443518, EPI_ISL_10443519, EPI_ISL_10443520, EPI_ISL_10443523, EPI_ISL_10443525, EPI_ISL_10443526, EPI_ISL_10443533, EPI_ISL_10443535, EPI_ISL_10443563, EPI_ISL_10443572, EPI_ISL_10443573, EPI_ISL_10443575, EPI_ISL_10443577, EPI_ISL_10443584, EPI_ISL_10443592, EPI_ISL_10443595, EPI_ISL_10443599, EPI_ISL_10443601, EPI_ISL_10443603, EPI_ISL_10443605, EPI_ISL_10443606, EPI_ISL_10443608, EPI_ISL_10443611, EPI_ISL_10443613, EPI_ISL_10443615, EPI_ISL_10443617, EPI_ISL_10443623, EPI_ISL_10443627, EPI_ISL_10443632, EPI_ISL_10443636, EPI_ISL_10443647, EPI_ISL_10443648, EPI_ISL_10443649, EPI_ISL_10443656, EPI_ISL_10443658, EPI_ISL_10443661, EPI_ISL_10443671, EPI_ISL_10443674, EPI_ISL_10443675, EPI_ISL_10443678, EPI_ISL_10450198, EPI_ISL_10451247, EPI_ISL_10455028, EPI_ISL_10457041, EPI_ISL_10481797, EPI_ISL_10482336, EPI_ISL_10482850, EPI_ISL_10497475, EPI_ISL_10502293, EPI_ISL_10504027, EPI_ISL_10504064, EPI_ISL_10504107, EPI_ISL_10504108, EPI_ISL_10504167, EPI_ISL_10504187, EPI_ISL_10504203, EPI_ISL_10504205, EPI_ISL_10504239, EPI_ISL_10504258, EPI_ISL_10504269, EPI_ISL_10504319, EPI_ISL_10504339, EPI_ISL_10504345, EPI_ISL_10504347, EPI_ISL_10504381, EPI_ISL_10504387, EPI_ISL_10504420, EPI_ISL_10504441, EPI_ISL_10504471, EPI_ISL_10504485, EPI_ISL_10504647, EPI_ISL_10514956, EPI_ISL_10519096, EPI_ISL_10523937, EPI_ISL_10524685, EPI_ISL_10524790, EPI_ISL_10524808, EPI_ISL_10543999, EPI_ISL_10544042, EPI_ISL_10544102, EPI_ISL_10544144, EPI_ISL_10544474, EPI_ISL_10544507, EPI_ISL_10544984, EPI_ISL_10544986, EPI_ISL_10545423, EPI_ISL_10545452, EPI_ISL_10546178, EPI_ISL_10546476, EPI_ISL_10547155, EPI_ISL_10547219, EPI_ISL_10547244, EPI_ISL_10547258, EPI_ISL_10547266, EPI_ISL_10547268, EPI_ISL_10547296, EPI_ISL_10547299, EPI_ISL_10547314, EPI_ISL_10547344, EPI_ISL_10547683, EPI_ISL_10548055, EPI_ISL_10548071, EPI_ISL_10548077, EPI_ISL_10548443, EPI_ISL_10548461, EPI_ISL_10549286, EPI_ISL_10555394, EPI_ISL_10555425, EPI_ISL_10555459, EPI_ISL_10555540, EPI_ISL_10555586, EPI_ISL_10555615, EPI_ISL_10555636, EPI_ISL_10555668, EPI_ISL_10559721, EPI_ISL_10560171, EPI_ISL_10565822, EPI_ISL_10567103, EPI_ISL_10568889, EPI_ISL_10571268, EPI_ISL_10579550, EPI_ISL_10579638, EPI_ISL_10579782, EPI_ISL_10580591, EPI_ISL_10580610, EPI_ISL_10580791, EPI_ISL_10580798, EPI_ISL_10590704, EPI_ISL_10594394, EPI_ISL_10596188, EPI_ISL_10596244, EPI_ISL_10623440, EPI_ISL_10630173, EPI_ISL_10630208, EPI_ISL_10630286, EPI_ISL_10630294, EPI_ISL_10630387, EPI_ISL_10630426, EPI_ISL_10630509, EPI_ISL_10631057, EPI_ISL_10631087, EPI_ISL_10631441, EPI_ISL_10632161, EPI_ISL_10633951, EPI_ISL_10634024, EPI_ISL_10638400, EPI_ISL_10638438, EPI_ISL_10638461, EPI_ISL_10638464, EPI_ISL_10638466, EPI_ISL_10638483, EPI_ISL_10638490, EPI_ISL_10638493, EPI_ISL_10638507, EPI_ISL_10638513, EPI_ISL_10638522, EPI_ISL_10638528, EPI_ISL_10638529, EPI_ISL_10638534, EPI_ISL_10638544, EPI_ISL_10638548, EPI_ISL_10638569, EPI_ISL_10638681, EPI_ISL_10638682, EPI_ISL_10638684, EPI_ISL_10638687, EPI_ISL_10639921, EPI_ISL_10640459, EPI_ISL_10645365, EPI_ISL_10646183, EPI_ISL_10646223, EPI_ISL_10646224, EPI_ISL_10646625, EPI_ISL_10646659, EPI_ISL_10646660, EPI_ISL_10646664, EPI_ISL_10646678, EPI_ISL_10652475, EPI_ISL_10661114, EPI_ISL_10661119, EPI_ISL_10661136, EPI_ISL_10667120, EPI_ISL_10673551, EPI_ISL_10676563, EPI_ISL_10678674, EPI_ISL_10682237, EPI_ISL_10682241, EPI_ISL_10682298, EPI_ISL_10682367, EPI_ISL_10682374, EPI_ISL_10699409, EPI_ISL_10703587, EPI_ISL_10703923, EPI_ISL_10707346, EPI_ISL_10714913, EPI_ISL_10714914, EPI_ISL_10715650, EPI_ISL_10716749, EPI_ISL_10716750, EPI_ISL_10716751, EPI_ISL_10716752, EPI_ISL_10716753, EPI_ISL_10716754, EPI_ISL_10716755, EPI_ISL_10716756, EPI_ISL_10716757, EPI_ISL_10716758, EPI_ISL_10716759, EPI_ISL_10716760, EPI_ISL_10716761, EPI_ISL_10716762, EPI_ISL_10716763, EPI_ISL_10716764, EPI_ISL_10716765, EPI_ISL_10716766, EPI_ISL_10716767, EPI_ISL_10716768, EPI_ISL_10716769, EPI_ISL_10716770, EPI_ISL_10716771, EPI_ISL_10716772, EPI_ISL_10716773, EPI_ISL_10716774, EPI_ISL_10716775, EPI_ISL_10716776, EPI_ISL_10716777, EPI_ISL_10716778, EPI_ISL_10716779, EPI_ISL_10716780, EPI_ISL_10716781, EPI_ISL_10716782, EPI_ISL_10716783, EPI_ISL_10716784, EPI_ISL_10716785, EPI_ISL_10716786, EPI_ISL_10716787, EPI_ISL_10716788, EPI_ISL_10716789, EPI_ISL_10716790, EPI_ISL_10716791, EPI_ISL_10716793, EPI_ISL_10716794, EPI_ISL_10716795, EPI_ISL_10716796, EPI_ISL_10716797, EPI_ISL_10716798, EPI_ISL_10716799, EPI_ISL_10716800, EPI_ISL_10716801, EPI_ISL_10716802, EPI_ISL_10716803, EPI_ISL_10716804, EPI_ISL_10716805, EPI_ISL_10716806, EPI_ISL_10716807, EPI_ISL_10716809, EPI_ISL_10716810, EPI_ISL_10716811, EPI_ISL_10716812, EPI_ISL_10716813, EPI_ISL_10716814, EPI_ISL_10716816, EPI_ISL_10720898, EPI_ISL_10720899, EPI_ISL_10720917, EPI_ISL_10720921, EPI_ISL_10720922, EPI_ISL_10720936, EPI_ISL_10720939, EPI_ISL_10720942, EPI_ISL_10720945, EPI_ISL_10720959, EPI_ISL_10720964, EPI_ISL_10720967, EPI_ISL_10720972, EPI_ISL_10720980, EPI_ISL_10720994, EPI_ISL_10721000, EPI_ISL_10721008, EPI_ISL_10721019, EPI_ISL_10721027, EPI_ISL_10721028, EPI_ISL_10721030, EPI_ISL_10721034, EPI_ISL_10721045, EPI_ISL_10721046, EPI_ISL_10721063, EPI_ISL_10721065, EPI_ISL_10721071, EPI_ISL_10721081, EPI_ISL_10721083, EPI_ISL_10721087, EPI_ISL_10721089, EPI_ISL_10721092, EPI_ISL_10721094, EPI_ISL_10721096, EPI_ISL_10721097, EPI_ISL_10721098, EPI_ISL_10721101, EPI_ISL_10721107, EPI_ISL_10721109, EPI_ISL_10721112, EPI_ISL_10721115, EPI_ISL_10721130, EPI_ISL_10721131, EPI_ISL_10721136, EPI_ISL_10721137, EPI_ISL_10721139, EPI_ISL_10721144, EPI_ISL_10721149, EPI_ISL_10721152, EPI_ISL_10721155, EPI_ISL_10721157, EPI_ISL_10721163, EPI_ISL_10721178, EPI_ISL_10721181, EPI_ISL_10721192, EPI_ISL_10721199, EPI_ISL_10721211, EPI_ISL_10721221, EPI_ISL_10721236, EPI_ISL_10721237, EPI_ISL_10721238, EPI_ISL_10724195, EPI_ISL_10729907, EPI_ISL_10729913, EPI_ISL_10729914, EPI_ISL_10729923, EPI_ISL_10729942, EPI_ISL_10729960, EPI_ISL_10729962, EPI_ISL_10729987, EPI_ISL_10730596, EPI_ISL_10737360, EPI_ISL_10737642, EPI_ISL_10737702, EPI_ISL_10737703, EPI_ISL_10737959, EPI_ISL_10738130, EPI_ISL_10738152, EPI_ISL_10739357, EPI_ISL_10743603, EPI_ISL_10744853, EPI_ISL_10745444, EPI_ISL_10745973, EPI_ISL_10746047, EPI_ISL_10746077, EPI_ISL_10746272, EPI_ISL_10746292, EPI_ISL_10746326, EPI_ISL_10746390, EPI_ISL_10746411, EPI_ISL_10746428, EPI_ISL_10746508, EPI_ISL_10746552, EPI_ISL_10746590, EPI_ISL_10746695, EPI_ISL_10746698, EPI_ISL_10747159, EPI_ISL_10747334, EPI_ISL_10748103, EPI_ISL_10748266, EPI_ISL_10748279, EPI_ISL_10748300, EPI_ISL_10748460, EPI_ISL_10748490, EPI_ISL_10748497, EPI_ISL_10748499, EPI_ISL_10748731, EPI_ISL_10748742, EPI_ISL_10748765, EPI_ISL_10748789, EPI_ISL_10748796, EPI_ISL_10749120, EPI_ISL_10749363, EPI_ISL_10751290, EPI_ISL_10752622, EPI_ISL_10752779, EPI_ISL_10753373, EPI_ISL_10761887, EPI_ISL_10765065, EPI_ISL_10766003, EPI_ISL_10767929, EPI_ISL_10774009, EPI_ISL_10778001, EPI_ISL_10786839, EPI_ISL_10787136, EPI_ISL_10792746, EPI_ISL_10794919, EPI_ISL_10794943, EPI_ISL_10795137, EPI_ISL_10804609, EPI_ISL_10808697, EPI_ISL_10816282, EPI_ISL_10818640, EPI_ISL_10824995, EPI_ISL_10825050, EPI_ISL_10825072, EPI_ISL_10825080, EPI_ISL_10827078, EPI_ISL_10827098, EPI_ISL_10827110, EPI_ISL_10827811, EPI_ISL_10828375, EPI_ISL_10830097, EPI_ISL_10832248, EPI_ISL_10832490, EPI_ISL_10832873, EPI_ISL_10834277, EPI_ISL_10834286, EPI_ISL_10834301, EPI_ISL_10834303, EPI_ISL_10834339, EPI_ISL_10835529, EPI_ISL_10835646, EPI_ISL_10836497, EPI_ISL_10839829, EPI_ISL_10846979, EPI_ISL_10846993, EPI_ISL_10847003, EPI_ISL_10847047, EPI_ISL_10856645, EPI_ISL_10858431, EPI_ISL_10858434, EPI_ISL_10858504, EPI_ISL_10858534, EPI_ISL_10858576, EPI_ISL_10859516, EPI_ISL_10863825, EPI_ISL_10863916, EPI_ISL_10864352, EPI_ISL_10864402, EPI_ISL_10864691, EPI_ISL_10869496, EPI_ISL_10875769, EPI_ISL_10876749, EPI_ISL_10885204, EPI_ISL_10889622, EPI_ISL_10889699, EPI_ISL_10894052, EPI_ISL_10899907, EPI_ISL_10901492, EPI_ISL_10901542, EPI_ISL_10901650, EPI_ISL_10902039, EPI_ISL_10902076, EPI_ISL_10903411, EPI_ISL_10905381, EPI_ISL_10905390, EPI_ISL_10906700, EPI_ISL_10907939, EPI_ISL_10908621, EPI_ISL_10909171, EPI_ISL_10910544, EPI_ISL_10910885, EPI_ISL_10911406, EPI_ISL_10911449, EPI_ISL_10916131, EPI_ISL_10916325, EPI_ISL_10916342, EPI_ISL_10916454, EPI_ISL_10916811, EPI_ISL_10916964, EPI_ISL_10917724, EPI_ISL_10918548, EPI_ISL_10918663, EPI_ISL_10918757, EPI_ISL_10918877, EPI_ISL_10918897, EPI_ISL_10919073, EPI_ISL_10919348, EPI_ISL_10919390, EPI_ISL_10919417, EPI_ISL_10920188, EPI_ISL_10920207, EPI_ISL_10920295, EPI_ISL_10920318, EPI_ISL_10920323, EPI_ISL_10920387, EPI_ISL_10920420, EPI_ISL_10920485, EPI_ISL_10929576, EPI_ISL_10929737, EPI_ISL_10931583, EPI_ISL_10940696, EPI_ISL_10944316, EPI_ISL_10944320, EPI_ISL_10944473, EPI_ISL_10944567, EPI_ISL_10944816, EPI_ISL_10945552, EPI_ISL_10952144, EPI_ISL_10952168, EPI_ISL_10952171, EPI_ISL_10953120, EPI_ISL_10958600, EPI_ISL_10958671, EPI_ISL_10958693, EPI_ISL_10958704, EPI_ISL_10958730, EPI_ISL_10969481, EPI_ISL_10981425, EPI_ISL_10984859, EPI_ISL_10985594, EPI_ISL_10985626, EPI_ISL_10989445, EPI_ISL_10989720, EPI_ISL_10989838, EPI_ISL_10989839, EPI_ISL_10993276, EPI_ISL_10993277, EPI_ISL_10993342, EPI_ISL_10993606, EPI_ISL_10993614, EPI_ISL_10993840, EPI_ISL_10996854, EPI_ISL_10996875, EPI_ISL_10996887, EPI_ISL_10996895, EPI_ISL_10997448, EPI_ISL_10999171")
    EPI_11mil_12mil = stringlist_to_strings_set("EPI_ISL_11522614, EPI_ISL_11001378, EPI_ISL_11001397, EPI_ISL_11001406, EPI_ISL_11001410, EPI_ISL_11001422, EPI_ISL_11001431, EPI_ISL_11001508, EPI_ISL_11001616, EPI_ISL_11001834, EPI_ISL_11001864, EPI_ISL_11001898, EPI_ISL_11001909, EPI_ISL_11001910, EPI_ISL_11001940, EPI_ISL_11002001, EPI_ISL_11002017, EPI_ISL_11002051, EPI_ISL_11002068, EPI_ISL_11007865, EPI_ISL_11011841, EPI_ISL_11011842, EPI_ISL_11011899, EPI_ISL_11011902, EPI_ISL_11011912, EPI_ISL_11011916, EPI_ISL_11011921, EPI_ISL_11011925, EPI_ISL_11011943, EPI_ISL_11011947, EPI_ISL_11011948, EPI_ISL_11011954, EPI_ISL_11011956, EPI_ISL_11011959, EPI_ISL_11011977, EPI_ISL_11011979, EPI_ISL_11011981, EPI_ISL_11011985, EPI_ISL_11012065, EPI_ISL_11012067, EPI_ISL_11012071, EPI_ISL_11012083, EPI_ISL_11012095, EPI_ISL_11012096, EPI_ISL_11012105, EPI_ISL_11012146, EPI_ISL_11012162, EPI_ISL_11012215, EPI_ISL_11012226, EPI_ISL_11012236, EPI_ISL_11012240, EPI_ISL_11012245, EPI_ISL_11012261, EPI_ISL_11012315, EPI_ISL_11012413, EPI_ISL_11012456, EPI_ISL_11012472, EPI_ISL_11012512, EPI_ISL_11012531, EPI_ISL_11012543, EPI_ISL_11012621, EPI_ISL_11012652, EPI_ISL_11012696, EPI_ISL_11012965, EPI_ISL_11015460, EPI_ISL_11015499, EPI_ISL_11016046, EPI_ISL_11022678, EPI_ISL_11023750, EPI_ISL_11023776, EPI_ISL_11023778, EPI_ISL_11027868, EPI_ISL_11035832, EPI_ISL_11036960, EPI_ISL_11036979, EPI_ISL_11037012, EPI_ISL_11037077, EPI_ISL_11057571, EPI_ISL_11057573, EPI_ISL_11057577, EPI_ISL_11057578, EPI_ISL_11057581, EPI_ISL_11057586, EPI_ISL_11057587, EPI_ISL_11057588, EPI_ISL_11057589, EPI_ISL_11057593, EPI_ISL_11057599, EPI_ISL_11057601, EPI_ISL_11057616, EPI_ISL_11057681, EPI_ISL_11057697, EPI_ISL_11057712, EPI_ISL_11057720, EPI_ISL_11057741, EPI_ISL_11057775, EPI_ISL_11057776, EPI_ISL_11057791, EPI_ISL_11057819, EPI_ISL_11057998, EPI_ISL_11062632, EPI_ISL_11063127, EPI_ISL_11063642, EPI_ISL_11063656, EPI_ISL_11064020, EPI_ISL_11068208, EPI_ISL_11069952, EPI_ISL_11070263, EPI_ISL_11072733, EPI_ISL_11072760, EPI_ISL_11072840, EPI_ISL_11073708, EPI_ISL_11074533, EPI_ISL_11074547, EPI_ISL_11074949, EPI_ISL_11075203, EPI_ISL_11075204, EPI_ISL_11075208, EPI_ISL_11075232, EPI_ISL_11095421, EPI_ISL_11101743, EPI_ISL_11102862, EPI_ISL_11102868, EPI_ISL_11102875, EPI_ISL_11102889, EPI_ISL_11102890, EPI_ISL_11102891, EPI_ISL_11102900, EPI_ISL_11102924, EPI_ISL_11102931, EPI_ISL_11102937, EPI_ISL_11102940, EPI_ISL_11102941, EPI_ISL_11102942, EPI_ISL_11102950, EPI_ISL_11102956, EPI_ISL_11102957, EPI_ISL_11103042, EPI_ISL_11103086, EPI_ISL_11104664, EPI_ISL_11104679, EPI_ISL_11104750, EPI_ISL_11104787, EPI_ISL_11104817, EPI_ISL_11104830, EPI_ISL_11104878, EPI_ISL_11104879, EPI_ISL_11104936, EPI_ISL_11104946, EPI_ISL_11104949, EPI_ISL_11104969, EPI_ISL_11105281, EPI_ISL_11105304, EPI_ISL_11105376, EPI_ISL_11105377, EPI_ISL_11105589, EPI_ISL_11105593, EPI_ISL_11105661, EPI_ISL_11105665, EPI_ISL_11106012, EPI_ISL_11106554, EPI_ISL_11106847, EPI_ISL_11108553, EPI_ISL_11108569, EPI_ISL_11108594, EPI_ISL_11108623, EPI_ISL_11108654, EPI_ISL_11108683, EPI_ISL_11108722, EPI_ISL_11108748, EPI_ISL_11108767, EPI_ISL_11108772, EPI_ISL_11108774, EPI_ISL_11108803, EPI_ISL_11108815, EPI_ISL_11108817, EPI_ISL_11133720, EPI_ISL_11133721, EPI_ISL_11133722, EPI_ISL_11134084, EPI_ISL_11134695, EPI_ISL_11141677, EPI_ISL_11141682, EPI_ISL_11144326, EPI_ISL_11145334, EPI_ISL_11148047, EPI_ISL_11168875, EPI_ISL_11168895, EPI_ISL_11168902, EPI_ISL_11168912, EPI_ISL_11168920, EPI_ISL_11168923, EPI_ISL_11168926, EPI_ISL_11168929, EPI_ISL_11168930, EPI_ISL_11168932, EPI_ISL_11168988, EPI_ISL_11168992, EPI_ISL_11168995, EPI_ISL_11169013, EPI_ISL_11169014, EPI_ISL_11169015, EPI_ISL_11169017, EPI_ISL_11169022, EPI_ISL_11169028, EPI_ISL_11173013, EPI_ISL_11173017, EPI_ISL_11173018, EPI_ISL_11173041, EPI_ISL_11173049, EPI_ISL_11173060, EPI_ISL_11173065, EPI_ISL_11173069, EPI_ISL_11173070, EPI_ISL_11173076, EPI_ISL_11173079, EPI_ISL_11173084, EPI_ISL_11173088, EPI_ISL_11173353, EPI_ISL_11173361, EPI_ISL_11173417, EPI_ISL_11179918, EPI_ISL_11179924, EPI_ISL_11209814, EPI_ISL_11220075, EPI_ISL_11220597, EPI_ISL_11222931, EPI_ISL_11225546, EPI_ISL_11225824, EPI_ISL_11228601, EPI_ISL_11228602, EPI_ISL_11228630, EPI_ISL_11228665, EPI_ISL_11228678, EPI_ISL_11228700, EPI_ISL_11235456, EPI_ISL_11235471, EPI_ISL_11235483, EPI_ISL_11235496, EPI_ISL_11235499, EPI_ISL_11235563, EPI_ISL_11235578, EPI_ISL_11235580, EPI_ISL_11235581, EPI_ISL_11235583, EPI_ISL_11235627, EPI_ISL_11235843, EPI_ISL_11235856, EPI_ISL_11235861, EPI_ISL_11235862, EPI_ISL_11235863, EPI_ISL_11235865, EPI_ISL_11235879, EPI_ISL_11235890, EPI_ISL_11235919, EPI_ISL_11235925, EPI_ISL_11235929, EPI_ISL_11235937, EPI_ISL_11236108, EPI_ISL_11236172, EPI_ISL_11236449, EPI_ISL_11237359, EPI_ISL_11246565, EPI_ISL_11260036, EPI_ISL_11260040, EPI_ISL_11260049, EPI_ISL_11260136, EPI_ISL_11260148, EPI_ISL_11260154, EPI_ISL_11262602, EPI_ISL_11268266, EPI_ISL_11268277, EPI_ISL_11268631, EPI_ISL_11270606, EPI_ISL_11271349, EPI_ISL_11278381, EPI_ISL_11278382, EPI_ISL_11284205, EPI_ISL_11291795, EPI_ISL_11299954, EPI_ISL_11299975, EPI_ISL_11311005, EPI_ISL_11323423, EPI_ISL_11323501, EPI_ISL_11323538, EPI_ISL_11323548, EPI_ISL_11323552, EPI_ISL_11323584, EPI_ISL_11323592, EPI_ISL_11323651, EPI_ISL_11323672, EPI_ISL_11323696, EPI_ISL_11323720, EPI_ISL_11325780, EPI_ISL_11330485, EPI_ISL_11333623, EPI_ISL_11333761, EPI_ISL_11344101, EPI_ISL_11349130, EPI_ISL_11351998, EPI_ISL_11369287, EPI_ISL_11381121, EPI_ISL_11381144, EPI_ISL_11397865, EPI_ISL_11404058, EPI_ISL_11404085, EPI_ISL_11404186, EPI_ISL_11404235, EPI_ISL_11404242, EPI_ISL_11404262, EPI_ISL_11404274, EPI_ISL_11404733, EPI_ISL_11410657, EPI_ISL_11414660, EPI_ISL_11414941, EPI_ISL_11415188, EPI_ISL_11416082, EPI_ISL_11436707, EPI_ISL_11441807, EPI_ISL_11442637, EPI_ISL_11450574, EPI_ISL_11451045, EPI_ISL_11451135, EPI_ISL_11451317, EPI_ISL_11451340, EPI_ISL_11451721, EPI_ISL_11451747, EPI_ISL_11452037, EPI_ISL_11452179, EPI_ISL_11452204, EPI_ISL_11454385, EPI_ISL_11454553, EPI_ISL_11454667, EPI_ISL_11454932, EPI_ISL_11455246, EPI_ISL_11466767, EPI_ISL_11468033, EPI_ISL_11468040, EPI_ISL_11468560, EPI_ISL_11468703, EPI_ISL_11468704, EPI_ISL_11468712, EPI_ISL_11468998, EPI_ISL_11469089, EPI_ISL_11471268, EPI_ISL_11471391, EPI_ISL_11471530, EPI_ISL_11471595, EPI_ISL_11472799, EPI_ISL_11473010, EPI_ISL_11473294, EPI_ISL_11473344, EPI_ISL_11474040, EPI_ISL_11474069, EPI_ISL_11474248, EPI_ISL_11474345, EPI_ISL_11474631, EPI_ISL_11474689, EPI_ISL_11477582, EPI_ISL_11477660, EPI_ISL_11478575, EPI_ISL_11480313, EPI_ISL_11483474, EPI_ISL_11483944, EPI_ISL_11483969, EPI_ISL_11486118, EPI_ISL_11486134, EPI_ISL_11486916, EPI_ISL_11487008, EPI_ISL_11490008, EPI_ISL_11494181, EPI_ISL_11495508, EPI_ISL_11503117, EPI_ISL_11505283, EPI_ISL_11505390, EPI_ISL_11507500, EPI_ISL_11507601, EPI_ISL_11519651, EPI_ISL_11520066, EPI_ISL_11521151, EPI_ISL_11521416, EPI_ISL_11521422, EPI_ISL_11521427, EPI_ISL_11521431, EPI_ISL_11521432, EPI_ISL_11521468, EPI_ISL_11521537, EPI_ISL_11521679, EPI_ISL_11522492, EPI_ISL_11522623, EPI_ISL_11522626, EPI_ISL_11525794, EPI_ISL_11525892, EPI_ISL_11525987, EPI_ISL_11526041, EPI_ISL_11532512, EPI_ISL_11533032, EPI_ISL_11533037, EPI_ISL_11533038, EPI_ISL_11533039, EPI_ISL_11533042, EPI_ISL_11533048, EPI_ISL_11533055, EPI_ISL_11533059, EPI_ISL_11533060, EPI_ISL_11533069, EPI_ISL_11533082, EPI_ISL_11533083, EPI_ISL_11533084, EPI_ISL_11533107, EPI_ISL_11533112, EPI_ISL_11533950, EPI_ISL_11534031, EPI_ISL_11534153, EPI_ISL_11534154, EPI_ISL_11538263, EPI_ISL_11543286, EPI_ISL_11543382, EPI_ISL_11543420, EPI_ISL_11543431, EPI_ISL_11555534, EPI_ISL_11555548, EPI_ISL_11555594, EPI_ISL_11559326, EPI_ISL_11559410, EPI_ISL_11559413, EPI_ISL_11559417, EPI_ISL_11559448, EPI_ISL_11563413, EPI_ISL_11563417, EPI_ISL_11563418, EPI_ISL_11565882, EPI_ISL_11565885, EPI_ISL_11566152, EPI_ISL_11566753, EPI_ISL_11575298, EPI_ISL_11575324, EPI_ISL_11575447, EPI_ISL_11575462, EPI_ISL_11575466, EPI_ISL_11575478, EPI_ISL_11575481, EPI_ISL_11578018, EPI_ISL_11579649, EPI_ISL_11579784, EPI_ISL_11579804, EPI_ISL_11579812, EPI_ISL_11579831, EPI_ISL_11580138, EPI_ISL_11580145, EPI_ISL_11582258, EPI_ISL_11582259, EPI_ISL_11582428, EPI_ISL_11582444, EPI_ISL_11582544, EPI_ISL_11582650, EPI_ISL_11582673, EPI_ISL_11583316, EPI_ISL_11583338, EPI_ISL_11583341, EPI_ISL_11584302, EPI_ISL_11584306, EPI_ISL_11584307, EPI_ISL_11584311, EPI_ISL_11584334, EPI_ISL_11584335, EPI_ISL_11586351, EPI_ISL_11586743, EPI_ISL_11588831, EPI_ISL_11588876, EPI_ISL_11591856, EPI_ISL_11593249, EPI_ISL_11594770, EPI_ISL_11595467, EPI_ISL_11598753, EPI_ISL_11610925, EPI_ISL_11612817, EPI_ISL_11612972, EPI_ISL_11612981, EPI_ISL_11613645, EPI_ISL_11614059, EPI_ISL_11615640, EPI_ISL_11616224, EPI_ISL_11618940, EPI_ISL_11619152, EPI_ISL_11619182, EPI_ISL_11619311, EPI_ISL_11619523, EPI_ISL_11621075, EPI_ISL_11636756, EPI_ISL_11651621, EPI_ISL_11651726, EPI_ISL_11651735, EPI_ISL_11654722, EPI_ISL_11656053, EPI_ISL_11656172, EPI_ISL_11656610, EPI_ISL_11656659, EPI_ISL_11657596, EPI_ISL_11657892, EPI_ISL_11657965, EPI_ISL_11662146, EPI_ISL_11663835, EPI_ISL_11666737, EPI_ISL_11666773, EPI_ISL_11667158, EPI_ISL_11667216, EPI_ISL_11672579, EPI_ISL_11672755, EPI_ISL_11672970, EPI_ISL_11677016, EPI_ISL_11681633, EPI_ISL_11682511, EPI_ISL_11687875, EPI_ISL_11687889, EPI_ISL_11687901, EPI_ISL_11735262, EPI_ISL_11738966, EPI_ISL_11738979, EPI_ISL_11738981, EPI_ISL_11739588, EPI_ISL_11744191, EPI_ISL_11744197, EPI_ISL_11744203, EPI_ISL_11744225, EPI_ISL_11744241, EPI_ISL_11745458, EPI_ISL_11750312, EPI_ISL_11754783, EPI_ISL_11755178, EPI_ISL_11764066, EPI_ISL_11764145, EPI_ISL_11764370, EPI_ISL_11765223, EPI_ISL_11765233, EPI_ISL_11765239, EPI_ISL_11765314, EPI_ISL_11776032, EPI_ISL_11777698, EPI_ISL_11778037, EPI_ISL_11780300, EPI_ISL_11780326, EPI_ISL_11780428, EPI_ISL_11792579, EPI_ISL_11792580, EPI_ISL_11799231, EPI_ISL_11799253, EPI_ISL_11799254, EPI_ISL_11799412, EPI_ISL_11799422, EPI_ISL_11799425, EPI_ISL_11799426, EPI_ISL_11799427, EPI_ISL_11799428, EPI_ISL_11799429, EPI_ISL_11799448, EPI_ISL_11799451, EPI_ISL_11799459, EPI_ISL_11799460, EPI_ISL_11799463, EPI_ISL_11799472, EPI_ISL_11799473, EPI_ISL_11799482, EPI_ISL_11799483, EPI_ISL_11800533, EPI_ISL_11805895, EPI_ISL_11816334, EPI_ISL_11817226, EPI_ISL_11817393, EPI_ISL_11818911, EPI_ISL_11822283, EPI_ISL_11822392, EPI_ISL_11822405, EPI_ISL_11822410, EPI_ISL_11822441, EPI_ISL_11822445, EPI_ISL_11822452, EPI_ISL_11833006, EPI_ISL_11833041, EPI_ISL_11833043, EPI_ISL_11833045, EPI_ISL_11834694, EPI_ISL_11834787, EPI_ISL_11835704, EPI_ISL_11838824, EPI_ISL_11838984, EPI_ISL_11839213, EPI_ISL_11839231, EPI_ISL_11839266, EPI_ISL_11840926, EPI_ISL_11841862, EPI_ISL_11851894, EPI_ISL_11851906, EPI_ISL_11852879, EPI_ISL_11854563, EPI_ISL_11857754, EPI_ISL_11857995, EPI_ISL_11858417, EPI_ISL_11858533, EPI_ISL_11858709, EPI_ISL_11858787, EPI_ISL_11858901, EPI_ISL_11860869, EPI_ISL_11861621, EPI_ISL_11873762, EPI_ISL_11887370, EPI_ISL_11887719, EPI_ISL_11887726, EPI_ISL_11890159, EPI_ISL_11890160, EPI_ISL_11890193, EPI_ISL_11897161, EPI_ISL_11897201, EPI_ISL_11897281, EPI_ISL_11897326, EPI_ISL_11897379, EPI_ISL_11898663, EPI_ISL_11898850, EPI_ISL_11899077, EPI_ISL_11901662, EPI_ISL_11901676, EPI_ISL_11901688, EPI_ISL_11901702, EPI_ISL_11901703, EPI_ISL_11901787, EPI_ISL_11905620, EPI_ISL_11908129, EPI_ISL_11924873, EPI_ISL_11927468, EPI_ISL_11927549, EPI_ISL_11927559, EPI_ISL_11927927, EPI_ISL_11929258, EPI_ISL_11929260, EPI_ISL_11929325, EPI_ISL_11929348, EPI_ISL_11929386, EPI_ISL_11930116, EPI_ISL_11936029, EPI_ISL_11937846, EPI_ISL_11946506, EPI_ISL_11946951, EPI_ISL_11947876, EPI_ISL_11947967, EPI_ISL_11948994, EPI_ISL_11950245, EPI_ISL_11952162, EPI_ISL_11953205, EPI_ISL_11953779, EPI_ISL_11954832, EPI_ISL_11969387, EPI_ISL_11969394, EPI_ISL_11971083, EPI_ISL_11976102, EPI_ISL_11988824, EPI_ISL_11999927")
    EPI_12mil_13mil = stringlist_to_strings("")
    EPI_13mil_14mil = stringlist_to_strings("")
    EPI_16mil_17mil = stringlist_to_strings_set("EPI_ISL_16472666, EPI_ISL_16815237, EPI_ISL_16897661, EPI_ISL_16371526, EPI_ISL_16654387, EPI_ISL_16654388, EPI_ISL_16670567, EPI_ISL_16742612, EPI_ISL_16991534, EPI_ISL_16876388, EPI_ISL_16206521, EPI_ISL_16097562, EPI_ISL_16201324, EPI_ISL_16897651, EPI_ISL_16012710, EPI_ISL_16012943, EPI_ISL_16012954, EPI_ISL_16012987, EPI_ISL_16013209, EPI_ISL_16013227, EPI_ISL_16013267, EPI_ISL_16013309, EPI_ISL_16013449, EPI_ISL_16113255, EPI_ISL_16113336, EPI_ISL_16113399, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16114505, EPI_ISL_16270893, EPI_ISL_16271048, EPI_ISL_16271102, EPI_ISL_16271255, EPI_ISL_16271333, EPI_ISL_16271386, EPI_ISL_16397554, EPI_ISL_16397699, EPI_ISL_16397720, EPI_ISL_16397721, EPI_ISL_16397783, EPI_ISL_16397860, EPI_ISL_16397883, EPI_ISL_16397988, EPI_ISL_16398016, EPI_ISL_16398020, EPI_ISL_16398022, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16542189, EPI_ISL_16542267, EPI_ISL_16542369, EPI_ISL_16542453, EPI_ISL_16542552, EPI_ISL_16542938, EPI_ISL_16599288, EPI_ISL_16599299, EPI_ISL_16599325, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599460, EPI_ISL_16599467, EPI_ISL_16599492, EPI_ISL_16599495, EPI_ISL_16599498, EPI_ISL_16599525, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599554, EPI_ISL_16599636, EPI_ISL_16599649, EPI_ISL_16599747, EPI_ISL_16599794, EPI_ISL_16599826, EPI_ISL_16599842, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599959, EPI_ISL_16599980, EPI_ISL_16761912, EPI_ISL_16761930, EPI_ISL_16761962, EPI_ISL_16761978, EPI_ISL_16762022, EPI_ISL_16762028, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762066, EPI_ISL_16762070, EPI_ISL_16762084, EPI_ISL_16762132, EPI_ISL_16762148, EPI_ISL_16762176, EPI_ISL_16762245, EPI_ISL_16762249, EPI_ISL_16762255, EPI_ISL_16762269, EPI_ISL_16762273, EPI_ISL_16762297, EPI_ISL_16762310, EPI_ISL_16762327, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762455, EPI_ISL_16762458, EPI_ISL_16762462, EPI_ISL_16762475, EPI_ISL_16762559, EPI_ISL_16762600, EPI_ISL_16762611, EPI_ISL_16762614, EPI_ISL_16762631, EPI_ISL_16762652, EPI_ISL_16762653, EPI_ISL_16762654, EPI_ISL_16762699, EPI_ISL_16762700, EPI_ISL_16762714, EPI_ISL_16762726, EPI_ISL_16762735, EPI_ISL_16762753, EPI_ISL_16919296, EPI_ISL_16919303, EPI_ISL_16919306, EPI_ISL_16919315, EPI_ISL_16919337, EPI_ISL_16919348, EPI_ISL_16919362, EPI_ISL_16919365, EPI_ISL_16919367, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919388, EPI_ISL_16919396, EPI_ISL_16919399, EPI_ISL_16919412, EPI_ISL_16919430, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919456, EPI_ISL_16919467, EPI_ISL_16919468, EPI_ISL_16919496, EPI_ISL_16919504, EPI_ISL_16919523, EPI_ISL_16919526, EPI_ISL_16919546, EPI_ISL_16919558, EPI_ISL_16919586, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919640, EPI_ISL_16919660, EPI_ISL_16919662, EPI_ISL_16919670, EPI_ISL_16919680, EPI_ISL_16919695, EPI_ISL_16919706, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919772, EPI_ISL_16919794, EPI_ISL_16919796, EPI_ISL_16919804, EPI_ISL_16919806, EPI_ISL_16919810, EPI_ISL_16919841, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919852, EPI_ISL_16919856, EPI_ISL_16919870, EPI_ISL_16919894, EPI_ISL_16919897, EPI_ISL_16919902, EPI_ISL_16919906, EPI_ISL_16919919, EPI_ISL_16919932, EPI_ISL_16919944, EPI_ISL_16919962, EPI_ISL_16001302, EPI_ISL_16020838, EPI_ISL_16024683, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16088909, EPI_ISL_16089016, EPI_ISL_16092776, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16148354, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16173291, EPI_ISL_16173302, EPI_ISL_16189719, EPI_ISL_16204016, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16248743, EPI_ISL_16260886, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16276826, EPI_ISL_16281961, EPI_ISL_16281966, EPI_ISL_16293327, EPI_ISL_16302637, EPI_ISL_16327676, EPI_ISL_16364514, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16378441, EPI_ISL_16381367, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16382491, EPI_ISL_16390727, EPI_ISL_16402352, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16496661, EPI_ISL_16499231, EPI_ISL_16507911, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16528798, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16570393, EPI_ISL_16574023, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16608560, EPI_ISL_16612080, EPI_ISL_16613416, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668033, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16729840, EPI_ISL_16734294, EPI_ISL_16742171, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760525, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760564, EPI_ISL_16760565, EPI_ISL_16763262, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16825422, EPI_ISL_16835014, EPI_ISL_16845486, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16860526, EPI_ISL_16861003, EPI_ISL_16870757, EPI_ISL_16879371, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16893748, EPI_ISL_16895710, EPI_ISL_16895713, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907588, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16921497, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943850, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16966278, EPI_ISL_16968369, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16020838, EPI_ISL_16041723, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16089016, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16189719, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16212482, EPI_ISL_16226002, EPI_ISL_16248743, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16292066, EPI_ISL_16292121, EPI_ISL_16293327, EPI_ISL_16328978, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16390727, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449525, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16459438, EPI_ISL_16465329, EPI_ISL_16465341, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16513739, EPI_ISL_16527367, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570393, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16607325, EPI_ISL_16612080, EPI_ISL_16612088, EPI_ISL_16613416, EPI_ISL_16638962, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16724327, EPI_ISL_16729840, EPI_ISL_16734284, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760565, EPI_ISL_16760567, EPI_ISL_16765385, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16835014, EPI_ISL_16837809, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16861003, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16895710, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16900417, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943847, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16097754, EPI_ISL_16119602, EPI_ISL_16119610, EPI_ISL_16119629, EPI_ISL_16171387, EPI_ISL_16204016, EPI_ISL_16212482, EPI_ISL_16307563, EPI_ISL_16357154, EPI_ISL_16368857, EPI_ISL_16369646, EPI_ISL_16377998, EPI_ISL_16382423, EPI_ISL_16390790, EPI_ISL_16402352, EPI_ISL_16440038, EPI_ISL_16441289, EPI_ISL_16449517, EPI_ISL_16457740, EPI_ISL_16465341, EPI_ISL_16475710, EPI_ISL_16483723, EPI_ISL_16496661, EPI_ISL_16510324, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16529241, EPI_ISL_16536287, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16574237, EPI_ISL_16606996, EPI_ISL_16607319, EPI_ISL_16607322, EPI_ISL_16607325, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16670793, EPI_ISL_16702339, EPI_ISL_16702350, EPI_ISL_16702360, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702393, EPI_ISL_16702396, EPI_ISL_16733307, EPI_ISL_16734294, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760541, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760562, EPI_ISL_16760565, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16823432, EPI_ISL_16845486, EPI_ISL_16870757, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907559, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907602, EPI_ISL_16915893, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943857, EPI_ISL_16943858, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16993581, EPI_ISL_16073446, EPI_ISL_16861820, EPI_ISL_16510293, EPI_ISL_16815208, EPI_ISL_16457333, EPI_ISL_16937010, EPI_ISL_16536298, EPI_ISL_16536270, EPI_ISL_16536294, EPI_ISL_16536310, EPI_ISL_16536267, EPI_ISL_16913912, EPI_ISL_16170920, EPI_ISL_16201285, EPI_ISL_16201296, EPI_ISL_16201312, EPI_ISL_16201314, EPI_ISL_16201319, EPI_ISL_16201326, EPI_ISL_16201331, EPI_ISL_16201278, EPI_ISL_16201334, EPI_ISL_16940694, EPI_ISL_16861820, EPI_ISL_16012710, EPI_ISL_16012943, EPI_ISL_16012954, EPI_ISL_16012987, EPI_ISL_16013209, EPI_ISL_16013227, EPI_ISL_16013267, EPI_ISL_16013309, EPI_ISL_16013449, EPI_ISL_16113255, EPI_ISL_16113336, EPI_ISL_16113399, EPI_ISL_16113938, EPI_ISL_16114486, EPI_ISL_16114505, EPI_ISL_16270893, EPI_ISL_16271048, EPI_ISL_16271102, EPI_ISL_16271255, EPI_ISL_16271333, EPI_ISL_16271386, EPI_ISL_16397554, EPI_ISL_16397699, EPI_ISL_16397720, EPI_ISL_16397721, EPI_ISL_16397783, EPI_ISL_16397860, EPI_ISL_16397883, EPI_ISL_16397988, EPI_ISL_16398016, EPI_ISL_16398020, EPI_ISL_16398022, EPI_ISL_16398035, EPI_ISL_16398060, EPI_ISL_16398063, EPI_ISL_16398072, EPI_ISL_16454140, EPI_ISL_16454168, EPI_ISL_16542189, EPI_ISL_16542267, EPI_ISL_16542369, EPI_ISL_16542453, EPI_ISL_16542552, EPI_ISL_16542938, EPI_ISL_16599288, EPI_ISL_16599299, EPI_ISL_16599325, EPI_ISL_16599380, EPI_ISL_16599389, EPI_ISL_16599460, EPI_ISL_16599467, EPI_ISL_16599492, EPI_ISL_16599495, EPI_ISL_16599498, EPI_ISL_16599525, EPI_ISL_16599538, EPI_ISL_16599540, EPI_ISL_16599554, EPI_ISL_16599636, EPI_ISL_16599649, EPI_ISL_16599747, EPI_ISL_16599794, EPI_ISL_16599826, EPI_ISL_16599842, EPI_ISL_16599844, EPI_ISL_16599878, EPI_ISL_16599959, EPI_ISL_16599980, EPI_ISL_16761912, EPI_ISL_16761930, EPI_ISL_16761962, EPI_ISL_16761978, EPI_ISL_16762022, EPI_ISL_16762028, EPI_ISL_16762035, EPI_ISL_16762052, EPI_ISL_16762066, EPI_ISL_16762070, EPI_ISL_16762084, EPI_ISL_16762132, EPI_ISL_16762148, EPI_ISL_16762176, EPI_ISL_16762245, EPI_ISL_16762249, EPI_ISL_16762255, EPI_ISL_16762269, EPI_ISL_16762273, EPI_ISL_16762297, EPI_ISL_16762310, EPI_ISL_16762327, EPI_ISL_16762407, EPI_ISL_16762427, EPI_ISL_16762455, EPI_ISL_16762458, EPI_ISL_16762462, EPI_ISL_16762475, EPI_ISL_16762559, EPI_ISL_16762600, EPI_ISL_16762611, EPI_ISL_16762614, EPI_ISL_16762631, EPI_ISL_16762652, EPI_ISL_16762653, EPI_ISL_16762654, EPI_ISL_16762699, EPI_ISL_16762700, EPI_ISL_16762714, EPI_ISL_16762726, EPI_ISL_16762735, EPI_ISL_16762753, EPI_ISL_16919296, EPI_ISL_16919303, EPI_ISL_16919306, EPI_ISL_16919315, EPI_ISL_16919337, EPI_ISL_16919348, EPI_ISL_16919362, EPI_ISL_16919365, EPI_ISL_16919367, EPI_ISL_16919368, EPI_ISL_16919380, EPI_ISL_16919388, EPI_ISL_16919396, EPI_ISL_16919399, EPI_ISL_16919412, EPI_ISL_16919430, EPI_ISL_16919435, EPI_ISL_16919455, EPI_ISL_16919456, EPI_ISL_16919467, EPI_ISL_16919468, EPI_ISL_16919496, EPI_ISL_16919504, EPI_ISL_16919523, EPI_ISL_16919526, EPI_ISL_16919546, EPI_ISL_16919558, EPI_ISL_16919586, EPI_ISL_16919592, EPI_ISL_16919598, EPI_ISL_16919609, EPI_ISL_16919640, EPI_ISL_16919660, EPI_ISL_16919662, EPI_ISL_16919670, EPI_ISL_16919680, EPI_ISL_16919695, EPI_ISL_16919706, EPI_ISL_16919718, EPI_ISL_16919762, EPI_ISL_16919772, EPI_ISL_16919794, EPI_ISL_16919796, EPI_ISL_16919804, EPI_ISL_16919806, EPI_ISL_16919810, EPI_ISL_16919841, EPI_ISL_16919843, EPI_ISL_16919845, EPI_ISL_16919852, EPI_ISL_16919856, EPI_ISL_16919870, EPI_ISL_16919894, EPI_ISL_16919897, EPI_ISL_16919902, EPI_ISL_16919906, EPI_ISL_16919919, EPI_ISL_16919932, EPI_ISL_16919944, EPI_ISL_16919962, EPI_ISL_16001302, EPI_ISL_16020838, EPI_ISL_16024683, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16088909, EPI_ISL_16089016, EPI_ISL_16092776, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16148354, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16173291, EPI_ISL_16173302, EPI_ISL_16189719, EPI_ISL_16204016, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16248743, EPI_ISL_16260886, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16276826, EPI_ISL_16281961, EPI_ISL_16281966, EPI_ISL_16293327, EPI_ISL_16302637, EPI_ISL_16327676, EPI_ISL_16364514, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16378441, EPI_ISL_16381367, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16382491, EPI_ISL_16390727, EPI_ISL_16402352, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16496661, EPI_ISL_16499231, EPI_ISL_16507911, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16528798, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16570393, EPI_ISL_16574023, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16608560, EPI_ISL_16612080, EPI_ISL_16613416, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668033, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16729840, EPI_ISL_16734294, EPI_ISL_16742171, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760525, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760564, EPI_ISL_16760565, EPI_ISL_16763262, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16825422, EPI_ISL_16835014, EPI_ISL_16845486, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16860526, EPI_ISL_16861003, EPI_ISL_16870757, EPI_ISL_16879371, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16893748, EPI_ISL_16895710, EPI_ISL_16895713, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907588, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16921497, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943850, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16966278, EPI_ISL_16968369, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16020838, EPI_ISL_16041723, EPI_ISL_16047006, EPI_ISL_16049706, EPI_ISL_16089016, EPI_ISL_16097810, EPI_ISL_16119629, EPI_ISL_16169012, EPI_ISL_16169559, EPI_ISL_16171387, EPI_ISL_16189719, EPI_ISL_16205980, EPI_ISL_16206468, EPI_ISL_16212482, EPI_ISL_16226002, EPI_ISL_16248743, EPI_ISL_16268007, EPI_ISL_16268596, EPI_ISL_16292066, EPI_ISL_16292121, EPI_ISL_16293327, EPI_ISL_16328978, EPI_ISL_16368859, EPI_ISL_16368863, EPI_ISL_16368870, EPI_ISL_16377998, EPI_ISL_16381431, EPI_ISL_16382369, EPI_ISL_16382423, EPI_ISL_16390727, EPI_ISL_16431012, EPI_ISL_16440289, EPI_ISL_16449525, EPI_ISL_16449531, EPI_ISL_16449569, EPI_ISL_16459438, EPI_ISL_16465329, EPI_ISL_16465341, EPI_ISL_16465346, EPI_ISL_16465349, EPI_ISL_16475710, EPI_ISL_16485170, EPI_ISL_16510336, EPI_ISL_16513678, EPI_ISL_16513739, EPI_ISL_16527367, EPI_ISL_16534033, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570393, EPI_ISL_16574144, EPI_ISL_16574237, EPI_ISL_16598797, EPI_ISL_16607325, EPI_ISL_16612080, EPI_ISL_16612088, EPI_ISL_16613416, EPI_ISL_16638962, EPI_ISL_16649909, EPI_ISL_16649933, EPI_ISL_16655461, EPI_ISL_16655462, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16665774, EPI_ISL_16665862, EPI_ISL_16668112, EPI_ISL_16668536, EPI_ISL_16668786, EPI_ISL_16702345, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702391, EPI_ISL_16724327, EPI_ISL_16729840, EPI_ISL_16734284, EPI_ISL_16742359, EPI_ISL_16742486, EPI_ISL_16742561, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760542, EPI_ISL_16760544, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760565, EPI_ISL_16760567, EPI_ISL_16765385, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16835014, EPI_ISL_16837809, EPI_ISL_16847576, EPI_ISL_16860429, EPI_ISL_16861003, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16887308, EPI_ISL_16895710, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16900417, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907578, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907598, EPI_ISL_16907601, EPI_ISL_16907602, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943847, EPI_ISL_16943855, EPI_ISL_16943857, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16943896, EPI_ISL_16947614, EPI_ISL_16976030, EPI_ISL_16986705, EPI_ISL_16997282, EPI_ISL_16097754, EPI_ISL_16119602, EPI_ISL_16119610, EPI_ISL_16119629, EPI_ISL_16171387, EPI_ISL_16204016, EPI_ISL_16212482, EPI_ISL_16307563, EPI_ISL_16357154, EPI_ISL_16368857, EPI_ISL_16369646, EPI_ISL_16377998, EPI_ISL_16382423, EPI_ISL_16390790, EPI_ISL_16402352, EPI_ISL_16440038, EPI_ISL_16441289, EPI_ISL_16449517, EPI_ISL_16457740, EPI_ISL_16465341, EPI_ISL_16475710, EPI_ISL_16483723, EPI_ISL_16496661, EPI_ISL_16510324, EPI_ISL_16524791, EPI_ISL_16527367, EPI_ISL_16529241, EPI_ISL_16536287, EPI_ISL_16547367, EPI_ISL_16549005, EPI_ISL_16570256, EPI_ISL_16574237, EPI_ISL_16606996, EPI_ISL_16607319, EPI_ISL_16607322, EPI_ISL_16607325, EPI_ISL_16656602, EPI_ISL_16656603, EPI_ISL_16660760, EPI_ISL_16660761, EPI_ISL_16670793, EPI_ISL_16702339, EPI_ISL_16702350, EPI_ISL_16702360, EPI_ISL_16702362, EPI_ISL_16702365, EPI_ISL_16702381, EPI_ISL_16702383, EPI_ISL_16702388, EPI_ISL_16702390, EPI_ISL_16702393, EPI_ISL_16702396, EPI_ISL_16733307, EPI_ISL_16734294, EPI_ISL_16760522, EPI_ISL_16760527, EPI_ISL_16760534, EPI_ISL_16760541, EPI_ISL_16760553, EPI_ISL_16760557, EPI_ISL_16760562, EPI_ISL_16760565, EPI_ISL_16823403, EPI_ISL_16823418, EPI_ISL_16823432, EPI_ISL_16845486, EPI_ISL_16870757, EPI_ISL_16882917, EPI_ISL_16882928, EPI_ISL_16883034, EPI_ISL_16883038, EPI_ISL_16883059, EPI_ISL_16883062, EPI_ISL_16883095, EPI_ISL_16883117, EPI_ISL_16883170, EPI_ISL_16895724, EPI_ISL_16895777, EPI_ISL_16907559, EPI_ISL_16907561, EPI_ISL_16907564, EPI_ISL_16907572, EPI_ISL_16907575, EPI_ISL_16907579, EPI_ISL_16907581, EPI_ISL_16907587, EPI_ISL_16907591, EPI_ISL_16907597, EPI_ISL_16907602, EPI_ISL_16915893, EPI_ISL_16940242, EPI_ISL_16940243, EPI_ISL_16943841, EPI_ISL_16943847, EPI_ISL_16943857, EPI_ISL_16943858, EPI_ISL_16943859, EPI_ISL_16943861, EPI_ISL_16943862, EPI_ISL_16943865, EPI_ISL_16943874, EPI_ISL_16943888, EPI_ISL_16943889, EPI_ISL_16993581")
    Spike_I794N = stringlist_to_strings_set("EPI_ISL_1407032, EPI_ISL_1646281, EPI_ISL_2221068, EPI_ISL_2661550-2661609, EPI_ISL_2681488-2681550, EPI_ISL_2970972, EPI_ISL_2970981, EPI_ISL_2970989, EPI_ISL_2970996, EPI_ISL_2970999, EPI_ISL_2971051, EPI_ISL_2971057, EPI_ISL_2971077, EPI_ISL_2971085-2971086, EPI_ISL_2971089, EPI_ISL_2971099, EPI_ISL_2971123, EPI_ISL_2971141, EPI_ISL_2971157, EPI_ISL_2971165, EPI_ISL_2996654, EPI_ISL_2996657, EPI_ISL_2996702, EPI_ISL_2996712, EPI_ISL_2996720, EPI_ISL_3373334, EPI_ISL_3613280, EPI_ISL_3613461, EPI_ISL_3613545, EPI_ISL_3613599, EPI_ISL_3613648, EPI_ISL_3613711, EPI_ISL_3613807, EPI_ISL_3614113, EPI_ISL_3759977, EPI_ISL_3801323, EPI_ISL_3923183, EPI_ISL_3998693-3998695, EPI_ISL_3998698-3998700, EPI_ISL_3998702, EPI_ISL_3998704-3998705, EPI_ISL_3998708, EPI_ISL_3998710-3998711, EPI_ISL_3998713-3998714, EPI_ISL_3998716, EPI_ISL_3998718-3998720, EPI_ISL_3998722-3998724, EPI_ISL_3998726, EPI_ISL_3998728, EPI_ISL_3998730-3998735, EPI_ISL_3998737-3998741, EPI_ISL_3998743-3998745, EPI_ISL_3998747-3998748, EPI_ISL_3998751-3998752, EPI_ISL_3998754-3998757, EPI_ISL_3998760-3998768, EPI_ISL_3998771-3998772, EPI_ISL_3998775-3998777, EPI_ISL_3998781, EPI_ISL_3998784, EPI_ISL_4470628-4470629, EPI_ISL_4470633, EPI_ISL_4470646-4470647, EPI_ISL_5350573, EPI_ISL_6566104, EPI_ISL_6596059, EPI_ISL_6596063, EPI_ISL_6596067-6596072, EPI_ISL_6596074-6596076, EPI_ISL_6596079-6596080, EPI_ISL_6596082-6596088, EPI_ISL_6596090-6596091, EPI_ISL_6596094, EPI_ISL_6596096, EPI_ISL_6596098, EPI_ISL_6596100-6596101, EPI_ISL_6596103, EPI_ISL_6596106-6596107, EPI_ISL_6596109, EPI_ISL_6596111-6596117, EPI_ISL_6596119, EPI_ISL_6596122-6596124, EPI_ISL_6596127, EPI_ISL_6596129-6596131, EPI_ISL_6596133, EPI_ISL_6596135, EPI_ISL_6596137, EPI_ISL_6601688, EPI_ISL_6601692-6601693, EPI_ISL_6601695-6601698, EPI_ISL_6601701-6601708, EPI_ISL_6601710, EPI_ISL_6601712-6601713, EPI_ISL_6601715-6601716, EPI_ISL_6601718, EPI_ISL_6601720-6601722, EPI_ISL_6601724, EPI_ISL_6601727-6601731, EPI_ISL_6601733, EPI_ISL_6601735-6601740, EPI_ISL_6601742-6601744, EPI_ISL_6601747-6601748, EPI_ISL_6601750-6601759, EPI_ISL_6601762-6601766, EPI_ISL_6601770, EPI_ISL_6601774, EPI_ISL_6601776, EPI_ISL_6601778-6601780, EPI_ISL_6601782-6601783, EPI_ISL_6601785-6601786, EPI_ISL_6601788, EPI_ISL_6601790-6601793, EPI_ISL_6601795-6601797, EPI_ISL_6601799-6601802, EPI_ISL_6601805, EPI_ISL_6601807-6601808, EPI_ISL_6601810-6601822, EPI_ISL_6601824-6601833, EPI_ISL_6601835, EPI_ISL_6601837-6601839, EPI_ISL_6601841-6601842, EPI_ISL_6601844-6601846, EPI_ISL_6601849, EPI_ISL_6601851-6601858, EPI_ISL_6601860, EPI_ISL_6601862-6601866, EPI_ISL_6601872-6601880, EPI_ISL_6601884-6601886, EPI_ISL_6601891, EPI_ISL_6601894, EPI_ISL_6601899-6601906, EPI_ISL_6601908-6601909, EPI_ISL_6601911-6601912, EPI_ISL_6601914-6601922, EPI_ISL_6601924, EPI_ISL_6601926-6601929, EPI_ISL_6601931-6601932, EPI_ISL_6601935-6601936, EPI_ISL_6601938-6601939, EPI_ISL_6601941-6601944, EPI_ISL_6601948-6601949, EPI_ISL_6601951-6601957, EPI_ISL_6601961-6601965, EPI_ISL_6601969-6601979, EPI_ISL_6601981-6601982, EPI_ISL_6601985-6601991, EPI_ISL_6601993-6602003, EPI_ISL_6602005-6602006, EPI_ISL_6602008, EPI_ISL_6602010, EPI_ISL_6602012-6602021, EPI_ISL_6602023-6602028, EPI_ISL_6602032-6602033, EPI_ISL_6602035-6602036, EPI_ISL_6602038-6602039, EPI_ISL_6602041, EPI_ISL_6603775-6603778, EPI_ISL_6603780-6603782, EPI_ISL_6603784-6603786, EPI_ISL_6603790-6603793, EPI_ISL_6603795-6603800, EPI_ISL_6603802-6603809, EPI_ISL_6603813-6603823, EPI_ISL_6603825-6603829, EPI_ISL_6603831-6603833, EPI_ISL_6603835, EPI_ISL_6603837-6603842, EPI_ISL_6603847, EPI_ISL_6603850-6603857, EPI_ISL_6603860-6603862, EPI_ISL_6603864, EPI_ISL_6603866-6603874, EPI_ISL_6603876-6603880, EPI_ISL_6603882-6603884, EPI_ISL_6603886-6603888, EPI_ISL_6603891-6603894, EPI_ISL_6603896-6603898, EPI_ISL_6603901-6603903, EPI_ISL_6603905-6603906, EPI_ISL_6603911-6603925, EPI_ISL_6603927-6603940, EPI_ISL_6603943, EPI_ISL_6603945-6603949, EPI_ISL_6645158, EPI_ISL_6645161, EPI_ISL_6645169, EPI_ISL_6645175, EPI_ISL_6645183, EPI_ISL_6645192, EPI_ISL_6645196, EPI_ISL_6645211, EPI_ISL_6645216, EPI_ISL_6645223, EPI_ISL_6645228, EPI_ISL_6645246-6645247, EPI_ISL_6645252, EPI_ISL_6645260, EPI_ISL_6645266, EPI_ISL_6645279, EPI_ISL_6645291, EPI_ISL_6645307, EPI_ISL_6645326, EPI_ISL_6645338, EPI_ISL_6645344, EPI_ISL_6645360, EPI_ISL_6645371, EPI_ISL_6645381, EPI_ISL_6645396, EPI_ISL_6645403, EPI_ISL_6645409, EPI_ISL_6645419, EPI_ISL_6645434, EPI_ISL_6645441, EPI_ISL_6645462, EPI_ISL_6645471, EPI_ISL_6645487, EPI_ISL_6645499, EPI_ISL_6645504, EPI_ISL_6645513, EPI_ISL_6645547, EPI_ISL_6645554, EPI_ISL_6645560, EPI_ISL_6645570, EPI_ISL_6645574, EPI_ISL_6645590, EPI_ISL_6645600, EPI_ISL_6645603, EPI_ISL_6645613, EPI_ISL_6645625, EPI_ISL_6645635, EPI_ISL_6645645, EPI_ISL_6645653, EPI_ISL_6645664, EPI_ISL_6645673, EPI_ISL_6645678, EPI_ISL_6645685, EPI_ISL_6645695, EPI_ISL_6645700, EPI_ISL_6645705, EPI_ISL_6645714, EPI_ISL_6645742, EPI_ISL_6645749, EPI_ISL_6645769, EPI_ISL_6645778, EPI_ISL_6645785, EPI_ISL_6645801, EPI_ISL_6645814, EPI_ISL_6645821, EPI_ISL_6645827, EPI_ISL_6645836, EPI_ISL_6645845, EPI_ISL_6645852, EPI_ISL_6645861, EPI_ISL_6645868, EPI_ISL_6645875, EPI_ISL_6645881, EPI_ISL_6645885, EPI_ISL_6645891, EPI_ISL_6645896, EPI_ISL_6645909, EPI_ISL_6645916, EPI_ISL_6645924, EPI_ISL_6645932, EPI_ISL_6645937, EPI_ISL_6645943, EPI_ISL_6645950, EPI_ISL_6645971, EPI_ISL_6645984-6645985, EPI_ISL_6645997, EPI_ISL_6646022, EPI_ISL_6646030, EPI_ISL_6646034, EPI_ISL_6646044, EPI_ISL_6646052, EPI_ISL_6646057, EPI_ISL_6646065, EPI_ISL_6646084, EPI_ISL_6646092, EPI_ISL_6646101, EPI_ISL_6646107, EPI_ISL_6646110, EPI_ISL_6646123, EPI_ISL_6646127, EPI_ISL_6646135, EPI_ISL_6646145, EPI_ISL_6646152, EPI_ISL_6646160, EPI_ISL_6646168, EPI_ISL_6646180, EPI_ISL_6646188-6646189, EPI_ISL_6646199, EPI_ISL_6646211, EPI_ISL_6646224-6646225, EPI_ISL_6711328, EPI_ISL_6711340, EPI_ISL_6711344, EPI_ISL_6711349, EPI_ISL_6711357, EPI_ISL_6711362, EPI_ISL_6711380, EPI_ISL_6711386, EPI_ISL_6711392, EPI_ISL_6711400, EPI_ISL_6711422, EPI_ISL_6711433, EPI_ISL_6711448, EPI_ISL_6711452, EPI_ISL_6711458, EPI_ISL_6711460-6711463, EPI_ISL_6711465-6711467, EPI_ISL_6711469-6711471, EPI_ISL_6711473, EPI_ISL_6711476, EPI_ISL_6711478, EPI_ISL_6711482, EPI_ISL_6711484, EPI_ISL_6711486-6711489, EPI_ISL_6711492-6711494, EPI_ISL_6711497-6711498, EPI_ISL_6711500-6711501, EPI_ISL_6711515-6711516, EPI_ISL_6711527, EPI_ISL_6711533, EPI_ISL_6711537, EPI_ISL_6711547, EPI_ISL_6711562, EPI_ISL_6711569, EPI_ISL_6711571, EPI_ISL_6930232-6930233, EPI_ISL_6954331-6954332, EPI_ISL_6954334-6954337, EPI_ISL_6954339-6954341, EPI_ISL_6954344-6954345, EPI_ISL_6954347, EPI_ISL_6954349, EPI_ISL_6954351-6954354, EPI_ISL_6954356-6954362, EPI_ISL_6954367, EPI_ISL_6954370-6954372, EPI_ISL_6954374-6954377, EPI_ISL_6954379-6954381, EPI_ISL_6954383-6954397, EPI_ISL_6954399, EPI_ISL_6954401-6954409, EPI_ISL_6954411-6954414, EPI_ISL_6954416-6954422, EPI_ISL_6954424-6954425, EPI_ISL_6954427-6954429, EPI_ISL_6954431-6954443, EPI_ISL_6954445-6954449, EPI_ISL_6954451-6954455, EPI_ISL_6954458-6954459, EPI_ISL_6954461-6954470, EPI_ISL_6954472-6954492, EPI_ISL_7239290, EPI_ISL_7239302-7239303, EPI_ISL_7239307, EPI_ISL_7239312, EPI_ISL_7239332, EPI_ISL_7239344, EPI_ISL_7239351, EPI_ISL_7239381-7239382, EPI_ISL_7239408, EPI_ISL_7239420-7239421, EPI_ISL_7239427, EPI_ISL_7239446, EPI_ISL_7239455, EPI_ISL_7239463, EPI_ISL_7239472, EPI_ISL_7239489, EPI_ISL_7239498, EPI_ISL_7239506, EPI_ISL_7239513, EPI_ISL_7239522, EPI_ISL_7239529, EPI_ISL_7239572, EPI_ISL_7239578, EPI_ISL_7239596, EPI_ISL_7239605, EPI_ISL_7239618, EPI_ISL_7239628-7239629, EPI_ISL_7239636, EPI_ISL_7239646, EPI_ISL_7239657, EPI_ISL_7239662, EPI_ISL_7239673, EPI_ISL_7239684-7239685, EPI_ISL_7239694, EPI_ISL_7239701, EPI_ISL_7239708, EPI_ISL_7239721, EPI_ISL_7239738, EPI_ISL_7239756, EPI_ISL_7239761-7239762, EPI_ISL_7338768, EPI_ISL_8476117-8476121, EPI_ISL_8476124-8476125, EPI_ISL_8674922-8674940, EPI_ISL_8674971, EPI_ISL_8676018, EPI_ISL_8676067-8676096, EPI_ISL_10678677, EPI_ISL_10834506, EPI_ISL_11096096, EPI_ISL_11173595, EPI_ISL_11323484, EPI_ISL_12293481-12293484, EPI_ISL_12293486-12293532, EPI_ISL_12293534-12293546, EPI_ISL_12293548, EPI_ISL_12293550-12293572, EPI_ISL_12293574-12293575, EPI_ISL_12293577-12293584, EPI_ISL_12293586-12293607, EPI_ISL_12293609-12293622, EPI_ISL_12293624-12293633, EPI_ISL_12367081, EPI_ISL_12623196-12623201, EPI_ISL_12623203-12623204, EPI_ISL_12623206-12623212, EPI_ISL_12623214-12623218, EPI_ISL_12623220-12623227, EPI_ISL_12623229-12623249, EPI_ISL_12623251-12623252, EPI_ISL_12623254-12623255, EPI_ISL_12623257-12623258, EPI_ISL_12623260-12623263, EPI_ISL_12623266-12623268, EPI_ISL_12623270, EPI_ISL_12771908, EPI_ISL_12784697-12784707, EPI_ISL_12784709-12784711, EPI_ISL_12784713-12784742, EPI_ISL_12784744-12784754, EPI_ISL_12784756-12784766, EPI_ISL_12784768-12784839, EPI_ISL_12784841-12784849, EPI_ISL_12784851-12784863, EPI_ISL_12911909-12911910, EPI_ISL_13773508-13773509, EPI_ISL_13773511-13773512, EPI_ISL_13773515, EPI_ISL_13773518, EPI_ISL_13773520-13773521, EPI_ISL_13774111, EPI_ISL_13774114-13774115, EPI_ISL_13774118-13774119, EPI_ISL_13774121, EPI_ISL_13774123, EPI_ISL_13774127-13774132, EPI_ISL_13774135-13774136, EPI_ISL_13774138-13774141, EPI_ISL_13778407, EPI_ISL_13778414, EPI_ISL_13812267, EPI_ISL_14253453, EPI_ISL_14643049, EPI_ISL_14643088-14643093, EPI_ISL_16680115, EPI_ISL_16762665, EPI_ISL_17546505, EPI_ISL_18032876, EPI_ISL_18936208, EPI_ISL_18936222")
    K113N_Q115H = stringlist_to_strings_set("EPI_ISL_16932114, EPI_ISL_16869750, EPI_ISL_16931402, EPI_ISL_16201312, EPI_ISL_16536270, EPI_ISL_16957026, EPI_ISL_16675021, EPI_ISL_16675029, EPI_ISL_16675040, EPI_ISL_16675042, EPI_ISL_16675044, EPI_ISL_16675045, EPI_ISL_16675046, EPI_ISL_16675048, EPI_ISL_16675051, EPI_ISL_16675052, EPI_ISL_16675054, EPI_ISL_16675056, EPI_ISL_16675058, EPI_ISL_16675061, EPI_ISL_16675066, EPI_ISL_16675070, EPI_ISL_16201278, EPI_ISL_16201285, EPI_ISL_16201296, EPI_ISL_16201314, EPI_ISL_16201319, EPI_ISL_16201326, EPI_ISL_16201331, EPI_ISL_16201334, EPI_ISL_16913912, EPI_ISL_16536267, EPI_ISL_16536288, EPI_ISL_16536294, EPI_ISL_16536298, EPI_ISL_16536307, EPI_ISL_16536310, EPI_ISL_16177586, EPI_ISL_16176426, EPI_ISL_16177390, EPI_ISL_16177585, EPI_ISL_16177885, EPI_ISL_16182290, EPI_ISL_16181062, EPI_ISL_16181097, EPI_ISL_16182291, EPI_ISL_16182292, EPI_ISL_16194844, EPI_ISL_16927816, EPI_ISL_16927912, EPI_ISL_16928225, EPI_ISL_16928351, EPI_ISL_16929433, EPI_ISL_16929568, EPI_ISL_16933210, EPI_ISL_16933982, EPI_ISL_16934074, EPI_ISL_16934187, EPI_ISL_16934367, EPI_ISL_16930495, EPI_ISL_16396196, EPI_ISL_16041963, EPI_ISL_16193316, EPI_ISL_16510293, EPI_ISL_16117273, EPI_ISL_16117275, EPI_ISL_16504003, EPI_ISL_16019601, EPI_ISL_16766839, EPI_ISL_16564702, EPI_ISL_16638878, EPI_ISL_16564608, EPI_ISL_16548907, EPI_ISL_16760558, EPI_ISL_16038755, EPI_ISL_16139659, EPI_ISL_16371647, EPI_ISL_16370921, EPI_ISL_16396871, EPI_ISL_16896486, EPI_ISL_16896496, EPI_ISL_16170925, EPI_ISL_16170920, EPI_ISL_16386231, EPI_ISL_16118427, EPI_ISL_16535377, EPI_ISL_16260047, EPI_ISL_16401734, EPI_ISL_16815607, EPI_ISL_16269297, EPI_ISL_16815605, EPI_ISL_16815315, EPI_ISL_16977445, EPI_ISL_672597, EPI_ISL_12293485, EPI_ISL_12293533, EPI_ISL_12293547, EPI_ISL_12293573, EPI_ISL_12293576, EPI_ISL_12293585, EPI_ISL_12293623, EPI_ISL_12623202, EPI_ISL_12623213, EPI_ISL_12623228, EPI_ISL_12623250, EPI_ISL_12623253, EPI_ISL_12623256, EPI_ISL_12623264, EPI_ISL_12623269, EPI_ISL_12784708, EPI_ISL_12784712, EPI_ISL_12784743, EPI_ISL_12784755, EPI_ISL_12784767, EPI_ISL_12955745, EPI_ISL_12955833, EPI_ISL_13135506, EPI_ISL_13612385, EPI_ISL_13774112-13774113, EPI_ISL_13774116-13774117, EPI_ISL_13774120, EPI_ISL_13774122, EPI_ISL_13774124-13774126, EPI_ISL_13774133-13774134, EPI_ISL_13774137, EPI_ISL_13774142-13774143, EPI_ISL_14807247, EPI_ISL_17372592")
    mink = stringlist_to_strings_set("EPI_ISL_16811138, EPI_ISL_16811146, EPI_ISL_16811151, EPI_ISL_16811154, EPI_ISL_16994016, EPI_ISL_16994017, EPI_ISL_16994018, EPI_ISL_16994087")
    combined_lists = union(chr_epi_2025_04_07, AT1, AV1, B1_243_2, B1_616, B1_632, B1_633, B1_638, C12, manual, EPI_8mil_9mil, EPI_9mil_10mil, EPI_10mil_11mil, EPI_11mil_12mil, Spike_I794N, K113N_Q115H, India, Mexico, Indonesia, Thailand, Turkey, Texas, MOV, EPI_16mil_17mil, mink)
    

function chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)
    date = Dates.format(today(), "yyyy-mm-dd")
    excluded_AA = Set(["S:T19I", "S:A67V", "S:V67A", "S:T95I", "S:G142D", "S:D142G", "S:Y145D", "S:L212I", "S:G339D", "S:S371F", "S:S371L", "S:L371F", "S:S373P", "S:S375F", "S:F375S", "S:T376A", "S:A376T", "S:D405N", "S:N405D", "S:R408S", "S:S408R", "S:K417N", "S:N417K", "S:K440N", "S:Q493R", "S:G496S", "S:N679K", "S:I794N", "S:N856K", "S:Q954H", "S:N969K", "S:L981F", "ORF1b:N498I", "ORF1b:F685Y", "S:S27A"])
    epi_key(m) = (length(split(m, "_")[3]), parse(Int, split(m, "_")[3]))
    seq_country = Dict{String, String}()
    seq_US_state = Dict{String, String}()
    seq_clade = Dict{String, String}()
    seq_pango = Dict{String, String}()
    seq_pango_unaliased = Dict{String, String}()
    seq_collection_date = Dict{String, String}()
    seq_date_index = Dict{String, Int}()
    seq_date_tuple = Dict{String, Tuple{Int, Int, Int}}()
    seq_lab_dict = Dict{String, String}()
    seq_nuc_del_ranges_ct = Dict{String, Int}()

    chr_candidates = Vector{String}()
    bad_chr_candidates = Vector{String}()
    bad_country_chr_candidates = Vector{String}()
    #chr_epis_list = chronic_epis("chronics_combined_2024_11_23__3902_seq_v2.ndjson")
    chr_epis_list = combined_lists
    no_date_to_index_key = 0
    for line in eachline(ndjson)
        batcheck = split(line, "/")[2]
        if batcheck ≠ "bat"
            j = JSON3.read(line)
            name = EPI_ISL(j.seqName)
            
            seq_country[name] = country(j.seqName)
            seq_US_state[name] = US_state(j.seqName)
            seq_date = ""
            try
                seq_clade[name] = j.clade
            catch
                seq_clade[name] = "Unknown"
            end
            try
                seq_pango[name] = j.customNodeAttributes.Nextclade_pango
            catch
                seq_pango[name] = "Unknown"
            end
            try
                seq_pango_unaliased[name] = j.customNodeAttributes.partiallyAliased
            catch
                seq_pango_unaliased[name] = "Unknown"
            end
            seq_date = "0-0-0"
            try
                seq_date = sequence_date(j.seqName)
            catch
                seq_date = "0-0-0"
            end
            lab = ""
            if !isempty(seq_lab(j.seqName))
                lab = seq_lab(j.seqName)
            end
            seq_lab_dict[name] = lab
            seq_yr = join(seq_date[1:4])
            if all(isdigit, seq_yr)
                seq_year = parse(Int, seq_yr)
            else
                seq_year = 0
            end
            seq_month = 0
            seq_day = 0
            if length(seq_date) ≥ 7
                seq_mo = join(seq_date[6:7])
                if all(isdigit, seq_mo)
                    seq_month = parse(Int, seq_mo)
                end
            end
            if length(seq_date) ≥ 10
                seq_da = join(seq_date[9:10])
                if all(isdigit, seq_da)
                    seq_day = parse(Int, seq_da)
                end
            end
            seq_collection_date[name] = seq_date
#            date_index = ""
#            if haskey(date_to_index, (seq_year, seq_month, seq_day))
#                date_index = date_to_index[(seq_year, seq_month, seq_day)]
#            else
#                no_date_to_index_key += 1
#            end
#            seq_date_index[name] = date_index
            date_tuple = (seq_year, seq_month, seq_day)
            seq_date_tuple[name] = date_tuple
            chr_muts_ct = 0
            labeled_subs_total = 0
            bad_countries = Set(["Kuwait", "India", "Bangladesh", "Thailand", "Pakistan", "Saudi Arabia"])
            if !(name in combined_lists) && haskey(j, "privateAaMutations")
                if haskey(j.privateAaMutations, "ORF1a") && haskey(j.privateAaMutations, "ORF1b") && haskey(j.privateAaMutations, "S") && haskey(j.privateAaMutations, "ORF3a") && haskey(j.privateAaMutations, "ORF6")  && haskey(j.privateAaMutations, "E")  && haskey(j.privateAaMutations, "M") && haskey(j.privateAaMutations, "ORF7a") && haskey(j.privateAaMutations, "ORF7b") && haskey(j.privateAaMutations, "ORF8") && haskey(j.privateAaMutations, "N") && haskey(j.privateAaMutations, "ORF9b")
                    genes = ["ORF1a", "ORF1b", "ORF3a", "ORF6", "ORF7a", "ORF7b", "ORF8", "ORF9b", "S", "E", "M", "N"]
                    gene_AA_subs = Set([j.privateAaMutations.ORF1a.privateSubstitutions, j.privateAaMutations.ORF1b.privateSubstitutions, j.privateAaMutations.ORF3a.privateSubstitutions, j.privateAaMutations.ORF6.privateSubstitutions, j.privateAaMutations.ORF7a.privateSubstitutions, j.privateAaMutations.ORF7b.privateSubstitutions, j.privateAaMutations.ORF8.privateSubstitutions, j.privateAaMutations.ORF9b.privateSubstitutions, j.privateAaMutations.S.privateSubstitutions, j.privateAaMutations.E.privateSubstitutions, j.privateAaMutations.M.privateSubstitutions, j.privateAaMutations.N.privateSubstitutions])
                    gene_AA_subs_WT = j.aaSubstitutions
                    labeled_subs_total = j.privateNucMutations.totalLabeledSubstitutions
                    subs = Set(j.privateNucMutations.privateSubstitutions)
                    dels = Set(j.privateNucMutations.privateDeletionRanges)
                    nuc_dels = Set(j.privateNucMutations.privateDeletions)
                    AA_deletions = Set([j.privateAaMutations.ORF1a.privateDeletionRanges, j.privateAaMutations.ORF1b.privateDeletionRanges, j.privateAaMutations.ORF3a.privateDeletionRanges, j.privateAaMutations.ORF6.privateDeletionRanges, j.privateAaMutations.ORF7a.privateDeletionRanges, j.privateAaMutations.ORF7b.privateDeletionRanges, j.privateAaMutations.ORF8.privateDeletionRanges, j.privateAaMutations.ORF9b.privateDeletionRanges, j.privateAaMutations.S.privateDeletionRanges, j.privateAaMutations.E.privateDeletionRanges, j.privateAaMutations.M.privateDeletionRanges, j.privateAaMutations.N.privateDeletionRanges])

                    seq_nuc_del_ranges_ct[name] = length(dels)

                    subs_WT = Set(j.substitutions)
                    # WT_dels = Set(j.deletions)
                    nuc_dels_WT = Set(j.deletions)
                    gene_AA_del_WT = j.aaDeletions
                    del_ct = 0
                    if j.privateNucMutations.totalReversionSubstitutions < revs_thresh
                        if j.privateNucMutations.totalPrivateSubstitutions - j.privateNucMutations.totalReversionSubstitutions ≥ minmut && j.privateNucMutations.totalPrivateSubstitutions ≤ maxmut
                            if seq_country[name] ≠ "India" && seq_country[name] ≠ "INDIA" && seq_country[name] ≠ "Pakistan" && seq_country[name] ≠ "Senegal" && seq_country[name] ≠ "Palestine" && seq_country[name] ≠ "Mongolia"
#                                if seq_pango[name] ≠ "B" seq_pango[name] ≠ "B.1" && seq_pango[name] ≠ "B.1.1"
                                    for i in gene_AA_subs
                                        for k in i
                                            pos = k.pos + 1
                                            AA_sub = k.cdsName * ":" * k.refAa * string(pos) * k.qryAa
                                            if !(k.refAa == "-") && !(AA_sub in excluded_AA)
                                                if AA_sub in chronic_search_muts_v2
                                                    chr_muts_ct += 1
                                                end
                                            end
                                        end
                                    end
#                                end
                            end
                        end
                    end
                    if chr_muts_ct ≥ min_chr_mut && !(seq_country in bad_countries)
                        if labeled_subs_total < 9
                            push!(chr_candidates, name)
                        end
                        if labeled_subs_total ≥ 9
                            push!(bad_chr_candidates, name)
                        end
                    end
                    if chr_muts_ct ≥ min_chr_mut && seq_country in bad_countries
                        push!(bad_country_chr_candidates, name)
                    end
                end
            end
        end
    end
    println("no_date_to_index_key count = $(no_date_to_index_key)")
    open("$(date)_$(ndjson_name)_minMut$(minmut)_minChrMut$(min_chr_mut)_$(date).txt", "w") do g
        ln = length(chr_candidates)
        ln_bad = length(bad_chr_candidates)
        println("Number of New Chronic Candidates = $(ln)")
        println(g, "Number of New Chronic Candidates = $(ln)")
        chr_candidates_sort = sort(chr_candidates)
        bad_chr_candidates_sort = sort(bad_chr_candidates)
        bad_country_chr_candidates_sort = sort(bad_country_chr_candidates)
        for seq in chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^15)
        print(g, "\n"^15)
        println("Number of Likely Bad New Chronic Candidates = $(ln_bad)")
        println(g, "Number of Likely Bad New Chronic Candidates = $(ln_bad)")
        for seq in bad_chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^15)
        print(g, "\n"^15)
        println("Number of Bad-Country New Chronic Candidates = $(ln_bad)")
        println(g, "Number of Bad-Country New Chronic Candidates = $(ln_bad)")
        for seq in bad_chr_candidates_sort
            print(g, "$(seq), ")
            print("$(seq), ")
        end
        print("\n"^5)
        print(g, "\n"^5)
    end
end

2025-05-04_1212PM


chronic_search (generic function with 1 method)

In [148]:
date_right_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_right_now)
@time begin
    chronic_search("EPI_ISL_16000001-17000000.ndjson", "16_17mil_chr_candidates", 7, 120, 4, 3)
end
#   chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)
# 786.434928 seconds
# 648.830901 seconds
# 673.509452 seconds
# 870.509452 seconds
# 1165.752907 seconds
#

2025-04-10_716PM
no_date_to_index_key count = 0
Number of New Chronic Candidates = 296
EPI_ISL_16004831, EPI_ISL_16004970, EPI_ISL_16005246, EPI_ISL_16008663, EPI_ISL_16019629, EPI_ISL_16019756, EPI_ISL_16026774, EPI_ISL_16034832, EPI_ISL_16036598, EPI_ISL_16041249, EPI_ISL_16045424, EPI_ISL_16046320, EPI_ISL_16053307, EPI_ISL_16059748, EPI_ISL_16064284, EPI_ISL_16067377, EPI_ISL_16068583, EPI_ISL_16072774, EPI_ISL_16073472, EPI_ISL_16074714, EPI_ISL_16075627, EPI_ISL_16076461, EPI_ISL_16080261, EPI_ISL_16082206, EPI_ISL_16092756, EPI_ISL_16102479, EPI_ISL_16109108, EPI_ISL_16121996, EPI_ISL_16122454, EPI_ISL_16127331, EPI_ISL_16131989, EPI_ISL_16136652, EPI_ISL_16140011, EPI_ISL_16142211, EPI_ISL_16151441, EPI_ISL_16153726, EPI_ISL_16158374, EPI_ISL_16160419, EPI_ISL_16161381, EPI_ISL_16161395, EPI_ISL_16161404, EPI_ISL_16167391, EPI_ISL_16170579, EPI_ISL_16176110, EPI_ISL_16177082, EPI_ISL_16178844, EPI_ISL_16179499, EPI_ISL_16179863, EPI_ISL_16181575, EPI_ISL_16187400, EPI_ISL_16194

In [43]:
@time begin
    chronic_search("EPI_ISL_12000001-13000000.ndjson", "EPI_ISL_12000001-13000000", 7, 120, 4, 3)
end
#   chronic_search(ndjson, ndjson_name, minmut, maxmut, min_chr_mut, revs_thresh)

Number of New Chronic Candidates = 193

EPI_ISL_12001596, EPI_ISL_12006559, EPI_ISL_12019192, EPI_ISL_12032564, EPI_ISL_12049230, EPI_ISL_12052759, EPI_ISL_12052893, EPI_ISL_12053470, EPI_ISL_12060673, EPI_ISL_12070255, EPI_ISL_12080363, EPI_ISL_12102432, EPI_ISL_12102455, EPI_ISL_12102706, EPI_ISL_12109390, EPI_ISL_12146277, EPI_ISL_12146400, EPI_ISL_12146644, EPI_ISL_12146852, EPI_ISL_12146859, EPI_ISL_12146860, EPI_ISL_12146869, EPI_ISL_12146883, EPI_ISL_12148503, EPI_ISL_12160726, EPI_ISL_12161756, EPI_ISL_12170546, EPI_ISL_12171169, EPI_ISL_12171478, EPI_ISL_12171499, EPI_ISL_12174134, EPI_ISL_12174539, EPI_ISL_12174571, EPI_ISL_12174595, EPI_ISL_12175156, EPI_ISL_12175970, EPI_ISL_12227593, EPI_ISL_12245664, EPI_ISL_12261783, EPI_ISL_12264741, EPI_ISL_12274819, EPI_ISL_12293485, EPI_ISL_12293533, EPI_ISL_12293547, EPI_ISL_12293573, EPI_ISL_12293576, EPI_ISL_12293585, EPI_ISL_12293623, EPI_ISL_12293792, EPI_ISL_12293816, EPI_ISL_12315249, EPI_ISL_12328868, EPI_ISL_12329924, EPI_IS

In [2]:
using Pkg
Pkg.add("FASTX")
using FASTX

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [c2308a5c] + FASTX v2.1.7
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [67c07d97] + Automa v1.1.0
  [47718e42] + BioGenerics v0.1.5
⌅ [7e6ae17a] + BioSequences v3.1.6
  [3c28c6f8] + BioSymbols v5.1.3
  [c2308a5c] + FASTX v2.1.7
  [fdea26ae] + SIMD v3.7.1
  [354b36f9] + StringViews v1.3.4
  [7200193e] + Twiddle v1.1.2
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`


In [42]:
println(pwd())
cd("/Users/ryhisner")
println(pwd())
function read_fasta(filepath::String)
    reader = FASTX.FASTA.Reader(open(filepath, "r"))
    fasta_in = [record for record in reader]
    close(reader)
    return[String(FASTX.FASTX.description(rec)) for rec in fasta_in],
    [uppercase(String(FASTX.FASTA.sequence(rec))) for rec in fasta_in]
end
headers, seqs = read_fasta("chronics_2024_12_31__4491_seq_v2_frameshifts_fixed.fasta")
head_set_2024_12_31 = Set{String}()
for head in headers
    epi = split(head, "|")[2]
    push!(head_set_2024_12_31, epi)
end
chr_2024_12_31_set = Set{String}()
for line in eachline("chronics_2024_12_31__4491_seq_v2_frameshifts_fixed.ndjson")
    j = JSON3.read(line)
    name = EPI_ISL(j.seqName)
    push!(chr_2024_12_31_set, name)
end


/Users/ryhisner
/Users/ryhisner
diff_1_2 = Set{String}()
diff_2_1 = Set{String}()



new_seq = 1
diff_new1_new2 = Set{String}()


In [49]:
chr_old1 = length(head_set_2024_12_31)
println(chr_old1)
chr_old2 = length(chr_2024_12_31_set)
println(chr_old2)

4492
4492


In [35]:
EPI_ISLs_2025_4_14 = Set{String}()
EPI_ISLs_2025_5_7 = Set(["EPI_ISL_402124", "EPI_ISL_474824", "EPI_ISL_508674", "EPI_ISL_510148", "EPI_ISL_516999", "EPI_ISL_517000", "EPI_ISL_528438", "EPI_ISL_534720", "EPI_ISL_539541", "EPI_ISL_539542", "EPI_ISL_540582", "EPI_ISL_593478", "EPI_ISL_593479", "EPI_ISL_593480", "EPI_ISL_593553", "EPI_ISL_593554", "EPI_ISL_593555", "EPI_ISL_593556", "EPI_ISL_593557", "EPI_ISL_593558", "EPI_ISL_596228", "EPI_ISL_602912", "EPI_ISL_654166", "EPI_ISL_654170", "EPI_ISL_654172", "EPI_ISL_654173", "EPI_ISL_654182", "EPI_ISL_654186", "EPI_ISL_654191", "EPI_ISL_654194", "EPI_ISL_678289", "EPI_ISL_686537", "EPI_ISL_732971", "EPI_ISL_753676", "EPI_ISL_776770", "EPI_ISL_801876", "EPI_ISL_812862", "EPI_ISL_831496", "EPI_ISL_856750", "EPI_ISL_872778", "EPI_ISL_941340", "EPI_ISL_949208", "EPI_ISL_959309", "EPI_ISL_979349", "EPI_ISL_1014810", "EPI_ISL_1039159", "EPI_ISL_1039839", "EPI_ISL_1059094", "EPI_ISL_1081672", "EPI_ISL_1081673", "EPI_ISL_1090851", "EPI_ISL_1104882", "EPI_ISL_1105146", "EPI_ISL_1105179", "EPI_ISL_1105235", "EPI_ISL_1158169", "EPI_ISL_1200867", "EPI_ISL_1200912", "EPI_ISL_1209365", "EPI_ISL_1225620", "EPI_ISL_1241756", "EPI_ISL_1248458", "EPI_ISL_1248485", "EPI_ISL_1248497", "EPI_ISL_1257978", "EPI_ISL_1261009", "EPI_ISL_1295569", "EPI_ISL_1295575", "EPI_ISL_1309105", "EPI_ISL_1347940", "EPI_ISL_1347941", "EPI_ISL_1347942", "EPI_ISL_1366562", "EPI_ISL_1366563", "EPI_ISL_1366564", "EPI_ISL_1366565", "EPI_ISL_1366566", "EPI_ISL_1366567", "EPI_ISL_1366568", "EPI_ISL_1366569", "EPI_ISL_1366570", "EPI_ISL_1366571", "EPI_ISL_1366572", "EPI_ISL_1366573", "EPI_ISL_1366792", "EPI_ISL_1372287", "EPI_ISL_1372288", "EPI_ISL_1376523", "EPI_ISL_1378739", "EPI_ISL_1403404", "EPI_ISL_1469973", "EPI_ISL_1470396", "EPI_ISL_1473700", "EPI_ISL_1477334", "EPI_ISL_1483302", "EPI_ISL_1490655", "EPI_ISL_1490669", "EPI_ISL_1495749", "EPI_ISL_1517099", "EPI_ISL_1522107", "EPI_ISL_1534324", "EPI_ISL_1547461", "EPI_ISL_1575358", "EPI_ISL_1578495", "EPI_ISL_1626185", "EPI_ISL_1637040", "EPI_ISL_1668821", "EPI_ISL_1668822", "EPI_ISL_1668823", "EPI_ISL_1668824", "EPI_ISL_1668825", "EPI_ISL_1670378", "EPI_ISL_1671330", "EPI_ISL_1675190", "EPI_ISL_1675203", "EPI_ISL_1678377", "EPI_ISL_1738308", "EPI_ISL_1743263", "EPI_ISL_1744401", "EPI_ISL_1756179", "EPI_ISL_1756180", "EPI_ISL_1792929", "EPI_ISL_1829054", "EPI_ISL_1829108", "EPI_ISL_1840893", "EPI_ISL_1855854", "EPI_ISL_1904901", "EPI_ISL_1904903", "EPI_ISL_1908476", "EPI_ISL_1909055", "EPI_ISL_1935116", "EPI_ISL_1935282", "EPI_ISL_1941336", "EPI_ISL_1941816", "EPI_ISL_1965009", "EPI_ISL_1965714", "EPI_ISL_1965722", "EPI_ISL_2001260", "EPI_ISL_2001292", "EPI_ISL_2030332", "EPI_ISL_2035047", "EPI_ISL_2036230", "EPI_ISL_2080876", "EPI_ISL_2096935", "EPI_ISL_2098974", "EPI_ISL_2140680", "EPI_ISL_2142447", "EPI_ISL_2159603", "EPI_ISL_2170618", "EPI_ISL_2179080", "EPI_ISL_2179597", "EPI_ISL_2179598", "EPI_ISL_2179600", "EPI_ISL_2179601", "EPI_ISL_2179635", "EPI_ISL_2193387", "EPI_ISL_2193781", "EPI_ISL_2193790", "EPI_ISL_2229473", "EPI_ISL_2232987", "EPI_ISL_2232988", "EPI_ISL_2245655", "EPI_ISL_2246946", "EPI_ISL_2272316", "EPI_ISL_2281463", "EPI_ISL_2285732", "EPI_ISL_2289324", "EPI_ISL_2331631", "EPI_ISL_2335139", "EPI_ISL_2355027", "EPI_ISL_2373667", "EPI_ISL_2373676", "EPI_ISL_2373689", "EPI_ISL_2373915", "EPI_ISL_2373976", "EPI_ISL_2376734", "EPI_ISL_2385134", "EPI_ISL_2397307", "EPI_ISL_2397308", "EPI_ISL_2397309", "EPI_ISL_2397310", "EPI_ISL_2397311", "EPI_ISL_2397312", "EPI_ISL_2397313", "EPI_ISL_2403056", "EPI_ISL_2408213", "EPI_ISL_2443102", "EPI_ISL_2443306", "EPI_ISL_2451852", "EPI_ISL_2453771", "EPI_ISL_2456706", "EPI_ISL_2466638", "EPI_ISL_2482552", "EPI_ISL_2482891", "EPI_ISL_2484152", "EPI_ISL_2492266", "EPI_ISL_2501697", "EPI_ISL_2504017", "EPI_ISL_2510252", "EPI_ISL_2526835", "EPI_ISL_2537393", "EPI_ISL_2545260", "EPI_ISL_2567482", "EPI_ISL_2567516", "EPI_ISL_2589968", "EPI_ISL_2598472", "EPI_ISL_2621566", "EPI_ISL_2622092", "EPI_ISL_2626505", "EPI_ISL_2629070", "EPI_ISL_2629071", "EPI_ISL_2629072", "EPI_ISL_2629073", "EPI_ISL_2629074", "EPI_ISL_2629075", "EPI_ISL_2646107", "EPI_ISL_2652487", "EPI_ISL_2658958", "EPI_ISL_2658962", "EPI_ISL_2658963", "EPI_ISL_2658970", "EPI_ISL_2658971", "EPI_ISL_2658972", "EPI_ISL_2681259", "EPI_ISL_2686814", "EPI_ISL_2686837", "EPI_ISL_2713004", "EPI_ISL_2716246", "EPI_ISL_2746031", "EPI_ISL_2758178", "EPI_ISL_2758179", "EPI_ISL_2776212", "EPI_ISL_2790083", "EPI_ISL_2791260", "EPI_ISL_2810326", "EPI_ISL_2811857", "EPI_ISL_2817504", "EPI_ISL_2820526", "EPI_ISL_2828407", "EPI_ISL_2833904", "EPI_ISL_2858161", "EPI_ISL_2858877", "EPI_ISL_2860316", "EPI_ISL_2868572", "EPI_ISL_2868616", "EPI_ISL_2876377", "EPI_ISL_2894033", "EPI_ISL_2903438", "EPI_ISL_2931876", "EPI_ISL_2931884", "EPI_ISL_2931896", "EPI_ISL_2931903", "EPI_ISL_2955288", "EPI_ISL_2955320", "EPI_ISL_2966985", "EPI_ISL_2978243", "EPI_ISL_2978352", "EPI_ISL_2984725", "EPI_ISL_2990101", "EPI_ISL_2993722", "EPI_ISL_3010321", "EPI_ISL_3029841", "EPI_ISL_3030114", "EPI_ISL_3030118", "EPI_ISL_3030145", "EPI_ISL_3030738", "EPI_ISL_3032627", "EPI_ISL_3033635", "EPI_ISL_3039352", "EPI_ISL_3045789", "EPI_ISL_3053903", "EPI_ISL_3061061", "EPI_ISL_3063770", "EPI_ISL_3064314", "EPI_ISL_3129808", "EPI_ISL_3130077", "EPI_ISL_3130081", "EPI_ISL_3130177", "EPI_ISL_3130302", "EPI_ISL_3132631", "EPI_ISL_3133023", "EPI_ISL_3152200", "EPI_ISL_3153240", "EPI_ISL_3164424", "EPI_ISL_3185346", "EPI_ISL_3209041", "EPI_ISL_3212959", "EPI_ISL_3215721", "EPI_ISL_3215722", "EPI_ISL_3215726", "EPI_ISL_3246237", "EPI_ISL_3251444", "EPI_ISL_3251605", "EPI_ISL_3259560", "EPI_ISL_3275376", "EPI_ISL_3339536", "EPI_ISL_3356734", "EPI_ISL_3358574", "EPI_ISL_3370176", "EPI_ISL_3394321", "EPI_ISL_3396491", "EPI_ISL_3414767", "EPI_ISL_3414889", "EPI_ISL_3415104", "EPI_ISL_3415226", "EPI_ISL_3426474", "EPI_ISL_3446827", "EPI_ISL_3447712", "EPI_ISL_3453279", "EPI_ISL_3457423", "EPI_ISL_3459118", "EPI_ISL_3471360", "EPI_ISL_3475993", "EPI_ISL_3497667", "EPI_ISL_3503811", "EPI_ISL_3504036", "EPI_ISL_3509013", "EPI_ISL_3556945", "EPI_ISL_3634003", "EPI_ISL_3634004", "EPI_ISL_3635506", "EPI_ISL_3657112", "EPI_ISL_3666069", "EPI_ISL_3712919", "EPI_ISL_3771882", "EPI_ISL_3779849", "EPI_ISL_3813731", "EPI_ISL_3838306", "EPI_ISL_3891136", "EPI_ISL_3933252", "EPI_ISL_3937027", "EPI_ISL_3957778", "EPI_ISL_3958461", "EPI_ISL_3958994", "EPI_ISL_3982251", "EPI_ISL_4029567", "EPI_ISL_4052911", "EPI_ISL_4072038", "EPI_ISL_4096626", "EPI_ISL_4096639", "EPI_ISL_4114033", "EPI_ISL_4124532", "EPI_ISL_4178790", "EPI_ISL_4193135", "EPI_ISL_4198270", "EPI_ISL_4203869", "EPI_ISL_4251611", "EPI_ISL_4261403", "EPI_ISL_4261411", "EPI_ISL_4281194", "EPI_ISL_4284228", "EPI_ISL_4295678", "EPI_ISL_4298277", "EPI_ISL_4298278", "EPI_ISL_4298279", "EPI_ISL_4309817", "EPI_ISL_4415808", "EPI_ISL_4440075", "EPI_ISL_4515444", "EPI_ISL_4525691", "EPI_ISL_4525698", "EPI_ISL_4525700", "EPI_ISL_4531313", "EPI_ISL_4536418", "EPI_ISL_4576991", "EPI_ISL_4577474", "EPI_ISL_4625101", "EPI_ISL_4652284", "EPI_ISL_4769386", "EPI_ISL_4775547", "EPI_ISL_4875939", "EPI_ISL_4930863", "EPI_ISL_4935777", "EPI_ISL_4935949", "EPI_ISL_4936095", "EPI_ISL_4936533", "EPI_ISL_4949584", "EPI_ISL_5018695", "EPI_ISL_5033183", "EPI_ISL_5056434", "EPI_ISL_5059980", "EPI_ISL_5099310", "EPI_ISL_5132437", "EPI_ISL_5132595", "EPI_ISL_5145656", "EPI_ISL_5196003", "EPI_ISL_5265214", "EPI_ISL_5280146", "EPI_ISL_5307398", "EPI_ISL_5323016", "EPI_ISL_5332877", "EPI_ISL_5332878", "EPI_ISL_5395558", "EPI_ISL_5446154", "EPI_ISL_5463914", "EPI_ISL_5532714", "EPI_ISL_5592605", "EPI_ISL_5620309", "EPI_ISL_5621224", "EPI_ISL_5627313", "EPI_ISL_5628248", "EPI_ISL_5639913", "EPI_ISL_5640459", "EPI_ISL_5649323", "EPI_ISL_5650474", "EPI_ISL_5655562", "EPI_ISL_5680241", "EPI_ISL_5692774", "EPI_ISL_5749185", "EPI_ISL_5780324", "EPI_ISL_5814411", "EPI_ISL_5865553", "EPI_ISL_5892132", "EPI_ISL_5922347", "EPI_ISL_5935407", "EPI_ISL_5944665", "EPI_ISL_5944669", "EPI_ISL_5944842", "EPI_ISL_5944948", "EPI_ISL_5946914", "EPI_ISL_6017746", "EPI_ISL_6017747", "EPI_ISL_6076460", "EPI_ISL_6208674", "EPI_ISL_6208675", "EPI_ISL_6208676", "EPI_ISL_6227177", "EPI_ISL_6227208", "EPI_ISL_6262165", "EPI_ISL_6281381", "EPI_ISL_6324366", "EPI_ISL_6327943", "EPI_ISL_6381841", "EPI_ISL_6384755", "EPI_ISL_6574278", "EPI_ISL_6584511", "EPI_ISL_6604686", "EPI_ISL_6605003", "EPI_ISL_6605659", "EPI_ISL_6628662", "EPI_ISL_6642599", "EPI_ISL_6666037", "EPI_ISL_6698637", "EPI_ISL_6710721", "EPI_ISL_6735468", "EPI_ISL_6737833", "EPI_ISL_6757093", "EPI_ISL_6783610", "EPI_ISL_6810267", "EPI_ISL_6814822", "EPI_ISL_6826536", "EPI_ISL_6842893", "EPI_ISL_6863316", "EPI_ISL_6863457", "EPI_ISL_6865741", "EPI_ISL_6930836", "EPI_ISL_6938691", "EPI_ISL_6976497", "EPI_ISL_6977941", "EPI_ISL_7000663", "EPI_ISL_7015624", "EPI_ISL_7015625", "EPI_ISL_7135374", "EPI_ISL_7159687", "EPI_ISL_7175748", "EPI_ISL_7204318", "EPI_ISL_7205152", "EPI_ISL_7334013", "EPI_ISL_7361483", "EPI_ISL_7361527", "EPI_ISL_7452581", "EPI_ISL_7452603", "EPI_ISL_7456462", "EPI_ISL_7458003", "EPI_ISL_7502990", "EPI_ISL_7503221", "EPI_ISL_7507393", "EPI_ISL_7592661", "EPI_ISL_7592687", "EPI_ISL_7652115", "EPI_ISL_7660179", "EPI_ISL_7707631", "EPI_ISL_7711813", "EPI_ISL_7729239", "EPI_ISL_7806535", "EPI_ISL_7806536", "EPI_ISL_7806549", "EPI_ISL_7813896", "EPI_ISL_7861580", "EPI_ISL_7876524", "EPI_ISL_7908114", "EPI_ISL_7961502", "EPI_ISL_7976931", "EPI_ISL_7980711", "EPI_ISL_8001538", "EPI_ISL_8035582", "EPI_ISL_8131224", "EPI_ISL_8132253", "EPI_ISL_8151798", "EPI_ISL_8153087", "EPI_ISL_8166542", "EPI_ISL_8189765", "EPI_ISL_8189775", "EPI_ISL_8204828", "EPI_ISL_8205040", "EPI_ISL_8207600", "EPI_ISL_8215783", "EPI_ISL_8215787", "EPI_ISL_8251200", "EPI_ISL_8263463", "EPI_ISL_8376888", "EPI_ISL_8479639", "EPI_ISL_8479640", "EPI_ISL_8563217", "EPI_ISL_8563218", "EPI_ISL_8563219", "EPI_ISL_8615077", "EPI_ISL_8627379", "EPI_ISL_8669281", "EPI_ISL_8712661", "EPI_ISL_8725398", "EPI_ISL_8725399", "EPI_ISL_8725400", "EPI_ISL_8725401", "EPI_ISL_8725402", "EPI_ISL_8725403", "EPI_ISL_8725404", "EPI_ISL_8725405", "EPI_ISL_8725406", "EPI_ISL_8725407", "EPI_ISL_8725408", "EPI_ISL_8725409", "EPI_ISL_8732699", "EPI_ISL_8732807", "EPI_ISL_8732841", "EPI_ISL_8750545", "EPI_ISL_8754305", "EPI_ISL_8766992", "EPI_ISL_8800409", "EPI_ISL_8806077", "EPI_ISL_8806082", "EPI_ISL_8806084", "EPI_ISL_8819629", "EPI_ISL_8825833", "EPI_ISL_8828662", "EPI_ISL_8887845", "EPI_ISL_8887874", "EPI_ISL_8973278", "EPI_ISL_9021214", "EPI_ISL_9141923", "EPI_ISL_9155607", "EPI_ISL_9201951", "EPI_ISL_9242265", "EPI_ISL_9242269", "EPI_ISL_9316982", "EPI_ISL_9319180", "EPI_ISL_9570633", "EPI_ISL_9630717", "EPI_ISL_9637481", "EPI_ISL_9637840", "EPI_ISL_9702285", "EPI_ISL_9735644", "EPI_ISL_9844246", "EPI_ISL_9869512", "EPI_ISL_9873278", "EPI_ISL_9907655", "EPI_ISL_9949797", "EPI_ISL_9949799", "EPI_ISL_10124646", "EPI_ISL_10127751", "EPI_ISL_10128185", "EPI_ISL_10132276", "EPI_ISL_10185453", "EPI_ISL_10195257", "EPI_ISL_10195262", "EPI_ISL_10195263", "EPI_ISL_10195264", "EPI_ISL_10195305", "EPI_ISL_10230612", "EPI_ISL_10239201", "EPI_ISL_10251304", "EPI_ISL_10329391", "EPI_ISL_10329558", "EPI_ISL_10352747", "EPI_ISL_10397517", "EPI_ISL_10451205", "EPI_ISL_10451252", "EPI_ISL_10548912", "EPI_ISL_10548913", "EPI_ISL_10548915", "EPI_ISL_10548916", "EPI_ISL_10548917", "EPI_ISL_10548918", "EPI_ISL_10548919", "EPI_ISL_10548920", "EPI_ISL_10548921", "EPI_ISL_10548922", "EPI_ISL_10549162", "EPI_ISL_10549163", "EPI_ISL_10549164", "EPI_ISL_10549165", "EPI_ISL_10549166", "EPI_ISL_10590270", "EPI_ISL_10590760", "EPI_ISL_10591808", "EPI_ISL_10657890", "EPI_ISL_10681118", "EPI_ISL_10706292", "EPI_ISL_10712909", "EPI_ISL_10717525", "EPI_ISL_10815044", "EPI_ISL_10816730", "EPI_ISL_10816731", "EPI_ISL_10816732", "EPI_ISL_10816733", "EPI_ISL_10816734", "EPI_ISL_10816735", "EPI_ISL_10816736", "EPI_ISL_10816737", "EPI_ISL_10816738", "EPI_ISL_10816739", "EPI_ISL_10816741", "EPI_ISL_10816742", "EPI_ISL_10816743", "EPI_ISL_10816744", "EPI_ISL_10824028", "EPI_ISL_10876034", "EPI_ISL_10876749", "EPI_ISL_10899907", "EPI_ISL_10942195", "EPI_ISL_10981395", "EPI_ISL_10995323", "EPI_ISL_10998528", "EPI_ISL_11025821", "EPI_ISL_11025897", "EPI_ISL_11030507", "EPI_ISL_11036385", "EPI_ISL_11036386", "EPI_ISL_11036389", "EPI_ISL_11036399", "EPI_ISL_11036451", "EPI_ISL_11036666", "EPI_ISL_11036688", "EPI_ISL_11036712", "EPI_ISL_11036915", "EPI_ISL_11036917", "EPI_ISL_11050902", "EPI_ISL_11055609", "EPI_ISL_11106543", "EPI_ISL_11110730", "EPI_ISL_11167907", "EPI_ISL_11173072", "EPI_ISL_11219235", "EPI_ISL_11219236", "EPI_ISL_11221773", "EPI_ISL_11221782", "EPI_ISL_11222620", "EPI_ISL_11229672", "EPI_ISL_11239958", "EPI_ISL_11242266", "EPI_ISL_11248919", "EPI_ISL_11256669", "EPI_ISL_11290054", "EPI_ISL_11295642", "EPI_ISL_11296415", "EPI_ISL_11349763", "EPI_ISL_11356268", "EPI_ISL_11356269", "EPI_ISL_11403393", "EPI_ISL_11424263", "EPI_ISL_11424578", "EPI_ISL_11437359", "EPI_ISL_11482304", "EPI_ISL_11503909", "EPI_ISL_11504189", "EPI_ISL_11517385", "EPI_ISL_11565840", "EPI_ISL_11576757", "EPI_ISL_11600821", "EPI_ISL_11600822", "EPI_ISL_11621597", "EPI_ISL_11657715", "EPI_ISL_11661806", "EPI_ISL_11695384", "EPI_ISL_11742572", "EPI_ISL_11742812", "EPI_ISL_11747289", "EPI_ISL_11778548", "EPI_ISL_11787443", "EPI_ISL_11798407", "EPI_ISL_11798458", "EPI_ISL_11801453", "EPI_ISL_11816904", "EPI_ISL_11825798", "EPI_ISL_11826326", "EPI_ISL_11826898", "EPI_ISL_11871462", "EPI_ISL_11886470", "EPI_ISL_11886479", "EPI_ISL_11886624", "EPI_ISL_11897546", "EPI_ISL_11919916", "EPI_ISL_11961223", "EPI_ISL_11968830", "EPI_ISL_11968876", "EPI_ISL_11970393", "EPI_ISL_11976211", "EPI_ISL_11992954", "EPI_ISL_11994571", "EPI_ISL_11995938", "EPI_ISL_12001179", "EPI_ISL_12001180", "EPI_ISL_12021469", "EPI_ISL_12039060", "EPI_ISL_12060087", "EPI_ISL_12063598", "EPI_ISL_12063599", "EPI_ISL_12063600", "EPI_ISL_12063601", "EPI_ISL_12063602", "EPI_ISL_12079999", "EPI_ISL_12080356", "EPI_ISL_12083619", "EPI_ISL_12089943", "EPI_ISL_12097355", "EPI_ISL_12108965", "EPI_ISL_12109074", "EPI_ISL_12127282", "EPI_ISL_12137683", "EPI_ISL_12139045", "EPI_ISL_12139066", "EPI_ISL_12145506", "EPI_ISL_12146579", "EPI_ISL_12146859", "EPI_ISL_12146865", "EPI_ISL_12146872", "EPI_ISL_12148419", "EPI_ISL_12150077", "EPI_ISL_12150259", "EPI_ISL_12150361", "EPI_ISL_12150484", "EPI_ISL_12155759", "EPI_ISL_12155809", "EPI_ISL_12157165", "EPI_ISL_12157166", "EPI_ISL_12157187", "EPI_ISL_12158960", "EPI_ISL_12162547", "EPI_ISL_12168418", "EPI_ISL_12171333", "EPI_ISL_12171674", "EPI_ISL_12172132", "EPI_ISL_12172842", "EPI_ISL_12173486", "EPI_ISL_12173730", "EPI_ISL_12173879", "EPI_ISL_12174612", "EPI_ISL_12174734", "EPI_ISL_12174735", "EPI_ISL_12174736", "EPI_ISL_12174739", "EPI_ISL_12174942", "EPI_ISL_12175020", "EPI_ISL_12175024", "EPI_ISL_12175203", "EPI_ISL_12176184", "EPI_ISL_12207682", "EPI_ISL_12220762", "EPI_ISL_12240087", "EPI_ISL_12240983", "EPI_ISL_12251561", "EPI_ISL_12259859", "EPI_ISL_12261629", "EPI_ISL_12268492", "EPI_ISL_12278477", "EPI_ISL_12278678", "EPI_ISL_12278997", "EPI_ISL_12284821", "EPI_ISL_12293549", "EPI_ISL_12296891", "EPI_ISL_12310532", "EPI_ISL_12323992", "EPI_ISL_12325408", "EPI_ISL_12350967", "EPI_ISL_12351281", "EPI_ISL_12355622", "EPI_ISL_12371147", "EPI_ISL_12401928", "EPI_ISL_12422504", "EPI_ISL_12425033", "EPI_ISL_12430022", "EPI_ISL_12444737", "EPI_ISL_12447557", "EPI_ISL_12454820", "EPI_ISL_12454840", "EPI_ISL_12464077", "EPI_ISL_12467081", "EPI_ISL_12467157", "EPI_ISL_12473693", "EPI_ISL_12475004", "EPI_ISL_12486436", "EPI_ISL_12486761", "EPI_ISL_12488441", "EPI_ISL_12495863", "EPI_ISL_12501519", "EPI_ISL_12511246", "EPI_ISL_12530780", "EPI_ISL_12531462", "EPI_ISL_12535815", "EPI_ISL_12539663", "EPI_ISL_12564261", "EPI_ISL_12568162", "EPI_ISL_12568208", "EPI_ISL_12575298", "EPI_ISL_12579825", "EPI_ISL_12581914", "EPI_ISL_12590958", "EPI_ISL_12602903", "EPI_ISL_12611697", "EPI_ISL_12611721", "EPI_ISL_12616586", "EPI_ISL_12622901", "EPI_ISL_12622902", "EPI_ISL_12623284", "EPI_ISL_12628520", "EPI_ISL_12639714", "EPI_ISL_12639788", "EPI_ISL_12639917", "EPI_ISL_12645823", "EPI_ISL_12646116", "EPI_ISL_12646785", "EPI_ISL_12647336", "EPI_ISL_12652423", "EPI_ISL_12654179", "EPI_ISL_12661097", "EPI_ISL_12663222", "EPI_ISL_12664764", "EPI_ISL_12680798", "EPI_ISL_12685124", "EPI_ISL_12685126", "EPI_ISL_12691923", "EPI_ISL_12698937", "EPI_ISL_12701772", "EPI_ISL_12701820", "EPI_ISL_12701858", "EPI_ISL_12701867", "EPI_ISL_12701871", "EPI_ISL_12701895", "EPI_ISL_12703517", "EPI_ISL_12707647", "EPI_ISL_12710394", "EPI_ISL_12717870", "EPI_ISL_12735772", "EPI_ISL_12739317", "EPI_ISL_12739425", "EPI_ISL_12741485", "EPI_ISL_12742126", "EPI_ISL_12742222", "EPI_ISL_12744193", "EPI_ISL_12754976", "EPI_ISL_12763802", "EPI_ISL_12765459", "EPI_ISL_12771614", "EPI_ISL_12784028", "EPI_ISL_12789812", "EPI_ISL_12789846", "EPI_ISL_12799990", "EPI_ISL_12805921", "EPI_ISL_12808264", "EPI_ISL_12809016", "EPI_ISL_12811012", "EPI_ISL_12822481", "EPI_ISL_12822483", "EPI_ISL_12829036", "EPI_ISL_12830215", "EPI_ISL_12830827", "EPI_ISL_12843060", "EPI_ISL_12844170", "EPI_ISL_12851188", "EPI_ISL_12851233", "EPI_ISL_12851285", "EPI_ISL_12862705", "EPI_ISL_12862970", "EPI_ISL_12871249", "EPI_ISL_12879252", "EPI_ISL_12892238", "EPI_ISL_12892312", "EPI_ISL_12892482", "EPI_ISL_12896994", "EPI_ISL_12903760", "EPI_ISL_12906172", "EPI_ISL_12911895", "EPI_ISL_12911898", "EPI_ISL_12914019", "EPI_ISL_12926555", "EPI_ISL_12926955", "EPI_ISL_12932770", "EPI_ISL_12953136", "EPI_ISL_12953419", "EPI_ISL_12955795", "EPI_ISL_12958668", "EPI_ISL_12961699", "EPI_ISL_12968380", "EPI_ISL_12980420", "EPI_ISL_12993020", "EPI_ISL_12995035", "EPI_ISL_12995123", "EPI_ISL_12995230", "EPI_ISL_12995281", "EPI_ISL_12995422", "EPI_ISL_12995533", "EPI_ISL_13000815", "EPI_ISL_13001148", "EPI_ISL_13011225", "EPI_ISL_13011226", "EPI_ISL_13019919", "EPI_ISL_13026163", "EPI_ISL_13028133", "EPI_ISL_13040401", "EPI_ISL_13044133", "EPI_ISL_13047387", "EPI_ISL_13050078", "EPI_ISL_13050270", "EPI_ISL_13050793", "EPI_ISL_13051431", "EPI_ISL_13051740", "EPI_ISL_13051790", "EPI_ISL_13051970", "EPI_ISL_13052096", "EPI_ISL_13052204", "EPI_ISL_13055324", "EPI_ISL_13055618", "EPI_ISL_13056142", "EPI_ISL_13056190", "EPI_ISL_13065554", "EPI_ISL_13066396", "EPI_ISL_13066472", "EPI_ISL_13066603", "EPI_ISL_13073691", "EPI_ISL_13085784", "EPI_ISL_13085991", "EPI_ISL_13086417", "EPI_ISL_13086737", "EPI_ISL_13086826", "EPI_ISL_13086831", "EPI_ISL_13088769", "EPI_ISL_13088942", "EPI_ISL_13089020", "EPI_ISL_13089384", "EPI_ISL_13091908", "EPI_ISL_13091912", "EPI_ISL_13091925", "EPI_ISL_13091929", "EPI_ISL_13092725", "EPI_ISL_13092906", "EPI_ISL_13093369", "EPI_ISL_13093922", "EPI_ISL_13094368", "EPI_ISL_13108591", "EPI_ISL_13110029", "EPI_ISL_13110031", "EPI_ISL_13123133", "EPI_ISL_13129353", "EPI_ISL_13129387", "EPI_ISL_13131091", "EPI_ISL_13131118", "EPI_ISL_13131514", "EPI_ISL_13132070", "EPI_ISL_13133128", "EPI_ISL_13133359", "EPI_ISL_13152570", "EPI_ISL_13157537", "EPI_ISL_13157638", "EPI_ISL_13158312", "EPI_ISL_13158314", "EPI_ISL_13160040", "EPI_ISL_13166402", "EPI_ISL_13166803", "EPI_ISL_13169047", "EPI_ISL_13169050", "EPI_ISL_13172328", "EPI_ISL_13172329", "EPI_ISL_13176279", "EPI_ISL_13176281", "EPI_ISL_13178754", "EPI_ISL_13183984", "EPI_ISL_13187136", "EPI_ISL_13192066", "EPI_ISL_13192072", "EPI_ISL_13192202", "EPI_ISL_13199746", "EPI_ISL_13199759", "EPI_ISL_13202578", "EPI_ISL_13210987", "EPI_ISL_13211404", "EPI_ISL_13214299", "EPI_ISL_13215742", "EPI_ISL_13230467", "EPI_ISL_13242111", "EPI_ISL_13242155", "EPI_ISL_13244164", "EPI_ISL_13244243", "EPI_ISL_13244707", "EPI_ISL_13251406", "EPI_ISL_13251514", "EPI_ISL_13251524", "EPI_ISL_13253132", "EPI_ISL_13253164", "EPI_ISL_13253244", "EPI_ISL_13253296", "EPI_ISL_13253344", "EPI_ISL_13253353", "EPI_ISL_13253416", "EPI_ISL_13253427", "EPI_ISL_13253489", "EPI_ISL_13253493", "EPI_ISL_13253550", "EPI_ISL_13253551", "EPI_ISL_13253570", "EPI_ISL_13264387", "EPI_ISL_13271922", "EPI_ISL_13272001", "EPI_ISL_13272223", "EPI_ISL_13273550", "EPI_ISL_13284168", "EPI_ISL_13288721", "EPI_ISL_13289213", "EPI_ISL_13289619", "EPI_ISL_13289774", "EPI_ISL_13294595", "EPI_ISL_13299119", "EPI_ISL_13304429", "EPI_ISL_13304451", "EPI_ISL_13304482", "EPI_ISL_13306995", "EPI_ISL_13307081", "EPI_ISL_13312837", "EPI_ISL_13314960", "EPI_ISL_13317150", "EPI_ISL_13317160", "EPI_ISL_13318832", "EPI_ISL_13319769", "EPI_ISL_13320180", "EPI_ISL_13322028", "EPI_ISL_13322954", "EPI_ISL_13322975", "EPI_ISL_13323290", "EPI_ISL_13328732", "EPI_ISL_13329792", "EPI_ISL_13332433", "EPI_ISL_13332459", "EPI_ISL_13332921", "EPI_ISL_13333047", "EPI_ISL_13338249", "EPI_ISL_13339980", "EPI_ISL_13345908", "EPI_ISL_13349432", "EPI_ISL_13350581", "EPI_ISL_13354243", "EPI_ISL_13354366", "EPI_ISL_13356160", "EPI_ISL_13356194", "EPI_ISL_13357233", "EPI_ISL_13358809", "EPI_ISL_13358893", "EPI_ISL_13358894", "EPI_ISL_13358962", "EPI_ISL_13361313", "EPI_ISL_13361419", "EPI_ISL_13362130", "EPI_ISL_13368501", "EPI_ISL_13368552", "EPI_ISL_13369326", "EPI_ISL_13371824", "EPI_ISL_13376289", "EPI_ISL_13385215", "EPI_ISL_13386427", "EPI_ISL_13388894", "EPI_ISL_13389863", "EPI_ISL_13389864", "EPI_ISL_13394010", "EPI_ISL_13398372", "EPI_ISL_13398391", "EPI_ISL_13400530", "EPI_ISL_13405240", "EPI_ISL_13406133", "EPI_ISL_13407391", "EPI_ISL_13407748", "EPI_ISL_13407802", "EPI_ISL_13408380", "EPI_ISL_13410054", "EPI_ISL_13410128", "EPI_ISL_13412509", "EPI_ISL_13417422", "EPI_ISL_13417595", "EPI_ISL_13417637", "EPI_ISL_13422063", "EPI_ISL_13425805", "EPI_ISL_13426860", "EPI_ISL_13426861", "EPI_ISL_13426862", "EPI_ISL_13440246", "EPI_ISL_13440269", "EPI_ISL_13440488", "EPI_ISL_13443257", "EPI_ISL_13454711", "EPI_ISL_13455972", "EPI_ISL_13463270", "EPI_ISL_13464004", "EPI_ISL_13465109", "EPI_ISL_13466588", "EPI_ISL_13466629", "EPI_ISL_13466644", "EPI_ISL_13467321", "EPI_ISL_13467676", "EPI_ISL_13470130", "EPI_ISL_13470158", "EPI_ISL_13476507", "EPI_ISL_13477158", "EPI_ISL_13478276", "EPI_ISL_13478413", "EPI_ISL_13478421", "EPI_ISL_13478425", "EPI_ISL_13478448", "EPI_ISL_13478487", "EPI_ISL_13480851", "EPI_ISL_13481704", "EPI_ISL_13482848", "EPI_ISL_13483067", "EPI_ISL_13483538", "EPI_ISL_13483605", "EPI_ISL_13483653", "EPI_ISL_13489191", "EPI_ISL_13501768", "EPI_ISL_13502856", "EPI_ISL_13502894", "EPI_ISL_13503354", "EPI_ISL_13503362", "EPI_ISL_13503413", "EPI_ISL_13504084", "EPI_ISL_13504103", "EPI_ISL_13504568", "EPI_ISL_13518485", "EPI_ISL_13519541", "EPI_ISL_13522554", "EPI_ISL_13522707", "EPI_ISL_13529086", "EPI_ISL_13534848", "EPI_ISL_13535844", "EPI_ISL_13536846", "EPI_ISL_13536992", "EPI_ISL_13538620", "EPI_ISL_13538621", "EPI_ISL_13539746", "EPI_ISL_13552269", "EPI_ISL_13560139", "EPI_ISL_13563849", "EPI_ISL_13563900", "EPI_ISL_13564453", "EPI_ISL_13564901", "EPI_ISL_13566717", "EPI_ISL_13571629", "EPI_ISL_13572579", "EPI_ISL_13572829", "EPI_ISL_13573543", "EPI_ISL_13573707", "EPI_ISL_13578717", "EPI_ISL_13585459", "EPI_ISL_13585850", "EPI_ISL_13592668", "EPI_ISL_13605240", "EPI_ISL_13605930", "EPI_ISL_13608346", "EPI_ISL_13611346", "EPI_ISL_13611373", "EPI_ISL_13612632", "EPI_ISL_13614328", "EPI_ISL_13615804", "EPI_ISL_13615826", "EPI_ISL_13616458", "EPI_ISL_13617390", "EPI_ISL_13617475", "EPI_ISL_13617493", "EPI_ISL_13619646", "EPI_ISL_13622657", "EPI_ISL_13622831", "EPI_ISL_13622874", "EPI_ISL_13624440", "EPI_ISL_13624441", "EPI_ISL_13626226", "EPI_ISL_13633558", "EPI_ISL_13633729", "EPI_ISL_13636934", "EPI_ISL_13637141", "EPI_ISL_13637734", "EPI_ISL_13638494", "EPI_ISL_13642059", "EPI_ISL_13653660", "EPI_ISL_13665096", "EPI_ISL_13665110", "EPI_ISL_13666764", "EPI_ISL_13677872", "EPI_ISL_13688333", "EPI_ISL_13691966", "EPI_ISL_13692397", "EPI_ISL_13694663", "EPI_ISL_13696042", "EPI_ISL_13698648", "EPI_ISL_13700128", "EPI_ISL_13700243", "EPI_ISL_13700756", "EPI_ISL_13701029", "EPI_ISL_13701810", "EPI_ISL_13710357", "EPI_ISL_13715129", "EPI_ISL_13715746", "EPI_ISL_13716400", "EPI_ISL_13716943", "EPI_ISL_13723111", "EPI_ISL_13723805", "EPI_ISL_13726415", "EPI_ISL_13726482", "EPI_ISL_13729045", "EPI_ISL_13732799", "EPI_ISL_13734474", "EPI_ISL_13734683", "EPI_ISL_13738059", "EPI_ISL_13740111", "EPI_ISL_13740674", "EPI_ISL_13741330", "EPI_ISL_13744798", "EPI_ISL_13744799", "EPI_ISL_13745638", "EPI_ISL_13745641", "EPI_ISL_13748166", "EPI_ISL_13750726", "EPI_ISL_13750730", "EPI_ISL_13750760", "EPI_ISL_13750771", "EPI_ISL_13750936", "EPI_ISL_13750937", "EPI_ISL_13751254", "EPI_ISL_13755996", "EPI_ISL_13757795", "EPI_ISL_13757902", "EPI_ISL_13757914", "EPI_ISL_13759674", "EPI_ISL_13759811", "EPI_ISL_13760929", "EPI_ISL_13762803", "EPI_ISL_13764852", "EPI_ISL_13765234", "EPI_ISL_13769441", "EPI_ISL_13774980", "EPI_ISL_13776103", "EPI_ISL_13776118", "EPI_ISL_13778691", "EPI_ISL_13780813", "EPI_ISL_13788917", "EPI_ISL_13794761", "EPI_ISL_13795106", "EPI_ISL_13795308", "EPI_ISL_13802466", "EPI_ISL_13805350", "EPI_ISL_13806026", "EPI_ISL_13806197", "EPI_ISL_13810913", "EPI_ISL_13812067", "EPI_ISL_13824458", "EPI_ISL_13826362", "EPI_ISL_13830194", "EPI_ISL_13830195", "EPI_ISL_13830196", "EPI_ISL_13830197", "EPI_ISL_13830445", "EPI_ISL_13830454", "EPI_ISL_13839105", "EPI_ISL_13839285", "EPI_ISL_13842068", "EPI_ISL_13844161", "EPI_ISL_13850726", "EPI_ISL_13855446", "EPI_ISL_13856866", "EPI_ISL_13858143", "EPI_ISL_13858664", "EPI_ISL_13860426", "EPI_ISL_13860879", "EPI_ISL_13866687", "EPI_ISL_13866688", "EPI_ISL_13866691", "EPI_ISL_13867130", "EPI_ISL_13867135", "EPI_ISL_13871326", "EPI_ISL_13873100", "EPI_ISL_13875348", "EPI_ISL_13875677", "EPI_ISL_13876290", "EPI_ISL_13876318", "EPI_ISL_13876612", "EPI_ISL_13876760", "EPI_ISL_13881123", "EPI_ISL_13884353", "EPI_ISL_13884360", "EPI_ISL_13884439", "EPI_ISL_13889482", "EPI_ISL_13891697", "EPI_ISL_13896135", "EPI_ISL_13896156", "EPI_ISL_13896578", "EPI_ISL_13900930", "EPI_ISL_13907795", "EPI_ISL_13907925", "EPI_ISL_13908668", "EPI_ISL_13913047", "EPI_ISL_13915527", "EPI_ISL_13915530", "EPI_ISL_13925703", "EPI_ISL_13931117", "EPI_ISL_13931975", "EPI_ISL_13931997", "EPI_ISL_13932039", "EPI_ISL_13932080", "EPI_ISL_13937613", "EPI_ISL_13939223", "EPI_ISL_13947566", "EPI_ISL_13948007", "EPI_ISL_13951654", "EPI_ISL_13957684", "EPI_ISL_13958481", "EPI_ISL_13958566", "EPI_ISL_13963277", "EPI_ISL_13963776", "EPI_ISL_13963832", "EPI_ISL_13967204", "EPI_ISL_13968198", "EPI_ISL_13970237", "EPI_ISL_13970242", "EPI_ISL_13970249", "EPI_ISL_13970257", "EPI_ISL_13970276", "EPI_ISL_13975822", "EPI_ISL_13981101", "EPI_ISL_13984460", "EPI_ISL_13986492", "EPI_ISL_13986494", "EPI_ISL_13986497", "EPI_ISL_13986498", "EPI_ISL_13986501", "EPI_ISL_13989484", "EPI_ISL_13991375", "EPI_ISL_13993708", "EPI_ISL_13994781", "EPI_ISL_14000155", "EPI_ISL_14004519", "EPI_ISL_14005794", "EPI_ISL_14010460", "EPI_ISL_14010462", "EPI_ISL_14011475", "EPI_ISL_14015047", "EPI_ISL_14019093", "EPI_ISL_14019109", "EPI_ISL_14019330", "EPI_ISL_14020697", "EPI_ISL_14022780", "EPI_ISL_14022892", "EPI_ISL_14023662", "EPI_ISL_14027304", "EPI_ISL_14027788", "EPI_ISL_14028215", "EPI_ISL_14029314", "EPI_ISL_14029803", "EPI_ISL_14030175", "EPI_ISL_14032717", "EPI_ISL_14035970", "EPI_ISL_14036069", "EPI_ISL_14041069", "EPI_ISL_14044698", "EPI_ISL_14044704", "EPI_ISL_14046291", "EPI_ISL_14047361", "EPI_ISL_14050544", "EPI_ISL_14051041", "EPI_ISL_14051049", "EPI_ISL_14051052", "EPI_ISL_14057503", "EPI_ISL_14062117", "EPI_ISL_14064598", "EPI_ISL_14064601", "EPI_ISL_14066591", "EPI_ISL_14066852", "EPI_ISL_14071587", "EPI_ISL_14071795", "EPI_ISL_14097542", "EPI_ISL_14097629", "EPI_ISL_14124074", "EPI_ISL_14127563", "EPI_ISL_14128514", "EPI_ISL_14134678", "EPI_ISL_14135854", "EPI_ISL_14147202", "EPI_ISL_14155218", "EPI_ISL_14158264", "EPI_ISL_14161024", "EPI_ISL_14170603", "EPI_ISL_14171301", "EPI_ISL_14172905", "EPI_ISL_14173767", "EPI_ISL_14175092", "EPI_ISL_14175097", "EPI_ISL_14175103", "EPI_ISL_14175182", "EPI_ISL_14181740", "EPI_ISL_14193000", "EPI_ISL_14193833", "EPI_ISL_14196068", "EPI_ISL_14196701", "EPI_ISL_14197724", "EPI_ISL_14198080", "EPI_ISL_14200342", "EPI_ISL_14200801", "EPI_ISL_14203206", "EPI_ISL_14203613", "EPI_ISL_14208740", "EPI_ISL_14208835", "EPI_ISL_14209372", "EPI_ISL_14209934", "EPI_ISL_14210619", "EPI_ISL_14211976", "EPI_ISL_14215014", "EPI_ISL_14215818", "EPI_ISL_14216595", "EPI_ISL_14217745", "EPI_ISL_14222817", "EPI_ISL_14223595", "EPI_ISL_14223736", "EPI_ISL_14223750", "EPI_ISL_14224871", "EPI_ISL_14225747", "EPI_ISL_14226883", "EPI_ISL_14228030", "EPI_ISL_14229584", "EPI_ISL_14230429", "EPI_ISL_14230544", "EPI_ISL_14231739", "EPI_ISL_14231749", "EPI_ISL_14231751", "EPI_ISL_14232221", "EPI_ISL_14234017", "EPI_ISL_14236377", "EPI_ISL_14241722", "EPI_ISL_14243471", "EPI_ISL_14243503", "EPI_ISL_14243509", "EPI_ISL_14249979", "EPI_ISL_14257394", "EPI_ISL_14257902", "EPI_ISL_14257905", "EPI_ISL_14259114", "EPI_ISL_14259141", "EPI_ISL_14259905", "EPI_ISL_14260215", "EPI_ISL_14261704", "EPI_ISL_14262778", "EPI_ISL_14263077", "EPI_ISL_14277057", "EPI_ISL_14285562", "EPI_ISL_14287370", "EPI_ISL_14288695", "EPI_ISL_14289901", "EPI_ISL_14289906", "EPI_ISL_14292615", "EPI_ISL_14292645", "EPI_ISL_14292727", "EPI_ISL_14292796", "EPI_ISL_14296586", "EPI_ISL_14298637", "EPI_ISL_14299991", "EPI_ISL_14301376", "EPI_ISL_14305792", "EPI_ISL_14311909", "EPI_ISL_14311965", "EPI_ISL_14312743", "EPI_ISL_14321789", "EPI_ISL_14321842", "EPI_ISL_14329477", "EPI_ISL_14340832", "EPI_ISL_14353536", "EPI_ISL_14356738", "EPI_ISL_14359010", "EPI_ISL_14377405", "EPI_ISL_14377421", "EPI_ISL_14382623", "EPI_ISL_14386850", "EPI_ISL_14387989", "EPI_ISL_14389796", "EPI_ISL_14391372", "EPI_ISL_14393120", "EPI_ISL_14393933", "EPI_ISL_14409468", "EPI_ISL_14416474", "EPI_ISL_14417374", "EPI_ISL_14425116", "EPI_ISL_14425894", "EPI_ISL_14426235", "EPI_ISL_14426336", "EPI_ISL_14430592", "EPI_ISL_14433737", "EPI_ISL_14434989", "EPI_ISL_14436225", "EPI_ISL_14437098", "EPI_ISL_14439513", "EPI_ISL_14439514", "EPI_ISL_14439530", "EPI_ISL_14439649", "EPI_ISL_14439686", "EPI_ISL_14440238", "EPI_ISL_14448667", "EPI_ISL_14455168", "EPI_ISL_14459779", "EPI_ISL_14462783", "EPI_ISL_14464386", "EPI_ISL_14467169", "EPI_ISL_14469810", "EPI_ISL_14470997", "EPI_ISL_14471721", "EPI_ISL_14478172", "EPI_ISL_14478208", "EPI_ISL_14479146", "EPI_ISL_14479735", "EPI_ISL_14483275", "EPI_ISL_14485183", "EPI_ISL_14485890", "EPI_ISL_14487304", "EPI_ISL_14487315", "EPI_ISL_14490179", "EPI_ISL_14493139", "EPI_ISL_14493608", "EPI_ISL_14493822", "EPI_ISL_14493989", "EPI_ISL_14494098", "EPI_ISL_14494322", "EPI_ISL_14496407", "EPI_ISL_14496491", "EPI_ISL_14496822", "EPI_ISL_14497316", "EPI_ISL_14498244", "EPI_ISL_14502420", "EPI_ISL_14503169", "EPI_ISL_14503437", "EPI_ISL_14504973", "EPI_ISL_14505974", "EPI_ISL_14507199", "EPI_ISL_14507200", "EPI_ISL_14508231", "EPI_ISL_14509715", "EPI_ISL_14513137", "EPI_ISL_14518038", "EPI_ISL_14518039", "EPI_ISL_14518040", "EPI_ISL_14518101", "EPI_ISL_14518137", "EPI_ISL_14523885", "EPI_ISL_14527351", "EPI_ISL_14527924", "EPI_ISL_14535112", "EPI_ISL_14540192", "EPI_ISL_14544667", "EPI_ISL_14545270", "EPI_ISL_14551066", "EPI_ISL_14552116", "EPI_ISL_14553998", "EPI_ISL_14554016", "EPI_ISL_14554020", "EPI_ISL_14554042", "EPI_ISL_14554047", "EPI_ISL_14556650", "EPI_ISL_14560721", "EPI_ISL_14560826", "EPI_ISL_14560873", "EPI_ISL_14561036", "EPI_ISL_14561487", "EPI_ISL_14561987", "EPI_ISL_14562000", "EPI_ISL_14562820", "EPI_ISL_14571645", "EPI_ISL_14572777", "EPI_ISL_14573093", "EPI_ISL_14574572", "EPI_ISL_14577541", "EPI_ISL_14577981", "EPI_ISL_14578599", "EPI_ISL_14583728", "EPI_ISL_14596883", "EPI_ISL_14599772", "EPI_ISL_14599778", "EPI_ISL_14599779", "EPI_ISL_14599780", "EPI_ISL_14602583", "EPI_ISL_14602992", "EPI_ISL_14606016", "EPI_ISL_14608770", "EPI_ISL_14610722", "EPI_ISL_14610723", "EPI_ISL_14610724", "EPI_ISL_14610725", "EPI_ISL_14610726", "EPI_ISL_14610727", "EPI_ISL_14610728", "EPI_ISL_14610729", "EPI_ISL_14610730", "EPI_ISL_14610731", "EPI_ISL_14610732", "EPI_ISL_14610733", "EPI_ISL_14610734", "EPI_ISL_14613632", "EPI_ISL_14613671", "EPI_ISL_14616144", "EPI_ISL_14616443", "EPI_ISL_14616543", "EPI_ISL_14616681", "EPI_ISL_14618772", "EPI_ISL_14619952", "EPI_ISL_14623599", "EPI_ISL_14624407", "EPI_ISL_14625263", "EPI_ISL_14630170", "EPI_ISL_14647032", "EPI_ISL_14650043", "EPI_ISL_14652006", "EPI_ISL_14665394", "EPI_ISL_14666760", "EPI_ISL_14666762", "EPI_ISL_14666764", "EPI_ISL_14666765", "EPI_ISL_14666766", "EPI_ISL_14667656", "EPI_ISL_14667834", "EPI_ISL_14669160", "EPI_ISL_14676287", "EPI_ISL_14681429", "EPI_ISL_14683500", "EPI_ISL_14687471", "EPI_ISL_14691921", "EPI_ISL_14694460", "EPI_ISL_14695466", "EPI_ISL_14699501", "EPI_ISL_14699555", "EPI_ISL_14700183", "EPI_ISL_14700285", "EPI_ISL_14701161", "EPI_ISL_14701325", "EPI_ISL_14702629", "EPI_ISL_14706169", "EPI_ISL_14707196", "EPI_ISL_14707197", "EPI_ISL_14707948", "EPI_ISL_14710821", "EPI_ISL_14710834", "EPI_ISL_14711613", "EPI_ISL_14711614", "EPI_ISL_14715522", "EPI_ISL_14721837", "EPI_ISL_14721894", "EPI_ISL_14722630", "EPI_ISL_14725600", "EPI_ISL_14727457", "EPI_ISL_14728608", "EPI_ISL_14728814", "EPI_ISL_14732990", "EPI_ISL_14744620", "EPI_ISL_14744804", "EPI_ISL_14744809", "EPI_ISL_14745146", "EPI_ISL_14746124", "EPI_ISL_14746196", "EPI_ISL_14746271", "EPI_ISL_14746905", "EPI_ISL_14747246", "EPI_ISL_14747247", "EPI_ISL_14747621", "EPI_ISL_14752384", "EPI_ISL_14754570", "EPI_ISL_14755727", "EPI_ISL_14755766", "EPI_ISL_14755933", "EPI_ISL_14762572", "EPI_ISL_14763447", "EPI_ISL_14763711", "EPI_ISL_14763898", "EPI_ISL_14765653", "EPI_ISL_14766331", "EPI_ISL_14766361", "EPI_ISL_14766363", "EPI_ISL_14766433", "EPI_ISL_14770484", "EPI_ISL_14771903", "EPI_ISL_14772260", "EPI_ISL_14773203", "EPI_ISL_14773569", "EPI_ISL_14773570", "EPI_ISL_14773839", "EPI_ISL_14778614", "EPI_ISL_14780448", "EPI_ISL_14785887", "EPI_ISL_14786677", "EPI_ISL_14788048", "EPI_ISL_14788099", "EPI_ISL_14788860", "EPI_ISL_14789391", "EPI_ISL_14789392", "EPI_ISL_14789508", "EPI_ISL_14791420", "EPI_ISL_14793146", "EPI_ISL_14793618", "EPI_ISL_14796633", "EPI_ISL_14804570", "EPI_ISL_14806018", "EPI_ISL_14806413", "EPI_ISL_14809350", "EPI_ISL_14811078", "EPI_ISL_14812412", "EPI_ISL_14813068", "EPI_ISL_14813161", "EPI_ISL_14813215", "EPI_ISL_14813300", "EPI_ISL_14813995", "EPI_ISL_14816346", "EPI_ISL_14817985", "EPI_ISL_14832276", "EPI_ISL_14832977", "EPI_ISL_14836181", "EPI_ISL_14837867", "EPI_ISL_14838049", "EPI_ISL_14841625", "EPI_ISL_14842196", "EPI_ISL_14845057", "EPI_ISL_14846399", "EPI_ISL_14846400", "EPI_ISL_14847727", "EPI_ISL_14856139", "EPI_ISL_14859457", "EPI_ISL_14859713", "EPI_ISL_14859716", "EPI_ISL_14859829", "EPI_ISL_14862263", "EPI_ISL_14886333", "EPI_ISL_14888736", "EPI_ISL_14890020", "EPI_ISL_14891391", "EPI_ISL_14891763", "EPI_ISL_14891765", "EPI_ISL_14892114", "EPI_ISL_14892395", "EPI_ISL_14892970", "EPI_ISL_14896074", "EPI_ISL_14901195", "EPI_ISL_14901198", "EPI_ISL_14901422", "EPI_ISL_14901439", "EPI_ISL_14901444", "EPI_ISL_14901672", "EPI_ISL_14901693", "EPI_ISL_14903212", "EPI_ISL_14904331", "EPI_ISL_14907633", "EPI_ISL_14912863", "EPI_ISL_14913043", "EPI_ISL_14916417", "EPI_ISL_14919989", "EPI_ISL_14920419", "EPI_ISL_14921805", "EPI_ISL_14922117", "EPI_ISL_14922327", "EPI_ISL_14922929", "EPI_ISL_14922954", "EPI_ISL_14924448", "EPI_ISL_14925471", "EPI_ISL_14925480", "EPI_ISL_14925483", "EPI_ISL_14925487", "EPI_ISL_14926371", "EPI_ISL_14929774", "EPI_ISL_14931103", "EPI_ISL_14934229", "EPI_ISL_14934234", "EPI_ISL_14935361", "EPI_ISL_14935895", "EPI_ISL_14935908", "EPI_ISL_14935930", "EPI_ISL_14935931", "EPI_ISL_14937654", "EPI_ISL_14937852", "EPI_ISL_14937864", "EPI_ISL_14942094", "EPI_ISL_14942184", "EPI_ISL_14942530", "EPI_ISL_14943290", "EPI_ISL_14945818", "EPI_ISL_14946958", "EPI_ISL_14949065", "EPI_ISL_14950282", "EPI_ISL_14950545", "EPI_ISL_14951595", "EPI_ISL_14951609", "EPI_ISL_14951892", "EPI_ISL_14952059", "EPI_ISL_14952205", "EPI_ISL_14952220", "EPI_ISL_14953487", "EPI_ISL_14955378", "EPI_ISL_14959923", "EPI_ISL_14960752", "EPI_ISL_14960911", "EPI_ISL_14961972", "EPI_ISL_14962212", "EPI_ISL_14962429", "EPI_ISL_14962617", "EPI_ISL_14971640", "EPI_ISL_14975894", "EPI_ISL_14977865", "EPI_ISL_14980656", "EPI_ISL_14992324", "EPI_ISL_14993023", "EPI_ISL_14994380", "EPI_ISL_14995958", "EPI_ISL_14997830", "EPI_ISL_15003867", "EPI_ISL_15005362", "EPI_ISL_15005560", "EPI_ISL_15010697", "EPI_ISL_15012784", "EPI_ISL_15013151", "EPI_ISL_15013344", "EPI_ISL_15014035", "EPI_ISL_15014516", "EPI_ISL_15015307", "EPI_ISL_15017244", "EPI_ISL_15018502", "EPI_ISL_15020427", "EPI_ISL_15022783", "EPI_ISL_15022788", "EPI_ISL_15024848", "EPI_ISL_15026124", "EPI_ISL_15030291", "EPI_ISL_15030370", "EPI_ISL_15032101", "EPI_ISL_15036387", "EPI_ISL_15038137", "EPI_ISL_15040430", "EPI_ISL_15040463", "EPI_ISL_15040845", "EPI_ISL_15040855", "EPI_ISL_15040867", "EPI_ISL_15044102", "EPI_ISL_15047139", "EPI_ISL_15048524", "EPI_ISL_15050379", "EPI_ISL_15051633", "EPI_ISL_15058728", "EPI_ISL_15058903", "EPI_ISL_15071928", "EPI_ISL_15072261", "EPI_ISL_15072543", "EPI_ISL_15072550", "EPI_ISL_15072553", "EPI_ISL_15072554", "EPI_ISL_15072999", "EPI_ISL_15075043", "EPI_ISL_15075836", "EPI_ISL_15076071", "EPI_ISL_15077422", "EPI_ISL_15078481", "EPI_ISL_15080832", "EPI_ISL_15083622", "EPI_ISL_15084091", "EPI_ISL_15085357", "EPI_ISL_15085883", "EPI_ISL_15085910", "EPI_ISL_15086100", "EPI_ISL_15086132", "EPI_ISL_15086246", "EPI_ISL_15088435", "EPI_ISL_15088854", "EPI_ISL_15090098", "EPI_ISL_15093244", "EPI_ISL_15093817", "EPI_ISL_15093818", "EPI_ISL_15094085", "EPI_ISL_15096597", "EPI_ISL_15096672", "EPI_ISL_15098367", "EPI_ISL_15101602", "EPI_ISL_15107059", "EPI_ISL_15107248", "EPI_ISL_15107529", "EPI_ISL_15108940", "EPI_ISL_15108982", "EPI_ISL_15109913", "EPI_ISL_15111016", "EPI_ISL_15111369", "EPI_ISL_15114528", "EPI_ISL_15114696", "EPI_ISL_15116712", "EPI_ISL_15118484", "EPI_ISL_15119416", "EPI_ISL_15120575", "EPI_ISL_15125352", "EPI_ISL_15126616", "EPI_ISL_15129252", "EPI_ISL_15137908", "EPI_ISL_15137948", "EPI_ISL_15140027", "EPI_ISL_15140068", "EPI_ISL_15145892", "EPI_ISL_15145981", "EPI_ISL_15156399", "EPI_ISL_15157666", "EPI_ISL_15159584", "EPI_ISL_15160596", "EPI_ISL_15161674", "EPI_ISL_15169761", "EPI_ISL_15169791", "EPI_ISL_15170512", "EPI_ISL_15171102", "EPI_ISL_15172949", "EPI_ISL_15173621", "EPI_ISL_15175083", "EPI_ISL_15177304", "EPI_ISL_15177330", "EPI_ISL_15177334", "EPI_ISL_15178067", "EPI_ISL_15178215", "EPI_ISL_15184076", "EPI_ISL_15184330", "EPI_ISL_15191490", "EPI_ISL_15191491", "EPI_ISL_15191642", "EPI_ISL_15191714", "EPI_ISL_15191804", "EPI_ISL_15193406", "EPI_ISL_15195342", "EPI_ISL_15195410", "EPI_ISL_15195634", "EPI_ISL_15195645", "EPI_ISL_15195724", "EPI_ISL_15198987", "EPI_ISL_15211295", "EPI_ISL_15211305", "EPI_ISL_15213088", "EPI_ISL_15215446", "EPI_ISL_15215974", "EPI_ISL_15216639", "EPI_ISL_15218165", "EPI_ISL_15222709", "EPI_ISL_15229199", "EPI_ISL_15231108", "EPI_ISL_15236061", "EPI_ISL_15236355", "EPI_ISL_15241555", "EPI_ISL_15247582", "EPI_ISL_15248681", "EPI_ISL_15250551", "EPI_ISL_15251240", "EPI_ISL_15251241", "EPI_ISL_15251242", "EPI_ISL_15251243", "EPI_ISL_15257404", "EPI_ISL_15264055", "EPI_ISL_15268715", "EPI_ISL_15268834", "EPI_ISL_15271184", "EPI_ISL_15273578", "EPI_ISL_15275240", "EPI_ISL_15278730", "EPI_ISL_15278787", "EPI_ISL_15279743", "EPI_ISL_15284364", "EPI_ISL_15284373", "EPI_ISL_15284374", "EPI_ISL_15284586", "EPI_ISL_15286527", "EPI_ISL_15287393", "EPI_ISL_15288333", "EPI_ISL_15292331", "EPI_ISL_15294656", "EPI_ISL_15296403", "EPI_ISL_15305368", "EPI_ISL_15305862", "EPI_ISL_15306067", "EPI_ISL_15307010", "EPI_ISL_15307651", "EPI_ISL_15310561", "EPI_ISL_15312119", "EPI_ISL_15314949", "EPI_ISL_15316398", "EPI_ISL_15317884", "EPI_ISL_15322793", "EPI_ISL_15325687", "EPI_ISL_15325946", "EPI_ISL_15328668", "EPI_ISL_15330077", "EPI_ISL_15330418", "EPI_ISL_15331994", "EPI_ISL_15332094", "EPI_ISL_15332912", "EPI_ISL_15333287", "EPI_ISL_15333310", "EPI_ISL_15338015", "EPI_ISL_15338081", "EPI_ISL_15340355", "EPI_ISL_15341321", "EPI_ISL_15347054", "EPI_ISL_15348674", "EPI_ISL_15348926", "EPI_ISL_15348927", "EPI_ISL_15348928", "EPI_ISL_15348929", "EPI_ISL_15352149", "EPI_ISL_15354679", "EPI_ISL_15354775", "EPI_ISL_15357057", "EPI_ISL_15357237", "EPI_ISL_15357960", "EPI_ISL_15357988", "EPI_ISL_15362650", "EPI_ISL_15363544", "EPI_ISL_15368893", "EPI_ISL_15370137", "EPI_ISL_15370885", "EPI_ISL_15370889", "EPI_ISL_15376124", "EPI_ISL_15376348", "EPI_ISL_15376376", "EPI_ISL_15379719", "EPI_ISL_15380518", "EPI_ISL_15384507", "EPI_ISL_15384545", "EPI_ISL_15385232", "EPI_ISL_15385971", "EPI_ISL_15387248", "EPI_ISL_15387296", "EPI_ISL_15387509", "EPI_ISL_15387686", "EPI_ISL_15388463", "EPI_ISL_15389278", "EPI_ISL_15392529", "EPI_ISL_15393294", "EPI_ISL_15393302", "EPI_ISL_15396381", "EPI_ISL_15397576", "EPI_ISL_15403852", "EPI_ISL_15403865", "EPI_ISL_15403898", "EPI_ISL_15408226", "EPI_ISL_15408697", "EPI_ISL_15409673", "EPI_ISL_15415649", "EPI_ISL_15416349", "EPI_ISL_15420040", "EPI_ISL_15420138", "EPI_ISL_15420212", "EPI_ISL_15420431", "EPI_ISL_15420631", "EPI_ISL_15423234", "EPI_ISL_15424211", "EPI_ISL_15424276", "EPI_ISL_15424884", "EPI_ISL_15427653", "EPI_ISL_15434634", "EPI_ISL_15434919", "EPI_ISL_15435185", "EPI_ISL_15436140", "EPI_ISL_15436494", "EPI_ISL_15436498", "EPI_ISL_15436499", "EPI_ISL_15442625", "EPI_ISL_15442735", "EPI_ISL_15446553", "EPI_ISL_15456143", "EPI_ISL_15462878", "EPI_ISL_15463120", "EPI_ISL_15471419", "EPI_ISL_15471420", "EPI_ISL_15472394", "EPI_ISL_15472759", "EPI_ISL_15473942", "EPI_ISL_15473981", "EPI_ISL_15473994", "EPI_ISL_15476105", "EPI_ISL_15476158", "EPI_ISL_15476180", "EPI_ISL_15476724", "EPI_ISL_15479591", "EPI_ISL_15481002", "EPI_ISL_15486348", "EPI_ISL_15490572", "EPI_ISL_15490728", "EPI_ISL_15492743", "EPI_ISL_15492887", "EPI_ISL_15494260", "EPI_ISL_15494897", "EPI_ISL_15495028", "EPI_ISL_15495125", "EPI_ISL_15496641", "EPI_ISL_15501222", "EPI_ISL_15502864", "EPI_ISL_15505215", "EPI_ISL_15505985", "EPI_ISL_15506333", "EPI_ISL_15507204", "EPI_ISL_15507296", "EPI_ISL_15507616", "EPI_ISL_15508143", "EPI_ISL_15509746", "EPI_ISL_15509755", "EPI_ISL_15509992", "EPI_ISL_15511119", "EPI_ISL_15511841", "EPI_ISL_15511842", "EPI_ISL_15511843", "EPI_ISL_15513583", "EPI_ISL_15513588", "EPI_ISL_15513663", "EPI_ISL_15514216", "EPI_ISL_15514265", "EPI_ISL_15514302", "EPI_ISL_15514923", "EPI_ISL_15523458", "EPI_ISL_15523591", "EPI_ISL_15528152", "EPI_ISL_15528174", "EPI_ISL_15528328", "EPI_ISL_15528329", "EPI_ISL_15528330", "EPI_ISL_15528331", "EPI_ISL_15528332", "EPI_ISL_15528333", "EPI_ISL_15528334", "EPI_ISL_15535800", "EPI_ISL_15537619", "EPI_ISL_15538513", "EPI_ISL_15538645", "EPI_ISL_15542503", "EPI_ISL_15546790", "EPI_ISL_15549778", "EPI_ISL_15549981", "EPI_ISL_15550525", "EPI_ISL_15557544", "EPI_ISL_15568780", "EPI_ISL_15579728", "EPI_ISL_15579786", "EPI_ISL_15580359", "EPI_ISL_15580699", "EPI_ISL_15581446", "EPI_ISL_15581681", "EPI_ISL_15581931", "EPI_ISL_15581932", "EPI_ISL_15581939", "EPI_ISL_15582076", "EPI_ISL_15582517", "EPI_ISL_15583376", "EPI_ISL_15585338", "EPI_ISL_15587950", "EPI_ISL_15588132", "EPI_ISL_15594682", "EPI_ISL_15595518", "EPI_ISL_15598104", "EPI_ISL_15598966", "EPI_ISL_15602198", "EPI_ISL_15604595", "EPI_ISL_15606951", "EPI_ISL_15607872", "EPI_ISL_15608835", "EPI_ISL_15609106", "EPI_ISL_15609107", "EPI_ISL_15610726", "EPI_ISL_15610881", "EPI_ISL_15612047", "EPI_ISL_15612048", "EPI_ISL_15614101", "EPI_ISL_15614116", "EPI_ISL_15614150", "EPI_ISL_15614383", "EPI_ISL_15614456", "EPI_ISL_15614490", "EPI_ISL_15616889", "EPI_ISL_15617621", "EPI_ISL_15617635", "EPI_ISL_15619675", "EPI_ISL_15626705", "EPI_ISL_15626859", "EPI_ISL_15626955", "EPI_ISL_15628252", "EPI_ISL_15630041", "EPI_ISL_15631537", "EPI_ISL_15635022", "EPI_ISL_15636641", "EPI_ISL_15637121", "EPI_ISL_15637182", "EPI_ISL_15638597", "EPI_ISL_15639067", "EPI_ISL_15642936", "EPI_ISL_15642980", "EPI_ISL_15649157", "EPI_ISL_15650076", "EPI_ISL_15650225", "EPI_ISL_15653695", "EPI_ISL_15654640", "EPI_ISL_15656922", "EPI_ISL_15659847", "EPI_ISL_15661609", "EPI_ISL_15666595", "EPI_ISL_15667047", "EPI_ISL_15669004", "EPI_ISL_15671244", "EPI_ISL_15671388", "EPI_ISL_15671577", "EPI_ISL_15671878", "EPI_ISL_15671888", "EPI_ISL_15673934", "EPI_ISL_15675248", "EPI_ISL_15678339", "EPI_ISL_15681826", "EPI_ISL_15685722", "EPI_ISL_15685724", "EPI_ISL_15685810", "EPI_ISL_15687681", "EPI_ISL_15687965", "EPI_ISL_15688500", "EPI_ISL_15692625", "EPI_ISL_15693169", "EPI_ISL_15693174", "EPI_ISL_15693676", "EPI_ISL_15700160", "EPI_ISL_15703834", "EPI_ISL_15705061", "EPI_ISL_15712450", "EPI_ISL_15712747", "EPI_ISL_15715244", "EPI_ISL_15719141", "EPI_ISL_15719142", "EPI_ISL_15719143", "EPI_ISL_15720605", "EPI_ISL_15721137", "EPI_ISL_15721185", "EPI_ISL_15721190", "EPI_ISL_15723589", "EPI_ISL_15725799", "EPI_ISL_15728467", "EPI_ISL_15728613", "EPI_ISL_15728673", "EPI_ISL_15729287", "EPI_ISL_15729288", "EPI_ISL_15729308", "EPI_ISL_15729309", "EPI_ISL_15729310", "EPI_ISL_15729311", "EPI_ISL_15729315", "EPI_ISL_15729341", "EPI_ISL_15729358", "EPI_ISL_15731233", "EPI_ISL_15731409", "EPI_ISL_15732413", "EPI_ISL_15736424", "EPI_ISL_15739498", "EPI_ISL_15739617", "EPI_ISL_15741410", "EPI_ISL_15742450", "EPI_ISL_15743318", "EPI_ISL_15743816", "EPI_ISL_15749060", "EPI_ISL_15754145", "EPI_ISL_15754794", "EPI_ISL_15758565", "EPI_ISL_15758745", "EPI_ISL_15760224", "EPI_ISL_15760382", "EPI_ISL_15760554", "EPI_ISL_15760609", "EPI_ISL_15760812", "EPI_ISL_15761520", "EPI_ISL_15761543", "EPI_ISL_15761663", "EPI_ISL_15763216", "EPI_ISL_15765022", "EPI_ISL_15768827", "EPI_ISL_15773132", "EPI_ISL_15773881", "EPI_ISL_15773907", "EPI_ISL_15776989", "EPI_ISL_15778527", "EPI_ISL_15779724", "EPI_ISL_15780387", "EPI_ISL_15781197", "EPI_ISL_15781220", "EPI_ISL_15781937", "EPI_ISL_15785782", "EPI_ISL_15786114", "EPI_ISL_15786255", "EPI_ISL_15789537", "EPI_ISL_15790179", "EPI_ISL_15790657", "EPI_ISL_15791223", "EPI_ISL_15791252", "EPI_ISL_15792351", "EPI_ISL_15793981", "EPI_ISL_15797751", "EPI_ISL_15798331", "EPI_ISL_15801425", "EPI_ISL_15801499", "EPI_ISL_15801515", "EPI_ISL_15803807", "EPI_ISL_15815337", "EPI_ISL_15815525", "EPI_ISL_15815889", "EPI_ISL_15818486", "EPI_ISL_15820055", "EPI_ISL_15822919", "EPI_ISL_15824080", "EPI_ISL_15824099", "EPI_ISL_15824207", "EPI_ISL_15825627", "EPI_ISL_15826800", "EPI_ISL_15826867", "EPI_ISL_15829108", "EPI_ISL_15837751", "EPI_ISL_15837827", "EPI_ISL_15838124", "EPI_ISL_15839941", "EPI_ISL_15840082", "EPI_ISL_15843473", "EPI_ISL_15843671", "EPI_ISL_15844032", "EPI_ISL_15844165", "EPI_ISL_15845753", "EPI_ISL_15845778", "EPI_ISL_15845946", "EPI_ISL_15846023", "EPI_ISL_15846264", "EPI_ISL_15849690", "EPI_ISL_15850759", "EPI_ISL_15850865", "EPI_ISL_15850872", "EPI_ISL_15853809", "EPI_ISL_15853943", "EPI_ISL_15856103", "EPI_ISL_15856463", "EPI_ISL_15856822", "EPI_ISL_15857383", "EPI_ISL_15857468", "EPI_ISL_15860163", "EPI_ISL_15864217", "EPI_ISL_15864218", "EPI_ISL_15865257", "EPI_ISL_15865301", "EPI_ISL_15865421", "EPI_ISL_15865482", "EPI_ISL_15866887", "EPI_ISL_15873398", "EPI_ISL_15874567", "EPI_ISL_15878818", "EPI_ISL_15882637", "EPI_ISL_15883009", "EPI_ISL_15883480", "EPI_ISL_15887656", "EPI_ISL_15894376", "EPI_ISL_15895625", "EPI_ISL_15896804", "EPI_ISL_15896845", "EPI_ISL_15896923", "EPI_ISL_15897067", "EPI_ISL_15897092", "EPI_ISL_15898992", "EPI_ISL_15900796", "EPI_ISL_15905452", "EPI_ISL_15905511", "EPI_ISL_15910198", "EPI_ISL_15911160", "EPI_ISL_15912221", "EPI_ISL_15912222", "EPI_ISL_15912223", "EPI_ISL_15912224", "EPI_ISL_15914119", "EPI_ISL_15917272", "EPI_ISL_15920181", "EPI_ISL_15920505", "EPI_ISL_15920753", "EPI_ISL_15920754", "EPI_ISL_15920755", "EPI_ISL_15926083", "EPI_ISL_15926723", "EPI_ISL_15928156", "EPI_ISL_15928909", "EPI_ISL_15929002", "EPI_ISL_15929151", "EPI_ISL_15932554", "EPI_ISL_15934274", "EPI_ISL_15937718", "EPI_ISL_15938074", "EPI_ISL_15941879", "EPI_ISL_15941880", "EPI_ISL_15945504", "EPI_ISL_15949533", "EPI_ISL_15953870", "EPI_ISL_15955458", "EPI_ISL_15958934", "EPI_ISL_15959369", "EPI_ISL_15961456", "EPI_ISL_15962045", "EPI_ISL_15965716", "EPI_ISL_15966131", "EPI_ISL_15966527", "EPI_ISL_15969420", "EPI_ISL_15969421", "EPI_ISL_15969437", "EPI_ISL_15969438", "EPI_ISL_15970088", "EPI_ISL_15970187", "EPI_ISL_15976036", "EPI_ISL_15982641", "EPI_ISL_15984958", "EPI_ISL_15992803", "EPI_ISL_15998627", "EPI_ISL_16001974", "EPI_ISL_16001995", "EPI_ISL_16003255", "EPI_ISL_16005246", "EPI_ISL_16005457", "EPI_ISL_16006665", "EPI_ISL_16007931", "EPI_ISL_16008877", "EPI_ISL_16012424", "EPI_ISL_16012845", "EPI_ISL_16013074", "EPI_ISL_16013086", "EPI_ISL_16015099", "EPI_ISL_16017107", "EPI_ISL_16018930", "EPI_ISL_16019056", "EPI_ISL_16019744", "EPI_ISL_16019756", "EPI_ISL_16024407", "EPI_ISL_16024682", "EPI_ISL_16027431", "EPI_ISL_16027937", "EPI_ISL_16029135", "EPI_ISL_16029382", "EPI_ISL_16029654", "EPI_ISL_16030181", "EPI_ISL_16033087", "EPI_ISL_16033186", "EPI_ISL_16034832", "EPI_ISL_16036598", "EPI_ISL_16039444", "EPI_ISL_16043974", "EPI_ISL_16044428", "EPI_ISL_16045410", "EPI_ISL_16045424", "EPI_ISL_16046711", "EPI_ISL_16050095", "EPI_ISL_16050127", "EPI_ISL_16054133", "EPI_ISL_16054451", "EPI_ISL_16054953", "EPI_ISL_16054963", "EPI_ISL_16055434", "EPI_ISL_16055461", "EPI_ISL_16055527", "EPI_ISL_16055697", "EPI_ISL_16055721", "EPI_ISL_16056123", "EPI_ISL_16057031", "EPI_ISL_16060790", "EPI_ISL_16062229", "EPI_ISL_16064284", "EPI_ISL_16066333", "EPI_ISL_16068281", "EPI_ISL_16068583", "EPI_ISL_16068914", "EPI_ISL_16073080", "EPI_ISL_16073469", "EPI_ISL_16073474", "EPI_ISL_16074871", "EPI_ISL_16075086", "EPI_ISL_16075127", "EPI_ISL_16079016", "EPI_ISL_16080170", "EPI_ISL_16080871", "EPI_ISL_16082206", "EPI_ISL_16091870", "EPI_ISL_16101516", "EPI_ISL_16102479", "EPI_ISL_16102480", "EPI_ISL_16109108", "EPI_ISL_16111875", "EPI_ISL_16113331", "EPI_ISL_16113603", "EPI_ISL_16113806", "EPI_ISL_16114505", "EPI_ISL_16114631", "EPI_ISL_16115703", "EPI_ISL_16116190", "EPI_ISL_16116234", "EPI_ISL_16116659", "EPI_ISL_16116707", "EPI_ISL_16119498", "EPI_ISL_16119508", "EPI_ISL_16119512", "EPI_ISL_16119517", "EPI_ISL_16119519", "EPI_ISL_16119805", "EPI_ISL_16131965", "EPI_ISL_16131986", "EPI_ISL_16131997", "EPI_ISL_16136901", "EPI_ISL_16137616", "EPI_ISL_16151030", "EPI_ISL_16151463", "EPI_ISL_16151651", "EPI_ISL_16153650", "EPI_ISL_16153658", "EPI_ISL_16153800", "EPI_ISL_16157875", "EPI_ISL_16158109", "EPI_ISL_16158197", "EPI_ISL_16158326", "EPI_ISL_16158363", "EPI_ISL_16158374", "EPI_ISL_16160252", "EPI_ISL_16160296", "EPI_ISL_16160313", "EPI_ISL_16165250", "EPI_ISL_16167761", "EPI_ISL_16178283", "EPI_ISL_16178634", "EPI_ISL_16178844", "EPI_ISL_16179355", "EPI_ISL_16180564", "EPI_ISL_16180574", "EPI_ISL_16181797", "EPI_ISL_16181828", "EPI_ISL_16181950", "EPI_ISL_16183022", "EPI_ISL_16190977", "EPI_ISL_16191476", "EPI_ISL_16196150", "EPI_ISL_16196167", "EPI_ISL_16197958", "EPI_ISL_16201173", "EPI_ISL_16204855", "EPI_ISL_16215808", "EPI_ISL_16218191", "EPI_ISL_16219709", "EPI_ISL_16219753", "EPI_ISL_16221691", "EPI_ISL_16224408", "EPI_ISL_16230801", "EPI_ISL_16231330", "EPI_ISL_16231873", "EPI_ISL_16233000", "EPI_ISL_16233650", "EPI_ISL_16233865", "EPI_ISL_16234790", "EPI_ISL_16235313", "EPI_ISL_16235462", "EPI_ISL_16235523", "EPI_ISL_16235930", "EPI_ISL_16240521", "EPI_ISL_16244367", "EPI_ISL_16244373", "EPI_ISL_16244408", "EPI_ISL_16244923", "EPI_ISL_16245232", "EPI_ISL_16245289", "EPI_ISL_16245329", "EPI_ISL_16245400", "EPI_ISL_16245414", "EPI_ISL_16245433", "EPI_ISL_16245601", "EPI_ISL_16245627", "EPI_ISL_16247208", "EPI_ISL_16247263", "EPI_ISL_16247490", "EPI_ISL_16247545", "EPI_ISL_16247708", "EPI_ISL_16251186", "EPI_ISL_16251466", "EPI_ISL_16251579", "EPI_ISL_16257294", "EPI_ISL_16259227", "EPI_ISL_16264400", "EPI_ISL_16265325", "EPI_ISL_16268074", "EPI_ISL_16270258", "EPI_ISL_16271245", "EPI_ISL_16271444", "EPI_ISL_16271604", "EPI_ISL_16273936", "EPI_ISL_16281373", "EPI_ISL_16284103", "EPI_ISL_16284207", "EPI_ISL_16284311", "EPI_ISL_16287253", "EPI_ISL_16287690", "EPI_ISL_16290877", "EPI_ISL_16293662", "EPI_ISL_16312661", "EPI_ISL_16327295", "EPI_ISL_16331524", "EPI_ISL_16334679", "EPI_ISL_16338847", "EPI_ISL_16338862", "EPI_ISL_16343084", "EPI_ISL_16343221", "EPI_ISL_16344593", "EPI_ISL_16348840", "EPI_ISL_16348868", "EPI_ISL_16351967", "EPI_ISL_16354229", "EPI_ISL_16355537", "EPI_ISL_16356453", "EPI_ISL_16356458", "EPI_ISL_16356910", "EPI_ISL_16358915", "EPI_ISL_16359990", "EPI_ISL_16360495", "EPI_ISL_16365715", "EPI_ISL_16368903", "EPI_ISL_16369869", "EPI_ISL_16370037", "EPI_ISL_16375356", "EPI_ISL_16378181", "EPI_ISL_16379359", "EPI_ISL_16380313", "EPI_ISL_16381332", "EPI_ISL_16381679", "EPI_ISL_16384522", "EPI_ISL_16385377", "EPI_ISL_16385455", "EPI_ISL_16385456", "EPI_ISL_16391752", "EPI_ISL_16394844", "EPI_ISL_16394922", "EPI_ISL_16395667", "EPI_ISL_16398472", "EPI_ISL_16399824", "EPI_ISL_16400033", "EPI_ISL_16400342", "EPI_ISL_16402001", "EPI_ISL_16402612", "EPI_ISL_16413336", "EPI_ISL_16414127", "EPI_ISL_16414190", "EPI_ISL_16422834", "EPI_ISL_16424130", "EPI_ISL_16425691", "EPI_ISL_16428100", "EPI_ISL_16428101", "EPI_ISL_16428102", "EPI_ISL_16429066", "EPI_ISL_16434308", "EPI_ISL_16435903", "EPI_ISL_16436533", "EPI_ISL_16439413", "EPI_ISL_16440696", "EPI_ISL_16443129", "EPI_ISL_16443688", "EPI_ISL_16443874", "EPI_ISL_16444178", "EPI_ISL_16444600", "EPI_ISL_16445144", "EPI_ISL_16446802", "EPI_ISL_16446804", "EPI_ISL_16452054", "EPI_ISL_16454044", "EPI_ISL_16460278", "EPI_ISL_16460823", "EPI_ISL_16461302", "EPI_ISL_16464657", "EPI_ISL_16467436", "EPI_ISL_16470710", "EPI_ISL_16470892", "EPI_ISL_16471153", "EPI_ISL_16471554", "EPI_ISL_16471730", "EPI_ISL_16471943", "EPI_ISL_16473435", "EPI_ISL_16473762", "EPI_ISL_16474400", "EPI_ISL_16475138", "EPI_ISL_16475139", "EPI_ISL_16479826", "EPI_ISL_16482060", "EPI_ISL_16489594", "EPI_ISL_16491494", "EPI_ISL_16492150", "EPI_ISL_16492585", "EPI_ISL_16492756", "EPI_ISL_16493396", "EPI_ISL_16493785", "EPI_ISL_16497702", "EPI_ISL_16498515", "EPI_ISL_16503949", "EPI_ISL_16507701", "EPI_ISL_16507896", "EPI_ISL_16507927", "EPI_ISL_16520597", "EPI_ISL_16520598", "EPI_ISL_16520637", "EPI_ISL_16520640", "EPI_ISL_16520641", "EPI_ISL_16520711", "EPI_ISL_16520763", "EPI_ISL_16520788", "EPI_ISL_16521101", "EPI_ISL_16524906", "EPI_ISL_16528641", "EPI_ISL_16528645", "EPI_ISL_16528903", "EPI_ISL_16534552", "EPI_ISL_16535376", "EPI_ISL_16536212", "EPI_ISL_16539692", "EPI_ISL_16541774", "EPI_ISL_16542381", "EPI_ISL_16542553", "EPI_ISL_16544506", "EPI_ISL_16553339", "EPI_ISL_16567779", "EPI_ISL_16574574", "EPI_ISL_16577679", "EPI_ISL_16577762", "EPI_ISL_16581578", "EPI_ISL_16584104", "EPI_ISL_16585126", "EPI_ISL_16585286", "EPI_ISL_16586683", "EPI_ISL_16586702", "EPI_ISL_16587574", "EPI_ISL_16597363", "EPI_ISL_16598790", "EPI_ISL_16598814", "EPI_ISL_16599846", "EPI_ISL_16607452", "EPI_ISL_16607500", "EPI_ISL_16607535", "EPI_ISL_16611498", "EPI_ISL_16611571", "EPI_ISL_16613287", "EPI_ISL_16613482", "EPI_ISL_16615201", "EPI_ISL_16615597", "EPI_ISL_16615617", "EPI_ISL_16615668", "EPI_ISL_16616642", "EPI_ISL_16625690", "EPI_ISL_16626611", "EPI_ISL_16626666", "EPI_ISL_16627067", "EPI_ISL_16628854", "EPI_ISL_16630260", "EPI_ISL_16630261", "EPI_ISL_16636917", "EPI_ISL_16637089", "EPI_ISL_16637607", "EPI_ISL_16637631", "EPI_ISL_16638190", "EPI_ISL_16638453", "EPI_ISL_16640597", "EPI_ISL_16643406", "EPI_ISL_16647535", "EPI_ISL_16647537", "EPI_ISL_16647544", "EPI_ISL_16647740", "EPI_ISL_16649988", "EPI_ISL_16650006", "EPI_ISL_16650011", "EPI_ISL_16653618", "EPI_ISL_16669313", "EPI_ISL_16669829", "EPI_ISL_16672282", "EPI_ISL_16672301", "EPI_ISL_16672327", "EPI_ISL_16672352", "EPI_ISL_16676267", "EPI_ISL_16677015", "EPI_ISL_16678917", "EPI_ISL_16678946", "EPI_ISL_16679654", "EPI_ISL_16681451", "EPI_ISL_16681917", "EPI_ISL_16682342", "EPI_ISL_16682837", "EPI_ISL_16688219", "EPI_ISL_16688525", "EPI_ISL_16688688", "EPI_ISL_16688713", "EPI_ISL_16689887", "EPI_ISL_16691397", "EPI_ISL_16691487", "EPI_ISL_16694176", "EPI_ISL_16695383", "EPI_ISL_16695435", "EPI_ISL_16697861", "EPI_ISL_16700160", "EPI_ISL_16702838", "EPI_ISL_16705882", "EPI_ISL_16706498", "EPI_ISL_16708798", "EPI_ISL_16711038", "EPI_ISL_16711095", "EPI_ISL_16711417", "EPI_ISL_16711531", "EPI_ISL_16716967", "EPI_ISL_16721930", "EPI_ISL_16722183", "EPI_ISL_16722215", "EPI_ISL_16722270", "EPI_ISL_16722970", "EPI_ISL_16723215", "EPI_ISL_16725887", "EPI_ISL_16727241", "EPI_ISL_16728257", "EPI_ISL_16728383", "EPI_ISL_16728411", "EPI_ISL_16729998", "EPI_ISL_16730248", "EPI_ISL_16731753", "EPI_ISL_16735633", "EPI_ISL_16736343", "EPI_ISL_16736346", "EPI_ISL_16736400", "EPI_ISL_16738641", "EPI_ISL_16739045", "EPI_ISL_16739452", "EPI_ISL_16740104", "EPI_ISL_16740406", "EPI_ISL_16741567", "EPI_ISL_16741573", "EPI_ISL_16749999", "EPI_ISL_16750878", "EPI_ISL_16751721", "EPI_ISL_16751722", "EPI_ISL_16751789", "EPI_ISL_16751791", "EPI_ISL_16751977", "EPI_ISL_16752073", "EPI_ISL_16752138", "EPI_ISL_16757168", "EPI_ISL_16757210", "EPI_ISL_16758981", "EPI_ISL_16764861", "EPI_ISL_16765888", "EPI_ISL_16766196", "EPI_ISL_16807740", "EPI_ISL_16807787", "EPI_ISL_16807844", "EPI_ISL_16811091", "EPI_ISL_16812565", "EPI_ISL_16815494", "EPI_ISL_16816293", "EPI_ISL_16818458", "EPI_ISL_16818471", "EPI_ISL_16825124", "EPI_ISL_16825157", "EPI_ISL_16825222", "EPI_ISL_16828876", "EPI_ISL_16828896", "EPI_ISL_16829188", "EPI_ISL_16831507", "EPI_ISL_16832953", "EPI_ISL_16833893", "EPI_ISL_16834974", "EPI_ISL_16834985", "EPI_ISL_16835399", "EPI_ISL_16842787", "EPI_ISL_16842790", "EPI_ISL_16847425", "EPI_ISL_16847642", "EPI_ISL_16847674", "EPI_ISL_16847675", "EPI_ISL_16847676", "EPI_ISL_16847677", "EPI_ISL_16847696", "EPI_ISL_16847697", "EPI_ISL_16853227", "EPI_ISL_16853229", "EPI_ISL_16853597", "EPI_ISL_16856355", "EPI_ISL_16856565", "EPI_ISL_16856637", "EPI_ISL_16856833", "EPI_ISL_16857514", "EPI_ISL_16857776", "EPI_ISL_16858310", "EPI_ISL_16858617", "EPI_ISL_16858667", "EPI_ISL_16861084", "EPI_ISL_16862130", "EPI_ISL_16863260", "EPI_ISL_16863802", "EPI_ISL_16866580", "EPI_ISL_16867553", "EPI_ISL_16868647", "EPI_ISL_16868654", "EPI_ISL_16868655", "EPI_ISL_16868993", "EPI_ISL_16869007", "EPI_ISL_16869905", "EPI_ISL_16875565", "EPI_ISL_16875752", "EPI_ISL_16876039", "EPI_ISL_16876784", "EPI_ISL_16877428", "EPI_ISL_16878720", "EPI_ISL_16880439", "EPI_ISL_16883240", "EPI_ISL_16883873", "EPI_ISL_16884622", "EPI_ISL_16890808", "EPI_ISL_16893283", "EPI_ISL_16894717", "EPI_ISL_16895138", "EPI_ISL_16895290", "EPI_ISL_16895522", "EPI_ISL_16897500", "EPI_ISL_16903492", "EPI_ISL_16903494", "EPI_ISL_16904444", "EPI_ISL_16904536", "EPI_ISL_16908472", "EPI_ISL_16909804", "EPI_ISL_16910025", "EPI_ISL_16910165", "EPI_ISL_16910272", "EPI_ISL_16911226", "EPI_ISL_16911909", "EPI_ISL_16913144", "EPI_ISL_16921530", "EPI_ISL_16921542", "EPI_ISL_16925257", "EPI_ISL_16927736", "EPI_ISL_16931901", "EPI_ISL_16939487", "EPI_ISL_16941750", "EPI_ISL_16941980", "EPI_ISL_16942000", "EPI_ISL_16942017", "EPI_ISL_16945429", "EPI_ISL_16946783", "EPI_ISL_16947592", "EPI_ISL_16947625", "EPI_ISL_16951592", "EPI_ISL_16953425", "EPI_ISL_16953741", "EPI_ISL_16954486", "EPI_ISL_16955471", "EPI_ISL_16957015", "EPI_ISL_16966997", "EPI_ISL_16967082", "EPI_ISL_16967083", "EPI_ISL_16967084", "EPI_ISL_16967085", "EPI_ISL_16967086", "EPI_ISL_16969756", "EPI_ISL_16969757", "EPI_ISL_16970279", "EPI_ISL_16973343", "EPI_ISL_16977317", "EPI_ISL_16977653", "EPI_ISL_16977749", "EPI_ISL_16979482", "EPI_ISL_16980683", "EPI_ISL_16981030", "EPI_ISL_16981047", "EPI_ISL_16981048", "EPI_ISL_16981102", "EPI_ISL_16987088", "EPI_ISL_16987375", "EPI_ISL_16987376", "EPI_ISL_16988527", "EPI_ISL_16989157", "EPI_ISL_16989160", "EPI_ISL_16995247", "EPI_ISL_16995491", "EPI_ISL_16995525", "EPI_ISL_16995951", "EPI_ISL_16997638", "EPI_ISL_17001974", "EPI_ISL_17001987", "EPI_ISL_17002442", "EPI_ISL_17006258", "EPI_ISL_17008393", "EPI_ISL_17008502", "EPI_ISL_17016219", "EPI_ISL_17016344", "EPI_ISL_17018731", "EPI_ISL_17020636", "EPI_ISL_17021187", "EPI_ISL_17022063", "EPI_ISL_17022081", "EPI_ISL_17024099", "EPI_ISL_17025560", "EPI_ISL_17025998", "EPI_ISL_17026052", "EPI_ISL_17026537", "EPI_ISL_17027430", "EPI_ISL_17032070", "EPI_ISL_17032664", "EPI_ISL_17035345", "EPI_ISL_17036551", "EPI_ISL_17037388", "EPI_ISL_17040117", "EPI_ISL_17040120", "EPI_ISL_17040127", "EPI_ISL_17040130", "EPI_ISL_17040131", "EPI_ISL_17040132", "EPI_ISL_17040133", "EPI_ISL_17040134", "EPI_ISL_17040184", "EPI_ISL_17041105", "EPI_ISL_17041117", "EPI_ISL_17041143", "EPI_ISL_17044002", "EPI_ISL_17046406", "EPI_ISL_17047667", "EPI_ISL_17050958", "EPI_ISL_17051743", "EPI_ISL_17056159", "EPI_ISL_17057279", "EPI_ISL_17061716", "EPI_ISL_17062081", "EPI_ISL_17065016", "EPI_ISL_17067007", "EPI_ISL_17068616", "EPI_ISL_17068621", "EPI_ISL_17073286", "EPI_ISL_17074720", "EPI_ISL_17075299", "EPI_ISL_17076011", "EPI_ISL_17076926", "EPI_ISL_17077233", "EPI_ISL_17077446", "EPI_ISL_17079150", "EPI_ISL_17079151", "EPI_ISL_17079427", "EPI_ISL_17080036", "EPI_ISL_17080146", "EPI_ISL_17080283", "EPI_ISL_17080510", "EPI_ISL_17081567", "EPI_ISL_17084330", "EPI_ISL_17086936", "EPI_ISL_17086958", "EPI_ISL_17090268", "EPI_ISL_17090730", "EPI_ISL_17092242", "EPI_ISL_17092260", "EPI_ISL_17099321", "EPI_ISL_17099446", "EPI_ISL_17101049", "EPI_ISL_17101231", "EPI_ISL_17104807", "EPI_ISL_17105674", "EPI_ISL_17105677", "EPI_ISL_17105777", "EPI_ISL_17105786", "EPI_ISL_17105804", "EPI_ISL_17106895", "EPI_ISL_17109738", "EPI_ISL_17109787", "EPI_ISL_17112198", "EPI_ISL_17113114", "EPI_ISL_17118740", "EPI_ISL_17119370", "EPI_ISL_17126699", "EPI_ISL_17126727", "EPI_ISL_17127510", "EPI_ISL_17129671", "EPI_ISL_17139969", "EPI_ISL_17144489", "EPI_ISL_17149697", "EPI_ISL_17150312", "EPI_ISL_17150571", "EPI_ISL_17152522", "EPI_ISL_17152602", "EPI_ISL_17152816", "EPI_ISL_17154843", "EPI_ISL_17154893", "EPI_ISL_17158601", "EPI_ISL_17158659", "EPI_ISL_17158660", "EPI_ISL_17158661", "EPI_ISL_17158662", "EPI_ISL_17158663", "EPI_ISL_17158664", "EPI_ISL_17158665", "EPI_ISL_17164529", "EPI_ISL_17165387", "EPI_ISL_17165528", "EPI_ISL_17170921", "EPI_ISL_17173754", "EPI_ISL_17174278", "EPI_ISL_17174323", "EPI_ISL_17175107", "EPI_ISL_17180776", "EPI_ISL_17182281", "EPI_ISL_17188691", "EPI_ISL_17188772", "EPI_ISL_17188836", "EPI_ISL_17189286", "EPI_ISL_17189360", "EPI_ISL_17189372", "EPI_ISL_17189865", "EPI_ISL_17190813", "EPI_ISL_17191784", "EPI_ISL_17193988", "EPI_ISL_17194121", "EPI_ISL_17194564", "EPI_ISL_17195807", "EPI_ISL_17198415", "EPI_ISL_17199165", "EPI_ISL_17199381", "EPI_ISL_17199743", "EPI_ISL_17200348", "EPI_ISL_17200520", "EPI_ISL_17201694", "EPI_ISL_17202051", "EPI_ISL_17205892", "EPI_ISL_17206016", "EPI_ISL_17206140", "EPI_ISL_17207424", "EPI_ISL_17210230", "EPI_ISL_17210689", "EPI_ISL_17214413", "EPI_ISL_17214693", "EPI_ISL_17214774", "EPI_ISL_17214805", "EPI_ISL_17214933", "EPI_ISL_17215427", "EPI_ISL_17215676", "EPI_ISL_17215686", "EPI_ISL_17215790", "EPI_ISL_17216822", "EPI_ISL_17216978", "EPI_ISL_17221710", "EPI_ISL_17222365", "EPI_ISL_17223438", "EPI_ISL_17226531", "EPI_ISL_17226637", "EPI_ISL_17232350", "EPI_ISL_17232448", "EPI_ISL_17237921", "EPI_ISL_17239049", "EPI_ISL_17239405", "EPI_ISL_17239499", "EPI_ISL_17241376", "EPI_ISL_17244668", "EPI_ISL_17245140", "EPI_ISL_17245198", "EPI_ISL_17245226", "EPI_ISL_17245255", "EPI_ISL_17246876", "EPI_ISL_17246931", "EPI_ISL_17247186", "EPI_ISL_17247325", "EPI_ISL_17247333", "EPI_ISL_17251028", "EPI_ISL_17252934", "EPI_ISL_17253364", "EPI_ISL_17253589", "EPI_ISL_17257608", "EPI_ISL_17262137", "EPI_ISL_17265160", "EPI_ISL_17270165", "EPI_ISL_17270215", "EPI_ISL_17270470", "EPI_ISL_17270618", "EPI_ISL_17270950", "EPI_ISL_17270964", "EPI_ISL_17270974", "EPI_ISL_17271226", "EPI_ISL_17271272", "EPI_ISL_17272946", "EPI_ISL_17273453", "EPI_ISL_17273454", "EPI_ISL_17273455", "EPI_ISL_17273456", "EPI_ISL_17275616", "EPI_ISL_17275984", "EPI_ISL_17276025", "EPI_ISL_17276098", "EPI_ISL_17276962", "EPI_ISL_17284010", "EPI_ISL_17284045", "EPI_ISL_17284573", "EPI_ISL_17284791", "EPI_ISL_17284792", "EPI_ISL_17284793", "EPI_ISL_17284794", "EPI_ISL_17284795", "EPI_ISL_17285690", "EPI_ISL_17288589", "EPI_ISL_17290740", "EPI_ISL_17292666", "EPI_ISL_17292834", "EPI_ISL_17297993", "EPI_ISL_17298321", "EPI_ISL_17298323", "EPI_ISL_17299688", "EPI_ISL_17300150", "EPI_ISL_17304801", "EPI_ISL_17304899", "EPI_ISL_17305358", "EPI_ISL_17319411", "EPI_ISL_17319528", "EPI_ISL_17319601", "EPI_ISL_17321362", "EPI_ISL_17322993", "EPI_ISL_17334027", "EPI_ISL_17342544", "EPI_ISL_17344004", "EPI_ISL_17344178", "EPI_ISL_17344660", "EPI_ISL_17345445", "EPI_ISL_17347577", "EPI_ISL_17348219", "EPI_ISL_17349770", "EPI_ISL_17349983", "EPI_ISL_17350301", "EPI_ISL_17352192", "EPI_ISL_17358767", "EPI_ISL_17359772", "EPI_ISL_17370155", "EPI_ISL_17371048", "EPI_ISL_17374605", "EPI_ISL_17374609", "EPI_ISL_17374807", "EPI_ISL_17376230", "EPI_ISL_17381216", "EPI_ISL_17387122", "EPI_ISL_17389140", "EPI_ISL_17389210", "EPI_ISL_17389223", "EPI_ISL_17389779", "EPI_ISL_17390660", "EPI_ISL_17390743", "EPI_ISL_17390873", "EPI_ISL_17391460", "EPI_ISL_17394837", "EPI_ISL_17397497", "EPI_ISL_17408352", "EPI_ISL_17409157", "EPI_ISL_17411543", "EPI_ISL_17414235", "EPI_ISL_17414543", "EPI_ISL_17418615", "EPI_ISL_17421962", "EPI_ISL_17423074", "EPI_ISL_17424014", "EPI_ISL_17429770", "EPI_ISL_17430458", "EPI_ISL_17430487", "EPI_ISL_17431238", "EPI_ISL_17434223", "EPI_ISL_17434227", "EPI_ISL_17437940", "EPI_ISL_17440507", "EPI_ISL_17441169", "EPI_ISL_17441208", "EPI_ISL_17441815", "EPI_ISL_17445401", "EPI_ISL_17446132", "EPI_ISL_17464711", "EPI_ISL_17466081", "EPI_ISL_17470229", "EPI_ISL_17470269", "EPI_ISL_17471181", "EPI_ISL_17471185", "EPI_ISL_17471619", "EPI_ISL_17471674", "EPI_ISL_17472531", "EPI_ISL_17475799", "EPI_ISL_17476568", "EPI_ISL_17476871", "EPI_ISL_17477106", "EPI_ISL_17480516", "EPI_ISL_17481180", "EPI_ISL_17481517", "EPI_ISL_17481597", "EPI_ISL_17482811", "EPI_ISL_17482813", "EPI_ISL_17482815", "EPI_ISL_17482819", "EPI_ISL_17494731", "EPI_ISL_17497461", "EPI_ISL_17497688", "EPI_ISL_17497854", "EPI_ISL_17497868", "EPI_ISL_17501536", "EPI_ISL_17501576", "EPI_ISL_17501763", "EPI_ISL_17502219", "EPI_ISL_17502972", "EPI_ISL_17503268", "EPI_ISL_17503711", "EPI_ISL_17504816", "EPI_ISL_17504835", "EPI_ISL_17505072", "EPI_ISL_17508749", "EPI_ISL_17509597", "EPI_ISL_17510495", "EPI_ISL_17510856", "EPI_ISL_17511096", "EPI_ISL_17511836", "EPI_ISL_17512412", "EPI_ISL_17512876", "EPI_ISL_17512968", "EPI_ISL_17513218", "EPI_ISL_17513312", "EPI_ISL_17514540", "EPI_ISL_17515086", "EPI_ISL_17515177", "EPI_ISL_17516651", "EPI_ISL_17516658", "EPI_ISL_17516659", "EPI_ISL_17517664", "EPI_ISL_17517834", "EPI_ISL_17517844", "EPI_ISL_17521302", "EPI_ISL_17521772", "EPI_ISL_17521778", "EPI_ISL_17522610", "EPI_ISL_17522687", "EPI_ISL_17522934", "EPI_ISL_17523535", "EPI_ISL_17523620", "EPI_ISL_17523782", "EPI_ISL_17523873", "EPI_ISL_17524106", "EPI_ISL_17524432", "EPI_ISL_17524502", "EPI_ISL_17524503", "EPI_ISL_17535664", "EPI_ISL_17535979", "EPI_ISL_17541088", "EPI_ISL_17541797", "EPI_ISL_17543006", "EPI_ISL_17544283", "EPI_ISL_17545970", "EPI_ISL_17546923", "EPI_ISL_17547529", "EPI_ISL_17547545", "EPI_ISL_17548526", "EPI_ISL_17549129", "EPI_ISL_17550129", "EPI_ISL_17550538", "EPI_ISL_17553063", "EPI_ISL_17553974", "EPI_ISL_17556705", "EPI_ISL_17559150", "EPI_ISL_17559165", "EPI_ISL_17559166", "EPI_ISL_17559167", "EPI_ISL_17559168", "EPI_ISL_17563568", "EPI_ISL_17565071", "EPI_ISL_17565211", "EPI_ISL_17565212", "EPI_ISL_17566854", "EPI_ISL_17579120", "EPI_ISL_17580420", "EPI_ISL_17583157", "EPI_ISL_17584277", "EPI_ISL_17585020", "EPI_ISL_17585021", "EPI_ISL_17585022", "EPI_ISL_17585023", "EPI_ISL_17585036", "EPI_ISL_17585039", "EPI_ISL_17586115", "EPI_ISL_17587423", "EPI_ISL_17587656", "EPI_ISL_17587671", "EPI_ISL_17587859", "EPI_ISL_17588127", "EPI_ISL_17588216", "EPI_ISL_17588460", "EPI_ISL_17589845", "EPI_ISL_17590449", "EPI_ISL_17590486", "EPI_ISL_17591005", "EPI_ISL_17591014", "EPI_ISL_17591015", "EPI_ISL_17591028", "EPI_ISL_17592236", "EPI_ISL_17592618", "EPI_ISL_17593692", "EPI_ISL_17593865", "EPI_ISL_17595116", "EPI_ISL_17595117", "EPI_ISL_17595967", "EPI_ISL_17595980", "EPI_ISL_17597954", "EPI_ISL_17598384", "EPI_ISL_17599326", "EPI_ISL_17599427", "EPI_ISL_17600948", "EPI_ISL_17600958", "EPI_ISL_17600978", "EPI_ISL_17600988", "EPI_ISL_17601066", "EPI_ISL_17601144", "EPI_ISL_17601196", "EPI_ISL_17601219", "EPI_ISL_17601261", "EPI_ISL_17601276", "EPI_ISL_17601933", "EPI_ISL_17602469", "EPI_ISL_17602756", "EPI_ISL_17605514", "EPI_ISL_17612035", "EPI_ISL_17612050", "EPI_ISL_17612051", "EPI_ISL_17612052", "EPI_ISL_17615127", "EPI_ISL_17615387", "EPI_ISL_17617168", "EPI_ISL_17617538", "EPI_ISL_17618306", "EPI_ISL_17621930", "EPI_ISL_17623470", "EPI_ISL_17623785", "EPI_ISL_17623810", "EPI_ISL_17626289", "EPI_ISL_17628383", "EPI_ISL_17628855", "EPI_ISL_17630096", "EPI_ISL_17632950", "EPI_ISL_17633442", "EPI_ISL_17634290", "EPI_ISL_17634585", "EPI_ISL_17634799", "EPI_ISL_17637409", "EPI_ISL_17637499", "EPI_ISL_17637946", "EPI_ISL_17640029", "EPI_ISL_17640079", "EPI_ISL_17642112", "EPI_ISL_17642765", "EPI_ISL_17643093", "EPI_ISL_17644186", "EPI_ISL_17645081", "EPI_ISL_17645416", "EPI_ISL_17646422", "EPI_ISL_17648924", "EPI_ISL_17651803", "EPI_ISL_17652508", "EPI_ISL_17652513", "EPI_ISL_17654325", "EPI_ISL_17654831", "EPI_ISL_17655018", "EPI_ISL_17656002", "EPI_ISL_17657287", "EPI_ISL_17658392", "EPI_ISL_17658574", "EPI_ISL_17659247", "EPI_ISL_17659794", "EPI_ISL_17661435", "EPI_ISL_17661709", "EPI_ISL_17661736", "EPI_ISL_17661772", "EPI_ISL_17662111", "EPI_ISL_17664370", "EPI_ISL_17666708", "EPI_ISL_17667360", "EPI_ISL_17667562", "EPI_ISL_17669441", "EPI_ISL_17669457", "EPI_ISL_17671157", "EPI_ISL_17671162", "EPI_ISL_17671689", "EPI_ISL_17675889", "EPI_ISL_17677128", "EPI_ISL_17677325", "EPI_ISL_17678395", "EPI_ISL_17679253", "EPI_ISL_17679612", "EPI_ISL_17680172", "EPI_ISL_17683135", "EPI_ISL_17683747", "EPI_ISL_17683879", "EPI_ISL_17683882", "EPI_ISL_17683902", "EPI_ISL_17683926", "EPI_ISL_17684194", "EPI_ISL_17685960", "EPI_ISL_17685982", "EPI_ISL_17686409", "EPI_ISL_17686485", "EPI_ISL_17686694", "EPI_ISL_17686736", "EPI_ISL_17688072", "EPI_ISL_17689247", "EPI_ISL_17695348", "EPI_ISL_17696086", "EPI_ISL_17696551", "EPI_ISL_17696575", "EPI_ISL_17697616", "EPI_ISL_17699149", "EPI_ISL_17699879", "EPI_ISL_17699884", "EPI_ISL_17700051", "EPI_ISL_17700270", "EPI_ISL_17701278", "EPI_ISL_17701782", "EPI_ISL_17703815", "EPI_ISL_17704713", "EPI_ISL_17705564", "EPI_ISL_17706013", "EPI_ISL_17706030", "EPI_ISL_17708288", "EPI_ISL_17709926", "EPI_ISL_17710268", "EPI_ISL_17710278", "EPI_ISL_17710307", "EPI_ISL_17710673", "EPI_ISL_17710974", "EPI_ISL_17711012", "EPI_ISL_17711646", "EPI_ISL_17712964", "EPI_ISL_17713423", "EPI_ISL_17713709", "EPI_ISL_17714880", "EPI_ISL_17714882", "EPI_ISL_17714902", "EPI_ISL_17714948", "EPI_ISL_17715122", "EPI_ISL_17715974", "EPI_ISL_17716296", "EPI_ISL_17718358", "EPI_ISL_17718497", "EPI_ISL_17719162", "EPI_ISL_17721620", "EPI_ISL_17721941", "EPI_ISL_17722142", "EPI_ISL_17722884", "EPI_ISL_17726746", "EPI_ISL_17727194", "EPI_ISL_17728144", "EPI_ISL_17728250", "EPI_ISL_17729454", "EPI_ISL_17731387", "EPI_ISL_17731388", "EPI_ISL_17732098", "EPI_ISL_17733269", "EPI_ISL_17734236", "EPI_ISL_17735972", "EPI_ISL_17736284", "EPI_ISL_17736388", "EPI_ISL_17737562", "EPI_ISL_17739108", "EPI_ISL_17739543", "EPI_ISL_17741957", "EPI_ISL_17743681", "EPI_ISL_17744022", "EPI_ISL_17747309", "EPI_ISL_17759354", "EPI_ISL_17759925", "EPI_ISL_17760156", "EPI_ISL_17760279", "EPI_ISL_17762387", "EPI_ISL_17762760", "EPI_ISL_17763721", "EPI_ISL_17764011", "EPI_ISL_17764066", "EPI_ISL_17764072", "EPI_ISL_17764496", "EPI_ISL_17766060", "EPI_ISL_17766100", "EPI_ISL_17766112", "EPI_ISL_17767434", "EPI_ISL_17767435", "EPI_ISL_17767436", "EPI_ISL_17767437", "EPI_ISL_17769030", "EPI_ISL_17769081", "EPI_ISL_17769169", "EPI_ISL_17769170", "EPI_ISL_17769216", "EPI_ISL_17769229", "EPI_ISL_17769310", "EPI_ISL_17769888", "EPI_ISL_17770729", "EPI_ISL_17770732", "EPI_ISL_17770736", "EPI_ISL_17770779", "EPI_ISL_17771047", "EPI_ISL_17771051", "EPI_ISL_17772012", "EPI_ISL_17775344", "EPI_ISL_17776736", "EPI_ISL_17777061", "EPI_ISL_17777067", "EPI_ISL_17777729", "EPI_ISL_17778593", "EPI_ISL_17778602", "EPI_ISL_17780724", "EPI_ISL_17780726", "EPI_ISL_17780860", "EPI_ISL_17780886", "EPI_ISL_17781122", "EPI_ISL_17781585", "EPI_ISL_17781712", "EPI_ISL_17782148", "EPI_ISL_17782366", "EPI_ISL_17782502", "EPI_ISL_17783358", "EPI_ISL_17784545", "EPI_ISL_17784546", "EPI_ISL_17784547", "EPI_ISL_17784552", "EPI_ISL_17784558", "EPI_ISL_17784569", "EPI_ISL_17784585", "EPI_ISL_17784593", "EPI_ISL_17784775", "EPI_ISL_17784803", "EPI_ISL_17784804", "EPI_ISL_17786165", "EPI_ISL_17786546", "EPI_ISL_17786769", "EPI_ISL_17786827", "EPI_ISL_17787009", "EPI_ISL_17787597", "EPI_ISL_17787864", "EPI_ISL_17788384", "EPI_ISL_17789385", "EPI_ISL_17789475", "EPI_ISL_17789808", "EPI_ISL_17790033", "EPI_ISL_17790116", "EPI_ISL_17791306", "EPI_ISL_17791796", "EPI_ISL_17792172", "EPI_ISL_17792191", "EPI_ISL_17794816", "EPI_ISL_17796500", "EPI_ISL_17796537", "EPI_ISL_17796598", "EPI_ISL_17796704", "EPI_ISL_17797704", "EPI_ISL_17798165", "EPI_ISL_17799068", "EPI_ISL_17799108", "EPI_ISL_17802597", "EPI_ISL_17803325", "EPI_ISL_17803653", "EPI_ISL_17806504", "EPI_ISL_17806524", "EPI_ISL_17809334", "EPI_ISL_17809574", "EPI_ISL_17810512", "EPI_ISL_17812915", "EPI_ISL_17813049", "EPI_ISL_17813537", "EPI_ISL_17813637", "EPI_ISL_17813862", "EPI_ISL_17815222", "EPI_ISL_17816174", "EPI_ISL_17817657", "EPI_ISL_17817985", "EPI_ISL_17818039", "EPI_ISL_17819921", "EPI_ISL_17820257", "EPI_ISL_17820258", "EPI_ISL_17820602", "EPI_ISL_17821850", "EPI_ISL_17823538", "EPI_ISL_17824292", "EPI_ISL_17824608", "EPI_ISL_17824670", "EPI_ISL_17826285", "EPI_ISL_17830573", "EPI_ISL_17830591", "EPI_ISL_17830762", "EPI_ISL_17831005", "EPI_ISL_17831639", "EPI_ISL_17831941", "EPI_ISL_17833161", "EPI_ISL_17833549", "EPI_ISL_17837092", "EPI_ISL_17837097", "EPI_ISL_17837134", "EPI_ISL_17837135", "EPI_ISL_17837188", "EPI_ISL_17837432", "EPI_ISL_17837459", "EPI_ISL_17837460", "EPI_ISL_17837914", "EPI_ISL_17837915", "EPI_ISL_17838109", "EPI_ISL_17838506", "EPI_ISL_17850070", "EPI_ISL_17850078", "EPI_ISL_17851276", "EPI_ISL_17853355", "EPI_ISL_17853579", "EPI_ISL_17855226", "EPI_ISL_17856975", "EPI_ISL_17857949", "EPI_ISL_17857950", "EPI_ISL_17859477", "EPI_ISL_17860390", "EPI_ISL_17860984", "EPI_ISL_17862677", "EPI_ISL_17871595", "EPI_ISL_17879222", "EPI_ISL_17884376", "EPI_ISL_17884518", "EPI_ISL_17885064", "EPI_ISL_17885128", "EPI_ISL_17885331", "EPI_ISL_17885459", "EPI_ISL_17891004", "EPI_ISL_17899627", "EPI_ISL_17949029", "EPI_ISL_17949339", "EPI_ISL_17949978", "EPI_ISL_17950840", "EPI_ISL_17952015", "EPI_ISL_17952019", "EPI_ISL_17953343", "EPI_ISL_17953610", "EPI_ISL_17954106", "EPI_ISL_17954662", "EPI_ISL_17954669", "EPI_ISL_17954940", "EPI_ISL_17956164", "EPI_ISL_17958015", "EPI_ISL_17959424", "EPI_ISL_17960600", "EPI_ISL_17960747", "EPI_ISL_17964403", "EPI_ISL_17964415", "EPI_ISL_17964828", "EPI_ISL_17965635", "EPI_ISL_17965636", "EPI_ISL_17966200", "EPI_ISL_17966205", "EPI_ISL_17968777", "EPI_ISL_17968962", "EPI_ISL_17969108", "EPI_ISL_17971223", "EPI_ISL_17971936", "EPI_ISL_17972242", "EPI_ISL_17972372", "EPI_ISL_17973367", "EPI_ISL_17974574", "EPI_ISL_17974688", "EPI_ISL_17974927", "EPI_ISL_17974952", "EPI_ISL_17975003", "EPI_ISL_17975174", "EPI_ISL_17976113", "EPI_ISL_17976116", "EPI_ISL_17976531", "EPI_ISL_17977985", "EPI_ISL_17978344", "EPI_ISL_17978693", "EPI_ISL_17978839", "EPI_ISL_17979017", "EPI_ISL_17979965", "EPI_ISL_17979979", "EPI_ISL_17979981", "EPI_ISL_17982411", "EPI_ISL_17982453", "EPI_ISL_17982543", "EPI_ISL_17985757", "EPI_ISL_17988396", "EPI_ISL_17989190", "EPI_ISL_17989433", "EPI_ISL_17989516", "EPI_ISL_17989740", "EPI_ISL_17989749", "EPI_ISL_17989792", "EPI_ISL_17989829", "EPI_ISL_17989860", "EPI_ISL_17990304", "EPI_ISL_17993966", "EPI_ISL_17994784", "EPI_ISL_17994786", "EPI_ISL_17995488", "EPI_ISL_17995513", "EPI_ISL_17995955", "EPI_ISL_17996897", "EPI_ISL_17997249", "EPI_ISL_17997251", "EPI_ISL_17997917", "EPI_ISL_17997982", "EPI_ISL_17998406", "EPI_ISL_18000155", "EPI_ISL_18000245", "EPI_ISL_18000414", "EPI_ISL_18000654", "EPI_ISL_18000825", "EPI_ISL_18001789", "EPI_ISL_18001862", "EPI_ISL_18008246", "EPI_ISL_18008262", "EPI_ISL_18008673", "EPI_ISL_18009591", "EPI_ISL_18009602", "EPI_ISL_18010720", "EPI_ISL_18011449", "EPI_ISL_18011518", "EPI_ISL_18012526", "EPI_ISL_18012547", "EPI_ISL_18012806", "EPI_ISL_18014700", "EPI_ISL_18016999", "EPI_ISL_18019246", "EPI_ISL_18028785", "EPI_ISL_18029979", "EPI_ISL_18030390", "EPI_ISL_18030391", "EPI_ISL_18030395", "EPI_ISL_18031842", "EPI_ISL_18032297", "EPI_ISL_18032322", "EPI_ISL_18032338", "EPI_ISL_18033013", "EPI_ISL_18033516", "EPI_ISL_18033631", "EPI_ISL_18034109", "EPI_ISL_18037119", "EPI_ISL_18037474", "EPI_ISL_18037476", "EPI_ISL_18037744", "EPI_ISL_18038269", "EPI_ISL_18039728", "EPI_ISL_18040070", "EPI_ISL_18041130", "EPI_ISL_18041968", "EPI_ISL_18042110", "EPI_ISL_18044024", "EPI_ISL_18044164", "EPI_ISL_18044400", "EPI_ISL_18044754", "EPI_ISL_18044755", "EPI_ISL_18044759", "EPI_ISL_18045809", "EPI_ISL_18048708", "EPI_ISL_18048972", "EPI_ISL_18048978", "EPI_ISL_18049009", "EPI_ISL_18049160", "EPI_ISL_18049161", "EPI_ISL_18049174", "EPI_ISL_18049809", "EPI_ISL_18049917", "EPI_ISL_18050065", "EPI_ISL_18050520", "EPI_ISL_18050523", "EPI_ISL_18051914", "EPI_ISL_18051918", "EPI_ISL_18052440", "EPI_ISL_18052776", "EPI_ISL_18052929", "EPI_ISL_18053022", "EPI_ISL_18053315", "EPI_ISL_18054466", "EPI_ISL_18054899", "EPI_ISL_18056643", "EPI_ISL_18056644", "EPI_ISL_18056759", "EPI_ISL_18056769", "EPI_ISL_18058525", "EPI_ISL_18058567", "EPI_ISL_18058881", "EPI_ISL_18059074", "EPI_ISL_18059075", "EPI_ISL_18059076", "EPI_ISL_18059726", "EPI_ISL_18060973", "EPI_ISL_18062475", "EPI_ISL_18064366", "EPI_ISL_18064383", "EPI_ISL_18064405", "EPI_ISL_18064413", "EPI_ISL_18064431", "EPI_ISL_18064456", "EPI_ISL_18070310", "EPI_ISL_18071883", "EPI_ISL_18071901", "EPI_ISL_18072343", "EPI_ISL_18073924", "EPI_ISL_18074072", "EPI_ISL_18075985", "EPI_ISL_18076065", "EPI_ISL_18076069", "EPI_ISL_18076165", "EPI_ISL_18076251", "EPI_ISL_18076473", "EPI_ISL_18077275", "EPI_ISL_18078878", "EPI_ISL_18079417", "EPI_ISL_18080566", "EPI_ISL_18083488", "EPI_ISL_18091808", "EPI_ISL_18093840", "EPI_ISL_18094397", "EPI_ISL_18094429", "EPI_ISL_18094476", "EPI_ISL_18094560", "EPI_ISL_18094842", "EPI_ISL_18095961", "EPI_ISL_18097327", "EPI_ISL_18097349", "EPI_ISL_18097786", "EPI_ISL_18098270", "EPI_ISL_18098276", "EPI_ISL_18098299", "EPI_ISL_18098479", "EPI_ISL_18098976", "EPI_ISL_18099952", "EPI_ISL_18100455", "EPI_ISL_18100457", "EPI_ISL_18100607", "EPI_ISL_18104072", "EPI_ISL_18104501", "EPI_ISL_18106416", "EPI_ISL_18106460", "EPI_ISL_18106464", "EPI_ISL_18106662", "EPI_ISL_18106788", "EPI_ISL_18106910", "EPI_ISL_18106912", "EPI_ISL_18106920", "EPI_ISL_18106929", "EPI_ISL_18106930", "EPI_ISL_18106931", "EPI_ISL_18106933", "EPI_ISL_18106934", "EPI_ISL_18106950", "EPI_ISL_18106951", "EPI_ISL_18107900", "EPI_ISL_18109285", "EPI_ISL_18110014", "EPI_ISL_18110496", "EPI_ISL_18110776", "EPI_ISL_18111020", "EPI_ISL_18111021", "EPI_ISL_18111040", "EPI_ISL_18111041", "EPI_ISL_18111086", "EPI_ISL_18112015", "EPI_ISL_18115442", "EPI_ISL_18115451", "EPI_ISL_18115956", "EPI_ISL_18116015", "EPI_ISL_18116176", "EPI_ISL_18118289", "EPI_ISL_18118388", "EPI_ISL_18118556", "EPI_ISL_18118855", "EPI_ISL_18119265", "EPI_ISL_18120201", "EPI_ISL_18123396", "EPI_ISL_18124840", "EPI_ISL_18125049", "EPI_ISL_18125050", "EPI_ISL_18126021", "EPI_ISL_18126834", "EPI_ISL_18127203", "EPI_ISL_18127526", "EPI_ISL_18127527", "EPI_ISL_18127685", "EPI_ISL_18127834", "EPI_ISL_18129019", "EPI_ISL_18129038", "EPI_ISL_18129213", "EPI_ISL_18129656", "EPI_ISL_18129944", "EPI_ISL_18131053", "EPI_ISL_18131109", "EPI_ISL_18134315", "EPI_ISL_18134392", "EPI_ISL_18134395", "EPI_ISL_18134442", "EPI_ISL_18134610", "EPI_ISL_18134691", "EPI_ISL_18134700", "EPI_ISL_18134706", "EPI_ISL_18134984", "EPI_ISL_18135040", "EPI_ISL_18136392", "EPI_ISL_18136968", "EPI_ISL_18139400", "EPI_ISL_18139409", "EPI_ISL_18141686", "EPI_ISL_18141739", "EPI_ISL_18141844", "EPI_ISL_18142202", "EPI_ISL_18142317", "EPI_ISL_18142357", "EPI_ISL_18142978", "EPI_ISL_18142994", "EPI_ISL_18146885", "EPI_ISL_18147456", "EPI_ISL_18147966", "EPI_ISL_18151975", "EPI_ISL_18151976", "EPI_ISL_18151977", "EPI_ISL_18153000", "EPI_ISL_18159587", "EPI_ISL_18160510", "EPI_ISL_18160518", "EPI_ISL_18160530", "EPI_ISL_18160538", "EPI_ISL_18162564", "EPI_ISL_18162567", "EPI_ISL_18163680", "EPI_ISL_18164441", "EPI_ISL_18166642", "EPI_ISL_18166643", "EPI_ISL_18168780", "EPI_ISL_18169117", "EPI_ISL_18205057", "EPI_ISL_18207613", "EPI_ISL_18210510", "EPI_ISL_18212559", "EPI_ISL_18213104", "EPI_ISL_18215123", "EPI_ISL_18215226", "EPI_ISL_18215482", "EPI_ISL_18215552", "EPI_ISL_18217564", "EPI_ISL_18217995", "EPI_ISL_18218776", "EPI_ISL_18219916", "EPI_ISL_18219931", "EPI_ISL_18219970", "EPI_ISL_18220073", "EPI_ISL_18220494", "EPI_ISL_18220495", "EPI_ISL_18220498", "EPI_ISL_18221521", "EPI_ISL_18221524", "EPI_ISL_18221527", "EPI_ISL_18221982", "EPI_ISL_18221985", "EPI_ISL_18222367", "EPI_ISL_18224410", "EPI_ISL_18224514", "EPI_ISL_18225473", "EPI_ISL_18227366", "EPI_ISL_18227596", "EPI_ISL_18227611", "EPI_ISL_18227624", "EPI_ISL_18227629", "EPI_ISL_18228307", "EPI_ISL_18232124", "EPI_ISL_18233906", "EPI_ISL_18234431", "EPI_ISL_18236180", "EPI_ISL_18236291", "EPI_ISL_18237063", "EPI_ISL_18238117", "EPI_ISL_18241087", "EPI_ISL_18241705", "EPI_ISL_18241707", "EPI_ISL_18241719", "EPI_ISL_18245571", "EPI_ISL_18247259", "EPI_ISL_18248695", "EPI_ISL_18249682", "EPI_ISL_18253248", "EPI_ISL_18253249", "EPI_ISL_18255994", "EPI_ISL_18256173", "EPI_ISL_18256714", "EPI_ISL_18256980", "EPI_ISL_18258766", "EPI_ISL_18259784", "EPI_ISL_18260202", "EPI_ISL_18263919", "EPI_ISL_18263945", "EPI_ISL_18263981", "EPI_ISL_18269234", "EPI_ISL_18271265", "EPI_ISL_18273982", "EPI_ISL_18274346", "EPI_ISL_18276415", "EPI_ISL_18277439", "EPI_ISL_18277736", "EPI_ISL_18278627", "EPI_ISL_18278909", "EPI_ISL_18279614", "EPI_ISL_18281186", "EPI_ISL_18281259", "EPI_ISL_18281287", "EPI_ISL_18281288", "EPI_ISL_18281494", "EPI_ISL_18281574", "EPI_ISL_18282077", "EPI_ISL_18282082", "EPI_ISL_18286773", "EPI_ISL_18287351", "EPI_ISL_18290890", "EPI_ISL_18290989", "EPI_ISL_18291808", "EPI_ISL_18292038", "EPI_ISL_18292398", "EPI_ISL_18294574", "EPI_ISL_18295441", "EPI_ISL_18298019", "EPI_ISL_18299948", "EPI_ISL_18301587", "EPI_ISL_18302636", "EPI_ISL_18303012", "EPI_ISL_18303592", "EPI_ISL_18303595", "EPI_ISL_18303758", "EPI_ISL_18306254", "EPI_ISL_18306922", "EPI_ISL_18308642", "EPI_ISL_18311951", "EPI_ISL_18313683", "EPI_ISL_18315747", "EPI_ISL_18315789", "EPI_ISL_18319306", "EPI_ISL_18319903", "EPI_ISL_18319904", "EPI_ISL_18319906", "EPI_ISL_18319907", "EPI_ISL_18320079", "EPI_ISL_18321271", "EPI_ISL_18322273", "EPI_ISL_18322420", "EPI_ISL_18322438", "EPI_ISL_18323536", "EPI_ISL_18324107", "EPI_ISL_18324168", "EPI_ISL_18324491", "EPI_ISL_18324976", "EPI_ISL_18325145", "EPI_ISL_18325563", "EPI_ISL_18326430", "EPI_ISL_18326597", "EPI_ISL_18326806", "EPI_ISL_18326807", "EPI_ISL_18330957", "EPI_ISL_18330966", "EPI_ISL_18331347", "EPI_ISL_18334903", "EPI_ISL_18334945", "EPI_ISL_18334986", "EPI_ISL_18335084", "EPI_ISL_18336165", "EPI_ISL_18336602", "EPI_ISL_18336862", "EPI_ISL_18337738", "EPI_ISL_18338137", "EPI_ISL_18338143", "EPI_ISL_18338144", "EPI_ISL_18338502", "EPI_ISL_18338504", "EPI_ISL_18338709", "EPI_ISL_18342412", "EPI_ISL_18343598", "EPI_ISL_18345777", "EPI_ISL_18345926", "EPI_ISL_18346109", "EPI_ISL_18351588", "EPI_ISL_18352473", "EPI_ISL_18352485", "EPI_ISL_18352489", "EPI_ISL_18359229	", "EPI_ISL_18359328", "EPI_ISL_18359679", "EPI_ISL_18360507", "EPI_ISL_18360944", "EPI_ISL_18361202", "EPI_ISL_18362265", "EPI_ISL_18362515", "EPI_ISL_18363170", "EPI_ISL_18363300", "EPI_ISL_18365170", "EPI_ISL_18365256", "EPI_ISL_18367086", "EPI_ISL_18367563", "EPI_ISL_18367586", "EPI_ISL_18367599", "EPI_ISL_18367615", "EPI_ISL_18367908", "EPI_ISL_18367992", "EPI_ISL_18370898", "EPI_ISL_18370960", "EPI_ISL_18370967", "EPI_ISL_18371749", "EPI_ISL_18373201", "EPI_ISL_18377021", "EPI_ISL_18377214", "EPI_ISL_18377245", "EPI_ISL_18377248", "EPI_ISL_18378384", "EPI_ISL_18380731", "EPI_ISL_18381066", "EPI_ISL_18383121", "EPI_ISL_18383423", "EPI_ISL_18384846", "EPI_ISL_18384936", "EPI_ISL_18385358", "EPI_ISL_18385924", "EPI_ISL_18386091", "EPI_ISL_18386114", "EPI_ISL_18387037", "EPI_ISL_18388509", "EPI_ISL_18388585", "EPI_ISL_18389783", "EPI_ISL_18391451", "EPI_ISL_18391597", "EPI_ISL_18392259", "EPI_ISL_18392502", "EPI_ISL_18392841", "EPI_ISL_18393366", "EPI_ISL_18395551", "EPI_ISL_18398210", "EPI_ISL_18398259", "EPI_ISL_18400843", "EPI_ISL_18400856", "EPI_ISL_18400946", "EPI_ISL_18400987", "EPI_ISL_18401313", "EPI_ISL_18403047", "EPI_ISL_18403051", "EPI_ISL_18403054", "EPI_ISL_18403509", "EPI_ISL_18403523", "EPI_ISL_18404585", "EPI_ISL_18405535", "EPI_ISL_18405621", "EPI_ISL_18406078", "EPI_ISL_18406394", "EPI_ISL_18408561", "EPI_ISL_18410987", "EPI_ISL_18414567", "EPI_ISL_18414568", "EPI_ISL_18414808", "EPI_ISL_18415823", "EPI_ISL_18415832", "EPI_ISL_18415834", "EPI_ISL_18415840", "EPI_ISL_18415854", "EPI_ISL_18416870", "EPI_ISL_18417129", "EPI_ISL_18417211", "EPI_ISL_18419485", "EPI_ISL_18419748", "EPI_ISL_18421674", "EPI_ISL_18422693", "EPI_ISL_18422715", "EPI_ISL_18422771", "EPI_ISL_18423785", "EPI_ISL_18423814", "EPI_ISL_18423907", "EPI_ISL_18424281", "EPI_ISL_18424468", "EPI_ISL_18426836", "EPI_ISL_18428844", "EPI_ISL_18429684", "EPI_ISL_18429702", "EPI_ISL_18429725", "EPI_ISL_18429773", "EPI_ISL_18429797", "EPI_ISL_18432077", "EPI_ISL_18433350", "EPI_ISL_18434194", "EPI_ISL_18435557", "EPI_ISL_18435892", "EPI_ISL_18435949", "EPI_ISL_18436693", "EPI_ISL_18437342", "EPI_ISL_18438723", "EPI_ISL_18439733", "EPI_ISL_18440037", "EPI_ISL_18440370", "EPI_ISL_18440660", "EPI_ISL_18440866", "EPI_ISL_18441868", "EPI_ISL_18443784", "EPI_ISL_18443944", "EPI_ISL_18448894", "EPI_ISL_18449647", "EPI_ISL_18449794", "EPI_ISL_18449820", "EPI_ISL_18449892", "EPI_ISL_18450249", "EPI_ISL_18450812", "EPI_ISL_18451678", "EPI_ISL_18453400", "EPI_ISL_18455292", "EPI_ISL_18455564", "EPI_ISL_18455706", "EPI_ISL_18455950", "EPI_ISL_18457808", "EPI_ISL_18459512", "EPI_ISL_18461774", "EPI_ISL_18462852", "EPI_ISL_18463490", "EPI_ISL_18463766", "EPI_ISL_18466251", "EPI_ISL_18468149", "EPI_ISL_18470400", "EPI_ISL_18472311", "EPI_ISL_18472312", "EPI_ISL_18473559", "EPI_ISL_18474126", "EPI_ISL_18474555", "EPI_ISL_18474665", "EPI_ISL_18475534", "EPI_ISL_18475535", "EPI_ISL_18480741", "EPI_ISL_18486919", "EPI_ISL_18487225", "EPI_ISL_18489646", "EPI_ISL_18489793", "EPI_ISL_18489829", "EPI_ISL_18491841", "EPI_ISL_18491844", "EPI_ISL_18492277", "EPI_ISL_18492305", "EPI_ISL_18492307", "EPI_ISL_18492412", "EPI_ISL_18492455", "EPI_ISL_18493129", "EPI_ISL_18495416", "EPI_ISL_18496252", "EPI_ISL_18496585", "EPI_ISL_18498001", "EPI_ISL_18498420", "EPI_ISL_18498499", "EPI_ISL_18500316", "EPI_ISL_18500771", "EPI_ISL_18501087", "EPI_ISL_18503287", "EPI_ISL_18509817", "EPI_ISL_18512421", "EPI_ISL_18512438", "EPI_ISL_18513936", "EPI_ISL_18514552", "EPI_ISL_18515280", "EPI_ISL_18515328", "EPI_ISL_18515343", "EPI_ISL_18515511", "EPI_ISL_18515749", "EPI_ISL_18516916", "EPI_ISL_18518769", "EPI_ISL_18518932", "EPI_ISL_18519113", "EPI_ISL_18520677", "EPI_ISL_18520678", "EPI_ISL_18521575", "EPI_ISL_18521765", "EPI_ISL_18522184", "EPI_ISL_18522580", "EPI_ISL_18523129", "EPI_ISL_18524926", "EPI_ISL_18525067", "EPI_ISL_18525840", "EPI_ISL_18526641", "EPI_ISL_18528453", "EPI_ISL_18529555", "EPI_ISL_18530445", "EPI_ISL_18530449", "EPI_ISL_18536100", "EPI_ISL_18536853", "EPI_ISL_18537013", "EPI_ISL_18537032", "EPI_ISL_18537373", "EPI_ISL_18537428", "EPI_ISL_18537814", "EPI_ISL_18538015", "EPI_ISL_18543268", "EPI_ISL_18543705", "EPI_ISL_18546112", "EPI_ISL_18546287", "EPI_ISL_18546551", "EPI_ISL_18546715", "EPI_ISL_18551440", "EPI_ISL_18552697", "EPI_ISL_18553587", "EPI_ISL_18554053", "EPI_ISL_18556084", "EPI_ISL_18556539", "EPI_ISL_18557145", "EPI_ISL_18558360", "EPI_ISL_18558385", "EPI_ISL_18558412", "EPI_ISL_18558468", "EPI_ISL_18558477", "EPI_ISL_18559317", "EPI_ISL_18560556", "EPI_ISL_18560725", "EPI_ISL_18560872", "EPI_ISL_18561098", "EPI_ISL_18563181", "EPI_ISL_18563821", "EPI_ISL_18564403", "EPI_ISL_18564755", "EPI_ISL_18566696", "EPI_ISL_18567985", "EPI_ISL_18568124", "EPI_ISL_18576266", "EPI_ISL_18576754", "EPI_ISL_18577842", "EPI_ISL_18577862", "EPI_ISL_18577966", "EPI_ISL_18578195", "EPI_ISL_18579981", "EPI_ISL_18580011", "EPI_ISL_18580750", "EPI_ISL_18580874", "EPI_ISL_18581347", "EPI_ISL_18584141", "EPI_ISL_18588773", "EPI_ISL_18589012", "EPI_ISL_18589243", "EPI_ISL_18589475", "EPI_ISL_18589669", "EPI_ISL_18590820", "EPI_ISL_18591717", "EPI_ISL_18593579", "EPI_ISL_18594183", "EPI_ISL_18594233", "EPI_ISL_18594266", "EPI_ISL_18595212", "EPI_ISL_18598503", "EPI_ISL_18598525", "EPI_ISL_18603922", "EPI_ISL_18604501", "EPI_ISL_18604502", "EPI_ISL_18604503", "EPI_ISL_18604504", "EPI_ISL_18604505", "EPI_ISL_18605012", "EPI_ISL_18605132", "EPI_ISL_18606460", "EPI_ISL_18607149", "EPI_ISL_18607150", "EPI_ISL_18608694", "EPI_ISL_18609973", "EPI_ISL_18612246", "EPI_ISL_18615968", "EPI_ISL_18622139", "EPI_ISL_18624843", "EPI_ISL_18625316", "EPI_ISL_18626713", "EPI_ISL_18626714", "EPI_ISL_18626750", "EPI_ISL_18630930", "EPI_ISL_18633829", "EPI_ISL_18634703", "EPI_ISL_18635526", "EPI_ISL_18635546", "EPI_ISL_18635599", "EPI_ISL_18635961", "EPI_ISL_18639151", "EPI_ISL_18640058", "EPI_ISL_18641470", "EPI_ISL_18641499", "EPI_ISL_18642608", "EPI_ISL_18646912", "EPI_ISL_18646945", "EPI_ISL_18648209", "EPI_ISL_18652556", "EPI_ISL_18653898", "EPI_ISL_18654501", "EPI_ISL_18668995", "EPI_ISL_18672102", "EPI_ISL_18674382", "EPI_ISL_18677704", "EPI_ISL_18681665", "EPI_ISL_18681942", "EPI_ISL_18681975", "EPI_ISL_18683430", "EPI_ISL_18687895", "EPI_ISL_18689964", "EPI_ISL_18693860", "EPI_ISL_18694187", "EPI_ISL_18702737", "EPI_ISL_18712665", "EPI_ISL_18712806", "EPI_ISL_18712994", "EPI_ISL_18713198", "EPI_ISL_18714360", "EPI_ISL_18714857", "EPI_ISL_18715065", "EPI_ISL_18715624", "EPI_ISL_18717261", "EPI_ISL_18727247", "EPI_ISL_18727294", "EPI_ISL_18729637", "EPI_ISL_18730563", "EPI_ISL_18740049", "EPI_ISL_18742836", "EPI_ISL_18743266", "EPI_ISL_18743442", "EPI_ISL_18743504", "EPI_ISL_18743721", "EPI_ISL_18759772", "EPI_ISL_18760241", "EPI_ISL_18763780", "EPI_ISL_18766752", "EPI_ISL_18772294", "EPI_ISL_18777179", "EPI_ISL_18778198", "EPI_ISL_18778941", "EPI_ISL_18782000", "EPI_ISL_18782699", "EPI_ISL_18784365", "EPI_ISL_18784385", "EPI_ISL_18784404", "EPI_ISL_18784438", "EPI_ISL_18787342", "EPI_ISL_18787343", "EPI_ISL_18792770", "EPI_ISL_18796930", "EPI_ISL_18797833", "EPI_ISL_18798193", "EPI_ISL_18798198", "EPI_ISL_18798202", "EPI_ISL_18798204", "EPI_ISL_18798234", "EPI_ISL_18799019", "EPI_ISL_18801405", "EPI_ISL_18803784", "EPI_ISL_18805014", "EPI_ISL_18809509", "EPI_ISL_18809652", "EPI_ISL_18811047", "EPI_ISL_18811325", "EPI_ISL_18811375", "EPI_ISL_18811389", "EPI_ISL_18814271", "EPI_ISL_18814272", "EPI_ISL_18814273", "EPI_ISL_18815556", "EPI_ISL_18816278", "EPI_ISL_18816628", "EPI_ISL_18817415", "EPI_ISL_18818949", "EPI_ISL_18820996", "EPI_ISL_18824608", "EPI_ISL_18825438", "EPI_ISL_18828912", "EPI_ISL_18829708", "EPI_ISL_18832540", "EPI_ISL_18832845", "EPI_ISL_18839074", "EPI_ISL_18839275", "EPI_ISL_18839824", "EPI_ISL_18839850", "EPI_ISL_18839932", "EPI_ISL_18841696", "EPI_ISL_18842709", "EPI_ISL_18846230", "EPI_ISL_18851473", "EPI_ISL_18853389", "EPI_ISL_18853925", "EPI_ISL_18854197", "EPI_ISL_18854790", "EPI_ISL_18854974", "EPI_ISL_18856190", "EPI_ISL_18856328", "EPI_ISL_18859868", "EPI_ISL_18860821", "EPI_ISL_18863734", "EPI_ISL_18864357", "EPI_ISL_18864823", "EPI_ISL_18865364", "EPI_ISL_18865427", "EPI_ISL_18868940", "EPI_ISL_18869806", "EPI_ISL_18870297", "EPI_ISL_18873437", "EPI_ISL_18873899", "EPI_ISL_18874096", "EPI_ISL_18874749", "EPI_ISL_18875008", "EPI_ISL_18876158", "EPI_ISL_18876159", "EPI_ISL_18876160", "EPI_ISL_18876161", "EPI_ISL_18876162", "EPI_ISL_18876164", "EPI_ISL_18876166", "EPI_ISL_18876167", "EPI_ISL_18876171", "EPI_ISL_18876174", "EPI_ISL_18876176", "EPI_ISL_18876178", "EPI_ISL_18876356", "EPI_ISL_18877288", "EPI_ISL_18877358", "EPI_ISL_18877379", "EPI_ISL_18877954", "EPI_ISL_18878665", "EPI_ISL_18880075", "EPI_ISL_18882888", "EPI_ISL_18884114", "EPI_ISL_18885028", "EPI_ISL_18885545", "EPI_ISL_18886798", "EPI_ISL_18888145", "EPI_ISL_18888340", "EPI_ISL_18889879", "EPI_ISL_18892766", "EPI_ISL_18896020", "EPI_ISL_18897180", "EPI_ISL_18897181", "EPI_ISL_18899656", "EPI_ISL_18899840", "EPI_ISL_18900159", "EPI_ISL_18901224", "EPI_ISL_18901411", "EPI_ISL_18901513", "EPI_ISL_18901762", "EPI_ISL_18901997", "EPI_ISL_18904234", "EPI_ISL_18906411", "EPI_ISL_18906577", "EPI_ISL_18908924", "EPI_ISL_18914858", "EPI_ISL_18915560", "EPI_ISL_18916107", "EPI_ISL_18916798", "EPI_ISL_18916914", "EPI_ISL_18917364", "EPI_ISL_18917482", "EPI_ISL_18917496", "EPI_ISL_18917541", "EPI_ISL_18918931", "EPI_ISL_18918932", "EPI_ISL_18918944", "EPI_ISL_18919477", "EPI_ISL_18919544", "EPI_ISL_18924428", "EPI_ISL_18927022", "EPI_ISL_18927251", "EPI_ISL_18927258", "EPI_ISL_18927292", "EPI_ISL_18927500", "EPI_ISL_18927514", "EPI_ISL_18927587", "EPI_ISL_18930149", "EPI_ISL_18930177", "EPI_ISL_18930182", "EPI_ISL_18930190", "EPI_ISL_18930681", "EPI_ISL_18931503", "EPI_ISL_18931504", "EPI_ISL_18931535", "EPI_ISL_18931538", "EPI_ISL_18931542", "EPI_ISL_18932507", "EPI_ISL_18933211", "EPI_ISL_18933315", "EPI_ISL_18933410", "EPI_ISL_18933606", "EPI_ISL_18933647", "EPI_ISL_18935631", "EPI_ISL_18936396", "EPI_ISL_18939566", "EPI_ISL_18939949", "EPI_ISL_18945392", "EPI_ISL_18945748", "EPI_ISL_18947086", "EPI_ISL_18947189", "EPI_ISL_18947944", "EPI_ISL_18948077", "EPI_ISL_18948078", "EPI_ISL_18948422", "EPI_ISL_18948613", "EPI_ISL_18948756", "EPI_ISL_18948757", "EPI_ISL_18949161", "EPI_ISL_18949911", "EPI_ISL_18952873", "EPI_ISL_18953132", "EPI_ISL_18953134", "EPI_ISL_18953234", "EPI_ISL_18953461", "EPI_ISL_18953493", "EPI_ISL_18953503", "EPI_ISL_18953522", "EPI_ISL_18955186", "EPI_ISL_18956070", "EPI_ISL_18956087", "EPI_ISL_18956161", "EPI_ISL_18958247", "EPI_ISL_18958295", "EPI_ISL_18958383", "EPI_ISL_18960149", "EPI_ISL_18960183", "EPI_ISL_18960224", "EPI_ISL_18962896", "EPI_ISL_18962928", "EPI_ISL_18963535", "EPI_ISL_18963652", "EPI_ISL_18965082", "EPI_ISL_18965451", "EPI_ISL_18966035", "EPI_ISL_18966186", "EPI_ISL_18968116", "EPI_ISL_18968120", "EPI_ISL_18969119", "EPI_ISL_18969697", "EPI_ISL_18969735", "EPI_ISL_18970692", "EPI_ISL_18971336", "EPI_ISL_18972187", "EPI_ISL_18972699", "EPI_ISL_18972708", "EPI_ISL_18972719", "EPI_ISL_18972734", "EPI_ISL_18972995", "EPI_ISL_18975064", "EPI_ISL_18975148", "EPI_ISL_18975189", "EPI_ISL_18975333", "EPI_ISL_18977928", "EPI_ISL_18977929", "EPI_ISL_18979948", "EPI_ISL_18980164", "EPI_ISL_18981298", "EPI_ISL_18982763", "EPI_ISL_18986641", "EPI_ISL_18986642", "EPI_ISL_18986747", "EPI_ISL_18987173", "EPI_ISL_18987258", "EPI_ISL_18987259", "EPI_ISL_18987545", "EPI_ISL_18990023", "EPI_ISL_18992540", "EPI_ISL_18999516", "EPI_ISL_19000455", "EPI_ISL_19002243", "EPI_ISL_19002832", "EPI_ISL_19003837", "EPI_ISL_19004571", "EPI_ISL_19005468", "EPI_ISL_19005470", "EPI_ISL_19005472", "EPI_ISL_19005481", "EPI_ISL_19006725", "EPI_ISL_19008875", "EPI_ISL_19009123", "EPI_ISL_19009131", "EPI_ISL_19010135", "EPI_ISL_19012038", "EPI_ISL_19012114", "EPI_ISL_19012460", "EPI_ISL_19012472", "EPI_ISL_19012554", "EPI_ISL_19012663", "EPI_ISL_19013418", "EPI_ISL_19014517", "EPI_ISL_19014601", "EPI_ISL_19015115", "EPI_ISL_19015882", "EPI_ISL_19016044", "EPI_ISL_19016053", "EPI_ISL_19016370", "EPI_ISL_19017499", "EPI_ISL_19019122", "EPI_ISL_19019127", "EPI_ISL_19019128", "EPI_ISL_19021045", "EPI_ISL_19021046", "EPI_ISL_19022071", "EPI_ISL_19022479", "EPI_ISL_19024260", "EPI_ISL_19026222", "EPI_ISL_19026577", "EPI_ISL_19026932", "EPI_ISL_19029499", "EPI_ISL_19030027", "EPI_ISL_19030112", "EPI_ISL_19030117", "EPI_ISL_19030129", "EPI_ISL_19030183", "EPI_ISL_19031583", "EPI_ISL_19032054", "EPI_ISL_19032056", "EPI_ISL_19032179", "EPI_ISL_19032598", "EPI_ISL_19033249", "EPI_ISL_19035801", "EPI_ISL_19036115", "EPI_ISL_19036116", "EPI_ISL_19036117", "EPI_ISL_19036644", "EPI_ISL_19036878", "EPI_ISL_19036921", "EPI_ISL_19041379", "EPI_ISL_19041967", "EPI_ISL_19042675", "EPI_ISL_19043835", "EPI_ISL_19043836", "EPI_ISL_19044085", "EPI_ISL_19044163", "EPI_ISL_19044218", "EPI_ISL_19044280", "EPI_ISL_19044474", "EPI_ISL_19044644", "EPI_ISL_19044737", "EPI_ISL_19046891", "EPI_ISL_19046892", "EPI_ISL_19047408", "EPI_ISL_19049407", "EPI_ISL_19049586", "EPI_ISL_19050509", "EPI_ISL_19051602", "EPI_ISL_19051976", "EPI_ISL_19052024", "EPI_ISL_19052034", "EPI_ISL_19052090", "EPI_ISL_19052102", "EPI_ISL_19052117", "EPI_ISL_19053178", "EPI_ISL_19053803", "EPI_ISL_19053806", "EPI_ISL_19053808", "EPI_ISL_19054784", "EPI_ISL_19055364", "EPI_ISL_19055368", "EPI_ISL_19055382", "EPI_ISL_19055909", "EPI_ISL_19058120", "EPI_ISL_19060002", "EPI_ISL_19060327", "EPI_ISL_19060922", "EPI_ISL_19061139", "EPI_ISL_19061646", "EPI_ISL_19062162", "EPI_ISL_19062523", "EPI_ISL_19062524", "EPI_ISL_19064175", "EPI_ISL_19065060", "EPI_ISL_19065882", "EPI_ISL_19066171", "EPI_ISL_19066817", "EPI_ISL_19067782", "EPI_ISL_19067788", "EPI_ISL_19070471", "EPI_ISL_19071618", "EPI_ISL_19073169", "EPI_ISL_19073731", "EPI_ISL_19073732", "EPI_ISL_19073800", "EPI_ISL_19073807", "EPI_ISL_19074673", "EPI_ISL_19075261", "EPI_ISL_19075267", "EPI_ISL_19075350", "EPI_ISL_19076088", "EPI_ISL_19081415", "EPI_ISL_19081416", "EPI_ISL_19081419", "EPI_ISL_19081422", "EPI_ISL_19081425", "EPI_ISL_19082080", "EPI_ISL_19082092", "EPI_ISL_19082200", "EPI_ISL_19082275", "EPI_ISL_19082487", "EPI_ISL_19085329", "EPI_ISL_19085381", "EPI_ISL_19085539", "EPI_ISL_19085540", "EPI_ISL_19085542", "EPI_ISL_19086378", "EPI_ISL_19086393", "EPI_ISL_19086523", "EPI_ISL_19091019", "EPI_ISL_19091144", "EPI_ISL_19094153", "EPI_ISL_19094369", "EPI_ISL_19095154", "EPI_ISL_19095156", "EPI_ISL_19095501", "EPI_ISL_19095606", "EPI_ISL_19095734", "EPI_ISL_19095768", "EPI_ISL_19100255", "EPI_ISL_19100981", "EPI_ISL_19105055", "EPI_ISL_19106368", "EPI_ISL_19106842", "EPI_ISL_19106844", "EPI_ISL_19106999", "EPI_ISL_19108198", "EPI_ISL_19108199", "EPI_ISL_19108200", "EPI_ISL_19108208", "EPI_ISL_19108340", "EPI_ISL_19108650", "EPI_ISL_19108704", "EPI_ISL_19108706", "EPI_ISL_19108725", "EPI_ISL_19117393", "EPI_ISL_19131415", "EPI_ISL_19131416", "EPI_ISL_19131417", "EPI_ISL_19131418", "EPI_ISL_19131419", "EPI_ISL_19132785", "EPI_ISL_19132846", "EPI_ISL_19133800", "EPI_ISL_19135448", "EPI_ISL_19135481", "EPI_ISL_19135505", "EPI_ISL_19135510", "EPI_ISL_19135511", "EPI_ISL_19137782", "EPI_ISL_19137844", "EPI_ISL_19140762", "EPI_ISL_19141912", "EPI_ISL_19142916", "EPI_ISL_19143060", "EPI_ISL_19143411", "EPI_ISL_19143412", "EPI_ISL_19143551", "EPI_ISL_19143854", "EPI_ISL_19143866", "EPI_ISL_19143953", "EPI_ISL_19143954", "EPI_ISL_19145335", "EPI_ISL_19146181", "EPI_ISL_19146282", "EPI_ISL_19146283", "EPI_ISL_19146308", "EPI_ISL_19146747", "EPI_ISL_19147578", "EPI_ISL_19148775", "EPI_ISL_19151438", "EPI_ISL_19153859", "EPI_ISL_19158778", "EPI_ISL_19158782", "EPI_ISL_19159275", "EPI_ISL_19159918", "EPI_ISL_19159922", "EPI_ISL_19159923", "EPI_ISL_19161805", "EPI_ISL_19164072", "EPI_ISL_19164956", "EPI_ISL_19165347", "EPI_ISL_19165721", "EPI_ISL_19167420", "EPI_ISL_19167714", "EPI_ISL_19169236", "EPI_ISL_19169239", "EPI_ISL_19169240", "EPI_ISL_19169597", "EPI_ISL_19169598", "EPI_ISL_19173640", "EPI_ISL_19173786", "EPI_ISL_19175160", "EPI_ISL_19175165", "EPI_ISL_19175555", "EPI_ISL_19175939", "EPI_ISL_19176642", "EPI_ISL_19176853", "EPI_ISL_19176930", "EPI_ISL_19177366", "EPI_ISL_19177549", "EPI_ISL_19177604", "EPI_ISL_19177633", "EPI_ISL_19178043", "EPI_ISL_19178311", "EPI_ISL_19179820", "EPI_ISL_19182830", "EPI_ISL_19182893", "EPI_ISL_19183095", "EPI_ISL_19183664", "EPI_ISL_19183910", "EPI_ISL_19183917", "EPI_ISL_19183924", "EPI_ISL_19184000", "EPI_ISL_19184031", "EPI_ISL_19184858", "EPI_ISL_19185391", "EPI_ISL_19186372", "EPI_ISL_19187674", "EPI_ISL_19187980", "EPI_ISL_19188471", "EPI_ISL_19189250", "EPI_ISL_19190171", "EPI_ISL_19190915", "EPI_ISL_19191046", "EPI_ISL_19192623", "EPI_ISL_19192806", "EPI_ISL_19192819", "EPI_ISL_19193007", "EPI_ISL_19193543", "EPI_ISL_19193606", "EPI_ISL_19193607", "EPI_ISL_19193617", "EPI_ISL_19195977", "EPI_ISL_19195978", "EPI_ISL_19195979", "EPI_ISL_19195980", "EPI_ISL_19195981", "EPI_ISL_19195982", "EPI_ISL_19196018", "EPI_ISL_19196019", "EPI_ISL_19196020", "EPI_ISL_19196021", "EPI_ISL_19197906", "EPI_ISL_19198127", "EPI_ISL_19198259", "EPI_ISL_19199717", "EPI_ISL_19199719", "EPI_ISL_19200980", "EPI_ISL_19202065", "EPI_ISL_19202066", "EPI_ISL_19203296", "EPI_ISL_19205927", "EPI_ISL_19209656", "EPI_ISL_19210836", "EPI_ISL_19211459", "EPI_ISL_19213222", "EPI_ISL_19213351", "EPI_ISL_19214251", "EPI_ISL_19214614", "EPI_ISL_19216679", "EPI_ISL_19216828", "EPI_ISL_19217899", "EPI_ISL_19219811", "EPI_ISL_19221626", "EPI_ISL_19223353", "EPI_ISL_19225911", "EPI_ISL_19226495", "EPI_ISL_19227219", "EPI_ISL_19228129", "EPI_ISL_19229097", "EPI_ISL_19229565", "EPI_ISL_19230859", "EPI_ISL_19230867", "EPI_ISL_19230971", "EPI_ISL_19232972", "EPI_ISL_19234843", "EPI_ISL_19237942", "EPI_ISL_19239019", "EPI_ISL_19239628", "EPI_ISL_19239713", "EPI_ISL_19243158", "EPI_ISL_19243166", "EPI_ISL_19243171", "EPI_ISL_19243231", "EPI_ISL_19243516", "EPI_ISL_19243810", "EPI_ISL_19244002", "EPI_ISL_19245397", "EPI_ISL_19245398", "EPI_ISL_19245399", "EPI_ISL_19251154", "EPI_ISL_19254798", "EPI_ISL_19256132", "EPI_ISL_19256137", "EPI_ISL_19256138", "EPI_ISL_19256151", "EPI_ISL_19256152", "EPI_ISL_19257013", "EPI_ISL_19257014", "EPI_ISL_19257017", "EPI_ISL_19257108", "EPI_ISL_19257977", "EPI_ISL_19259365", "EPI_ISL_19259369", "EPI_ISL_19259383", "EPI_ISL_19260849", "EPI_ISL_19260850", "EPI_ISL_19261083", "EPI_ISL_19261711", "EPI_ISL_19264977", "EPI_ISL_19265076", "EPI_ISL_19267631", "EPI_ISL_19268349", "EPI_ISL_19269083", "EPI_ISL_19269483", "EPI_ISL_19271166", "EPI_ISL_19271182", "EPI_ISL_19271183", "EPI_ISL_19271567", "EPI_ISL_19271746", "EPI_ISL_19273074", "EPI_ISL_19277027", "EPI_ISL_19277033", "EPI_ISL_19280260", "EPI_ISL_19281017", "EPI_ISL_19282183", "EPI_ISL_19282184", "EPI_ISL_19283996", "EPI_ISL_19286133", "EPI_ISL_19286135", "EPI_ISL_19286138", "EPI_ISL_19286139", "EPI_ISL_19286260", "EPI_ISL_19287266", "EPI_ISL_19287684", "EPI_ISL_19288763", "EPI_ISL_19290906", "EPI_ISL_19292343", "EPI_ISL_19292838", "EPI_ISL_19292841", "EPI_ISL_19297116", "EPI_ISL_19297576", "EPI_ISL_19298817", "EPI_ISL_19299051", "EPI_ISL_19300378", "EPI_ISL_19300385", "EPI_ISL_19300400", "EPI_ISL_19301040", "EPI_ISL_19302363", "EPI_ISL_19302407", "EPI_ISL_19304883", "EPI_ISL_19308329", "EPI_ISL_19308670", "EPI_ISL_19308767", "EPI_ISL_19308875", "EPI_ISL_19308876", "EPI_ISL_19308877", "EPI_ISL_19308906", "EPI_ISL_19309906", "EPI_ISL_19310234", "EPI_ISL_19311057", "EPI_ISL_19311769", "EPI_ISL_19312520", "EPI_ISL_19318476", "EPI_ISL_19319006", "EPI_ISL_19320511", "EPI_ISL_19322462", "EPI_ISL_19324934", "EPI_ISL_19326359", "EPI_ISL_19331433", "EPI_ISL_19332294", "EPI_ISL_19333087", "EPI_ISL_19335441", "EPI_ISL_19340140", "EPI_ISL_19341128", "EPI_ISL_19341144", "EPI_ISL_19344211", "EPI_ISL_19345465", "EPI_ISL_19345845", "EPI_ISL_19346102", "EPI_ISL_19346538", "EPI_ISL_19348673", "EPI_ISL_19351027", "EPI_ISL_19351032", "EPI_ISL_19351033", "EPI_ISL_19351648", "EPI_ISL_19351927", "EPI_ISL_19359999", "EPI_ISL_19360941", "EPI_ISL_19362874", "EPI_ISL_19362955", "EPI_ISL_19363093", "EPI_ISL_19364322", "EPI_ISL_19364675", "EPI_ISL_19365917", "EPI_ISL_19369553", "EPI_ISL_19369713", "EPI_ISL_19374392", "EPI_ISL_19374843", "EPI_ISL_19374844", "EPI_ISL_19374845", "EPI_ISL_19380467", "EPI_ISL_19381264", "EPI_ISL_19381428", "EPI_ISL_19381638", "EPI_ISL_19381992", "EPI_ISL_19381994", "EPI_ISL_19382602", "EPI_ISL_19383694", "EPI_ISL_19384121", "EPI_ISL_19385914", "EPI_ISL_19385980", "EPI_ISL_19387703", "EPI_ISL_19388165", "EPI_ISL_19388758", "EPI_ISL_19391206", "EPI_ISL_19391216", "EPI_ISL_19393434", "EPI_ISL_19393708", "EPI_ISL_19398369", "EPI_ISL_19405918", "EPI_ISL_19408692", "EPI_ISL_19408693", "EPI_ISL_19410044", "EPI_ISL_19410056", "EPI_ISL_19410058", "EPI_ISL_19411869", "EPI_ISL_19412054", "EPI_ISL_19414452", "EPI_ISL_19414669", "EPI_ISL_19414842", "EPI_ISL_19415183", "EPI_ISL_19415272", "EPI_ISL_19415273", "EPI_ISL_19415333", "EPI_ISL_19417987", "EPI_ISL_19418385", "EPI_ISL_19418789", "EPI_ISL_19419991", "EPI_ISL_19425659", "EPI_ISL_19427049", "EPI_ISL_19427050", "EPI_ISL_19427051", "EPI_ISL_19428450", "EPI_ISL_19428673", "EPI_ISL_19431719", "EPI_ISL_19433335", "EPI_ISL_19434973", "EPI_ISL_19438222", "EPI_ISL_19441794", "EPI_ISL_19446721", "EPI_ISL_19446726", "EPI_ISL_19447859", "EPI_ISL_19450094", "EPI_ISL_19452022", "EPI_ISL_19456759", "EPI_ISL_19457982", "EPI_ISL_19458104", "EPI_ISL_19458866", "EPI_ISL_19459469", "EPI_ISL_19463531", "EPI_ISL_19463787", "EPI_ISL_19463811", "EPI_ISL_19464534", "EPI_ISL_19465468", "EPI_ISL_19467713", "EPI_ISL_19467725", "EPI_ISL_19468710", "EPI_ISL_19473728", "EPI_ISL_19474606", "EPI_ISL_19474613", "EPI_ISL_19477107", "EPI_ISL_19478383", "EPI_ISL_19478598", "EPI_ISL_19479514", "EPI_ISL_19480237", "EPI_ISL_19482235", "EPI_ISL_19483178", "EPI_ISL_19483184", "EPI_ISL_19483313", "EPI_ISL_19486141", "EPI_ISL_19488917", "EPI_ISL_19491347", "EPI_ISL_19495855", "EPI_ISL_19498393", "EPI_ISL_19499089", "EPI_ISL_19499133", "EPI_ISL_19499199", "EPI_ISL_19499228", "EPI_ISL_19499640", "EPI_ISL_19499789", "EPI_ISL_19500407", "EPI_ISL_19502648", "EPI_ISL_19503869", "EPI_ISL_19506293", "EPI_ISL_19506337", "EPI_ISL_19506387", "EPI_ISL_19506391", "EPI_ISL_19506410", "EPI_ISL_19506412", "EPI_ISL_19506489", "EPI_ISL_19506517", "EPI_ISL_19506572", "EPI_ISL_19508878", "EPI_ISL_19510518", "EPI_ISL_19511239", "EPI_ISL_19511242", "EPI_ISL_19511245", "EPI_ISL_19511259", "EPI_ISL_19511353", "EPI_ISL_19512930", "EPI_ISL_19513189", "EPI_ISL_19513373", "EPI_ISL_19513376", "EPI_ISL_19517945", "EPI_ISL_19519860", "EPI_ISL_19521406", "EPI_ISL_19522101", "EPI_ISL_19526975", "EPI_ISL_19527216", "EPI_ISL_19527499", "EPI_ISL_19529679", "EPI_ISL_19529699", "EPI_ISL_19530201", "EPI_ISL_19530213", "EPI_ISL_19530220", "EPI_ISL_19532518", "EPI_ISL_19535971", "EPI_ISL_19536358", "EPI_ISL_19537504", "EPI_ISL_19537517", "EPI_ISL_19537806", "EPI_ISL_19539586", "EPI_ISL_19539602", "EPI_ISL_19541318", "EPI_ISL_19543648", "EPI_ISL_19543801", "EPI_ISL_19546384", "EPI_ISL_19550039", "EPI_ISL_19550107", "EPI_ISL_19551038", "EPI_ISL_19551134", "EPI_ISL_19555102", "EPI_ISL_19555114", "EPI_ISL_19555115", "EPI_ISL_19557993", "EPI_ISL_19558034", "EPI_ISL_19560436", "EPI_ISL_19560828", "EPI_ISL_19560838", "EPI_ISL_19561072", "EPI_ISL_19561079", "EPI_ISL_19562666", "EPI_ISL_19575892", "EPI_ISL_19578009", "EPI_ISL_19578384", "EPI_ISL_19585411", "EPI_ISL_19585472", "EPI_ISL_19588050", "EPI_ISL_19588617", "EPI_ISL_19588630", "EPI_ISL_19589209", "EPI_ISL_19589226", "EPI_ISL_19589644", "EPI_ISL_19591990", "EPI_ISL_19600727", "EPI_ISL_19601813", "EPI_ISL_19603076", "EPI_ISL_19603396", "EPI_ISL_19603766", "EPI_ISL_19606872", "EPI_ISL_19606873", "EPI_ISL_19606893", "EPI_ISL_19613477", "EPI_ISL_19613481", "EPI_ISL_19615536", "EPI_ISL_19615541", "EPI_ISL_19615759", "EPI_ISL_19616027", "EPI_ISL_19616479", "EPI_ISL_19618593", "EPI_ISL_19619689", "EPI_ISL_19619734", "EPI_ISL_19619757", "EPI_ISL_19620010", "EPI_ISL_19621954", "EPI_ISL_19622266", "EPI_ISL_19623358", "EPI_ISL_19623503", "EPI_ISL_19627737", "EPI_ISL_19627776", "EPI_ISL_19627778", "EPI_ISL_19630975", "EPI_ISL_19631111", "EPI_ISL_19632962", "EPI_ISL_19634022", "EPI_ISL_19636173", "EPI_ISL_19638051", "EPI_ISL_19639319", "EPI_ISL_19640624", "EPI_ISL_19641389", "EPI_ISL_19641840", "EPI_ISL_19642771", "EPI_ISL_19643417", "EPI_ISL_19643444", "EPI_ISL_19643542", "EPI_ISL_19643616", "EPI_ISL_19643666", "EPI_ISL_19645977", "EPI_ISL_19648297", "EPI_ISL_19655163", "EPI_ISL_19656451", "EPI_ISL_19657328", "EPI_ISL_19657519", "EPI_ISL_19657528", "EPI_ISL_19660180", "EPI_ISL_19661881", "EPI_ISL_19661997", "EPI_ISL_19665170", "EPI_ISL_19666574", "EPI_ISL_19671026", "EPI_ISL_19671598", "EPI_ISL_19671605", "EPI_ISL_19671606", "EPI_ISL_19671607", "EPI_ISL_19671887", "EPI_ISL_19673194", "EPI_ISL_19673201", "EPI_ISL_19676136", "EPI_ISL_19680534", "EPI_ISL_19683226", "EPI_ISL_19683235", "EPI_ISL_19683886", "EPI_ISL_19685790", "EPI_ISL_19685811", "EPI_ISL_19685815", "EPI_ISL_19685993", "EPI_ISL_19688052", "EPI_ISL_19689294", "EPI_ISL_19690695", "EPI_ISL_19693383", "EPI_ISL_19693416", "EPI_ISL_19693419", "EPI_ISL_19696986", "EPI_ISL_19703902", "EPI_ISL_19704129", "EPI_ISL_19704643", "EPI_ISL_19704646", "EPI_ISL_19706712", "EPI_ISL_19706713", "EPI_ISL_19710331", "EPI_ISL_19710359", "EPI_ISL_19711118", "EPI_ISL_19711122", "EPI_ISL_19713810", "EPI_ISL_19713821", "EPI_ISL_19713832", "EPI_ISL_19714066", "EPI_ISL_19715063", "EPI_ISL_19717002", "EPI_ISL_19719176", "EPI_ISL_19719216", "EPI_ISL_19719218", "EPI_ISL_19720736", "EPI_ISL_19720749", "EPI_ISL_19720751", "EPI_ISL_19720757", "EPI_ISL_19720765", "EPI_ISL_19720774", "EPI_ISL_19720781", "EPI_ISL_19721494", "EPI_ISL_19721498", "EPI_ISL_19729765", "EPI_ISL_19729811", "EPI_ISL_19729840", "EPI_ISL_19730527", "EPI_ISL_19735517", "EPI_ISL_19736097", "EPI_ISL_19736266", "EPI_ISL_19736299", "EPI_ISL_19736349", "EPI_ISL_19736527", "EPI_ISL_19739887", "EPI_ISL_19742521", "EPI_ISL_19743439", "EPI_ISL_19743558", "EPI_ISL_19748970", "EPI_ISL_19749286", "EPI_ISL_19749348", "EPI_ISL_19750081", "EPI_ISL_19750087", "EPI_ISL_19750479", "EPI_ISL_19751001", "EPI_ISL_19752288", "EPI_ISL_19755631", "EPI_ISL_19760571", "EPI_ISL_19761402", "EPI_ISL_19765520", "EPI_ISL_19765527", "EPI_ISL_19766581", "EPI_ISL_19766606", "EPI_ISL_19770265", "EPI_ISL_19771106", "EPI_ISL_19773769", "EPI_ISL_19773770", "EPI_ISL_19775232", "EPI_ISL_19775886", "EPI_ISL_19776542", "EPI_ISL_19782005", "EPI_ISL_19783524", "EPI_ISL_19788439", "EPI_ISL_19791384", "EPI_ISL_19794365", "EPI_ISL_19795466", "EPI_ISL_19799368", "EPI_ISL_19799392", "EPI_ISL_19799712", "EPI_ISL_19799810", "EPI_ISL_19800009", "EPI_ISL_19800089", "EPI_ISL_19800094", "EPI_ISL_19800098", "EPI_ISL_19800151", "EPI_ISL_19801979", "EPI_ISL_19803172", "EPI_ISL_19803215", "EPI_ISL_19803225", "EPI_ISL_19803382", "EPI_ISL_19804293", "EPI_ISL_19804491", "EPI_ISL_19805234", "EPI_ISL_19806441", "EPI_ISL_19806502", "EPI_ISL_19806836", "EPI_ISL_19806981", "EPI_ISL_19807128", "EPI_ISL_19807229", "EPI_ISL_19808706", "EPI_ISL_19810078", "EPI_ISL_19810444", "EPI_ISL_19810684", "EPI_ISL_19812168", "EPI_ISL_19814897", "EPI_ISL_19815994", "EPI_ISL_19816020", "EPI_ISL_19816100", "EPI_ISL_19816143", "EPI_ISL_19816182", "EPI_ISL_19816207", "EPI_ISL_19816266", "EPI_ISL_19816276", "EPI_ISL_19816297", "EPI_ISL_19816344", "EPI_ISL_19816356", "EPI_ISL_19816434", "EPI_ISL_19819753", "EPI_ISL_19820321", "EPI_ISL_19822739", "EPI_ISL_19824345", "EPI_ISL_19826284", "EPI_ISL_19826623", "EPI_ISL_19828130", "EPI_ISL_19828714", "EPI_ISL_19831372", "EPI_ISL_19831976", "EPI_ISL_19836303", "EPI_ISL_19838390", "EPI_ISL_19841148", "EPI_ISL_19845799", "EPI_ISL_19845890", "EPI_ISL_19846112", "EPI_ISL_19846130", "EPI_ISL_19846377", "EPI_ISL_19846620", "EPI_ISL_19846658", "EPI_ISL_19848250", "EPI_ISL_19848532", "EPI_ISL_19848571", "EPI_ISL_19850000", "EPI_ISL_19851510", "EPI_ISL_19852484", "EPI_ISL_19853694", "EPI_ISL_19853731", "EPI_ISL_19856123", "EPI_ISL_19859249", "EPI_ISL_19859991", "EPI_ISL_19860001", "EPI_ISL_19860628", "EPI_ISL_19860631", "EPI_ISL_19860648", "EPI_ISL_19860650", "EPI_ISL_19862012"])
for line in eachline("chronics_2025_04_14__5383seq.ndjson")
    j = JSON3.read(line)
    name = EPI_ISL(j.seqName)
    push!(EPI_ISLs_2025_4_14, name)
end
println(length(EPI_ISLs_2025_4_14))
println(length(EPI_ISLs_2025_5_7))

5382
5572


In [39]:
date_now = Dates.format(now(), "yyyy-mm-dd_IMMp")
println(date_now)
diff_1_2 = setdiff(EPI_ISLs_2025_4_14, EPI_ISLs_2025_5_7)
diff_2_1 = setdiff(EPI_ISLs_2025_5_7, EPI_ISLs_2025_4_14)
diff_12_arr = String[]
diff_21_arr = String[]
for seq in diff_1_2
    push!(diff_12_arr, seq)
end
for seq in diff_2_1
    push!(diff_21_arr, seq)
end
diff_12_sort = sort(diff_12_arr, by = x -> (length(x), x))
diff_21_sort = sort(diff_21_arr, by = x -> (length(x), x))
for s in diff_12_sort
    print(s, ", ")
end
println("\n"^5)
open("__new_chr_seqs_$(date_now).txt", "w") do g
    for s in diff_21_sort
        print(s, ", ")
        print(g, s, ", ")
    end
end

2025-05-07_541AM
EPI_ISL_1376522, EPI_ISL_14917292, EPI_ISL_15175896, EPI_ISL_16154520, EPI_ISL_16157031, EPI_ISL_18359229, 





EPI_ISL_402124, EPI_ISL_1909055, EPI_ISL_2589968, EPI_ISL_3215721, EPI_ISL_3215722, EPI_ISL_3215726, EPI_ISL_6698637, EPI_ISL_7205152, EPI_ISL_7876524, EPI_ISL_12968380, EPI_ISL_13092906, EPI_ISL_13323290, EPI_ISL_13503362, EPI_ISL_13710357, EPI_ISL_13732799, EPI_ISL_13925703, EPI_ISL_13986498, EPI_ISL_14234017, EPI_ISL_14257902, EPI_ISL_14377405, EPI_ISL_14377421, EPI_ISL_14494098, EPI_ISL_14496407, EPI_ISL_14561987, EPI_ISL_14562000, EPI_ISL_14846399, EPI_ISL_14846400, EPI_ISL_14960752, EPI_ISL_15040430, EPI_ISL_15058903, EPI_ISL_15120575, EPI_ISL_15215974, EPI_ISL_15333287, EPI_ISL_15420138, EPI_ISL_15424276, EPI_ISL_15463120, EPI_ISL_15473942, EPI_ISL_15490728, EPI_ISL_15760609, EPI_ISL_15840082, EPI_ISL_15910198, EPI_ISL_15959369, EPI_ISL_16003255, EPI_ISL_16005246, EPI_ISL_16019744, EPI_ISL_16019756, EPI_ISL_16034832, EPI_ISL_16036598, EPI_ISL_16045424

In [57]:

set1 = stringlist_to_strings_set("EPI_ISL_474824, EPI_ISL_508674, EPI_ISL_510148, EPI_ISL_516999, EPI_ISL_517000, EPI_ISL_528438, EPI_ISL_534720, EPI_ISL_539541, EPI_ISL_539542, EPI_ISL_593478, EPI_ISL_593479, EPI_ISL_593480, EPI_ISL_593553, EPI_ISL_593554, EPI_ISL_593555, EPI_ISL_593556, EPI_ISL_593557, EPI_ISL_593558, EPI_ISL_596228, EPI_ISL_602912, EPI_ISL_654166, EPI_ISL_654170, EPI_ISL_654172, EPI_ISL_654173, EPI_ISL_654182, EPI_ISL_654186, EPI_ISL_654191, EPI_ISL_654194, EPI_ISL_678289, EPI_ISL_686537, EPI_ISL_732971, EPI_ISL_753676, EPI_ISL_776770, EPI_ISL_801876, EPI_ISL_812862, EPI_ISL_831496, EPI_ISL_856750, EPI_ISL_872778, EPI_ISL_941340, EPI_ISL_949208, EPI_ISL_959309, EPI_ISL_979349, EPI_ISL_1014810, EPI_ISL_1039159, EPI_ISL_1039839, EPI_ISL_1059094, EPI_ISL_1081672, EPI_ISL_1081673, EPI_ISL_1090851, EPI_ISL_1104882, EPI_ISL_1105146, EPI_ISL_1105179, EPI_ISL_1105235, EPI_ISL_1158169, EPI_ISL_1200867, EPI_ISL_1200912, EPI_ISL_1209365, EPI_ISL_1225620, EPI_ISL_1241756, EPI_ISL_1248458, EPI_ISL_1248485, EPI_ISL_1248497, EPI_ISL_1257978, EPI_ISL_1261009, EPI_ISL_1295569, EPI_ISL_1295575, EPI_ISL_1309105, EPI_ISL_1347940, EPI_ISL_1347941, EPI_ISL_1347942, EPI_ISL_1366562, EPI_ISL_1366563, EPI_ISL_1366564, EPI_ISL_1366565, EPI_ISL_1366566, EPI_ISL_1366567, EPI_ISL_1366568, EPI_ISL_1366569, EPI_ISL_1366570, EPI_ISL_1366571, EPI_ISL_1366572, EPI_ISL_1366573, EPI_ISL_1366792, EPI_ISL_1372287, EPI_ISL_1372288, EPI_ISL_1376522, EPI_ISL_1376523, EPI_ISL_1378739, EPI_ISL_1403404, EPI_ISL_1469973, EPI_ISL_1470396, EPI_ISL_1473700, EPI_ISL_1477334, EPI_ISL_1483302, EPI_ISL_1490655, EPI_ISL_1490669, EPI_ISL_1495749, EPI_ISL_1522107, EPI_ISL_1534324, EPI_ISL_1547461, EPI_ISL_1575358, EPI_ISL_1578495, EPI_ISL_1626185, EPI_ISL_1637040, EPI_ISL_1668821, EPI_ISL_1668822, EPI_ISL_1668823, EPI_ISL_1668824, EPI_ISL_1668825, EPI_ISL_1670378, EPI_ISL_1671330, EPI_ISL_1675190, EPI_ISL_1675203, EPI_ISL_1738308, EPI_ISL_1743263, EPI_ISL_1744401, EPI_ISL_1756179, EPI_ISL_1756180, EPI_ISL_1792929, EPI_ISL_1829054, EPI_ISL_1829108, EPI_ISL_1840893, EPI_ISL_1855854, EPI_ISL_1904901, EPI_ISL_1904903, EPI_ISL_1908476, EPI_ISL_1935116, EPI_ISL_1935282, EPI_ISL_1941336, EPI_ISL_1941816, EPI_ISL_1965714, EPI_ISL_1965722, EPI_ISL_2001260, EPI_ISL_2001292, EPI_ISL_2030332, EPI_ISL_2035047, EPI_ISL_2080876, EPI_ISL_2096935, EPI_ISL_2140680, EPI_ISL_2142447, EPI_ISL_2159603, EPI_ISL_2170618, EPI_ISL_2179080, EPI_ISL_2179597, EPI_ISL_2179598, EPI_ISL_2179600, EPI_ISL_2179601, EPI_ISL_2179635, EPI_ISL_2193387, EPI_ISL_2193781, EPI_ISL_2193790, EPI_ISL_2229473, EPI_ISL_2232987, EPI_ISL_2232988, EPI_ISL_2245655, EPI_ISL_2246946, EPI_ISL_2272316, EPI_ISL_2281463, EPI_ISL_2285732, EPI_ISL_2289324, EPI_ISL_2335139, EPI_ISL_2355027, EPI_ISL_2373667, EPI_ISL_2373676, EPI_ISL_2373689, EPI_ISL_2373915, EPI_ISL_2373976, EPI_ISL_2376734, EPI_ISL_2385134, EPI_ISL_2397307, EPI_ISL_2397308, EPI_ISL_2397309, EPI_ISL_2397310, EPI_ISL_2397311, EPI_ISL_2397312, EPI_ISL_2397313, EPI_ISL_2403056, EPI_ISL_2408213, EPI_ISL_2443102, EPI_ISL_2443306, EPI_ISL_2451852, EPI_ISL_2453771, EPI_ISL_2456706, EPI_ISL_2466638, EPI_ISL_2482552, EPI_ISL_2482891, EPI_ISL_2484152, EPI_ISL_2492266, EPI_ISL_2501697, EPI_ISL_2504017, EPI_ISL_2510252, EPI_ISL_2526835, EPI_ISL_2537393, EPI_ISL_2545260, EPI_ISL_2567482, EPI_ISL_2567516, EPI_ISL_2598472, EPI_ISL_2621566, EPI_ISL_2622092, EPI_ISL_2626505, EPI_ISL_2629070, EPI_ISL_2629071, EPI_ISL_2629072, EPI_ISL_2629073, EPI_ISL_2629074, EPI_ISL_2629075, EPI_ISL_2646107, EPI_ISL_2658958, EPI_ISL_2658962, EPI_ISL_2658963, EPI_ISL_2658970, EPI_ISL_2658971, EPI_ISL_2658972, EPI_ISL_2681259, EPI_ISL_2686814, EPI_ISL_2686837, EPI_ISL_2713004, EPI_ISL_2716246, EPI_ISL_2746031, EPI_ISL_2758178, EPI_ISL_2758179, EPI_ISL_2776212, EPI_ISL_2790083, EPI_ISL_2791260, EPI_ISL_2810326, EPI_ISL_2811857, EPI_ISL_2817504, EPI_ISL_2820526, EPI_ISL_2828407, EPI_ISL_2833904, EPI_ISL_2858161, EPI_ISL_2858877, EPI_ISL_2860316, EPI_ISL_2868572, EPI_ISL_2868616, EPI_ISL_2876377, EPI_ISL_2894033, EPI_ISL_2903438, EPI_ISL_2931876, EPI_ISL_2931884, EPI_ISL_2931896, EPI_ISL_2931903, EPI_ISL_2955288, EPI_ISL_2955320, EPI_ISL_2966985, EPI_ISL_2978243, EPI_ISL_2978352, EPI_ISL_2984725, EPI_ISL_2990101, EPI_ISL_2993722, EPI_ISL_3010321, EPI_ISL_3029841, EPI_ISL_3030114, EPI_ISL_3030118, EPI_ISL_3030145, EPI_ISL_3030738, EPI_ISL_3032627, EPI_ISL_3033635, EPI_ISL_3039352, EPI_ISL_3045789, EPI_ISL_3053903, EPI_ISL_3061061, EPI_ISL_3064314, EPI_ISL_3129808, EPI_ISL_3130077, EPI_ISL_3130081, EPI_ISL_3130177, EPI_ISL_3130302, EPI_ISL_3132631, EPI_ISL_3133023, EPI_ISL_3153240, EPI_ISL_3164424, EPI_ISL_3185346, EPI_ISL_3209041, EPI_ISL_3212959, EPI_ISL_3246237, EPI_ISL_3251444, EPI_ISL_3251605, EPI_ISL_3259560, EPI_ISL_3275376, EPI_ISL_3339536, EPI_ISL_3356734, EPI_ISL_3358574, EPI_ISL_3370176, EPI_ISL_3394321, EPI_ISL_3396491, EPI_ISL_3414767, EPI_ISL_3414889, EPI_ISL_3415104, EPI_ISL_3415226, EPI_ISL_3426474, EPI_ISL_3446827, EPI_ISL_3453279, EPI_ISL_3457423, EPI_ISL_3459118, EPI_ISL_3471360, EPI_ISL_3475993, EPI_ISL_3497667, EPI_ISL_3503811, EPI_ISL_3504036, EPI_ISL_3509013, EPI_ISL_3556945, EPI_ISL_3634003, EPI_ISL_3634004, EPI_ISL_3635506, EPI_ISL_3657112, EPI_ISL_3712919, EPI_ISL_3771882, EPI_ISL_3779849, EPI_ISL_3838306, EPI_ISL_3891136, EPI_ISL_3933252, EPI_ISL_3937027, EPI_ISL_3957778, EPI_ISL_3958461, EPI_ISL_3958994, EPI_ISL_3982251, EPI_ISL_4029567, EPI_ISL_4052911, EPI_ISL_4072038, EPI_ISL_4096626, EPI_ISL_4096639, EPI_ISL_4114033, EPI_ISL_4124532, EPI_ISL_4178790, EPI_ISL_4193135, EPI_ISL_4198270, EPI_ISL_4251611, EPI_ISL_4261403, EPI_ISL_4261411, EPI_ISL_4281194, EPI_ISL_4284228, EPI_ISL_4295678, EPI_ISL_4298277, EPI_ISL_4298278, EPI_ISL_4298279, EPI_ISL_4309817, EPI_ISL_4415808, EPI_ISL_4440075, EPI_ISL_4515444, EPI_ISL_4525691, EPI_ISL_4525698, EPI_ISL_4525700, EPI_ISL_4531313, EPI_ISL_4536418, EPI_ISL_4576991, EPI_ISL_4577474, EPI_ISL_4625101, EPI_ISL_4652284, EPI_ISL_4769386, EPI_ISL_4775547, EPI_ISL_4875939, EPI_ISL_4930863, EPI_ISL_4935777, EPI_ISL_4935949, EPI_ISL_4936095, EPI_ISL_4936533, EPI_ISL_4949584, EPI_ISL_5018695, EPI_ISL_5033183, EPI_ISL_5056434, EPI_ISL_5059980, EPI_ISL_5099310, EPI_ISL_5132437, EPI_ISL_5132595, EPI_ISL_5145656, EPI_ISL_5196003, EPI_ISL_5265214, EPI_ISL_5280146, EPI_ISL_5307398, EPI_ISL_5323016, EPI_ISL_5332877, EPI_ISL_5332878, EPI_ISL_5395558, EPI_ISL_5446154, EPI_ISL_5463914, EPI_ISL_5532714, EPI_ISL_5592605, EPI_ISL_5620309, EPI_ISL_5621224, EPI_ISL_5628248, EPI_ISL_5639913, EPI_ISL_5640459, EPI_ISL_5649323, EPI_ISL_5650474, EPI_ISL_5655562, EPI_ISL_5680241, EPI_ISL_5692774, EPI_ISL_5749185, EPI_ISL_5780324, EPI_ISL_5814411, EPI_ISL_5865553, EPI_ISL_5892132, EPI_ISL_5922347, EPI_ISL_5935407, EPI_ISL_5944665, EPI_ISL_5944669, EPI_ISL_5944842, EPI_ISL_5944948, EPI_ISL_5946914, EPI_ISL_6017746, EPI_ISL_6017747, EPI_ISL_6076460, EPI_ISL_6208674, EPI_ISL_6208675, EPI_ISL_6208676, EPI_ISL_6227177, EPI_ISL_6227208, EPI_ISL_6262165, EPI_ISL_6324366, EPI_ISL_6327943, EPI_ISL_6381841, EPI_ISL_6384755, EPI_ISL_6574278, EPI_ISL_6584511, EPI_ISL_6604686, EPI_ISL_6605003, EPI_ISL_6605659, EPI_ISL_6628662, EPI_ISL_6642599, EPI_ISL_6666037, EPI_ISL_6710721, EPI_ISL_6735468, EPI_ISL_6737833, EPI_ISL_6783610, EPI_ISL_6810267, EPI_ISL_6814822, EPI_ISL_6826536, EPI_ISL_6842893, EPI_ISL_6863316, EPI_ISL_6863457, EPI_ISL_6865741, EPI_ISL_6938691, EPI_ISL_6976497, EPI_ISL_6977941, EPI_ISL_7000663, EPI_ISL_7015624, EPI_ISL_7015625, EPI_ISL_7135374, EPI_ISL_7159687, EPI_ISL_7204318, EPI_ISL_7334013, EPI_ISL_7361483, EPI_ISL_7361527, EPI_ISL_7452581, EPI_ISL_7452603, EPI_ISL_7456462, EPI_ISL_7458003, EPI_ISL_7502990, EPI_ISL_7503221, EPI_ISL_7592661, EPI_ISL_7592687, EPI_ISL_7652115, EPI_ISL_7660179, EPI_ISL_7707631, EPI_ISL_7711813, EPI_ISL_7729239, EPI_ISL_7806535, EPI_ISL_7806536, EPI_ISL_7806549, EPI_ISL_7813896, EPI_ISL_7861580, EPI_ISL_7908114, EPI_ISL_7961502, EPI_ISL_7976931, EPI_ISL_7980711, EPI_ISL_8001538, EPI_ISL_8035582, EPI_ISL_8131224, EPI_ISL_8132253, EPI_ISL_8151798, EPI_ISL_8153087, EPI_ISL_8166542, EPI_ISL_8189765, EPI_ISL_8189775, EPI_ISL_8204828, EPI_ISL_8205040, EPI_ISL_8207600, EPI_ISL_8215783, EPI_ISL_8251200, EPI_ISL_8263463, EPI_ISL_8376888, EPI_ISL_8479639, EPI_ISL_8479640, EPI_ISL_8563217, EPI_ISL_8563218, EPI_ISL_8563219, EPI_ISL_8615077, EPI_ISL_8627379, EPI_ISL_8669281, EPI_ISL_8712661, EPI_ISL_8725398, EPI_ISL_8725399, EPI_ISL_8725400, EPI_ISL_8725401, EPI_ISL_8725402, EPI_ISL_8725403, EPI_ISL_8725404, EPI_ISL_8725405, EPI_ISL_8725406, EPI_ISL_8725407, EPI_ISL_8725408, EPI_ISL_8725409, EPI_ISL_8732699, EPI_ISL_8732807, EPI_ISL_8732841, EPI_ISL_8750545, EPI_ISL_8754305, EPI_ISL_8766992, EPI_ISL_8800409, EPI_ISL_8806077, EPI_ISL_8806082, EPI_ISL_8806084, EPI_ISL_8819629, EPI_ISL_8825833, EPI_ISL_8828662, EPI_ISL_8887845, EPI_ISL_8887874, EPI_ISL_8973278, EPI_ISL_9021214, EPI_ISL_9141923, EPI_ISL_9155607, EPI_ISL_9201951, EPI_ISL_9242265, EPI_ISL_9242269, EPI_ISL_9316982, EPI_ISL_9319180, EPI_ISL_9630717, EPI_ISL_9637481, EPI_ISL_9637840, EPI_ISL_9702285, EPI_ISL_9735644, EPI_ISL_9844246, EPI_ISL_9873278, EPI_ISL_9907655, EPI_ISL_9949797, EPI_ISL_9949799, EPI_ISL_10124646, EPI_ISL_10127751, EPI_ISL_10128185, EPI_ISL_10132276, EPI_ISL_10185453, EPI_ISL_10195257, EPI_ISL_10195262, EPI_ISL_10195263, EPI_ISL_10195264, EPI_ISL_10195305, EPI_ISL_10230612, EPI_ISL_10239201, EPI_ISL_10251304, EPI_ISL_10329391, EPI_ISL_10329558, EPI_ISL_10352747, EPI_ISL_10397517, EPI_ISL_10451205, EPI_ISL_10451252, EPI_ISL_10548912, EPI_ISL_10548913, EPI_ISL_10548915, EPI_ISL_10548916, EPI_ISL_10548917, EPI_ISL_10548918, EPI_ISL_10548919, EPI_ISL_10548920, EPI_ISL_10548921, EPI_ISL_10548922, EPI_ISL_10549162, EPI_ISL_10549163, EPI_ISL_10549164, EPI_ISL_10549165, EPI_ISL_10549166, EPI_ISL_10591808, EPI_ISL_10681118, EPI_ISL_10706292, EPI_ISL_10712909, EPI_ISL_10815044, EPI_ISL_10816730, EPI_ISL_10816731, EPI_ISL_10816732, EPI_ISL_10816733, EPI_ISL_10816734, EPI_ISL_10816735, EPI_ISL_10816736, EPI_ISL_10816737, EPI_ISL_10816738, EPI_ISL_10816739, EPI_ISL_10816741, EPI_ISL_10816742, EPI_ISL_10816743, EPI_ISL_10816744, EPI_ISL_10824028, EPI_ISL_10876034, EPI_ISL_10876749, EPI_ISL_10899907, EPI_ISL_10942195, EPI_ISL_10981395, EPI_ISL_10995323, EPI_ISL_11025821, EPI_ISL_11025897, EPI_ISL_11030507, EPI_ISL_11036385, EPI_ISL_11036386, EPI_ISL_11036389, EPI_ISL_11036399, EPI_ISL_11036451, EPI_ISL_11036666, EPI_ISL_11036688, EPI_ISL_11036712, EPI_ISL_11036915, EPI_ISL_11036917, EPI_ISL_11050902, EPI_ISL_11055609, EPI_ISL_11110730, EPI_ISL_11167907, EPI_ISL_11173072, EPI_ISL_11219235, EPI_ISL_11219236, EPI_ISL_11221773, EPI_ISL_11221782, EPI_ISL_11222620, EPI_ISL_11229672, EPI_ISL_11248919, EPI_ISL_11256669, EPI_ISL_11290054, EPI_ISL_11295642, EPI_ISL_11296415, EPI_ISL_11356268, EPI_ISL_11356269, EPI_ISL_11403393, EPI_ISL_11437359, EPI_ISL_11482304, EPI_ISL_11503909, EPI_ISL_11504189, EPI_ISL_11517385, EPI_ISL_11565840, EPI_ISL_11576757, EPI_ISL_11600821, EPI_ISL_11600822, EPI_ISL_11621597, EPI_ISL_11657715, EPI_ISL_11661806, EPI_ISL_11695384, EPI_ISL_11742572, EPI_ISL_11742812, EPI_ISL_11747289, EPI_ISL_11778548, EPI_ISL_11787443, EPI_ISL_11798407, EPI_ISL_11798458, EPI_ISL_11801453, EPI_ISL_11816904, EPI_ISL_11825798, EPI_ISL_11826326, EPI_ISL_11826898, EPI_ISL_11871462, EPI_ISL_11886470, EPI_ISL_11886479, EPI_ISL_11886624, EPI_ISL_11897546, EPI_ISL_11961223, EPI_ISL_11968830, EPI_ISL_11968876, EPI_ISL_11970393, EPI_ISL_11976211, EPI_ISL_11994571, EPI_ISL_11995938, EPI_ISL_12001179, EPI_ISL_12001180, EPI_ISL_12021469, EPI_ISL_12039060, EPI_ISL_12063598, EPI_ISL_12063599, EPI_ISL_12063600, EPI_ISL_12063601, EPI_ISL_12063602, EPI_ISL_12079999, EPI_ISL_12080356, EPI_ISL_12083619, EPI_ISL_12089943, EPI_ISL_12095320, EPI_ISL_12097355, EPI_ISL_12108965, EPI_ISL_12109074, EPI_ISL_12127282, EPI_ISL_12137683, EPI_ISL_12139045, EPI_ISL_12139066, EPI_ISL_12145506, EPI_ISL_12146579, EPI_ISL_12148419, EPI_ISL_12150077, EPI_ISL_12150259, EPI_ISL_12150361, EPI_ISL_12150484, EPI_ISL_12155759, EPI_ISL_12155809, EPI_ISL_12157165, EPI_ISL_12157166, EPI_ISL_12157187, EPI_ISL_12158960, EPI_ISL_12162547, EPI_ISL_12168418, EPI_ISL_12171333, EPI_ISL_12171674, EPI_ISL_12172132, EPI_ISL_12172842, EPI_ISL_12173486, EPI_ISL_12173730, EPI_ISL_12173879, EPI_ISL_12174612, EPI_ISL_12174734, EPI_ISL_12174735, EPI_ISL_12174736, EPI_ISL_12174739, EPI_ISL_12174942, EPI_ISL_12175020, EPI_ISL_12175024, EPI_ISL_12175203, EPI_ISL_12176184, EPI_ISL_12207682, EPI_ISL_12220762, EPI_ISL_12240087, EPI_ISL_12240983, EPI_ISL_12278477, EPI_ISL_12278678, EPI_ISL_12278997, EPI_ISL_12284821, EPI_ISL_12293549, EPI_ISL_12293608, EPI_ISL_12323992, EPI_ISL_12325408, EPI_ISL_12350967, EPI_ISL_12351281, EPI_ISL_12355622, EPI_ISL_12422504, EPI_ISL_12425033, EPI_ISL_12430022, EPI_ISL_12464077, EPI_ISL_12467081, EPI_ISL_12473693, EPI_ISL_12475004, EPI_ISL_12486436, EPI_ISL_12488441, EPI_ISL_12501519, EPI_ISL_12511246, EPI_ISL_12530780, EPI_ISL_12531462, EPI_ISL_12539663, EPI_ISL_12564261, EPI_ISL_12568162, EPI_ISL_12568208, EPI_ISL_12575298, EPI_ISL_12590958, EPI_ISL_12602903, EPI_ISL_12611697, EPI_ISL_12611721, EPI_ISL_12616586, EPI_ISL_12622901, EPI_ISL_12622902, EPI_ISL_12623284, EPI_ISL_12639714, EPI_ISL_12639788, EPI_ISL_12639917, EPI_ISL_12645823, EPI_ISL_12646116, EPI_ISL_12647336, EPI_ISL_12652423, EPI_ISL_12654179, EPI_ISL_12685124, EPI_ISL_12685126, EPI_ISL_12698937, EPI_ISL_12701772, EPI_ISL_12701820, EPI_ISL_12701858, EPI_ISL_12701867, EPI_ISL_12701871, EPI_ISL_12701895, EPI_ISL_12703517, EPI_ISL_12710394, EPI_ISL_12735772, EPI_ISL_12739317, EPI_ISL_12739425, EPI_ISL_12742126, EPI_ISL_12744193, EPI_ISL_12753427, EPI_ISL_12754976, EPI_ISL_12763802, EPI_ISL_12771614, EPI_ISL_12784028, EPI_ISL_12789812, EPI_ISL_12789846, EPI_ISL_12799990, EPI_ISL_12805921, EPI_ISL_12808264, EPI_ISL_12809016, EPI_ISL_12811012, EPI_ISL_12822481, EPI_ISL_12822483, EPI_ISL_12829036, EPI_ISL_12830215, EPI_ISL_12830827, EPI_ISL_12843060, EPI_ISL_12844170, EPI_ISL_12851188, EPI_ISL_12851233, EPI_ISL_12851285, EPI_ISL_12862705, EPI_ISL_12871249, EPI_ISL_12896994, EPI_ISL_12903760, EPI_ISL_12906172, EPI_ISL_12911895, EPI_ISL_12911898, EPI_ISL_12914019, EPI_ISL_12926555, EPI_ISL_12926955, EPI_ISL_12932770, EPI_ISL_12953136, EPI_ISL_12953419, EPI_ISL_12955795, EPI_ISL_12958668, EPI_ISL_12961699, EPI_ISL_12980420, EPI_ISL_12993020, EPI_ISL_12995035, EPI_ISL_12995230, EPI_ISL_12995281, EPI_ISL_12995422, EPI_ISL_13000815, EPI_ISL_13001148, EPI_ISL_13011225, EPI_ISL_13011226, EPI_ISL_13019919, EPI_ISL_13026163, EPI_ISL_13028133, EPI_ISL_13040401, EPI_ISL_13044133, EPI_ISL_13047387, EPI_ISL_13050078, EPI_ISL_13050270, EPI_ISL_13050793, EPI_ISL_13051740, EPI_ISL_13051790, EPI_ISL_13051970, EPI_ISL_13052096, EPI_ISL_13052204, EPI_ISL_13055324, EPI_ISL_13055618, EPI_ISL_13056142, EPI_ISL_13056190, EPI_ISL_13065554, EPI_ISL_13066396, EPI_ISL_13066472, EPI_ISL_13066603, EPI_ISL_13085784, EPI_ISL_13085991, EPI_ISL_13086417, EPI_ISL_13086737, EPI_ISL_13086826, EPI_ISL_13086831, EPI_ISL_13088769, EPI_ISL_13088942, EPI_ISL_13089020, EPI_ISL_13089384, EPI_ISL_13091912, EPI_ISL_13091925, EPI_ISL_13091929, EPI_ISL_13092725, EPI_ISL_13093369, EPI_ISL_13093922, EPI_ISL_13094368, EPI_ISL_13108591, EPI_ISL_13110029, EPI_ISL_13110031, EPI_ISL_13123133, EPI_ISL_13129353, EPI_ISL_13129387, EPI_ISL_13131091, EPI_ISL_13131118, EPI_ISL_13132070, EPI_ISL_13133128, EPI_ISL_13133359, EPI_ISL_13152570, EPI_ISL_13155404, EPI_ISL_13157537, EPI_ISL_13157638, EPI_ISL_13158312, EPI_ISL_13158314, EPI_ISL_13160040, EPI_ISL_13166402, EPI_ISL_13166803, EPI_ISL_13169047, EPI_ISL_13169050, EPI_ISL_13172328, EPI_ISL_13172329, EPI_ISL_13176279, EPI_ISL_13176281, EPI_ISL_13178754, EPI_ISL_13183984, EPI_ISL_13192066, EPI_ISL_13192072, EPI_ISL_13192202, EPI_ISL_13199746, EPI_ISL_13199759, EPI_ISL_13202578, EPI_ISL_13203062, EPI_ISL_13210987, EPI_ISL_13211404, EPI_ISL_13214299, EPI_ISL_13215742, EPI_ISL_13230467, EPI_ISL_13231553, EPI_ISL_13232492, EPI_ISL_13232499, EPI_ISL_13242111, EPI_ISL_13242155, EPI_ISL_13244164, EPI_ISL_13244243, EPI_ISL_13244707, EPI_ISL_13251406, EPI_ISL_13251514, EPI_ISL_13253132, EPI_ISL_13253164, EPI_ISL_13253244, EPI_ISL_13253296, EPI_ISL_13253344, EPI_ISL_13253416, EPI_ISL_13253427, EPI_ISL_13253489, EPI_ISL_13253493, EPI_ISL_13253550, EPI_ISL_13253551, EPI_ISL_13253570, EPI_ISL_13264387, EPI_ISL_13271922, EPI_ISL_13272001, EPI_ISL_13272223, EPI_ISL_13273550, EPI_ISL_13284168, EPI_ISL_13288721, EPI_ISL_13289213, EPI_ISL_13289619, EPI_ISL_13289774, EPI_ISL_13294595, EPI_ISL_13299119, EPI_ISL_13304429, EPI_ISL_13304451, EPI_ISL_13304482, EPI_ISL_13306995, EPI_ISL_13307081, EPI_ISL_13312837, EPI_ISL_13314960, EPI_ISL_13319769, EPI_ISL_13322028, EPI_ISL_13322954, EPI_ISL_13322975, EPI_ISL_13328732, EPI_ISL_13332433, EPI_ISL_13332459, EPI_ISL_13332921, EPI_ISL_13333047, EPI_ISL_13338249, EPI_ISL_13339980, EPI_ISL_13345908, EPI_ISL_13349432, EPI_ISL_13350581, EPI_ISL_13354243, EPI_ISL_13356160, EPI_ISL_13356194, EPI_ISL_13357233, EPI_ISL_13358809, EPI_ISL_13358893, EPI_ISL_13358894, EPI_ISL_13358962, EPI_ISL_13361313, EPI_ISL_13361419, EPI_ISL_13362130, EPI_ISL_13368501, EPI_ISL_13368552, EPI_ISL_13369326, EPI_ISL_13376289, EPI_ISL_13385215, EPI_ISL_13386427, EPI_ISL_13388894, EPI_ISL_13389863, EPI_ISL_13389864, EPI_ISL_13394010, EPI_ISL_13398372, EPI_ISL_13398391, EPI_ISL_13400530, EPI_ISL_13405240, EPI_ISL_13406133, EPI_ISL_13407391, EPI_ISL_13407802, EPI_ISL_13408380, EPI_ISL_13410054, EPI_ISL_13410128, EPI_ISL_13412509, EPI_ISL_13417422, EPI_ISL_13417595, EPI_ISL_13417637, EPI_ISL_13422063, EPI_ISL_13425805, EPI_ISL_13426860, EPI_ISL_13426861, EPI_ISL_13426862, EPI_ISL_13440246, EPI_ISL_13440269, EPI_ISL_13440488, EPI_ISL_13443257, EPI_ISL_13454711, EPI_ISL_13455972, EPI_ISL_13463270, EPI_ISL_13464004, EPI_ISL_13465109, EPI_ISL_13466588, EPI_ISL_13466629, EPI_ISL_13466644, EPI_ISL_13467321, EPI_ISL_13467676, EPI_ISL_13470130, EPI_ISL_13470158, EPI_ISL_13476507, EPI_ISL_13477158, EPI_ISL_13478276, EPI_ISL_13478421, EPI_ISL_13478425, EPI_ISL_13478448, EPI_ISL_13478487, EPI_ISL_13480851, EPI_ISL_13481704, EPI_ISL_13482848, EPI_ISL_13483067, EPI_ISL_13483538, EPI_ISL_13483605, EPI_ISL_13483653, EPI_ISL_13489191, EPI_ISL_13501768, EPI_ISL_13502856, EPI_ISL_13502894, EPI_ISL_13503354, EPI_ISL_13503413, EPI_ISL_13504084, EPI_ISL_13504103, EPI_ISL_13504568, EPI_ISL_13518485, EPI_ISL_13519541, EPI_ISL_13522554, EPI_ISL_13522707, EPI_ISL_13529086, EPI_ISL_13534848, EPI_ISL_13535844, EPI_ISL_13536846, EPI_ISL_13536992, EPI_ISL_13538620, EPI_ISL_13538621, EPI_ISL_13539746, EPI_ISL_13552269, EPI_ISL_13560139, EPI_ISL_13563849, EPI_ISL_13563900, EPI_ISL_13564453, EPI_ISL_13564901, EPI_ISL_13566717, EPI_ISL_13571629, EPI_ISL_13572579, EPI_ISL_13572829, EPI_ISL_13573543, EPI_ISL_13573707, EPI_ISL_13578717, EPI_ISL_13585459, EPI_ISL_13585850, EPI_ISL_13592668, EPI_ISL_13605930, EPI_ISL_13608346, EPI_ISL_13611346, EPI_ISL_13611373, EPI_ISL_13612631, EPI_ISL_13612636, EPI_ISL_13614328, EPI_ISL_13615804, EPI_ISL_13615826, EPI_ISL_13616458, EPI_ISL_13617390, EPI_ISL_13617475, EPI_ISL_13617493, EPI_ISL_13619646, EPI_ISL_13622657, EPI_ISL_13622831, EPI_ISL_13622874, EPI_ISL_13624440, EPI_ISL_13624441, EPI_ISL_13626226, EPI_ISL_13633558, EPI_ISL_13633729, EPI_ISL_13636934, EPI_ISL_13637141, EPI_ISL_13637734, EPI_ISL_13638494, EPI_ISL_13642059, EPI_ISL_13653660, EPI_ISL_13665096, EPI_ISL_13665110, EPI_ISL_13666764, EPI_ISL_13677872, EPI_ISL_13688333, EPI_ISL_13691966, EPI_ISL_13692397, EPI_ISL_13694663, EPI_ISL_13696042, EPI_ISL_13698648, EPI_ISL_13700128, EPI_ISL_13700243, EPI_ISL_13700756, EPI_ISL_13701029, EPI_ISL_13701810, EPI_ISL_13710357, EPI_ISL_13715129, EPI_ISL_13715746, EPI_ISL_13716400, EPI_ISL_13716943, EPI_ISL_13723111, EPI_ISL_13723805, EPI_ISL_13726415, EPI_ISL_13726482, EPI_ISL_13729045, EPI_ISL_13734474, EPI_ISL_13734683, EPI_ISL_13738059, EPI_ISL_13740111, EPI_ISL_13740674, EPI_ISL_13741330, EPI_ISL_13744798, EPI_ISL_13744799, EPI_ISL_13745638, EPI_ISL_13745641, EPI_ISL_13748166, EPI_ISL_13750726, EPI_ISL_13750730, EPI_ISL_13750760, EPI_ISL_13750771, EPI_ISL_13750936, EPI_ISL_13750937, EPI_ISL_13751254, EPI_ISL_13755996, EPI_ISL_13757795, EPI_ISL_13757902, EPI_ISL_13757914, EPI_ISL_13759674, EPI_ISL_13759811, EPI_ISL_13760929, EPI_ISL_13762803, EPI_ISL_13764852, EPI_ISL_13765234, EPI_ISL_13769441, EPI_ISL_13774980, EPI_ISL_13776103, EPI_ISL_13776118, EPI_ISL_13780813, EPI_ISL_13788917, EPI_ISL_13794761, EPI_ISL_13795106, EPI_ISL_13795308, EPI_ISL_13802466, EPI_ISL_13805350, EPI_ISL_13806026, EPI_ISL_13806197, EPI_ISL_13810913, EPI_ISL_13812067, EPI_ISL_13824458, EPI_ISL_13826362, EPI_ISL_13830194, EPI_ISL_13830195, EPI_ISL_13830196, EPI_ISL_13830197, EPI_ISL_13830445, EPI_ISL_13830454, EPI_ISL_13839105, EPI_ISL_13839285, EPI_ISL_13842068, EPI_ISL_13844161, EPI_ISL_13850726, EPI_ISL_13855446, EPI_ISL_13856866, EPI_ISL_13858143, EPI_ISL_13858664, EPI_ISL_13860426, EPI_ISL_13860879, EPI_ISL_13866687, EPI_ISL_13866688, EPI_ISL_13866691, EPI_ISL_13867130, EPI_ISL_13867135, EPI_ISL_13871326, EPI_ISL_13873100, EPI_ISL_13875348, EPI_ISL_13875677, EPI_ISL_13876290, EPI_ISL_13876318, EPI_ISL_13876612, EPI_ISL_13876760, EPI_ISL_13881123, EPI_ISL_13884353, EPI_ISL_13884360, EPI_ISL_13884439, EPI_ISL_13889482, EPI_ISL_13891697, EPI_ISL_13896135, EPI_ISL_13896156, EPI_ISL_13896578, EPI_ISL_13900930, EPI_ISL_13907795, EPI_ISL_13907925, EPI_ISL_13908668, EPI_ISL_13913047, EPI_ISL_13915527, EPI_ISL_13915530, EPI_ISL_13931117, EPI_ISL_13931975, EPI_ISL_13931997, EPI_ISL_13932039, EPI_ISL_13932080, EPI_ISL_13937613, EPI_ISL_13939223, EPI_ISL_13947566, EPI_ISL_13948007, EPI_ISL_13951654, EPI_ISL_13957684, EPI_ISL_13958481, EPI_ISL_13958566, EPI_ISL_13963277, EPI_ISL_13963776, EPI_ISL_13963832, EPI_ISL_13967204, EPI_ISL_13968198, EPI_ISL_13970237, EPI_ISL_13970242, EPI_ISL_13970249, EPI_ISL_13970257, EPI_ISL_13970276, EPI_ISL_13975822, EPI_ISL_13981101, EPI_ISL_13984460, EPI_ISL_13986494, EPI_ISL_13986497, EPI_ISL_13986498, EPI_ISL_13986501, EPI_ISL_13989484, EPI_ISL_13991375, EPI_ISL_13994781, EPI_ISL_14000155, EPI_ISL_14005794, EPI_ISL_14011475, EPI_ISL_14015047, EPI_ISL_14019093, EPI_ISL_14019109, EPI_ISL_14019330, EPI_ISL_14020697, EPI_ISL_14022780, EPI_ISL_14022892, EPI_ISL_14023662, EPI_ISL_14027304, EPI_ISL_14027788, EPI_ISL_14028215, EPI_ISL_14029314, EPI_ISL_14029803, EPI_ISL_14030175, EPI_ISL_14032717, EPI_ISL_14035970, EPI_ISL_14036069, EPI_ISL_14044698, EPI_ISL_14044704, EPI_ISL_14046291, EPI_ISL_14047361, EPI_ISL_14051041, EPI_ISL_14051049, EPI_ISL_14051052, EPI_ISL_14064598, EPI_ISL_14064601, EPI_ISL_14066591, EPI_ISL_14066852, EPI_ISL_14071587, EPI_ISL_14071795, EPI_ISL_14097542, EPI_ISL_14124074, EPI_ISL_14127563, EPI_ISL_14134678, EPI_ISL_14147202, EPI_ISL_14155218, EPI_ISL_14158264, EPI_ISL_14161024, EPI_ISL_14170603, EPI_ISL_14172905, EPI_ISL_14173767, EPI_ISL_14175092, EPI_ISL_14175097, EPI_ISL_14175182, EPI_ISL_14180178, EPI_ISL_14181740, EPI_ISL_14193000, EPI_ISL_14193833, EPI_ISL_14196068, EPI_ISL_14196701, EPI_ISL_14197724, EPI_ISL_14198080, EPI_ISL_14200342, EPI_ISL_14200801, EPI_ISL_14203206, EPI_ISL_14203613, EPI_ISL_14208835, EPI_ISL_14209372, EPI_ISL_14209934, EPI_ISL_14211976, EPI_ISL_14215014, EPI_ISL_14215818, EPI_ISL_14216595, EPI_ISL_14217745, EPI_ISL_14222817, EPI_ISL_14223595, EPI_ISL_14223736, EPI_ISL_14223750, EPI_ISL_14224871, EPI_ISL_14226883, EPI_ISL_14228030, EPI_ISL_14229584, EPI_ISL_14230429, EPI_ISL_14230544, EPI_ISL_14231739, EPI_ISL_14231749, EPI_ISL_14231751, EPI_ISL_14232221, EPI_ISL_14236377, EPI_ISL_14241722, EPI_ISL_14243471, EPI_ISL_14243503, EPI_ISL_14249979, EPI_ISL_14257905, EPI_ISL_14259114, EPI_ISL_14259141, EPI_ISL_14259905, EPI_ISL_14260215, EPI_ISL_14261704, EPI_ISL_14263077, EPI_ISL_14277057, EPI_ISL_14285562, EPI_ISL_14287370, EPI_ISL_14289901, EPI_ISL_14292615, EPI_ISL_14292645, EPI_ISL_14292727, EPI_ISL_14292796, EPI_ISL_14296586, EPI_ISL_14298637, EPI_ISL_14299991, EPI_ISL_14301376, EPI_ISL_14311909, EPI_ISL_14311965, EPI_ISL_14312743, EPI_ISL_14321789, EPI_ISL_14321842, EPI_ISL_14329477, EPI_ISL_14353536, EPI_ISL_14356738, EPI_ISL_14359010, EPI_ISL_14377405, EPI_ISL_14377421, EPI_ISL_14382623, EPI_ISL_14386850, EPI_ISL_14387989, EPI_ISL_14389796, EPI_ISL_14391372, EPI_ISL_14393120, EPI_ISL_14416474, EPI_ISL_14417374, EPI_ISL_14425116, EPI_ISL_14425894, EPI_ISL_14426235, EPI_ISL_14426336, EPI_ISL_14430592, EPI_ISL_14433737, EPI_ISL_14434989, EPI_ISL_14436225, EPI_ISL_14437098, EPI_ISL_14439513, EPI_ISL_14439514, EPI_ISL_14439530, EPI_ISL_14439649, EPI_ISL_14439686, EPI_ISL_14448667, EPI_ISL_14455168, EPI_ISL_14459779, EPI_ISL_14462783, EPI_ISL_14464386, EPI_ISL_14467169, EPI_ISL_14469810, EPI_ISL_14471721, EPI_ISL_14478208, EPI_ISL_14479146, EPI_ISL_14479735, EPI_ISL_14483275, EPI_ISL_14485183, EPI_ISL_14485890, EPI_ISL_14487304, EPI_ISL_14487315, EPI_ISL_14493139, EPI_ISL_14493608, EPI_ISL_14493822, EPI_ISL_14493989, EPI_ISL_14496407, EPI_ISL_14496491, EPI_ISL_14497316, EPI_ISL_14498244, EPI_ISL_14502420, EPI_ISL_14503169, EPI_ISL_14503437, EPI_ISL_14504973, EPI_ISL_14505974, EPI_ISL_14507199, EPI_ISL_14507200, EPI_ISL_14508231, EPI_ISL_14509715, EPI_ISL_14513137, EPI_ISL_14518038, EPI_ISL_14518039, EPI_ISL_14518040, EPI_ISL_14518101, EPI_ISL_14518137, EPI_ISL_14527351, EPI_ISL_14535112, EPI_ISL_14540192, EPI_ISL_14544667, EPI_ISL_14545270, EPI_ISL_14551066, EPI_ISL_14556650, EPI_ISL_14560721, EPI_ISL_14561487, EPI_ISL_14562820, EPI_ISL_14571645, EPI_ISL_14572777, EPI_ISL_14573093, EPI_ISL_14577981, EPI_ISL_14578599, EPI_ISL_14583728, EPI_ISL_14596883, EPI_ISL_14599772, EPI_ISL_14602583, EPI_ISL_14602992, EPI_ISL_14606016, EPI_ISL_14610722, EPI_ISL_14610723, EPI_ISL_14610724, EPI_ISL_14610725, EPI_ISL_14610726, EPI_ISL_14610727, EPI_ISL_14610728, EPI_ISL_14610729, EPI_ISL_14610730, EPI_ISL_14610731, EPI_ISL_14610732, EPI_ISL_14610733, EPI_ISL_14610734, EPI_ISL_14613632, EPI_ISL_14613671, EPI_ISL_14616144, EPI_ISL_14616681, EPI_ISL_14618772, EPI_ISL_14619952, EPI_ISL_14623599, EPI_ISL_14624407, EPI_ISL_14625263, EPI_ISL_14647032, EPI_ISL_14650043, EPI_ISL_14652006, EPI_ISL_14665394, EPI_ISL_14666760, EPI_ISL_14667656, EPI_ISL_14667834, EPI_ISL_14669160, EPI_ISL_14676287, EPI_ISL_14681429, EPI_ISL_14683500, EPI_ISL_14687471, EPI_ISL_14691921, EPI_ISL_14694460, EPI_ISL_14699501, EPI_ISL_14700183, EPI_ISL_14700285, EPI_ISL_14701161, EPI_ISL_14706169, EPI_ISL_14707196, EPI_ISL_14707197, EPI_ISL_14710821, EPI_ISL_14710834, EPI_ISL_14711613, EPI_ISL_14711614, EPI_ISL_14715522, EPI_ISL_14721837, EPI_ISL_14721894, EPI_ISL_14722630, EPI_ISL_14725600, EPI_ISL_14727457, EPI_ISL_14728608, EPI_ISL_14728814, EPI_ISL_14732990, EPI_ISL_14744620, EPI_ISL_14744804, EPI_ISL_14744809, EPI_ISL_14745146, EPI_ISL_14746124, EPI_ISL_14746196, EPI_ISL_14746271, EPI_ISL_14747246, EPI_ISL_14747247, EPI_ISL_14747621, EPI_ISL_14752384, EPI_ISL_14754570, EPI_ISL_14755727, EPI_ISL_14755766, EPI_ISL_14755933, EPI_ISL_14763711, EPI_ISL_14763898, EPI_ISL_14766331, EPI_ISL_14766361, EPI_ISL_14766363, EPI_ISL_14770484, EPI_ISL_14771903, EPI_ISL_14772260, EPI_ISL_14773203, EPI_ISL_14773569, EPI_ISL_14773570, EPI_ISL_14778614, EPI_ISL_14785887, EPI_ISL_14786677, EPI_ISL_14788048, EPI_ISL_14788099, EPI_ISL_14788860, EPI_ISL_14789391, EPI_ISL_14789392, EPI_ISL_14789508, EPI_ISL_14791420, EPI_ISL_14793146, EPI_ISL_14793618, EPI_ISL_14806018, EPI_ISL_14806413, EPI_ISL_14809350, EPI_ISL_14811078, EPI_ISL_14812412, EPI_ISL_14813068, EPI_ISL_14813161, EPI_ISL_14813215, EPI_ISL_14813300, EPI_ISL_14813995, EPI_ISL_14816346, EPI_ISL_14817985, EPI_ISL_14832977, EPI_ISL_14834854, EPI_ISL_14837867, EPI_ISL_14838049, EPI_ISL_14841625, EPI_ISL_14845057, EPI_ISL_14847727, EPI_ISL_14856139, EPI_ISL_14859457, EPI_ISL_14859716, EPI_ISL_14862263, EPI_ISL_14886333, EPI_ISL_14888736, EPI_ISL_14890020, EPI_ISL_14891391, EPI_ISL_14891763, EPI_ISL_14891765, EPI_ISL_14892114, EPI_ISL_14892395, EPI_ISL_14892970, EPI_ISL_14901195, EPI_ISL_14901198, EPI_ISL_14901422, EPI_ISL_14901439, EPI_ISL_14901444, EPI_ISL_14901672, EPI_ISL_14903212, EPI_ISL_14904331, EPI_ISL_14912863, EPI_ISL_14913043, EPI_ISL_14916417, EPI_ISL_14917292, EPI_ISL_14919989, EPI_ISL_14920419, EPI_ISL_14921805, EPI_ISL_14922117, EPI_ISL_14922327, EPI_ISL_14922954, EPI_ISL_14924448, EPI_ISL_14925471, EPI_ISL_14925480, EPI_ISL_14925487, EPI_ISL_14929774, EPI_ISL_14931103, EPI_ISL_14934229, EPI_ISL_14934234, EPI_ISL_14935361, EPI_ISL_14935895, EPI_ISL_14935908, EPI_ISL_14935930, EPI_ISL_14935931, EPI_ISL_14937654, EPI_ISL_14937864, EPI_ISL_14942184, EPI_ISL_14942530, EPI_ISL_14943290, EPI_ISL_14945818, EPI_ISL_14946958, EPI_ISL_14949065, EPI_ISL_14950282, EPI_ISL_14950545, EPI_ISL_14951595, EPI_ISL_14951609, EPI_ISL_14951892, EPI_ISL_14952059, EPI_ISL_14952220, EPI_ISL_14953487, EPI_ISL_14960752, EPI_ISL_14960911, EPI_ISL_14961972, EPI_ISL_14962212, EPI_ISL_14962429, EPI_ISL_14962617, EPI_ISL_14975894, EPI_ISL_14980656, EPI_ISL_14992324, EPI_ISL_14993023, EPI_ISL_14995958, EPI_ISL_14997830, EPI_ISL_15005362, EPI_ISL_15005560, EPI_ISL_15010697, EPI_ISL_15012784, EPI_ISL_15013151, EPI_ISL_15013344, EPI_ISL_15014035, EPI_ISL_15014516, EPI_ISL_15015307, EPI_ISL_15017244, EPI_ISL_15018502, EPI_ISL_15020427, EPI_ISL_15022783, EPI_ISL_15022788, EPI_ISL_15024848, EPI_ISL_15026124, EPI_ISL_15030291, EPI_ISL_15030370, EPI_ISL_15032101, EPI_ISL_15036387, EPI_ISL_15038137, EPI_ISL_15040463, EPI_ISL_15040845, EPI_ISL_15040855, EPI_ISL_15040867, EPI_ISL_15044102, EPI_ISL_15048524, EPI_ISL_15050379, EPI_ISL_15051633, EPI_ISL_15058728, EPI_ISL_15058903, EPI_ISL_15071928, EPI_ISL_15072261, EPI_ISL_15072543, EPI_ISL_15072550, EPI_ISL_15072553, EPI_ISL_15072554, EPI_ISL_15072999, EPI_ISL_15075043, EPI_ISL_15075836, EPI_ISL_15076071, EPI_ISL_15077422, EPI_ISL_15078481, EPI_ISL_15080832, EPI_ISL_15084091, EPI_ISL_15085357, EPI_ISL_15085883, EPI_ISL_15085910, EPI_ISL_15086100, EPI_ISL_15086132, EPI_ISL_15086246, EPI_ISL_15088435, EPI_ISL_15088854, EPI_ISL_15090098, EPI_ISL_15093244, EPI_ISL_15093817, EPI_ISL_15093818, EPI_ISL_15094085, EPI_ISL_15096672, EPI_ISL_15098367, EPI_ISL_15101602, EPI_ISL_15107059, EPI_ISL_15107248, EPI_ISL_15107529, EPI_ISL_15108940, EPI_ISL_15108982, EPI_ISL_15109913, EPI_ISL_15111016, EPI_ISL_15111369, EPI_ISL_15114528, EPI_ISL_15114696, EPI_ISL_15116712, EPI_ISL_15118484, EPI_ISL_15119416, EPI_ISL_15120575, EPI_ISL_15125352, EPI_ISL_15126616, EPI_ISL_15129252, EPI_ISL_15137908, EPI_ISL_15137948, EPI_ISL_15140027, EPI_ISL_15140068, EPI_ISL_15145892, EPI_ISL_15145981, EPI_ISL_15157666, EPI_ISL_15159584, EPI_ISL_15160596, EPI_ISL_15161674, EPI_ISL_15169791, EPI_ISL_15170512, EPI_ISL_15172949, EPI_ISL_15173621, EPI_ISL_15175083, EPI_ISL_15175896, EPI_ISL_15177304, EPI_ISL_15177330, EPI_ISL_15177334, EPI_ISL_15178067, EPI_ISL_15178215, EPI_ISL_15184076, EPI_ISL_15184330, EPI_ISL_15191490, EPI_ISL_15191491, EPI_ISL_15191642, EPI_ISL_15191714, EPI_ISL_15191804, EPI_ISL_15193406, EPI_ISL_15195634, EPI_ISL_15195645, EPI_ISL_15198987, EPI_ISL_15211295, EPI_ISL_15211305, EPI_ISL_15213088, EPI_ISL_15215446, EPI_ISL_15216639, EPI_ISL_15218165, EPI_ISL_15222709, EPI_ISL_15229199, EPI_ISL_15231108, EPI_ISL_15236061, EPI_ISL_15236355, EPI_ISL_15241555, EPI_ISL_15248681, EPI_ISL_15250551, EPI_ISL_15251240, EPI_ISL_15251241, EPI_ISL_15251242, EPI_ISL_15251243, EPI_ISL_15257404, EPI_ISL_15264055, EPI_ISL_15268715, EPI_ISL_15268834, EPI_ISL_15271184, EPI_ISL_15273578, EPI_ISL_15275240, EPI_ISL_15278730, EPI_ISL_15278787, EPI_ISL_15279743, EPI_ISL_15284364, EPI_ISL_15284373, EPI_ISL_15284586, EPI_ISL_15286527, EPI_ISL_15287393, EPI_ISL_15288333, EPI_ISL_15292331, EPI_ISL_15294656, EPI_ISL_15296403, EPI_ISL_15305862, EPI_ISL_15306067, EPI_ISL_15307010, EPI_ISL_15307651, EPI_ISL_15310561, EPI_ISL_15312119, EPI_ISL_15314949, EPI_ISL_15316398, EPI_ISL_15317884, EPI_ISL_15325687, EPI_ISL_15325946, EPI_ISL_15328668, EPI_ISL_15330077, EPI_ISL_15330418, EPI_ISL_15331994, EPI_ISL_15332094, EPI_ISL_15332912, EPI_ISL_15333310, EPI_ISL_15338015, EPI_ISL_15338081, EPI_ISL_15340355, EPI_ISL_15341321, EPI_ISL_15347054, EPI_ISL_15348926, EPI_ISL_15348927, EPI_ISL_15348928, EPI_ISL_15348929, EPI_ISL_15352149, EPI_ISL_15354679, EPI_ISL_15354775, EPI_ISL_15357057, EPI_ISL_15357237, EPI_ISL_15357960, EPI_ISL_15357988, EPI_ISL_15362650, EPI_ISL_15363544, EPI_ISL_15368893, EPI_ISL_15370137, EPI_ISL_15370885, EPI_ISL_15370889, EPI_ISL_15376124, EPI_ISL_15376348, EPI_ISL_15376376, EPI_ISL_15379719, EPI_ISL_15380518, EPI_ISL_15384507, EPI_ISL_15385232, EPI_ISL_15385971, EPI_ISL_15387248, EPI_ISL_15387296, EPI_ISL_15387686, EPI_ISL_15388463, EPI_ISL_15389278, EPI_ISL_15392529, EPI_ISL_15393294, EPI_ISL_15393302, EPI_ISL_15397576, EPI_ISL_15403865, EPI_ISL_15403898, EPI_ISL_15408226, EPI_ISL_15408697, EPI_ISL_15409673, EPI_ISL_15415649, EPI_ISL_15416349, EPI_ISL_15420040, EPI_ISL_15420212, EPI_ISL_15420431, EPI_ISL_15420631, EPI_ISL_15423234, EPI_ISL_15424211, EPI_ISL_15424884, EPI_ISL_15434634, EPI_ISL_15435185, EPI_ISL_15436140, EPI_ISL_15436494, EPI_ISL_15436498, EPI_ISL_15436499, EPI_ISL_15442625, EPI_ISL_15442735, EPI_ISL_15446553, EPI_ISL_15456143, EPI_ISL_15462878, EPI_ISL_15471419, EPI_ISL_15471420, EPI_ISL_15472394, EPI_ISL_15472759, EPI_ISL_15476105, EPI_ISL_15476158, EPI_ISL_15476180, EPI_ISL_15476724, EPI_ISL_15479591, EPI_ISL_15481002, EPI_ISL_15486348, EPI_ISL_15490572, EPI_ISL_15492743, EPI_ISL_15492887, EPI_ISL_15494260, EPI_ISL_15494897, EPI_ISL_15495028, EPI_ISL_15496641, EPI_ISL_15505215, EPI_ISL_15505985, EPI_ISL_15506333, EPI_ISL_15507204, EPI_ISL_15507296, EPI_ISL_15507616, EPI_ISL_15508143, EPI_ISL_15509746, EPI_ISL_15509755, EPI_ISL_15511119, EPI_ISL_15511841, EPI_ISL_15511842, EPI_ISL_15511843, EPI_ISL_15513583, EPI_ISL_15513588, EPI_ISL_15513663, EPI_ISL_15514216, EPI_ISL_15514265, EPI_ISL_15514302, EPI_ISL_15514923, EPI_ISL_15523458, EPI_ISL_15523591, EPI_ISL_15528152, EPI_ISL_15528174, EPI_ISL_15528328, EPI_ISL_15528329, EPI_ISL_15528330, EPI_ISL_15528331, EPI_ISL_15528332, EPI_ISL_15528333, EPI_ISL_15528334, EPI_ISL_15535800, EPI_ISL_15537619, EPI_ISL_15538513, EPI_ISL_15538645, EPI_ISL_15542503, EPI_ISL_15546790, EPI_ISL_15549778, EPI_ISL_15549981, EPI_ISL_15550525, EPI_ISL_15579728, EPI_ISL_15579786, EPI_ISL_15580359, EPI_ISL_15580699, EPI_ISL_15581446, EPI_ISL_15581681, EPI_ISL_15581931, EPI_ISL_15581932, EPI_ISL_15581939, EPI_ISL_15582076, EPI_ISL_15582517, EPI_ISL_15583376, EPI_ISL_15585338, EPI_ISL_15587950, EPI_ISL_15588132, EPI_ISL_15594682, EPI_ISL_15595518, EPI_ISL_15598104, EPI_ISL_15598966, EPI_ISL_15602198, EPI_ISL_15604595, EPI_ISL_15606951, EPI_ISL_15607872, EPI_ISL_15608835, EPI_ISL_15609106, EPI_ISL_15609107, EPI_ISL_15610881, EPI_ISL_15612047, EPI_ISL_15612048, EPI_ISL_15614383, EPI_ISL_15614490, EPI_ISL_15616889, EPI_ISL_15617621, EPI_ISL_15617635, EPI_ISL_15619675, EPI_ISL_15626705, EPI_ISL_15628252, EPI_ISL_15630041, EPI_ISL_15631537, EPI_ISL_15635022, EPI_ISL_15637121, EPI_ISL_15639067, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15649157, EPI_ISL_15650076, EPI_ISL_15650225, EPI_ISL_15653695, EPI_ISL_15654640, EPI_ISL_15656922, EPI_ISL_15659847, EPI_ISL_15661609, EPI_ISL_15666595, EPI_ISL_15667047, EPI_ISL_15669004, EPI_ISL_15671244, EPI_ISL_15671388, EPI_ISL_15671577, EPI_ISL_15671878, EPI_ISL_15671888, EPI_ISL_15673934, EPI_ISL_15675248, EPI_ISL_15678339, EPI_ISL_15685722, EPI_ISL_15685724, EPI_ISL_15685810, EPI_ISL_15687681, EPI_ISL_15687965, EPI_ISL_15688500, EPI_ISL_15692625, EPI_ISL_15693169, EPI_ISL_15693174, EPI_ISL_15693676, EPI_ISL_15700160, EPI_ISL_15703834, EPI_ISL_15705061, EPI_ISL_15712450, EPI_ISL_15715244, EPI_ISL_15719141, EPI_ISL_15719142, EPI_ISL_15719143, EPI_ISL_15720605, EPI_ISL_15721137, EPI_ISL_15721185, EPI_ISL_15721190, EPI_ISL_15723589, EPI_ISL_15725799, EPI_ISL_15728467, EPI_ISL_15728673, EPI_ISL_15729287, EPI_ISL_15729288, EPI_ISL_15729308, EPI_ISL_15729309, EPI_ISL_15729310, EPI_ISL_15729311, EPI_ISL_15729315, EPI_ISL_15729341, EPI_ISL_15729358, EPI_ISL_15731233, EPI_ISL_15731409, EPI_ISL_15732413, EPI_ISL_15736424, EPI_ISL_15739498, EPI_ISL_15739617, EPI_ISL_15741410, EPI_ISL_15743318, EPI_ISL_15743816, EPI_ISL_15749060, EPI_ISL_15754145, EPI_ISL_15754794, EPI_ISL_15758565, EPI_ISL_15758745, EPI_ISL_15760224, EPI_ISL_15760382, EPI_ISL_15760554, EPI_ISL_15760812, EPI_ISL_15761520, EPI_ISL_15761543, EPI_ISL_15761663, EPI_ISL_15763216, EPI_ISL_15765022, EPI_ISL_15768827, EPI_ISL_15776989, EPI_ISL_15778527, EPI_ISL_15779724, EPI_ISL_15780387, EPI_ISL_15781197, EPI_ISL_15781220, EPI_ISL_15781937, EPI_ISL_15782049, EPI_ISL_15785782, EPI_ISL_15786114, EPI_ISL_15786255, EPI_ISL_15790657, EPI_ISL_15791223, EPI_ISL_15791252, EPI_ISL_15792351, EPI_ISL_15793981, EPI_ISL_15797751, EPI_ISL_15798331, EPI_ISL_15801425, EPI_ISL_15801499, EPI_ISL_15801515, EPI_ISL_15803807, EPI_ISL_15815337, EPI_ISL_15815525, EPI_ISL_15815889, EPI_ISL_15818486, EPI_ISL_15820055, EPI_ISL_15822919, EPI_ISL_15824080, EPI_ISL_15824099, EPI_ISL_15824207, EPI_ISL_15825627, EPI_ISL_15826800, EPI_ISL_15826867, EPI_ISL_15829108, EPI_ISL_15837751, EPI_ISL_15837827, EPI_ISL_15838124, EPI_ISL_15839941, EPI_ISL_15843473, EPI_ISL_15843671, EPI_ISL_15844032, EPI_ISL_15844165, EPI_ISL_15845753, EPI_ISL_15845778, EPI_ISL_15845946, EPI_ISL_15846023, EPI_ISL_15846264, EPI_ISL_15849690, EPI_ISL_15850759, EPI_ISL_15850865, EPI_ISL_15850872, EPI_ISL_15853809, EPI_ISL_15853943, EPI_ISL_15856103, EPI_ISL_15856463, EPI_ISL_15856822, EPI_ISL_15857468, EPI_ISL_15860163, EPI_ISL_15864217, EPI_ISL_15864218, EPI_ISL_15865257, EPI_ISL_15865301, EPI_ISL_15865421, EPI_ISL_15865482, EPI_ISL_15866887, EPI_ISL_15873398, EPI_ISL_15874567, EPI_ISL_15878818, EPI_ISL_15883009, EPI_ISL_15883480, EPI_ISL_15887656, EPI_ISL_15894376, EPI_ISL_15895625, EPI_ISL_15896804, EPI_ISL_15896845, EPI_ISL_15896923, EPI_ISL_15897067, EPI_ISL_15897092, EPI_ISL_15898992, EPI_ISL_15900796, EPI_ISL_15905452, EPI_ISL_15905511, EPI_ISL_15911160, EPI_ISL_15912221, EPI_ISL_15912222, EPI_ISL_15912223, EPI_ISL_15912224, EPI_ISL_15914119, EPI_ISL_15917272, EPI_ISL_15920181, EPI_ISL_15920505, EPI_ISL_15920753, EPI_ISL_15920754, EPI_ISL_15920755, EPI_ISL_15926083, EPI_ISL_15926723, EPI_ISL_15928156, EPI_ISL_15928909, EPI_ISL_15929151, EPI_ISL_15932554, EPI_ISL_15934274, EPI_ISL_15937718, EPI_ISL_15938074, EPI_ISL_15941879, EPI_ISL_15941880, EPI_ISL_15945504, EPI_ISL_15955458, EPI_ISL_15958934, EPI_ISL_15961456, EPI_ISL_15962045, EPI_ISL_15966527, EPI_ISL_15969420, EPI_ISL_15969421, EPI_ISL_15969437, EPI_ISL_15969438, EPI_ISL_15970088, EPI_ISL_15970187, EPI_ISL_15982641, EPI_ISL_15984958, EPI_ISL_15992803, EPI_ISL_15998627, EPI_ISL_16001974, EPI_ISL_16001995, EPI_ISL_16005457, EPI_ISL_16006665, EPI_ISL_16007931, EPI_ISL_16008877, EPI_ISL_16012424, EPI_ISL_16013074, EPI_ISL_16013086, EPI_ISL_16015099, EPI_ISL_16017107, EPI_ISL_16018930, EPI_ISL_16019056, EPI_ISL_16024407, EPI_ISL_16024682, EPI_ISL_16027431, EPI_ISL_16027937, EPI_ISL_16029135, EPI_ISL_16029382, EPI_ISL_16029654, EPI_ISL_16030181, EPI_ISL_16033087, EPI_ISL_16039444, EPI_ISL_16043974, EPI_ISL_16045410, EPI_ISL_16046711, EPI_ISL_16050127, EPI_ISL_16054133, EPI_ISL_16054451, EPI_ISL_16054953, EPI_ISL_16054963, EPI_ISL_16055434, EPI_ISL_16055461, EPI_ISL_16055527, EPI_ISL_16055697, EPI_ISL_16055721, EPI_ISL_16056123, EPI_ISL_16057031, EPI_ISL_16060790, EPI_ISL_16062229, EPI_ISL_16066333, EPI_ISL_16068281, EPI_ISL_16068914, EPI_ISL_16073469, EPI_ISL_16073474, EPI_ISL_16075086, EPI_ISL_16075127, EPI_ISL_16079016, EPI_ISL_16080170, EPI_ISL_16080871, EPI_ISL_16091870, EPI_ISL_16102480, EPI_ISL_16111875, EPI_ISL_16113331, EPI_ISL_16113603, EPI_ISL_16113806, EPI_ISL_16114631, EPI_ISL_16115703, EPI_ISL_16116190, EPI_ISL_16116234, EPI_ISL_16116659, EPI_ISL_16116707, EPI_ISL_16119498, EPI_ISL_16119508, EPI_ISL_16119512, EPI_ISL_16119517, EPI_ISL_16119519, EPI_ISL_16119805, EPI_ISL_16131965, EPI_ISL_16131986, EPI_ISL_16131997, EPI_ISL_16136901, EPI_ISL_16137616, EPI_ISL_16151030, EPI_ISL_16151463, EPI_ISL_16151651, EPI_ISL_16153650, EPI_ISL_16153658, EPI_ISL_16153800, EPI_ISL_16154520, EPI_ISL_16157031, EPI_ISL_16157875, EPI_ISL_16158109, EPI_ISL_16158326, EPI_ISL_16158363, EPI_ISL_16160252, EPI_ISL_16160296, EPI_ISL_16160313, EPI_ISL_16165250, EPI_ISL_16167761, EPI_ISL_16178634, EPI_ISL_16179355, EPI_ISL_16180574, EPI_ISL_16181797, EPI_ISL_16181828, EPI_ISL_16181950, EPI_ISL_16183022, EPI_ISL_16190977, EPI_ISL_16191476, EPI_ISL_16196167, EPI_ISL_16197958, EPI_ISL_16201173, EPI_ISL_16215808, EPI_ISL_16218191, EPI_ISL_16219709, EPI_ISL_16219753, EPI_ISL_16221691, EPI_ISL_16230801, EPI_ISL_16233000, EPI_ISL_16233650, EPI_ISL_16233865, EPI_ISL_16234790, EPI_ISL_16235313, EPI_ISL_16235462, EPI_ISL_16235523, EPI_ISL_16235930, EPI_ISL_16244367, EPI_ISL_16244373, EPI_ISL_16244408, EPI_ISL_16244923, EPI_ISL_16245232, EPI_ISL_16245289, EPI_ISL_16245433, EPI_ISL_16245601, EPI_ISL_16245627, EPI_ISL_16247208, EPI_ISL_16247263, EPI_ISL_16247490, EPI_ISL_16247545, EPI_ISL_16257294, EPI_ISL_16264400, EPI_ISL_16265325, EPI_ISL_16268074, EPI_ISL_16270258, EPI_ISL_16271444, EPI_ISL_16271604, EPI_ISL_16273936, EPI_ISL_16284103, EPI_ISL_16284311, EPI_ISL_16287253, EPI_ISL_16287690, EPI_ISL_16290877, EPI_ISL_16293662, EPI_ISL_16312661, EPI_ISL_16327295, EPI_ISL_16334679, EPI_ISL_16338847, EPI_ISL_16338862, EPI_ISL_16343221, EPI_ISL_16348840, EPI_ISL_16348868, EPI_ISL_16351967, EPI_ISL_16354229, EPI_ISL_16355537, EPI_ISL_16356453, EPI_ISL_16356910, EPI_ISL_16358915, EPI_ISL_16360495, EPI_ISL_16365715, EPI_ISL_16368903, EPI_ISL_16369869, EPI_ISL_16370037, EPI_ISL_16378181, EPI_ISL_16379359, EPI_ISL_16380313, EPI_ISL_16381332, EPI_ISL_16381679, EPI_ISL_16384522, EPI_ISL_16385455, EPI_ISL_16385456, EPI_ISL_16391752, EPI_ISL_16394844, EPI_ISL_16394922, EPI_ISL_16395667, EPI_ISL_16398472, EPI_ISL_16399824, EPI_ISL_16400033, EPI_ISL_16414127, EPI_ISL_16422834, EPI_ISL_16424130, EPI_ISL_16425691, EPI_ISL_16428100, EPI_ISL_16428101, EPI_ISL_16428102, EPI_ISL_16429066, EPI_ISL_16436533, EPI_ISL_16439413, EPI_ISL_16443129, EPI_ISL_16443688, EPI_ISL_16443874, EPI_ISL_16444600, EPI_ISL_16452054, EPI_ISL_16454044, EPI_ISL_16460823, EPI_ISL_16461302, EPI_ISL_16464657, EPI_ISL_16467436, EPI_ISL_16470710, EPI_ISL_16471153, EPI_ISL_16473435, EPI_ISL_16473762, EPI_ISL_16474400, EPI_ISL_16479826, EPI_ISL_16482060, EPI_ISL_16489594, EPI_ISL_16491494, EPI_ISL_16492150, EPI_ISL_16492585, EPI_ISL_16492756, EPI_ISL_16493785, EPI_ISL_16497702, EPI_ISL_16498515, EPI_ISL_16503949, EPI_ISL_16507701, EPI_ISL_16507927, EPI_ISL_16520597, EPI_ISL_16520598, EPI_ISL_16520637, EPI_ISL_16520640, EPI_ISL_16520641, EPI_ISL_16520711, EPI_ISL_16520763, EPI_ISL_16520788, EPI_ISL_16521101, EPI_ISL_16524906, EPI_ISL_16528641, EPI_ISL_16528645, EPI_ISL_16528903, EPI_ISL_16535376, EPI_ISL_16536212, EPI_ISL_16539692, EPI_ISL_16541774, EPI_ISL_16542553, EPI_ISL_16544506, EPI_ISL_16567779, EPI_ISL_16574574, EPI_ISL_16581578, EPI_ISL_16584104, EPI_ISL_16586683, EPI_ISL_16586702, EPI_ISL_16587574, EPI_ISL_16597363, EPI_ISL_16607452, EPI_ISL_16607500, EPI_ISL_16607535, EPI_ISL_16611498, EPI_ISL_16611571, EPI_ISL_16613287, EPI_ISL_16613482, EPI_ISL_16615597, EPI_ISL_16615617, EPI_ISL_16615668, EPI_ISL_16616642, EPI_ISL_16625690, EPI_ISL_16626611, EPI_ISL_16626666, EPI_ISL_16627067, EPI_ISL_16628854, EPI_ISL_16630260, EPI_ISL_16630261, EPI_ISL_16637607, EPI_ISL_16637631, EPI_ISL_16638190, EPI_ISL_16638453, EPI_ISL_16643406, EPI_ISL_16649988, EPI_ISL_16653618, EPI_ISL_16669313, EPI_ISL_16669829, EPI_ISL_16672301, EPI_ISL_16672327, EPI_ISL_16672352, EPI_ISL_16676267, EPI_ISL_16677015, EPI_ISL_16678917, EPI_ISL_16678946, EPI_ISL_16679654, EPI_ISL_16681917, EPI_ISL_16682342, EPI_ISL_16688219, EPI_ISL_16688525, EPI_ISL_16688688, EPI_ISL_16688713, EPI_ISL_16691397, EPI_ISL_16691487, EPI_ISL_16694176, EPI_ISL_16695435, EPI_ISL_16697861, EPI_ISL_16700160, EPI_ISL_16702838, EPI_ISL_16705882, EPI_ISL_16706498, EPI_ISL_16708798, EPI_ISL_16711038, EPI_ISL_16711095, EPI_ISL_16711417, EPI_ISL_16711531, EPI_ISL_16716967, EPI_ISL_16721930, EPI_ISL_16722183, EPI_ISL_16722215, EPI_ISL_16722270, EPI_ISL_16722970, EPI_ISL_16723215, EPI_ISL_16725887, EPI_ISL_16727241, EPI_ISL_16728257, EPI_ISL_16728383, EPI_ISL_16728411, EPI_ISL_16731753, EPI_ISL_16736400, EPI_ISL_16739452, EPI_ISL_16740104, EPI_ISL_16740406, EPI_ISL_16741567, EPI_ISL_16741573, EPI_ISL_16749999, EPI_ISL_16750878, EPI_ISL_16751721, EPI_ISL_16751722, EPI_ISL_16751789, EPI_ISL_16751791, EPI_ISL_16751977, EPI_ISL_16752073, EPI_ISL_16752138, EPI_ISL_16757168, EPI_ISL_16757210, EPI_ISL_16758981, EPI_ISL_16764861, EPI_ISL_16765888, EPI_ISL_16766196, EPI_ISL_16811091, EPI_ISL_16812565, EPI_ISL_16815494, EPI_ISL_16816293, EPI_ISL_16818458, EPI_ISL_16818471, EPI_ISL_16825222, EPI_ISL_16828876, EPI_ISL_16828896, EPI_ISL_16829188, EPI_ISL_16831507, EPI_ISL_16833893, EPI_ISL_16834974, EPI_ISL_16835399, EPI_ISL_16842787, EPI_ISL_16842790, EPI_ISL_16847425, EPI_ISL_16847642, EPI_ISL_16847674, EPI_ISL_16847675, EPI_ISL_16847676, EPI_ISL_16847677, EPI_ISL_16853227, EPI_ISL_16853229, EPI_ISL_16853597, EPI_ISL_16856355, EPI_ISL_16856565, EPI_ISL_16856637, EPI_ISL_16856833, EPI_ISL_16857514, EPI_ISL_16857776, EPI_ISL_16857903, EPI_ISL_16858310, EPI_ISL_16858617, EPI_ISL_16858667, EPI_ISL_16861084, EPI_ISL_16863260, EPI_ISL_16866580, EPI_ISL_16868647, EPI_ISL_16868655, EPI_ISL_16868993, EPI_ISL_16869007, EPI_ISL_16875752, EPI_ISL_16876039, EPI_ISL_16876784, EPI_ISL_16877428, EPI_ISL_16878720, EPI_ISL_16883240, EPI_ISL_16883873, EPI_ISL_16884622, EPI_ISL_16894717, EPI_ISL_16895138, EPI_ISL_16895290, EPI_ISL_16903492, EPI_ISL_16903494, EPI_ISL_16904536, EPI_ISL_16908472, EPI_ISL_16910025, EPI_ISL_16910165, EPI_ISL_16910272, EPI_ISL_16921530, EPI_ISL_16925257, EPI_ISL_16927736, EPI_ISL_16931901, EPI_ISL_16941750, EPI_ISL_16942000, EPI_ISL_16945429, EPI_ISL_16946783, EPI_ISL_16947592, EPI_ISL_16947625, EPI_ISL_16951592, EPI_ISL_16953741, EPI_ISL_16954486, EPI_ISL_16955471, EPI_ISL_16957015, EPI_ISL_16966997, EPI_ISL_16967082, EPI_ISL_16967083, EPI_ISL_16967084, EPI_ISL_16967085, EPI_ISL_16967086, EPI_ISL_16969756, EPI_ISL_16969757, EPI_ISL_16970279, EPI_ISL_16973343, EPI_ISL_16977317, EPI_ISL_16977653, EPI_ISL_16977749, EPI_ISL_16979482, EPI_ISL_16980683, EPI_ISL_16981030, EPI_ISL_16981102, EPI_ISL_16987088, EPI_ISL_16987375, EPI_ISL_16987376, EPI_ISL_16995491, EPI_ISL_16995525, EPI_ISL_16997638, EPI_ISL_17001974, EPI_ISL_17001987, EPI_ISL_17002442, EPI_ISL_17006258, EPI_ISL_17008393, EPI_ISL_17008502, EPI_ISL_17016219, EPI_ISL_17018731, EPI_ISL_17020636, EPI_ISL_17021187, EPI_ISL_17022063, EPI_ISL_17022081, EPI_ISL_17024099, EPI_ISL_17025560, EPI_ISL_17025998, EPI_ISL_17026052, EPI_ISL_17026537, EPI_ISL_17027430, EPI_ISL_17032070, EPI_ISL_17032664, EPI_ISL_17035345, EPI_ISL_17036551, EPI_ISL_17037388, EPI_ISL_17040133, EPI_ISL_17041105, EPI_ISL_17041117, EPI_ISL_17041143, EPI_ISL_17046406, EPI_ISL_17047667, EPI_ISL_17050958, EPI_ISL_17051743, EPI_ISL_17056159, EPI_ISL_17057279, EPI_ISL_17065016, EPI_ISL_17067007, EPI_ISL_17068616, EPI_ISL_17068621, EPI_ISL_17073286, EPI_ISL_17076011, EPI_ISL_17076926, EPI_ISL_17077233, EPI_ISL_17077446, EPI_ISL_17079150, EPI_ISL_17079151, EPI_ISL_17079427, EPI_ISL_17080036, EPI_ISL_17080146, EPI_ISL_17080283, EPI_ISL_17080510, EPI_ISL_17081567, EPI_ISL_17084330, EPI_ISL_17086936, EPI_ISL_17086958, EPI_ISL_17090268, EPI_ISL_17090730, EPI_ISL_17092242, EPI_ISL_17092260, EPI_ISL_17099321, EPI_ISL_17099446, EPI_ISL_17101049, EPI_ISL_17104807, EPI_ISL_17105674, EPI_ISL_17105677, EPI_ISL_17105777, EPI_ISL_17105786, EPI_ISL_17105804, EPI_ISL_17106895, EPI_ISL_17109738, EPI_ISL_17109787, EPI_ISL_17112198, EPI_ISL_17113114, EPI_ISL_17118740, EPI_ISL_17126699, EPI_ISL_17126727, EPI_ISL_17127510, EPI_ISL_17129671, EPI_ISL_17139969, EPI_ISL_17149697, EPI_ISL_17150312, EPI_ISL_17150571, EPI_ISL_17152522, EPI_ISL_17152602, EPI_ISL_17152816, EPI_ISL_17154843, EPI_ISL_17154893, EPI_ISL_17158601, EPI_ISL_17158659, EPI_ISL_17158660, EPI_ISL_17158661, EPI_ISL_17158662, EPI_ISL_17158663, EPI_ISL_17158664, EPI_ISL_17158665, EPI_ISL_17164529, EPI_ISL_17165387, EPI_ISL_17165528, EPI_ISL_17170921, EPI_ISL_17173754, EPI_ISL_17174278, EPI_ISL_17174323, EPI_ISL_17175107, EPI_ISL_17180776, EPI_ISL_17182281, EPI_ISL_17188691, EPI_ISL_17188772, EPI_ISL_17188836, EPI_ISL_17189286, EPI_ISL_17189360, EPI_ISL_17189372, EPI_ISL_17190813, EPI_ISL_17191784, EPI_ISL_17193988, EPI_ISL_17194121, EPI_ISL_17194564, EPI_ISL_17195807, EPI_ISL_17199165, EPI_ISL_17199381, EPI_ISL_17199743, EPI_ISL_17200348, EPI_ISL_17200520, EPI_ISL_17201694, EPI_ISL_17202051, EPI_ISL_17205892, EPI_ISL_17206016, EPI_ISL_17206140, EPI_ISL_17207424, EPI_ISL_17210230, EPI_ISL_17210689, EPI_ISL_17214413, EPI_ISL_17214693, EPI_ISL_17214774, EPI_ISL_17214805, EPI_ISL_17214933, EPI_ISL_17215427, EPI_ISL_17215676, EPI_ISL_17215686, EPI_ISL_17215790, EPI_ISL_17216822, EPI_ISL_17216978, EPI_ISL_17222365, EPI_ISL_17223438, EPI_ISL_17226531, EPI_ISL_17232350, EPI_ISL_17232448, EPI_ISL_17237921, EPI_ISL_17239049, EPI_ISL_17239405, EPI_ISL_17239499, EPI_ISL_17241376, EPI_ISL_17244668, EPI_ISL_17245140, EPI_ISL_17245198, EPI_ISL_17245255, EPI_ISL_17246876, EPI_ISL_17246931, EPI_ISL_17247186, EPI_ISL_17247325, EPI_ISL_17247333, EPI_ISL_17251028, EPI_ISL_17252934, EPI_ISL_17253364, EPI_ISL_17253589, EPI_ISL_17257608, EPI_ISL_17262137, EPI_ISL_17265160, EPI_ISL_17270165, EPI_ISL_17270950, EPI_ISL_17270964, EPI_ISL_17270974, EPI_ISL_17271226, EPI_ISL_17271272, EPI_ISL_17272946, EPI_ISL_17273453, EPI_ISL_17273454, EPI_ISL_17273455, EPI_ISL_17273456, EPI_ISL_17275616, EPI_ISL_17275984, EPI_ISL_17276025, EPI_ISL_17276962, EPI_ISL_17284010, EPI_ISL_17284045, EPI_ISL_17284573, EPI_ISL_17285690, EPI_ISL_17288589, EPI_ISL_17290740, EPI_ISL_17292666, EPI_ISL_17292834, EPI_ISL_17297993, EPI_ISL_17298321, EPI_ISL_17298323, EPI_ISL_17299688, EPI_ISL_17300150, EPI_ISL_17304801, EPI_ISL_17304899, EPI_ISL_17305358, EPI_ISL_17319411, EPI_ISL_17319528, EPI_ISL_17319601, EPI_ISL_17321362, EPI_ISL_17322993, EPI_ISL_17334027, EPI_ISL_17342544, EPI_ISL_17344004, EPI_ISL_17344178, EPI_ISL_17344660, EPI_ISL_17345445, EPI_ISL_17347577, EPI_ISL_17348219, EPI_ISL_17349770, EPI_ISL_17349983, EPI_ISL_17350301, EPI_ISL_17352192, EPI_ISL_17359772, EPI_ISL_17370155, EPI_ISL_17374605, EPI_ISL_17374609, EPI_ISL_17374807, EPI_ISL_17376230, EPI_ISL_17381216, EPI_ISL_17387122, EPI_ISL_17389140, EPI_ISL_17389210, EPI_ISL_17389223, EPI_ISL_17389779, EPI_ISL_17390660, EPI_ISL_17390743, EPI_ISL_17390873, EPI_ISL_17391460, EPI_ISL_17394837, EPI_ISL_17397497, EPI_ISL_17408352, EPI_ISL_17409157, EPI_ISL_17411543, EPI_ISL_17414235, EPI_ISL_17414543, EPI_ISL_17421962, EPI_ISL_17423074, EPI_ISL_17424014, EPI_ISL_17429770, EPI_ISL_17430458, EPI_ISL_17430487, EPI_ISL_17431238, EPI_ISL_17434223, EPI_ISL_17434227, EPI_ISL_17437940, EPI_ISL_17440507, EPI_ISL_17441169, EPI_ISL_17441208, EPI_ISL_17441815, EPI_ISL_17445401, EPI_ISL_17446132, EPI_ISL_17464711, EPI_ISL_17466081, EPI_ISL_17470229, EPI_ISL_17470269, EPI_ISL_17471181, EPI_ISL_17471619, EPI_ISL_17471674, EPI_ISL_17472531, EPI_ISL_17475799, EPI_ISL_17476568, EPI_ISL_17476871, EPI_ISL_17477106, EPI_ISL_17480516, EPI_ISL_17481180, EPI_ISL_17481517, EPI_ISL_17481597, EPI_ISL_17482811, EPI_ISL_17482813, EPI_ISL_17482815, EPI_ISL_17482819, EPI_ISL_17494731, EPI_ISL_17497461, EPI_ISL_17497688, EPI_ISL_17497868, EPI_ISL_17501536, EPI_ISL_17501576, EPI_ISL_17501763, EPI_ISL_17502219, EPI_ISL_17502972, EPI_ISL_17503268, EPI_ISL_17503711, EPI_ISL_17504816, EPI_ISL_17504835, EPI_ISL_17505072, EPI_ISL_17508749, EPI_ISL_17509597, EPI_ISL_17510495, EPI_ISL_17510856, EPI_ISL_17511096, EPI_ISL_17511836, EPI_ISL_17512412, EPI_ISL_17512876, EPI_ISL_17512968, EPI_ISL_17513312, EPI_ISL_17514540, EPI_ISL_17515086, EPI_ISL_17515177, EPI_ISL_17516651, EPI_ISL_17516658, EPI_ISL_17516659, EPI_ISL_17517664, EPI_ISL_17517834, EPI_ISL_17517844, EPI_ISL_17521302, EPI_ISL_17522610, EPI_ISL_17522687, EPI_ISL_17522934, EPI_ISL_17523535, EPI_ISL_17523620, EPI_ISL_17523782, EPI_ISL_17523873, EPI_ISL_17524106, EPI_ISL_17524502, EPI_ISL_17524503, EPI_ISL_17535664, EPI_ISL_17535979, EPI_ISL_17541088, EPI_ISL_17541797, EPI_ISL_17543006, EPI_ISL_17544283, EPI_ISL_17545970, EPI_ISL_17547529, EPI_ISL_17547545, EPI_ISL_17548526, EPI_ISL_17549129, EPI_ISL_17550129, EPI_ISL_17550538, EPI_ISL_17553063, EPI_ISL_17553974, EPI_ISL_17556705, EPI_ISL_17559150, EPI_ISL_17559165, EPI_ISL_17559166, EPI_ISL_17559167, EPI_ISL_17559168, EPI_ISL_17563568, EPI_ISL_17565071, EPI_ISL_17565211, EPI_ISL_17565212, EPI_ISL_17566854, EPI_ISL_17579120, EPI_ISL_17583157, EPI_ISL_17584277, EPI_ISL_17585020, EPI_ISL_17585021, EPI_ISL_17585022, EPI_ISL_17585023, EPI_ISL_17585036, EPI_ISL_17585039, EPI_ISL_17586115, EPI_ISL_17587423, EPI_ISL_17587656, EPI_ISL_17587671, EPI_ISL_17587859, EPI_ISL_17588127, EPI_ISL_17588216, EPI_ISL_17588460, EPI_ISL_17589845, EPI_ISL_17590449, EPI_ISL_17590486, EPI_ISL_17591005, EPI_ISL_17591014, EPI_ISL_17591015, EPI_ISL_17591028, EPI_ISL_17592236, EPI_ISL_17592618, EPI_ISL_17593692, EPI_ISL_17593865, EPI_ISL_17595116, EPI_ISL_17595117, EPI_ISL_17595980, EPI_ISL_17597954, EPI_ISL_17598384, EPI_ISL_17599326, EPI_ISL_17599427, EPI_ISL_17600948, EPI_ISL_17600958, EPI_ISL_17600978, EPI_ISL_17600988, EPI_ISL_17601066, EPI_ISL_17601144, EPI_ISL_17601196, EPI_ISL_17601219, EPI_ISL_17601261, EPI_ISL_17601276, EPI_ISL_17601933, EPI_ISL_17602469, EPI_ISL_17602756, EPI_ISL_17605514, EPI_ISL_17612035, EPI_ISL_17612050, EPI_ISL_17612051, EPI_ISL_17612052, EPI_ISL_17615127, EPI_ISL_17615387, EPI_ISL_17617168, EPI_ISL_17617538, EPI_ISL_17618306, EPI_ISL_17621930, EPI_ISL_17623470, EPI_ISL_17623785, EPI_ISL_17623810, EPI_ISL_17626289, EPI_ISL_17628383, EPI_ISL_17628855, EPI_ISL_17630096, EPI_ISL_17632950, EPI_ISL_17634290, EPI_ISL_17634585, EPI_ISL_17634799, EPI_ISL_17637409, EPI_ISL_17637499, EPI_ISL_17637946, EPI_ISL_17640029, EPI_ISL_17640079, EPI_ISL_17642765, EPI_ISL_17643093, EPI_ISL_17644186, EPI_ISL_17645081, EPI_ISL_17645416, EPI_ISL_17646422, EPI_ISL_17648924, EPI_ISL_17651803, EPI_ISL_17652508, EPI_ISL_17652513, EPI_ISL_17654325, EPI_ISL_17654831, EPI_ISL_17655018, EPI_ISL_17656002, EPI_ISL_17657287, EPI_ISL_17658392, EPI_ISL_17659247, EPI_ISL_17659794, EPI_ISL_17661435, EPI_ISL_17661709, EPI_ISL_17661736, EPI_ISL_17661772, EPI_ISL_17662111, EPI_ISL_17664370, EPI_ISL_17666708, EPI_ISL_17667360, EPI_ISL_17667562, EPI_ISL_17669441, EPI_ISL_17669457, EPI_ISL_17671157, EPI_ISL_17671162, EPI_ISL_17671689, EPI_ISL_17675889, EPI_ISL_17677128, EPI_ISL_17677325, EPI_ISL_17678395, EPI_ISL_17679253, EPI_ISL_17679612, EPI_ISL_17680172, EPI_ISL_17683135, EPI_ISL_17683747, EPI_ISL_17683879, EPI_ISL_17683882, EPI_ISL_17683902, EPI_ISL_17683926, EPI_ISL_17684194, EPI_ISL_17685960, EPI_ISL_17685982, EPI_ISL_17686409, EPI_ISL_17686485, EPI_ISL_17686694, EPI_ISL_17686736, EPI_ISL_17688072, EPI_ISL_17689247, EPI_ISL_17695348, EPI_ISL_17696086, EPI_ISL_17696551, EPI_ISL_17696575, EPI_ISL_17697616, EPI_ISL_17699149, EPI_ISL_17699879, EPI_ISL_17700051, EPI_ISL_17700270, EPI_ISL_17701278, EPI_ISL_17701782, EPI_ISL_17703815, EPI_ISL_17704713, EPI_ISL_17706013, EPI_ISL_17706030, EPI_ISL_17708288, EPI_ISL_17710268, EPI_ISL_17710278, EPI_ISL_17710307, EPI_ISL_17710673, EPI_ISL_17710974, EPI_ISL_17711012, EPI_ISL_17711646, EPI_ISL_17713423, EPI_ISL_17713709, EPI_ISL_17714880, EPI_ISL_17714902, EPI_ISL_17714948, EPI_ISL_17715122, EPI_ISL_17715974, EPI_ISL_17716296, EPI_ISL_17718358, EPI_ISL_17718497, EPI_ISL_17719162, EPI_ISL_17721620, EPI_ISL_17721941, EPI_ISL_17722142, EPI_ISL_17722884, EPI_ISL_17726746, EPI_ISL_17727194, EPI_ISL_17728144, EPI_ISL_17728250, EPI_ISL_17731387, EPI_ISL_17731388, EPI_ISL_17732098, EPI_ISL_17733269, EPI_ISL_17734236, EPI_ISL_17735972, EPI_ISL_17736284, EPI_ISL_17737562, EPI_ISL_17739108, EPI_ISL_17741957, EPI_ISL_17743681, EPI_ISL_17744022, EPI_ISL_17747309, EPI_ISL_17759354, EPI_ISL_17759925, EPI_ISL_17760156, EPI_ISL_17762387, EPI_ISL_17762760, EPI_ISL_17763721, EPI_ISL_17764011, EPI_ISL_17764066, EPI_ISL_17764072, EPI_ISL_17764496, EPI_ISL_17766060, EPI_ISL_17766100, EPI_ISL_17766112, EPI_ISL_17767434, EPI_ISL_17767435, EPI_ISL_17767436, EPI_ISL_17767437, EPI_ISL_17769081, EPI_ISL_17769169, EPI_ISL_17769216, EPI_ISL_17769229, EPI_ISL_17769310, EPI_ISL_17769888, EPI_ISL_17770729, EPI_ISL_17770732, EPI_ISL_17770736, EPI_ISL_17770779, EPI_ISL_17771047, EPI_ISL_17771051, EPI_ISL_17775344, EPI_ISL_17776736, EPI_ISL_17777061, EPI_ISL_17777067, EPI_ISL_17777729, EPI_ISL_17778593, EPI_ISL_17778602, EPI_ISL_17780724, EPI_ISL_17780726, EPI_ISL_17780860, EPI_ISL_17780886, EPI_ISL_17781122, EPI_ISL_17781585, EPI_ISL_17781712, EPI_ISL_17782148, EPI_ISL_17782366, EPI_ISL_17782502, EPI_ISL_17783358, EPI_ISL_17784545, EPI_ISL_17784546, EPI_ISL_17784547, EPI_ISL_17784552, EPI_ISL_17784558, EPI_ISL_17784569, EPI_ISL_17784585, EPI_ISL_17784593, EPI_ISL_17784775, EPI_ISL_17784803, EPI_ISL_17784804, EPI_ISL_17786165, EPI_ISL_17786546, EPI_ISL_17786769, EPI_ISL_17786827, EPI_ISL_17787009, EPI_ISL_17787597, EPI_ISL_17787864, EPI_ISL_17788384, EPI_ISL_17789385, EPI_ISL_17789475, EPI_ISL_17789808, EPI_ISL_17790033, EPI_ISL_17790116, EPI_ISL_17791306, EPI_ISL_17791796, EPI_ISL_17792172, EPI_ISL_17792191, EPI_ISL_17794816, EPI_ISL_17796500, EPI_ISL_17796537, EPI_ISL_17796598, EPI_ISL_17796704, EPI_ISL_17797704, EPI_ISL_17798165, EPI_ISL_17799068, EPI_ISL_17799108, EPI_ISL_17802597, EPI_ISL_17803325, EPI_ISL_17803653, EPI_ISL_17806504, EPI_ISL_17806524, EPI_ISL_17809334, EPI_ISL_17809574, EPI_ISL_17812915, EPI_ISL_17813049, EPI_ISL_17813537, EPI_ISL_17813637, EPI_ISL_17813862, EPI_ISL_17815222, EPI_ISL_17816174, EPI_ISL_17817657, EPI_ISL_17817985, EPI_ISL_17818039, EPI_ISL_17819921, EPI_ISL_17820257, EPI_ISL_17820258, EPI_ISL_17820602, EPI_ISL_17821850, EPI_ISL_17823538, EPI_ISL_17824292, EPI_ISL_17824608, EPI_ISL_17824670, EPI_ISL_17826285, EPI_ISL_17830573, EPI_ISL_17830591, EPI_ISL_17830762, EPI_ISL_17831005, EPI_ISL_17831639, EPI_ISL_17831941, EPI_ISL_17833161, EPI_ISL_17833549, EPI_ISL_17837092, EPI_ISL_17837097, EPI_ISL_17837134, EPI_ISL_17837135, EPI_ISL_17837188, EPI_ISL_17837432, EPI_ISL_17837459, EPI_ISL_17837460, EPI_ISL_17837914, EPI_ISL_17837915, EPI_ISL_17838109, EPI_ISL_17838506, EPI_ISL_17850070, EPI_ISL_17850078, EPI_ISL_17851276, EPI_ISL_17853355, EPI_ISL_17853579, EPI_ISL_17855226, EPI_ISL_17856975, EPI_ISL_17857949, EPI_ISL_17857950, EPI_ISL_17859477, EPI_ISL_17860390, EPI_ISL_17860984, EPI_ISL_17862677, EPI_ISL_17871595, EPI_ISL_17879222, EPI_ISL_17884376, EPI_ISL_17884518, EPI_ISL_17885064, EPI_ISL_17885128, EPI_ISL_17885331, EPI_ISL_17885459, EPI_ISL_17891004, EPI_ISL_17899627, EPI_ISL_17949029, EPI_ISL_17949339, EPI_ISL_17949978, EPI_ISL_17952015, EPI_ISL_17952019, EPI_ISL_17953343, EPI_ISL_17953610, EPI_ISL_17954106, EPI_ISL_17954662, EPI_ISL_17954669, EPI_ISL_17954940, EPI_ISL_17956164, EPI_ISL_17958015, EPI_ISL_17959424, EPI_ISL_17960600, EPI_ISL_17960747, EPI_ISL_17964403, EPI_ISL_17964415, EPI_ISL_17964828, EPI_ISL_17965636, EPI_ISL_17966200, EPI_ISL_17966205, EPI_ISL_17968777, EPI_ISL_17968962, EPI_ISL_17969108, EPI_ISL_17971223, EPI_ISL_17971936, EPI_ISL_17972242, EPI_ISL_17972372, EPI_ISL_17973367, EPI_ISL_17974574, EPI_ISL_17974688, EPI_ISL_17974927, EPI_ISL_17974952, EPI_ISL_17975003, EPI_ISL_17975174, EPI_ISL_17976113, EPI_ISL_17976116, EPI_ISL_17977985, EPI_ISL_17978344, EPI_ISL_17978693, EPI_ISL_17978839, EPI_ISL_17979017, EPI_ISL_17979965, EPI_ISL_17979979, EPI_ISL_17979981, EPI_ISL_17982411, EPI_ISL_17982453, EPI_ISL_17982543, EPI_ISL_17985757, EPI_ISL_17988396, EPI_ISL_17989190, EPI_ISL_17989433, EPI_ISL_17989516, EPI_ISL_17989740, EPI_ISL_17989749, EPI_ISL_17989792, EPI_ISL_17989829, EPI_ISL_17990304, EPI_ISL_17993966, EPI_ISL_17994784, EPI_ISL_17994786, EPI_ISL_17995488, EPI_ISL_17995513, EPI_ISL_17995955, EPI_ISL_17996897, EPI_ISL_17997249, EPI_ISL_17997251, EPI_ISL_17997917, EPI_ISL_17997982, EPI_ISL_17998406, EPI_ISL_18000155, EPI_ISL_18000245, EPI_ISL_18000414, EPI_ISL_18000654, EPI_ISL_18000825, EPI_ISL_18001789, EPI_ISL_18001862, EPI_ISL_18008246, EPI_ISL_18008262, EPI_ISL_18008673, EPI_ISL_18009591, EPI_ISL_18009602, EPI_ISL_18010720, EPI_ISL_18011518, EPI_ISL_18012526, EPI_ISL_18012547, EPI_ISL_18012806, EPI_ISL_18014700, EPI_ISL_18016999, EPI_ISL_18019246, EPI_ISL_18028785, EPI_ISL_18029979, EPI_ISL_18030390, EPI_ISL_18030391, EPI_ISL_18030395, EPI_ISL_18031842, EPI_ISL_18032297, EPI_ISL_18032322, EPI_ISL_18032338, EPI_ISL_18033013, EPI_ISL_18033516, EPI_ISL_18033631, EPI_ISL_18034109, EPI_ISL_18037119, EPI_ISL_18037474, EPI_ISL_18037476, EPI_ISL_18037744, EPI_ISL_18038269, EPI_ISL_18039728, EPI_ISL_18040070, EPI_ISL_18041130, EPI_ISL_18041968, EPI_ISL_18042110, EPI_ISL_18044024, EPI_ISL_18044164, EPI_ISL_18044400, EPI_ISL_18044754, EPI_ISL_18044755, EPI_ISL_18044759, EPI_ISL_18045809, EPI_ISL_18048708, EPI_ISL_18048972, EPI_ISL_18048978, EPI_ISL_18049009, EPI_ISL_18049161, EPI_ISL_18049174, EPI_ISL_18049917, EPI_ISL_18050065, EPI_ISL_18050520, EPI_ISL_18050523, EPI_ISL_18051914, EPI_ISL_18051918, EPI_ISL_18052440, EPI_ISL_18052776, EPI_ISL_18052929, EPI_ISL_18053022, EPI_ISL_18053315, EPI_ISL_18054466, EPI_ISL_18054899, EPI_ISL_18056643, EPI_ISL_18056644, EPI_ISL_18056759, EPI_ISL_18056769, EPI_ISL_18058525, EPI_ISL_18058567, EPI_ISL_18058881, EPI_ISL_18059074, EPI_ISL_18059075, EPI_ISL_18059076, EPI_ISL_18059726, EPI_ISL_18060973, EPI_ISL_18062475, EPI_ISL_18064366, EPI_ISL_18064383, EPI_ISL_18064405, EPI_ISL_18064413, EPI_ISL_18064431, EPI_ISL_18064456, EPI_ISL_18070310, EPI_ISL_18071883, EPI_ISL_18071901, EPI_ISL_18072343, EPI_ISL_18073924, EPI_ISL_18074072, EPI_ISL_18075985, EPI_ISL_18076065, EPI_ISL_18076069, EPI_ISL_18076165, EPI_ISL_18076251, EPI_ISL_18076473, EPI_ISL_18077275, EPI_ISL_18078878, EPI_ISL_18079417, EPI_ISL_18080566, EPI_ISL_18083488, EPI_ISL_18091808, EPI_ISL_18093840, EPI_ISL_18094397, EPI_ISL_18094429, EPI_ISL_18094476, EPI_ISL_18094560, EPI_ISL_18095961, EPI_ISL_18097327, EPI_ISL_18097349, EPI_ISL_18097786, EPI_ISL_18098270, EPI_ISL_18098276, EPI_ISL_18098299, EPI_ISL_18098479, EPI_ISL_18098976, EPI_ISL_18099952, EPI_ISL_18100455, EPI_ISL_18100457, EPI_ISL_18100607, EPI_ISL_18104072, EPI_ISL_18104501, EPI_ISL_18106416, EPI_ISL_18106460, EPI_ISL_18106464, EPI_ISL_18106662, EPI_ISL_18106788, EPI_ISL_18106910, EPI_ISL_18106912, EPI_ISL_18106920, EPI_ISL_18106929, EPI_ISL_18106930, EPI_ISL_18106931, EPI_ISL_18106933, EPI_ISL_18106934, EPI_ISL_18106950, EPI_ISL_18106951, EPI_ISL_18107900, EPI_ISL_18109285, EPI_ISL_18110014, EPI_ISL_18110496, EPI_ISL_18110776, EPI_ISL_18111020, EPI_ISL_18111021, EPI_ISL_18111040, EPI_ISL_18111041, EPI_ISL_18111086, EPI_ISL_18112015, EPI_ISL_18115442, EPI_ISL_18115451, EPI_ISL_18115956, EPI_ISL_18116015, EPI_ISL_18116176, EPI_ISL_18118289, EPI_ISL_18118388, EPI_ISL_18118556, EPI_ISL_18118855, EPI_ISL_18119265, EPI_ISL_18120201, EPI_ISL_18123396, EPI_ISL_18124840, EPI_ISL_18125049, EPI_ISL_18125050, EPI_ISL_18126834, EPI_ISL_18127203, EPI_ISL_18127526, EPI_ISL_18127527, EPI_ISL_18127685, EPI_ISL_18127834, EPI_ISL_18129019, EPI_ISL_18129038, EPI_ISL_18129213, EPI_ISL_18129656, EPI_ISL_18129944, EPI_ISL_18131053, EPI_ISL_18131109, EPI_ISL_18134315, EPI_ISL_18134392, EPI_ISL_18134395, EPI_ISL_18134442, EPI_ISL_18134610, EPI_ISL_18134691, EPI_ISL_18134700, EPI_ISL_18134706, EPI_ISL_18134984, EPI_ISL_18135040, EPI_ISL_18136392, EPI_ISL_18136968, EPI_ISL_18139400, EPI_ISL_18139409, EPI_ISL_18141686, EPI_ISL_18141739, EPI_ISL_18141844, EPI_ISL_18142202, EPI_ISL_18142317, EPI_ISL_18142978, EPI_ISL_18142994, EPI_ISL_18146885, EPI_ISL_18147456, EPI_ISL_18147966, EPI_ISL_18151975, EPI_ISL_18151976, EPI_ISL_18151977, EPI_ISL_18153000, EPI_ISL_18159587, EPI_ISL_18160510, EPI_ISL_18160530, EPI_ISL_18160538, EPI_ISL_18162564, EPI_ISL_18162567, EPI_ISL_18163680, EPI_ISL_18164441, EPI_ISL_18166642, EPI_ISL_18166643, EPI_ISL_18168780, EPI_ISL_18169117, EPI_ISL_18205057, EPI_ISL_18207613, EPI_ISL_18210510, EPI_ISL_18212559, EPI_ISL_18213104, EPI_ISL_18215123, EPI_ISL_18215226, EPI_ISL_18215482, EPI_ISL_18215552, EPI_ISL_18217564, EPI_ISL_18217995, EPI_ISL_18218776, EPI_ISL_18219916, EPI_ISL_18219931, EPI_ISL_18219970, EPI_ISL_18220073, EPI_ISL_18220494, EPI_ISL_18220495, EPI_ISL_18220498, EPI_ISL_18221521, EPI_ISL_18221524, EPI_ISL_18221527, EPI_ISL_18221982, EPI_ISL_18221985, EPI_ISL_18222367, EPI_ISL_18224410, EPI_ISL_18224514, EPI_ISL_18225473, EPI_ISL_18227366, EPI_ISL_18227596, EPI_ISL_18227611, EPI_ISL_18227624, EPI_ISL_18227629, EPI_ISL_18228307, EPI_ISL_18232124, EPI_ISL_18233906, EPI_ISL_18234431, EPI_ISL_18236180, EPI_ISL_18236291, EPI_ISL_18237063, EPI_ISL_18238117, EPI_ISL_18241087, EPI_ISL_18241705, EPI_ISL_18241707, EPI_ISL_18241719, EPI_ISL_18245571, EPI_ISL_18247259, EPI_ISL_18248695, EPI_ISL_18249682, EPI_ISL_18253248, EPI_ISL_18253249, EPI_ISL_18255994, EPI_ISL_18256173, EPI_ISL_18256714, EPI_ISL_18256980, EPI_ISL_18258766, EPI_ISL_18259784, EPI_ISL_18260202, EPI_ISL_18263945, EPI_ISL_18263981, EPI_ISL_18271265, EPI_ISL_18273982, EPI_ISL_18274346, EPI_ISL_18276415, EPI_ISL_18277439, EPI_ISL_18277736, EPI_ISL_18278627, EPI_ISL_18278909, EPI_ISL_18279614, EPI_ISL_18281186, EPI_ISL_18281259, EPI_ISL_18281287, EPI_ISL_18281288, EPI_ISL_18281494, EPI_ISL_18281574, EPI_ISL_18282077, EPI_ISL_18282082, EPI_ISL_18286773, EPI_ISL_18287351, EPI_ISL_18290989, EPI_ISL_18291808, EPI_ISL_18292038, EPI_ISL_18292398, EPI_ISL_18294574, EPI_ISL_18295441, EPI_ISL_18298019, EPI_ISL_18299948, EPI_ISL_18301587, EPI_ISL_18302636, EPI_ISL_18303012, EPI_ISL_18303592, EPI_ISL_18303595, EPI_ISL_18303758, EPI_ISL_18306254, EPI_ISL_18306922, EPI_ISL_18308642, EPI_ISL_18311951, EPI_ISL_18313683, EPI_ISL_18315747, EPI_ISL_18315789, EPI_ISL_18319306, EPI_ISL_18319903, EPI_ISL_18319904, EPI_ISL_18319906, EPI_ISL_18319907, EPI_ISL_18320079, EPI_ISL_18321271, EPI_ISL_18322273, EPI_ISL_18322420, EPI_ISL_18322438, EPI_ISL_18323536, EPI_ISL_18324107, EPI_ISL_18324168, EPI_ISL_18324491, EPI_ISL_18324976, EPI_ISL_18325145, EPI_ISL_18325563, EPI_ISL_18326430, EPI_ISL_18326597, EPI_ISL_18326806, EPI_ISL_18326807, EPI_ISL_18330957, EPI_ISL_18330966, EPI_ISL_18331347, EPI_ISL_18334903, EPI_ISL_18334945, EPI_ISL_18334986, EPI_ISL_18335084, EPI_ISL_18336165, EPI_ISL_18336602, EPI_ISL_18336862, EPI_ISL_18337738, EPI_ISL_18338137, EPI_ISL_18338143, EPI_ISL_18338144, EPI_ISL_18338502, EPI_ISL_18338504, EPI_ISL_18338709, EPI_ISL_18342412, EPI_ISL_18343598, EPI_ISL_18345777, EPI_ISL_18345926, EPI_ISL_18346109, EPI_ISL_18351588, EPI_ISL_18352473, EPI_ISL_18352485, EPI_ISL_18352489, EPI_ISL_18359229, EPI_ISL_18359328, EPI_ISL_18359679, EPI_ISL_18360507, EPI_ISL_18360944, EPI_ISL_18361202, EPI_ISL_18362265, EPI_ISL_18362515, EPI_ISL_18363170, EPI_ISL_18363300, EPI_ISL_18365170, EPI_ISL_18365256, EPI_ISL_18367086, EPI_ISL_18367563, EPI_ISL_18367586, EPI_ISL_18367599, EPI_ISL_18367615, EPI_ISL_18367908, EPI_ISL_18367992, EPI_ISL_18370898, EPI_ISL_18370960, EPI_ISL_18370967, EPI_ISL_18371749, EPI_ISL_18373201, EPI_ISL_18377021, EPI_ISL_18377214, EPI_ISL_18377245, EPI_ISL_18377248, EPI_ISL_18378384, EPI_ISL_18380731, EPI_ISL_18381066, EPI_ISL_18383121, EPI_ISL_18383423, EPI_ISL_18384846, EPI_ISL_18384936, EPI_ISL_18385358, EPI_ISL_18385924, EPI_ISL_18386091, EPI_ISL_18386114, EPI_ISL_18387037, EPI_ISL_18388509, EPI_ISL_18388585, EPI_ISL_18389783, EPI_ISL_18391451, EPI_ISL_18392259, EPI_ISL_18392502, EPI_ISL_18393366, EPI_ISL_18395551, EPI_ISL_18398210, EPI_ISL_18398259, EPI_ISL_18400843, EPI_ISL_18400856, EPI_ISL_18400946, EPI_ISL_18400987, EPI_ISL_18401313, EPI_ISL_18403047, EPI_ISL_18403051, EPI_ISL_18403054, EPI_ISL_18403509, EPI_ISL_18403523, EPI_ISL_18404585, EPI_ISL_18405535, EPI_ISL_18405621, EPI_ISL_18406078, EPI_ISL_18406394, EPI_ISL_18408561, EPI_ISL_18410987, EPI_ISL_18414567, EPI_ISL_18414568, EPI_ISL_18414808, EPI_ISL_18415823, EPI_ISL_18415832, EPI_ISL_18415834, EPI_ISL_18415840, EPI_ISL_18415854, EPI_ISL_18416870, EPI_ISL_18417129, EPI_ISL_18417211, EPI_ISL_18419485, EPI_ISL_18419748, EPI_ISL_18421674, EPI_ISL_18422693, EPI_ISL_18422715, EPI_ISL_18422771, EPI_ISL_18423785, EPI_ISL_18423814, EPI_ISL_18423907, EPI_ISL_18424281, EPI_ISL_18424468, EPI_ISL_18426836, EPI_ISL_18428844, EPI_ISL_18429684, EPI_ISL_18429702, EPI_ISL_18429725, EPI_ISL_18429773, EPI_ISL_18429797, EPI_ISL_18432077, EPI_ISL_18433350, EPI_ISL_18434194, EPI_ISL_18435557, EPI_ISL_18435892, EPI_ISL_18435949, EPI_ISL_18436693, EPI_ISL_18437342, EPI_ISL_18438723, EPI_ISL_18439733, EPI_ISL_18440037, EPI_ISL_18440370, EPI_ISL_18440660, EPI_ISL_18440866, EPI_ISL_18441868, EPI_ISL_18443784, EPI_ISL_18443944, EPI_ISL_18448894, EPI_ISL_18449647, EPI_ISL_18449794, EPI_ISL_18449820, EPI_ISL_18449892, EPI_ISL_18450249, EPI_ISL_18450812, EPI_ISL_18451678, EPI_ISL_18453400, EPI_ISL_18455292, EPI_ISL_18455564, EPI_ISL_18455706, EPI_ISL_18455950, EPI_ISL_18457808, EPI_ISL_18459512, EPI_ISL_18461774, EPI_ISL_18462852, EPI_ISL_18463490, EPI_ISL_18463766, EPI_ISL_18466251, EPI_ISL_18468149, EPI_ISL_18470400, EPI_ISL_18472311, EPI_ISL_18472312, EPI_ISL_18473559, EPI_ISL_18474126, EPI_ISL_18474555, EPI_ISL_18475534, EPI_ISL_18475535, EPI_ISL_18480741, EPI_ISL_18486919, EPI_ISL_18487225, EPI_ISL_18489646, EPI_ISL_18489793, EPI_ISL_18489829, EPI_ISL_18491841, EPI_ISL_18491844, EPI_ISL_18492277, EPI_ISL_18492305, EPI_ISL_18492307, EPI_ISL_18492412, EPI_ISL_18492455, EPI_ISL_18493129, EPI_ISL_18495416, EPI_ISL_18496252, EPI_ISL_18496585, EPI_ISL_18498001, EPI_ISL_18498420, EPI_ISL_18498499, EPI_ISL_18500316, EPI_ISL_18500771, EPI_ISL_18501087, EPI_ISL_18503287, EPI_ISL_18509817, EPI_ISL_18512421, EPI_ISL_18512438, EPI_ISL_18513936, EPI_ISL_18514552, EPI_ISL_18515280, EPI_ISL_18515328, EPI_ISL_18515343, EPI_ISL_18515511, EPI_ISL_18515749, EPI_ISL_18516916, EPI_ISL_18518769, EPI_ISL_18518932, EPI_ISL_18519113, EPI_ISL_18520677, EPI_ISL_18520678, EPI_ISL_18521765, EPI_ISL_18522184, EPI_ISL_18522580, EPI_ISL_18523129, EPI_ISL_18524926, EPI_ISL_18525067, EPI_ISL_18525840, EPI_ISL_18526641, EPI_ISL_18529555, EPI_ISL_18530445, EPI_ISL_18530449, EPI_ISL_18536100, EPI_ISL_18536853, EPI_ISL_18537013, EPI_ISL_18537032, EPI_ISL_18537373, EPI_ISL_18537428, EPI_ISL_18537814, EPI_ISL_18538015, EPI_ISL_18543268, EPI_ISL_18546112, EPI_ISL_18546287, EPI_ISL_18546551, EPI_ISL_18546715, EPI_ISL_18551440, EPI_ISL_18552697, EPI_ISL_18553587, EPI_ISL_18554053, EPI_ISL_18556084, EPI_ISL_18556539, EPI_ISL_18557145, EPI_ISL_18558360, EPI_ISL_18558385, EPI_ISL_18558412, EPI_ISL_18558468, EPI_ISL_18558477, EPI_ISL_18559317, EPI_ISL_18560556, EPI_ISL_18560725, EPI_ISL_18560872, EPI_ISL_18561098, EPI_ISL_18563181, EPI_ISL_18563821, EPI_ISL_18564403, EPI_ISL_18564755, EPI_ISL_18566696, EPI_ISL_18567985, EPI_ISL_18568124, EPI_ISL_18576266, EPI_ISL_18576754, EPI_ISL_18577842, EPI_ISL_18577861, EPI_ISL_18577862, EPI_ISL_18577966, EPI_ISL_18578195, EPI_ISL_18579981, EPI_ISL_18580011, EPI_ISL_18580750, EPI_ISL_18580874, EPI_ISL_18581347, EPI_ISL_18584141, EPI_ISL_18588773, EPI_ISL_18589012, EPI_ISL_18589243, EPI_ISL_18589475, EPI_ISL_18589669, EPI_ISL_18590820, EPI_ISL_18591717, EPI_ISL_18593579, EPI_ISL_18594183, EPI_ISL_18594233, EPI_ISL_18594266, EPI_ISL_18595212, EPI_ISL_18598503, EPI_ISL_18598525, EPI_ISL_18603922, EPI_ISL_18604501, EPI_ISL_18604502, EPI_ISL_18604503, EPI_ISL_18604504, EPI_ISL_18604505, EPI_ISL_18605012, EPI_ISL_18605132, EPI_ISL_18606460, EPI_ISL_18607149, EPI_ISL_18607150, EPI_ISL_18609973, EPI_ISL_18612246, EPI_ISL_18615968, EPI_ISL_18622139, EPI_ISL_18624843, EPI_ISL_18625316, EPI_ISL_18626713, EPI_ISL_18626714, EPI_ISL_18626750, EPI_ISL_18630930, EPI_ISL_18633829, EPI_ISL_18634703, EPI_ISL_18635526, EPI_ISL_18635546, EPI_ISL_18635599, EPI_ISL_18635961, EPI_ISL_18639151, EPI_ISL_18640058, EPI_ISL_18641470, EPI_ISL_18641499, EPI_ISL_18642608, EPI_ISL_18646912, EPI_ISL_18646945, EPI_ISL_18648209, EPI_ISL_18652556, EPI_ISL_18653898, EPI_ISL_18654501, EPI_ISL_18668995, EPI_ISL_18672102, EPI_ISL_18677704, EPI_ISL_18681665, EPI_ISL_18681942, EPI_ISL_18681975, EPI_ISL_18683430, EPI_ISL_18687895, EPI_ISL_18689964, EPI_ISL_18693860, EPI_ISL_18694187, EPI_ISL_18702737, EPI_ISL_18712665, EPI_ISL_18712806, EPI_ISL_18712994, EPI_ISL_18713198, EPI_ISL_18714360, EPI_ISL_18714857, EPI_ISL_18715065, EPI_ISL_18715624, EPI_ISL_18717261, EPI_ISL_18727247, EPI_ISL_18727294, EPI_ISL_18729637, EPI_ISL_18730563, EPI_ISL_18740049, EPI_ISL_18742836, EPI_ISL_18743266, EPI_ISL_18743442, EPI_ISL_18743504, EPI_ISL_18743721, EPI_ISL_18759772, EPI_ISL_18760241, EPI_ISL_18763780, EPI_ISL_18766752, EPI_ISL_18772294, EPI_ISL_18777179, EPI_ISL_18778198, EPI_ISL_18778941, EPI_ISL_18782000, EPI_ISL_18782699, EPI_ISL_18784365, EPI_ISL_18784385, EPI_ISL_18784404, EPI_ISL_18784438, EPI_ISL_18787342, EPI_ISL_18787343, EPI_ISL_18792770, EPI_ISL_18796930, EPI_ISL_18797833, EPI_ISL_18798193, EPI_ISL_18798198, EPI_ISL_18798202, EPI_ISL_18798204, EPI_ISL_18798234, EPI_ISL_18799019, EPI_ISL_18801405, EPI_ISL_18803784, EPI_ISL_18805014, EPI_ISL_18809509, EPI_ISL_18809652, EPI_ISL_18811047, EPI_ISL_18811325, EPI_ISL_18811375, EPI_ISL_18811389, EPI_ISL_18814271, EPI_ISL_18814272, EPI_ISL_18814273, EPI_ISL_18815556, EPI_ISL_18816278, EPI_ISL_18816628, EPI_ISL_18817415, EPI_ISL_18818949, EPI_ISL_18824608, EPI_ISL_18825438, EPI_ISL_18829708, EPI_ISL_18832845, EPI_ISL_18839275, EPI_ISL_18839824, EPI_ISL_18839850, EPI_ISL_18839932, EPI_ISL_18841696, EPI_ISL_18842709, EPI_ISL_18846230, EPI_ISL_18851473, EPI_ISL_18853389, EPI_ISL_18853925, EPI_ISL_18854197, EPI_ISL_18854790, EPI_ISL_18854974, EPI_ISL_18856328, EPI_ISL_18859868, EPI_ISL_18860821, EPI_ISL_18863734, EPI_ISL_18864357, EPI_ISL_18864823, EPI_ISL_18865364, EPI_ISL_18865427, EPI_ISL_18868940, EPI_ISL_18869806, EPI_ISL_18870297, EPI_ISL_18873437, EPI_ISL_18873899, EPI_ISL_18874096, EPI_ISL_18874749, EPI_ISL_18875008, EPI_ISL_18876158, EPI_ISL_18876159, EPI_ISL_18876160, EPI_ISL_18876161, EPI_ISL_18876171, EPI_ISL_18876178, EPI_ISL_18876356, EPI_ISL_18877288, EPI_ISL_18877379, EPI_ISL_18877954, EPI_ISL_18878665, EPI_ISL_18880075, EPI_ISL_18882888, EPI_ISL_18884114, EPI_ISL_18885028, EPI_ISL_18885545, EPI_ISL_18886798, EPI_ISL_18888145, EPI_ISL_18888340, EPI_ISL_18889879, EPI_ISL_18892766, EPI_ISL_18896020, EPI_ISL_18897180, EPI_ISL_18897181, EPI_ISL_18899656, EPI_ISL_18899840, EPI_ISL_18900159, EPI_ISL_18901224, EPI_ISL_18901411, EPI_ISL_18901513, EPI_ISL_18901762, EPI_ISL_18901997, EPI_ISL_18904234, EPI_ISL_18906411, EPI_ISL_18906577, EPI_ISL_18908924, EPI_ISL_18915560, EPI_ISL_18916107, EPI_ISL_18916798, EPI_ISL_18916914, EPI_ISL_18917364, EPI_ISL_18917482, EPI_ISL_18917496, EPI_ISL_18917541, EPI_ISL_18918931, EPI_ISL_18918932, EPI_ISL_18918944, EPI_ISL_18919477, EPI_ISL_18919544, EPI_ISL_18924428, EPI_ISL_18927022, EPI_ISL_18927251, EPI_ISL_18927258, EPI_ISL_18927292, EPI_ISL_18927500, EPI_ISL_18930149, EPI_ISL_18930177, EPI_ISL_18930182, EPI_ISL_18930190, EPI_ISL_18930681, EPI_ISL_18931503, EPI_ISL_18931504, EPI_ISL_18931535, EPI_ISL_18931538, EPI_ISL_18931542, EPI_ISL_18932507, EPI_ISL_18933211, EPI_ISL_18933315, EPI_ISL_18933410, EPI_ISL_18933606, EPI_ISL_18935631, EPI_ISL_18936396, EPI_ISL_18939566, EPI_ISL_18939949, EPI_ISL_18945392, EPI_ISL_18945748, EPI_ISL_18947086, EPI_ISL_18947189, EPI_ISL_18947944, EPI_ISL_18948077, EPI_ISL_18948078, EPI_ISL_18948422, EPI_ISL_18948613, EPI_ISL_18948756, EPI_ISL_18948757, EPI_ISL_18949161, EPI_ISL_18949911, EPI_ISL_18952873, EPI_ISL_18953132, EPI_ISL_18953134, EPI_ISL_18953234, EPI_ISL_18953461, EPI_ISL_18953503, EPI_ISL_18955186, EPI_ISL_18956070, EPI_ISL_18956087, EPI_ISL_18956161, EPI_ISL_18958247, EPI_ISL_18958295, EPI_ISL_18958383, EPI_ISL_18960149, EPI_ISL_18960183, EPI_ISL_18960224, EPI_ISL_18962896, EPI_ISL_18962928, EPI_ISL_18963652, EPI_ISL_18965082, EPI_ISL_18965451, EPI_ISL_18966035, EPI_ISL_18966186, EPI_ISL_18968116, EPI_ISL_18968120, EPI_ISL_18969119, EPI_ISL_18969697, EPI_ISL_18969735, EPI_ISL_18970692, EPI_ISL_18971336, EPI_ISL_18972187, EPI_ISL_18972699, EPI_ISL_18972708, EPI_ISL_18972719, EPI_ISL_18972734, EPI_ISL_18972995, EPI_ISL_18975064, EPI_ISL_18975148, EPI_ISL_18975189, EPI_ISL_18975333, EPI_ISL_18977928, EPI_ISL_18977929, EPI_ISL_18979948, EPI_ISL_18980164, EPI_ISL_18981298, EPI_ISL_18982763, EPI_ISL_18986641, EPI_ISL_18986642, EPI_ISL_18986747, EPI_ISL_18987173, EPI_ISL_18987259, EPI_ISL_18990023, EPI_ISL_18992540, EPI_ISL_18999516, EPI_ISL_19000455, EPI_ISL_19002243, EPI_ISL_19002832, EPI_ISL_19003837, EPI_ISL_19004571, EPI_ISL_19005468, EPI_ISL_19005470, EPI_ISL_19005472, EPI_ISL_19005481, EPI_ISL_19008875, EPI_ISL_19009123, EPI_ISL_19009131, EPI_ISL_19010135, EPI_ISL_19012038, EPI_ISL_19012114, EPI_ISL_19012460, EPI_ISL_19012472, EPI_ISL_19012554, EPI_ISL_19012663, EPI_ISL_19013418, EPI_ISL_19014517, EPI_ISL_19014601, EPI_ISL_19015115, EPI_ISL_19015882, EPI_ISL_19016044, EPI_ISL_19016053, EPI_ISL_19016370, EPI_ISL_19017499, EPI_ISL_19019122, EPI_ISL_19019127, EPI_ISL_19019128, EPI_ISL_19021045, EPI_ISL_19021046, EPI_ISL_19022071, EPI_ISL_19022479, EPI_ISL_19024260, EPI_ISL_19026222, EPI_ISL_19026577, EPI_ISL_19026932, EPI_ISL_19029499, EPI_ISL_19030027, EPI_ISL_19030112, EPI_ISL_19030117, EPI_ISL_19030129, EPI_ISL_19030183, EPI_ISL_19031583, EPI_ISL_19032054, EPI_ISL_19032056, EPI_ISL_19032179, EPI_ISL_19032598, EPI_ISL_19033249, EPI_ISL_19035801, EPI_ISL_19036115, EPI_ISL_19036116, EPI_ISL_19036117, EPI_ISL_19036644, EPI_ISL_19036878, EPI_ISL_19036921, EPI_ISL_19041379, EPI_ISL_19041967, EPI_ISL_19042675, EPI_ISL_19043835, EPI_ISL_19043836, EPI_ISL_19044085, EPI_ISL_19044163, EPI_ISL_19044218, EPI_ISL_19044280, EPI_ISL_19044474, EPI_ISL_19044644, EPI_ISL_19044737, EPI_ISL_19046891, EPI_ISL_19046892, EPI_ISL_19047408, EPI_ISL_19049407, EPI_ISL_19049586, EPI_ISL_19050509, EPI_ISL_19051602, EPI_ISL_19053178, EPI_ISL_19053803, EPI_ISL_19053806, EPI_ISL_19053808, EPI_ISL_19054784, EPI_ISL_19055364, EPI_ISL_19055368, EPI_ISL_19055909, EPI_ISL_19058120, EPI_ISL_19060002, EPI_ISL_19060327, EPI_ISL_19060922, EPI_ISL_19061139, EPI_ISL_19061646, EPI_ISL_19062162, EPI_ISL_19062523, EPI_ISL_19062524, EPI_ISL_19064175, EPI_ISL_19065060, EPI_ISL_19065882, EPI_ISL_19066171, EPI_ISL_19066817, EPI_ISL_19067782, EPI_ISL_19067788, EPI_ISL_19070471, EPI_ISL_19071618, EPI_ISL_19073169, EPI_ISL_19073731, EPI_ISL_19073732, EPI_ISL_19073800, EPI_ISL_19073807, EPI_ISL_19074673, EPI_ISL_19075261, EPI_ISL_19075267, EPI_ISL_19075350, EPI_ISL_19076088, EPI_ISL_19081415, EPI_ISL_19081416, EPI_ISL_19081419, EPI_ISL_19081422, EPI_ISL_19081425, EPI_ISL_19082080, EPI_ISL_19082200, EPI_ISL_19082275, EPI_ISL_19082487, EPI_ISL_19085329, EPI_ISL_19085381, EPI_ISL_19085539, EPI_ISL_19085540, EPI_ISL_19085542, EPI_ISL_19086378, EPI_ISL_19086393, EPI_ISL_19086523, EPI_ISL_19091019, EPI_ISL_19091144, EPI_ISL_19094153, EPI_ISL_19094369, EPI_ISL_19095154, EPI_ISL_19095156, EPI_ISL_19095501, EPI_ISL_19095606, EPI_ISL_19095734, EPI_ISL_19095768, EPI_ISL_19100255, EPI_ISL_19105055, EPI_ISL_19106368, EPI_ISL_19106842, EPI_ISL_19106844, EPI_ISL_19106999, EPI_ISL_19108198, EPI_ISL_19108199, EPI_ISL_19108200, EPI_ISL_19108208, EPI_ISL_19108340, EPI_ISL_19108650, EPI_ISL_19108704, EPI_ISL_19108706, EPI_ISL_19108725, EPI_ISL_19131415, EPI_ISL_19131416, EPI_ISL_19131417, EPI_ISL_19131418, EPI_ISL_19131419, EPI_ISL_19132785, EPI_ISL_19132846, EPI_ISL_19133800, EPI_ISL_19135481, EPI_ISL_19135505, EPI_ISL_19135510, EPI_ISL_19135511, EPI_ISL_19137782, EPI_ISL_19137844, EPI_ISL_19140762, EPI_ISL_19141912, EPI_ISL_19142916, EPI_ISL_19143060, EPI_ISL_19143411, EPI_ISL_19143412, EPI_ISL_19143551, EPI_ISL_19143854, EPI_ISL_19143866, EPI_ISL_19143953, EPI_ISL_19143954, EPI_ISL_19145335, EPI_ISL_19146181, EPI_ISL_19146282, EPI_ISL_19146283, EPI_ISL_19146308, EPI_ISL_19146747, EPI_ISL_19147578, EPI_ISL_19148775, EPI_ISL_19151438, EPI_ISL_19153859, EPI_ISL_19158778, EPI_ISL_19158782, EPI_ISL_19159275, EPI_ISL_19159918, EPI_ISL_19159922, EPI_ISL_19159923, EPI_ISL_19161805, EPI_ISL_19164072, EPI_ISL_19164956, EPI_ISL_19165347, EPI_ISL_19165721, EPI_ISL_19167420, EPI_ISL_19167714, EPI_ISL_19169236, EPI_ISL_19169239, EPI_ISL_19169240, EPI_ISL_19169597, EPI_ISL_19169598, EPI_ISL_19173640, EPI_ISL_19173786, EPI_ISL_19175160, EPI_ISL_19175165, EPI_ISL_19175555, EPI_ISL_19175939, EPI_ISL_19176642, EPI_ISL_19176853, EPI_ISL_19176930, EPI_ISL_19177549, EPI_ISL_19177604, EPI_ISL_19177633, EPI_ISL_19178043, EPI_ISL_19178311, EPI_ISL_19179820, EPI_ISL_19182830, EPI_ISL_19182893, EPI_ISL_19183095, EPI_ISL_19183664, EPI_ISL_19183910, EPI_ISL_19183917, EPI_ISL_19183924, EPI_ISL_19184000, EPI_ISL_19184031, EPI_ISL_19184858, EPI_ISL_19185391, EPI_ISL_19186372, EPI_ISL_19187674, EPI_ISL_19187980, EPI_ISL_19188471, EPI_ISL_19189250, EPI_ISL_19190171, EPI_ISL_19190915, EPI_ISL_19191046, EPI_ISL_19192806, EPI_ISL_19192819, EPI_ISL_19193007, EPI_ISL_19193543, EPI_ISL_19193606, EPI_ISL_19193607, EPI_ISL_19193617, EPI_ISL_19195977, EPI_ISL_19195978, EPI_ISL_19195979, EPI_ISL_19195980, EPI_ISL_19195981, EPI_ISL_19195982, EPI_ISL_19196018, EPI_ISL_19196019, EPI_ISL_19196020, EPI_ISL_19196021, EPI_ISL_19197906, EPI_ISL_19198127, EPI_ISL_19198259, EPI_ISL_19199717, EPI_ISL_19199719, EPI_ISL_19200980, EPI_ISL_19202065, EPI_ISL_19202066, EPI_ISL_19203296, EPI_ISL_19209656, EPI_ISL_19210836, EPI_ISL_19211459, EPI_ISL_19213222, EPI_ISL_19213351, EPI_ISL_19214251, EPI_ISL_19214614, EPI_ISL_19216679, EPI_ISL_19216828, EPI_ISL_19217899, EPI_ISL_19219811, EPI_ISL_19219892, EPI_ISL_19221626, EPI_ISL_19223353, EPI_ISL_19225911, EPI_ISL_19226495, EPI_ISL_19227219, EPI_ISL_19228129, EPI_ISL_19229097, EPI_ISL_19229565, EPI_ISL_19230859, EPI_ISL_19230867, EPI_ISL_19230971, EPI_ISL_19232972, EPI_ISL_19234843, EPI_ISL_19237942, EPI_ISL_19239019, EPI_ISL_19239628, EPI_ISL_19239713, EPI_ISL_19243158, EPI_ISL_19243166, EPI_ISL_19243171, EPI_ISL_19243231, EPI_ISL_19243516, EPI_ISL_19243810, EPI_ISL_19244002, EPI_ISL_19245397, EPI_ISL_19245398, EPI_ISL_19245399, EPI_ISL_19251154, EPI_ISL_19254798, EPI_ISL_19256132, EPI_ISL_19256137, EPI_ISL_19256138, EPI_ISL_19256151, EPI_ISL_19256152, EPI_ISL_19257108, EPI_ISL_19257977, EPI_ISL_19259365, EPI_ISL_19259369, EPI_ISL_19259383, EPI_ISL_19260849, EPI_ISL_19260850, EPI_ISL_19261083, EPI_ISL_19264977, EPI_ISL_19265076, EPI_ISL_19267631, EPI_ISL_19268349, EPI_ISL_19269083, EPI_ISL_19269483, EPI_ISL_19271166, EPI_ISL_19271182, EPI_ISL_19271183, EPI_ISL_19271567, EPI_ISL_19271746, EPI_ISL_19273074, EPI_ISL_19277027, EPI_ISL_19280260, EPI_ISL_19281017, EPI_ISL_19282183, EPI_ISL_19282184, EPI_ISL_19283996, EPI_ISL_19286133, EPI_ISL_19286135, EPI_ISL_19286138, EPI_ISL_19286139, EPI_ISL_19286260, EPI_ISL_19287266, EPI_ISL_19287684, EPI_ISL_19288763, EPI_ISL_19290906, EPI_ISL_19292343, EPI_ISL_19292838, EPI_ISL_19292841, EPI_ISL_19297116, EPI_ISL_19297576, EPI_ISL_19298817, EPI_ISL_19299051, EPI_ISL_19300378, EPI_ISL_19300385, EPI_ISL_19300400, EPI_ISL_19301040, EPI_ISL_19302407, EPI_ISL_19304883, EPI_ISL_19308329, EPI_ISL_19308670, EPI_ISL_19308767, EPI_ISL_19308875, EPI_ISL_19308876, EPI_ISL_19308877, EPI_ISL_19308906, EPI_ISL_19309906, EPI_ISL_19310234, EPI_ISL_19311057, EPI_ISL_19311769, EPI_ISL_19312520, EPI_ISL_19318476, EPI_ISL_19319006, EPI_ISL_19320511, EPI_ISL_19322462, EPI_ISL_19324934, EPI_ISL_19326359, EPI_ISL_19331433, EPI_ISL_19332294, EPI_ISL_19333087, EPI_ISL_19340140, EPI_ISL_19341128, EPI_ISL_19341144, EPI_ISL_19344211, EPI_ISL_19345465, EPI_ISL_19345485, EPI_ISL_19346102, EPI_ISL_19346538, EPI_ISL_19348673, EPI_ISL_19351027, EPI_ISL_19351032, EPI_ISL_19351033, EPI_ISL_19351648, EPI_ISL_19351927, EPI_ISL_19359999, EPI_ISL_19360941, EPI_ISL_19362874, EPI_ISL_19362955, EPI_ISL_19363093, EPI_ISL_19364322, EPI_ISL_19364675, EPI_ISL_19369553, EPI_ISL_19369713, EPI_ISL_19374392, EPI_ISL_19374843, EPI_ISL_19374844, EPI_ISL_19374845, EPI_ISL_19380467, EPI_ISL_19381264, EPI_ISL_19381428, EPI_ISL_19381638, EPI_ISL_19381992, EPI_ISL_19381994, EPI_ISL_19382602, EPI_ISL_19383694, EPI_ISL_19384121, EPI_ISL_19385914, EPI_ISL_19385980, EPI_ISL_19387703, EPI_ISL_19388165, EPI_ISL_19388758, EPI_ISL_19391206, EPI_ISL_19391216, EPI_ISL_19393434, EPI_ISL_19393708, EPI_ISL_19398369, EPI_ISL_19405918, EPI_ISL_19408692, EPI_ISL_19408693, EPI_ISL_19410044, EPI_ISL_19410056, EPI_ISL_19410058, EPI_ISL_19411869, EPI_ISL_19412054, EPI_ISL_19414452, EPI_ISL_19414669, EPI_ISL_19414842, EPI_ISL_19415183, EPI_ISL_19415272, EPI_ISL_19415273, EPI_ISL_19415333, EPI_ISL_19417987, EPI_ISL_19418385, EPI_ISL_19418789, EPI_ISL_19419991, EPI_ISL_19425659, EPI_ISL_19427049, EPI_ISL_19427050, EPI_ISL_19427051, EPI_ISL_19428450, EPI_ISL_19428673, EPI_ISL_19431719, EPI_ISL_19433335, EPI_ISL_19434973, EPI_ISL_19438222, EPI_ISL_19441794, EPI_ISL_19446721, EPI_ISL_19446726, EPI_ISL_19447859, EPI_ISL_19452022, EPI_ISL_19456759, EPI_ISL_19457982, EPI_ISL_19458104, EPI_ISL_19458866, EPI_ISL_19463787, EPI_ISL_19463811, EPI_ISL_19464534, EPI_ISL_19465468, EPI_ISL_19467713, EPI_ISL_19467725, EPI_ISL_19468710, EPI_ISL_19473728, EPI_ISL_19474606, EPI_ISL_19474613, EPI_ISL_19477107, EPI_ISL_19478383, EPI_ISL_19478598, EPI_ISL_19479514, EPI_ISL_19480237, EPI_ISL_19482235, EPI_ISL_19483178, EPI_ISL_19483184, EPI_ISL_19483313, EPI_ISL_19486141, EPI_ISL_19488917, EPI_ISL_19491347, EPI_ISL_19495855, EPI_ISL_19498393, EPI_ISL_19499089, EPI_ISL_19499199, EPI_ISL_19499228, EPI_ISL_19499640, EPI_ISL_19499789, EPI_ISL_19500407, EPI_ISL_19502648, EPI_ISL_19503869, EPI_ISL_19506293, EPI_ISL_19506337, EPI_ISL_19506387, EPI_ISL_19506391, EPI_ISL_19506410, EPI_ISL_19506412, EPI_ISL_19506489, EPI_ISL_19506517, EPI_ISL_19506572, EPI_ISL_19508878, EPI_ISL_19510518, EPI_ISL_19511239, EPI_ISL_19511242, EPI_ISL_19511245, EPI_ISL_19511259, EPI_ISL_19511353, EPI_ISL_19512930, EPI_ISL_19513189, EPI_ISL_19513373, EPI_ISL_19513376, EPI_ISL_19517945, EPI_ISL_19519860, EPI_ISL_19521406, EPI_ISL_19522101, EPI_ISL_19526975, EPI_ISL_19527216, EPI_ISL_19527499, EPI_ISL_19529679, EPI_ISL_19529699, EPI_ISL_19530201, EPI_ISL_19530213, EPI_ISL_19530220, EPI_ISL_19532518, EPI_ISL_19535971, EPI_ISL_19536358, EPI_ISL_19537504, EPI_ISL_19537517, EPI_ISL_19537806, EPI_ISL_19539586, EPI_ISL_19539602, EPI_ISL_19541318, EPI_ISL_19543648, EPI_ISL_19543801, EPI_ISL_19546384, EPI_ISL_19550039, EPI_ISL_19550107, EPI_ISL_19551038, EPI_ISL_19551134, EPI_ISL_19555102, EPI_ISL_19555114, EPI_ISL_19555115, EPI_ISL_19557993, EPI_ISL_19558034, EPI_ISL_19560828, EPI_ISL_19560838, EPI_ISL_19561072, EPI_ISL_19561079, EPI_ISL_19562666, EPI_ISL_19575892, EPI_ISL_19578009, EPI_ISL_19578384, EPI_ISL_19585411, EPI_ISL_19585472, EPI_ISL_19588050, EPI_ISL_19588617, EPI_ISL_19588630, EPI_ISL_19589209, EPI_ISL_19589226, EPI_ISL_19589644, EPI_ISL_19591990, EPI_ISL_19600727, EPI_ISL_19601813, EPI_ISL_19603076, EPI_ISL_19603396, EPI_ISL_19603766, EPI_ISL_19606872, EPI_ISL_19606873, EPI_ISL_19606893, EPI_ISL_19613477, EPI_ISL_19613481, EPI_ISL_19615536, EPI_ISL_19615541, EPI_ISL_19615759, EPI_ISL_19616027, EPI_ISL_19616479, EPI_ISL_19618593, EPI_ISL_19619689, EPI_ISL_19619734, EPI_ISL_19619757, EPI_ISL_19620010, EPI_ISL_19621954, EPI_ISL_19622266, EPI_ISL_19623358, EPI_ISL_19623503, EPI_ISL_19627737, EPI_ISL_19627776, EPI_ISL_19627778, EPI_ISL_19630975, EPI_ISL_19631111, EPI_ISL_19632962, EPI_ISL_19634022, EPI_ISL_19636173, EPI_ISL_19638051, EPI_ISL_19639319, EPI_ISL_19640624, EPI_ISL_19641389, EPI_ISL_19641840, EPI_ISL_19642771, EPI_ISL_19643417, EPI_ISL_19643444, EPI_ISL_19643542, EPI_ISL_19643616, EPI_ISL_19643666, EPI_ISL_19645977, EPI_ISL_19648297, EPI_ISL_19655163, EPI_ISL_19656451, EPI_ISL_19657328, EPI_ISL_19657519, EPI_ISL_19657528, EPI_ISL_19660180, EPI_ISL_19661881, EPI_ISL_19661997, EPI_ISL_19666574, EPI_ISL_19671026, EPI_ISL_19671598, EPI_ISL_19671605, EPI_ISL_19671606, EPI_ISL_19671607, EPI_ISL_19671887, EPI_ISL_19673194, EPI_ISL_19673201, EPI_ISL_19676136, EPI_ISL_19680534, EPI_ISL_19683226, EPI_ISL_19683235, EPI_ISL_19683886, EPI_ISL_19685790, EPI_ISL_19685811, EPI_ISL_19685815, EPI_ISL_19685993, EPI_ISL_19689294, EPI_ISL_19690695, EPI_ISL_19693383, EPI_ISL_19693416, EPI_ISL_19693419, EPI_ISL_19696986, EPI_ISL_19703902, EPI_ISL_19704129, EPI_ISL_19704643, EPI_ISL_19706712, EPI_ISL_19706713, EPI_ISL_19710331, EPI_ISL_19710359, EPI_ISL_19711118, EPI_ISL_19711122, EPI_ISL_19713810, EPI_ISL_19713821, EPI_ISL_19713832, EPI_ISL_19714066, EPI_ISL_19717002")
set2 = stringlist_to_strings_set("EPI_ISL_474824, EPI_ISL_508674, EPI_ISL_510148, EPI_ISL_516999, EPI_ISL_517000, EPI_ISL_528438, EPI_ISL_534720, EPI_ISL_539541, EPI_ISL_539542, EPI_ISL_593478, EPI_ISL_593479, EPI_ISL_593480, EPI_ISL_593553, EPI_ISL_593554, EPI_ISL_593555, EPI_ISL_593556, EPI_ISL_593557, EPI_ISL_593558, EPI_ISL_596228, EPI_ISL_602912, EPI_ISL_654166, EPI_ISL_654170, EPI_ISL_654172, EPI_ISL_654173, EPI_ISL_654182, EPI_ISL_654186, EPI_ISL_654191, EPI_ISL_654194, EPI_ISL_678289, EPI_ISL_686537, EPI_ISL_732971, EPI_ISL_753676, EPI_ISL_776770, EPI_ISL_801876, EPI_ISL_812862, EPI_ISL_831496, EPI_ISL_856750, EPI_ISL_872778, EPI_ISL_941340, EPI_ISL_949208, EPI_ISL_959309, EPI_ISL_979349, EPI_ISL_1014810, EPI_ISL_1039159, EPI_ISL_1039839, EPI_ISL_1059094, EPI_ISL_1081672, EPI_ISL_1081673, EPI_ISL_1090851, EPI_ISL_1104882, EPI_ISL_1105146, EPI_ISL_1105179, EPI_ISL_1105235, EPI_ISL_1158169, EPI_ISL_1200867, EPI_ISL_1200912, EPI_ISL_1209365, EPI_ISL_1225620, EPI_ISL_1241756, EPI_ISL_1248458, EPI_ISL_1248485, EPI_ISL_1248497, EPI_ISL_1257978, EPI_ISL_1261009, EPI_ISL_1295569, EPI_ISL_1295575, EPI_ISL_1309105, EPI_ISL_1347940, EPI_ISL_1347941, EPI_ISL_1347942, EPI_ISL_1366562, EPI_ISL_1366563, EPI_ISL_1366564, EPI_ISL_1366565, EPI_ISL_1366566, EPI_ISL_1366567, EPI_ISL_1366568, EPI_ISL_1366569, EPI_ISL_1366570, EPI_ISL_1366571, EPI_ISL_1366572, EPI_ISL_1366573, EPI_ISL_1366792, EPI_ISL_1372287, EPI_ISL_1372288, EPI_ISL_1376522, EPI_ISL_1376523, EPI_ISL_1378739, EPI_ISL_1403404, EPI_ISL_1469973, EPI_ISL_1470396, EPI_ISL_1473700, EPI_ISL_1477334, EPI_ISL_1483302, EPI_ISL_1490655, EPI_ISL_1490669, EPI_ISL_1495749, EPI_ISL_1522107, EPI_ISL_1534324, EPI_ISL_1547461, EPI_ISL_1575358, EPI_ISL_1578495, EPI_ISL_1626185, EPI_ISL_1637040, EPI_ISL_1668821, EPI_ISL_1668822, EPI_ISL_1668823, EPI_ISL_1668824, EPI_ISL_1668825, EPI_ISL_1670378, EPI_ISL_1671330, EPI_ISL_1675190, EPI_ISL_1675203, EPI_ISL_1738308, EPI_ISL_1743263, EPI_ISL_1744401, EPI_ISL_1756179, EPI_ISL_1756180, EPI_ISL_1792929, EPI_ISL_1829054, EPI_ISL_1829108, EPI_ISL_1840893, EPI_ISL_1855854, EPI_ISL_1904901, EPI_ISL_1904903, EPI_ISL_1908476, EPI_ISL_1935116, EPI_ISL_1935282, EPI_ISL_1941336, EPI_ISL_1941816, EPI_ISL_1965714, EPI_ISL_1965722, EPI_ISL_2001260, EPI_ISL_2001292, EPI_ISL_2030332, EPI_ISL_2035047, EPI_ISL_2080876, EPI_ISL_2096935, EPI_ISL_2140680, EPI_ISL_2142447, EPI_ISL_2159603, EPI_ISL_2170618, EPI_ISL_2179080, EPI_ISL_2179597, EPI_ISL_2179598, EPI_ISL_2179600, EPI_ISL_2179601, EPI_ISL_2179635, EPI_ISL_2193387, EPI_ISL_2193781, EPI_ISL_2193790, EPI_ISL_2229473, EPI_ISL_2232987, EPI_ISL_2232988, EPI_ISL_2245655, EPI_ISL_2246946, EPI_ISL_2272316, EPI_ISL_2281463, EPI_ISL_2285732, EPI_ISL_2289324, EPI_ISL_2335139, EPI_ISL_2355027, EPI_ISL_2373667, EPI_ISL_2373676, EPI_ISL_2373689, EPI_ISL_2373915, EPI_ISL_2373976, EPI_ISL_2376734, EPI_ISL_2385134, EPI_ISL_2397307, EPI_ISL_2397308, EPI_ISL_2397309, EPI_ISL_2397310, EPI_ISL_2397311, EPI_ISL_2397312, EPI_ISL_2397313, EPI_ISL_2403056, EPI_ISL_2408213, EPI_ISL_2443102, EPI_ISL_2443306, EPI_ISL_2451852, EPI_ISL_2453771, EPI_ISL_2456706, EPI_ISL_2466638, EPI_ISL_2482552, EPI_ISL_2482891, EPI_ISL_2484152, EPI_ISL_2492266, EPI_ISL_2501697, EPI_ISL_2504017, EPI_ISL_2510252, EPI_ISL_2526835, EPI_ISL_2537393, EPI_ISL_2545260, EPI_ISL_2567482, EPI_ISL_2567516, EPI_ISL_2597478, EPI_ISL_2598472, EPI_ISL_2621566, EPI_ISL_2622092, EPI_ISL_2626505, EPI_ISL_2629070, EPI_ISL_2629071, EPI_ISL_2629072, EPI_ISL_2629073, EPI_ISL_2629074, EPI_ISL_2629075, EPI_ISL_2646107, EPI_ISL_2658958, EPI_ISL_2658962, EPI_ISL_2658963, EPI_ISL_2658970, EPI_ISL_2658971, EPI_ISL_2658972, EPI_ISL_2681259, EPI_ISL_2686814, EPI_ISL_2686837, EPI_ISL_2713004, EPI_ISL_2716246, EPI_ISL_2746031, EPI_ISL_2758178, EPI_ISL_2758179, EPI_ISL_2776212, EPI_ISL_2790083, EPI_ISL_2791260, EPI_ISL_2810326, EPI_ISL_2811857, EPI_ISL_2817504, EPI_ISL_2820526, EPI_ISL_2828407, EPI_ISL_2833904, EPI_ISL_2858161, EPI_ISL_2858877, EPI_ISL_2860316, EPI_ISL_2868572, EPI_ISL_2868616, EPI_ISL_2876377, EPI_ISL_2894033, EPI_ISL_2903438, EPI_ISL_2931876, EPI_ISL_2931884, EPI_ISL_2931896, EPI_ISL_2931903, EPI_ISL_2955288, EPI_ISL_2955320, EPI_ISL_2966985, EPI_ISL_2978243, EPI_ISL_2978352, EPI_ISL_2984725, EPI_ISL_2990101, EPI_ISL_2993722, EPI_ISL_3010321, EPI_ISL_3022702, EPI_ISL_3029841, EPI_ISL_3030114, EPI_ISL_3030118, EPI_ISL_3030145, EPI_ISL_3030738, EPI_ISL_3032627, EPI_ISL_3033635, EPI_ISL_3039352, EPI_ISL_3045789, EPI_ISL_3053903, EPI_ISL_3061061, EPI_ISL_3063770, EPI_ISL_3064314, EPI_ISL_3129808, EPI_ISL_3130077, EPI_ISL_3130081, EPI_ISL_3130177, EPI_ISL_3130302, EPI_ISL_3132631, EPI_ISL_3133023, EPI_ISL_3153240, EPI_ISL_3164424, EPI_ISL_3185346, EPI_ISL_3209041, EPI_ISL_3212959, EPI_ISL_3246237, EPI_ISL_3251444, EPI_ISL_3251605, EPI_ISL_3259560, EPI_ISL_3275376, EPI_ISL_3339536, EPI_ISL_3356734, EPI_ISL_3358574, EPI_ISL_3370176, EPI_ISL_3394321, EPI_ISL_3396491, EPI_ISL_3414767, EPI_ISL_3414889, EPI_ISL_3415104, EPI_ISL_3415226, EPI_ISL_3426474, EPI_ISL_3446827, EPI_ISL_3453279, EPI_ISL_3457423, EPI_ISL_3459118, EPI_ISL_3471360, EPI_ISL_3475993, EPI_ISL_3497667, EPI_ISL_3503811, EPI_ISL_3504036, EPI_ISL_3509013, EPI_ISL_3556945, EPI_ISL_3634003, EPI_ISL_3634004, EPI_ISL_3635506, EPI_ISL_3657112, EPI_ISL_3712919, EPI_ISL_3771882, EPI_ISL_3779849, EPI_ISL_3838306, EPI_ISL_3891136, EPI_ISL_3933252, EPI_ISL_3937027, EPI_ISL_3957778, EPI_ISL_3958461, EPI_ISL_3958994, EPI_ISL_3982251, EPI_ISL_4029567, EPI_ISL_4052911, EPI_ISL_4072038, EPI_ISL_4096626, EPI_ISL_4096639, EPI_ISL_4114033, EPI_ISL_4124532, EPI_ISL_4178790, EPI_ISL_4193135, EPI_ISL_4198270, EPI_ISL_4251611, EPI_ISL_4261403, EPI_ISL_4261411, EPI_ISL_4281194, EPI_ISL_4284228, EPI_ISL_4295678, EPI_ISL_4298277, EPI_ISL_4298278, EPI_ISL_4298279, EPI_ISL_4309817, EPI_ISL_4415808, EPI_ISL_4440075, EPI_ISL_4515444, EPI_ISL_4525691, EPI_ISL_4525698, EPI_ISL_4525700, EPI_ISL_4531313, EPI_ISL_4536418, EPI_ISL_4576991, EPI_ISL_4577474, EPI_ISL_4625101, EPI_ISL_4652284, EPI_ISL_4769386, EPI_ISL_4775547, EPI_ISL_4875939, EPI_ISL_4930863, EPI_ISL_4935777, EPI_ISL_4935949, EPI_ISL_4936095, EPI_ISL_4936533, EPI_ISL_4949584, EPI_ISL_5018695, EPI_ISL_5033183, EPI_ISL_5056434, EPI_ISL_5059980, EPI_ISL_5099310, EPI_ISL_5132437, EPI_ISL_5132595, EPI_ISL_5145656, EPI_ISL_5196003, EPI_ISL_5265214, EPI_ISL_5280146, EPI_ISL_5307398, EPI_ISL_5323016, EPI_ISL_5332877, EPI_ISL_5332878, EPI_ISL_5395558, EPI_ISL_5446154, EPI_ISL_5463914, EPI_ISL_5532714, EPI_ISL_5592605, EPI_ISL_5620309, EPI_ISL_5621224, EPI_ISL_5628248, EPI_ISL_5639913, EPI_ISL_5640459, EPI_ISL_5649323, EPI_ISL_5650474, EPI_ISL_5655562, EPI_ISL_5680241, EPI_ISL_5692774, EPI_ISL_5749185, EPI_ISL_5780324, EPI_ISL_5814411, EPI_ISL_5865553, EPI_ISL_5892132, EPI_ISL_5922347, EPI_ISL_5935407, EPI_ISL_5944665, EPI_ISL_5944669, EPI_ISL_5944842, EPI_ISL_5944948, EPI_ISL_5946914, EPI_ISL_6017746, EPI_ISL_6017747, EPI_ISL_6076460, EPI_ISL_6208674, EPI_ISL_6208675, EPI_ISL_6208676, EPI_ISL_6227177, EPI_ISL_6227208, EPI_ISL_6262165, EPI_ISL_6324366, EPI_ISL_6327943, EPI_ISL_6381841, EPI_ISL_6384755, EPI_ISL_6574278, EPI_ISL_6584511, EPI_ISL_6604686, EPI_ISL_6605003, EPI_ISL_6605659, EPI_ISL_6628662, EPI_ISL_6642599, EPI_ISL_6666037, EPI_ISL_6710721, EPI_ISL_6735468, EPI_ISL_6737833, EPI_ISL_6783610, EPI_ISL_6810267, EPI_ISL_6814822, EPI_ISL_6826536, EPI_ISL_6842893, EPI_ISL_6863316, EPI_ISL_6863457, EPI_ISL_6865741, EPI_ISL_6938691, EPI_ISL_6976497, EPI_ISL_6977941, EPI_ISL_7000663, EPI_ISL_7015624, EPI_ISL_7015625, EPI_ISL_7135374, EPI_ISL_7159687, EPI_ISL_7204318, EPI_ISL_7334013, EPI_ISL_7361483, EPI_ISL_7361527, EPI_ISL_7452581, EPI_ISL_7452603, EPI_ISL_7456462, EPI_ISL_7458003, EPI_ISL_7502990, EPI_ISL_7503221, EPI_ISL_7592661, EPI_ISL_7592687, EPI_ISL_7652115, EPI_ISL_7660179, EPI_ISL_7707631, EPI_ISL_7711813, EPI_ISL_7729239, EPI_ISL_7806535, EPI_ISL_7806536, EPI_ISL_7806549, EPI_ISL_7813896, EPI_ISL_7861580, EPI_ISL_7908114, EPI_ISL_7961502, EPI_ISL_7976931, EPI_ISL_7980711, EPI_ISL_8001538, EPI_ISL_8035582, EPI_ISL_8131224, EPI_ISL_8132253, EPI_ISL_8151798, EPI_ISL_8153087, EPI_ISL_8166542, EPI_ISL_8189765, EPI_ISL_8189775, EPI_ISL_8204828, EPI_ISL_8205040, EPI_ISL_8207600, EPI_ISL_8215783, EPI_ISL_8251200, EPI_ISL_8263463, EPI_ISL_8376888, EPI_ISL_8479639, EPI_ISL_8479640, EPI_ISL_8563217, EPI_ISL_8563218, EPI_ISL_8563219, EPI_ISL_8615077, EPI_ISL_8627379, EPI_ISL_8669281, EPI_ISL_8712661, EPI_ISL_8725398, EPI_ISL_8725399, EPI_ISL_8725400, EPI_ISL_8725401, EPI_ISL_8725402, EPI_ISL_8725403, EPI_ISL_8725404, EPI_ISL_8725405, EPI_ISL_8725406, EPI_ISL_8725407, EPI_ISL_8725408, EPI_ISL_8725409, EPI_ISL_8732699, EPI_ISL_8732807, EPI_ISL_8732841, EPI_ISL_8750545, EPI_ISL_8754305, EPI_ISL_8766992, EPI_ISL_8800409, EPI_ISL_8806077, EPI_ISL_8806082, EPI_ISL_8806084, EPI_ISL_8819629, EPI_ISL_8825833, EPI_ISL_8828662, EPI_ISL_8887845, EPI_ISL_8887874, EPI_ISL_8973278, EPI_ISL_9021214, EPI_ISL_9141923, EPI_ISL_9155607, EPI_ISL_9201951, EPI_ISL_9242265, EPI_ISL_9242269, EPI_ISL_9316982, EPI_ISL_9319180, EPI_ISL_9630717, EPI_ISL_9637481, EPI_ISL_9637840, EPI_ISL_9702285, EPI_ISL_9735644, EPI_ISL_9844246, EPI_ISL_9873278, EPI_ISL_9907655, EPI_ISL_9949797, EPI_ISL_9949799, EPI_ISL_10124646, EPI_ISL_10127751, EPI_ISL_10128185, EPI_ISL_10132276, EPI_ISL_10185453, EPI_ISL_10195257, EPI_ISL_10195262, EPI_ISL_10195263, EPI_ISL_10195264, EPI_ISL_10195305, EPI_ISL_10230612, EPI_ISL_10239201, EPI_ISL_10251304, EPI_ISL_10329391, EPI_ISL_10329558, EPI_ISL_10352747, EPI_ISL_10397517, EPI_ISL_10451205, EPI_ISL_10451252, EPI_ISL_10548912, EPI_ISL_10548913, EPI_ISL_10548915, EPI_ISL_10548916, EPI_ISL_10548917, EPI_ISL_10548918, EPI_ISL_10548919, EPI_ISL_10548920, EPI_ISL_10548921, EPI_ISL_10548922, EPI_ISL_10549162, EPI_ISL_10549163, EPI_ISL_10549164, EPI_ISL_10549165, EPI_ISL_10549166, EPI_ISL_10591808, EPI_ISL_10681118, EPI_ISL_10706292, EPI_ISL_10712909, EPI_ISL_10815044, EPI_ISL_10816730, EPI_ISL_10816731, EPI_ISL_10816732, EPI_ISL_10816733, EPI_ISL_10816734, EPI_ISL_10816735, EPI_ISL_10816736, EPI_ISL_10816737, EPI_ISL_10816738, EPI_ISL_10816739, EPI_ISL_10816741, EPI_ISL_10816742, EPI_ISL_10816743, EPI_ISL_10816744, EPI_ISL_10824028, EPI_ISL_10876034, EPI_ISL_10876749, EPI_ISL_10899907, EPI_ISL_10942195, EPI_ISL_10981395, EPI_ISL_10995323, EPI_ISL_11025821, EPI_ISL_11025897, EPI_ISL_11030507, EPI_ISL_11036385, EPI_ISL_11036386, EPI_ISL_11036389, EPI_ISL_11036399, EPI_ISL_11036451, EPI_ISL_11036666, EPI_ISL_11036688, EPI_ISL_11036712, EPI_ISL_11036915, EPI_ISL_11036917, EPI_ISL_11050902, EPI_ISL_11055609, EPI_ISL_11110730, EPI_ISL_11167907, EPI_ISL_11173072, EPI_ISL_11219235, EPI_ISL_11219236, EPI_ISL_11221773, EPI_ISL_11221782, EPI_ISL_11222620, EPI_ISL_11229672, EPI_ISL_11248919, EPI_ISL_11256669, EPI_ISL_11290054, EPI_ISL_11295642, EPI_ISL_11296415, EPI_ISL_11356268, EPI_ISL_11356269, EPI_ISL_11403393, EPI_ISL_11437359, EPI_ISL_11482304, EPI_ISL_11503909, EPI_ISL_11504189, EPI_ISL_11517385, EPI_ISL_11565840, EPI_ISL_11576757, EPI_ISL_11600821, EPI_ISL_11600822, EPI_ISL_11621597, EPI_ISL_11657715, EPI_ISL_11661806, EPI_ISL_11695384, EPI_ISL_11742572, EPI_ISL_11742812, EPI_ISL_11747289, EPI_ISL_11778548, EPI_ISL_11787443, EPI_ISL_11798407, EPI_ISL_11798458, EPI_ISL_11801453, EPI_ISL_11816904, EPI_ISL_11825798, EPI_ISL_11826326, EPI_ISL_11826898, EPI_ISL_11871462, EPI_ISL_11886470, EPI_ISL_11886479, EPI_ISL_11886624, EPI_ISL_11897546, EPI_ISL_11961223, EPI_ISL_11968830, EPI_ISL_11968876, EPI_ISL_11970393, EPI_ISL_11976211, EPI_ISL_11994571, EPI_ISL_11995938, EPI_ISL_12001179, EPI_ISL_12001180, EPI_ISL_12021469, EPI_ISL_12039060, EPI_ISL_12063598, EPI_ISL_12063599, EPI_ISL_12063600, EPI_ISL_12063601, EPI_ISL_12063602, EPI_ISL_12079999, EPI_ISL_12080356, EPI_ISL_12083619, EPI_ISL_12089943, EPI_ISL_12095320, EPI_ISL_12097355, EPI_ISL_12108965, EPI_ISL_12109074, EPI_ISL_12127282, EPI_ISL_12137683, EPI_ISL_12139045, EPI_ISL_12139066, EPI_ISL_12145506, EPI_ISL_12146579, EPI_ISL_12148419, EPI_ISL_12150077, EPI_ISL_12150259, EPI_ISL_12150361, EPI_ISL_12150484, EPI_ISL_12155759, EPI_ISL_12155809, EPI_ISL_12157165, EPI_ISL_12157166, EPI_ISL_12157187, EPI_ISL_12158960, EPI_ISL_12162547, EPI_ISL_12168418, EPI_ISL_12171333, EPI_ISL_12171674, EPI_ISL_12172132, EPI_ISL_12172842, EPI_ISL_12173486, EPI_ISL_12173730, EPI_ISL_12173879, EPI_ISL_12174612, EPI_ISL_12174734, EPI_ISL_12174735, EPI_ISL_12174736, EPI_ISL_12174739, EPI_ISL_12174942, EPI_ISL_12175020, EPI_ISL_12175024, EPI_ISL_12175203, EPI_ISL_12176184, EPI_ISL_12207682, EPI_ISL_12220762, EPI_ISL_12240087, EPI_ISL_12240983, EPI_ISL_12278477, EPI_ISL_12278678, EPI_ISL_12278997, EPI_ISL_12284821, EPI_ISL_12293549, EPI_ISL_12293608, EPI_ISL_12323992, EPI_ISL_12325408, EPI_ISL_12350967, EPI_ISL_12351281, EPI_ISL_12355622, EPI_ISL_12422504, EPI_ISL_12425033, EPI_ISL_12430022, EPI_ISL_12464077, EPI_ISL_12467081, EPI_ISL_12473693, EPI_ISL_12475004, EPI_ISL_12486436, EPI_ISL_12488441, EPI_ISL_12501519, EPI_ISL_12511246, EPI_ISL_12530780, EPI_ISL_12531462, EPI_ISL_12539663, EPI_ISL_12564261, EPI_ISL_12568162, EPI_ISL_12568208, EPI_ISL_12575298, EPI_ISL_12590958, EPI_ISL_12602903, EPI_ISL_12611697, EPI_ISL_12611721, EPI_ISL_12616586, EPI_ISL_12622901, EPI_ISL_12622902, EPI_ISL_12623284, EPI_ISL_12639714, EPI_ISL_12639788, EPI_ISL_12639917, EPI_ISL_12645823, EPI_ISL_12646116, EPI_ISL_12647336, EPI_ISL_12652423, EPI_ISL_12654179, EPI_ISL_12685124, EPI_ISL_12685126, EPI_ISL_12698937, EPI_ISL_12701772, EPI_ISL_12701820, EPI_ISL_12701858, EPI_ISL_12701867, EPI_ISL_12701871, EPI_ISL_12701895, EPI_ISL_12703517, EPI_ISL_12710394, EPI_ISL_12735772, EPI_ISL_12739317, EPI_ISL_12739425, EPI_ISL_12742126, EPI_ISL_12744193, EPI_ISL_12753427, EPI_ISL_12754976, EPI_ISL_12763802, EPI_ISL_12771614, EPI_ISL_12784028, EPI_ISL_12789812, EPI_ISL_12789846, EPI_ISL_12799990, EPI_ISL_12805921, EPI_ISL_12808264, EPI_ISL_12809016, EPI_ISL_12811012, EPI_ISL_12822481, EPI_ISL_12822483, EPI_ISL_12829036, EPI_ISL_12830215, EPI_ISL_12830827, EPI_ISL_12843060, EPI_ISL_12844170, EPI_ISL_12851188, EPI_ISL_12851233, EPI_ISL_12851285, EPI_ISL_12862705, EPI_ISL_12871249, EPI_ISL_12896994, EPI_ISL_12903760, EPI_ISL_12906172, EPI_ISL_12911895, EPI_ISL_12911898, EPI_ISL_12914019, EPI_ISL_12926555, EPI_ISL_12926955, EPI_ISL_12932770, EPI_ISL_12953136, EPI_ISL_12953419, EPI_ISL_12955795, EPI_ISL_12958668, EPI_ISL_12961699, EPI_ISL_12980420, EPI_ISL_12993020, EPI_ISL_12995035, EPI_ISL_12995230, EPI_ISL_12995281, EPI_ISL_12995422, EPI_ISL_13000815, EPI_ISL_13001148, EPI_ISL_13011225, EPI_ISL_13011226, EPI_ISL_13019919, EPI_ISL_13026163, EPI_ISL_13028133, EPI_ISL_13040401, EPI_ISL_13044133, EPI_ISL_13047387, EPI_ISL_13050078, EPI_ISL_13050270, EPI_ISL_13050793, EPI_ISL_13051740, EPI_ISL_13051790, EPI_ISL_13051970, EPI_ISL_13052096, EPI_ISL_13052204, EPI_ISL_13055324, EPI_ISL_13055618, EPI_ISL_13056142, EPI_ISL_13056190, EPI_ISL_13065554, EPI_ISL_13066396, EPI_ISL_13066472, EPI_ISL_13066603, EPI_ISL_13085784, EPI_ISL_13085991, EPI_ISL_13086417, EPI_ISL_13086737, EPI_ISL_13086826, EPI_ISL_13086831, EPI_ISL_13088769, EPI_ISL_13088942, EPI_ISL_13089020, EPI_ISL_13089384, EPI_ISL_13091912, EPI_ISL_13091925, EPI_ISL_13091929, EPI_ISL_13092725, EPI_ISL_13093369, EPI_ISL_13093922, EPI_ISL_13094368, EPI_ISL_13108591, EPI_ISL_13110029, EPI_ISL_13110031, EPI_ISL_13123133, EPI_ISL_13129353, EPI_ISL_13129387, EPI_ISL_13131091, EPI_ISL_13131118, EPI_ISL_13132070, EPI_ISL_13133128, EPI_ISL_13133359, EPI_ISL_13152570, EPI_ISL_13155404, EPI_ISL_13157537, EPI_ISL_13157638, EPI_ISL_13158312, EPI_ISL_13158314, EPI_ISL_13160040, EPI_ISL_13166402, EPI_ISL_13166803, EPI_ISL_13169047, EPI_ISL_13169050, EPI_ISL_13172328, EPI_ISL_13172329, EPI_ISL_13176279, EPI_ISL_13176281, EPI_ISL_13178754, EPI_ISL_13183984, EPI_ISL_13192066, EPI_ISL_13192072, EPI_ISL_13192202, EPI_ISL_13199746, EPI_ISL_13199759, EPI_ISL_13202578, EPI_ISL_13203062, EPI_ISL_13210987, EPI_ISL_13211404, EPI_ISL_13214299, EPI_ISL_13215742, EPI_ISL_13230467, EPI_ISL_13231553, EPI_ISL_13232492, EPI_ISL_13232499, EPI_ISL_13242111, EPI_ISL_13242155, EPI_ISL_13244164, EPI_ISL_13244243, EPI_ISL_13244707, EPI_ISL_13251406, EPI_ISL_13251514, EPI_ISL_13253132, EPI_ISL_13253164, EPI_ISL_13253244, EPI_ISL_13253296, EPI_ISL_13253344, EPI_ISL_13253416, EPI_ISL_13253427, EPI_ISL_13253489, EPI_ISL_13253493, EPI_ISL_13253550, EPI_ISL_13253551, EPI_ISL_13253570, EPI_ISL_13264387, EPI_ISL_13271922, EPI_ISL_13272001, EPI_ISL_13272223, EPI_ISL_13273550, EPI_ISL_13284168, EPI_ISL_13288721, EPI_ISL_13289213, EPI_ISL_13289619, EPI_ISL_13289774, EPI_ISL_13294595, EPI_ISL_13299119, EPI_ISL_13304429, EPI_ISL_13304451, EPI_ISL_13304482, EPI_ISL_13306995, EPI_ISL_13307081, EPI_ISL_13312837, EPI_ISL_13314960, EPI_ISL_13319769, EPI_ISL_13322028, EPI_ISL_13322954, EPI_ISL_13322975, EPI_ISL_13328732, EPI_ISL_13332433, EPI_ISL_13332459, EPI_ISL_13332921, EPI_ISL_13333047, EPI_ISL_13338249, EPI_ISL_13339980, EPI_ISL_13345908, EPI_ISL_13349432, EPI_ISL_13350581, EPI_ISL_13354243, EPI_ISL_13356160, EPI_ISL_13356194, EPI_ISL_13357233, EPI_ISL_13358809, EPI_ISL_13358893, EPI_ISL_13358894, EPI_ISL_13358962, EPI_ISL_13361313, EPI_ISL_13361419, EPI_ISL_13362130, EPI_ISL_13368501, EPI_ISL_13368552, EPI_ISL_13369326, EPI_ISL_13376289, EPI_ISL_13385215, EPI_ISL_13386427, EPI_ISL_13388894, EPI_ISL_13389863, EPI_ISL_13389864, EPI_ISL_13394010, EPI_ISL_13398372, EPI_ISL_13398391, EPI_ISL_13400530, EPI_ISL_13405240, EPI_ISL_13406133, EPI_ISL_13407391, EPI_ISL_13407802, EPI_ISL_13408380, EPI_ISL_13410054, EPI_ISL_13410128, EPI_ISL_13412509, EPI_ISL_13417422, EPI_ISL_13417595, EPI_ISL_13417637, EPI_ISL_13422063, EPI_ISL_13425805, EPI_ISL_13426860, EPI_ISL_13426861, EPI_ISL_13426862, EPI_ISL_13440246, EPI_ISL_13440269, EPI_ISL_13440488, EPI_ISL_13443257, EPI_ISL_13454711, EPI_ISL_13455972, EPI_ISL_13463270, EPI_ISL_13464004, EPI_ISL_13465109, EPI_ISL_13466588, EPI_ISL_13466629, EPI_ISL_13466644, EPI_ISL_13467321, EPI_ISL_13467676, EPI_ISL_13470130, EPI_ISL_13470158, EPI_ISL_13476507, EPI_ISL_13477158, EPI_ISL_13478276, EPI_ISL_13478421, EPI_ISL_13478425, EPI_ISL_13478448, EPI_ISL_13478487, EPI_ISL_13480851, EPI_ISL_13481704, EPI_ISL_13482848, EPI_ISL_13483067, EPI_ISL_13483538, EPI_ISL_13483605, EPI_ISL_13483653, EPI_ISL_13489191, EPI_ISL_13501768, EPI_ISL_13502856, EPI_ISL_13502894, EPI_ISL_13503354, EPI_ISL_13503413, EPI_ISL_13504084, EPI_ISL_13504103, EPI_ISL_13504568, EPI_ISL_13518485, EPI_ISL_13519541, EPI_ISL_13522554, EPI_ISL_13522707, EPI_ISL_13529086, EPI_ISL_13534848, EPI_ISL_13535844, EPI_ISL_13536846, EPI_ISL_13536992, EPI_ISL_13538620, EPI_ISL_13538621, EPI_ISL_13539746, EPI_ISL_13552269, EPI_ISL_13560139, EPI_ISL_13563849, EPI_ISL_13563900, EPI_ISL_13564453, EPI_ISL_13564901, EPI_ISL_13566717, EPI_ISL_13571629, EPI_ISL_13572579, EPI_ISL_13572829, EPI_ISL_13573543, EPI_ISL_13573707, EPI_ISL_13578717, EPI_ISL_13585459, EPI_ISL_13585850, EPI_ISL_13592668, EPI_ISL_13605930, EPI_ISL_13608346, EPI_ISL_13611346, EPI_ISL_13611373, EPI_ISL_13612631, EPI_ISL_13612636, EPI_ISL_13614328, EPI_ISL_13615804, EPI_ISL_13615826, EPI_ISL_13616458, EPI_ISL_13617390, EPI_ISL_13617475, EPI_ISL_13617493, EPI_ISL_13619646, EPI_ISL_13622657, EPI_ISL_13622831, EPI_ISL_13622874, EPI_ISL_13624440, EPI_ISL_13624441, EPI_ISL_13626226, EPI_ISL_13633558, EPI_ISL_13633729, EPI_ISL_13636934, EPI_ISL_13637141, EPI_ISL_13637734, EPI_ISL_13638494, EPI_ISL_13642059, EPI_ISL_13653660, EPI_ISL_13665096, EPI_ISL_13665110, EPI_ISL_13666764, EPI_ISL_13677872, EPI_ISL_13688333, EPI_ISL_13691966, EPI_ISL_13692397, EPI_ISL_13694663, EPI_ISL_13696042, EPI_ISL_13698648, EPI_ISL_13700128, EPI_ISL_13700243, EPI_ISL_13700756, EPI_ISL_13701029, EPI_ISL_13701810, EPI_ISL_13710357, EPI_ISL_13715129, EPI_ISL_13715746, EPI_ISL_13716400, EPI_ISL_13716943, EPI_ISL_13723111, EPI_ISL_13723805, EPI_ISL_13726415, EPI_ISL_13726482, EPI_ISL_13729045, EPI_ISL_13734474, EPI_ISL_13734683, EPI_ISL_13738059, EPI_ISL_13740111, EPI_ISL_13740674, EPI_ISL_13741330, EPI_ISL_13744798, EPI_ISL_13744799, EPI_ISL_13745638, EPI_ISL_13745641, EPI_ISL_13748166, EPI_ISL_13750726, EPI_ISL_13750730, EPI_ISL_13750760, EPI_ISL_13750771, EPI_ISL_13750936, EPI_ISL_13750937, EPI_ISL_13751254, EPI_ISL_13755996, EPI_ISL_13757795, EPI_ISL_13757902, EPI_ISL_13757914, EPI_ISL_13759674, EPI_ISL_13759811, EPI_ISL_13760929, EPI_ISL_13762803, EPI_ISL_13764852, EPI_ISL_13765234, EPI_ISL_13769441, EPI_ISL_13774980, EPI_ISL_13776103, EPI_ISL_13776118, EPI_ISL_13780813, EPI_ISL_13788917, EPI_ISL_13794761, EPI_ISL_13795106, EPI_ISL_13795308, EPI_ISL_13802466, EPI_ISL_13805350, EPI_ISL_13806026, EPI_ISL_13806197, EPI_ISL_13810913, EPI_ISL_13812067, EPI_ISL_13824458, EPI_ISL_13826362, EPI_ISL_13830194, EPI_ISL_13830195, EPI_ISL_13830196, EPI_ISL_13830197, EPI_ISL_13830445, EPI_ISL_13830454, EPI_ISL_13839105, EPI_ISL_13839285, EPI_ISL_13842068, EPI_ISL_13844161, EPI_ISL_13850726, EPI_ISL_13855446, EPI_ISL_13856866, EPI_ISL_13858143, EPI_ISL_13858664, EPI_ISL_13860426, EPI_ISL_13860879, EPI_ISL_13866687, EPI_ISL_13866688, EPI_ISL_13866691, EPI_ISL_13867130, EPI_ISL_13867135, EPI_ISL_13871326, EPI_ISL_13873100, EPI_ISL_13875348, EPI_ISL_13875677, EPI_ISL_13876290, EPI_ISL_13876318, EPI_ISL_13876612, EPI_ISL_13876760, EPI_ISL_13881123, EPI_ISL_13884353, EPI_ISL_13884360, EPI_ISL_13884439, EPI_ISL_13889482, EPI_ISL_13891697, EPI_ISL_13896135, EPI_ISL_13896156, EPI_ISL_13896578, EPI_ISL_13900930, EPI_ISL_13907795, EPI_ISL_13907925, EPI_ISL_13908668, EPI_ISL_13913047, EPI_ISL_13915527, EPI_ISL_13915530, EPI_ISL_13931117, EPI_ISL_13931975, EPI_ISL_13931997, EPI_ISL_13932039, EPI_ISL_13932080, EPI_ISL_13937613, EPI_ISL_13939223, EPI_ISL_13947566, EPI_ISL_13948007, EPI_ISL_13951654, EPI_ISL_13957684, EPI_ISL_13958481, EPI_ISL_13958566, EPI_ISL_13963277, EPI_ISL_13963776, EPI_ISL_13963832, EPI_ISL_13967204, EPI_ISL_13968198, EPI_ISL_13970237, EPI_ISL_13970242, EPI_ISL_13970249, EPI_ISL_13970257, EPI_ISL_13970276, EPI_ISL_13975822, EPI_ISL_13981101, EPI_ISL_13984460, EPI_ISL_13986494, EPI_ISL_13986497, EPI_ISL_13986498, EPI_ISL_13986501, EPI_ISL_13989484, EPI_ISL_13991375, EPI_ISL_13994781, EPI_ISL_14000155, EPI_ISL_14005794, EPI_ISL_14011475, EPI_ISL_14015047, EPI_ISL_14019093, EPI_ISL_14019109, EPI_ISL_14019330, EPI_ISL_14020697, EPI_ISL_14022780, EPI_ISL_14022892, EPI_ISL_14023662, EPI_ISL_14027304, EPI_ISL_14027788, EPI_ISL_14028215, EPI_ISL_14029314, EPI_ISL_14029803, EPI_ISL_14030175, EPI_ISL_14032717, EPI_ISL_14035970, EPI_ISL_14036069, EPI_ISL_14044698, EPI_ISL_14044704, EPI_ISL_14046291, EPI_ISL_14047361, EPI_ISL_14051041, EPI_ISL_14051049, EPI_ISL_14051052, EPI_ISL_14064598, EPI_ISL_14064601, EPI_ISL_14066591, EPI_ISL_14066852, EPI_ISL_14071587, EPI_ISL_14071795, EPI_ISL_14097542, EPI_ISL_14124074, EPI_ISL_14127563, EPI_ISL_14134678, EPI_ISL_14147202, EPI_ISL_14155218, EPI_ISL_14158264, EPI_ISL_14161024, EPI_ISL_14170603, EPI_ISL_14172905, EPI_ISL_14173767, EPI_ISL_14175092, EPI_ISL_14175097, EPI_ISL_14175182, EPI_ISL_14180178, EPI_ISL_14181740, EPI_ISL_14193000, EPI_ISL_14193833, EPI_ISL_14196068, EPI_ISL_14196701, EPI_ISL_14197724, EPI_ISL_14198080, EPI_ISL_14200342, EPI_ISL_14200801, EPI_ISL_14203206, EPI_ISL_14203613, EPI_ISL_14208835, EPI_ISL_14209372, EPI_ISL_14209934, EPI_ISL_14211976, EPI_ISL_14215014, EPI_ISL_14215818, EPI_ISL_14216595, EPI_ISL_14217745, EPI_ISL_14222817, EPI_ISL_14223595, EPI_ISL_14223736, EPI_ISL_14223750, EPI_ISL_14224871, EPI_ISL_14226883, EPI_ISL_14228030, EPI_ISL_14229584, EPI_ISL_14230429, EPI_ISL_14230544, EPI_ISL_14231739, EPI_ISL_14231749, EPI_ISL_14231751, EPI_ISL_14232221, EPI_ISL_14236377, EPI_ISL_14241722, EPI_ISL_14243471, EPI_ISL_14243503, EPI_ISL_14249979, EPI_ISL_14257905, EPI_ISL_14259114, EPI_ISL_14259141, EPI_ISL_14259905, EPI_ISL_14260215, EPI_ISL_14261704, EPI_ISL_14263077, EPI_ISL_14277057, EPI_ISL_14285562, EPI_ISL_14287370, EPI_ISL_14289901, EPI_ISL_14292615, EPI_ISL_14292645, EPI_ISL_14292727, EPI_ISL_14292796, EPI_ISL_14296586, EPI_ISL_14298637, EPI_ISL_14299991, EPI_ISL_14301376, EPI_ISL_14311909, EPI_ISL_14311965, EPI_ISL_14312743, EPI_ISL_14321789, EPI_ISL_14321842, EPI_ISL_14329477, EPI_ISL_14353536, EPI_ISL_14356738, EPI_ISL_14359010, EPI_ISL_14377405, EPI_ISL_14377421, EPI_ISL_14382623, EPI_ISL_14386850, EPI_ISL_14387989, EPI_ISL_14389796, EPI_ISL_14391372, EPI_ISL_14393120, EPI_ISL_14416474, EPI_ISL_14417374, EPI_ISL_14425116, EPI_ISL_14425894, EPI_ISL_14426235, EPI_ISL_14426336, EPI_ISL_14430592, EPI_ISL_14433737, EPI_ISL_14434989, EPI_ISL_14436225, EPI_ISL_14437098, EPI_ISL_14439513, EPI_ISL_14439514, EPI_ISL_14439530, EPI_ISL_14439649, EPI_ISL_14439686, EPI_ISL_14448667, EPI_ISL_14455168, EPI_ISL_14459779, EPI_ISL_14462783, EPI_ISL_14464386, EPI_ISL_14467169, EPI_ISL_14469810, EPI_ISL_14471721, EPI_ISL_14478208, EPI_ISL_14479146, EPI_ISL_14479735, EPI_ISL_14483275, EPI_ISL_14485183, EPI_ISL_14485890, EPI_ISL_14487304, EPI_ISL_14487315, EPI_ISL_14493139, EPI_ISL_14493608, EPI_ISL_14493822, EPI_ISL_14493989, EPI_ISL_14496407, EPI_ISL_14496491, EPI_ISL_14497316, EPI_ISL_14498244, EPI_ISL_14502420, EPI_ISL_14503169, EPI_ISL_14503437, EPI_ISL_14504973, EPI_ISL_14505974, EPI_ISL_14507199, EPI_ISL_14507200, EPI_ISL_14508231, EPI_ISL_14509715, EPI_ISL_14513137, EPI_ISL_14518038, EPI_ISL_14518039, EPI_ISL_14518040, EPI_ISL_14518101, EPI_ISL_14518137, EPI_ISL_14527351, EPI_ISL_14535112, EPI_ISL_14540192, EPI_ISL_14544667, EPI_ISL_14545270, EPI_ISL_14551066, EPI_ISL_14556650, EPI_ISL_14560721, EPI_ISL_14561487, EPI_ISL_14562820, EPI_ISL_14571645, EPI_ISL_14572777, EPI_ISL_14573093, EPI_ISL_14577981, EPI_ISL_14578599, EPI_ISL_14583728, EPI_ISL_14596883, EPI_ISL_14599772, EPI_ISL_14602583, EPI_ISL_14602992, EPI_ISL_14606016, EPI_ISL_14610722, EPI_ISL_14610723, EPI_ISL_14610724, EPI_ISL_14610725, EPI_ISL_14610726, EPI_ISL_14610727, EPI_ISL_14610728, EPI_ISL_14610729, EPI_ISL_14610730, EPI_ISL_14610731, EPI_ISL_14610732, EPI_ISL_14610733, EPI_ISL_14610734, EPI_ISL_14613632, EPI_ISL_14613671, EPI_ISL_14616144, EPI_ISL_14616681, EPI_ISL_14618772, EPI_ISL_14619952, EPI_ISL_14623599, EPI_ISL_14624407, EPI_ISL_14625263, EPI_ISL_14647032, EPI_ISL_14650043, EPI_ISL_14652006, EPI_ISL_14665394, EPI_ISL_14666760, EPI_ISL_14667656, EPI_ISL_14667834, EPI_ISL_14669160, EPI_ISL_14676287, EPI_ISL_14681429, EPI_ISL_14683500, EPI_ISL_14687471, EPI_ISL_14691921, EPI_ISL_14694460, EPI_ISL_14699501, EPI_ISL_14700183, EPI_ISL_14700285, EPI_ISL_14701161, EPI_ISL_14706169, EPI_ISL_14707196, EPI_ISL_14707197, EPI_ISL_14710821, EPI_ISL_14710834, EPI_ISL_14711613, EPI_ISL_14711614, EPI_ISL_14715522, EPI_ISL_14721837, EPI_ISL_14721894, EPI_ISL_14722630, EPI_ISL_14725600, EPI_ISL_14727457, EPI_ISL_14728608, EPI_ISL_14728814, EPI_ISL_14732990, EPI_ISL_14744620, EPI_ISL_14744804, EPI_ISL_14744809, EPI_ISL_14745146, EPI_ISL_14746124, EPI_ISL_14746196, EPI_ISL_14746271, EPI_ISL_14747246, EPI_ISL_14747247, EPI_ISL_14747621, EPI_ISL_14752384, EPI_ISL_14754570, EPI_ISL_14755727, EPI_ISL_14755766, EPI_ISL_14755933, EPI_ISL_14763711, EPI_ISL_14763898, EPI_ISL_14766331, EPI_ISL_14766361, EPI_ISL_14766363, EPI_ISL_14770484, EPI_ISL_14771903, EPI_ISL_14772260, EPI_ISL_14773203, EPI_ISL_14773569, EPI_ISL_14773570, EPI_ISL_14778614, EPI_ISL_14785887, EPI_ISL_14786677, EPI_ISL_14788048, EPI_ISL_14788099, EPI_ISL_14788860, EPI_ISL_14789391, EPI_ISL_14789392, EPI_ISL_14789508, EPI_ISL_14791420, EPI_ISL_14793146, EPI_ISL_14793618, EPI_ISL_14806018, EPI_ISL_14806413, EPI_ISL_14809350, EPI_ISL_14811078, EPI_ISL_14812412, EPI_ISL_14813068, EPI_ISL_14813161, EPI_ISL_14813215, EPI_ISL_14813300, EPI_ISL_14813995, EPI_ISL_14816346, EPI_ISL_14817985, EPI_ISL_14832977, EPI_ISL_14834854, EPI_ISL_14837867, EPI_ISL_14838049, EPI_ISL_14841625, EPI_ISL_14845057, EPI_ISL_14847727, EPI_ISL_14856139, EPI_ISL_14859457, EPI_ISL_14859716, EPI_ISL_14862263, EPI_ISL_14886333, EPI_ISL_14888736, EPI_ISL_14890020, EPI_ISL_14891391, EPI_ISL_14891763, EPI_ISL_14891765, EPI_ISL_14892114, EPI_ISL_14892395, EPI_ISL_14892970, EPI_ISL_14901195, EPI_ISL_14901198, EPI_ISL_14901422, EPI_ISL_14901439, EPI_ISL_14901444, EPI_ISL_14901672, EPI_ISL_14903212, EPI_ISL_14904331, EPI_ISL_14912863, EPI_ISL_14913043, EPI_ISL_14916417, EPI_ISL_14917292, EPI_ISL_14919989, EPI_ISL_14920419, EPI_ISL_14921805, EPI_ISL_14922117, EPI_ISL_14922327, EPI_ISL_14922954, EPI_ISL_14924448, EPI_ISL_14925471, EPI_ISL_14925480, EPI_ISL_14925487, EPI_ISL_14929774, EPI_ISL_14931103, EPI_ISL_14934229, EPI_ISL_14934234, EPI_ISL_14935361, EPI_ISL_14935895, EPI_ISL_14935908, EPI_ISL_14935930, EPI_ISL_14935931, EPI_ISL_14937654, EPI_ISL_14937864, EPI_ISL_14942184, EPI_ISL_14942530, EPI_ISL_14943290, EPI_ISL_14945818, EPI_ISL_14946958, EPI_ISL_14949065, EPI_ISL_14950282, EPI_ISL_14950545, EPI_ISL_14951595, EPI_ISL_14951609, EPI_ISL_14951892, EPI_ISL_14952059, EPI_ISL_14952220, EPI_ISL_14953487, EPI_ISL_14960752, EPI_ISL_14960911, EPI_ISL_14961972, EPI_ISL_14962212, EPI_ISL_14962429, EPI_ISL_14962617, EPI_ISL_14975894, EPI_ISL_14980656, EPI_ISL_14992324, EPI_ISL_14993023, EPI_ISL_14995958, EPI_ISL_14997830, EPI_ISL_15005362, EPI_ISL_15005560, EPI_ISL_15010697, EPI_ISL_15012784, EPI_ISL_15013151, EPI_ISL_15013344, EPI_ISL_15014035, EPI_ISL_15014516, EPI_ISL_15015307, EPI_ISL_15017244, EPI_ISL_15018502, EPI_ISL_15020427, EPI_ISL_15022783, EPI_ISL_15022788, EPI_ISL_15024848, EPI_ISL_15026124, EPI_ISL_15030291, EPI_ISL_15030370, EPI_ISL_15032101, EPI_ISL_15036387, EPI_ISL_15038137, EPI_ISL_15040463, EPI_ISL_15040845, EPI_ISL_15040855, EPI_ISL_15040867, EPI_ISL_15044102, EPI_ISL_15048524, EPI_ISL_15050379, EPI_ISL_15051633, EPI_ISL_15058728, EPI_ISL_15058903, EPI_ISL_15071928, EPI_ISL_15072261, EPI_ISL_15072543, EPI_ISL_15072550, EPI_ISL_15072553, EPI_ISL_15072554, EPI_ISL_15072999, EPI_ISL_15075043, EPI_ISL_15075836, EPI_ISL_15076071, EPI_ISL_15077422, EPI_ISL_15078481, EPI_ISL_15080832, EPI_ISL_15084091, EPI_ISL_15085357, EPI_ISL_15085883, EPI_ISL_15085910, EPI_ISL_15086100, EPI_ISL_15086132, EPI_ISL_15086246, EPI_ISL_15088435, EPI_ISL_15088854, EPI_ISL_15090098, EPI_ISL_15093244, EPI_ISL_15093817, EPI_ISL_15093818, EPI_ISL_15094085, EPI_ISL_15096672, EPI_ISL_15098367, EPI_ISL_15101602, EPI_ISL_15107059, EPI_ISL_15107248, EPI_ISL_15107529, EPI_ISL_15108940, EPI_ISL_15108982, EPI_ISL_15109913, EPI_ISL_15111016, EPI_ISL_15111369, EPI_ISL_15114528, EPI_ISL_15114696, EPI_ISL_15116712, EPI_ISL_15118484, EPI_ISL_15119416, EPI_ISL_15120575, EPI_ISL_15125352, EPI_ISL_15126616, EPI_ISL_15129252, EPI_ISL_15137908, EPI_ISL_15137948, EPI_ISL_15140027, EPI_ISL_15140068, EPI_ISL_15145892, EPI_ISL_15145981, EPI_ISL_15157666, EPI_ISL_15159584, EPI_ISL_15160596, EPI_ISL_15161674, EPI_ISL_15169791, EPI_ISL_15170512, EPI_ISL_15172949, EPI_ISL_15173621, EPI_ISL_15175083, EPI_ISL_15175896, EPI_ISL_15177304, EPI_ISL_15177330, EPI_ISL_15177334, EPI_ISL_15178067, EPI_ISL_15178215, EPI_ISL_15184076, EPI_ISL_15184330, EPI_ISL_15191490, EPI_ISL_15191491, EPI_ISL_15191642, EPI_ISL_15191714, EPI_ISL_15191804, EPI_ISL_15193406, EPI_ISL_15195634, EPI_ISL_15195645, EPI_ISL_15198987, EPI_ISL_15211295, EPI_ISL_15211305, EPI_ISL_15213088, EPI_ISL_15215446, EPI_ISL_15216639, EPI_ISL_15218165, EPI_ISL_15222709, EPI_ISL_15229199, EPI_ISL_15231108, EPI_ISL_15236061, EPI_ISL_15236355, EPI_ISL_15241555, EPI_ISL_15248681, EPI_ISL_15250551, EPI_ISL_15251240, EPI_ISL_15251241, EPI_ISL_15251242, EPI_ISL_15251243, EPI_ISL_15257404, EPI_ISL_15264055, EPI_ISL_15268715, EPI_ISL_15268834, EPI_ISL_15271184, EPI_ISL_15273578, EPI_ISL_15275240, EPI_ISL_15278730, EPI_ISL_15278787, EPI_ISL_15279743, EPI_ISL_15284364, EPI_ISL_15284373, EPI_ISL_15284586, EPI_ISL_15286527, EPI_ISL_15287393, EPI_ISL_15288333, EPI_ISL_15292331, EPI_ISL_15294656, EPI_ISL_15296403, EPI_ISL_15305862, EPI_ISL_15306067, EPI_ISL_15307010, EPI_ISL_15307651, EPI_ISL_15310561, EPI_ISL_15312119, EPI_ISL_15314949, EPI_ISL_15316398, EPI_ISL_15317884, EPI_ISL_15325687, EPI_ISL_15325946, EPI_ISL_15328668, EPI_ISL_15330077, EPI_ISL_15330418, EPI_ISL_15331994, EPI_ISL_15332094, EPI_ISL_15332912, EPI_ISL_15333310, EPI_ISL_15338015, EPI_ISL_15338081, EPI_ISL_15340355, EPI_ISL_15341321, EPI_ISL_15347054, EPI_ISL_15348926, EPI_ISL_15348927, EPI_ISL_15348928, EPI_ISL_15348929, EPI_ISL_15352149, EPI_ISL_15354679, EPI_ISL_15354775, EPI_ISL_15357057, EPI_ISL_15357237, EPI_ISL_15357960, EPI_ISL_15357988, EPI_ISL_15362650, EPI_ISL_15363544, EPI_ISL_15368893, EPI_ISL_15370137, EPI_ISL_15370885, EPI_ISL_15370889, EPI_ISL_15376124, EPI_ISL_15376348, EPI_ISL_15376376, EPI_ISL_15379719, EPI_ISL_15380518, EPI_ISL_15384507, EPI_ISL_15385232, EPI_ISL_15385971, EPI_ISL_15387248, EPI_ISL_15387296, EPI_ISL_15387686, EPI_ISL_15388463, EPI_ISL_15389278, EPI_ISL_15392529, EPI_ISL_15393294, EPI_ISL_15393302, EPI_ISL_15397576, EPI_ISL_15403865, EPI_ISL_15403898, EPI_ISL_15408226, EPI_ISL_15408697, EPI_ISL_15409673, EPI_ISL_15415649, EPI_ISL_15416349, EPI_ISL_15420040, EPI_ISL_15420212, EPI_ISL_15420431, EPI_ISL_15420631, EPI_ISL_15423234, EPI_ISL_15424211, EPI_ISL_15424884, EPI_ISL_15434634, EPI_ISL_15435185, EPI_ISL_15436140, EPI_ISL_15436494, EPI_ISL_15436498, EPI_ISL_15436499, EPI_ISL_15442625, EPI_ISL_15442735, EPI_ISL_15446553, EPI_ISL_15456143, EPI_ISL_15462878, EPI_ISL_15471419, EPI_ISL_15471420, EPI_ISL_15472394, EPI_ISL_15472759, EPI_ISL_15476105, EPI_ISL_15476158, EPI_ISL_15476180, EPI_ISL_15476724, EPI_ISL_15479591, EPI_ISL_15481002, EPI_ISL_15486348, EPI_ISL_15490572, EPI_ISL_15492743, EPI_ISL_15492887, EPI_ISL_15494260, EPI_ISL_15494897, EPI_ISL_15495028, EPI_ISL_15496641, EPI_ISL_15505215, EPI_ISL_15505985, EPI_ISL_15506333, EPI_ISL_15507204, EPI_ISL_15507296, EPI_ISL_15507616, EPI_ISL_15508143, EPI_ISL_15509746, EPI_ISL_15509755, EPI_ISL_15511119, EPI_ISL_15511841, EPI_ISL_15511842, EPI_ISL_15511843, EPI_ISL_15513583, EPI_ISL_15513588, EPI_ISL_15513663, EPI_ISL_15514216, EPI_ISL_15514265, EPI_ISL_15514302, EPI_ISL_15514923, EPI_ISL_15523458, EPI_ISL_15523591, EPI_ISL_15528152, EPI_ISL_15528174, EPI_ISL_15528328, EPI_ISL_15528329, EPI_ISL_15528330, EPI_ISL_15528331, EPI_ISL_15528332, EPI_ISL_15528333, EPI_ISL_15528334, EPI_ISL_15535800, EPI_ISL_15537619, EPI_ISL_15538513, EPI_ISL_15538645, EPI_ISL_15542503, EPI_ISL_15546790, EPI_ISL_15549778, EPI_ISL_15549981, EPI_ISL_15550525, EPI_ISL_15579728, EPI_ISL_15579786, EPI_ISL_15580359, EPI_ISL_15580699, EPI_ISL_15581446, EPI_ISL_15581681, EPI_ISL_15581931, EPI_ISL_15581932, EPI_ISL_15581939, EPI_ISL_15582076, EPI_ISL_15582517, EPI_ISL_15583376, EPI_ISL_15585338, EPI_ISL_15587950, EPI_ISL_15588132, EPI_ISL_15594682, EPI_ISL_15595518, EPI_ISL_15598104, EPI_ISL_15598966, EPI_ISL_15602198, EPI_ISL_15604595, EPI_ISL_15606951, EPI_ISL_15607872, EPI_ISL_15608835, EPI_ISL_15609106, EPI_ISL_15609107, EPI_ISL_15610881, EPI_ISL_15612047, EPI_ISL_15612048, EPI_ISL_15614383, EPI_ISL_15614490, EPI_ISL_15616889, EPI_ISL_15617621, EPI_ISL_15617635, EPI_ISL_15619675, EPI_ISL_15626705, EPI_ISL_15628252, EPI_ISL_15630041, EPI_ISL_15631537, EPI_ISL_15635022, EPI_ISL_15637121, EPI_ISL_15639067, EPI_ISL_15642936, EPI_ISL_15642980, EPI_ISL_15649157, EPI_ISL_15650076, EPI_ISL_15650225, EPI_ISL_15653695, EPI_ISL_15654640, EPI_ISL_15656922, EPI_ISL_15659847, EPI_ISL_15661609, EPI_ISL_15666595, EPI_ISL_15667047, EPI_ISL_15669004, EPI_ISL_15671244, EPI_ISL_15671388, EPI_ISL_15671577, EPI_ISL_15671878, EPI_ISL_15671888, EPI_ISL_15673934, EPI_ISL_15675248, EPI_ISL_15678339, EPI_ISL_15685722, EPI_ISL_15685724, EPI_ISL_15685810, EPI_ISL_15687681, EPI_ISL_15687965, EPI_ISL_15688500, EPI_ISL_15692625, EPI_ISL_15693169, EPI_ISL_15693174, EPI_ISL_15693676, EPI_ISL_15700160, EPI_ISL_15703834, EPI_ISL_15705061, EPI_ISL_15712450, EPI_ISL_15715244, EPI_ISL_15719141, EPI_ISL_15719142, EPI_ISL_15719143, EPI_ISL_15720605, EPI_ISL_15721137, EPI_ISL_15721185, EPI_ISL_15721190, EPI_ISL_15723589, EPI_ISL_15725799, EPI_ISL_15728467, EPI_ISL_15728673, EPI_ISL_15729287, EPI_ISL_15729288, EPI_ISL_15729308, EPI_ISL_15729309, EPI_ISL_15729310, EPI_ISL_15729311, EPI_ISL_15729315, EPI_ISL_15729341, EPI_ISL_15729358, EPI_ISL_15731233, EPI_ISL_15731409, EPI_ISL_15732413, EPI_ISL_15736424, EPI_ISL_15739498, EPI_ISL_15739617, EPI_ISL_15741410, EPI_ISL_15743318, EPI_ISL_15743816, EPI_ISL_15749060, EPI_ISL_15754145, EPI_ISL_15754794, EPI_ISL_15758565, EPI_ISL_15758745, EPI_ISL_15760224, EPI_ISL_15760382, EPI_ISL_15760554, EPI_ISL_15760812, EPI_ISL_15761520, EPI_ISL_15761543, EPI_ISL_15761663, EPI_ISL_15763216, EPI_ISL_15765022, EPI_ISL_15768827, EPI_ISL_15776989, EPI_ISL_15778527, EPI_ISL_15779724, EPI_ISL_15780387, EPI_ISL_15781197, EPI_ISL_15781220, EPI_ISL_15781937, EPI_ISL_15782049, EPI_ISL_15785782, EPI_ISL_15786114, EPI_ISL_15786255, EPI_ISL_15790657, EPI_ISL_15791223, EPI_ISL_15791252, EPI_ISL_15792351, EPI_ISL_15793981, EPI_ISL_15797751, EPI_ISL_15798331, EPI_ISL_15801425, EPI_ISL_15801499, EPI_ISL_15801515, EPI_ISL_15803807, EPI_ISL_15815337, EPI_ISL_15815525, EPI_ISL_15815889, EPI_ISL_15818486, EPI_ISL_15820055, EPI_ISL_15822919, EPI_ISL_15824080, EPI_ISL_15824099, EPI_ISL_15824207, EPI_ISL_15825627, EPI_ISL_15826800, EPI_ISL_15826867, EPI_ISL_15829108, EPI_ISL_15837751, EPI_ISL_15837827, EPI_ISL_15838124, EPI_ISL_15839941, EPI_ISL_15843473, EPI_ISL_15843671, EPI_ISL_15844032, EPI_ISL_15844165, EPI_ISL_15845753, EPI_ISL_15845778, EPI_ISL_15845946, EPI_ISL_15846023, EPI_ISL_15846264, EPI_ISL_15849690, EPI_ISL_15850759, EPI_ISL_15850865, EPI_ISL_15850872, EPI_ISL_15853809, EPI_ISL_15853943, EPI_ISL_15856103, EPI_ISL_15856463, EPI_ISL_15856822, EPI_ISL_15857468, EPI_ISL_15860163, EPI_ISL_15864217, EPI_ISL_15864218, EPI_ISL_15865257, EPI_ISL_15865301, EPI_ISL_15865421, EPI_ISL_15865482, EPI_ISL_15866887, EPI_ISL_15873398, EPI_ISL_15874567, EPI_ISL_15878818, EPI_ISL_15883009, EPI_ISL_15883480, EPI_ISL_15887656, EPI_ISL_15894376, EPI_ISL_15895625, EPI_ISL_15896804, EPI_ISL_15896845, EPI_ISL_15896923, EPI_ISL_15897067, EPI_ISL_15897092, EPI_ISL_15898992, EPI_ISL_15900796, EPI_ISL_15905452, EPI_ISL_15905511, EPI_ISL_15911160, EPI_ISL_15912221, EPI_ISL_15912222, EPI_ISL_15912223, EPI_ISL_15912224, EPI_ISL_15914119, EPI_ISL_15917272, EPI_ISL_15920181, EPI_ISL_15920505, EPI_ISL_15920753, EPI_ISL_15920754, EPI_ISL_15920755, EPI_ISL_15926083, EPI_ISL_15926723, EPI_ISL_15928156, EPI_ISL_15928909, EPI_ISL_15929151, EPI_ISL_15932554, EPI_ISL_15934274, EPI_ISL_15937718, EPI_ISL_15938074, EPI_ISL_15941879, EPI_ISL_15941880, EPI_ISL_15945504, EPI_ISL_15955458, EPI_ISL_15958934, EPI_ISL_15961456, EPI_ISL_15962045, EPI_ISL_15966527, EPI_ISL_15969420, EPI_ISL_15969421, EPI_ISL_15969437, EPI_ISL_15969438, EPI_ISL_15970088, EPI_ISL_15970187, EPI_ISL_15982641, EPI_ISL_15984958, EPI_ISL_15992803, EPI_ISL_15998627, EPI_ISL_16001974, EPI_ISL_16001995, EPI_ISL_16005457, EPI_ISL_16006665, EPI_ISL_16007931, EPI_ISL_16008877, EPI_ISL_16012424, EPI_ISL_16013074, EPI_ISL_16013086, EPI_ISL_16015099, EPI_ISL_16017107, EPI_ISL_16018930, EPI_ISL_16019056, EPI_ISL_16024407, EPI_ISL_16024682, EPI_ISL_16027431, EPI_ISL_16027937, EPI_ISL_16029135, EPI_ISL_16029382, EPI_ISL_16029654, EPI_ISL_16030181, EPI_ISL_16033087, EPI_ISL_16039444, EPI_ISL_16043974, EPI_ISL_16045410, EPI_ISL_16046711, EPI_ISL_16050127, EPI_ISL_16054133, EPI_ISL_16054451, EPI_ISL_16054953, EPI_ISL_16054963, EPI_ISL_16055434, EPI_ISL_16055461, EPI_ISL_16055527, EPI_ISL_16055697, EPI_ISL_16055721, EPI_ISL_16056123, EPI_ISL_16057031, EPI_ISL_16060790, EPI_ISL_16062229, EPI_ISL_16066333, EPI_ISL_16068281, EPI_ISL_16068914, EPI_ISL_16073469, EPI_ISL_16073474, EPI_ISL_16075086, EPI_ISL_16075127, EPI_ISL_16079016, EPI_ISL_16080170, EPI_ISL_16080871, EPI_ISL_16091870, EPI_ISL_16102480, EPI_ISL_16111875, EPI_ISL_16113331, EPI_ISL_16113603, EPI_ISL_16113806, EPI_ISL_16114631, EPI_ISL_16115703, EPI_ISL_16116190, EPI_ISL_16116234, EPI_ISL_16116659, EPI_ISL_16116707, EPI_ISL_16119498, EPI_ISL_16119508, EPI_ISL_16119512, EPI_ISL_16119517, EPI_ISL_16119519, EPI_ISL_16119805, EPI_ISL_16131965, EPI_ISL_16131986, EPI_ISL_16131997, EPI_ISL_16136901, EPI_ISL_16137616, EPI_ISL_16151030, EPI_ISL_16151463, EPI_ISL_16151651, EPI_ISL_16153650, EPI_ISL_16153658, EPI_ISL_16153800, EPI_ISL_16154520, EPI_ISL_16157031, EPI_ISL_16157875, EPI_ISL_16158109, EPI_ISL_16158326, EPI_ISL_16158363, EPI_ISL_16160252, EPI_ISL_16160296, EPI_ISL_16160313, EPI_ISL_16165250, EPI_ISL_16167761, EPI_ISL_16178634, EPI_ISL_16179355, EPI_ISL_16180574, EPI_ISL_16181797, EPI_ISL_16181828, EPI_ISL_16181950, EPI_ISL_16183022, EPI_ISL_16190977, EPI_ISL_16191476, EPI_ISL_16196150, EPI_ISL_16196167, EPI_ISL_16197958, EPI_ISL_16201173, EPI_ISL_16215808, EPI_ISL_16218191, EPI_ISL_16219709, EPI_ISL_16219753, EPI_ISL_16221691, EPI_ISL_16230801, EPI_ISL_16233000, EPI_ISL_16233650, EPI_ISL_16233865, EPI_ISL_16234790, EPI_ISL_16235313, EPI_ISL_16235462, EPI_ISL_16235523, EPI_ISL_16235930, EPI_ISL_16244367, EPI_ISL_16244373, EPI_ISL_16244408, EPI_ISL_16244923, EPI_ISL_16245232, EPI_ISL_16245289, EPI_ISL_16245433, EPI_ISL_16245601, EPI_ISL_16245627, EPI_ISL_16247208, EPI_ISL_16247263, EPI_ISL_16247490, EPI_ISL_16247545, EPI_ISL_16257294, EPI_ISL_16264400, EPI_ISL_16265325, EPI_ISL_16268074, EPI_ISL_16270258, EPI_ISL_16271444, EPI_ISL_16271604, EPI_ISL_16273936, EPI_ISL_16284103, EPI_ISL_16284311, EPI_ISL_16287253, EPI_ISL_16287690, EPI_ISL_16290877, EPI_ISL_16293662, EPI_ISL_16312661, EPI_ISL_16327295, EPI_ISL_16334679, EPI_ISL_16338847, EPI_ISL_16338862, EPI_ISL_16343221, EPI_ISL_16348840, EPI_ISL_16348868, EPI_ISL_16351967, EPI_ISL_16354229, EPI_ISL_16355537, EPI_ISL_16356453, EPI_ISL_16356910, EPI_ISL_16358915, EPI_ISL_16360495, EPI_ISL_16365715, EPI_ISL_16368903, EPI_ISL_16369869, EPI_ISL_16370037, EPI_ISL_16378181, EPI_ISL_16379359, EPI_ISL_16380313, EPI_ISL_16381332, EPI_ISL_16381679, EPI_ISL_16384522, EPI_ISL_16385455, EPI_ISL_16385456, EPI_ISL_16391752, EPI_ISL_16394844, EPI_ISL_16394922, EPI_ISL_16395667, EPI_ISL_16398472, EPI_ISL_16399824, EPI_ISL_16400033, EPI_ISL_16414127, EPI_ISL_16422834, EPI_ISL_16424130, EPI_ISL_16425691, EPI_ISL_16428100, EPI_ISL_16428101, EPI_ISL_16428102, EPI_ISL_16429066, EPI_ISL_16436533, EPI_ISL_16439413, EPI_ISL_16443129, EPI_ISL_16443688, EPI_ISL_16443874, EPI_ISL_16444600, EPI_ISL_16452054, EPI_ISL_16454044, EPI_ISL_16460823, EPI_ISL_16461302, EPI_ISL_16464657, EPI_ISL_16467436, EPI_ISL_16470710, EPI_ISL_16471153, EPI_ISL_16473435, EPI_ISL_16473762, EPI_ISL_16474400, EPI_ISL_16479826, EPI_ISL_16482060, EPI_ISL_16489594, EPI_ISL_16491494, EPI_ISL_16492150, EPI_ISL_16492585, EPI_ISL_16492756, EPI_ISL_16493785, EPI_ISL_16497702, EPI_ISL_16498515, EPI_ISL_16503949, EPI_ISL_16507701, EPI_ISL_16507927, EPI_ISL_16520597, EPI_ISL_16520598, EPI_ISL_16520637, EPI_ISL_16520640, EPI_ISL_16520641, EPI_ISL_16520711, EPI_ISL_16520763, EPI_ISL_16520788, EPI_ISL_16521101, EPI_ISL_16524906, EPI_ISL_16528641, EPI_ISL_16528645, EPI_ISL_16528903, EPI_ISL_16535376, EPI_ISL_16536212, EPI_ISL_16539692, EPI_ISL_16541774, EPI_ISL_16542553, EPI_ISL_16544506, EPI_ISL_16567779, EPI_ISL_16574574, EPI_ISL_16581578, EPI_ISL_16584104, EPI_ISL_16586683, EPI_ISL_16586702, EPI_ISL_16587574, EPI_ISL_16597363, EPI_ISL_16607452, EPI_ISL_16607500, EPI_ISL_16607535, EPI_ISL_16611498, EPI_ISL_16611571, EPI_ISL_16613287, EPI_ISL_16613482, EPI_ISL_16615597, EPI_ISL_16615617, EPI_ISL_16615668, EPI_ISL_16616642, EPI_ISL_16625690, EPI_ISL_16626611, EPI_ISL_16626666, EPI_ISL_16627067, EPI_ISL_16628854, EPI_ISL_16630260, EPI_ISL_16630261, EPI_ISL_16637607, EPI_ISL_16637631, EPI_ISL_16638190, EPI_ISL_16638453, EPI_ISL_16643406, EPI_ISL_16649988, EPI_ISL_16653618, EPI_ISL_16669313, EPI_ISL_16669829, EPI_ISL_16672301, EPI_ISL_16672327, EPI_ISL_16672352, EPI_ISL_16676267, EPI_ISL_16677015, EPI_ISL_16678917, EPI_ISL_16678946, EPI_ISL_16679654, EPI_ISL_16681917, EPI_ISL_16682342, EPI_ISL_16688219, EPI_ISL_16688525, EPI_ISL_16688688, EPI_ISL_16688713, EPI_ISL_16691397, EPI_ISL_16691487, EPI_ISL_16694176, EPI_ISL_16695435, EPI_ISL_16697861, EPI_ISL_16700160, EPI_ISL_16702838, EPI_ISL_16705882, EPI_ISL_16706498, EPI_ISL_16708798, EPI_ISL_16711038, EPI_ISL_16711095, EPI_ISL_16711417, EPI_ISL_16711531, EPI_ISL_16716967, EPI_ISL_16721930, EPI_ISL_16722183, EPI_ISL_16722215, EPI_ISL_16722270, EPI_ISL_16722970, EPI_ISL_16723215, EPI_ISL_16725887, EPI_ISL_16727241, EPI_ISL_16728257, EPI_ISL_16728383, EPI_ISL_16728411, EPI_ISL_16731753, EPI_ISL_16736400, EPI_ISL_16739452, EPI_ISL_16740104, EPI_ISL_16740406, EPI_ISL_16741567, EPI_ISL_16741573, EPI_ISL_16749999, EPI_ISL_16750878, EPI_ISL_16751721, EPI_ISL_16751722, EPI_ISL_16751789, EPI_ISL_16751791, EPI_ISL_16751977, EPI_ISL_16752073, EPI_ISL_16752138, EPI_ISL_16757168, EPI_ISL_16757210, EPI_ISL_16758981, EPI_ISL_16764861, EPI_ISL_16765888, EPI_ISL_16766196, EPI_ISL_16811091, EPI_ISL_16812565, EPI_ISL_16815494, EPI_ISL_16816293, EPI_ISL_16818458, EPI_ISL_16818471, EPI_ISL_16825222, EPI_ISL_16828876, EPI_ISL_16828896, EPI_ISL_16829188, EPI_ISL_16831507, EPI_ISL_16833893, EPI_ISL_16834974, EPI_ISL_16835399, EPI_ISL_16842787, EPI_ISL_16842790, EPI_ISL_16847425, EPI_ISL_16847642, EPI_ISL_16847674, EPI_ISL_16847675, EPI_ISL_16847676, EPI_ISL_16847677, EPI_ISL_16853227, EPI_ISL_16853229, EPI_ISL_16853597, EPI_ISL_16856355, EPI_ISL_16856565, EPI_ISL_16856637, EPI_ISL_16856833, EPI_ISL_16857514, EPI_ISL_16857776, EPI_ISL_16857903, EPI_ISL_16858310, EPI_ISL_16858617, EPI_ISL_16858667, EPI_ISL_16861084, EPI_ISL_16863260, EPI_ISL_16866580, EPI_ISL_16868647, EPI_ISL_16868655, EPI_ISL_16868993, EPI_ISL_16869007, EPI_ISL_16875752, EPI_ISL_16876039, EPI_ISL_16876784, EPI_ISL_16877428, EPI_ISL_16878720, EPI_ISL_16883240, EPI_ISL_16883873, EPI_ISL_16884622, EPI_ISL_16894717, EPI_ISL_16895138, EPI_ISL_16895290, EPI_ISL_16903492, EPI_ISL_16903494, EPI_ISL_16904536, EPI_ISL_16908472, EPI_ISL_16910025, EPI_ISL_16910165, EPI_ISL_16910272, EPI_ISL_16921530, EPI_ISL_16925257, EPI_ISL_16927736, EPI_ISL_16931901, EPI_ISL_16941750, EPI_ISL_16942000, EPI_ISL_16945429, EPI_ISL_16946783, EPI_ISL_16947592, EPI_ISL_16947625, EPI_ISL_16951592, EPI_ISL_16953741, EPI_ISL_16954486, EPI_ISL_16955471, EPI_ISL_16957015, EPI_ISL_16966997, EPI_ISL_16967082, EPI_ISL_16967083, EPI_ISL_16967084, EPI_ISL_16967085, EPI_ISL_16967086, EPI_ISL_16969756, EPI_ISL_16969757, EPI_ISL_16970279, EPI_ISL_16973343, EPI_ISL_16977317, EPI_ISL_16977653, EPI_ISL_16977749, EPI_ISL_16979482, EPI_ISL_16980683, EPI_ISL_16981030, EPI_ISL_16981102, EPI_ISL_16987088, EPI_ISL_16987375, EPI_ISL_16987376, EPI_ISL_16995491, EPI_ISL_16995525, EPI_ISL_16997638, EPI_ISL_17001974, EPI_ISL_17001987, EPI_ISL_17002442, EPI_ISL_17006258, EPI_ISL_17008393, EPI_ISL_17008502, EPI_ISL_17016219, EPI_ISL_17018731, EPI_ISL_17020636, EPI_ISL_17021187, EPI_ISL_17022063, EPI_ISL_17022081, EPI_ISL_17024099, EPI_ISL_17025560, EPI_ISL_17025998, EPI_ISL_17026052, EPI_ISL_17026537, EPI_ISL_17027430, EPI_ISL_17032070, EPI_ISL_17032664, EPI_ISL_17035345, EPI_ISL_17036551, EPI_ISL_17037388, EPI_ISL_17040133, EPI_ISL_17041105, EPI_ISL_17041117, EPI_ISL_17041143, EPI_ISL_17046406, EPI_ISL_17047667, EPI_ISL_17050958, EPI_ISL_17051743, EPI_ISL_17056159, EPI_ISL_17057279, EPI_ISL_17065016, EPI_ISL_17067007, EPI_ISL_17068616, EPI_ISL_17068621, EPI_ISL_17073286, EPI_ISL_17076011, EPI_ISL_17076926, EPI_ISL_17077233, EPI_ISL_17077446, EPI_ISL_17079150, EPI_ISL_17079151, EPI_ISL_17079427, EPI_ISL_17080036, EPI_ISL_17080146, EPI_ISL_17080283, EPI_ISL_17080510, EPI_ISL_17081567, EPI_ISL_17084330, EPI_ISL_17086936, EPI_ISL_17086958, EPI_ISL_17090268, EPI_ISL_17090730, EPI_ISL_17092242, EPI_ISL_17092260, EPI_ISL_17099321, EPI_ISL_17099446, EPI_ISL_17101049, EPI_ISL_17104807, EPI_ISL_17105674, EPI_ISL_17105677, EPI_ISL_17105777, EPI_ISL_17105786, EPI_ISL_17105804, EPI_ISL_17106895, EPI_ISL_17109738, EPI_ISL_17109787, EPI_ISL_17112198, EPI_ISL_17113114, EPI_ISL_17118740, EPI_ISL_17126699, EPI_ISL_17126727, EPI_ISL_17127510, EPI_ISL_17129671, EPI_ISL_17139969, EPI_ISL_17149697, EPI_ISL_17150312, EPI_ISL_17150571, EPI_ISL_17152522, EPI_ISL_17152602, EPI_ISL_17152816, EPI_ISL_17154843, EPI_ISL_17154893, EPI_ISL_17158601, EPI_ISL_17158659, EPI_ISL_17158660, EPI_ISL_17158661, EPI_ISL_17158662, EPI_ISL_17158663, EPI_ISL_17158664, EPI_ISL_17158665, EPI_ISL_17164529, EPI_ISL_17165387, EPI_ISL_17165528, EPI_ISL_17170921, EPI_ISL_17173754, EPI_ISL_17174278, EPI_ISL_17174323, EPI_ISL_17175107, EPI_ISL_17180776, EPI_ISL_17182281, EPI_ISL_17188691, EPI_ISL_17188772, EPI_ISL_17188836, EPI_ISL_17189286, EPI_ISL_17189360, EPI_ISL_17189372, EPI_ISL_17190813, EPI_ISL_17191784, EPI_ISL_17193988, EPI_ISL_17194121, EPI_ISL_17194564, EPI_ISL_17195807, EPI_ISL_17199165, EPI_ISL_17199381, EPI_ISL_17199743, EPI_ISL_17200348, EPI_ISL_17200520, EPI_ISL_17201694, EPI_ISL_17202051, EPI_ISL_17205892, EPI_ISL_17206016, EPI_ISL_17206140, EPI_ISL_17207424, EPI_ISL_17210230, EPI_ISL_17210689, EPI_ISL_17214413, EPI_ISL_17214693, EPI_ISL_17214774, EPI_ISL_17214805, EPI_ISL_17214933, EPI_ISL_17215427, EPI_ISL_17215676, EPI_ISL_17215686, EPI_ISL_17215790, EPI_ISL_17216822, EPI_ISL_17216978, EPI_ISL_17222365, EPI_ISL_17223438, EPI_ISL_17226531, EPI_ISL_17232350, EPI_ISL_17232448, EPI_ISL_17237921, EPI_ISL_17239049, EPI_ISL_17239405, EPI_ISL_17239499, EPI_ISL_17241376, EPI_ISL_17244668, EPI_ISL_17245140, EPI_ISL_17245198, EPI_ISL_17245255, EPI_ISL_17246876, EPI_ISL_17246931, EPI_ISL_17247186, EPI_ISL_17247325, EPI_ISL_17247333, EPI_ISL_17251028, EPI_ISL_17252934, EPI_ISL_17253364, EPI_ISL_17253589, EPI_ISL_17257608, EPI_ISL_17262137, EPI_ISL_17265160, EPI_ISL_17270165, EPI_ISL_17270950, EPI_ISL_17270964, EPI_ISL_17270974, EPI_ISL_17271226, EPI_ISL_17271272, EPI_ISL_17272946, EPI_ISL_17273453, EPI_ISL_17273454, EPI_ISL_17273455, EPI_ISL_17273456, EPI_ISL_17275616, EPI_ISL_17275984, EPI_ISL_17276025, EPI_ISL_17276962, EPI_ISL_17284010, EPI_ISL_17284045, EPI_ISL_17284573, EPI_ISL_17285690, EPI_ISL_17288589, EPI_ISL_17290740, EPI_ISL_17292666, EPI_ISL_17292834, EPI_ISL_17297993, EPI_ISL_17298321, EPI_ISL_17298323, EPI_ISL_17299688, EPI_ISL_17300150, EPI_ISL_17304801, EPI_ISL_17304899, EPI_ISL_17305358, EPI_ISL_17319411, EPI_ISL_17319528, EPI_ISL_17319601, EPI_ISL_17321362, EPI_ISL_17322993, EPI_ISL_17334027, EPI_ISL_17342544, EPI_ISL_17344004, EPI_ISL_17344178, EPI_ISL_17344660, EPI_ISL_17345445, EPI_ISL_17347577, EPI_ISL_17348219, EPI_ISL_17349770, EPI_ISL_17349983, EPI_ISL_17350301, EPI_ISL_17352192, EPI_ISL_17359772, EPI_ISL_17370155, EPI_ISL_17374605, EPI_ISL_17374609, EPI_ISL_17374807, EPI_ISL_17376230, EPI_ISL_17381216, EPI_ISL_17387122, EPI_ISL_17389140, EPI_ISL_17389210, EPI_ISL_17389223, EPI_ISL_17389779, EPI_ISL_17390660, EPI_ISL_17390743, EPI_ISL_17390873, EPI_ISL_17391460, EPI_ISL_17394837, EPI_ISL_17397497, EPI_ISL_17408352, EPI_ISL_17409157, EPI_ISL_17411543, EPI_ISL_17414235, EPI_ISL_17414543, EPI_ISL_17421962, EPI_ISL_17423074, EPI_ISL_17424014, EPI_ISL_17429770, EPI_ISL_17430458, EPI_ISL_17430487, EPI_ISL_17431238, EPI_ISL_17434223, EPI_ISL_17434227, EPI_ISL_17437940, EPI_ISL_17440507, EPI_ISL_17441169, EPI_ISL_17441208, EPI_ISL_17441815, EPI_ISL_17445401, EPI_ISL_17446132, EPI_ISL_17464711, EPI_ISL_17466081, EPI_ISL_17470229, EPI_ISL_17470269, EPI_ISL_17471181, EPI_ISL_17471619, EPI_ISL_17471674, EPI_ISL_17472531, EPI_ISL_17475799, EPI_ISL_17476568, EPI_ISL_17476871, EPI_ISL_17477106, EPI_ISL_17480516, EPI_ISL_17481180, EPI_ISL_17481517, EPI_ISL_17481597, EPI_ISL_17482811, EPI_ISL_17482813, EPI_ISL_17482815, EPI_ISL_17482819, EPI_ISL_17494731, EPI_ISL_17497461, EPI_ISL_17497688, EPI_ISL_17497868, EPI_ISL_17501536, EPI_ISL_17501576, EPI_ISL_17501763, EPI_ISL_17502219, EPI_ISL_17502972, EPI_ISL_17503268, EPI_ISL_17503711, EPI_ISL_17504816, EPI_ISL_17504835, EPI_ISL_17505072, EPI_ISL_17508749, EPI_ISL_17509597, EPI_ISL_17510495, EPI_ISL_17510856, EPI_ISL_17511096, EPI_ISL_17511836, EPI_ISL_17512412, EPI_ISL_17512876, EPI_ISL_17512968, EPI_ISL_17513312, EPI_ISL_17514540, EPI_ISL_17515086, EPI_ISL_17515177, EPI_ISL_17516651, EPI_ISL_17516658, EPI_ISL_17516659, EPI_ISL_17517664, EPI_ISL_17517834, EPI_ISL_17517844, EPI_ISL_17521302, EPI_ISL_17522610, EPI_ISL_17522687, EPI_ISL_17522934, EPI_ISL_17523535, EPI_ISL_17523620, EPI_ISL_17523782, EPI_ISL_17523873, EPI_ISL_17524106, EPI_ISL_17524502, EPI_ISL_17524503, EPI_ISL_17535664, EPI_ISL_17535979, EPI_ISL_17541088, EPI_ISL_17541797, EPI_ISL_17543006, EPI_ISL_17544283, EPI_ISL_17545970, EPI_ISL_17547529, EPI_ISL_17547545, EPI_ISL_17548526, EPI_ISL_17549129, EPI_ISL_17550129, EPI_ISL_17550538, EPI_ISL_17553063, EPI_ISL_17553974, EPI_ISL_17556705, EPI_ISL_17559150, EPI_ISL_17559165, EPI_ISL_17559166, EPI_ISL_17559167, EPI_ISL_17559168, EPI_ISL_17563568, EPI_ISL_17565071, EPI_ISL_17565211, EPI_ISL_17565212, EPI_ISL_17566854, EPI_ISL_17579120, EPI_ISL_17583157, EPI_ISL_17584277, EPI_ISL_17585020, EPI_ISL_17585021, EPI_ISL_17585022, EPI_ISL_17585023, EPI_ISL_17585036, EPI_ISL_17585039, EPI_ISL_17586115, EPI_ISL_17587423, EPI_ISL_17587656, EPI_ISL_17587671, EPI_ISL_17587859, EPI_ISL_17588127, EPI_ISL_17588216, EPI_ISL_17588460, EPI_ISL_17589845, EPI_ISL_17590449, EPI_ISL_17590486, EPI_ISL_17591005, EPI_ISL_17591014, EPI_ISL_17591015, EPI_ISL_17591028, EPI_ISL_17592236, EPI_ISL_17592618, EPI_ISL_17593692, EPI_ISL_17593865, EPI_ISL_17595116, EPI_ISL_17595117, EPI_ISL_17595980, EPI_ISL_17597954, EPI_ISL_17598384, EPI_ISL_17599326, EPI_ISL_17599427, EPI_ISL_17600948, EPI_ISL_17600958, EPI_ISL_17600978, EPI_ISL_17600988, EPI_ISL_17601066, EPI_ISL_17601144, EPI_ISL_17601196, EPI_ISL_17601219, EPI_ISL_17601261, EPI_ISL_17601276, EPI_ISL_17601933, EPI_ISL_17602469, EPI_ISL_17602756, EPI_ISL_17605514, EPI_ISL_17612035, EPI_ISL_17612050, EPI_ISL_17612051, EPI_ISL_17612052, EPI_ISL_17615127, EPI_ISL_17615387, EPI_ISL_17617168, EPI_ISL_17617538, EPI_ISL_17618306, EPI_ISL_17621930, EPI_ISL_17623470, EPI_ISL_17623785, EPI_ISL_17623810, EPI_ISL_17626289, EPI_ISL_17628383, EPI_ISL_17628855, EPI_ISL_17630096, EPI_ISL_17632950, EPI_ISL_17634290, EPI_ISL_17634585, EPI_ISL_17634799, EPI_ISL_17637409, EPI_ISL_17637499, EPI_ISL_17637946, EPI_ISL_17640029, EPI_ISL_17640079, EPI_ISL_17642765, EPI_ISL_17643093, EPI_ISL_17644186, EPI_ISL_17645081, EPI_ISL_17645416, EPI_ISL_17646422, EPI_ISL_17648924, EPI_ISL_17651803, EPI_ISL_17652508, EPI_ISL_17652513, EPI_ISL_17654325, EPI_ISL_17654831, EPI_ISL_17655018, EPI_ISL_17656002, EPI_ISL_17657287, EPI_ISL_17658392, EPI_ISL_17659247, EPI_ISL_17659794, EPI_ISL_17661435, EPI_ISL_17661709, EPI_ISL_17661736, EPI_ISL_17661772, EPI_ISL_17662111, EPI_ISL_17664370, EPI_ISL_17666708, EPI_ISL_17667360, EPI_ISL_17667562, EPI_ISL_17669441, EPI_ISL_17669457, EPI_ISL_17671157, EPI_ISL_17671162, EPI_ISL_17671689, EPI_ISL_17675889, EPI_ISL_17677128, EPI_ISL_17677325, EPI_ISL_17678395, EPI_ISL_17679253, EPI_ISL_17679612, EPI_ISL_17680172, EPI_ISL_17683135, EPI_ISL_17683747, EPI_ISL_17683879, EPI_ISL_17683882, EPI_ISL_17683902, EPI_ISL_17683926, EPI_ISL_17684194, EPI_ISL_17685960, EPI_ISL_17685982, EPI_ISL_17686409, EPI_ISL_17686485, EPI_ISL_17686694, EPI_ISL_17686736, EPI_ISL_17688072, EPI_ISL_17689247, EPI_ISL_17695348, EPI_ISL_17696086, EPI_ISL_17696551, EPI_ISL_17696575, EPI_ISL_17697616, EPI_ISL_17699149, EPI_ISL_17699879, EPI_ISL_17700051, EPI_ISL_17700270, EPI_ISL_17701278, EPI_ISL_17701782, EPI_ISL_17703815, EPI_ISL_17704713, EPI_ISL_17706013, EPI_ISL_17706030, EPI_ISL_17708288, EPI_ISL_17710268, EPI_ISL_17710278, EPI_ISL_17710307, EPI_ISL_17710673, EPI_ISL_17710974, EPI_ISL_17711012, EPI_ISL_17711646, EPI_ISL_17713423, EPI_ISL_17713709, EPI_ISL_17714880, EPI_ISL_17714902, EPI_ISL_17714948, EPI_ISL_17715122, EPI_ISL_17715974, EPI_ISL_17716296, EPI_ISL_17718358, EPI_ISL_17718497, EPI_ISL_17719162, EPI_ISL_17721620, EPI_ISL_17721941, EPI_ISL_17722142, EPI_ISL_17722884, EPI_ISL_17726746, EPI_ISL_17727194, EPI_ISL_17728144, EPI_ISL_17728250, EPI_ISL_17731387, EPI_ISL_17731388, EPI_ISL_17732098, EPI_ISL_17733269, EPI_ISL_17734236, EPI_ISL_17735972, EPI_ISL_17736284, EPI_ISL_17737562, EPI_ISL_17739108, EPI_ISL_17741957, EPI_ISL_17743681, EPI_ISL_17744022, EPI_ISL_17747309, EPI_ISL_17759354, EPI_ISL_17759925, EPI_ISL_17760156, EPI_ISL_17762387, EPI_ISL_17762760, EPI_ISL_17763721, EPI_ISL_17764011, EPI_ISL_17764066, EPI_ISL_17764072, EPI_ISL_17764496, EPI_ISL_17766060, EPI_ISL_17766100, EPI_ISL_17766112, EPI_ISL_17767434, EPI_ISL_17767435, EPI_ISL_17767436, EPI_ISL_17767437, EPI_ISL_17769081, EPI_ISL_17769169, EPI_ISL_17769216, EPI_ISL_17769229, EPI_ISL_17769310, EPI_ISL_17769888, EPI_ISL_17770729, EPI_ISL_17770732, EPI_ISL_17770736, EPI_ISL_17770779, EPI_ISL_17771047, EPI_ISL_17771051, EPI_ISL_17775344, EPI_ISL_17776736, EPI_ISL_17777061, EPI_ISL_17777067, EPI_ISL_17777729, EPI_ISL_17778593, EPI_ISL_17778602, EPI_ISL_17780724, EPI_ISL_17780726, EPI_ISL_17780860, EPI_ISL_17780886, EPI_ISL_17781122, EPI_ISL_17781585, EPI_ISL_17781712, EPI_ISL_17782148, EPI_ISL_17782366, EPI_ISL_17782502, EPI_ISL_17783358, EPI_ISL_17784545, EPI_ISL_17784546, EPI_ISL_17784547, EPI_ISL_17784552, EPI_ISL_17784558, EPI_ISL_17784569, EPI_ISL_17784585, EPI_ISL_17784593, EPI_ISL_17784775, EPI_ISL_17784803, EPI_ISL_17784804, EPI_ISL_17786165, EPI_ISL_17786546, EPI_ISL_17786769, EPI_ISL_17786827, EPI_ISL_17787009, EPI_ISL_17787597, EPI_ISL_17787864, EPI_ISL_17788384, EPI_ISL_17789385, EPI_ISL_17789475, EPI_ISL_17789808, EPI_ISL_17790033, EPI_ISL_17790116, EPI_ISL_17791306, EPI_ISL_17791796, EPI_ISL_17792172, EPI_ISL_17792191, EPI_ISL_17794816, EPI_ISL_17796500, EPI_ISL_17796537, EPI_ISL_17796598, EPI_ISL_17796704, EPI_ISL_17797704, EPI_ISL_17798165, EPI_ISL_17799068, EPI_ISL_17799108, EPI_ISL_17802597, EPI_ISL_17803325, EPI_ISL_17803653, EPI_ISL_17806504, EPI_ISL_17806524, EPI_ISL_17809334, EPI_ISL_17809574, EPI_ISL_17812915, EPI_ISL_17813049, EPI_ISL_17813537, EPI_ISL_17813637, EPI_ISL_17813862, EPI_ISL_17815222, EPI_ISL_17816174, EPI_ISL_17817657, EPI_ISL_17817985, EPI_ISL_17818039, EPI_ISL_17819921, EPI_ISL_17820257, EPI_ISL_17820258, EPI_ISL_17820602, EPI_ISL_17821850, EPI_ISL_17823538, EPI_ISL_17824292, EPI_ISL_17824608, EPI_ISL_17824670, EPI_ISL_17826285, EPI_ISL_17830573, EPI_ISL_17830591, EPI_ISL_17830762, EPI_ISL_17831005, EPI_ISL_17831639, EPI_ISL_17831941, EPI_ISL_17833161, EPI_ISL_17833549, EPI_ISL_17837092, EPI_ISL_17837097, EPI_ISL_17837134, EPI_ISL_17837135, EPI_ISL_17837188, EPI_ISL_17837432, EPI_ISL_17837459, EPI_ISL_17837460, EPI_ISL_17837914, EPI_ISL_17837915, EPI_ISL_17838109, EPI_ISL_17838506, EPI_ISL_17850070, EPI_ISL_17850078, EPI_ISL_17851276, EPI_ISL_17853355, EPI_ISL_17853579, EPI_ISL_17855226, EPI_ISL_17856975, EPI_ISL_17857949, EPI_ISL_17857950, EPI_ISL_17859477, EPI_ISL_17860390, EPI_ISL_17860984, EPI_ISL_17862677, EPI_ISL_17871595, EPI_ISL_17879222, EPI_ISL_17884376, EPI_ISL_17884518, EPI_ISL_17885064, EPI_ISL_17885128, EPI_ISL_17885331, EPI_ISL_17885459, EPI_ISL_17891004, EPI_ISL_17899627, EPI_ISL_17949029, EPI_ISL_17949339, EPI_ISL_17949978, EPI_ISL_17952015, EPI_ISL_17952019, EPI_ISL_17953343, EPI_ISL_17953610, EPI_ISL_17954106, EPI_ISL_17954662, EPI_ISL_17954669, EPI_ISL_17954940, EPI_ISL_17956164, EPI_ISL_17958015, EPI_ISL_17959424, EPI_ISL_17960600, EPI_ISL_17960747, EPI_ISL_17964403, EPI_ISL_17964415, EPI_ISL_17964828, EPI_ISL_17965636, EPI_ISL_17966200, EPI_ISL_17966205, EPI_ISL_17968777, EPI_ISL_17968962, EPI_ISL_17969108, EPI_ISL_17971223, EPI_ISL_17971936, EPI_ISL_17972242, EPI_ISL_17972372, EPI_ISL_17973367, EPI_ISL_17974574, EPI_ISL_17974688, EPI_ISL_17974927, EPI_ISL_17974952, EPI_ISL_17975003, EPI_ISL_17975174, EPI_ISL_17976113, EPI_ISL_17976116, EPI_ISL_17977985, EPI_ISL_17978344, EPI_ISL_17978693, EPI_ISL_17978839, EPI_ISL_17979017, EPI_ISL_17979965, EPI_ISL_17979979, EPI_ISL_17979981, EPI_ISL_17982411, EPI_ISL_17982453, EPI_ISL_17982543, EPI_ISL_17985757, EPI_ISL_17988396, EPI_ISL_17989190, EPI_ISL_17989433, EPI_ISL_17989516, EPI_ISL_17989740, EPI_ISL_17989749, EPI_ISL_17989792, EPI_ISL_17989829, EPI_ISL_17990304, EPI_ISL_17993966, EPI_ISL_17994784, EPI_ISL_17994786, EPI_ISL_17995488, EPI_ISL_17995513, EPI_ISL_17995955, EPI_ISL_17996897, EPI_ISL_17997249, EPI_ISL_17997251, EPI_ISL_17997917, EPI_ISL_17997982, EPI_ISL_17998406, EPI_ISL_18000155, EPI_ISL_18000245, EPI_ISL_18000414, EPI_ISL_18000654, EPI_ISL_18000825, EPI_ISL_18001789, EPI_ISL_18001862, EPI_ISL_18008246, EPI_ISL_18008262, EPI_ISL_18008673, EPI_ISL_18009591, EPI_ISL_18009602, EPI_ISL_18010720, EPI_ISL_18011518, EPI_ISL_18012526, EPI_ISL_18012547, EPI_ISL_18012806, EPI_ISL_18014700, EPI_ISL_18016999, EPI_ISL_18019246, EPI_ISL_18028785, EPI_ISL_18029979, EPI_ISL_18030390, EPI_ISL_18030391, EPI_ISL_18030395, EPI_ISL_18031842, EPI_ISL_18032297, EPI_ISL_18032322, EPI_ISL_18032338, EPI_ISL_18033013, EPI_ISL_18033516, EPI_ISL_18033631, EPI_ISL_18034109, EPI_ISL_18037119, EPI_ISL_18037474, EPI_ISL_18037476, EPI_ISL_18037744, EPI_ISL_18038269, EPI_ISL_18039728, EPI_ISL_18040070, EPI_ISL_18041130, EPI_ISL_18041968, EPI_ISL_18042110, EPI_ISL_18044024, EPI_ISL_18044164, EPI_ISL_18044400, EPI_ISL_18044754, EPI_ISL_18044755, EPI_ISL_18044759, EPI_ISL_18045809, EPI_ISL_18048708, EPI_ISL_18048972, EPI_ISL_18048978, EPI_ISL_18049009, EPI_ISL_18049161, EPI_ISL_18049174, EPI_ISL_18049917, EPI_ISL_18050065, EPI_ISL_18050520, EPI_ISL_18050523, EPI_ISL_18051914, EPI_ISL_18051918, EPI_ISL_18052440, EPI_ISL_18052776, EPI_ISL_18052929, EPI_ISL_18053022, EPI_ISL_18053315, EPI_ISL_18054466, EPI_ISL_18054899, EPI_ISL_18056643, EPI_ISL_18056644, EPI_ISL_18056759, EPI_ISL_18056769, EPI_ISL_18058525, EPI_ISL_18058567, EPI_ISL_18058881, EPI_ISL_18059074, EPI_ISL_18059075, EPI_ISL_18059076, EPI_ISL_18059726, EPI_ISL_18060973, EPI_ISL_18062475, EPI_ISL_18064366, EPI_ISL_18064383, EPI_ISL_18064405, EPI_ISL_18064413, EPI_ISL_18064431, EPI_ISL_18064456, EPI_ISL_18070310, EPI_ISL_18071883, EPI_ISL_18071901, EPI_ISL_18072343, EPI_ISL_18073924, EPI_ISL_18074072, EPI_ISL_18075985, EPI_ISL_18076065, EPI_ISL_18076069, EPI_ISL_18076165, EPI_ISL_18076251, EPI_ISL_18076473, EPI_ISL_18077275, EPI_ISL_18078878, EPI_ISL_18079417, EPI_ISL_18080566, EPI_ISL_18083488, EPI_ISL_18091808, EPI_ISL_18093840, EPI_ISL_18094397, EPI_ISL_18094429, EPI_ISL_18094476, EPI_ISL_18094560, EPI_ISL_18095961, EPI_ISL_18097327, EPI_ISL_18097349, EPI_ISL_18097786, EPI_ISL_18098270, EPI_ISL_18098276, EPI_ISL_18098299, EPI_ISL_18098479, EPI_ISL_18098976, EPI_ISL_18099952, EPI_ISL_18100455, EPI_ISL_18100457, EPI_ISL_18100607, EPI_ISL_18104072, EPI_ISL_18104501, EPI_ISL_18106416, EPI_ISL_18106460, EPI_ISL_18106464, EPI_ISL_18106662, EPI_ISL_18106788, EPI_ISL_18106910, EPI_ISL_18106912, EPI_ISL_18106920, EPI_ISL_18106929, EPI_ISL_18106930, EPI_ISL_18106931, EPI_ISL_18106933, EPI_ISL_18106934, EPI_ISL_18106950, EPI_ISL_18106951, EPI_ISL_18107900, EPI_ISL_18109285, EPI_ISL_18110014, EPI_ISL_18110496, EPI_ISL_18110776, EPI_ISL_18111020, EPI_ISL_18111021, EPI_ISL_18111040, EPI_ISL_18111041, EPI_ISL_18111086, EPI_ISL_18112015, EPI_ISL_18115442, EPI_ISL_18115451, EPI_ISL_18115956, EPI_ISL_18116015, EPI_ISL_18116176, EPI_ISL_18118289, EPI_ISL_18118388, EPI_ISL_18118556, EPI_ISL_18118855, EPI_ISL_18119265, EPI_ISL_18120201, EPI_ISL_18123396, EPI_ISL_18124840, EPI_ISL_18125049, EPI_ISL_18125050, EPI_ISL_18126834, EPI_ISL_18127203, EPI_ISL_18127526, EPI_ISL_18127527, EPI_ISL_18127685, EPI_ISL_18127834, EPI_ISL_18129019, EPI_ISL_18129038, EPI_ISL_18129213, EPI_ISL_18129656, EPI_ISL_18129944, EPI_ISL_18131053, EPI_ISL_18131109, EPI_ISL_18134315, EPI_ISL_18134392, EPI_ISL_18134395, EPI_ISL_18134442, EPI_ISL_18134610, EPI_ISL_18134691, EPI_ISL_18134700, EPI_ISL_18134706, EPI_ISL_18134984, EPI_ISL_18135040, EPI_ISL_18136392, EPI_ISL_18136968, EPI_ISL_18139400, EPI_ISL_18139409, EPI_ISL_18141686, EPI_ISL_18141739, EPI_ISL_18141844, EPI_ISL_18142202, EPI_ISL_18142317, EPI_ISL_18142978, EPI_ISL_18142994, EPI_ISL_18146885, EPI_ISL_18147456, EPI_ISL_18147966, EPI_ISL_18151975, EPI_ISL_18151976, EPI_ISL_18151977, EPI_ISL_18153000, EPI_ISL_18159587, EPI_ISL_18160510, EPI_ISL_18160530, EPI_ISL_18160538, EPI_ISL_18162564, EPI_ISL_18162567, EPI_ISL_18163680, EPI_ISL_18164441, EPI_ISL_18166642, EPI_ISL_18166643, EPI_ISL_18168780, EPI_ISL_18169117, EPI_ISL_18205057, EPI_ISL_18207613, EPI_ISL_18210510, EPI_ISL_18212559, EPI_ISL_18213104, EPI_ISL_18215123, EPI_ISL_18215226, EPI_ISL_18215482, EPI_ISL_18215552, EPI_ISL_18217564, EPI_ISL_18217995, EPI_ISL_18218776, EPI_ISL_18219916, EPI_ISL_18219931, EPI_ISL_18219970, EPI_ISL_18220073, EPI_ISL_18220494, EPI_ISL_18220495, EPI_ISL_18220498, EPI_ISL_18221521, EPI_ISL_18221524, EPI_ISL_18221527, EPI_ISL_18221982, EPI_ISL_18221985, EPI_ISL_18222367, EPI_ISL_18224410, EPI_ISL_18224514, EPI_ISL_18225473, EPI_ISL_18227366, EPI_ISL_18227596, EPI_ISL_18227611, EPI_ISL_18227624, EPI_ISL_18227629, EPI_ISL_18228307, EPI_ISL_18232124, EPI_ISL_18233906, EPI_ISL_18234431, EPI_ISL_18236180, EPI_ISL_18236291, EPI_ISL_18237063, EPI_ISL_18238117, EPI_ISL_18241087, EPI_ISL_18241705, EPI_ISL_18241707, EPI_ISL_18241719, EPI_ISL_18245571, EPI_ISL_18247259, EPI_ISL_18248695, EPI_ISL_18249682, EPI_ISL_18253248, EPI_ISL_18253249, EPI_ISL_18255994, EPI_ISL_18256173, EPI_ISL_18256714, EPI_ISL_18256980, EPI_ISL_18258766, EPI_ISL_18259784, EPI_ISL_18260202, EPI_ISL_18263945, EPI_ISL_18263981, EPI_ISL_18271265, EPI_ISL_18273982, EPI_ISL_18274346, EPI_ISL_18276415, EPI_ISL_18277439, EPI_ISL_18277736, EPI_ISL_18278627, EPI_ISL_18278909, EPI_ISL_18279614, EPI_ISL_18281186, EPI_ISL_18281259, EPI_ISL_18281287, EPI_ISL_18281288, EPI_ISL_18281494, EPI_ISL_18281574, EPI_ISL_18282077, EPI_ISL_18282082, EPI_ISL_18286773, EPI_ISL_18287351, EPI_ISL_18290989, EPI_ISL_18291808, EPI_ISL_18292038, EPI_ISL_18292398, EPI_ISL_18294574, EPI_ISL_18295441, EPI_ISL_18298019, EPI_ISL_18299948, EPI_ISL_18301587, EPI_ISL_18302636, EPI_ISL_18303012, EPI_ISL_18303592, EPI_ISL_18303595, EPI_ISL_18303758, EPI_ISL_18306254, EPI_ISL_18306922, EPI_ISL_18308642, EPI_ISL_18311951, EPI_ISL_18313683, EPI_ISL_18315747, EPI_ISL_18315789, EPI_ISL_18319306, EPI_ISL_18319903, EPI_ISL_18319904, EPI_ISL_18319906, EPI_ISL_18319907, EPI_ISL_18320079, EPI_ISL_18321271, EPI_ISL_18322273, EPI_ISL_18322420, EPI_ISL_18322438, EPI_ISL_18323536, EPI_ISL_18324107, EPI_ISL_18324168, EPI_ISL_18324491, EPI_ISL_18324976, EPI_ISL_18325145, EPI_ISL_18325563, EPI_ISL_18326430, EPI_ISL_18326597, EPI_ISL_18326806, EPI_ISL_18326807, EPI_ISL_18330957, EPI_ISL_18330966, EPI_ISL_18331347, EPI_ISL_18334903, EPI_ISL_18334945, EPI_ISL_18334986, EPI_ISL_18335084, EPI_ISL_18336165, EPI_ISL_18336602, EPI_ISL_18336862, EPI_ISL_18337738, EPI_ISL_18338137, EPI_ISL_18338143, EPI_ISL_18338144, EPI_ISL_18338502, EPI_ISL_18338504, EPI_ISL_18338709, EPI_ISL_18342412, EPI_ISL_18343598, EPI_ISL_18345777, EPI_ISL_18345926, EPI_ISL_18346109, EPI_ISL_18351588, EPI_ISL_18352473, EPI_ISL_18352485, EPI_ISL_18352489, EPI_ISL_18359229, EPI_ISL_18359328, EPI_ISL_18359679, EPI_ISL_18360507, EPI_ISL_18360944, EPI_ISL_18361202, EPI_ISL_18362265, EPI_ISL_18362515, EPI_ISL_18363170, EPI_ISL_18363300, EPI_ISL_18365170, EPI_ISL_18365256, EPI_ISL_18367086, EPI_ISL_18367563, EPI_ISL_18367586, EPI_ISL_18367599, EPI_ISL_18367615, EPI_ISL_18367908, EPI_ISL_18367992, EPI_ISL_18370898, EPI_ISL_18370960, EPI_ISL_18370967, EPI_ISL_18371749, EPI_ISL_18373201, EPI_ISL_18377021, EPI_ISL_18377214, EPI_ISL_18377245, EPI_ISL_18377248, EPI_ISL_18378384, EPI_ISL_18380731, EPI_ISL_18381066, EPI_ISL_18383121, EPI_ISL_18383423, EPI_ISL_18384846, EPI_ISL_18384936, EPI_ISL_18385358, EPI_ISL_18385924, EPI_ISL_18386091, EPI_ISL_18386114, EPI_ISL_18387037, EPI_ISL_18388509, EPI_ISL_18388585, EPI_ISL_18389783, EPI_ISL_18391451, EPI_ISL_18392259, EPI_ISL_18392502, EPI_ISL_18393366, EPI_ISL_18395551, EPI_ISL_18398210, EPI_ISL_18398259, EPI_ISL_18400843, EPI_ISL_18400856, EPI_ISL_18400946, EPI_ISL_18400987, EPI_ISL_18401313, EPI_ISL_18403047, EPI_ISL_18403051, EPI_ISL_18403054, EPI_ISL_18403509, EPI_ISL_18403523, EPI_ISL_18404585, EPI_ISL_18405535, EPI_ISL_18405621, EPI_ISL_18406078, EPI_ISL_18406394, EPI_ISL_18408561, EPI_ISL_18410987, EPI_ISL_18414567, EPI_ISL_18414568, EPI_ISL_18414808, EPI_ISL_18415823, EPI_ISL_18415832, EPI_ISL_18415834, EPI_ISL_18415840, EPI_ISL_18415854, EPI_ISL_18416870, EPI_ISL_18417129, EPI_ISL_18417211, EPI_ISL_18419485, EPI_ISL_18419748, EPI_ISL_18421674, EPI_ISL_18422693, EPI_ISL_18422715, EPI_ISL_18422771, EPI_ISL_18423785, EPI_ISL_18423814, EPI_ISL_18423907, EPI_ISL_18424281, EPI_ISL_18424468, EPI_ISL_18426836, EPI_ISL_18428844, EPI_ISL_18429684, EPI_ISL_18429702, EPI_ISL_18429725, EPI_ISL_18429773, EPI_ISL_18429797, EPI_ISL_18432077, EPI_ISL_18433350, EPI_ISL_18434194, EPI_ISL_18435557, EPI_ISL_18435892, EPI_ISL_18435949, EPI_ISL_18436693, EPI_ISL_18437342, EPI_ISL_18438723, EPI_ISL_18439733, EPI_ISL_18440037, EPI_ISL_18440370, EPI_ISL_18440660, EPI_ISL_18440866, EPI_ISL_18441868, EPI_ISL_18443784, EPI_ISL_18443944, EPI_ISL_18448894, EPI_ISL_18449647, EPI_ISL_18449794, EPI_ISL_18449820, EPI_ISL_18449892, EPI_ISL_18450249, EPI_ISL_18450812, EPI_ISL_18451678, EPI_ISL_18453400, EPI_ISL_18455292, EPI_ISL_18455564, EPI_ISL_18455706, EPI_ISL_18455950, EPI_ISL_18457808, EPI_ISL_18459512, EPI_ISL_18461774, EPI_ISL_18462852, EPI_ISL_18463490, EPI_ISL_18463766, EPI_ISL_18466251, EPI_ISL_18468149, EPI_ISL_18470400, EPI_ISL_18472311, EPI_ISL_18472312, EPI_ISL_18473559, EPI_ISL_18474126, EPI_ISL_18474555, EPI_ISL_18475534, EPI_ISL_18475535, EPI_ISL_18480741, EPI_ISL_18486919, EPI_ISL_18487225, EPI_ISL_18489646, EPI_ISL_18489793, EPI_ISL_18489829, EPI_ISL_18491841, EPI_ISL_18491844, EPI_ISL_18492277, EPI_ISL_18492305, EPI_ISL_18492307, EPI_ISL_18492412, EPI_ISL_18492455, EPI_ISL_18493129, EPI_ISL_18495416, EPI_ISL_18496252, EPI_ISL_18496585, EPI_ISL_18498001, EPI_ISL_18498420, EPI_ISL_18498499, EPI_ISL_18500316, EPI_ISL_18500771, EPI_ISL_18501087, EPI_ISL_18503287, EPI_ISL_18509817, EPI_ISL_18512421, EPI_ISL_18512438, EPI_ISL_18513936, EPI_ISL_18514552, EPI_ISL_18515280, EPI_ISL_18515328, EPI_ISL_18515343, EPI_ISL_18515511, EPI_ISL_18515749, EPI_ISL_18516916, EPI_ISL_18518769, EPI_ISL_18518932, EPI_ISL_18519113, EPI_ISL_18520677, EPI_ISL_18520678, EPI_ISL_18521765, EPI_ISL_18522184, EPI_ISL_18522580, EPI_ISL_18523129, EPI_ISL_18524926, EPI_ISL_18525067, EPI_ISL_18525840, EPI_ISL_18526641, EPI_ISL_18529555, EPI_ISL_18530445, EPI_ISL_18530449, EPI_ISL_18536100, EPI_ISL_18536853, EPI_ISL_18537013, EPI_ISL_18537032, EPI_ISL_18537373, EPI_ISL_18537428, EPI_ISL_18537814, EPI_ISL_18538015, EPI_ISL_18543268, EPI_ISL_18546112, EPI_ISL_18546287, EPI_ISL_18546551, EPI_ISL_18546715, EPI_ISL_18551440, EPI_ISL_18552697, EPI_ISL_18553587, EPI_ISL_18554053, EPI_ISL_18556084, EPI_ISL_18556539, EPI_ISL_18557145, EPI_ISL_18558360, EPI_ISL_18558385, EPI_ISL_18558412, EPI_ISL_18558468, EPI_ISL_18558477, EPI_ISL_18559317, EPI_ISL_18560556, EPI_ISL_18560725, EPI_ISL_18560872, EPI_ISL_18561098, EPI_ISL_18563181, EPI_ISL_18563821, EPI_ISL_18564403, EPI_ISL_18564755, EPI_ISL_18566696, EPI_ISL_18567985, EPI_ISL_18568124, EPI_ISL_18576266, EPI_ISL_18576754, EPI_ISL_18577842, EPI_ISL_18577861, EPI_ISL_18577862, EPI_ISL_18577966, EPI_ISL_18578195, EPI_ISL_18579981, EPI_ISL_18580011, EPI_ISL_18580750, EPI_ISL_18580874, EPI_ISL_18581347, EPI_ISL_18584141, EPI_ISL_18588773, EPI_ISL_18589012, EPI_ISL_18589243, EPI_ISL_18589475, EPI_ISL_18589669, EPI_ISL_18590820, EPI_ISL_18591717, EPI_ISL_18593579, EPI_ISL_18594183, EPI_ISL_18594233, EPI_ISL_18594266, EPI_ISL_18595212, EPI_ISL_18598503, EPI_ISL_18598525, EPI_ISL_18603922, EPI_ISL_18604501, EPI_ISL_18604502, EPI_ISL_18604503, EPI_ISL_18604504, EPI_ISL_18604505, EPI_ISL_18605012, EPI_ISL_18605132, EPI_ISL_18606460, EPI_ISL_18607149, EPI_ISL_18607150, EPI_ISL_18609973, EPI_ISL_18612246, EPI_ISL_18615968, EPI_ISL_18622139, EPI_ISL_18624843, EPI_ISL_18625316, EPI_ISL_18626713, EPI_ISL_18626714, EPI_ISL_18626750, EPI_ISL_18630930, EPI_ISL_18633829, EPI_ISL_18634703, EPI_ISL_18635526, EPI_ISL_18635546, EPI_ISL_18635599, EPI_ISL_18635961, EPI_ISL_18639151, EPI_ISL_18640058, EPI_ISL_18641470, EPI_ISL_18641499, EPI_ISL_18642608, EPI_ISL_18646912, EPI_ISL_18646945, EPI_ISL_18648209, EPI_ISL_18652556, EPI_ISL_18653898, EPI_ISL_18654501, EPI_ISL_18668995, EPI_ISL_18672102, EPI_ISL_18677704, EPI_ISL_18681665, EPI_ISL_18681942, EPI_ISL_18681975, EPI_ISL_18683430, EPI_ISL_18687895, EPI_ISL_18689964, EPI_ISL_18693860, EPI_ISL_18694187, EPI_ISL_18702737, EPI_ISL_18712665, EPI_ISL_18712806, EPI_ISL_18712994, EPI_ISL_18713198, EPI_ISL_18714360, EPI_ISL_18714857, EPI_ISL_18715065, EPI_ISL_18715624, EPI_ISL_18717261, EPI_ISL_18727247, EPI_ISL_18727294, EPI_ISL_18729637, EPI_ISL_18730563, EPI_ISL_18740049, EPI_ISL_18742836, EPI_ISL_18743266, EPI_ISL_18743442, EPI_ISL_18743504, EPI_ISL_18743721, EPI_ISL_18759772, EPI_ISL_18760241, EPI_ISL_18763780, EPI_ISL_18766752, EPI_ISL_18772294, EPI_ISL_18777179, EPI_ISL_18778198, EPI_ISL_18778941, EPI_ISL_18782000, EPI_ISL_18782699, EPI_ISL_18784365, EPI_ISL_18784385, EPI_ISL_18784404, EPI_ISL_18784438, EPI_ISL_18787342, EPI_ISL_18787343, EPI_ISL_18792770, EPI_ISL_18796930, EPI_ISL_18797833, EPI_ISL_18798193, EPI_ISL_18798198, EPI_ISL_18798202, EPI_ISL_18798204, EPI_ISL_18798234, EPI_ISL_18799019, EPI_ISL_18801405, EPI_ISL_18803784, EPI_ISL_18805014, EPI_ISL_18809509, EPI_ISL_18809652, EPI_ISL_18811047, EPI_ISL_18811325, EPI_ISL_18811375, EPI_ISL_18811389, EPI_ISL_18814271, EPI_ISL_18814272, EPI_ISL_18814273, EPI_ISL_18815556, EPI_ISL_18816278, EPI_ISL_18816628, EPI_ISL_18817415, EPI_ISL_18818949, EPI_ISL_18824608, EPI_ISL_18825438, EPI_ISL_18829708, EPI_ISL_18832845, EPI_ISL_18839275, EPI_ISL_18839824, EPI_ISL_18839850, EPI_ISL_18839932, EPI_ISL_18841696, EPI_ISL_18842709, EPI_ISL_18846230, EPI_ISL_18851473, EPI_ISL_18853389, EPI_ISL_18853925, EPI_ISL_18854197, EPI_ISL_18854790, EPI_ISL_18854974, EPI_ISL_18856328, EPI_ISL_18859868, EPI_ISL_18860821, EPI_ISL_18863734, EPI_ISL_18864357, EPI_ISL_18864823, EPI_ISL_18865364, EPI_ISL_18865427, EPI_ISL_18868940, EPI_ISL_18869806, EPI_ISL_18870297, EPI_ISL_18873437, EPI_ISL_18873899, EPI_ISL_18874096, EPI_ISL_18874749, EPI_ISL_18875008, EPI_ISL_18876158, EPI_ISL_18876159, EPI_ISL_18876160, EPI_ISL_18876161, EPI_ISL_18876171, EPI_ISL_18876178, EPI_ISL_18876356, EPI_ISL_18877288, EPI_ISL_18877379, EPI_ISL_18877954, EPI_ISL_18878665, EPI_ISL_18880075, EPI_ISL_18882888, EPI_ISL_18884114, EPI_ISL_18885028, EPI_ISL_18885545, EPI_ISL_18886798, EPI_ISL_18888145, EPI_ISL_18888340, EPI_ISL_18889879, EPI_ISL_18892766, EPI_ISL_18896020, EPI_ISL_18897180, EPI_ISL_18897181, EPI_ISL_18899656, EPI_ISL_18899840, EPI_ISL_18900159, EPI_ISL_18901224, EPI_ISL_18901411, EPI_ISL_18901513, EPI_ISL_18901762, EPI_ISL_18901997, EPI_ISL_18904234, EPI_ISL_18906411, EPI_ISL_18906577, EPI_ISL_18908924, EPI_ISL_18915560, EPI_ISL_18916107, EPI_ISL_18916798, EPI_ISL_18916914, EPI_ISL_18917364, EPI_ISL_18917482, EPI_ISL_18917496, EPI_ISL_18917541, EPI_ISL_18918931, EPI_ISL_18918932, EPI_ISL_18918944, EPI_ISL_18919477, EPI_ISL_18919544, EPI_ISL_18924428, EPI_ISL_18927022, EPI_ISL_18927251, EPI_ISL_18927258, EPI_ISL_18927292, EPI_ISL_18927500, EPI_ISL_18930149, EPI_ISL_18930177, EPI_ISL_18930182, EPI_ISL_18930190, EPI_ISL_18930681, EPI_ISL_18931503, EPI_ISL_18931504, EPI_ISL_18931535, EPI_ISL_18931538, EPI_ISL_18931542, EPI_ISL_18932507, EPI_ISL_18933211, EPI_ISL_18933315, EPI_ISL_18933410, EPI_ISL_18933606, EPI_ISL_18935631, EPI_ISL_18936396, EPI_ISL_18939566, EPI_ISL_18939949, EPI_ISL_18945392, EPI_ISL_18945748, EPI_ISL_18947086, EPI_ISL_18947189, EPI_ISL_18947944, EPI_ISL_18948077, EPI_ISL_18948078, EPI_ISL_18948422, EPI_ISL_18948613, EPI_ISL_18948756, EPI_ISL_18948757, EPI_ISL_18949161, EPI_ISL_18949911, EPI_ISL_18952873, EPI_ISL_18953132, EPI_ISL_18953134, EPI_ISL_18953234, EPI_ISL_18953461, EPI_ISL_18953503, EPI_ISL_18955186, EPI_ISL_18956070, EPI_ISL_18956087, EPI_ISL_18956161, EPI_ISL_18958247, EPI_ISL_18958295, EPI_ISL_18958383, EPI_ISL_18960149, EPI_ISL_18960183, EPI_ISL_18960224, EPI_ISL_18962896, EPI_ISL_18962928, EPI_ISL_18963652, EPI_ISL_18965082, EPI_ISL_18965451, EPI_ISL_18966035, EPI_ISL_18966186, EPI_ISL_18968116, EPI_ISL_18968120, EPI_ISL_18969119, EPI_ISL_18969697, EPI_ISL_18969735, EPI_ISL_18970692, EPI_ISL_18971336, EPI_ISL_18972187, EPI_ISL_18972699, EPI_ISL_18972708, EPI_ISL_18972719, EPI_ISL_18972734, EPI_ISL_18972995, EPI_ISL_18975064, EPI_ISL_18975148, EPI_ISL_18975189, EPI_ISL_18975333, EPI_ISL_18977928, EPI_ISL_18977929, EPI_ISL_18979948, EPI_ISL_18980164, EPI_ISL_18981298, EPI_ISL_18982763, EPI_ISL_18986641, EPI_ISL_18986642, EPI_ISL_18986747, EPI_ISL_18987173, EPI_ISL_18987259, EPI_ISL_18990023, EPI_ISL_18992540, EPI_ISL_18999516, EPI_ISL_19000455, EPI_ISL_19002243, EPI_ISL_19002832, EPI_ISL_19003837, EPI_ISL_19004571, EPI_ISL_19005468, EPI_ISL_19005470, EPI_ISL_19005472, EPI_ISL_19005481, EPI_ISL_19008875, EPI_ISL_19009123, EPI_ISL_19009131, EPI_ISL_19010135, EPI_ISL_19012038, EPI_ISL_19012114, EPI_ISL_19012460, EPI_ISL_19012472, EPI_ISL_19012554, EPI_ISL_19012663, EPI_ISL_19013418, EPI_ISL_19014517, EPI_ISL_19014601, EPI_ISL_19015115, EPI_ISL_19015882, EPI_ISL_19016044, EPI_ISL_19016053, EPI_ISL_19016370, EPI_ISL_19017499, EPI_ISL_19019122, EPI_ISL_19019127, EPI_ISL_19019128, EPI_ISL_19021045, EPI_ISL_19021046, EPI_ISL_19022071, EPI_ISL_19022479, EPI_ISL_19024260, EPI_ISL_19026222, EPI_ISL_19026577, EPI_ISL_19026932, EPI_ISL_19029499, EPI_ISL_19030027, EPI_ISL_19030112, EPI_ISL_19030117, EPI_ISL_19030129, EPI_ISL_19030183, EPI_ISL_19031583, EPI_ISL_19032054, EPI_ISL_19032056, EPI_ISL_19032179, EPI_ISL_19032598, EPI_ISL_19033249, EPI_ISL_19035801, EPI_ISL_19036115, EPI_ISL_19036116, EPI_ISL_19036117, EPI_ISL_19036644, EPI_ISL_19036878, EPI_ISL_19036921, EPI_ISL_19041379, EPI_ISL_19041967, EPI_ISL_19042675, EPI_ISL_19043835, EPI_ISL_19043836, EPI_ISL_19044085, EPI_ISL_19044163, EPI_ISL_19044218, EPI_ISL_19044280, EPI_ISL_19044474, EPI_ISL_19044644, EPI_ISL_19044737, EPI_ISL_19046891, EPI_ISL_19046892, EPI_ISL_19047408, EPI_ISL_19049407, EPI_ISL_19049586, EPI_ISL_19050509, EPI_ISL_19051602, EPI_ISL_19053178, EPI_ISL_19053803, EPI_ISL_19053806, EPI_ISL_19053808, EPI_ISL_19054784, EPI_ISL_19055364, EPI_ISL_19055368, EPI_ISL_19055909, EPI_ISL_19058120, EPI_ISL_19060002, EPI_ISL_19060327, EPI_ISL_19060922, EPI_ISL_19061139, EPI_ISL_19061646, EPI_ISL_19062162, EPI_ISL_19062523, EPI_ISL_19062524, EPI_ISL_19064175, EPI_ISL_19065060, EPI_ISL_19065882, EPI_ISL_19066171, EPI_ISL_19066817, EPI_ISL_19067782, EPI_ISL_19067788, EPI_ISL_19070471, EPI_ISL_19071618, EPI_ISL_19073169, EPI_ISL_19073731, EPI_ISL_19073732, EPI_ISL_19073800, EPI_ISL_19073807, EPI_ISL_19074673, EPI_ISL_19075261, EPI_ISL_19075267, EPI_ISL_19075350, EPI_ISL_19076088, EPI_ISL_19081415, EPI_ISL_19081416, EPI_ISL_19081419, EPI_ISL_19081422, EPI_ISL_19081425, EPI_ISL_19082080, EPI_ISL_19082200, EPI_ISL_19082275, EPI_ISL_19082487, EPI_ISL_19085329, EPI_ISL_19085381, EPI_ISL_19085539, EPI_ISL_19085540, EPI_ISL_19085542, EPI_ISL_19086378, EPI_ISL_19086393, EPI_ISL_19086523, EPI_ISL_19091019, EPI_ISL_19091144, EPI_ISL_19094153, EPI_ISL_19094369, EPI_ISL_19095154, EPI_ISL_19095156, EPI_ISL_19095501, EPI_ISL_19095606, EPI_ISL_19095734, EPI_ISL_19095768, EPI_ISL_19100255, EPI_ISL_19105055, EPI_ISL_19106368, EPI_ISL_19106842, EPI_ISL_19106844, EPI_ISL_19106999, EPI_ISL_19108198, EPI_ISL_19108199, EPI_ISL_19108200, EPI_ISL_19108208, EPI_ISL_19108340, EPI_ISL_19108650, EPI_ISL_19108704, EPI_ISL_19108706, EPI_ISL_19108725, EPI_ISL_19131415, EPI_ISL_19131416, EPI_ISL_19131417, EPI_ISL_19131418, EPI_ISL_19131419, EPI_ISL_19132785, EPI_ISL_19132846, EPI_ISL_19133800, EPI_ISL_19135481, EPI_ISL_19135505, EPI_ISL_19135510, EPI_ISL_19135511, EPI_ISL_19137782, EPI_ISL_19137844, EPI_ISL_19140762, EPI_ISL_19141912, EPI_ISL_19142916, EPI_ISL_19143060, EPI_ISL_19143411, EPI_ISL_19143412, EPI_ISL_19143551, EPI_ISL_19143854, EPI_ISL_19143866, EPI_ISL_19143953, EPI_ISL_19143954, EPI_ISL_19145335, EPI_ISL_19146181, EPI_ISL_19146282, EPI_ISL_19146283, EPI_ISL_19146308, EPI_ISL_19146747, EPI_ISL_19147578, EPI_ISL_19148775, EPI_ISL_19151438, EPI_ISL_19153859, EPI_ISL_19158778, EPI_ISL_19158782, EPI_ISL_19159275, EPI_ISL_19159918, EPI_ISL_19159922, EPI_ISL_19159923, EPI_ISL_19161805, EPI_ISL_19164072, EPI_ISL_19164956, EPI_ISL_19165347, EPI_ISL_19165721, EPI_ISL_19167420, EPI_ISL_19167714, EPI_ISL_19169236, EPI_ISL_19169239, EPI_ISL_19169240, EPI_ISL_19169597, EPI_ISL_19169598, EPI_ISL_19173640, EPI_ISL_19173786, EPI_ISL_19175160, EPI_ISL_19175165, EPI_ISL_19175555, EPI_ISL_19175939, EPI_ISL_19176642, EPI_ISL_19176853, EPI_ISL_19176930, EPI_ISL_19177549, EPI_ISL_19177604, EPI_ISL_19177633, EPI_ISL_19178043, EPI_ISL_19178311, EPI_ISL_19179820, EPI_ISL_19182830, EPI_ISL_19182893, EPI_ISL_19183095, EPI_ISL_19183664, EPI_ISL_19183910, EPI_ISL_19183917, EPI_ISL_19183924, EPI_ISL_19184000, EPI_ISL_19184031, EPI_ISL_19184858, EPI_ISL_19185391, EPI_ISL_19186372, EPI_ISL_19187674, EPI_ISL_19187980, EPI_ISL_19188471, EPI_ISL_19189250, EPI_ISL_19190171, EPI_ISL_19190915, EPI_ISL_19191046, EPI_ISL_19192806, EPI_ISL_19192819, EPI_ISL_19193007, EPI_ISL_19193543, EPI_ISL_19193606, EPI_ISL_19193607, EPI_ISL_19193617, EPI_ISL_19195977, EPI_ISL_19195978, EPI_ISL_19195979, EPI_ISL_19195980, EPI_ISL_19195981, EPI_ISL_19195982, EPI_ISL_19196018, EPI_ISL_19196019, EPI_ISL_19196020, EPI_ISL_19196021, EPI_ISL_19197906, EPI_ISL_19198127, EPI_ISL_19198259, EPI_ISL_19199717, EPI_ISL_19199719, EPI_ISL_19200980, EPI_ISL_19202065, EPI_ISL_19202066, EPI_ISL_19203296, EPI_ISL_19209656, EPI_ISL_19210836, EPI_ISL_19211459, EPI_ISL_19213222, EPI_ISL_19213351, EPI_ISL_19214251, EPI_ISL_19214614, EPI_ISL_19216679, EPI_ISL_19216828, EPI_ISL_19217899, EPI_ISL_19219811, EPI_ISL_19219892, EPI_ISL_19221626, EPI_ISL_19223353, EPI_ISL_19225911, EPI_ISL_19226495, EPI_ISL_19227219, EPI_ISL_19228129, EPI_ISL_19229097, EPI_ISL_19229565, EPI_ISL_19230859, EPI_ISL_19230867, EPI_ISL_19230971, EPI_ISL_19232972, EPI_ISL_19234843, EPI_ISL_19237942, EPI_ISL_19239019, EPI_ISL_19239628, EPI_ISL_19239713, EPI_ISL_19243158, EPI_ISL_19243166, EPI_ISL_19243171, EPI_ISL_19243231, EPI_ISL_19243516, EPI_ISL_19243810, EPI_ISL_19244002, EPI_ISL_19245397, EPI_ISL_19245398, EPI_ISL_19245399, EPI_ISL_19251154, EPI_ISL_19254798, EPI_ISL_19256132, EPI_ISL_19256137, EPI_ISL_19256138, EPI_ISL_19256151, EPI_ISL_19256152, EPI_ISL_19257108, EPI_ISL_19257977, EPI_ISL_19259365, EPI_ISL_19259369, EPI_ISL_19259383, EPI_ISL_19260849, EPI_ISL_19260850, EPI_ISL_19261083, EPI_ISL_19264977, EPI_ISL_19265076, EPI_ISL_19267631, EPI_ISL_19268349, EPI_ISL_19269083, EPI_ISL_19269483, EPI_ISL_19271166, EPI_ISL_19271182, EPI_ISL_19271183, EPI_ISL_19271567, EPI_ISL_19271746, EPI_ISL_19273074, EPI_ISL_19277027, EPI_ISL_19280260, EPI_ISL_19281017, EPI_ISL_19282183, EPI_ISL_19282184, EPI_ISL_19283996, EPI_ISL_19286133, EPI_ISL_19286135, EPI_ISL_19286138, EPI_ISL_19286139, EPI_ISL_19286260, EPI_ISL_19287266, EPI_ISL_19287684, EPI_ISL_19288763, EPI_ISL_19290906, EPI_ISL_19292343, EPI_ISL_19292838, EPI_ISL_19292841, EPI_ISL_19297116, EPI_ISL_19297576, EPI_ISL_19298817, EPI_ISL_19299051, EPI_ISL_19300378, EPI_ISL_19300385, EPI_ISL_19300400, EPI_ISL_19301040, EPI_ISL_19302407, EPI_ISL_19304883, EPI_ISL_19308329, EPI_ISL_19308670, EPI_ISL_19308767, EPI_ISL_19308875, EPI_ISL_19308876, EPI_ISL_19308877, EPI_ISL_19308906, EPI_ISL_19309906, EPI_ISL_19310234, EPI_ISL_19311057, EPI_ISL_19311769, EPI_ISL_19312520, EPI_ISL_19318476, EPI_ISL_19319006, EPI_ISL_19320511, EPI_ISL_19322462, EPI_ISL_19324934, EPI_ISL_19326359, EPI_ISL_19331433, EPI_ISL_19332294, EPI_ISL_19333087, EPI_ISL_19340140, EPI_ISL_19341128, EPI_ISL_19341144, EPI_ISL_19344211, EPI_ISL_19345465, EPI_ISL_19345485, EPI_ISL_19346102, EPI_ISL_19346538, EPI_ISL_19348673, EPI_ISL_19351027, EPI_ISL_19351032, EPI_ISL_19351033, EPI_ISL_19351648, EPI_ISL_19351927, EPI_ISL_19359999, EPI_ISL_19360941, EPI_ISL_19362874, EPI_ISL_19362955, EPI_ISL_19363093, EPI_ISL_19364322, EPI_ISL_19364675, EPI_ISL_19369553, EPI_ISL_19369713, EPI_ISL_19374392, EPI_ISL_19374843, EPI_ISL_19374844, EPI_ISL_19374845, EPI_ISL_19380467, EPI_ISL_19381264, EPI_ISL_19381428, EPI_ISL_19381638, EPI_ISL_19381992, EPI_ISL_19381994, EPI_ISL_19382602, EPI_ISL_19383694, EPI_ISL_19384121, EPI_ISL_19385914, EPI_ISL_19385980, EPI_ISL_19387703, EPI_ISL_19388165, EPI_ISL_19388758, EPI_ISL_19391206, EPI_ISL_19391216, EPI_ISL_19393434, EPI_ISL_19393708, EPI_ISL_19398369, EPI_ISL_19405918, EPI_ISL_19408692, EPI_ISL_19408693, EPI_ISL_19410044, EPI_ISL_19410056, EPI_ISL_19410058, EPI_ISL_19411869, EPI_ISL_19412054, EPI_ISL_19414452, EPI_ISL_19414669, EPI_ISL_19414842, EPI_ISL_19415183, EPI_ISL_19415272, EPI_ISL_19415273, EPI_ISL_19415333, EPI_ISL_19417987, EPI_ISL_19418385, EPI_ISL_19418789, EPI_ISL_19419991, EPI_ISL_19425659, EPI_ISL_19427049, EPI_ISL_19427050, EPI_ISL_19427051, EPI_ISL_19428450, EPI_ISL_19428673, EPI_ISL_19431719, EPI_ISL_19433335, EPI_ISL_19434973, EPI_ISL_19438222, EPI_ISL_19441794, EPI_ISL_19446721, EPI_ISL_19446726, EPI_ISL_19447859, EPI_ISL_19452022, EPI_ISL_19456759, EPI_ISL_19457982, EPI_ISL_19458104, EPI_ISL_19458866, EPI_ISL_19463787, EPI_ISL_19463811, EPI_ISL_19464534, EPI_ISL_19465468, EPI_ISL_19467713, EPI_ISL_19467725, EPI_ISL_19468710, EPI_ISL_19473728, EPI_ISL_19474606, EPI_ISL_19474613, EPI_ISL_19477107, EPI_ISL_19478383, EPI_ISL_19478598, EPI_ISL_19479514, EPI_ISL_19480237, EPI_ISL_19482235, EPI_ISL_19483178, EPI_ISL_19483184, EPI_ISL_19483313, EPI_ISL_19486141, EPI_ISL_19488917, EPI_ISL_19491347, EPI_ISL_19495855, EPI_ISL_19498393, EPI_ISL_19499089, EPI_ISL_19499199, EPI_ISL_19499228, EPI_ISL_19499640, EPI_ISL_19499789, EPI_ISL_19500407, EPI_ISL_19502648, EPI_ISL_19503869, EPI_ISL_19506293, EPI_ISL_19506337, EPI_ISL_19506387, EPI_ISL_19506391, EPI_ISL_19506410, EPI_ISL_19506412, EPI_ISL_19506489, EPI_ISL_19506517, EPI_ISL_19506572, EPI_ISL_19508878, EPI_ISL_19510518, EPI_ISL_19511239, EPI_ISL_19511242, EPI_ISL_19511245, EPI_ISL_19511259, EPI_ISL_19511353, EPI_ISL_19512930, EPI_ISL_19513189, EPI_ISL_19513373, EPI_ISL_19513376, EPI_ISL_19517945, EPI_ISL_19519860, EPI_ISL_19521406, EPI_ISL_19522101, EPI_ISL_19526975, EPI_ISL_19527216, EPI_ISL_19527499, EPI_ISL_19529679, EPI_ISL_19529699, EPI_ISL_19530201, EPI_ISL_19530213, EPI_ISL_19530220, EPI_ISL_19532518, EPI_ISL_19535971, EPI_ISL_19536358, EPI_ISL_19537504, EPI_ISL_19537517, EPI_ISL_19537806, EPI_ISL_19539586, EPI_ISL_19539602, EPI_ISL_19541318, EPI_ISL_19543648, EPI_ISL_19543801, EPI_ISL_19546384, EPI_ISL_19550039, EPI_ISL_19550107, EPI_ISL_19551038, EPI_ISL_19551134, EPI_ISL_19555102, EPI_ISL_19555114, EPI_ISL_19555115, EPI_ISL_19557993, EPI_ISL_19558034, EPI_ISL_19560828, EPI_ISL_19560838, EPI_ISL_19561072, EPI_ISL_19561079, EPI_ISL_19562666, EPI_ISL_19575892, EPI_ISL_19578009, EPI_ISL_19578384, EPI_ISL_19585411, EPI_ISL_19585472, EPI_ISL_19588050, EPI_ISL_19588617, EPI_ISL_19588630, EPI_ISL_19589209, EPI_ISL_19589226, EPI_ISL_19589644, EPI_ISL_19591990, EPI_ISL_19600727, EPI_ISL_19601813, EPI_ISL_19603076, EPI_ISL_19603396, EPI_ISL_19603766, EPI_ISL_19606872, EPI_ISL_19606873, EPI_ISL_19606893, EPI_ISL_19613477, EPI_ISL_19613481, EPI_ISL_19615536, EPI_ISL_19615541, EPI_ISL_19615759, EPI_ISL_19616027, EPI_ISL_19616479, EPI_ISL_19618593, EPI_ISL_19619689, EPI_ISL_19619734, EPI_ISL_19619757, EPI_ISL_19620010, EPI_ISL_19621954, EPI_ISL_19622266, EPI_ISL_19623358, EPI_ISL_19623503, EPI_ISL_19627737, EPI_ISL_19627776, EPI_ISL_19627778, EPI_ISL_19630975, EPI_ISL_19631111, EPI_ISL_19632962, EPI_ISL_19634022, EPI_ISL_19636173, EPI_ISL_19638051, EPI_ISL_19639319, EPI_ISL_19640624, EPI_ISL_19641389, EPI_ISL_19641840, EPI_ISL_19642771, EPI_ISL_19643417, EPI_ISL_19643444, EPI_ISL_19643542, EPI_ISL_19643616, EPI_ISL_19643666, EPI_ISL_19645977, EPI_ISL_19648297, EPI_ISL_19655163, EPI_ISL_19656451, EPI_ISL_19657328, EPI_ISL_19657519, EPI_ISL_19657528, EPI_ISL_19660180, EPI_ISL_19661881, EPI_ISL_19661997, EPI_ISL_19666574, EPI_ISL_19671026, EPI_ISL_19671598, EPI_ISL_19671605, EPI_ISL_19671606, EPI_ISL_19671607, EPI_ISL_19671887, EPI_ISL_19673194, EPI_ISL_19673201, EPI_ISL_19676136, EPI_ISL_19680534, EPI_ISL_19683226, EPI_ISL_19683235, EPI_ISL_19683886, EPI_ISL_19685790, EPI_ISL_19685811, EPI_ISL_19685815, EPI_ISL_19685993, EPI_ISL_19689294, EPI_ISL_19690695, EPI_ISL_19693383, EPI_ISL_19693416, EPI_ISL_19693419, EPI_ISL_19696986, EPI_ISL_19703902, EPI_ISL_19704129, EPI_ISL_19704643, EPI_ISL_19706712, EPI_ISL_19706713, EPI_ISL_19710331, EPI_ISL_19710359, EPI_ISL_19711118, EPI_ISL_19711122, EPI_ISL_19713810, EPI_ISL_19713821, EPI_ISL_19713832, EPI_ISL_19714066")
diff_1_2 = setdiff(head_set_2024_12_31, chr_2024_12_31_set)
diff_2_1 = setdiff(chr_2024_12_31_set, head_set_2024_12_31)
println("diff_1_2 = $diff_1_2")
println("\n"^2)
println("diff_2_1 = $diff_2_1")
println("\n"^2)
chr_2025_2_10_set = Set(chr_2025_2_10)
diff_new_1_arr = Vector{String}()
diff_new_1 = setdiff(chr_2025_2_10_set, head_set_2024_12_31)
for seq in chr_epi_2025_02_11
    if !(seq in head_set_2024_12_31)
        push!(diff_new_1_arr, seq)
    end
end
println(length(diff_new_1_arr))
for seq in diff_new_1_arr
    print(seq, ", ")
end
println(length(diff_new_1_arr))
#diff_new_1_str = join(diff_new_1_arr, ",")
#println(diff_new_1)
println()
println()
diff_new_2 = setdiff(chr_2025_2_10_set, chr_2024_12_31_set)
println()
ct = 0
for seq in diff_new_1
    ct+= 1
#    print("$seq, ")
end
println("new_seq = $ct")
#println("\n"^8)
#println(diff_new_2)
diff_new1_new2 = setdiff(diff_new_1, diff_new_2)
println("diff_new1_new2 = $diff_new1_new2")
#println("\n"^8)
#println(diff_new1_new2)

diff_1_2 = Set{String}()
diff_2_1 = Set{String}()
422
EPI_ISL_602912, EPI_ISL_1014810, EPI_ISL_1059094, EPI_ISL_1090851, EPI_ISL_1200867, EPI_ISL_1200912, EPI_ISL_1261009, EPI_ISL_1376522, EPI_ISL_1376523, EPI_ISL_1547461, EPI_ISL_1744401, EPI_ISL_1829054, EPI_ISL_1829108, EPI_ISL_1941816, EPI_ISL_2170618, EPI_ISL_2397307, EPI_ISL_2397308, EPI_ISL_2397309, EPI_ISL_2397310, EPI_ISL_2397311, EPI_ISL_2397312, EPI_ISL_2397313, EPI_ISL_2758178, EPI_ISL_3061061, EPI_ISL_4515444, EPI_ISL_5307398, EPI_ISL_5628248, EPI_ISL_6605659, EPI_ISL_7502990, EPI_ISL_7503221, EPI_ISL_9201951, EPI_ISL_9630717, EPI_ISL_10128185, EPI_ISL_10329391, EPI_ISL_10451205, EPI_ISL_10451252, EPI_ISL_10681118, EPI_ISL_10876034, EPI_ISL_10876749, EPI_ISL_11219235, EPI_ISL_11219236, EPI_ISL_11295642, EPI_ISL_11482304, EPI_ISL_11742572, EPI_ISL_11747289, EPI_ISL_11995938, EPI_ISL_12021469, EPI_ISL_12207682, EPI_ISL_12350967, EPI_ISL_12530780, EPI_ISL_12531462, EPI_ISL_12623284, EPI_ISL_12753427, EPI_ISL_12754976, EPI_ISL